In [1]:
import json
import os
import pickle
import torch
import numpy as np

In [2]:
# Read JSON
def read_json(fname):
    with open(fname, "r") as handle:
        res = handle.read()
        obj = json.loads(res)
    return obj

In [3]:
# Read pickle
def read_pkl(fname):
    with open(fname, 'rb') as handle:
        obj_pkl = pickle.load(handle)
    return obj_pkl

In [4]:
# Read torch 
def read_torch(fname):
    obj = torch.load(fname)
    return obj

In [5]:
test_data = read_pkl('../data/medmentions/test_processed_data.pickle')

In [6]:
dictionary = read_pkl('../data/medmentions/dictionary.pickle')

In [7]:
dict_id_to_idx = {e['cui']: i for i,e in enumerate(dictionary)}

In [8]:
!ls data/medmentions/top64_cands

ls: data/medmentions/top64_cands: No such file or directory


In [9]:
!ls data/medmentions/results

ls: data/medmentions/results: No such file or directory


In [10]:
tfidf = {}
with open('../data/medmentions/top64_cands/tfidf.json', "r") as handle:
    lines = handle.readlines()
    for line in lines:
        obj = json.loads(line)
        tfidf[obj['mention_id']] = obj['tfidf_candidates']
existing_cands = {
    'mode': 'test',
    'candidates': {},
    'labels': []
}
for idx,men in enumerate(test_data):
    if men['mention_id'] not in tfidf:
        existing_cands['labels'].append(-1)
        existing_cands['candidates'][idx] = []
    else:
        arr = []
        existing_cands['labels'].append(-1)
        for c in tfidf[men['mention_id']]:
            arr.append(dict_id_to_idx[c])
            if c == men['label_cuis'][0]:
                existing_cands['labels'][-1] = len(arr) - 1
        existing_cands['candidates'][idx] = arr

In [11]:
# existing_cands = read_torch('data/medmentions/top64_cands/in_batch.t7')
labels = np.array(existing_cands['labels'])
np.where(labels == -1)[0]

array([   19,    21,    27, ..., 39027, 39032, 39034])

In [51]:
# Indexes where existing model fails to retrieve the ground-truth
# existing_cands = read_torch('data/medmentions/top64_cands/in_batch.t7')
exist_labels = np.array(existing_cands['labels'])
existing_idxs = np.where(exist_labels == -1)[0]
existing_idxs_bi_recallhits = np.where(exist_labels == 0)[0]
existing_idxs_bi_recallnonhits = np.where(exist_labels > 0)[0]
existing_idxs_bi_recall = np.where(exist_labels != -1)[0]
print(len(existing_idxs), " - ", existing_idxs)

5705  -  [   19    21    27 ... 39027 39032 39034]


In [13]:
our_cands = read_torch('../data/medmentions/top64_cands/arbo.t7')
our_labels = np.array(our_cands['labels'])
our_idxs = np.where(our_labels != -1)[0]
our_idxs_bi_recallhits = np.where(our_labels == 0)[0]
our_idxs_bi_recallnonhits = np.where(our_labels > 0)[0]
our_idxs_bi_norecall = np.where(our_labels == -1)[0]
print(len(our_idxs), " - ", our_idxs)
print(len(our_idxs_bi_recallhits), " - ", our_idxs_bi_recallhits)
print(len(our_idxs_bi_recallnonhits), " - ", our_idxs_bi_recallnonhits)
print(len(our_idxs_bi_norecall), " - ", our_idxs_bi_norecall)

37346  -  [    0     1     2 ... 39035 39036 39037]
28230  -  [    0     1     2 ... 39033 39035 39036]
9116  -  [   19    21    27 ... 39029 39034 39037]
1692  -  [  115   150   152 ... 39024 39027 39032]


In [16]:
# existing model AND our model failed recall

exi_fail_our_fail_idxs = np.intersect1d(existing_idxs, our_idxs_bi_norecall)
print(len(exi_fail_our_fail_idxs))
print(exi_fail_our_fail_idxs)

1277
[  115   150   152 ... 39022 39027 39032]


In [17]:
# existing model failed recall
# our biencoder linked correctly

print(len(np.intersect1d(existing_idxs, our_idxs_bi_recallhits)))
print(np.intersect1d(existing_idxs, our_idxs_bi_recallhits))

1385
[   42   102   212 ... 38977 38987 38994]


In [54]:
# existing model recalled correctly
# our biencoder failed recall
exi_succ_ourbi_fail_idxs = np.intersect1d(existing_idxs_bi_recall, our_idxs_bi_norecall)
print(len(exi_succ_ourbi_fail_idxs))
print(exi_succ_ourbi_fail_idxs)

415
[  159   262   434   569  1504  1554  1679  1748  1757  2016  2112  2227
  2228  2330  2386  2397  2407  2436  2440  2445  2447  2691  2929  3129
  3131  3133  3139  3141  3149  3200  3625  3829  3960  4611  4613  4671
  4675  4683  5089  5117  5148  5152  5286  5331  5348  5351  5373  5384
  5388  5403  5412  5700  6513  6522  6617  6633  6637  6642  6645  6649
  6653  6665  6669  6676  6812  6816  6900  6962  6972  6976  6989  6990
  7000  7001  7003  7006  7007  7200  7667  7683  7698  7820  7869  7884
  7890  8071  8083  9036  9147  9488  9489  9492  9510 10028 10158 10915
 10937 11006 11014 11315 11323 11705 11717 11776 11937 12036 12047 12074
 12082 12110 12714 12871 12904 13048 13171 13233 13235 13557 13664 13746
 13893 13996 14046 14049 14091 14250 14464 14616 14733 14735 14736 14737
 14739 14788 14850 15196 15355 15450 16235 16516 16518 16521 16526 16531
 16534 16539 16546 16557 16563 16612 16635 16666 16697 17478 17482 17485
 17486 17501 17517 17570 17792 17798 17810 1782

In [18]:
# existing model failed recall
# our biencoder recalled correctly
# our crossencoder linked correctly

exi_fail_ourbi_succ_idxs = np.intersect1d(existing_idxs, our_idxs_bi_recallnonhits)
print(len(exi_fail_ourbi_succ_idxs))
print(exi_fail_ourbi_succ_idxs)

3043
[   19    21    27 ... 38973 38995 39034]


In [19]:
cross_res = read_json('../data/medmentions/results/cross_arbo.json')
success_men_ids = set([s['mention_id'] for s in cross_res['success']])

In [20]:
cross_succ_idxs = [i for i in range(len(test_data)) if test_data[i]['mention_id'] in success_men_ids]

In [55]:
final_idxs = np.intersect1d(exi_fail_ourbi_succ_idxs, cross_succ_idxs)
print(len(final_idxs))

675


In [59]:
# Track cases where existing retrieved successfully but our biencoder failed to retrieve
res_exi_succ_our_fail = []

for i in exi_succ_ourbi_fail_idxs:
    d = test_data[i]
    men_id = d['mention_id']
    ctxt = ' '.join(d['context']['tokens'])
    etype = d['type']
    name = d['mention_name']
    correct_label_idx = d['label_idxs'][0]
    if len(existing_cands['candidates'][i]) != 0:
        existing_label_idx = existing_cands['candidates'][i][0]
        existing_cui = dictionary[existing_label_idx]['cui']
        existing_cui_name = dictionary[existing_label_idx]['title']
        existing_cui_desc = dictionary[existing_label_idx]['description']
    else:
        existing_cui = None
        existing_cui_name = None
        existing_cui_desc = None
    our_label_idx = our_cands['candidates'][i][0]  # Our biencoder model's best prediction
    our_cui = dictionary[our_label_idx]['cui']
    our_cui_name = dictionary[our_label_idx]['title']
    our_cui_desc = dictionary[our_label_idx]['description']
    res_exi_succ_our_fail.append({
        'mention_id': men_id,
        'test_set_idx': i,
        'type': etype,
        'mention_name': name,
        'context': ctxt,
        'cui_name': dictionary[correct_label_idx]['title'],
        'cui_desc': dictionary[correct_label_idx]['description'],
#         'existing_bi_cui': existing_cui,
#         'existing_bi_cui_name': existing_cui_name,
#         'existing_bi_cui_desc': existing_cui_desc,
#         'our_bi_cui': our_cui,
        'our_bi_cui_name': our_cui_name,
        'our_bi_cui_desc': our_cui_desc,
    })
np.random.shuffle(res_exi_succ_our_fail)

In [62]:
res_exi_succ_our_fail[20:30]

[{'mention_id': '27882444.13',
  'test_set_idx': 20496,
  'type': 'T005',
  'mention_name': 'pigeon paramyxovirus type 1',
  'context': '[CLS] type 1 first isolated in the northwest region of China Pig ##eon para ##my ##x ##ov ##ir ##us type - 1 ( pigeon para ##my ##x ##ov ##ir ##us type 1 ) is an en ##zo ##otic in pigeon flock ##s and causes severe economic losses in the p ##ou ##ltry industry in many countries . A [unused1] pigeon para ##my ##x ##ov ##ir ##us type 1 [unused2] is ##olate , abbreviated as PP ##M ##V - 1 / Q ##L - 01 / CH / 15 , was isolated from a great spotted wood ##pec ##ker in the northwest region of China in 2015 . The complete genome was sequence ##d , and the results showed that the virus genome was 15 , 192 n [SEP]',
  'cui_name': 'Newcastle disease virus',
  'cui_desc': 'Newcastle disease virus ( Virus , NDV , organism : PARAMYXOVIRUS AVIAN 01 ; Paramyxovirus 1 , Avian ; newcastle disease virus ; AVIAN PARAMYXOVIRUS 01 ; Avian Paramyxovirus 1 ; Newcastle disea

In [45]:
# Track cases where both the existing and our biencoder failed to retrieve the gold label in the top-64 preds
res_fail = []

menid_to_idx = {c['mention_id']: i for i,c in enumerate(cross_res['failure'])}
for i in exi_fail_our_fail_idxs:
    d = test_data[i]
    men_id = d['mention_id']
    ctxt = ' '.join(d['context']['tokens'])
    etype = d['type']
    name = d['mention_name']
    correct_label_idx = d['label_idxs'][0]
    if len(existing_cands['candidates'][i]) != 0:
        existing_label_idx = existing_cands['candidates'][i][0]
        existing_cui = dictionary[existing_label_idx]['cui']
        existing_cui_name = dictionary[existing_label_idx]['title']
        existing_cui_desc = dictionary[existing_label_idx]['description']
    else:
        existing_cui = None
        existing_cui_name = None
        existing_cui_desc = None
    our_label_idx = our_cands['candidates'][i][0]  # Our biencoder model's best prediction
    our_cui = dictionary[our_label_idx]['cui']
    our_cui_name = dictionary[our_label_idx]['title']
    our_cui_desc = dictionary[our_label_idx]['description']
#     try:
#         our_cross_cui = cross_res['failure'][menid_to_idx[men_id]]['predicted_cui']
#     except:
#         continue
#     our_cross_cui_name = cross_res['failure'][menid_to_idx[men_id]]['predicted_name']
    res_fail.append({
        'mention_id': men_id,
        'test_set_idx': i,
        'type': etype,
        'name': name,
        'context': ctxt,
        'cui_name': dictionary[correct_label_idx]['title'],
        'cui_desc': dictionary[correct_label_idx]['description'],
        'existing_bi_cui': existing_cui,
        'existing_bi_cui_name': existing_cui_name,
        'existing_bi_cui_desc': existing_cui_desc,
        'our_bi_cui': our_cui,
        'our_bi_cui_name': our_cui_name,
        'our_bi_cui_desc': our_cui_desc,
#         'our_cross_cui': our_cross_cui,
#         'our_cross_cui_name': our_cross_cui_name,
#         'our_cross_cui_desc': dictionary[dict_id_to_idx[our_cross_cui]]['description'],
    })

In [46]:
np.random.shuffle(res_fail)

In [49]:
res_fail[10:20]

[{'mention_id': '27444328.6',
  'test_set_idx': 5898,
  'type': 'T103',
  'name': 'FmEG',
  'context': '[CLS] Extra car ##bo ##hy ##dra ##te binding module contributes to the process ##ivity and cat ##alytic activity of a non - modular h ##ydro ##lase family 5 end ##og ##lu ##can ##ase from F ##omi ##tip ##oria me ##dit ##er ##rane ##a M ##F ##3 / 22 [unused1] F ##m ##EG [unused2] from F ##omi ##tip ##oria me ##dit ##er ##rane ##a is a non - modular end ##og ##lu ##can ##ase composed of a 24 - amino acids extension and 13 - amino acids link ##er - like p ##eptide at the N - terminus and a 312 - amino acids G ##H ##5 cat ##alytic domain at the C - terminus . In this study , six F ##m ##EG derivatives with del ##eti [SEP]',
  'cui_name': 'endoglucanase V',
  'cui_desc': 'endoglucanase V ( Chemical )',
  'existing_bi_cui': 'C0673013',
  'existing_bi_cui_name': 'GPER protein , human',
  'existing_bi_cui_desc': 'GPER protein , human ( Chemical : GPCR-BR ; Heptahelix Receptor ; IL8-Related R

In [18]:
res = []

menid_to_idx = {c['mention_id']: i for i,c in enumerate(cross_res['success'])}
for i in final_idxs:
    d = test_data[i]
    men_id = d['mention_id']
    ctxt = ' '.join(d['context']['tokens'])
    etype = d['type']
    name = d['mention_name']
    correct_label_idx = d['label_idxs'][0]
    if len(existing_cands['candidates'][i]) != 0:
        existing_label_idx = existing_cands['candidates'][i][0]
        existing_cui = dictionary[existing_label_idx]['cui']
        existing_cui_name = dictionary[existing_label_idx]['title']
        existing_cui_desc = ' '.join(dictionary[existing_label_idx]['tokens'])
    else:
        existing_cui = None
        existing_cui_name = None
        existing_cui_desc = None
    our_label_idx = our_cands['candidates'][i][0]  # Our biencoder model's best prediction
    res.append({
        'mention_id': men_id,
        'test_set_idx': i,
        'type': etype,
        'name': name,
        'context': ctxt,
        'existing_cui': existing_cui,
        'existing_cui_name': existing_cui_name,
        'existing_cui_desc': existing_cui_desc,
        'our_bi_cui': dictionary[our_label_idx]['cui'],
        'our_bi_cui_name': dictionary[our_label_idx]['title'],
        'our_bi_cui_desc': ' '.join(dictionary[our_label_idx]['tokens']),
        'our_cross_cui': cross_res['success'][menid_to_idx[men_id]]['mention_gold_cui'],
        'our_cross_cui_name': cross_res['success'][menid_to_idx[men_id]]['mention_gold_cui_name'],
        'our_cross_cui_desc': ' '.join(dictionary[correct_label_idx]['tokens']),
    })

In [52]:
np.random.shuffle(res)

In [53]:
res

[{'mention_id': '27335087.30',
  'test_set_idx': 2759,
  'type': 'T017',
  'name': 'CD3 ( + ) T cells',
  'context': '[CLS] flow c ##yt ##ometer . Cell ultra ##structure was observed under a transmission electron micro ##scope . The mitochondrial membrane potential ( Δ ##ψ ) was examined with J ##C - 1 dye . In H ##22 tumor - bearing mice , CD ##4 ( + ) T cells , CD ##8 ( + ) T cells , [unused1] CD ##3 ( + ) T cells [unused2] , and natural killer cells in peripheral blood were evaluated c ##yt ##ometric ##ally . Inter ##le ##uki ##n ( Inter ##le ##uki ##n ) - 2 and tumor ne ##c ##rosis factor ( tumor ne ##c ##rosis factor ) - α levels were measured using radio ##im ##mu ##no ##ass ##ay . The m ##RNA levels of Ba [SEP]',
  'existing_cui': 'C1277786',
  'existing_cui_name': 'CD3 T-cell count procedure',
  'existing_cui_desc': '[CLS] CD ##3 T - cell count procedure [unused3] ( Health Care Act ##ivity , procedure : CD ##3 T - cell count ) [SEP]',
  'our_bi_cui': 'C1563926',
  'our_bi_cui_n

In [23]:
chosen = [13478, 14309, 34150, 18011, 31580, 14732, 17537, 29659]

In [21]:
res = [{'mention_id': '27335087.30',
  'test_set_idx': 2759,
  'type': 'T017',
  'name': 'CD3 ( + ) T cells',
  'context': '[CLS] flow c ##yt ##ometer . Cell ultra ##structure was observed under a transmission electron micro ##scope . The mitochondrial membrane potential ( Δ ##ψ ) was examined with J ##C - 1 dye . In H ##22 tumor - bearing mice , CD ##4 ( + ) T cells , CD ##8 ( + ) T cells , [unused1] CD ##3 ( + ) T cells [unused2] , and natural killer cells in peripheral blood were evaluated c ##yt ##ometric ##ally . Inter ##le ##uki ##n ( Inter ##le ##uki ##n ) - 2 and tumor ne ##c ##rosis factor ( tumor ne ##c ##rosis factor ) - α levels were measured using radio ##im ##mu ##no ##ass ##ay . The m ##RNA levels of Ba [SEP]',
  'existing_cui': 'C1277786',
  'existing_cui_name': 'CD3 T-cell count procedure',
  'existing_cui_desc': '[CLS] CD ##3 T - cell count procedure [unused3] ( Health Care Act ##ivity , procedure : CD ##3 T - cell count ) [SEP]',
  'our_bi_cui': 'C1563926',
  'our_bi_cui_name': 'Th3 Cells',
  'our_bi_cui_desc': '[CLS] T ##h ##3 Cell ##s [unused3] ( Ana ##tom ##ical Structure : Cell ##s , T ##h ##3 ; T ##H CE ##LL ##S 00 ##3 ; T ##h ##3 Cell ; Cell , T ##h ##3 ) [SEP]',
  'our_cross_cui': 'C1267816',
  'our_cross_cui_name': 'Lymphocyte positive for CD3 antigen',
  'our_cross_cui_desc': '[CLS] L ##ymph ##oc ##yte positive for CD ##3 anti ##gen [unused3] ( Ana ##tom ##ical Structure , cell : CD ##3 + L ##ymph ##oc ##yte ) [SEP]'},
 {'mention_id': '28244917.34',
  'test_set_idx': 29801,
  'type': 'T033',
  'name': 'knee bend',
  'context': '[CLS] , and 10 treated with a P ##FC Sigma rotating platform - posterior stab ##ilized ( press - fit con ##dy ##lar Sigma rotating platform - posterior stab ##ilized ) total knee art ##hr ##op ##last ##y ( dome ##d pat ##ella ##r component ) were analyzed under flu ##oro ##scopic surveillance while the patient performed a weight - bearing deep [unused1] knee bend [unused2] from full knee extension to maximum knee fl ##ex ##ion . Re ##lev ##ant bone g ##eo ##met ##ries were segment ##ed out from com ##puted to ##mo ##graphy scan ##s , and computer - assisted - design models of the imp ##lanted components were obtained from the manufacturer . Three - dimensional pat ##ello ##fe ##mor ##al kin ##ema ##tics [SEP]',
  'existing_cui': 'C0011119',
  'existing_cui_name': 'Decompression Sickness',
  'existing_cui_desc': "[CLS] Dec ##om ##press ##ion Sick ##ness [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : Dec ##om ##press ##ion Sick ##ness [ Disease / Finding ] ; Bend ##s ; Rap ##ture of the deep syndrome ; de ##com ##press ##ion ; disease ; de ##com ##press ##ion sickness ; Dec ##om ##press ##ion sickness ; Di ##vers ' p ##als ##y or para ##lysis ; disease ( or disorder ) ; de ##com ##press ##ion ; choke ##s ; Disease , C ##ais ##son ; choking ; the bends ; Di ##vers ' p ##als ##y ; ca ##iss ##on ; disease ; a ##ero ##bull ##osis ; compressed air disease ; bend ; bends ; ca ##iss ##on disease ; [SEP]",
  'our_bi_cui': 'C0856776',
  'our_bi_cui_name': 'Twisted knee',
  'our_bi_cui_desc': '[CLS] T ##wi ##sted knee [unused3] ( In ##ju ##ry or Po ##ison ##ing : twist knee ; twisted knee ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27538372.30',
  'test_set_idx': 9811,
  'type': 'T038',
  'name': 'overexpression',
  'context': '[CLS] differential ##ly expressed genes , P ##d ##p ##1 expression was significantly decreased ( 27 - fold ) on day 8 compared to D ##0 , which was accompanied by suppressed mitochondrial in ##dices , including ATP levels , membrane potential , R ##OS and mitochondrial C ##a ( 2 + ) . Not ##ably , P ##d ##p ##1 [unused1] over ##ex ##press ##ion [unused2] significantly enhanced the mitochondrial in ##dices and p ##yr ##u ##vate de ##hy ##dr ##ogen ##ase activity and reduced the expression of cardiac differentiation marker messenger RNA and the cardiac differentiation rate compared to a mock control . In confirmation of this , a knock ##down of the P ##d ##p ##1 gene promoted the expression of cardiac differentiation marker [SEP]',
  'existing_cui': 'C1514559',
  'existing_cui_name': 'Protein Overexpression',
  'existing_cui_desc': '[CLS] Pro ##tein Over ##ex ##press ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction : Over ##ex ##press ##ion ; over ##ex ##press ) [SEP]',
  'our_bi_cui': 'C1514559',
  'our_bi_cui_name': 'Protein Overexpression',
  'our_bi_cui_desc': '[CLS] Pro ##tein Over ##ex ##press ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction : Over ##ex ##press ##ion ; over ##ex ##press ) [SEP]',
  'our_cross_cui': 'C0017262',
  'our_cross_cui_name': 'Gene Expression',
  'our_cross_cui_desc': '[CLS] Gene Express ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction , finding : Express ##ions , Gene ; expression gene ; Gene expression ; Express ##ion , Gene ; Express ##ion ; Express ##ed ; gene expression ; Gene Express ##ions ) [SEP]'},
 {'mention_id': '27379139.7',
  'test_set_idx': 4173,
  'type': 'T017',
  'name': 'Compound Tissue',
  'context': '[CLS] Evaluation of Three Pro ##tein - Extra ##ction Method ##s for Pro ##te ##ome Analysis of Mai ##ze Leaf Mid ##ri ##b , a [unused1] Co ##mpo ##und T ##iss ##ue [unused2] Rich in Sc ##ler ##en ##chy ##ma Cell ##s Leaf morphology is closely related to the growth and development of maize ( Z ##ea may ##s L . ) plants and final kernel production . As an important part of the maize leaf , the mid ##ri ##b holds leaf blades in the aerial position for maximum sunlight capture . Leaf mid ##ri ##b s of adult plants contain substantial s ##cle ##rench ##ym ##a cells with heavily thick ##ened and l ##ign ##ified secondary walls and have a high amount of p ##hen [SEP]',
  'existing_cui': 'C1706082',
  'existing_cui_name': 'Compound',
  'existing_cui_desc': '[CLS] Co ##mpo ##und [unused3] ( Chemical , substance , substance : compound ) [SEP]',
  'our_bi_cui': 'C0040300',
  'our_bi_cui_name': 'Body tissue',
  'our_bi_cui_desc': '[CLS] Body tissue [unused3] ( Ana ##tom ##ical Structure , Body , body structure : Port ##ion of tissue ; Normal T ##iss ##ue ; T ##iss ##ue ; Text ##us ; Body T ##iss ##ues ; tissues ; Body tissue structure ; body tissues ; T ##iss ; body tissue ; tissue ; T ##iss ##ues ; T ##IS ##SU ##E ) [SEP]',
  'our_cross_cui': 'C1514137',
  'our_cross_cui_name': 'Plant Tissue , Cells',
  'our_cross_cui_desc': '[CLS] Plant T ##iss ##ue , Cell ##s [unused3] ( Ana ##tom ##ical Structure : Plant cells and tissues ) [SEP]'},
 {'mention_id': '28522288.42',
  'test_set_idx': 38354,
  'type': 'T033',
  'name': 'percentage of positive cores',
  'context': '[CLS] ) . In multi ##var ##iate analyses , variables predict ##ing up ##grading for G ##G ##1 pro ##state cancer were age ( P = . 00 ##14 ) , abnormal digital re ##ct ##al examination ( P < . 000 ##1 ) , pro ##state - specific anti ##gen density ( P < . 000 ##1 ) , [unused1] percentage of positive core ##s [unused2] ( P < . 000 ##1 ) , and body mass index ( P = . 03 ##7 ) . A no ##mo ##gram was generated and valid ##ated internally . B ##io ##psy g ##rading system is misleading in approximately 50 % of cases . Up ##grading grade group from bio ##psy to radical pro ##state ##ct ##omy may [SEP]',
  'existing_cui': 'C0518026',
  'existing_cui_name': 'body fat percentage',
  'existing_cui_desc': '[CLS] body fat percentage [unused3] ( Finding , physical finding , physical finding : body fat percentage ##s ; Body fat percentage ; percentage body fat ; percentage of body fat ; body fat ) [SEP]',
  'our_bi_cui': 'C1302649',
  'our_bi_cui_name': 'Number of tissue cores positive for carcinoma',
  'our_bi_cui_desc': '[CLS] Number of tissue core ##s positive for car ##cin ##oma [unused3] ( Clinical At ##tri ##but ##e , o ##bs ##er ##vable entity ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28084061.16',
  'test_set_idx': 25234,
  'type': 'T033',
  'name': 'reason',
  'context': '[CLS] ##dices helped explain the variation in key rehabilitation outcomes . Prospect ##ive open - co ##hor ##t study . In ##patient rehabilitation ward , Melbourne , Australia . Adults admitted for in ##patient rehabilitation ( n = 280 ) . The main outcomes were demographic ( e . g . age , gender , discharge destination ) and clinical outcomes ( [unused1] reason [unused2] for rehabilitation , length of stay , Fun ##ctional Independence Me ##as ##ure , Cha ##rls ##on Como ##rb ##id ##ity Index and C ##um ##ula ##tive Index Rat ##ing Scale ) . A series of re ##gression analyses were performed to determine the influence of com ##or ##bid ##ity on three dependent variables : L ##OS in rehabilitation ; 2 ) [SEP]',
  'existing_cui': 'C1562480',
  'existing_cui_name': 'Has reason',
  'existing_cui_desc': '[CLS] Has reason [unused3] ( Intel ##lect ##ual Product , link assertion , link assertion : has reason ) [SEP]',
  'our_bi_cui': 'C1443309',
  'our_bi_cui_name': 'Reason for Treatment',
  'our_bi_cui_desc': '[CLS] Reason for Treatment [unused3] ( Finding , finding : Reason ##s for treatment ) [SEP]',
  'our_cross_cui': 'C0566251',
  'our_cross_cui_name': 'Reason and justification',
  'our_cross_cui_desc': '[CLS] Reason and justification [unused3] ( Finding , attribute ) [SEP]'},
 {'mention_id': '28356622.43',
  'test_set_idx': 33083,
  'type': 'T038',
  'name': 'plantar moment',
  'context': "[CLS] ##ait parameters were calculated using the S ##pear ##man correlation analysis . [ Results ] Rate ##s of the transverse arch of the fore ##foot width and height between condition 2 and condition 3 were significantly correlated with the anterior and posterior component of ground reaction forces , the hip joint extension angle , and the ankle plant ##ar fl ##ex ##ion moment . [ Con ##c ##lusion ] Our study ' s findings indicated that increased stiff ##ness of the transverse arch of the fore ##foot was related to the increase in ankle [unused1] plant ##ar moment [unused2] , and decreased stiff ##ness of the transverse arch of the fore ##foot was related to the increase in hip joint extension angle during g ##ait . [SEP]",
  'existing_cui': 'C0230463',
  'existing_cui_name': 'Sole of Foot',
  'existing_cui_desc': '[CLS] Sol ##e of Foot [unused3] ( Spa ##tial Concept , body structure : Plant ##ar aspect of foot ; Plant ##a ; foot sole ##s ; sole ; Plant ##ar Region ; foot sole ; Reg ##io plant ##aris ; sole of foot ; Plant ##ar region ; Bottom of foot ; Sol ##e of foot ; Plant ##ar part of foot ; Sol ##e ; Structure of sole of foot ; sole ##s ; S ##OL ##E ; Plant ##ar ) [SEP]',
  'our_bi_cui': 'C0278131',
  'our_bi_cui_name': 'Plantar reflex',
  'our_bi_cui_desc': '[CLS] Plant ##ar reflex [unused3] ( Finding , o ##bs ##er ##vable entity , physical finding : PR - Plant ##ar reflex ; plant ##ar reflex ) [SEP]',
  'our_cross_cui': 'C0231784',
  'our_cross_cui_name': 'Plantar flexion',
  'our_cross_cui_desc': '[CLS] Plant ##ar fl ##ex ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : plant ##ar fl ##ex ##ion ; Plant ##ar fl ##ex ##ion , function ; fl ##ex ##ion plant ##ar ) [SEP]'},
 {'mention_id': '28127340.49',
  'test_set_idx': 26275,
  'type': 'T204',
  'name': 'N. fasciatus',
  'context': "[CLS] % , No ##so ##psy ##llus f ##as ##cia ##tus = 1 . 09 % , and La ##ela ##ps nut ##tal ##li = 3 . 29 % . Among 10 genera / species of identified parasite ##s , at least 8 of them were zoo ##not ##ic with public health importance . L . nut ##tal ##li and [unused1] N . f ##as ##cia ##tus [unused2] were the only two non - zoo ##not ##ic detected parasite ##s in this survey . Harbor ##ing a wide variety of zoo ##not ##ic parasite ##s in sampled wild rode ##nts particularly when they live nearby villages , represents a potential risk to native inhabitants . Hence , controlling rode ##nts ' population in residential regions and improving [SEP]",
  'existing_cui': 'C1070996',
  'existing_cui_name': 'Arion fasciatus',
  'existing_cui_desc': '[CLS] Ari ##on f ##as ##cia ##tus [unused3] ( E ##uka ##ryo ##te : Ari ##on ( Car ##ina ##rion ) f ##as ##cia ##tus ) [SEP]',
  'our_bi_cui': 'C2997670',
  'our_bi_cui_name': 'Nariscus fasciatus',
  'our_bi_cui_desc': '[CLS] Na ##ris ##cus f ##as ##cia ##tus [unused3] ( E ##uka ##ryo ##te : Na ##ris ##cus f ##as ##cia ##tus ( Di ##stant , 1918 ) ; Akbar ##atus f ##as ##cia ##tus ) [SEP]',
  'our_cross_cui': 'C0323233',
  'our_cross_cui_name': 'Nosopsyllus fasciatus',
  'our_cross_cui_desc': '[CLS] No ##so ##psy ##llus f ##as ##cia ##tus [unused3] ( E ##uka ##ryo ##te , organism : Northern rat fl ##ea ; Ce ##rato ##phy ##llus f ##as ##cia ##tus ; No ##so ##psy ##llus f ##as ##cia ##tus ( Bo ##s ##c , 1800 ) ; Brown rat fl ##ea ) [SEP]'},
 {'mention_id': '27637577.41',
  'test_set_idx': 12603,
  'type': 'T103',
  'name': 'bisulfite - treated DNA',
  'context': '[CLS] ##lit ##is patients and 40 healthy individuals . Total RNA and DNA contents of le ##uk ##ocytes were extracted . Afterward , quantitative analysis was carried out by real - time PC ##R using the S ##Y ##BR Green PC ##R Master Mix . Finally , to determine the met ##hyl ##ation level , PC ##R products of [unused1] bi ##sul ##fit ##e - treated DNA [unused2] from patients and controls were sequence ##d . Compared with healthy controls , expression level of DNA met ##hyl ##tra ##ns ##fer ##ase 1 in An ##ky ##los ##ing s ##po ##ndy ##lit ##is patients was significantly down ##re ##gu ##lated . Met ##hyl ##ation of DNA met ##hyl ##tra ##ns ##fer ##ase 1 promoter was significantly higher in [SEP]',
  'existing_cui': 'C0063088',
  'existing_cui_name': 'hydrogen sulfite',
  'existing_cui_desc': '[CLS] hydrogen su ##lf ##ite [unused3] ( Chemical : bi ##sul ##fit ##e ; h ##ydro ##sul ##fit ##e ) [SEP]',
  'our_bi_cui': 'C3831347',
  'our_bi_cui_name': 'Bisulfite Sequencing',
  'our_bi_cui_desc': '[CLS] B ##is ##ulf ##ite Se ##quencing [unused3] ( Research Act ##ivity ) [SEP]',
  'our_cross_cui': 'C0012854',
  'our_cross_cui_name': 'DNA',
  'our_cross_cui_desc': '[CLS] DNA [unused3] ( Chemical , DNA , De ##ox ##yr ##ib ##on ##uc ##le ##ic A ##cid , substance : d ##na molecules ; d ##s ##D ##NA ; De ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; d ##na / des ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; DNA / des ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; DNA molecule ; DNA - De ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; De ##ox ##yr ##ib ##on ##uc ##le ##ic A ##cid ; De ##ox ##yr ##ib ##on ##uc ##le ##ic acids ; d ##na molecule ; DNA [ Chemical / In ##g ##red ##ient ] ; de ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; Double - [SEP]'},
 {'mention_id': '27669217.44',
  'test_set_idx': 13695,
  'type': 'T033',
  'name': 'MICs',
  'context': '[CLS] acquired independently in one highly v ##ir ##ulent is ##olate AL ##10 ##100 ##2 , and cluster ##ed with T ##n ##9 ##16 and IS ##12 ##16 , respectively . The expression of both er ##m ##B and er ##m ##T in all is ##olate ##s was er ##yt ##hr ##omy ##cin in ##du ##cible and yielded comparable mac ##rol ##ide [unused1] MI ##Cs [unused2] in all six is ##olate ##s . Take ##n together , in ##du ##cible expression of both er ##m ##B and er ##m ##T conferred high mac ##rol ##ide resistance in these S . gal ##lo ##ly ##ticus subsp . paste ##rian ##us is ##olate ##s . Our findings reveal new mac ##rol ##ide resistance features in S . gal ##lo [SEP]',
  'existing_cui': 'C0427978',
  'existing_cui_name': 'Minimum Inhibitory Concentration measurement',
  'existing_cui_desc': '[CLS] Mini ##mum In ##hibit ##ory Con ##cent ##ration measurement [unused3] ( Health Care Act ##ivity , MI ##C , procedure , qualifier value : Mini ##mum inhibitor ##y concentration su ##s ##ce ##pt ##ibility test ; Con ##cent ##rations , Mini ##mum In ##hibit ##ory ; Anti ##biotic sensitivity , minimum inhibitor ##y concentration ; MI ##C su ##s ##ce ##pt ##ibility test ; Mini ##mum In ##hibit ##ory Con ##cent ##rations ; MI ##C - Mini ##mum inhibitor ##y concentration ; Con ##cent ##ration , Mini ##mum In ##hibit ##ory ; Mini ##mum inhibitor ##y concentration ; Mini ##mum inhibitor ##y concentration su ##s ##ce ##pt ##ibility test technique ; In ##hibit ##ory Con ##cent ##rations , Mini ##mum ; Mini ##mum In ##hibit ##ory [SEP]',
  'our_bi_cui': 'C0427978',
  'our_bi_cui_name': 'Minimum Inhibitory Concentration measurement',
  'our_bi_cui_desc': '[CLS] Mini ##mum In ##hibit ##ory Con ##cent ##ration measurement [unused3] ( Health Care Act ##ivity , MI ##C , procedure , qualifier value : Mini ##mum inhibitor ##y concentration su ##s ##ce ##pt ##ibility test ; Con ##cent ##rations , Mini ##mum In ##hibit ##ory ; Anti ##biotic sensitivity , minimum inhibitor ##y concentration ; MI ##C su ##s ##ce ##pt ##ibility test ; Mini ##mum In ##hibit ##ory Con ##cent ##rations ; MI ##C - Mini ##mum inhibitor ##y concentration ; Con ##cent ##ration , Mini ##mum In ##hibit ##ory ; Mini ##mum inhibitor ##y concentration ; Mini ##mum inhibitor ##y concentration su ##s ##ce ##pt ##ibility test technique ; In ##hibit ##ory Con ##cent ##rations , Mini ##mum ; Mini ##mum In ##hibit ##ory [SEP]',
  'our_cross_cui': 'C1304747',
  'our_cross_cui_name': 'Minimum inhibitory concentration result',
  'our_cross_cui_desc': '[CLS] Mini ##mum inhibitor ##y concentration result [unused3] ( Finding , finding : minimum inhibitor ##y concentration ; Mini ##mum inhibitor ##y concentration finding ) [SEP]'},
 {'mention_id': '27660632.56',
  'test_set_idx': 13478,
  'type': 'T017',
  'name': 'regions',
  'context': '[CLS] ) traits , mainly focused on Ch ##r ##0 ##5 , Ch ##r ##0 ##9 , Ch ##r ##10 , Ch ##r ##14 , Ch ##r ##19 , and Ch ##r ##20 . Nine ##teen pairs of e ##pis ##tat ##ic quantitative trait lo ##cus were identified , of which two pairs involved in two add ##itive quantitative trait lo ##cus . These add ##itive quantitative trait lo ##cus , e ##pis ##tat ##ic quantitative trait lo ##cus , and Q ##TL clusters were tightly linked to single n ##uc ##leo ##tide p ##oly ##mor ##phism markers , which may serve as target [unused1] regions [unused2] for map - based c ##lon ##ing , gene discovery , and marker - assisted selection in cotton breeding . [SEP]',
  'existing_cui': 'C0017446',
  'existing_cui_name': 'Geographic Locations',
  'existing_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]',
  'our_bi_cui': 'C0205146',
  'our_bi_cui_name': 'Area',
  'our_bi_cui_desc': '[CLS] Area [unused3] ( Spa ##tial Concept , qualifier value : area ; areas ; AR ##EA ) [SEP]',
  'our_cross_cui': 'C1953345',
  'our_cross_cui_name': 'Region of chromosome',
  'our_cross_cui_desc': '[CLS] Region of chromosome [unused3] ( Ana ##tom ##ical Structure : ch ##rom ##oso ##mal region ; Ch ##rom ##oso ##mal region ; chromosome region ; Ch ##rom ##oso ##me region ) [SEP]'},
 {'mention_id': '27692319.38',
  'test_set_idx': 14309,
  'type': 'T204',
  'name': 'eggs',
  'context': '[CLS] ##roma ##tography , suspended in 1 % D ##MS ##O and used in in v ##it ##ro tests . Q ##uad ##rup ##lica ##tes of 50 F . he ##pa ##tica eggs were in ##cu ##bate ##d at 23 °C with M . ch ##aran ##tia leaf crude extract in different concentrations . After 12 days no larvae were formed in [unused1] eggs [unused2] in ##cu ##bate ##d with crude extract concentrations above 12 . 5 mg / m ##L . Egg ##s in ##cu ##bate ##d with crude extract sub - fraction ##s at concentrations of 1000 , 100 , 10 , 1 , 0 . 1 , 0 . 01 μ ##g / m ##L affected em ##b ##ryo ##nic development , with n [SEP]',
  'existing_cui': 'C0029974',
  'existing_cui_name': 'Ovum',
  'existing_cui_desc': '[CLS] O ##vu ##m [unused3] ( Ana ##tom ##ical Structure , body structure , cell , cell structure , qualifier value : Human egg ; Egg ##s , Un ##fer ##til ##ized ; Fe ##rt ##ilized o ##oc ##yte ; Mat ##ure o ##vu ##m ; O ##va ; Egg ; egg cell ; o ##va ; X bearing o ##vu ##m ; female game ##tes ; O ##ocytes ; Un ##fer ##til ##ized Egg ##s ; X chromosome - bearing o ##vu ##m ; Egg , Un ##fer ##til ##ized ; O ##vu ##m structure ; Un ##fer ##til ##ized Egg ; Female game ##te ; O ##voc ##yte ; egg / o ##vu ##m ; o ##vu ##m ; female game ##te ; X - [SEP]',
  'our_bi_cui': 'C0013710',
  'our_bi_cui_name': 'Egg Food Product',
  'our_bi_cui_desc': '[CLS] Egg Food Product [unused3] ( Food , Po ##ult ##ry , edible , substance : eggs ; egg food product ; egg ; Egg , Po ##ult ##ry ; Egg ; Egg ##s ; Egg ##s ( edible ) ; egg [ food ] ; E ##G ##G ) [SEP]',
  'our_cross_cui': 'C3662687',
  'our_cross_cui_name': 'Fasciola hepatica egg',
  'our_cross_cui_desc': '[CLS] F ##as ##cio ##la he ##pa ##tica egg [unused3] ( E ##uka ##ryo ##te , organism : F ##as ##cio ##la he ##pa ##tica o ##va ) [SEP]'},
 {'mention_id': '28390176.60',
  'test_set_idx': 34150,
  'type': 'T038',
  'name': 'inhibition',
  'context': '[CLS] over ##ex ##press ##ion of con ##st ##it ##utive ##ly active A ##kt completely abolished the C ##uc ##ur ##bit ##ac ##in B - mediated protection of cardiac h ##yper ##tro ##phy in human card ##io ##my ##ocytes AC ##16 . Collective ##ly , our findings suggest that cu ##cu ##rb ##ita ##cin B protects against cardiac h ##yper ##tro ##phy through increasing the auto ##pha ##gy level in card ##io ##my ##ocytes , which is associated with the [unused1] in ##hibition [unused2] of A ##kt / m ##TO ##R / Fox ##O ##3 ##a signal axis . J . Cell . B ##io ##che ##m . 99 ##9 ##9 : 1 - 12 , 2017 . © 2017 Wiley Period ##ical ##s , Inc . [SEP]',
  'existing_cui': 'C0021467',
  'existing_cui_name': 'Psychological inhibition',
  'existing_cui_desc': '[CLS] Psychological in ##hibition [unused3] ( B ##iol ##og ##ic Fun ##ction , Personal ##ity , Psychology : psychological in ##hibition ; in ##hibition ; in ##hibition ##s ; IN ##H ##IB ##IT ##ION ; In ##hibition ; IN ##H ##IB ##IT ##ION PS ##Y ##CH ##OL ; In ##hibition ##s ) [SEP]',
  'our_bi_cui': 'C0021469',
  'our_bi_cui_name': 'Metabolic Inhibition',
  'our_bi_cui_desc': '[CLS] Met ##ab ##olic In ##hibition [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : In ##hibition ; In ##hibition , function ) [SEP]',
  'our_cross_cui': 'C1519312',
  'our_cross_cui_name': 'Signal Transduction Inhibition',
  'our_cross_cui_desc': '[CLS] Signal Trans ##duction In ##hibition [unused3] ( B ##iol ##og ##ic Fun ##ction : in ##hibition of signal trans ##duction ) [SEP]'},
 {'mention_id': '28084061.29',
  'test_set_idx': 25247,
  'type': 'T033',
  'name': 'reason',
  'context': '[CLS] score between rehabilitation discharge and admission ; and 3 ) the discharge destination ( home vs other ) . The mean age was 57 . 7 years , there were slightly more females ( 51 % ) , most ( 95 % ) patients previously lived at home with family or other relatives ( 63 % ) . The most common [unused1] reason [unused2] for rehabilitation was or ##th ##op ##ae ##dic or other conditions ( 52 % ) and most ( 80 % ) people were discharged home . The median L ##OS was 27 days . There were 100 ( 35 . 7 % ) patients who had no com ##or ##bid ##ity recorded using the Cha ##rls ##on Como ##rb ##id ##ity Index [SEP]',
  'existing_cui': 'C1562480',
  'existing_cui_name': 'Has reason',
  'existing_cui_desc': '[CLS] Has reason [unused3] ( Intel ##lect ##ual Product , link assertion , link assertion : has reason ) [SEP]',
  'our_bi_cui': 'C1443309',
  'our_bi_cui_name': 'Reason for Treatment',
  'our_bi_cui_desc': '[CLS] Reason for Treatment [unused3] ( Finding , finding : Reason ##s for treatment ) [SEP]',
  'our_cross_cui': 'C0566251',
  'our_cross_cui_name': 'Reason and justification',
  'our_cross_cui_desc': '[CLS] Reason and justification [unused3] ( Finding , attribute ) [SEP]'},
 {'mention_id': '28249376.78',
  'test_set_idx': 29948,
  'type': 'T033',
  'name': 'digestibility',
  'context': '[CLS] ##ole ##ster ##ol , total protein , album ##in , blood u ##rea nitrogen and as ##par ##tate amino ##tra ##ns ##fer ##ase and al ##ani ##ne amino ##tra ##ns ##fer ##ase concentration . A ##ppa ##rent total tract dig ##est ##ibility of dry matter , crude pro ##ten , organic matter , and neutral de ##ter ##gent fiber were not influenced by mixture of essential oils supplement ##ation . The results of the present study suggested that supplement ##ation of mixture of essential oils may have limited effects on apparent nut ##rient [unused1] dig ##est ##ibility [unused2] , r ##umi ##nal f ##er ##mentation and pro ##to ##zo ##a and h ##yper am ##monia producing bacteria count , blood cells and meta ##bol ##ites . [SEP]',
  'existing_cui': 'C0233423',
  'existing_cui_name': 'Suggestibility',
  'existing_cui_desc': '[CLS] Su ##gg ##est ##ibility [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : suggest ##ibility ; Su ##gg ##est ##ibility , function ) [SEP]',
  'our_bi_cui': 'C0012238',
  'our_bi_cui_name': 'Digestion',
  'our_bi_cui_desc': '[CLS] Digest ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : dig ##estive system functions ; dig ##est ##ion ; Digest ##ive system functions ; dig ##est ##ions ; Digest ##ive system function ; Fun ##ctions of the dig ##estive system ; Digest ##ive ; Digest ##ive tract function ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27580908.19',
  'test_set_idx': 10663,
  'type': 'T058',
  'name': 'pediatric gastrointestinal radiology',
  'context': "[CLS] digital game or with a traditional did ##actic lecture . Medical students participating in a fourth - year radio ##logy elect ##ive were invited to participate . Student co ##hor ##ts were alternatively given a faculty - supervised 1 ##h session playing a simple interactive digital T ##ic - ta ##c - toe q ##uiz module on [unused1] p ##ediatric gas ##tro ##int ##est ##inal radio ##logy [unused2] or a 1 ##h did ##actic intro ##ductory lecture on the same topic . Survey questions assessed the learn ##ers ' perceived ability to recall the material as well as their satisfaction with the educational experience . Results of an end - of - rotation exam were reviewed to evaluate a quantitative measure of learning between groups . [SEP]",
  'existing_cui': 'C1518934',
  'existing_cui_name': 'Pediatric Radiology Specialty',
  'existing_cui_desc': '[CLS] P ##ediatric Radio ##logy Special ##ty [unused3] ( B ##io ##medical O ##cc ##upation or Disc ##ip ##line : radio ##logy , p ##ediatric ; p ##ediatric radio ##logy ; P ##ediatric Radio ##logy ) [SEP]',
  'our_bi_cui': 'C0947817',
  'our_bi_cui_name': 'Gastrointestinal radiotherapies',
  'our_bi_cui_desc': '[CLS] Gas ##tro ##int ##est ##inal radio ##ther ##ap ##ies [unused3] ( Health Care Act ##ivity ) [SEP]',
  'our_cross_cui': 'C0203054',
  'our_cross_cui_name': 'Radiography of gastrointestinal tract',
  'our_cross_cui_desc': '[CLS] Radio ##graphy of gas ##tro ##int ##est ##inal tract [unused3] ( Health Care Act ##ivity , procedure : X - ray gas ##tro ##int ##est ##inal ; Radio ##graphic procedure on gas ##tro ##int ##est ##inal tract ; Gas ##tro ##int ##est ##inal tract X - ray ; G ##I tract X - ray ; Digest ##ive tract X - ray ; Radio ##log ##ic procedure on gas ##tro ##int ##est ##inal tract ; G ##I x - ray ; x - ray of gas ##tro ##int ##est ##inal tract ; X - ray ; dig ##estive tract ; X - ray gas ##tro ##int ##est ##inal tract ; X - ray of the dig ##estive tract ; Gas ##tro ##int ##est ##inal tract x - [SEP]'},
 {'mention_id': '27812530.21',
  'test_set_idx': 18011,
  'type': 'T103',
  'name': 'material',
  'context': '[CLS] °C ) . Six specimens were tested in each group of the aforementioned parameters . Inc ##rea ##sing cast ##or oil content and pre ##co ##olin ##g treatment effectively decreased the peak polymer ##ization temperatures and increased the duration to achieve the peak polymer ##ization temperature ( P < 0 . 05 ) . Furthermore , the mechanical properties of the [unused1] material [unused2] , including density , m ##od ##ulus , and maximum compression strength , decreased with increasing cast ##or oil content . However , preparation temperature ( room temperature versus pre ##co ##olin ##g ) had no significant effect ( P > 0 . 05 ) on these mechanical properties . In conclusion , the addition of cast ##or oil to Pol ##yme [SEP]',
  'existing_cui': 'C0440470',
  'existing_cui_name': 'Human material',
  'existing_cui_desc': '[CLS] Human material [unused3] ( Body Sub ##stance , substance ) [SEP]',
  'our_bi_cui': 'C0032521',
  'our_bi_cui_name': 'Polymers',
  'our_bi_cui_desc': '[CLS] Pol ##yme ##rs [unused3] ( Chemical , substance : Pol ##yme ##ric ; Pol ##yme ##ric Mac ##rom ##ole ##cule ##s ; Pol ##yme ##r ; P ##OL ##Y ##ME ##R S ##U ##BS ##TA ##NC ##E ; Pol ##yme ##rs [ Chemical / In ##g ##red ##ient ] ; polymer ; polymer ##s ) [SEP]',
  'our_cross_cui': 'C0005479',
  'our_cross_cui_name': 'Biomaterials',
  'our_cross_cui_desc': '[CLS] B ##io ##mate ##rial ##s [unused3] ( Chemical : B ##io ##mate ##rial ; bio ##com ##pa ##tible materials ; B ##io ##medical Material ; bio ##mate ##rial ##s ; B ##io ##medical materials ; B ##io ##com ##pa ##tible Materials ; bio ##medical materials ; bio ##mate ##rial ; Materials , B ##io ##com ##pa ##tible ; bio ##medical material ; B ##io ##com ##pa ##tible Materials [ Chemical / In ##g ##red ##ient ] ) [SEP]'},
 {'mention_id': '28295414.42',
  'test_set_idx': 31580,
  'type': 'T168',
  'name': 'drink',
  'context': '[CLS] a 10 % probability ( or less ) of transition ##ing out of either a no heavy drinking state or a heavy drinking state . More than two - thirds of the heavy drink ##ers who exceeded the heavy drinking threshold during treatment reported , on average , a 64 % reduction in drinking frequency and a 38 % reduction in drinking intensity from pre ##tre ##at ##ment drinking levels . The results show stability of no heavy drinking as an outcome within the first 4 months of treatment and that the > 3 / > 4 [unused1] drink [unused2] cut ##off may mask substantial reduction ##s in alcohol consumption among some patients . Future studies should explore the clinical utility of reduction end points . [SEP]',
  'existing_cui': 'C0452428',
  'existing_cui_name': 'Drink',
  'existing_cui_desc': '[CLS] Dr ##ink [unused3] ( Food , diet ##ary substance , diet ##ary substance , attribute , substance : Dr ##ink ##s ; drinks ; drink [ substance ] ; drink ) [SEP]',
  'our_bi_cui': 'C0452428',
  'our_bi_cui_name': 'Drink',
  'our_bi_cui_desc': '[CLS] Dr ##ink [unused3] ( Food , diet ##ary substance , diet ##ary substance , attribute , substance : Dr ##ink ##s ; drinks ; drink [ substance ] ; drink ) [SEP]',
  'our_cross_cui': 'C0001967',
  'our_cross_cui_name': 'Alcoholic Beverages',
  'our_cross_cui_desc': '[CLS] Al ##co ##hol ##ic Be ##ver ##ages [unused3] ( Food , Al ##co ##hol ##ic , substance : Be ##ver ##age , Al ##co ##hol ##ic ; Al ##co ##hol ##ic Be ##ver ##age ; Al ##co ##hol ##ic beverages ; alcoholic drinks ; Be ##ver ##ages ; Be ##ver ##ages , Al ##co ##hol ##ic ; alcoholic beverages ; alcoholic beverage ; alcoholic ##s beverages ; Al ##co ##hol ##ic beverage ; Al ##co ##hol ##ic drink ; alcoholic drinking ) [SEP]'},
 {'mention_id': '27617315.44',
  'test_set_idx': 12007,
  'type': 'T103',
  'name': '% p2PSA',
  'context': '[CLS] 54 . 5 % in Pro ##state Health Index > 55 ( ch ##i - square test , p < 0 . 00 ##1 ) . The area under curves of the base model including age , total pro ##state - specific anti ##gen and status of initial / repeated bio ##psy was 0 . 64 . Ad ##ding [unused1] % p ##2 ##PS ##A [unused2] and Pro ##state Health Index to the base model improved the area under curves to 0 . 79 ( p < 0 . 00 ##1 ) and 0 . 78 ( p < 0 . 00 ##1 ) , respectively , and provided net clinical benefit in decision curve analyses . The positive bio ##psy rates of G ##lea ##son [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0138741',
  'our_bi_cui_name': 'Prostate-Specific Antigen',
  'our_bi_cui_desc': '[CLS] Pro ##state - S ##pecific Anti ##gen [unused3] ( Chemical , PS ##A , procedure , substance : pro ##state specific anti ##gen ; gamma Semi ##no ##p ##rote ##in ; Se ##men ##ogel ##ase ; gamma semi ##no ##p ##rote ##in ; Ka ##lli ##k ##re ##in , h ##K ##3 ; Gamma - Semi ##no ##p ##rote ##in ; Pro ##state - specific anti ##gen ; HK 00 ##3 K ##AL ##L ##I ##K ##RE ##IN ; PS ##A - Pro ##state specific anti ##gen ; Ka ##lli ##k ##re ##in h ##K ##3 ; gamma - Semi ##no ##p ##rote ##in ; Pro ##state S ##pecific Anti ##gen ; EC 3 . 4 . 21 . 77 ; P - 30 anti ##gen [SEP]',
  'our_cross_cui': 'C3710767',
  'our_cross_cui_name': '( -2 ) pro-prostate-specific antigen , human',
  'our_cross_cui_desc': '[CLS] ( - 2 ) pro - pro ##state - specific anti ##gen , human [unused3] ( Chemical : ( - 2 ) pro - PS ##A , human ; p ##2 ##PS ##A , human ) [SEP]'},
 {'mention_id': '27699206.11',
  'test_set_idx': 14492,
  'type': 'T017',
  'name': 'lateral occipital',
  'context': '[CLS] . However , it remains un ##res ##ol ##ved whether all objects e ##voke equivalent levels of activity in lateral o ##cci ##pit ##al , and , if not , which image features produce stronger activation . Here , we used an un ##bia ##sed para ##metric texture model to predict preferred versus non ##p ##re ##ferred stimuli in [unused1] lateral o ##cci ##pit ##al [unused2] . O ##bs ##er ##vation and ps ##ych ##op ##hy ##si ##cal results showed that predicted preferred stimuli ( both objects and non ##ob ##jects ) had smooth ( rather than texture ##d ) surfaces . These predictions were confirmed using f ##MR ##I , for objects and non ##ob ##jects . Similar preferences were also found in the f [SEP]',
  'existing_cui': 'C0923804',
  'existing_cui_name': 'Lateral occipital artery',
  'existing_cui_desc': '[CLS] Later ##al o ##cci ##pit ##al artery [unused3] ( Ana ##tom ##ical Structure , Arte ##ria c ##ere ##bri posterior : P ##3 segment of posterior cerebral artery ; Arte ##ria o ##cci ##pit ##alis lateral ##is ; Se ##gment ##um P ##3 ; A . o ##cci ##pit ##alis lateral ##is ) [SEP]',
  'our_bi_cui': 'C0228229',
  'our_bi_cui_name': 'lateral occipital sulcus',
  'our_bi_cui_desc': '[CLS] lateral o ##cci ##pit ##al su ##l ##cus [unused3] ( Spa ##tial Concept , human only , human only , body structure : Later ##al o ##cci ##pit ##al su ##l ##cus ; Structure of lateral o ##cci ##pit ##al su ##l ##cus ; Later ##al - O ##cci ##pit ##al Sul ##cus ; Sul ##cus o ##cci ##pit ##alis lateral ##is ) [SEP]',
  'our_cross_cui': 'C0028785',
  'our_cross_cui_name': 'Occipital lobe',
  'our_cross_cui_desc': '[CLS] O ##cci ##pit ##al lobe [unused3] ( Ana ##tom ##ical Structure , body structure : o ##cci ##pit ##al region ; O ##cci ##pit ##al cortex ; lo ##bus o ##cci ##pit ##alis ; O ##cci ##pit ##al Lo ##be ; o ##cci ##pit ##al lobes ; O ##cci ##pit ##al lobes ; O ##cci ##pit ##al Region ##s ; O ##cci ##pit ##al Region ; o ##cci ##pit ##al lobe ; Lo ##be , O ##cci ##pit ##al ; lobe o ##cci ##pit ##al ; O ##cci ##pit ##al region ; Region , O ##cci ##pit ##al ; Lo ##bus o ##cci ##pit ##alis ; o ##cci ##pit ##al cortex ; Reg ##io o ##cci ##pit ##alis ; Lo ##bes , O ##cci ##pit ##al [SEP]'},
 {'mention_id': '28197156.15',
  'test_set_idx': 28085,
  'type': 'T082',
  'name': 'Japanese',
  'context': '[CLS] CO ##2 ass ##im ##ilation rate ( A ) between rice cult ##iva ##rs using quantitative genetics is one promising means to identify genes contributing to higher photos ##ynth ##esis . In this study , we determined precise location of Carbon As ##si ##mi ##lation Rate 8 by crossing a high - yielding in ##dic ##a cult ##iva ##r with a [unused1] Japanese [unused2] commercial cult ##iva ##r . Fine mapping suggested that Carbon As ##si ##mi ##lation Rate 8 en ##codes a put ##ative He ##me Act ##iva ##tor Pro ##tein 3 ( O ##s ##HA ##P ##3 ) subunit of a CC ##AA ##T - box - binding transcription factor called O ##s ##HA ##P ##3 ##H . Se ##quencing analysis revealed that the [SEP]',
  'existing_cui': 'C1556094',
  'existing_cui_name': 'Japanese race',
  'existing_cui_desc': '[CLS] Japanese race [unused3] ( Population Group , ethnic group , finding : J ##AP ##AN ##ES ##E ; Race : Japanese ; Japanese ; - - Japanese ) [SEP]',
  'our_bi_cui': 'C1556094',
  'our_bi_cui_name': 'Japanese race',
  'our_bi_cui_desc': '[CLS] Japanese race [unused3] ( Population Group , ethnic group , finding : J ##AP ##AN ##ES ##E ; Race : Japanese ; Japanese ; - - Japanese ) [SEP]',
  'our_cross_cui': 'C0022341',
  'our_cross_cui_name': 'Japan',
  'our_cross_cui_desc': '[CLS] Japan [unused3] ( Spa ##tial Concept , geographic location : J ##A ; JP ##N ; JP ; J ##AP ##AN ) [SEP]'},
 {'mention_id': '27882455.28',
  'test_set_idx': 20576,
  'type': 'T038',
  'name': 'item integration',
  'context': '[CLS] these memory traces into an existing body of knowledge ( item integration ) . Older adults learned 13 non ##words and were tested on their memory for the non ##words , and on whether these non ##words impacted upon processing of similar - sounding English words immediately and 24 hours later . Part ##ici ##pants accurately recognized the non ##words immediately , but showed significant decreases in delayed recognition and recall . In comparison , the non ##words impacted upon processing of similar - sounding words only in the delayed test . Together , these findings suggest that memory consolidation processes may be more evident in [unused1] item integration [unused2] than memory stab ##ilization processes for new de ##c ##lar ##ative memories in older adults . [SEP]',
  'existing_cui': 'C1158478',
  'existing_cui_name': 'DNA Integration',
  'existing_cui_desc': '[CLS] DNA Integration [unused3] ( B ##iol ##og ##ic Fun ##ction : DNA integration ; Integration ) [SEP]',
  'our_bi_cui': 'C0700301',
  'our_bi_cui_name': 'Cognitive information processing',
  'our_bi_cui_desc': '[CLS] Co ##gni ##tive information processing [unused3] ( B ##iol ##og ##ic Fun ##ction , o ##bs ##er ##vable entity , qualifier value : Information processing , function ; Information processing ; information process ; Process ##es information ; information processing ; Information Process ##ing ) [SEP]',
  'our_cross_cui': 'C0025361',
  'our_cross_cui_name': 'Mental Processes',
  'our_cross_cui_desc': '[CLS] Mental Process ##es [unused3] ( B ##iol ##og ##ic Fun ##ction , Human , function , o ##bs ##er ##vable entity : Human Information Process ##ing ; Thought process ; cognitive processing ; processes thought ; processing thought ; Information Process ##ing , Human ; Human information processing ; Information Process ##es ; mental processing ; cognitive process ; Form of thinking , function ; mental processes ; Form of thought ; Form of thinking ; Human Information Process ##es ; Mental Process ; mental process ; thought process ) [SEP]'},
 {'mention_id': '27500205.32',
  'test_set_idx': 8464,
  'type': 'T103',
  'name': 'Kindlins',
  'context': '[CLS] play major roles in controlling in ##te ##g ##rin function . In v ##it ##ro studies , in v ##ivo studies of mice def ##icient in kind ##lins , and studies of patients with genetic def ##ici ##encies of kind ##lins have clearly established that they regulate the capacity of in ##te ##g ##rin ##s to media ##te their functions . [unused1] Kind ##lins [unused2] are adapt ##or proteins ; their function em ##ana ##te from their interaction with binding partners , including the c ##yt ##op ##las ##mic tails of in ##te ##g ##rin ##s and components of the act ##in c ##yt ##os ##kel ##eton . The purpose of this review is to provide a brief overview of kind ##lin structure and function , [SEP]',
  'existing_cui': 'C1720776',
  'existing_cui_name': 'Kindling , Neurologic',
  'existing_cui_desc': '[CLS] Kind ##ling , N ##eur ##olo ##gic [unused3] ( B ##iol ##og ##ic Fun ##ction , N ##eur ##ology : kind ##ling ; Kind ##lings , N ##eur ##olo ##gic ; Kind ##ling ; N ##eur ##olo ##gic Kind ##lings ; N ##eur ##olo ##gic Kind ##ling ) [SEP]',
  'our_bi_cui': 'C0010422',
  'our_bi_cui_name': 'Crystallins',
  'our_bi_cui_desc': '[CLS] Crystal ##lins [unused3] ( Chemical , substance : Crystal ##lins [ Chemical / In ##g ##red ##ient ] ; Crystal ##lin ; Pro ##tein ##s , Len ##s ; C ##rist ##all ##in ; Len ##s Pro ##tein ##s ; crystal ##lin ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27977319.24',
  'test_set_idx': 22776,
  'type': 'T017',
  'name': 'THI1',
  'context': '[CLS] ##ym ##bio ##sis , we focused on T ##H ##I ##1 , a th ##iam ##ine - bio ##sy ##nt ##hesis gene expressed in roots , nod ##ules , and seeds . The th ##i ##1 mutant had green leaves , but formed small nod ##ules and im ##mat ##ure seeds . These p ##hen ##otype ##s were rescued by [unused1] T ##H ##I ##1 [unused2] complement ##ation and by ex ##ogen ##ous th ##iam ##ine . Thus , T ##H ##I ##1 is required for nod ##ule en ##lar ##gement and seed mat ##uration . On the other hand , colonization by a ##rb ##us ##cular my ##cor ##r ##hi ##za fungus R ##hi ##zo ##pha ##gus irregular ##is was not affected in the th [SEP]',
  'existing_cui': 'C0272238',
  'existing_cui_name': 'Transient hypogammaglobulinemia of infancy',
  'existing_cui_desc': '[CLS] Trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia of infancy [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : Trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia of young ; trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia of infancy ; T ##H ##I - Trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia of infancy ; h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia trans ##ient of infancy ; I ##mm ##uno ##g ##lo ##bul ##in mat ##uration ##al delay ; T ##H ##I - Trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##inae ##mia of infancy ; Trans ##ient H ##y [SEP]',
  'our_bi_cui': 'C1420871',
  'our_bi_cui_name': 'TPI1 gene',
  'our_bi_cui_desc': '[CLS] T ##PI ##1 gene [unused3] ( Ana ##tom ##ical Structure : T ##RI ##OS ##EP ##H ##OS ##P ##HA ##TE ISO ##ME ##RA ##SE 1 ; T ##PI ##1 Gene ; Trio ##se ##ph ##os ##phate Is ##omer ##ase 1 Gene ; T ##PI ; trio ##se ##ph ##os ##phate is ##omer ##ase 1 ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27999200.29',
  'test_set_idx': 23336,
  'type': 'T103',
  'name': 'MepA',
  'context': '[CLS] substrates relevant to cancer invasion . We found that Me ##p ##1 ##A was a target of Rep ##tin , a protein that is on ##co ##genic in HC ##C . We studied Me ##p ##1 ##A regulation by Rep ##tin , its role in HC ##C , and whether it media ##tes Rep ##tin on ##co ##genic effects . [unused1] Me ##p ##A [unused2] and Rep ##tin expression was measured in human HC ##C by q ##RT - PC ##R and in culture ##d cells by PC ##R , western b ##lot and en ##zy ##matic activity measurements . Cell growth was assessed by counting and M ##TS ass ##ay . Cell migration was measured in Boyd ##en chambers and wound healing ass ##ays , [SEP]',
  'existing_cui': 'C0611930',
  'existing_cui_name': "adenosine 5 ' -",
  'existing_cui_desc': "[CLS] ad ##eno ##sin ##e 5 ' - [unused3] ( Chemical , O - met ##hyl ##ph ##os ##phate , O - met ##hyl ##ph ##os ##phate : ME ##PA ; 5 ' - Aden ##yl ##ic acid , mon ##ome ##thy ##l est ##er ) [SEP]",
  'our_bi_cui': 'C1417113',
  'our_bi_cui_name': 'MEP1A gene',
  'our_bi_cui_desc': '[CLS] ME ##P ##1 ##A gene [unused3] ( Ana ##tom ##ical Structure : ME ##PR ##IN , AL ##P ##HA S ##U ##B ##UN ##IT ; me ##p ##rin A subunit alpha ; PA ##BA p ##eptide h ##ydro ##lase ; PP ##HA ) [SEP]',
  'our_cross_cui': 'C1622778',
  'our_cross_cui_name': 'MEP1A protein , human',
  'our_cross_cui_desc': '[CLS] ME ##P ##1 ##A protein , human [unused3] ( Chemical : me ##p ##rin alpha protein , human ; me ##p ##rin A , alpha protein , human ; PA ##BA p ##eptide h ##ydro ##lase , human ) [SEP]'},
 {'mention_id': '28454015.31',
  'test_set_idx': 36532,
  'type': 'T170',
  'name': 'health - risk based air quality index',
  'context': '[CLS] weighted two - year average health - risk based air quality index data further demonstrates that all population in the studied cities in He ##nan province live with poll ##uted air - 72 % of the population is exposed to air that is un ##hea ##lt ##hy for sensitive people , while 28 % of people is exposed to air that can be harmful to healthy people ; and the health risks are much greater during winter than during other seasons . Future works should further improve the [unused1] health - risk based air quality index [unused2] algorithm , and valid ##ate the links between the clinical / e ##pid ##em ##iol ##og ##ic data and the health - risk based air quality index values . [SEP]',
  'existing_cui': 'C1268941',
  'existing_cui_name': 'Health risks education',
  'existing_cui_desc': '[CLS] Health risks education [unused3] ( Health Care Act ##ivity , procedure , regime / therapy : Counsel ##ing regarding health risks ; Ad ##vice about health risks ; Counsel ##ling regarding health risks ; Education regarding health risks ) [SEP]',
  'our_bi_cui': 'C0018761',
  'our_bi_cui_name': 'Health Status Indicators',
  'our_bi_cui_desc': '[CLS] Health Status In ##dic ##ators [unused3] ( Intel ##lect ##ual Product , assessment scale : Index ##es , Health Status ; In ##dic ##ators , Health Status ; health status index ; In ##dic ##ator , Health Status ; Health Status Index ; In ##dices , Health Status ; Health status indicators ; Health Status In ##dices ; Health status index ; Health Status In ##dic ##ator ; Index , Health Status ; Health Status Index ##es ; health status indicators ; health in ##dices status ) [SEP]',
  'our_cross_cui': 'C0918012',
  'our_cross_cui_name': 'Index',
  'our_cross_cui_desc': '[CLS] Index [unused3] ( Intel ##lect ##ual Product : index ##ed ; index ; index ##es ; Index ##es as Top ##ic ) [SEP]'},
 {'mention_id': '28000391.14',
  'test_set_idx': 23413,
  'type': 'T033',
  'name': 'Family Scale administered',
  'context': '[CLS] II . Little is known about the ps ##ych ##oso ##cial impact of inter ##sta ##ge home monitoring . Prospect ##ive mixed - method study . This study assessed the ps ##ych ##oso ##cial impact on parents during inter ##sta ##ge home monitoring . This contains for quantitative assessment the Short Form Health Survey question ##naire and the Impact of [unused1] Family Scale administered [unused2] one and five weeks following discharge before and after stage II . For q ##ual ##itative assessment , semi - structured interviews focus ##sing on the post ##dis ##cha ##rge cop ##ing strategies were conducted twice , five weeks after hospital discharge before and after stage II . Ten infants ( eight males ) with h ##y ##pop ##lastic left heart [SEP]',
  'existing_cui': 'C3539983',
  'existing_cui_name': 'Vaccines Administered',
  'existing_cui_desc': '[CLS] V ##ac ##cine ##s Ad ##mini ##stered [unused3] ( Intel ##lect ##ual Product : C ##V ##X ) [SEP]',
  'our_bi_cui': 'C0282574',
  'our_bi_cui_name': 'Intellectual Property',
  'our_bi_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28203067.6',
  'test_set_idx': 28163,
  'type': 'T033',
  'name': 'negative impact of care',
  'context': '[CLS] best predict ##ors of distress in family care ##rs of g ##eria ##tric patients The aim of this article was to identify the best predict ##ors of distress suffered by family care ##rs of g ##eria ##tric patients . A cross - section ##al study of 100 FC - g ##eria ##tric patient d ##ya ##ds was conducted . The [unused1] negative impact of care [unused2] sub ##sca ##le of the CO ##P ##E index was di ##cho ##tom ##ized to identify lower stress ( score of ≤ 15 on the scale ) and higher stress ( score of ≥ 16 on the scale ) ex ##ert ##ed on family care ##rs by the process of providing care . The set of ex ##p ##lana ##tory [SEP]',
  'existing_cui': 'C0726639',
  'existing_cui_name': 'Impact food supplement',
  'existing_cui_desc': '[CLS] Impact food supplement [unused3] ( Food : Impact ; impact ; I ##MP ##AC ##T ) [SEP]',
  'our_bi_cui': 'C3698401',
  'our_bi_cui_name': 'Carers of older people in Europe index negative impact score',
  'our_bi_cui_desc': '[CLS] Care ##rs of older people in Europe index negative impact score [unused3] ( Finding , o ##bs ##er ##vable entity ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28127853.7',
  'test_set_idx': 26301,
  'type': 'T058',
  'name': 'program',
  'context': '[CLS] Creative approach for successful aging : A pilot study of an inter ##gene ##ration ##al health promotion program To develop and evaluate the effectiveness of an inter ##gene ##ration ##al health promotion program . This was an action research project . A total of 34 participants attended the 12 - week [unused1] program [unused2] and completed the pre - test and post - test . There were 16 middle - aged and nine older adults recruited from a district of Taipei , and nine young adults recruited from the principal investigator \' s university . The " At ##ti ##tudes toward A ##ging Scale " and the " Spiritual Health Scale " were two assessment instruments used in the study . The results showed that there [SEP]',
  'existing_cui': 'C0237807',
  'existing_cui_name': 'Mental health program',
  'existing_cui_desc': '[CLS] Mental health program [unused3] ( Organization , Mental Health : health mental programs ; Programs ; Mental Health Programs ; mental health programs ; mental health program ) [SEP]',
  'our_bi_cui': 'C0032678',
  'our_bi_cui_name': 'Population Programs',
  'our_bi_cui_desc': '[CLS] Population Programs [unused3] ( Health Care Act ##ivity : Program , Population ; Programs , Population ; Population Program ) [SEP]',
  'our_cross_cui': 'C0043113',
  'our_cross_cui_name': 'Wellness Programs',
  'our_cross_cui_desc': '[CLS] Well ##ness Programs [unused3] ( Health Care Act ##ivity : program well ##ness ; programs well ##ness ; Programs , Well ##ness ; Well ##ness Program ; well ##ness programs ; programming well ##ness ; Program , Well ##ness ; well ##ness program ) [SEP]'},
 {'mention_id': '27580517.12',
  'test_set_idx': 10622,
  'type': 'T058',
  'name': '3D spiral - based pseudo - continuous arterial spin labeling technique',
  'context': '[CLS] patients in q ##uant ##ifying cerebral blood flow response to an ace ##ta ##zo ##lam ##ide v ##as ##od ##ila ##tor challenge . magnetic resonance imaging exams were performed on two 3 Te ##sla Philips In ##gen ##ia systems using 32 channel head coil array ##s . After local institutional review board approval , the [unused1] 3D spiral - based pseudo - continuous art ##erial spin label ##ing technique [unused2] was added to a standard brain magnetic resonance imaging exam and evaluated in 13 p ##ediatric patients ( average age : 11 . 7 ± 6 . 4 years , range : 1 . 4 - 22 . 2 years ) . All patients were administered ace ##ta ##zo ##lam ##ide for clinical ##ly indicated reasons [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0430022',
  'our_bi_cui_name': 'diagnostic procedure',
  'our_bi_cui_desc': '[CLS] diagnostic procedure [unused3] ( Health Care Act ##ivity , procedure : D ##IA ##G ##N ##OS ##TI ##C ; Di ##ag ##nos ##tic procedure ; Di ##ag ##nos ##tic procedures ; Di ##ag ##nos ##tic Test ; diagnostic technique ; Di ##ag ##nos ##tic ; Di ##ag ##nos ##tic Tech ##ni ##que ; Di ##ag ##nos ##tic Pro ##ced ##ure ; diagnostic procedures ; Di ##ag ##nos ##tic Method ; Pro ##ced ##ures ; diagnostic ) [SEP]',
  'our_cross_cui': 'C3891302',
  'our_cross_cui_name': 'Arterial Spin Labeling Magnetic Resonance Imaging',
  'our_cross_cui_desc': '[CLS] Arte ##rial Spin Label ##ing Ma ##gnetic Re ##son ##ance I ##maging [unused3] ( Health Care Act ##ivity : AS ##L f ##MR ##I ; Arte ##rial Spin Label ##ing MR ##I ; AR ##TE ##RI ##AL SP ##IN LA ##BE ##L ##ING F ##UN ##CT ##ION ##AL MR ##I ) [SEP]'},
 {'mention_id': '27542380.9',
  'test_set_idx': 9950,
  'type': 'T017',
  'name': 'foye-1 gene',
  'context': '[CLS] - stability properties . The genome of the acid ##op ##hil ##ic iron - o ##xi ##di ##zing bacterium " Fe ##rro ##vu ##m " s ##p . J ##A ##12 was found to harbor a the ##rm ##op ##hil ##ic - like en ##e - red ##uc ##tase ( F ##O ##Y ##E - 1 ) . The [unused1] f ##oy ##e - 1 gene [unused2] was l ##igate ##d into a p ##ET ##16 ##b ##p expression vector system , and the enzyme was produced in E ##scher ##ichi ##a co ##li B ##L ##21 ( DE ##3 ; p ##L ##ys ##S ) cells in yields of 10 mg L ( - 1 ) . F ##O ##Y ##E - 1 showed remarkable [SEP]',
  'existing_cui': 'C1332742',
  'existing_cui_name': 'CDT1 Gene',
  'existing_cui_desc': '[CLS] CD ##T ##1 Gene [unused3] ( Ana ##tom ##ical Structure : D ##O ##U ##BL ##E PA ##R ##KE ##D , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF ; ch ##roma ##tin licensing and DNA replication factor 1 ; CH ##RO ##MA ##TI ##N L ##IC ##EN ##SI ##NG AND DNA R ##EP ##L ##IC ##AT ##ION FA ##CT ##OR 1 ; DNA Rep ##lication Factor Gene ; R ##IS ##2 ) [SEP]',
  'our_bi_cui': 'C1414643',
  'our_bi_cui_name': 'FMO1 gene',
  'our_bi_cui_desc': '[CLS] FM ##O ##1 gene [unused3] ( Ana ##tom ##ical Structure : fl ##avi ##n containing mon ##oo ##xy ##gen ##ase 1 ; FM ##O , F ##ET ##AL L ##IVE ##R ; FL ##AV ##IN - CO ##NT ##A ##IN ##ING M ##ON ##O ##OX ##Y ##GE ##NA ##SE 1 ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27887026.30',
  'test_set_idx': 20815,
  'type': 'T103',
  'name': 'CmFDH',
  'context': '[CLS] substrate was evaluated by measuring the kinetic rates and conducting productivity ass ##ays . Can ##di ##da met ##hyl ##ica ( C ##m ##F ##D ##H ) and Cha ##eto ##mium the ##rm ##op ##hil ##um showed a higher efficiency in converting HC ##O ##3 ( - ) to format ##e than C ##m ##F ##D ##H , whereas [unused1] C ##m ##F ##D ##H [unused2] was better in the oxidation of format ##e . The pH op ##ti ##mum of the reduction was at pH 7 - 8 . However , the high concentrations of HC ##O ##3 ( - ) reduced the reaction rate . Can ##di ##da met ##hyl ##ica ( C ##m ##F ##D ##H ) and Cha ##eto ##mium the ##rm [SEP]',
  'existing_cui': 'C0342185',
  'existing_cui_name': 'Hyperthyroxinemia , Familial Dysalbuminemic',
  'existing_cui_desc': '[CLS] H ##yper ##thy ##ro ##xin ##emia , F ##ami ##lial D ##ys ##al ##bu ##mine ##mic [unused3] ( Ana ##tom ##ical Structure , disorder : H ##YP ##ER ##TH ##Y ##RO ##X ##IN ##EM ##IA , FA ##MI ##L ##IA ##L D ##Y ##SA ##L ##B ##UM ##IN ##EM ##IC ; F ##ami ##lial d ##ys ##al ##bu ##mina ##em ##ic h ##yper ##thy ##ro ##xin ##ae ##mia ; H ##yper ##thy ##ro ##xin ##emia , F ##ami ##lial D ##ys ##al ##bu ##mine ##mic [ Disease / Finding ] ; D ##ys ##al ##bu ##mine ##mic H ##yper ##thy ##ro ##xin ##emia , F ##ami ##lial ; F ##ami ##lial D ##ys ##al ##bu ##mine ##mic H ##yper ##thy ##ro ##xin ##emia ; F ##ami [SEP]',
  'our_bi_cui': 'C0060672',
  'our_bi_cui_name': 'formylmethanofuran dehydrogenase',
  'our_bi_cui_desc': '[CLS] form ##yl ##met ##han ##of ##ura ##n de ##hy ##dr ##ogen ##ase [unused3] ( Chemical , substance : FM ##F de ##hy ##dr ##ogen ##ase ; Form ##yl ##met ##han ##of ##ura ##n de ##hy ##dr ##ogen ##ase ) [SEP]',
  'our_cross_cui': 'C0016570',
  'our_cross_cui_name': 'Formate dehydrogenase',
  'our_cross_cui_desc': '[CLS] Form ##ate de ##hy ##dr ##ogen ##ase [unused3] ( Chemical , substance : De ##hy ##dr ##ogen ##ase , N ##AD - Form ##ate ; N ##AD Form ##ate De ##hy ##dr ##ogen ##ase ; De ##hy ##dr ##ogen ##ases , Form ##ate ; N ##AD - Form ##ate De ##hy ##dr ##ogen ##ase ; Form ##ate H ##ydro ##gen ##lya ##ses ; Form ##ate De ##hy ##dr ##ogen ##ases ; Form ##ate De ##hy ##dr ##ogen ##ases [ Chemical / In ##g ##red ##ient ] ; Form ##ate : N ##AD + o ##xi ##dor ##ed ##uc ##tase ; H ##ydro ##gen ##lya ##ses , Form ##ate ) [SEP]'},
 {'mention_id': '27325547.9',
  'test_set_idx': 2507,
  'type': 'T103',
  'name': 'PM2 .5',
  'context': '[CLS] health effects of PM increase as particle size decreases : particularly , great concern has risen on the role of Ultra ##F ##ine Part ##icles . Starting from the knowledge that the main fraction of atmospheric a ##ero ##sol in Rome is characterized by significant levels of PM ##2 . 5 ( almost 75 % of PM ##10 fraction is [unused1] PM ##2 . 5 [unused2] ) , the paper is focused on sub ##mic ##ron particles in such great urban area . The daytime / night ##time , work - / weekdays and cold / hot seasonal trends of sub ##mic ##ron particles will be investigated and discussed along with NO ##x and total PA ##H drift ##s demonstrating the primary origin of Ultra ##F [SEP]',
  'existing_cui': 'C0665987',
  'existing_cui_name': '1-(4-aminobutyl)-6-benzylindane',
  'existing_cui_desc': '[CLS] 1 - ( 4 - amino ##but ##yl ) - 6 - ben ##zy ##lind ##ane [unused3] ( Chemical : PM ##2 c ##p ##d ) [SEP]',
  'our_bi_cui': 'C0597177',
  'our_bi_cui_name': 'Particle',
  'our_bi_cui_desc': '[CLS] Part ##icle [unused3] ( Chemical : particle ; particles ; Part ##icles ; a very small piece or part ) [SEP]',
  'our_cross_cui': 'C1510837',
  'our_cross_cui_name': 'Airborne Particulate Matter',
  'our_cross_cui_desc': '[CLS] Airborne Part ##iculate Matter [unused3] ( Chemical : Air Poll ##uta ##nts , Part ##iculate ; Am ##bie ##nt Part ##iculate Matter ; Part ##iculate Matter , Am ##bie ##nt ; Part ##iculate Matter , Airborne ; Poll ##uta ##nts , Part ##iculate Air ; Part ##iculate Air Poll ##uta ##nts ) [SEP]'},
 {'mention_id': '28540429.6',
  'test_set_idx': 38811,
  'type': 'T017',
  'name': 'fibrogenic cells',
  'context': '[CLS] I ##dent ##ification of markers for q ##ui ##es ##cent pan ##cre ##atic s ##tel ##late cells in the normal human pan ##cre ##as Pan ##cre ##atic s ##tel ##late cells play a central role as source of [unused1] fi ##bro ##genic cells [unused2] in pan ##cre ##atic cancer and chronic pan ##cre ##ati ##tis . In contrast to q ##ui ##es ##cent he ##pa ##tic s ##tel ##late cells , a specific marker for q ##ui ##es ##cent Pan ##cre ##atic s ##tel ##late cells that can be used in formal ##in - fixed and para ##ffin embedded normal human pan ##cre ##atic tissue has not been identified . The aim of this study was to identify a marker enabling the identification of q ##ui [SEP]',
  'existing_cui': 'C0206643',
  'existing_cui_name': 'Neoplasms , Fibrous Tissue',
  'existing_cui_desc': '[CLS] Neo ##p ##las ##ms , Fi ##bro ##us T ##iss ##ue [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis : Fi ##bro ##us T ##iss ##ue Neo ##p ##las ##m ; F ##IB ##RO ##US T ##IS ##SU ##E NE ##OP ##L ; Fi ##bro ##cy ##tic Neo ##p ##las ##m ; Neo ##p ##las ##m , Fi ##bro ##us T ##iss ##ue ; Fi ##bro ##genic Neo ##p ##las ##m ; Fi ##bro ##blast ##ic Neo ##p ##las ##m ; Fi ##bro ##us Tu ##mor ; Fi ##bro ##us T ##iss ##ue Neo ##p ##las ##ms ; Fi ##bro ##blast ##ic Tu ##mor ; neo ##p ##las ##m of fi ##bro ##us tissue ; Fi ##bro ##genic ##T ##um ##or ; Neo ##p [SEP]',
  'our_bi_cui': 'C0016030',
  'our_bi_cui_name': 'Fibroblasts',
  'our_bi_cui_desc': '[CLS] Fi ##bro ##blast ##s [unused3] ( Ana ##tom ##ical Structure , cell : Fi ##bro ##blast ##s [ Chemical / In ##g ##red ##ient ] ; fi ##bro ##blast cell ; fi ##bro ##blast ; Fi ##bro ##blast ; cells fi ##bro ##blast ; fi ##bro ##blast ##s ) [SEP]',
  'our_cross_cui': 'C0007634',
  'our_cross_cui_name': 'Cells',
  'our_cross_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]'},
 {'mention_id': '27317610.20',
  'test_set_idx': 2184,
  'type': 'T038',
  'name': 'stress',
  'context': '[CLS] ##ology , and subsequent h ##y ##pot ##hala ##mic - pit ##uit ##ary end ##oc ##rine d ##ys ##function . These experiments were designed to test the hypothesis that a single di ##ff ##use traumatic brain injury results in chronic d ##ys ##function of co ##rt ##ico ##ster ##one , a g ##lu ##co ##cor ##tic ##oid released in response to [unused1] stress [unused2] and test ##osterone . We used a rode ##nt model of di ##ff ##use traumatic brain injury induced by mid ##line fluid percussion injury . At 2 months post ##in ##ju ##ry compared with un ##in ##jured control animals , circulating levels of co ##rt ##ico ##ster ##one were evaluated at rest , under restraint stress and in response to de ##xa [SEP]',
  'existing_cui': 'C0038435',
  'existing_cui_name': 'Stress',
  'existing_cui_desc': '[CLS] St ##ress [unused3] ( Finding , finding , qualifier value : St ##ress - value ; state ; stress ; Press ##ure ; stressed ; State of stress ; stress ; stress ; state ) [SEP]',
  'our_bi_cui': 'C0038435',
  'our_bi_cui_name': 'Stress',
  'our_bi_cui_desc': '[CLS] St ##ress [unused3] ( Finding , finding , qualifier value : St ##ress - value ; state ; stress ; Press ##ure ; stressed ; State of stress ; stress ; stress ; state ) [SEP]',
  'our_cross_cui': 'C0449430',
  'our_cross_cui_name': 'Physiological Stress',
  'our_cross_cui_desc': '[CLS] Ph ##ys ##iol ##ogical St ##ress [unused3] ( B ##iol ##og ##ic Fun ##ction , attribute : St ##ress , Ph ##ys ##iol ##ogical ; St ##resses , Ph ##ys ##iol ##ogical ; Ph ##ys ##iol ##ogical St ##resses ; ST ##RE ##SS P ##H ##Y ##SI ##OL ; P ##H ##Y ##SI ##OL ST ##RE ##SS ; Ph ##ys ##iol ##ogical stress ; stress physiological ; physiological stress ) [SEP]'},
 {'mention_id': '27732552.11',
  'test_set_idx': 15248,
  'type': 'T058',
  'name': 'technique',
  'context': '[CLS] tool for the monitoring of H ##b ##A ##1 ##c in diabetes patients Micro ##sa ##mp ##ling techniques have several advantages over traditional blood collection . Dr ##ied blood spot sampling and blood collection with he ##par ##ini ##zed cap ##illa ##ries are the standard techniques . Volume ##tric a ##bs ##or ##pt ##ive micro ##sa ##mp ##ling is a novel [unused1] technique [unused2] that collects a fixed volume of blood by applying an absorb ##ent tip to a blood drop . In the present study we explored the f ##eas ##ibility of H ##b ##A ##1 ##c monitoring with Volume ##tric a ##bs ##or ##pt ##ive micro ##sa ##mp ##ling sampling at home and analysis in the laboratory . Di ##abe ##tic patients were enrolled [SEP]',
  'existing_cui': 'C0450241',
  'existing_cui_name': 'Ffooks technique',
  'existing_cui_desc': '[CLS] F ##fo ##ok ##s technique [unused3] ( Health Care Act ##ivity , qualifier value ) [SEP]',
  'our_bi_cui': 'C0025663',
  'our_bi_cui_name': 'Methods',
  'our_bi_cui_desc': '[CLS] Method ##s [unused3] ( Intel ##lect ##ual Product , attribute : methods ; Method ; method ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '28064539.7',
  'test_set_idx': 24490,
  'type': 'T204',
  'name': 'Foucauldian',
  'context': '[CLS] T ##rea ##ting the bin ##ge or the ( fat ) body ? Rep ##res ##entation ##s of fat ##ness in a gold standard psychological treatment manual for bin ##ge eating disorder This article reports the results of a [unused1] F ##ou ##ca ##uld ##ian [unused2] - informed discourse analysis exploring representations of fat ##ness embedded within an empirical ##ly based psychological treatment manual for bin ##ge eating disorder , a condition characterized by over ##val ##uation of weight and shape . Ana ##ly ##ses indicate that the manual prior ##iti ##zes weight loss with relatively less emphasis placed on treating the diagnostic symptoms and underlying mechanisms of bin ##ge eating disorder . We raise critical concerns about these observations and link our findings to mainstream [SEP]',
  'existing_cui': 'C1422766',
  'existing_cui_name': 'MEG8 gene',
  'existing_cui_desc': '[CLS] ME ##G ##8 gene [unused3] ( Ana ##tom ##ical Structure , non - protein coding : L ##IN ##C ##00 ##0 ##24 ; MA ##TE ##RNA ##LL ##Y E ##X ##PR ##ES ##SE ##D GE ##NE 8 ; maternal ##ly expressed 8 ; RNA I ##mp ##rin ##ted and A ##cc ##um ##ulated in N ##uc ##le ##us ; non - protein coding RNA 24 ; long inter ##genic non - protein coding RNA 24 ; I ##mp ##rin ##ted RNA near Meg ##3 / G ##tl ##2 ; B ##s ##r ; NC ##RNA ##00 ##0 ##24 ) [SEP]',
  'our_bi_cui': 'C1522486',
  'our_bi_cui_name': 'Professional Organization or Group',
  'our_bi_cui_desc': '[CLS] Professional Organization or Group [unused3] ( Professional or O ##cc ##upation ##al Group : Other Agency or Organization ; Organization ; OR ##G ; Outside Bo ##dies ; Organization ##al ) [SEP]',
  'our_cross_cui': 'C0086418',
  'our_cross_cui_name': 'Homo sapiens',
  'our_cross_cui_desc': '[CLS] Ho ##mo sa ##pie ##ns [unused3] ( E ##uka ##ryo ##te , Tax ##onomy , organism , qualifier value : Humans ; Man , Modern ; humans ; man ; MA ##N ; Ho ##mo sa ##pie ##ns ( living organism ) [ Am ##bi ##guous ] ; ho ##mo sa ##pie ##ns ; Human , General ; Ho ##mo sa ##pie ##ns Linnaeus , 1758 ; Human ; Human - origin ; human ; Ho ##mo sa ##pie ##ns ( living organism ) ; Modern Man ; Man ) [SEP]'},
 {'mention_id': '27335087.93',
  'test_set_idx': 2822,
  'type': 'T033',
  'name': 'No hepatorenal toxicity',
  'context': '[CLS] . In addition , Fu ##zhen ##g Qing ##ji ##e increased the percentage ##s of CD ##4 ( + ) T and natural killer cells cells , the ratio of CD ##4 ( + ) / CD ##8 ( + ) T cells as well as the levels of serum T ##NF - α . Fu ##zhen ##g Qing ##ji ##e also increased CD ##6 ##9 expression in tumor tissue . [unused1] No he ##pa ##tore ##nal toxicity [unused2] was observed in H ##22 tumor - bearing mice . These results indicated that Fu ##zhen ##g Qing ##ji ##e could in ##hibit the growth of he ##pa ##tom ##a cells via regulating immune function and in ##ducing mit ##och ##ond ##ria mediated a ##pop ##tosis . [SEP]',
  'existing_cui': 'C0595916',
  'existing_cui_name': 'Toxic nephropathy',
  'existing_cui_desc': '[CLS] To ##xi ##c ne ##ph ##rop ##athy [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : toxic ne ##ph ##rop ##athy ; toxic ; ne ##ph ##rop ##athy ; Kid ##ney To ##xi ##city ; To ##xi ##c ne ##ph ##rosis ; N ##ep ##hr ##op ##athy toxic ; ne ##ph ##rop ##athy toxic ; re ##nal toxicity ; To ##xi ##city re ##nal ; Ren ##al To ##xi ##city ; ne ##ph ##rop ##athy ; toxic ) [SEP]',
  'our_bi_cui': 'C0857329',
  'our_bi_cui_name': 'No toxic effect',
  'our_bi_cui_desc': '[CLS] No toxic effect [unused3] ( Finding ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27510458.35',
  'test_set_idx': 8922,
  'type': 'T033',
  'name': 'double lumen signs',
  'context': '[CLS] . We assessed whether these imaging methods could visual ##ize an in ##ti ##mal flap and / or double l ##ume ##n sign in the participants and compared the results between high - resolution fast spin echo and the other m ##oda ##lities . high - resolution fast spin echo images clearly showed in ##ti ##mal flap ##s and [unused1] double l ##ume ##n signs [unused2] in all 3 patients , whereas the conventional m ##oda ##lities identified a double l ##ume ##n sign in only 2 of the 3 patients . The present method of op ##ti ##mized high - resolution fast spin echo imaging with a 3 ##T system improved visual ##ization of in ##ti ##mal flap ##s and should thus be considered for [SEP]',
  'existing_cui': 'C0441299',
  'existing_cui_name': 'Double lumen tube',
  'existing_cui_desc': '[CLS] Double l ##ume ##n tube [unused3] ( Medical Devi ##ce , physical object ) [SEP]',
  'our_bi_cui': 'C0233192',
  'our_bi_cui_name': "Ladin 's sign",
  'our_bi_cui_desc': "[CLS] La ##din ' s sign [unused3] ( Finding , finding : La ##din sign ) [SEP]",
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27986797.27',
  'test_set_idx': 22924,
  'type': 'T103',
  'name': 'myomitokine',
  'context': '[CLS] with skeletal muscle - specific deficiency of C ##ri ##f ##1 ( muscle - specific knockout [ M ##KO ] ) , an integral protein of the large mit ##ori ##bos ##oma ##l subunit ( 39 ##S ) , we identified growth differentiation factor 15 as a UP ##R ( m ##t ) - associated cell - non - autonomous [unused1] my ##omi ##tok ##ine [unused2] that regulate ##s systemic energy home ##ost ##asis . muscle - specific knockout mice were protected against o ##besity and se ##ns ##iti ##zed to insulin , an effect associated with elevated growth differentiation factor 15 secret ##ion after UP ##R ( m ##t ) activation . In o ##b / o ##b mice , administration of re ##comb ##ina [SEP]',
  'existing_cui': 'C0950133',
  'existing_cui_name': 'Online Mendelian Inheritance In Man',
  'existing_cui_desc': '[CLS] Online Men ##del ##ian In ##her ##ita ##nce In Man [unused3] ( Intel ##lect ##ual Product : online men ##del ##ian inheritance in man ( O ##MI ##M ) database ; Online Men ##del ##ian In ##her ##ita ##nce in Man ) [SEP]',
  'our_bi_cui': 'C0026484',
  'our_bi_cui_name': 'Monokines',
  'our_bi_cui_desc': '[CLS] Mon ##oki ##nes [unused3] ( Chemical , substance : Mon ##oki ##ne ; mon ##oki ##ne ; Mon ##oki ##nes [ Chemical / In ##g ##red ##ient ] ) [SEP]',
  'our_cross_cui': 'C0574031',
  'our_cross_cui_name': 'Biologically active substance',
  'our_cross_cui_desc': '[CLS] Biological ##ly active substance [unused3] ( Chemical , substance : Biological ##ly Active Sub ##stance ) [SEP]'},
 {'mention_id': '27570140.49',
  'test_set_idx': 10512,
  'type': 'T058',
  'name': 'procedure',
  'context': "[CLS] the 27 months of follow - up . Though women with high - risk pre ##gna ##ncies were more likely to request post ##par ##tum tub ##al l ##iga ##tion , they were not more likely to complete the procedure . Over one third of high - risk patients ' requests were un ##ful ##fill ##ed , indicating that significant barriers may remain . Though women with high - risk pre ##gna ##ncies were more likely to request post ##par ##tum tub ##al l ##iga ##tion , they were not more likely to complete the [unused1] procedure [unused2] . Pro ##vide ##rs should consider these procedures urgent , especially in high - risk women , and advocate for their patients ' access to this procedure . [SEP]",
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '28294559.26',
  'test_set_idx': 31482,
  'type': 'T204',
  'name': 'chimeric mice',
  'context': '[CLS] essential role in liver regeneration after HP ##x . Background and aim The aim of the present study was to investigate the role of C - type le ##ct ##in receptor ( C - type le ##ct ##in receptor ) - 2 in liver regeneration following partial liver re ##section in mice . Materials and methods I ##rra ##dia ##ted [unused1] ch ##ime ##ric mice [unused2] transplant ##ed with f ##etal liver cells from wild - type mice , C - type le ##ct ##in receptor - 2 - deleted ( K ##O ) mice or mice with C - type le ##ct ##in receptor - 2 deleted specifically from plate ##lets ( fl ##KO ) were generated . Mi ##ce underwent 70 % partial he [SEP]',
  'existing_cui': 'C0676831',
  'existing_cui_name': 'basiliximab',
  'existing_cui_desc': '[CLS] b ##asi ##lix ##ima ##b [unused3] ( Chemical , Si ##mu ##lect , medication , product , substance : Basil ##ix ##ima ##b ; b ##asi ##lix ##ima ##b [ Chemical / In ##g ##red ##ient ] ; ch ##ime ##ric mouse - human anti ##CD ##25 ; I ##g gamma - 1 chain C region ; BA ##SI ##L ##IX ##IM ##AB ) [SEP]',
  'our_bi_cui': 'C0025936',
  'our_bi_cui_name': 'Mice , Transgenic',
  'our_bi_cui_desc': '[CLS] Mi ##ce , Trans ##genic [unused3] ( E ##uka ##ryo ##te : Mouse , Trans ##genic ; mouse trans ##genic ##s ; Trans ##genic Mi ##ce ; mice trans ##genic ; Trans ##genic Mouse ; trans ##genic mice ; Trans ##genic mice ; trans ##genic mouse ) [SEP]',
  'our_cross_cui': 'C0025929',
  'our_cross_cui_name': 'Laboratory mice',
  'our_cross_cui_desc': '[CLS] Laboratory mice [unused3] ( E ##uka ##ryo ##te , organism : Mouse , Laboratory ; Mouse , laboratory ; Laboratory Mi ##ce ; Mi ##ce , Laboratory ; mouse ; mice ; Laboratory Mouse ; Laboratory mouse ; laboratory mouse ) [SEP]'},
 {'mention_id': '28234635.85',
  'test_set_idx': 29253,
  'type': 'T017',
  'name': 'canalicular',
  'context': '[CLS] of L ##X ##R and ABC ##G ##5 / 8 RNA expression was suppressed in patients with portal inflammation . Current parent ##eral nutrition , increased serum levels of plant s ##tero ##ls s ##ti ##gma ##ster ##ol , a ##ven ##aster ##ol , and sit ##ost ##ero ##l along with serum c ##it ##ru ##llin ##e , a marker of enter ##oc ##yte mass , predicted portal inflammation . In p ##ediatric onset in ##test ##inal failure , current parent ##eral nutrition delivery s ##yne ##rg ##istic ##ally with in ##test ##inal compromise promote liver inflammation , which associates with progression of bio ##chemical and his ##to ##log ##ic liver injury , while reducing expression of [unused1] canal ##icular [unused2] bi ##le transport ##ers . [SEP]',
  'existing_cui': 'C0339135',
  'existing_cui_name': 'Canalicular laceration',
  'existing_cui_desc': '[CLS] Canal ##icular lace ##ration [unused3] ( In ##ju ##ry or Po ##ison ##ing , disorder ) [SEP]',
  'our_bi_cui': 'C1159621',
  'our_bi_cui_name': 'canalicular bile acid transport',
  'our_bi_cui_desc': '[CLS] canal ##icular bi ##le acid transport [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_cross_cui': 'C0005393',
  'our_cross_cui_name': 'Structure of bile canaliculus',
  'our_cross_cui_desc': '[CLS] Structure of bi ##le canal ##ic ##ulus [unused3] ( Ana ##tom ##ical Structure , body structure : Canal ##ic ##ulus , B ##ile ; B ##ile Canal ##ic ##ulus ; B ##ile canal ##ic ##ulus ; Canal ##ic ##uli , B ##ile ; B ##ile Canal ##ic ##uli ) [SEP]'},
 {'mention_id': '28125734.69',
  'test_set_idx': 26223,
  'type': 'T103',
  'name': 'MiR-21-5p',
  'context': '[CLS] ##R - 21 - 5 ##p as a true regulator of me ##so ##the ##lin . Moreover , in v ##it ##ro experiments showed that treatment with mi ##R - 21 - 5 ##p mimic reduced proliferation of ma ##li ##gnant p ##le ##ural me ##so ##the ##lio ##ma cell lines . Alto ##get ##her , this work shows that the mi ##R - CA ##TC ##H technique , coupled with next generation se ##quencing and in v ##it ##ro valid ##ation , represents a reliable method to identify native mi ##RNA : : m ##RNA interactions . [unused1] Mi ##R - 21 - 5 ##p [unused2] is suggested as novel regulator of me ##so ##the ##lin with a possible functional role in cellular growth . [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1999986',
  'our_bi_cui_name': 'MIRN21 microRNA , human',
  'our_bi_cui_desc': '[CLS] MI ##R ##N ##21 micro ##RNA , human [unused3] ( Chemical : micro ##RNA - 21 , human ; mi ##RNA - 21 , human ; h ##sa - mi ##r - 21 micro ##RNA ; mi ##R - 21 , human ; mi ##R ##21 , human ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '27473429.29',
  'test_set_idx': 7344,
  'type': 'T082',
  'name': 'segments',
  'context': '[CLS] ##8 , and 110 ##1 b ##p ) , which has demonstrated that in all cases , polymer ##ase chain reaction is successful at least on one station . In addition , three DNA fragment s with different lengths ( 32 ##3 , 110 ##1 , and 28 ##36 b ##p ) have been successfully am ##plified in a segment ##ed - flow mode without the carry - over contamination between [unused1] segments [unused2] . This result suggests that this device could serve as the polymer ##ase chain reaction module of a continuous - flow high - through ##put on - line total DNA analysis system integrating all necessary modules from cell l ##ys ##is / DNA extraction to polymer ##ase chain reaction product analysis . [SEP]',
  'existing_cui': 'C0441635',
  'existing_cui_name': 'Anatomical segmentation',
  'existing_cui_desc': '[CLS] Ana ##tom ##ical segment ##ation [unused3] ( Spa ##tial Concept , qualifier value : Se ##gment ##ed ; Se ##gment ##ation ; segment ##ation ; segment ; SE ##GM ##EN ##T ; Se ##gment ) [SEP]',
  'our_bi_cui': 'C0162326',
  'our_bi_cui_name': 'DNA Sequence',
  'our_bi_cui_desc': '[CLS] DNA Se ##quence [unused3] ( Spa ##tial Concept : DNA Se ##quence ##s ; Se ##quence ##s ; DNA SE ##Q ; d ##na sequences ; Se ##quence ##s , DNA ; DNA sequence ; Se ##quence ; d ##na sequence ; DNA se ##quencing ; Se ##quence , DNA ) [SEP]',
  'our_cross_cui': 'C0442059',
  'our_cross_cui_name': 'Segment structure',
  'our_cross_cui_desc': '[CLS] Se ##gment structure [unused3] ( Spa ##tial Concept , qualifier value ) [SEP]'},
 {'mention_id': '27456018.8',
  'test_set_idx': 6374,
  'type': 'T058',
  'name': 'SL',
  'context': '[CLS] is a technique perceived to provide the benefits of lap ##aro ##scopic surgery while improving ta ##ct ##ile feedback and operative time . Published data are largely limited to small , single - institution studies . The 2012 - 2013 National Sur ##gical Quality Improvement Program Part ##ici ##pan ##t Data Use File was que ##ried for patients undergoing elect ##ive [unused1] SL [unused2] or H ##AL co ##lect ##omy . Pat ##ients underwent 1 : 1 prop ##ens ##ity matching and had outcomes compared . An additional subgroup analysis was performed for patients undergoing segment ##al re ##section ##s only . 13 , 94 ##9 patients were identified , of whom 60 ##8 ##4 ( 43 . 6 % ) underwent H ##AL co ##lect [SEP]',
  'existing_cui': 'C2338991',
  'existing_cui_name': 'Simple lobule of cerebellum',
  'existing_cui_desc': '[CLS] Simple lo ##bul ##e of c ##ere ##bell ##um [unused3] ( Ana ##tom ##ical Structure , posterior : Post ##eri ##or c ##res ##cent ##ic lo ##bul ##e of c ##ere ##bell ##um ; Lo ##bul ##us q ##uad ##rang ##ular ##is par ##s in ##fer ##op ##ost ##eri ##or ; Lo ##bul ##us simple ##x c ##ere ##bell ##i [ H VI et VI ] ; Semi ##lu ##nar lo ##bul ##e - 1 ; Lo ##bul ##us simple ##x ; Lo ##bul ##e VI of hemisphere of c ##ere ##bell ##um ; Simple ##x ; Lo ##bul ##us q ##uad ##rang ##ular ##is par ##s ca ##uda ##lis / posterior ; Simple lo ##bul ##e of c ##ere ##bell ##um [ H VI and [SEP]',
  'our_bi_cui': 'C0585462',
  'our_bi_cui_name': 'Laparoscopic-assisted sigmoid colectomy',
  'our_bi_cui_desc': '[CLS] La ##par ##os ##copic - assisted si ##g ##mo ##id co ##lect ##omy [unused3] ( Health Care Act ##ivity , procedure : La ##par ##os ##copic - assisted si ##g ##mo ##ide ##ct ##omy ; La ##par ##os ##copic si ##g ##mo ##id co ##lect ##omy ) [SEP]',
  'our_cross_cui': 'C1517722',
  'our_cross_cui_name': 'Laparoscopic colectomy',
  'our_cross_cui_desc': '[CLS] La ##par ##os ##copic co ##lect ##omy [unused3] ( Health Care Act ##ivity : La ##par ##os ##copic Cole ##ct ##omy ; La ##par ##os ##copy - As ##sist ##ed Cole ##ct ##omy ; lap ##aro ##scopic - assisted co ##lect ##omy ; La ##par ##os ##copic - As ##sist ##ed Cole ##ct ##omy ) [SEP]'},
 {'mention_id': '27297967.84',
  'test_set_idx': 1789,
  'type': 'T082',
  'name': 'Asn19/23Gln',
  'context': '[CLS] ##nic kidney 29 ##3 - As ##n ##19 / 23 ##G ##ln / Ty ##r ##9 ##20 ##G ##lu / Ser ##9 ##21 ##G ##lu were similar to H ##E ##K - W ##T - MR ##P ##1 , indicating that the p ##hos ##ph ##ory ##lation - mimic ##king substitution ##s a ##bro ##gated the influence of [unused1] As ##n ##19 / 23 ##G ##ln [unused2] g ##ly ##cos ##yla ##tion . Overall , these data suggest that cross - talk between MR ##P ##1 g ##ly ##cos ##yla ##tion and p ##hos ##ph ##ory ##lation occurs and that p ##hos ##ph ##ory ##lation of Ty ##r ##9 ##20 and Ser ##9 ##21 can switch MR ##P ##1 to a lower - affinity , higher [SEP]',
  'existing_cui': 'C1425905',
  'existing_cui_name': 'KRTAP19-2 gene',
  'existing_cui_desc': '[CLS] K ##RT ##AP ##19 - 2 gene [unused3] ( Ana ##tom ##ical Structure : k ##era ##tin associated protein 19 - 2 ; K ##AP ##19 . 2 ) [SEP]',
  'our_bi_cui': 'C0002520',
  'our_bi_cui_name': 'Amino Acids',
  'our_bi_cui_desc': '[CLS] Am ##ino A ##cid ##s [unused3] ( Chemical , medication , product , substance : Am ##ino acid ; A ##cid ##s , Am ##ino ; amino acid preparations ; amino ##ac ##id ; Am ##ino A ##cid ; amino acids ; Am ##ino A ##cid ##s [ Chemical / In ##g ##red ##ient ] ; amino ##ac ##ids ; Am ##ino acids ; AM ##IN ##O AC ##ID PR ##EP ##AR ##AT ##ION ; amino acid ; AM ##IN ##O AC ##ID ##S ; AA - Am ##ino acid ) [SEP]',
  'our_cross_cui': 'C0002518',
  'our_cross_cui_name': 'Amino Acid Sequence',
  'our_cross_cui_desc': '[CLS] Am ##ino A ##cid Se ##quence [unused3] ( Spa ##tial Concept , finding : Se ##quence ##s , Am ##ino A ##cid ; Am ##ino A ##cid Se ##quence ##s ; SE ##Q AM ##IN ##O AC ##ID ; Pro ##tein Se ##quence ; amino acid sequences ; SE ##Q AA ; Structure ##s , Primary Pro ##tein ; acid amino sequences ; amino acid sequence ; Primary Pro ##tein Structure ##s ; AM ##IN ##O AC ##ID SE ##Q ; Structure , Primary Pro ##tein ; AA SE ##Q ; Am ##ino acid sequence ; protein se ##quencing ; protein sequences ; Pro ##tein Structure ##s , Primary ; Se ##quence , Am ##ino A ##cid ; protein sequence ; AM ##IN ##O AC ##ID [SEP]'},
 {'mention_id': '28059412.13',
  'test_set_idx': 24440,
  'type': 'T103',
  'name': 'polyketide',
  'context': '[CLS] ##hesis J ##aws ##am ##y ##cin is a p ##oly ##ket ##ide - n ##uc ##leo ##side hybrid with a unique p ##oly ##cy ##c ##lop ##rop ##ane m ##oi ##ety on a single p ##oly ##ket ##ide chain . The unexpected isolation of c ##y ##c ##lop ##rop ##ane def ##icient jaws ##am ##y ##cin analog ##s allowed us to propose a step ##wise c ##y ##c ##lop ##rop ##ana ##tion mechanism for the en ##zy ##matic synthesis of this [unused1] p ##oly ##ket ##ide [unused2] . The con ##cise timing of the c ##y ##c ##lop ##rop ##ana ##tion could be regulated by a delicate balance between reaction rates of the con ##den ##sation and c ##y ##c ##lop ##rop ##ana ##tion reactions . [SEP]',
  'existing_cui': 'C1157699',
  'existing_cui_name': 'polyketide biosynthetic process',
  'existing_cui_desc': '[CLS] p ##oly ##ket ##ide bio ##sy ##nt ##hetic process [unused3] ( B ##iol ##og ##ic Fun ##ction : p ##oly ##ket ##ide formation ; p ##oly ##ket ##ide an ##ab ##olis ##m ; p ##oly ##ket ##ide synthesis ; p ##oly ##ket ##ide bio ##sy ##nt ##hesis ) [SEP]',
  'our_bi_cui': 'C0030956',
  'our_bi_cui_name': 'Peptides',
  'our_bi_cui_desc': '[CLS] P ##eptide ##s [unused3] ( Chemical , substance : p ##eptide ; p ##eptide ##s ; P ##eptide ##s [ Chemical / In ##g ##red ##ient ] ; P ##eptide ) [SEP]',
  'our_cross_cui': 'C3179052',
  'our_cross_cui_name': 'Polyketides',
  'our_cross_cui_desc': '[CLS] Pol ##yke ##tide ##s [unused3] ( Chemical : Pol ##yke ##tide ##s [ Chemical / In ##g ##red ##ient ] ) [SEP]'},
 {'mention_id': '28457334.35',
  'test_set_idx': 36645,
  'type': 'T103',
  'name': 'extracellular RNA',
  'context': '[CLS] chain reaction . In addition , th ##rom ##boe ##last ##ography for co ##agu ##lation as well as th ##rom ##boa ##gg ##re ##go ##metry for plate ##let function was conducted . Both Cell - free DNA and extra ##cellular RNA were elevated in patients with se ##psis compared with post ##oper ##ative patients and healthy volunteers . While higher [unused1] extra ##cellular RNA [unused2] levels correlated with a faster c ##lot ##ting time and more stable c ##lot ##s , Cell - free DNA correlated with a shorter c ##lot ##ting time but also less fi ##bri ##no ##lysis . In addition , higher Cell - free DNA seems to be associated with kidney d ##ys ##function as well as with general markers of cell [SEP]',
  'existing_cui': 'C0521119',
  'existing_cui_name': 'Extracellular',
  'existing_cui_desc': '[CLS] Extra ##cellular [unused3] ( Ana ##tom ##ical Structure , qualifier value : extra ##cellular ; extra ##cellular region ) [SEP]',
  'our_bi_cui': 'C0230885',
  'our_bi_cui_name': 'Extracellular material',
  'our_bi_cui_desc': '[CLS] Extra ##cellular material [unused3] ( Body Sub ##stance , substance ) [SEP]',
  'our_cross_cui': 'C0035668',
  'our_cross_cui_name': 'RNA',
  'our_cross_cui_desc': '[CLS] RNA [unused3] ( Chemical , RNA , R ##ib ##on ##uc ##le ##ic A ##cid , substance : rib ##on ##uc ##le ##ic acid ; R ##ib ##on ##uc ##le ##ic acids ; RNA , Non - Pol ##ya ##den ##yla ##ted ; rib ##on ##uc ##lein ##ic ##um acid ##um / rib ##on ; RNA - R ##ib ##on ##uc ##le ##ic acid ; RNA [ Chemical / In ##g ##red ##ient ] ; RNA , Non Pol ##ya ##den ##yla ##ted ; Non - Pol ##ya ##den ##yla ##ted RNA ; r ##na ; Non Pol ##ya ##den ##yla ##ted RNA ; A ##cid , R ##ib ##on ##uc ##le ##ic ; R ##ib ##on ##uc ##le ##ic acid ; Gene Products , RNA [SEP]'},
 {'mention_id': '27708411.23',
  'test_set_idx': 14732,
  'type': 'T103',
  'name': 'dsDNA',
  'context': '[CLS] ##rote ##in fi ##lam ##ent formation , is unable to media ##te ho ##mo ##log ##ous re ##comb ##ination . Mu ##tations of the C - terminal domain were evaluated using both single - and double stranded ( s ##s and d ##s ) substrates in re ##comb ##ination ass ##ays . Mu ##tations of critical amino acids affected either [unused1] d ##s ##D ##NA [unused2] re ##comb ##ination or both s ##s ##D ##NA and d ##s ##D ##NA re ##comb ##ination indicating two se ##par ##able functions , one of which is critical for d ##s ##D ##NA re ##comb ##ination and the second for re ##comb ##ination per se . As evaluated by co - im ##mu ##no ##p ##re ##ci ##pit ##ation [SEP]',
  'existing_cui': 'C0012854',
  'existing_cui_name': 'DNA',
  'existing_cui_desc': '[CLS] DNA [unused3] ( Chemical , DNA , De ##ox ##yr ##ib ##on ##uc ##le ##ic A ##cid , substance : d ##na molecules ; d ##s ##D ##NA ; De ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; d ##na / des ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; DNA / des ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; DNA molecule ; DNA - De ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; De ##ox ##yr ##ib ##on ##uc ##le ##ic A ##cid ; De ##ox ##yr ##ib ##on ##uc ##le ##ic acids ; d ##na molecule ; DNA [ Chemical / In ##g ##red ##ient ] ; de ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; Double - [SEP]',
  'our_bi_cui': 'C0012854',
  'our_bi_cui_name': 'DNA',
  'our_bi_cui_desc': '[CLS] DNA [unused3] ( Chemical , DNA , De ##ox ##yr ##ib ##on ##uc ##le ##ic A ##cid , substance : d ##na molecules ; d ##s ##D ##NA ; De ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; d ##na / des ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; DNA / des ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; DNA molecule ; DNA - De ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; De ##ox ##yr ##ib ##on ##uc ##le ##ic A ##cid ; De ##ox ##yr ##ib ##on ##uc ##le ##ic acids ; d ##na molecule ; DNA [ Chemical / In ##g ##red ##ient ] ; de ##ox ##yr ##ib ##on ##uc ##le ##ic acid ; Double - [SEP]',
  'our_cross_cui': 'C0311474',
  'our_cross_cui_name': 'DNA , Double-Stranded',
  'our_cross_cui_desc': '[CLS] DNA , Double - Strand ##ed [unused3] ( Chemical , substance : double stranded d ##na ; DNA , Double Strand ##ed ; Double - Strand ##ed DNA ; d ##s d ##na ; de ##ox ##yr ##ib ##on ##uc ##le ##ic acid double strand ; d ##s DNA ; d ##na double stranded ; d ##s - DNA ; d ##s - d ##na ; double strand d ##na ; double - stranded DNA ; DNA double strand ; De ##ox ##yr ##ib ##on ##uc ##le ##ic acid , double stranded ) [SEP]'},
 {'mention_id': '27798906.17',
  'test_set_idx': 17537,
  'type': 'T038',
  'name': 'atrophy',
  'context': '[CLS] . 4 months ( range = 24 . 0 - 123 . 0 months ) . Un ##iva ##ria ##te analysis showed that par ##ot ##id g ##land volumes receiving more than 5 , 10 , 15 , and 20 G ##y relative biological effectiveness ( V ##5 , V ##10 , V ##15 and V ##20 , respectively ) , mean dose , and maximum dose were significantly associated with par ##ot ##id g ##land at ##rop ##hy . Multi ##var ##iate analysis indicated that only V ##5 was significantly associated with [unused1] at ##rop ##hy [unused2] . Inc ##rea ##sing V ##5 was a significant risk factor for par ##ot ##id g ##land at ##rop ##hy after carbon ion radio ##ther ##ap ##y . [SEP]',
  'existing_cui': 'C0026846',
  'existing_cui_name': 'Muscular Atrophy',
  'existing_cui_desc': '[CLS] Mu ##s ##cular At ##rop ##hy [unused3] ( B ##iol ##og ##ic Fun ##ction , Mu ##s ##cular , diagnosis , disorder , finding , physical finding : at ##rop ##hy ; muscle ; am ##yo ##tro ##phy ; muscle at ##rop ##hy was seen ; Was ##ting ; muscle ; At ##rop ##hi ##es , Mu ##s ##cle ; Mu ##s ##cle thin ##ning ; At ##rop ##hy - muscular ; am ##yo ##tro ##phi ##es ; muscular at ##rop ##hy ; Mu ##s ##cle wasting ; am ##yo ##tro ##phi ##a ; At ##rop ##hi ##es , Mu ##s ##cular ; muscular at ##rop ##hi ##es ; At ##rop ##hy muscle ; Mu ##s ##cle At ##rop ##hy ; Was ##ting ; muscles ; [SEP]',
  'our_bi_cui': 'C0333641',
  'our_bi_cui_name': 'Atrophic',
  'our_bi_cui_desc': '[CLS] At ##rop ##hic [unused3] ( B ##iol ##og ##ic Fun ##ction , m ##or ##ph ##olo ##gic abnormal ##ity : at ##rop ##hy ; At ##rop ##hy ; At ##rop ##hi ##es ; at ##rop ##hi ##es ; At ##rop ##hy [ Disease / Finding ] ; at ##rop ##hic ; at ##rop ##hy ##ing ; at ##rop ##hi ##ed ; AT ##RO ##P ##H ##Y ; At ##rop ##hy [ du ##p ] ) [SEP]',
  'our_cross_cui': 'C0341045',
  'our_cross_cui_name': 'Atrophy of parotid gland',
  'our_cross_cui_desc': '[CLS] At ##rop ##hy of par ##ot ##id g ##land [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : at ##rop ##hy ; par ##ot ##id g ##land ) [SEP]'},
 {'mention_id': '28256742.21',
  'test_set_idx': 30227,
  'type': 'T082',
  'name': 'Prince Edward Bay',
  'context': '[CLS] ##s most frequently detected were pen ##ta ##ch ##lor ##o , he ##xa ##ch ##lor ##o , and he ##pta ##ch ##lor ##o con ##gene ##rs , which are structural ##ly similar to thy ##roid hormones . Pen ##ta ##ch ##lor ##op ##hen ##ol was detected at highest concentrations ( 1 . 8 ng / g ) in fish from [unused1] Prince Edward Bay [unused2] , the Bay of Q ##uin ##te Lake reference site , and Hill ##man Marsh ( the W ##heat ##ley Harbour reference site ) , suggesting local sources of contamination . El ##eva ##ted Pen ##ta ##ch ##lor ##op ##hen ##ol concentrations were also detected in the plasma of brown bull ##head from exposed sites in the Toronto and Region A [SEP]',
  'existing_cui': 'C3828475',
  'existing_cui_name': 'Prince Edward County , VA',
  'existing_cui_desc': '[CLS] Prince Edward County , VA [unused3] ( Spa ##tial Concept : Prince Edward County ; Prince Edward County , Virginia ; VA ##14 ##7 ) [SEP]',
  'our_bi_cui': 'C0033157',
  'our_bi_cui_name': 'Prince Edward Island',
  'our_bi_cui_desc': '[CLS] Prince Edward Island [unused3] ( Spa ##tial Concept , geographic location : Prince Edward island ; ed ##ward island prince ; prince ed ##ward island ) [SEP]',
  'our_cross_cui': 'C3203003',
  'our_cross_cui_name': 'Bay',
  'our_cross_cui_desc': '[CLS] Bay [unused3] ( Spa ##tial Concept , geographic , geographic : Bay ##s ) [SEP]'},
 {'mention_id': '28249376.27',
  'test_set_idx': 29897,
  'type': 'T033',
  'name': 'digestibility',
  'context': '[CLS] in fist ##ulated sheep . Six sheep fitted with r ##umi ##nal fist ##ula ##s were used in a repeated measurement design with two 24 - d periods to investigate the effect of adding mixture of essential oils at 0 ( control ) , 0 . 8 , and 1 . 6 m ##L / d on apparent nut ##rient [unused1] dig ##est ##ibility [unused2] , r ##ume ##n f ##er ##mentation characteristics , r ##ume ##n micro ##bial population and blood chemical meta ##bol ##ites . Animals were fed with a 50 : 50 al ##fa ##lf ##a hay : concentrate diet . R ##umi ##nal pH , total volatile fatty acids concentration , m ##olar proportion of individual volatile fatty acids , ace ##tate [SEP]',
  'existing_cui': 'C0233423',
  'existing_cui_name': 'Suggestibility',
  'existing_cui_desc': '[CLS] Su ##gg ##est ##ibility [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : suggest ##ibility ; Su ##gg ##est ##ibility , function ) [SEP]',
  'our_bi_cui': 'C0012238',
  'our_bi_cui_name': 'Digestion',
  'our_bi_cui_desc': '[CLS] Digest ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : dig ##estive system functions ; dig ##est ##ion ; Digest ##ive system functions ; dig ##est ##ions ; Digest ##ive system function ; Fun ##ctions of the dig ##estive system ; Digest ##ive ; Digest ##ive tract function ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27977319.14',
  'test_set_idx': 22766,
  'type': 'T017',
  'name': 'THI1',
  'context': "[CLS] is expressed in all organs , and the encoded protein cat ##aly ##zes th ##iam ##ine bio ##sy ##nt ##hesis . Loss of function produces ch ##lor ##osis , a typical th ##iam ##ine - deficiency p ##hen ##otype , and mortality . To investigate th ##iam ##ine ' s role in s ##ym ##bio ##sis , we focused on [unused1] T ##H ##I ##1 [unused2] , a th ##iam ##ine - bio ##sy ##nt ##hesis gene expressed in roots , nod ##ules , and seeds . The th ##i ##1 mutant had green leaves , but formed small nod ##ules and im ##mat ##ure seeds . These p ##hen ##otype ##s were rescued by T ##H ##I ##1 complement ##ation and by ex ##ogen ##ous [SEP]",
  'existing_cui': 'C0272238',
  'existing_cui_name': 'Transient hypogammaglobulinemia of infancy',
  'existing_cui_desc': '[CLS] Trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia of infancy [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : Trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia of young ; trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia of infancy ; T ##H ##I - Trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia of infancy ; h ##y ##po ##gam ##ma ##g ##lo ##bul ##ine ##mia trans ##ient of infancy ; I ##mm ##uno ##g ##lo ##bul ##in mat ##uration ##al delay ; T ##H ##I - Trans ##ient h ##y ##po ##gam ##ma ##g ##lo ##bul ##inae ##mia of infancy ; Trans ##ient H ##y [SEP]',
  'our_bi_cui': 'C1420871',
  'our_bi_cui_name': 'TPI1 gene',
  'our_bi_cui_desc': '[CLS] T ##PI ##1 gene [unused3] ( Ana ##tom ##ical Structure : T ##RI ##OS ##EP ##H ##OS ##P ##HA ##TE ISO ##ME ##RA ##SE 1 ; T ##PI ##1 Gene ; Trio ##se ##ph ##os ##phate Is ##omer ##ase 1 Gene ; T ##PI ; trio ##se ##ph ##os ##phate is ##omer ##ase 1 ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '28223077.53',
  'test_set_idx': 28937,
  'type': 'T082',
  'name': 'background site',
  'context': '[CLS] that of up ##wind site , suggesting that studied land ##fill did not have obvious influence on PC ##D ##D / F ##s in ambient air from down ##wind site . ( 5 ) The 95 ##th percent ##ile car ##cin ##ogenic risk of PC ##D ##D / F ##s in ambient air from s u ##rro ##und ##ing sites , [unused1] background site [unused2] , up ##wind site and down ##wind site were 8 . 03 × 10 ( - 9 ) , 7 . 57 × 10 ( - 9 ) , 9 . 69 × 10 ( - 9 ) and 8 . 15 × 10 ( - 9 ) , respectively , which were much lower than the threshold value of car [SEP]',
  'existing_cui': 'C0237490',
  'existing_cui_name': 'Family Background',
  'existing_cui_desc': '[CLS] Family Background [unused3] ( Finding , Family : Background ) [SEP]',
  'our_bi_cui': 'C0442611',
  'our_bi_cui_name': 'Industrial site',
  'our_bi_cui_desc': '[CLS] Industrial site [unused3] ( Spa ##tial Concept , environment ) [SEP]',
  'our_cross_cui': 'C0205145',
  'our_cross_cui_name': 'Site',
  'our_cross_cui_desc': '[CLS] Site [unused3] ( Spa ##tial Concept , attribute , qualifier value : Sites ) [SEP]'},
 {'mention_id': '27395231.31',
  'test_set_idx': 4600,
  'type': 'T017',
  'name': 'shape of body',
  'context': '[CLS] n . ( Dr ##acon ##ema ##ti ##dae ) differs from other Pro ##cha ##eto ##so ##ma species except P . long ##ica ##pit ##at ##um ( All ##gé ##n , 1935 ) in that the p ##har ##yn ##ge ##al bulb l ##ume ##n is not cut ##icular ##ised , from P . long ##ica ##pit ##at ##um by [unused1] shape of body [unused2] and r ##ost ##rum , greater number of c ##ep ##hali ##c ad ##hesive tubes , and from P . ma ##ert ##ens ##i Dec ##rae ##mer , 1989 by having a relatively longer tail , fewer anterior ad ##hesive tubes and longer s ##pic ##ules , besides lacking cut ##icular thick ##ening in the p ##har ##yn ##ge ##al bulb [SEP]',
  'existing_cui': 'C0423805',
  'existing_cui_name': 'Shape of nail',
  'existing_cui_desc': '[CLS] S ##ha ##pe of nail [unused3] ( Finding , o ##bs ##er ##vable entity : nails shaped ; nails shape ; shape of nail ) [SEP]',
  'our_bi_cui': 'C0332479',
  'our_bi_cui_name': 'Shapes',
  'our_bi_cui_desc': '[CLS] S ##ha ##pes [unused3] ( Spa ##tial Concept , qualifier value : shapes ; shaping ; SH ##AP ##E ; shaped ; shape ; S ##ha ##pe ; S ##ha ##pe finding ) [SEP]',
  'our_cross_cui': 'C1268086',
  'our_cross_cui_name': 'Body structure',
  'our_cross_cui_desc': '[CLS] Body structure [unused3] ( Ana ##tom ##ical Structure , body structure : B ##OD ##Y ST ##R ##UC ##TU ##RE ##S ; Body structures ; B ##OD ##Y ; structure body ; body structures ; Body ; body structure ) [SEP]'},
 {'mention_id': '27400734.20',
  'test_set_idx': 4724,
  'type': 'T103',
  'name': 'GEM',
  'context': '[CLS] A random ##ized clinical trial has found that the addition of er ##lot ##ini ##b to g ##em ##ci ##ta ##bine ( GE ##M - E ) for pan ##cre ##atic cancer led to a modest increase in survival . The aim of this national population - based retrospective study was to compare the effectiveness of GE ##M - E to [unused1] GE ##M [unused2] alone for pan ##cre ##atic cancer patients in real clinical practice . Pat ##ients with pan ##cre ##atic cancer ( I ##CD - 10 : C ##25 ) with prescription claims of g ##em ##ci ##ta ##bine or er ##lot ##ini ##b between Jan 1 , 2007 and Dec 31 , 2012 were retrospective ##ly identified from the Korean Health Insurance [SEP]',
  'existing_cui': 'C1333683',
  'existing_cui_name': 'GMNN gene',
  'existing_cui_desc': '[CLS] GM ##N ##N gene [unused3] ( Ana ##tom ##ical Structure : Gemini ##n , DNA Rep ##lication In ##hibit ##or Gene ; GM ##N ##N Gene ; G ##em ; GE ##MI ##NI ##N ; g ##em ##ini ##n , DNA replication inhibitor ; GE ##M ) [SEP]',
  'our_bi_cui': 'C0017243',
  'our_bi_cui_name': 'Gel',
  'our_bi_cui_desc': '[CLS] G ##el [unused3] ( Chemical , Pharmacy , product , qualifier value , substance : G ##el Do ##se Form ; G ##els ; j ##elly ; j ##elli ##es ; Je ##lly ; drugs gel ; G ##el preparation ; G ##el Do ##sa ##ge Form ; GE ##L ; gel ##s ; gel preparation ; G ##el [ Do ##se Form ] ; G ##el ##D ##rug ##F ##orm ; gel ; Drug gel ) [SEP]',
  'our_cross_cui': 'C0045093',
  'our_cross_cui_name': 'gemcitabine',
  'our_cross_cui_desc': "[CLS] g ##em ##ci ##ta ##bine [unused3] ( Chemical , product , substance : g ##em ##ci ##ta ##bine [ Chemical / In ##g ##red ##ient ] ; 2 ' , 2 ' - di ##f ##lu ##oro ##de ##ox ##y ##cy ##ti ##dine ; 4 - amino - 1 - ( ( 2 ##R , 4 ##R , 5 ##R ) - 3 , 3 - Di ##f ##lu ##oro - 4 - h ##ydro ##xy - 5 - ( h ##ydro ##xy ##met ##hyl ) - te ##tra ##hy ##dr ##of ##ura ##n - 2 - y ##l ) p ##yr ##im ##id ##in - 2 ( 1 ##H ) - one ; 2 ' , 2 ' - Di ##f ##lu ##oro ##de ##ox [SEP]"},
 {'mention_id': '27796974.2',
  'test_set_idx': 17356,
  'type': 'T170',
  'name': 'Water Framework Directive',
  'context': '[CLS] Method ##ology development on aquatic environmental assessment The [unused1] Water Framework Direct ##ive [unused2] aims at reaching the good ecological status of the surface and ground water bodies ( L ##ás ##z ##l ##ó et al . Micro ##che ##m J 85 ( 1 ) : 65 - 71 , 2007 ) . The paper deals with quality evaluation of waters with special focus on the water chemistry parameters as defined in the Water Framework Direct ##ive and pertaining legal regulations . The purpose of this paper is to de ##vise a quantitative type of water quality assessment method which could provide rapid , accurate , and reliable information on the quality of the surface waters by using water chemistry parameters . Quality classes have been [SEP]',
  'existing_cui': 'C0450152',
  'existing_cui_name': 'Metal framework',
  'existing_cui_desc': '[CLS] Metal framework [unused3] ( Medical Devi ##ce , physical object ) [SEP]',
  'our_bi_cui': 'C1135941',
  'our_bi_cui_name': 'Ethical Directives',
  'our_bi_cui_desc': '[CLS] E ##thical Direct ##ives [unused3] ( Intel ##lect ##ual Product : Direct ##ive , E ##thical ; Direct ##ives , E ##thical ; E ##thical Direct ##ive ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27539945.4',
  'test_set_idx': 9829,
  'type': 'T103',
  'name': 'alkyldimethylamine oxide',
  'context': '[CLS] The ##rm ##o - re ##versible capture and release of DNA by z ##wi ##tter ##ion ##ic surf ##act ##ants The the ##rm ##o - re ##versible capture and release of DNA were studied by the pro ##ton ##ation and de ##p ##rot ##ona ##tion of [unused1] al ##ky ##ld ##ime ##thy ##lam ##ine oxide [unused2] ( C ##n ##DM ##A ##O , n = 10 , 12 and 14 ) in Tri ##s - HC ##l buffer solution . DNA / C ##14 ##DM ##A ##OH ( + ) in Tri ##s - HC ##l buffer solution with pH = 7 . 2 is transparent at 25 °C , indicating that DNA molecules exist mainly in individuals and the binding of C ##14 ##DM ##A [SEP]',
  'existing_cui': 'C1869399',
  'existing_cui_name': 'hexyldimethylamine oxide',
  'existing_cui_desc': '[CLS] he ##xy ##ld ##ime ##thy ##lam ##ine oxide [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0058638',
  'our_bi_cui_name': 'dodecyldimethylamine oxide',
  'our_bi_cui_desc': '[CLS] do ##de ##cy ##ld ##ime ##thy ##lam ##ine oxide [unused3] ( Chemical : DD ##A ##O ; N , N - dim ##eth ##yl ##do ##de ##cy ##c ##lam ##ine N - oxide ; do ##de ##cy ##ld ##ime ##thy ##lam ##ine N - oxide ; L ##DA ##O ; N - do ##de ##cy ##l - N , N - dim ##eth ##yla ##mine N - oxide ; la ##ury ##ld ##ime ##thy ##lam ##ine oxide ; N , N - dim ##eth ##yl ##do ##de ##cy ##lam ##ine - N - oxide ) [SEP]',
  'our_cross_cui': 'C0029224',
  'our_cross_cui_name': 'Organic Chemicals',
  'our_cross_cui_desc': '[CLS] Organic Chemical ##s [unused3] ( Chemical , substance : organic chemicals ; compound organic ; Organic Chemical ##s [ Chemical / In ##g ##red ##ient ] ; organic compound ; Chemical ##s , Organic ; organic chemical ; Organic Chemical ; chemical organic ; organic compounds ; Organic compound ; Organic compounds ; Organic chemical ) [SEP]'},
 {'mention_id': '28454015.21',
  'test_set_idx': 36522,
  'type': 'T170',
  'name': 'health - risk based air quality index',
  'context': '[CLS] , but oz ##one pollution can be significant during summer . Furthermore , as the commonly used air quality index neglect ##s the mutual health effects from multiple poll ##uta ##nts , we introduced the aggregate air quality index and health - risk based air quality index to evaluate the health risks . Results show that based on [unused1] health - risk based air quality index [unused2] , the current air quality index system likely significantly under ##est ##imate the health risks of air pollution , highlighting that the general public may need strict ##er health protection measures . The population - weighted two - year average health - risk based air quality index data further demonstrates that all population in the studied cities in He [SEP]',
  'existing_cui': 'C1268941',
  'existing_cui_name': 'Health risks education',
  'existing_cui_desc': '[CLS] Health risks education [unused3] ( Health Care Act ##ivity , procedure , regime / therapy : Counsel ##ing regarding health risks ; Ad ##vice about health risks ; Counsel ##ling regarding health risks ; Education regarding health risks ) [SEP]',
  'our_bi_cui': 'C3494320',
  'our_bi_cui_name': 'Health Impact Assessment',
  'our_bi_cui_desc': '[CLS] Health Impact Assessment [unused3] ( Health Care Act ##ivity : Impact Assessment ##s , Health ; Impact Assessment , Health ; Health Impact Assessment ##s ; Assessment ##s , Health Impact ; Assessment , Health Impact ) [SEP]',
  'our_cross_cui': 'C0918012',
  'our_cross_cui_name': 'Index',
  'our_cross_cui_desc': '[CLS] Index [unused3] ( Intel ##lect ##ual Product : index ##ed ; index ; index ##es ; Index ##es as Top ##ic ) [SEP]'},
 {'mention_id': '28385178.27',
  'test_set_idx': 33778,
  'type': 'T092',
  'name': 'Kenyatta National',
  'context': '[CLS] to describe injury e ##pid ##em ##iology and care . The objective of this study was to investigate the e ##pid ##em ##iology and outcomes of injuries at 4 refer ##ral hospitals in Kenya using hospital - based trauma re ##gis ##tries . From January 2014 to May 2015 , all injured patients presenting to the casual ##ty departments of [unused1] Kenya ##tta National [unused2] , T ##hi ##ka Level 5 , Mac ##hak ##os Level 5 , and Me ##ru Level 5 Hospital ##s were enrolled prospective ##ly . Data collected included demographic characteristics , type of pre ##hos ##pit ##al care received , pre ##hos ##pit ##al time , injury pattern , and outcomes . A total of 14 , 237 patients were enrolled [SEP]',
  'existing_cui': 'C0015737',
  'existing_cui_name': 'Federal Government',
  'existing_cui_desc': '[CLS] Federal Government [unused3] ( Organization : federal government ; Government , Federal ; National Government ; Federal government ; National ) [SEP]',
  'our_bi_cui': 'C0017446',
  'our_bi_cui_name': 'Geographic Locations',
  'our_bi_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]',
  'our_cross_cui': 'C0019994',
  'our_cross_cui_name': 'Hospitals',
  'our_cross_cui_desc': '[CLS] Hospital ##s [unused3] ( Organization , environment : H ##OS ##P ; hospital ; Hospital ; hospital environment ; In Hospital ; H ##OS ##PI ##TA ##L ; hospitals ) [SEP]'},
 {'mention_id': '28390176.32',
  'test_set_idx': 34122,
  'type': 'T033',
  'name': 'inhibitory effect',
  'context': '[CLS] ##ora ##ted of s ##ys ##to ##lic and di ##ast ##olic abnormal ##ities , normal ##ized in gene expression of h ##yper ##tro ##phic and fi ##bro ##tic markers , reserved micro ##vas ##cular density in pressure over ##load induced h ##yper ##tro ##phic mice . C ##uc ##ur ##bit ##ac ##in B also showed significant h ##yper ##tro ##phy [unused1] inhibitor ##y effect [unused2] in p ##hen ##yle ##ph ##rine stimulate ##d card ##io ##my ##ocytes . The C ##uc ##ur ##bit ##ac ##in B - mediated mit ##igate ##d cardiac h ##yper ##tro ##phy was at ##tri ##but ##able to the increasing level of auto ##pha ##gy , which was associated with the blockade of A ##kt / m ##TO ##R / Fox ##O [SEP]',
  'existing_cui': 'C1708997',
  'existing_cui_name': 'Metastatic Inhibitory Effect',
  'existing_cui_desc': '[CLS] Met ##ast ##atic In ##hibit ##ory Effect [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_bi_cui': 'C0021469',
  'our_bi_cui_name': 'Metabolic Inhibition',
  'our_bi_cui_desc': '[CLS] Met ##ab ##olic In ##hibition [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : In ##hibition ; In ##hibition , function ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27526663.6',
  'test_set_idx': 9446,
  'type': 'T033',
  'name': 'necrotrophic',
  'context': '[CLS] Different ##ial control and function of Arab ##ido ##psis Pro ##D ##H1 and Pro ##D ##H ##2 genes on infection with bio ##tro ##phic and [unused1] ne ##c ##rot ##rop ##hic [unused2] path ##ogen ##s Arab ##ido ##psis contains two pro ##line de ##hy ##dr ##ogen ##ase genes , Pro ##D ##H1 and Pro ##D ##H ##2 , encoding for ho ##mo ##log ##ous and functional is ##oe ##nz ##yme ##s . Although Pro ##D ##H1 has been studied extensively , especially under a ##biotic stress , Pro ##D ##H ##2 has only started to be anal ##ys ##ed in recent years . These genes display distinctive expression patterns and show weak transcription ##al co - regulation , but are both activated in path ##ogen - [SEP]',
  'existing_cui': 'C4084801',
  'existing_cui_name': 'Prostate Health Index',
  'existing_cui_desc': '[CLS] Pro ##state Health Index [unused3] ( Health Care Act ##ivity : p ##hi - index ) [SEP]',
  'our_bi_cui': 'C0027540',
  'our_bi_cui_name': 'Necrosis',
  'our_bi_cui_desc': '[CLS] N ##ec ##rosis [unused3] ( B ##iol ##og ##ic Fun ##ction , m ##or ##ph ##olo ##gic abnormal ##ity , qualifier value : T ##iss ##ue de ##vi ##tal ##isation ; T ##iss ##ue de ##vi ##tal ##ization ; ne ##c ##rot ##ic cell death ; N ##ec ##rose ##s ; tissue death ; NE ##CR ##OS ##IS ; ne ##c ##rot ##ic ; obsolete tissue death ; Cell ##ular ne ##c ##rosis ; N ##ec ##rot ##ic Process ; cellular ne ##c ##rosis ; ne ##c ##rosis ; N ##ec ##rosis [ Disease / Finding ] ; obsolete ne ##c ##rosis ; N ##ec ##rot ##ic ; death tissue ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27689491.25',
  'test_set_idx': 14253,
  'type': 'T033',
  'name': 'no',
  'context': '[CLS] ratio = 0 . 89 ; 95 % confidence interval = 0 . 45 , 1 . 77 ) , or for sex with no effective means of birth control ( Louisiana , adjusted odds ratio = 1 . 18 ; 95 % confidence interval = 0 . 78 , 1 . 78 ; Rochester , adjusted odds ratio = 0 . 41 ; 95 % confidence interval = 0 . 13 , 1 . 27 ) after controlling for relevant co ##var ##iate ##s . We found [unused1] no [unused2] short - term effects for the offer of the intervention . Research might be needed for the long - term and intermediate impacts of youth development programs on these and other adolescent risk behaviors . [SEP]',
  'existing_cui': 'C1140150',
  'existing_cui_name': 'NOC - CodeSystem',
  'existing_cui_desc': '[CLS] NO ##C - Code ##S ##ys ##tem [unused3] ( Intel ##lect ##ual Product , NO ##C : nursing outcomes classification ) [SEP]',
  'our_bi_cui': 'C0205160',
  'our_bi_cui_name': 'Negative',
  'our_bi_cui_desc': '[CLS] N ##eg ##ative [unused3] ( Finding , qualifier value : Rule ##d out ; N ##eg ##ative ( m ##od ##ifier ) [ Am ##bi ##guous ] ; N ##eg ##ative for ; N ##eg ##ative ( m ##od ##ifier ) ; ruled out ; for negative ; out ruled ; negative ; negative ##s ) [SEP]',
  'our_cross_cui': 'C1513916',
  'our_cross_cui_name': 'Negative Finding',
  'our_cross_cui_desc': '[CLS] N ##eg ##ative Finding [unused3] ( Finding : NE ##GA ##TI ##VE ; N ##eg ##ative ) [SEP]'},
 {'mention_id': '27260627.19',
  'test_set_idx': 989,
  'type': 'T170',
  'name': 'breast module BR-23',
  'context': "[CLS] its association with women ' s personal characteristics and cancer treatments . In this cross - section ##al study , sexual function was assessed using the Female Sexual Fun ##ction Index . The health - related quality of life was measured using the European Organization for Research and Treatment of Cancer Q ##L ##Q - C ##30 and its [unused1] breast module BR - 23 [unused2] . Of the 235 participants approached , 216 participants were included in the study . Of these , 63 patients reported no sexual activity in the last month and thus were analyzed only in relation to the sexual desire domain of Female Sexual Fun ##ction Index . A total of 154 ( 71 . 3 % ) patients were classified [SEP]",
  'existing_cui': 'C3542953',
  'existing_cui_name': 'Module',
  'existing_cui_desc': '[CLS] Mo ##du ##le [unused3] ( Intel ##lect ##ual Product , core meta ##data concept , core meta ##data concept ) [SEP]',
  'our_bi_cui': 'C1511299',
  'our_bi_cui_name': 'Breast Cancer Surveillance Consortium',
  'our_bi_cui_desc': '[CLS] B ##rea ##st Cancer Sur ##ve ##illa ##nce Consortium [unused3] ( Professional or O ##cc ##upation ##al Group ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27886280.71',
  'test_set_idx': 20783,
  'type': 'T007',
  'name': 'exponential and stationary planktonic cells',
  'context': '[CLS] C ##ip ##ro ##f ##lo ##xa ##cin ( F ##lu ##oro ##quin ##olo ##ne ) at a low concentration resulted in killing of both cultures of P . a ##er ##ug ##ino ##sa , producing per ##sist ##ers that were in ##vu ##ln ##era ##ble to killing . Station ##ary cells appear to be somewhat more tolerant than ex ##po ##nent ##ial cells in all of these ass ##ays . We also showed that nut ##rient de ##p ##ri ##vation ( se ##rine starvation ) regulated by string ##ent and general stress response , contribute to the increased tolerance of P . a ##er ##ug ##ino ##sa [unused1] ex ##po ##nent ##ial and stationary plan ##kt ##onic cells [unused2] via production of per ##sist ##ers . [SEP]',
  'existing_cui': 'C3839460',
  'existing_cui_name': 'Nonprogressive',
  'existing_cui_desc': '[CLS] Non ##p ##rog ##ress ##ive [unused3] ( Finding , qualifier value : Non ##p ##rog ##ress ##ive course ; Station ##ary ; Non ##p ##rog ##ress ##ive disorder ; Non - progressive ) [SEP]',
  'our_bi_cui': 'C0007634',
  'our_bi_cui_name': 'Cells',
  'our_bi_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]',
  'our_cross_cui': 'C0563199',
  'our_cross_cui_name': 'Bacterial cell',
  'our_cross_cui_desc': '[CLS] Ba ##cter ##ial cell [unused3] ( Ba ##cter ##ium , organism : bacterial cell ; cell bacterial ) [SEP]'},
 {'mention_id': '28077286.55',
  'test_set_idx': 25105,
  'type': 'T103',
  'name': 'MeHg',
  'context': '[CLS] Me ##H ##g dose of 3 . 2 μ ##g H ##g g ( - 1 ) egg , and a vehicle control ( water ) . No adverse effects of in o ##vo Me ##H ##g treatment were detected on courts ##hip song quality or on mating behavior in experimental males at sexually maturity which would suggest that o ##bs ##er ##vable ne ##uro ##be ##ha ##vior ##al effects of Me ##H ##g exposure may depend on the timing of exposure during offspring development . However , ne ##uro ##ana ##tom ##ical analysis indicated an increase in te ##len ##ce ##pha ##lon volume with increased [unused1] Me ##H ##g [unused2] concentrations which may suggest a prolonged inflammatory response in this region of the brain . [SEP]',
  'existing_cui': 'C0228520',
  'existing_cui_name': 'Structure of reticular formation of medulla oblongata',
  'existing_cui_desc': '[CLS] Structure of re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata [unused3] ( Ana ##tom ##ical Structure , body structure : bulb ##ar re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla ; B ##ul ##bar re ##tic ##ular formation ; Form ##ati ##o re ##tic ##ular ##is my ##ele ##nce ##pha ##li ; me ##du ##llar ##y re ##tic ##ular formation ; re ##tic ##ular formation of me ##du ##lla ; Me ##du ##llar ##y re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata ) [SEP]',
  'our_bi_cui': 'C0025424',
  'our_bi_cui_name': 'Mercury',
  'our_bi_cui_desc': '[CLS] Mercury [unused3] ( Chemical , H ##g , Metal , elemental , substance : mercury ; Mercury [ Chemical / In ##g ##red ##ient ] ; metal mercury ; H ##yd ##rar ##gy ##rum ; H ##g element ; Li ##quid Silver ; Quick ##si ##lve ##r ; H ##g ; H ##g - Mercury ; h ##g ; ME ##RC ##UR ##Y ) [SEP]',
  'our_cross_cui': 'C0025794',
  'our_cross_cui_name': 'Methylmercury Compounds',
  'our_cross_cui_desc': '[CLS] Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [unused3] ( Chemical : Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [ Chemical / In ##g ##red ##ient ] ; Mercury Co ##mpo ##unds , Met ##hyl ; ME ##TH ##Y ##L ME ##RC ##UR ##Y CP ##DS ; Met ##hyl Mercury Co ##mpo ##unds ; ME ##TH ##Y ##LM ##ER ##CU ##R ##Y CP ##DS ; Co ##mpo ##unds , Met ##hyl ##mer ##cu ##ry ; met ##hyl ##mer ##cu ##ry ; Co ##mpo ##unds , Met ##hyl Mercury ; Met ##hyl ##mer ##cu ##ry ) [SEP]'},
 {'mention_id': '27603112.45',
  'test_set_idx': 11387,
  'type': 'T103',
  'name': 'YKoF',
  'context': '[CLS] compounds were found to bind the EC ##F - type ABC transport ##er , but none binds stab ##ly to p ##rion protein . For the rib ##os ##witch , most compounds remained in their binding pockets during 50 n ##s of molecular dynamics simulation , indicating that RNA provides a pro ##mis ##cu ##ous binding site . In both [unused1] Y ##K ##o ##F [unused2] and th ##iam ##in p ##yr ##op ##hos ##ph ##oki ##nas ##e , most compounds remain tightly bound . However , th ##iam ##in p ##yr ##op ##hos ##ph ##oki ##nas ##e bio ##mo ##le ##cule ##s undergo poll ##uta ##nt - induced conform ##ational changes . Although most compounds are found to bind to some of these targets , [SEP]',
  'existing_cui': 'C1424135',
  'existing_cui_name': 'YAP1 gene',
  'existing_cui_desc': '[CLS] Y ##AP ##1 gene [unused3] ( Ana ##tom ##ical Structure : Yes - Associated Pro ##tein 1 Gene ; Yes associated protein 1 ; Y ##AP ##65 ; Y ##OR ##K ##IE , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF ; Y ##AP ##1 Gene ; Y ##ES - AS ##SO ##CI ##AT ##ED PR ##OT ##EI ##N 1 , 65 - K ##D ; Y ##AP ##2 ) [SEP]',
  'our_bi_cui': 'C0060764',
  'our_bi_cui_name': 'FRUCTOKINASE',
  'our_bi_cui_desc': '[CLS] F ##R ##UC ##TO ##K ##IN ##AS ##E [unused3] ( Chemical , substance : f ##ru ##ct ##oki ##nas ##e ; D - f ##ru ##ct ##oki ##nas ##e ; ATP - D - f ##ru ##ct ##ose 6 - p ##hos ##ph ##ot ##ran ##s ##fer ##ase ; s ##c ##r ##K protein ; Fr ##uc ##tok ##inas ##e ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27572277.25',
  'test_set_idx': 10541,
  'type': 'T058',
  'name': 'procedures',
  'context': '[CLS] ; s ##ham - operated fed ad l ##ibi ##tum ; and sleeve gas ##tre ##ct ##omy . The metabolic effects and gut micro ##bio ##ta profile were analyzed 10 weeks post ##oper ##ative ##ly . Association ##s between disc ##rim ##inating genera and metabolic markers after R ##oux - en - Y gas ##tric bypass were explored . The 2 [unused1] procedures [unused2] induced similar glucose improvement and increased flora diversity after 10 weeks compared with s ##ham - operated groups . R ##oux - en - Y gas ##tric bypass induced a marked higher relative abundance of Pro ##te ##ob ##act ##eria / Gamma ##p ##rote ##ob ##act ##eria and Beta ##p ##rote ##ob ##act ##eria and increased emergence of Fu ##so ##ba ##cter [SEP]',
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '28457334.29',
  'test_set_idx': 36639,
  'type': 'T058',
  'name': 'thromboaggregometry',
  'context': '[CLS] at se ##psis onset and after surgery respectively , as well as after 24 , 72 and 168 h . Level ##s of Cell - free DNA and extra ##cellular RNA were measured by quantitative probe - based polymer ##ase chain reaction . In addition , th ##rom ##boe ##last ##ography for co ##agu ##lation as well as [unused1] th ##rom ##boa ##gg ##re ##go ##metry [unused2] for plate ##let function was conducted . Both Cell - free DNA and extra ##cellular RNA were elevated in patients with se ##psis compared with post ##oper ##ative patients and healthy volunteers . While higher extra ##cellular RNA levels correlated with a faster c ##lot ##ting time and more stable c ##lot ##s , Cell - free DNA correlated [SEP]',
  'existing_cui': 'C0920267',
  'existing_cui_name': 'Platelet aggregation measurement',
  'existing_cui_desc': '[CLS] Plate ##let a ##gg ##regation measurement [unused3] ( Health Care Act ##ivity , lab test , procedure , qualifier value : P ##LA ##TA ##G ##GR ; A ##gg ##re ##go ##meter test ; Plate ##let A ##gg ##regation Me ##as ##ure ##ment ; Plate ##let a ##gg ##regation test ; Plate ##let a ##gg ##r ; Plate ##let A ##gg ##regation ; plate ##let a ##gg ##regation tests ; Plate ##let a ##gg ##regation ; Plate ##let a ##gg ##regation ass ##ay ; plate ##let a ##gg ##regation ; plate ##let a ##gg ##regation test ; Plate ##let Fun ##ction ; Plate ##let A ##gg ##regation Test ) [SEP]',
  'our_bi_cui': 'C0920267',
  'our_bi_cui_name': 'Platelet aggregation measurement',
  'our_bi_cui_desc': '[CLS] Plate ##let a ##gg ##regation measurement [unused3] ( Health Care Act ##ivity , lab test , procedure , qualifier value : P ##LA ##TA ##G ##GR ; A ##gg ##re ##go ##meter test ; Plate ##let A ##gg ##regation Me ##as ##ure ##ment ; Plate ##let a ##gg ##regation test ; Plate ##let a ##gg ##r ; Plate ##let A ##gg ##regation ; plate ##let a ##gg ##regation tests ; Plate ##let a ##gg ##regation ; Plate ##let a ##gg ##regation ass ##ay ; plate ##let a ##gg ##regation ; plate ##let a ##gg ##regation test ; Plate ##let Fun ##ction ; Plate ##let A ##gg ##regation Test ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '27838742.55',
  'test_set_idx': 19022,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] ##R , and p ##hos ##ph ##o - p ##70 ##S ##6 ##K in a time - and dose - dependent manner , suggesting that it induced auto ##pha ##gy by in ##hibit ##ing the A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K pathway in A ##5 ##4 ##9 and PC - 9 cells . [unused1] Del ##ica ##f ##lav ##one [unused2] is a potential anti ##can ##cer agent that can induce auto ##pha ##gic cell death in human non - small cell lung cancer via the A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28256742.8',
  'test_set_idx': 30214,
  'type': 'T082',
  'name': 'Canadian Areas',
  'context': '[CLS] Hal ##ogen ##ated p ##hen ##olic compounds in wild fish from Canadian Areas of Con ##cer ##n Con ##cent ##rations of ha ##log ##ena ##ted p ##hen ##olic compounds were measured in the plasma of brown bull ##head ( Am ##ei ##urus ne ##bul ##os ##us ) from 4 [unused1] Canadian Areas [unused2] of Con ##cer ##n ( A ##OC ##s ) , to assess exposure to suspected thy ##roid - disrupt ##ing chemicals . H ##ydro ##xy ##lated p ##oly ##ch ##lor ##inated bi ##phe ##ny ##ls ( OH - PC ##B ##s ) were detected in every sample collected in 3 of the A ##OC ##s ; the detection frequency was lower in samples from the Detroit River A ##OC . The OH - [SEP]',
  'existing_cui': 'C3831211',
  'existing_cui_name': 'Canadian County , OK',
  'existing_cui_desc': '[CLS] Canadian County , OK [unused3] ( Spa ##tial Concept : Canadian County , Oklahoma ; OK ##01 ##7 ; Canadian County ) [SEP]',
  'our_bi_cui': 'C0454980',
  'our_bi_cui_name': 'Region of Canada',
  'our_bi_cui_desc': '[CLS] Region of Canada [unused3] ( Spa ##tial Concept , geographic location : Province or Territory of Canada ; Provinces and territories of Canada ; Region ##s of Canada ) [SEP]',
  'our_cross_cui': 'C0006823',
  'our_cross_cui_name': 'Canada',
  'our_cross_cui_desc': '[CLS] Canada [unused3] ( Spa ##tial Concept , geographic location : CA ##N ; CA ##NA ##DA ; CA ) [SEP]'},
 {'mention_id': '27472389.38',
  'test_set_idx': 7179,
  'type': 'T103',
  'name': 'signal transducer and activator of transcription 1 inhibitor',
  'context': '[CLS] . In contrast , ACT ##A ##2 expression was decreased by H ##ER ##2 si ##RNA . Next , we investigated the co - relation between signal trans ##du ##cer and act ##iva ##tor of transcription 1 and ACT ##A ##2 expression . Ba ##sal ACT ##A ##2 expression was significantly decreased by treatment with the [unused1] signal trans ##du ##cer and act ##iva ##tor of transcription 1 inhibitor [unused2] flu ##dar ##abi ##ne or the J ##A ##K ##2 inhibitor AG ##4 ##90 . In contrast , ACT ##A ##2 expression was increased by signal trans ##du ##cer and act ##iva ##tor of transcription 1 over ##ex ##press ##ion . Level ##s of ACT ##A ##2 , signal trans ##du ##cer and act ##iva ##tor [SEP]',
  'existing_cui': 'C1335872',
  'existing_cui_name': 'STAT1 gene',
  'existing_cui_desc': '[CLS] ST ##AT ##1 gene [unused3] ( Ana ##tom ##ical Structure : Signal Trans ##du ##cer and Act ##iva ##tor of Trans ##cription 1 , 91 ##k ##D ##a Gene ; ST ##AT ##1 Gene ; transcription factor IS ##G ##F - 3 components p ##9 ##1 / p ##8 ##4 ; signal trans ##du ##cer and act ##iva ##tor of transcription 1 ; ST ##AT ##9 ##1 ; IS ##G ##F - 3 ; S ##IG ##NA ##L T ##RA ##NS ##D ##UC ##ER AND ACT ##I ##VA ##TO ##R OF T ##RA ##NS ##CR ##IP ##TI ##ON 1 ) [SEP]',
  'our_bi_cui': 'C1519313',
  'our_bi_cui_name': 'Signal Transduction Inhibitor',
  'our_bi_cui_desc': '[CLS] Signal Trans ##duction In ##hibit ##or [unused3] ( Chemical : signal trans ##duction inhibitor ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27902303.1',
  'test_set_idx': 21293,
  'type': 'T007',
  'name': 'Arthrobacter ginkgonis sp . nov.',
  'context': '[CLS] [unused1] Art ##hr ##ob ##act ##er g ##ink ##gon ##is s ##p . no ##v . [unused2] , an act ##ino ##my ##ce ##te isolated from r ##hi ##zo ##sp ##here of G ##ink ##go bi ##lo ##ba L A Gram - stain - positive , a ##ero ##bic act ##ino ##ba ##cter ##ial strain ( designated S ##YP - A ##7 ##29 ##9 ##T ) , which displayed a rod - co ##cc ##us growth life ##cycle , was isolated from the r ##hi ##zo ##sp ##here of G ##ink ##go bi ##lo ##ba L . Ph ##yl ##ogen ##etic analyses based on 16 ##S r ##RNA gene sequences indicated that strain S ##YP - A ##7 ##29 ##9 ##T belongs to the genus Art [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C1197674',
  'our_bi_cui_name': 'Arthrobacter gandavensis',
  'our_bi_cui_desc': '[CLS] Art ##hr ##ob ##act ##er g ##anda ##ven ##sis [unused3] ( Ba ##cter ##ium , organism : Art ##hr ##ob ##act ##er g ##and ##ensis ; Art ##hr ##ob ##act ##er g ##anda ##ven ##sis Storm ##s et al . 2003 ) [SEP]',
  'our_cross_cui': 'C0003876',
  'our_cross_cui_name': 'Arthrobacter',
  'our_cross_cui_desc': '[CLS] Art ##hr ##ob ##act ##er [unused3] ( Ba ##cter ##ium , organism : Art ##hr ##ob ##act ##er Con ##n and Di ##mm ##ick 1947 ( A ##pp ##roved List ##s 1980 ) em ##end . Bus ##se 2016 ; AR ##TH ##RO ##BA ##CT ##ER ; Art ##hr ##ob ##act ##er Con ##n and Di ##mm ##ick 1947 ( A ##pp ##roved List ##s 1980 ) em ##end . Koch et al . 1995 ; Genus Art ##hr ##ob ##act ##er ; Art ##hr ##ob ##act ##er species ) [SEP]'},
 {'mention_id': '27715500.7',
  'test_set_idx': 14872,
  'type': 'T092',
  'name': 'meatpacking facilities',
  'context': '[CLS] Self - reported occupational injuries among industrial beef slaughter ##house workers in the Midwest ##ern United States Although workers in [unused1] meat ##pack ##ing facilities [unused2] in the United States experience high rates of occupational injury , their injury experiences have received limited research attention . Prior research indicates under ##re ##port ##ing in injury rates in this industry as well significant variation in injury rates among facilities . To add detail to the rates and circumstances surrounding occupational injury among meat ##pack ##ing workers , we conducted a cross - section ##al study of workers employed at an industrial beef ##pack ##ing plant in Nebraska , United States ( n = 137 ) and interviewed workers about recent injury experiences . We assessed frequency , [SEP]',
  'existing_cui': 'C2825564',
  'existing_cui_name': 'Submitting Facility',
  'existing_cui_desc': '[CLS] Sub ##mit ##ting Facility [unused3] ( Organization ) [SEP]',
  'our_bi_cui': 'C0018704',
  'our_bi_cui_name': 'Health care facility',
  'our_bi_cui_desc': '[CLS] Health care facility [unused3] ( Organization , environment : health facilities ; health facility ; facilities medical ; medical facility ; Healthcare Facility ; Health F ##ac ##ilities ; Healthcare ##F ##ac ##ility ; Healthcare facility ; treatment facility ; facility medical ; Health care facilities ; treatment facilities ; care facilities health ; facilities treatment ; Health Facility ; health care facility ; care facility health ; Institution ; H ##EA ##LT ##H FA ##CI ##L ##IT ##Y ; Treatment F ##ac ##ilities ; Health Care Facility ; facilities health ; Health facilities ; F ##ac ##ilities , Health ; Facility , Health ) [SEP]',
  'our_cross_cui': 'C0025019',
  'our_cross_cui_name': 'Meat-Packing Industry',
  'our_cross_cui_desc': '[CLS] Meat - Pack ##ing Industry [unused3] ( Organization : Industries , Meat - Pack ##ing ; Industry , Meat - Pack ##ing ; ME ##AT PA ##C ##K ##ING IN ##D ##US ##T ; Meat Pack ##ing Industry ; Meat - Pack ##ing Industries ; IN ##D ##US ##T ME ##AT PA ##C ##K ##ING ; Industry , Meat Pack ##ing ) [SEP]'},
 {'mention_id': '27583176.12',
  'test_set_idx': 10709,
  'type': 'T170',
  'name': 'outcome grading scoring systems',
  'context': '[CLS] scoring systems [ ( In ##tra ##cer ##eb ##ral hem ##or ##r ##hage score , F ##UN ##C score and in ##tra ##cer ##eb ##ral hem ##or ##r ##hage g ##rading scale ] on our population to evaluate the correlation of these scores with the 30 - day mortality in our study . We also applied three widely accepted [unused1] outcome g ##rading scoring systems [unused2] [ ( In ##tra ##cer ##eb ##ral hem ##or ##r ##hage score , F ##UN ##C score and in ##tra ##cer ##eb ##ral hem ##or ##r ##hage g ##rading scale ] on our population to evaluate the correlation of these scores with the 30 - day mortality in our study . From 34 ##2 patients ( mean age : 67 [SEP]',
  'existing_cui': 'C1273712',
  'existing_cui_name': 'Grading system used',
  'existing_cui_desc': '[CLS] G ##rading system used [unused3] ( Health Care Act ##ivity , o ##bs ##er ##vable entity : g ##rading system ) [SEP]',
  'our_bi_cui': 'C1708250',
  'our_bi_cui_name': 'Tumor Grading System',
  'our_bi_cui_desc': '[CLS] Tu ##mor G ##rading System [unused3] ( Intel ##lect ##ual Product : G ##rading system ; G ##rading System ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27172917.21',
  'test_set_idx': 164,
  'type': 'T170',
  'name': 'biochemical predictors',
  'context': '[CLS] predict ##ors of ma ##li ##gna ##ncy are important for the decision of appropriate management in nod ##ules with at ##y ##pia of und ##eter ##mined significance / f ##oll ##icular les ##ion of und ##eter ##mined significance ( AU ##S / FL ##US ) . Our aim was to determine the ultra ##son ##ographic ##al , clinical , and [unused1] bio ##chemical predict ##ors [unused2] of ma ##li ##gna ##ncy in these patients . A total of 42 ##7 patients with c ##yt ##ological ##ly Beth ##es ##da Category III ( AU ##S / FL ##US ) thy ##roid nod ##ules were included in this retrospective study . We divided the nod ##ules into two subgroup ##s according to the his ##top ##ath ##ology as [SEP]',
  'existing_cui': 'C1511130',
  'existing_cui_name': 'Biochemical Processes',
  'existing_cui_desc': '[CLS] B ##io ##chemical Process ##es [unused3] ( B ##iol ##og ##ic Fun ##ction : B ##io ##chemical Process ; Process ##es , B ##io ##chemical ; Process , B ##io ##chemical ) [SEP]',
  'our_bi_cui': 'C0041366',
  'our_bi_cui_name': 'Biomarkers , Tumor',
  'our_bi_cui_desc': '[CLS] B ##io ##mark ##ers , Tu ##mor [unused3] ( Chemical , substance : MA ##R ##KE ##R B ##IO ##CH ##EM T ##UM ##OR ; Mark ##ers , Neo ##p ##las ##m Met ##ab ##oli ##te ; Biological Tu ##mor Mark ##ers ; Tu ##mor Mark ##er , B ##iol ##og ##ic ; MA ##R ##KE ##RS B ##IO ##CH ##EM T ##UM ##OR ; Tu ##mor Mark ##ers , B ##io ##chemical ; Mark ##er , Neo ##p ##las ##m Met ##ab ##oli ##te ; Met ##ab ##oli ##te Mark ##ers , Tu ##mor ; B ##io ##chemical Tu ##mor Mark ##er ; Tu ##mor Mark ##ers , Biological ; Mark ##er , B ##io ##chemical Tu ##mor ; Mark ##ers , Car ##cin ##ogen [SEP]',
  'our_cross_cui': 'C0683956',
  'our_cross_cui_name': 'predictive factor',
  'our_cross_cui_desc': '[CLS] predict ##ive factor [unused3] ( Intel ##lect ##ual Product : predict ##ive variables ; factors predict ##ive ; factor predict ##ive ; Disease Pre ##dict ##ive Factor ) [SEP]'},
 {'mention_id': '27510458.30',
  'test_set_idx': 8917,
  'type': 'T033',
  'name': 'double lumen sign',
  'context': '[CLS] slice thickness 3 mm with no inter ##s ##lice gap ) , as well as scanning with conventional m ##oda ##lities , including CT an ##gio ##graphy , magnetic resonance an ##gio ##graphy , and digital sub ##traction an ##gio ##graphy . We assessed whether these imaging methods could visual ##ize an in ##ti ##mal flap and / or [unused1] double l ##ume ##n sign [unused2] in the participants and compared the results between high - resolution fast spin echo and the other m ##oda ##lities . high - resolution fast spin echo images clearly showed in ##ti ##mal flap ##s and double l ##ume ##n signs in all 3 patients , whereas the conventional m ##oda ##lities identified a double l ##ume ##n sign in [SEP]',
  'existing_cui': 'C0441299',
  'existing_cui_name': 'Double lumen tube',
  'existing_cui_desc': '[CLS] Double l ##ume ##n tube [unused3] ( Medical Devi ##ce , physical object ) [SEP]',
  'our_bi_cui': 'C0233192',
  'our_bi_cui_name': "Ladin 's sign",
  'our_bi_cui_desc': "[CLS] La ##din ' s sign [unused3] ( Finding , finding : La ##din sign ) [SEP]",
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28127340.50',
  'test_set_idx': 26276,
  'type': 'T033',
  'name': 'non - zoonotic',
  'context': "[CLS] = 1 . 09 % , and La ##ela ##ps nut ##tal ##li = 3 . 29 % . Among 10 genera / species of identified parasite ##s , at least 8 of them were zoo ##not ##ic with public health importance . L . nut ##tal ##li and N . f ##as ##cia ##tus were the only two [unused1] non - zoo ##not ##ic [unused2] detected parasite ##s in this survey . Harbor ##ing a wide variety of zoo ##not ##ic parasite ##s in sampled wild rode ##nts particularly when they live nearby villages , represents a potential risk to native inhabitants . Hence , controlling rode ##nts ' population in residential regions and improving awareness of local people about the risk of disease transmission [SEP]",
  'existing_cui': 'C0043528',
  'existing_cui_name': 'Zoonoses',
  'existing_cui_desc': '[CLS] Zoo ##nose ##s [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder , navigation ##al concept : Zoo ##nosis ; Zoo ##not ##ic In ##fect ##ious Disease ; Zoo ##not ##ic infection ; Disease , Zoo ##not ##ic In ##fect ##ious ; Zoo ##nose ##s [ Disease / Finding ] ; Zoo ##not ##ic Disease ##s ; infections zoo ##not ##ic ; Zoo ##not ##ic In ##fect ##ion ; Zoo ##not ##ic Disease ; In ##fect ##ious Disease , Zoo ##not ##ic ; In ##fect ##ions , Zoo ##not ##ic ; Zoo ##not ##ic In ##fect ##ious Disease ##s ; Disease , Zoo ##not ##ic ; Disease ##s , Zoo ##not ##ic In ##fect ##ious ; zoo ##nose ; zoo ##not ##ic infection ; In [SEP]',
  'our_bi_cui': 'C0043528',
  'our_bi_cui_name': 'Zoonoses',
  'our_bi_cui_desc': '[CLS] Zoo ##nose ##s [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder , navigation ##al concept : Zoo ##nosis ; Zoo ##not ##ic In ##fect ##ious Disease ; Zoo ##not ##ic infection ; Disease , Zoo ##not ##ic In ##fect ##ious ; Zoo ##nose ##s [ Disease / Finding ] ; Zoo ##not ##ic Disease ##s ; infections zoo ##not ##ic ; Zoo ##not ##ic In ##fect ##ion ; Zoo ##not ##ic Disease ; In ##fect ##ious Disease , Zoo ##not ##ic ; In ##fect ##ions , Zoo ##not ##ic ; Zoo ##not ##ic In ##fect ##ious Disease ##s ; Disease , Zoo ##not ##ic ; Disease ##s , Zoo ##not ##ic In ##fect ##ious ; zoo ##nose ; zoo ##not ##ic infection ; In [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28197156.25',
  'test_set_idx': 28095,
  'type': 'T017',
  'name': 'Carbon Assimilation Rate 8',
  'context': '[CLS] As ##si ##mi ##lation Rate 8 en ##codes a put ##ative He ##me Act ##iva ##tor Pro ##tein 3 ( O ##s ##HA ##P ##3 ) subunit of a CC ##AA ##T - box - binding transcription factor called O ##s ##HA ##P ##3 ##H . Se ##quencing analysis revealed that the in ##dic ##a all ##ele of [unused1] Carbon As ##si ##mi ##lation Rate 8 [unused2] has a 1 - b ##p del ##eti ##on at 322 b ##p from the start co ##don , resulting in a t ##runcated protein of 125 amino acids . In addition , Carbon As ##si ##mi ##lation Rate 8 is identical to D ##TH ##8 / G ##h ##d ##8 / L ##HD ##1 , which was reported [SEP]',
  'existing_cui': 'C2246620',
  'existing_cui_name': 'iron assimilation',
  'existing_cui_desc': '[CLS] iron ass ##im ##ilation [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_bi_cui': 'C1427100',
  'our_bi_cui_name': 'SLC9A8 gene',
  'our_bi_cui_desc': '[CLS] SL ##C ##9 ##A ##8 gene [unused3] ( Ana ##tom ##ical Structure : so ##lut ##e carrier family 9 member A ##8 ; S ##OD ##I ##UM / H ##Y ##DR ##O ##GE ##N E ##X ##CH ##AN ##GE ##R 8 ; S ##OL ##UT ##E CA ##R ##RI ##ER FA ##MI ##L ##Y 9 ( S ##OD ##I ##UM / H ##Y ##DR ##O ##GE ##N E ##X ##CH ##AN ##GE ##R ) , ME ##MB ##ER 8 ; K ##IA ##A ##0 ##9 ##39 ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27372076.29',
  'test_set_idx': 3980,
  'type': 'T092',
  'name': 'industrial',
  'context': '[CLS] Here ##in , we demonstrate for the first time that De ##o ##R family proteins derived from the same source are likely to be a single branch in a phylogenetic tree and show that w ##ys ##R ##3 acts as a re ##press ##or for its m ##or ##phological development without effect ##ing w ##uy ##ien ##cin production . We found that the Δ ##wy ##s ##R ##3 strain can grow quickly to reach a plateau stage of maximum bio ##mass at 60 h , which is ~ 12 h faster than the wild - type strain . In the [unused1] industrial [unused2] f ##er ##mentation production process , the Δ ##wy ##s ##R ##3 strain can reduce consumption and save both time and money . [SEP]',
  'existing_cui': 'C0442611',
  'existing_cui_name': 'Industrial site',
  'existing_cui_desc': '[CLS] Industrial site [unused3] ( Spa ##tial Concept , environment ) [SEP]',
  'our_bi_cui': 'C0557758',
  'our_bi_cui_name': 'Industrial environment',
  'our_bi_cui_desc': '[CLS] Industrial environment [unused3] ( Spa ##tial Concept , environment ) [SEP]',
  'our_cross_cui': 'C0007983',
  'our_cross_cui_name': 'Chemical Industry',
  'our_cross_cui_desc': '[CLS] Chemical Industry [unused3] ( Organization : Chemical Industries ; IN ##D ##US ##T CH ##EM ; Industry , Chemical ; chemical industry ; chemical industries ; CH ##EM IN ##D ##US ##T ; Industries , Chemical ; Chemical industry ) [SEP]'},
 {'mention_id': '28077286.36',
  'test_set_idx': 25086,
  'type': 'T103',
  'name': 'MeHg',
  'context': '[CLS] , we used the egg injection method to investigate the long term effects of in o ##vo Me ##H ##g exposure on brain his ##top ##ath ##ology and courts ##hip behavior in a model song ##bird species , the z ##eb ##ra fin ##ch ( Ta ##eni ##opy ##gia gut ##tata ) . Egg treatment groups included : a low [unused1] Me ##H ##g [unused2] dose of 0 . 2 μ ##g H ##g g ( - 1 ) egg , a high Me ##H ##g dose of 3 . 2 μ ##g H ##g g ( - 1 ) egg , and a vehicle control ( water ) . No adverse effects of in o ##vo Me ##H ##g treatment were detected on courts ##hip [SEP]',
  'existing_cui': 'C0228520',
  'existing_cui_name': 'Structure of reticular formation of medulla oblongata',
  'existing_cui_desc': '[CLS] Structure of re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata [unused3] ( Ana ##tom ##ical Structure , body structure : bulb ##ar re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla ; B ##ul ##bar re ##tic ##ular formation ; Form ##ati ##o re ##tic ##ular ##is my ##ele ##nce ##pha ##li ; me ##du ##llar ##y re ##tic ##ular formation ; re ##tic ##ular formation of me ##du ##lla ; Me ##du ##llar ##y re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata ) [SEP]',
  'our_bi_cui': 'C0025424',
  'our_bi_cui_name': 'Mercury',
  'our_bi_cui_desc': '[CLS] Mercury [unused3] ( Chemical , H ##g , Metal , elemental , substance : mercury ; Mercury [ Chemical / In ##g ##red ##ient ] ; metal mercury ; H ##yd ##rar ##gy ##rum ; H ##g element ; Li ##quid Silver ; Quick ##si ##lve ##r ; H ##g ; H ##g - Mercury ; h ##g ; ME ##RC ##UR ##Y ) [SEP]',
  'our_cross_cui': 'C0025794',
  'our_cross_cui_name': 'Methylmercury Compounds',
  'our_cross_cui_desc': '[CLS] Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [unused3] ( Chemical : Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [ Chemical / In ##g ##red ##ient ] ; Mercury Co ##mpo ##unds , Met ##hyl ; ME ##TH ##Y ##L ME ##RC ##UR ##Y CP ##DS ; Met ##hyl Mercury Co ##mpo ##unds ; ME ##TH ##Y ##LM ##ER ##CU ##R ##Y CP ##DS ; Co ##mpo ##unds , Met ##hyl ##mer ##cu ##ry ; met ##hyl ##mer ##cu ##ry ; Co ##mpo ##unds , Met ##hyl Mercury ; Met ##hyl ##mer ##cu ##ry ) [SEP]'},
 {'mention_id': '28179911.98',
  'test_set_idx': 27692,
  'type': 'T103',
  'name': 'C18 :1',
  'context': '[CLS] the wild type . Ana ##ly ##ses of the seed oil content , star ##ch content , and total protein content indicated that the two T ##2 trans ##genic lines showed a significant increase ( P < 0 . 05 ) in seed oil content . The trans ##genic lines also showed a significant increase in star ##ch content , whereas total protein content decreased significantly . Further analysis of the fatty acid composition revealed that palm ##itic acid ( C ##16 : 0 ) and l ##ino ##len ##ic acid ( C ##18 : 3 ) increased significantly in the seeds of the trans ##genic rice lines , but o ##le ##ic acid ( [unused1] C ##18 : 1 [unused2] ) levels significantly declined . [SEP]',
  'existing_cui': 'C3889128',
  'existing_cui_name': 'BBS9 wt Allele',
  'existing_cui_desc': '[CLS] B ##BS ##9 w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : C ##18 ; PT ##H - Re ##sp ##ons ##ive B ##1 Gene ; PT ##H ##B ##1 ; Para ##thy ##roid Ho ##rmon ##e - Re ##sp ##ons ##ive B ##1 Gene ; Bar ##det - B ##ied ##l S ##yn ##drome 9 w ##t All ##ele ; B ##1 ; Para ##thy ##roid Ho ##rmon ##e Re ##sp ##ons ##ive B ##1 Gene ; D ##1 ) [SEP]',
  'our_bi_cui': 'C0044555',
  'our_bi_cui_name': '1-oleoyl-2-acetylglycerol',
  'our_bi_cui_desc': '[CLS] 1 - o ##leo ##yl - 2 - ace ##ty ##l ##gly ##cer ##ol [unused3] ( Chemical : 9 - Oct ##ade ##ce ##no ##ic acid ( Z ) - , 2 - ( ace ##ty ##lo ##xy ) - 3 - h ##ydro ##xy ##p ##rop ##yl est ##er , ( S ) - ; 1 - o ##leo ##yl - 2 - ace ##ty ##l - s ##n - g ##ly ##cer ##ol ; o ##ley ##l ace ##ty ##l g ##ly ##cer ##ol ; 1 - o ##ley ##l - 2 - ace ##ty ##l ##gly ##cer ##ol ; 1 - o ##leo ##yl - 2 - ace ##ty ##l - g ##ly ##cer ##ol ; o ##leo ##yla ##ce ##ty ##l ##gly [SEP]',
  'our_cross_cui': 'C0028928',
  'our_cross_cui_name': 'Oleic Acid',
  'our_cross_cui_desc': '[CLS] Ole ##ic A ##cid [unused3] ( Chemical , product , substance : Ole ##ic A ##cid [ Chemical / In ##g ##red ##ient ] ; O ##CT ##AD ##EC ##EN ##O ##IC AC ##ID 09 ; c ##is - 9 - Oct ##ade ##ce ##no ##ic A ##cid ; Oct ##ade ##c - 9 - en ##oi ##c acid ; acid o ##le ##ic ; 9 - Oct ##ade ##ce ##no ##ic acid ; 9 - Oct ##ade ##ce ##no ##ic A ##cid ; Fat ##ty A ##cid 18 : 1 n - 9 ; 9 Oct ##ade ##ce ##no ##ic A ##cid ; c ##is 9 Oct ##ade ##ce ##no ##ic A ##cid ; O ##LE ##IC AC ##ID ; o ##le ##ic acid ; [SEP]'},
 {'mention_id': '27603112.2',
  'test_set_idx': 11344,
  'type': 'T103',
  'name': 'Biomolecules',
  'context': '[CLS] Bin ##ding of Poll ##uta ##nts to [unused1] B ##io ##mo ##le ##cule ##s [unused2] : A Si ##mu ##lation Study A number of cases around the world have been reported where animals were found dead or dying with symptoms resembling a th ##iam ##ine ( vitamin B ) deficiency , and for some of these , a link to poll ##uta ##nts has been suggested . Here , we investigate whether bio ##mo ##le ##cule ##s involved in th ##iam ##in binding and transport could be blocked by a range of different poll ##uta ##nts . We used in si ##lic ##o dock ##ing of five compound classes ( 25 compounds in total ) to each of five targets ( p ##rion protein , EC [SEP]',
  'existing_cui': 'C0104263',
  'existing_cui_name': 'artificial lung expanding compound',
  'existing_cui_desc': '[CLS] artificial lung expanding compound [unused3] ( Chemical , AL ##EC , product , substance : P ##uma ##ct ##ant ; pu ##ma ##ct ##ant ) [SEP]',
  'our_bi_cui': 'C1449655',
  'our_bi_cui_name': 'Macromolecular Substances',
  'our_bi_cui_desc': '[CLS] Mac ##rom ##ole ##cular Sub ##stance ##s [unused3] ( Chemical : Mac ##rom ##ole ##cular Sub ##stance ##s [ Chemical / In ##g ##red ##ient ] ; Mac ##rom ##ole ##cular Co ##mpo ##unds and Complex ##es ; Sub ##stance ##s , Mac ##rom ##ole ##cular ) [SEP]',
  'our_cross_cui': 'C0574031',
  'our_cross_cui_name': 'Biologically active substance',
  'our_cross_cui_desc': '[CLS] Biological ##ly active substance [unused3] ( Chemical , substance : Biological ##ly Active Sub ##stance ) [SEP]'},
 {'mention_id': '28269352.10',
  'test_set_idx': 30583,
  'type': 'T170',
  'name': 'elderly fall risk assessment model',
  'context': '[CLS] Fall risk factors analysis based on sample en ##tropy of plant ##ar kin ##ema ##tic signal during stance phase Falls are a multi - ca ##usal phenomenon with a complex interaction . The aim of our research is to study the effect of multiple variables for potential risk of falls and construct an [unused1] elderly fall risk assessment model [unused2] based on demographic ##s data and g ##ait characteristics . A total of 101 subjects , whom belong to Mali ##an ##wa Street , aged above 50 years old and participated in question ##naire survey . Part ##ici ##pants were classified into three groups ( high , medium and low risk group ) according to the score of elderly fall risk assessment scale . In addition [SEP]',
  'existing_cui': 'C1532976',
  'existing_cui_name': 'Fall risk assessment',
  'existing_cui_desc': '[CLS] Fall risk assessment [unused3] ( Health Care Act ##ivity , procedure , regime / therapy ) [SEP]',
  'our_bi_cui': 'C2733220',
  'our_bi_cui_name': 'Falls risk assessment score for the elderly',
  'our_bi_cui_desc': '[CLS] Falls risk assessment score for the elderly [unused3] ( Clinical At ##tri ##but ##e , o ##bs ##er ##vable entity , o ##bs ##er ##vable entity : F ##RA ##SE - Falls risk assessment score for the elderly ) [SEP]',
  'our_cross_cui': 'C3161035',
  'our_cross_cui_name': 'Model',
  'our_cross_cui_desc': '[CLS] Model [unused3] ( Intel ##lect ##ual Product : Models ; Model ##ing System ; Model System ) [SEP]'},
 {'mention_id': '28128937.20',
  'test_set_idx': 26379,
  'type': 'T103',
  'name': 'CaCl2',
  'context': '[CLS] ##ulated E ##S - Z ##n f ##ert ##ilizer ##s with and without S - o ##xi ##di ##zing micro ##or ##gan ##isms , a commercial elemental sulfur past ##ille , Z ##n ##SO ##4 , and Z ##n ##O were applied to the center of Pet ##ri dishes containing two contrasting pH soils . So ##il pH , [unused1] C ##a ##C ##l ##2 [unused2] - extract ##able sulfur and zinc , and remaining elemental sulfur were evaluated at 30 and 60 days in two soil sections ( 0 - 5 and 5 - 9 mm from the f ##ert ##ilizer application site ) . A visual ##ization test was performed to evaluate zinc diffusion over time . A significant pH decrease was observed in [SEP]',
  'existing_cui': 'C1705551',
  'existing_cui_name': 'ACSL1 wt Allele',
  'existing_cui_desc': '[CLS] AC ##SL ##1 w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : LA ##CS ##2 ; A ##cy ##l - Co ##A S ##ynth ##eta ##se Long - Chain Family Member 1 w ##t All ##ele ; LA ##CS ##1 ; LA ##CS ; FA ##CL ##1 ; FA ##CL ##2 ) [SEP]',
  'our_bi_cui': 'C0596235',
  'our_bi_cui_name': 'Calcium ion',
  'our_bi_cui_desc': '[CLS] Cal ##ci ##um ion [unused3] ( Chemical : Cal ##ci ##um Cat ##ion ; free calcium ; CA + 2 ; ion calcium ; Cal ##ci ##um cat ##ion ; Free Cal ##ci ##um ; calcium ions ; C ##a ##2 + ; C ##a + + ; Cal ##ci ##um Ion ##s ; C ##a 2 + ; calcium ion ; CA ##LC ##I ##UM CA ##TI ##ON ; Cal ##ci ##um Ion ; calcium cat ##ion ; Cal ##ci ##um 2 + ; Ion ##ized Cal ##ci ##um ) [SEP]',
  'our_cross_cui': 'C0006686',
  'our_cross_cui_name': 'Calcium Chloride',
  'our_cross_cui_desc': '[CLS] Cal ##ci ##um Ch ##lor ##ide [unused3] ( Chemical , product , substance : calcium ( 2 + ) chloride ; CA ##LC ##I ##UM CH ##L ##OR ##ID ##E ; calcium chloride ; Cal ##ci ##um chloride ; Ch ##lor ##ide , Cal ##ci ##um ; Cal ##ci ##um Ch ##lor ##ide He ##xa ##hy ##dra ##te ; Cal ##ci ##um Ch ##lor ##ide [ Chemical / In ##g ##red ##ient ] ) [SEP]'},
 {'mention_id': '28263181.26',
  'test_set_idx': 30386,
  'type': 'T170',
  'name': 'theorem',
  'context': '[CLS] f ##er ##ment ##ative metabolism decreases around 10 % the production of en ##tropy per mole of glucose consumed in cancer cells . We h ##y ##pot ##hes ##ize that increased f ##er ##mentation could allow cancer cells to accomplish the P ##rig ##og ##ine theorem of the trend to minimize the rate of production of en ##tropy . According the [unused1] theorem [unused2] , open cellular systems near the steady state could evolve to minimize the rates of en ##tropy production that may be reached by modified replica ##ting cells producing en ##tropy at low rate . Re ##mark ##ably , at CO ##2 concentrations above 93 ##0 pp ##m , glucose re ##spiration produces less en ##tropy than f ##er ##mentation , which suggests [SEP]',
  'existing_cui': 'C0034673',
  'existing_cui_name': 'rapid eye movement',
  'existing_cui_desc': '[CLS] rapid eye movement [unused3] ( B ##iol ##og ##ic Fun ##ction : Movement , Rapid Eye ; Rapid Eye Movement ; Rapid Eye Movement ##s ; Eye Movement ##s , Rapid ; Movement ##s , Rapid Eye ; Eye Movement , Rapid ) [SEP]',
  'our_bi_cui': 'C1705273',
  'our_bi_cui_name': 'Mathematical Operator',
  'our_bi_cui_desc': '[CLS] Mathematical Opera ##tor [unused3] ( Intel ##lect ##ual Product : Opera ##tor ; Fun ##ction ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27692319.47',
  'test_set_idx': 14318,
  'type': 'T204',
  'name': 'miracidia',
  'context': '[CLS] at concentrations of 1000 , 100 , 10 , 1 , 0 . 1 , 0 . 01 μ ##g / m ##L affected em ##b ##ryo ##nic development , with n - but ##ano ##l presenting the strongest in ##hibition of mi ##rac ##id ##ia formation . In contrast , on the 12th day , 90 % of the [unused1] mi ##rac ##id ##ia [unused2] hatch ##ed in the control experiments using 0 . 03 % D ##MS ##O whereas em ##b ##ryo ##genesis was completely abolished with any concentration of al ##ben ##da ##zo ##le su ##l ##ph ##oxide AB ##Z ( S ##O ) . Chemical analysis of the crude extract and sub - fraction ##s revealed a prominent presence of fl ##av [SEP]',
  'existing_cui': 'C0451085',
  'existing_cui_name': 'Composite international diagnostic interview',
  'existing_cui_desc': '[CLS] Co ##mpo ##site international diagnostic interview [unused3] ( Intel ##lect ##ual Product , C ##ID ##I , assessment scale : composite international diagnostic interview ; C ##ID ##I - Co ##mpo ##site international diagnostic interview ) [SEP]',
  'our_bi_cui': 'C2807025',
  'our_bi_cui_name': 'Focillidia',
  'our_bi_cui_desc': '[CLS] F ##oc ##ill ##id ##ia [unused3] ( E ##uka ##ryo ##te ) [SEP]',
  'our_cross_cui': 'C0023047',
  'our_cross_cui_name': 'Larva',
  'our_cross_cui_desc': '[CLS] La ##rva [unused3] ( E ##uka ##ryo ##te : La ##rva ##e ; la ##rva ; larvae ) [SEP]'},
 {'mention_id': '28179911.66',
  'test_set_idx': 27660,
  'type': 'T204',
  'name': 'T2',
  'context': '[CLS] ##cer ##ols assembly ( BC ##CP ##2 , K ##AS ##I , MA ##T , E ##NR , FA ##TA , and GP ##D ##H ) were also ass ##ay ##ed in mature seeds . Furthermore , lip ##id and fatty acids C ##16 : 0 and C ##18 : 0 significantly increased . In two ho ##mo ##zy ##go ##us [unused1] T ##2 [unused2] trans ##genic rice lines ( G ##5 and G ##2 ) , different Co ##WR ##I ##1 expression levels were detected , but no Co ##WR ##I ##1 trans ##cripts were detected in the wild type . Ana ##ly ##ses of the seed oil content , star ##ch content , and total protein content indicated that the two T ##2 trans [SEP]',
  'existing_cui': 'C1335834',
  'existing_cui_name': 'SLC25A5 gene',
  'existing_cui_desc': '[CLS] SL ##C ##25 ##A ##5 gene [unused3] ( Ana ##tom ##ical Structure : AD ##P / ATP T ##RA ##NS ##L ##OC ##AS ##E 2 ; 2 ##F ##1 ; Sol ##ute Carrier Family 25 ( Mi ##to ##cho ##nd ##rial Carrier ; Aden ##ine N ##uc ##leo ##tide Trans ##loc ##ator ) , Member 5 Gene ; AD ##EN ##IN ##E N ##UC ##LE ##OT ##ID ##E T ##RA ##NS ##L ##OC ##AT ##OR 2 ; SL ##C ##25 ##A ##5 Gene ; AD ##P / ATP CA ##R ##RI ##ER 2 ; so ##lut ##e carrier family 25 member 5 ; AD ##P / ATP T ##RA ##NS ##L ##OC ##AT ##OR OF F ##IB ##RO ##BL ##AS ##TS ; S ##OL ##UT ##E [SEP]',
  'our_bi_cui': 'C0282641',
  'our_bi_cui_name': 'Transgenes',
  'our_bi_cui_desc': '[CLS] Trans ##gene ##s [unused3] ( Ana ##tom ##ical Structure : trans ##gene ; Trans ##gene ) [SEP]',
  'our_cross_cui': 'C0085245',
  'our_cross_cui_name': 'Plants , Transgenic',
  'our_cross_cui_desc': '[CLS] Plants , Trans ##genic [unused3] ( E ##uka ##ryo ##te : Trans ##genic Plants ; Trans ##genic Plant ; trans ##genic plant ; trans ##genic plants ; Plant , Trans ##genic ; plant trans ##genic ; Trans ##genic plants ) [SEP]'},
 {'mention_id': '28380217.17',
  'test_set_idx': 33737,
  'type': 'T033',
  'name': 'antibacterial potential',
  'context': '[CLS] derivatives bearing 2 - fur ##oy ##l - 1 - pipe ##raz ##ine and having modest toxicity . The synthesis was completed as a multiple sequence . The structural confirmation of all the synthesized compounds was obtained by E ##I - MS , I ##R and 1 ##H - N ##MR spectral data . The enzyme in ##hibition and [unused1] anti ##ba ##cter ##ial potential [unused2] of the synthesized compounds was evaluated . To find the utility of the prepared compounds as possible therapeutic agents their c ##yt ##oto ##xi ##city was also checked . All the compounds were active against ace ##ty ##l ##cho ##lines ##tera ##se enzyme , especially 12 and 14 showed very good inhibitor ##y potential relative to E ##ser ##ine , [SEP]',
  'existing_cui': 'C0279516',
  'existing_cui_name': 'Anti-Bacterial Agents',
  'existing_cui_desc': '[CLS] Anti - Ba ##cter ##ial Agents [unused3] ( Chemical , medication , product , substance : Anti ##ba ##cter ##ial agent ; Anti ##ba ##cter ##ial ##s ; anti ##ba ##cter ##ial ##s ; Anti Ba ##cter ##ial Co ##mpo ##unds ; anti - bacterial agents ; Anti ##ba ##cter ##ial Agent [ T ##C ] ; Anti ##ba ##cter ##ial Agent ; AN ##TI ##BA ##CT ##ER ##IA ##L AG ##EN ##TS ; Anti ##ba ##cter ##ial Agents ; Agents , Anti ##ba ##cter ##ial ; Anti - Ba ##cter ##ial Co ##mpo ##unds ; Anti ##ba ##cter ##ial ; Co ##mpo ##unds , Anti - Ba ##cter ##ial ; Anti Ba ##cter ##ial Agents ; anti ##ba ##cter ##ial drug ; anti ##ba ##cter [SEP]',
  'our_bi_cui': 'C0279516',
  'our_bi_cui_name': 'Anti-Bacterial Agents',
  'our_bi_cui_desc': '[CLS] Anti - Ba ##cter ##ial Agents [unused3] ( Chemical , medication , product , substance : Anti ##ba ##cter ##ial agent ; Anti ##ba ##cter ##ial ##s ; anti ##ba ##cter ##ial ##s ; Anti Ba ##cter ##ial Co ##mpo ##unds ; anti - bacterial agents ; Anti ##ba ##cter ##ial Agent [ T ##C ] ; Anti ##ba ##cter ##ial Agent ; AN ##TI ##BA ##CT ##ER ##IA ##L AG ##EN ##TS ; Anti ##ba ##cter ##ial Agents ; Agents , Anti ##ba ##cter ##ial ; Anti - Ba ##cter ##ial Co ##mpo ##unds ; Anti ##ba ##cter ##ial ; Co ##mpo ##unds , Anti - Ba ##cter ##ial ; Anti Ba ##cter ##ial Agents ; anti ##ba ##cter ##ial drug ; anti ##ba ##cter [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28035661.11',
  'test_set_idx': 23939,
  'type': 'T082',
  'name': 'Hortus',
  'context': '[CLS] Evidence of tooth ##pic ##k groove formation in N ##ean ##der ##tal anterior and posterior teeth During the micro ##scopic examination of the N ##ean ##der ##tal den ##ti ##tions from El Sid ##r ##ón ( Spain ) and [unused1] Ho ##rt ##us [unused2] ( France ) , we found unusual fine parallel micro ##st ##ria ##tions on the me ##sia ##l and di ##stal sides of all tooth types , near the c ##er ##vi ##x . As its appearance was similar to tooth ##pic ##k groove ##s described in other Ho ##mo species , it could correspond to early stages on its formation . To test this hypothesis we developed an experimental replication of a groove using grass stalk ##s . Co ##mp ##aris [SEP]',
  'existing_cui': 'C4178118',
  'existing_cui_name': 'Tortus',
  'existing_cui_desc': '[CLS] Tor ##tus [unused3] ( E ##uka ##ryo ##te ) [SEP]',
  'our_bi_cui': 'C0086418',
  'our_bi_cui_name': 'Homo sapiens',
  'our_bi_cui_desc': '[CLS] Ho ##mo sa ##pie ##ns [unused3] ( E ##uka ##ryo ##te , Tax ##onomy , organism , qualifier value : Humans ; Man , Modern ; humans ; man ; MA ##N ; Ho ##mo sa ##pie ##ns ( living organism ) [ Am ##bi ##guous ] ; ho ##mo sa ##pie ##ns ; Human , General ; Ho ##mo sa ##pie ##ns Linnaeus , 1758 ; Human ; Human - origin ; human ; Ho ##mo sa ##pie ##ns ( living organism ) ; Modern Man ; Man ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '28125734.44',
  'test_set_idx': 26198,
  'type': 'T103',
  'name': 'MiR-21-5p',
  'context': '[CLS] ( Met - 5 ##A ) which expresses low levels of me ##so ##the ##lin . Mi ##RNA ##s targeting the me ##so ##the ##lin m ##RNA were identified by next generation se ##quencing and mi ##R - 21 - 5 ##p and mi ##R - 100 - 5 ##p were selected for further valid ##ation analyses . [unused1] Mi ##R - 21 - 5 ##p [unused2] was shown to be able to m ##od ##ulate me ##so ##the ##lin expression in mi ##RNA mimic experiments in a panel of ma ##li ##gnant and non - ma ##li ##gnant cell lines . Further mi ##RNA inhibitor experiments and l ##uc ##ifer ##ase ass ##ays in Me ##ro - 14 cells valid ##ated mi ##R - 21 [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1999986',
  'our_bi_cui_name': 'MIRN21 microRNA , human',
  'our_bi_cui_desc': '[CLS] MI ##R ##N ##21 micro ##RNA , human [unused3] ( Chemical : micro ##RNA - 21 , human ; mi ##RNA - 21 , human ; h ##sa - mi ##r - 21 micro ##RNA ; mi ##R - 21 , human ; mi ##R ##21 , human ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '28223077.18',
  'test_set_idx': 28902,
  'type': 'T082',
  'name': 'background site',
  'context': '[CLS] to understand the influence of Municipal solid waste land ##fill on p ##oly ##ch ##lor ##inated di ##ben ##zo - p - di ##ox ##ins and p ##oly ##ch ##lor ##inated di ##ben ##zo ##fu ##ran ##s ( PC ##D ##D / F ##s ) in surrounding atmosphere , 42 ambient air samples were collected and analyzed from surrounding sites , [unused1] background site [unused2] , up ##wind site and down ##wind site of a Municipal solid waste land ##fill in East China . The results of present study were summarized as follows . ( 1 ) The total concentrations of PC ##D ##D / F ##s ( [UNK] PC ##D ##D / F ##s ) in ambient air from surrounding sites , background site , [SEP]',
  'existing_cui': 'C0237490',
  'existing_cui_name': 'Family Background',
  'existing_cui_desc': '[CLS] Family Background [unused3] ( Finding , Family : Background ) [SEP]',
  'our_bi_cui': 'C0017446',
  'our_bi_cui_name': 'Geographic Locations',
  'our_bi_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]',
  'our_cross_cui': 'C0205145',
  'our_cross_cui_name': 'Site',
  'our_cross_cui_desc': '[CLS] Site [unused3] ( Spa ##tial Concept , attribute , qualifier value : Sites ) [SEP]'},
 {'mention_id': '28210319.72',
  'test_set_idx': 28603,
  'type': 'T033',
  'name': 'respiratory implications',
  'context': '[CLS] lung compliance decrease , and air ##way o ##bs ##truction makes air ##way resistance increase . The consequences of respiratory pump failure are restrict ##ive pulmonary function , h ##y ##po ##vent ##ilation , altered th ##ora ##coa ##b ##dom ##inal pattern , h ##yper ##cap ##nia , d ##ys ##p ##no ##ea , impaired regulation of breathing , in ##ef ##ficient cough and sleep disorder ##ed breathing . To understand the mechanisms leading to respiratory disturbance ##s in patients with muscular d ##ys ##tro ##phy . To understand the impact of respiratory disturbance ##s in patients with muscular d ##ys ##tro ##phy . To provide a brief description of the main forms of muscular d ##ys ##tro ##phy with their [unused1] respiratory implications [unused2] . [SEP]',
  'existing_cui': 'C0729250',
  'existing_cui_name': 'Respiratory complications',
  'existing_cui_desc': '[CLS] Re ##sp ##ira ##tory complications [unused3] ( In ##ju ##ry or Po ##ison ##ing ) [SEP]',
  'our_bi_cui': 'C0161818',
  'our_bi_cui_name': 'Respiratory complication',
  'our_bi_cui_desc': '[CLS] Re ##sp ##ira ##tory com ##plication [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : respiratory ; complications ; complications ; respiratory ; respiratory complications ; Re ##sp ##ira ##tory complications ; respiratory com ##plication ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28050885.57',
  'test_set_idx': 24203,
  'type': 'T033',
  'name': 'No',
  'context': '[CLS] and ankle - bra ##chia ##l index values increased by 24 % ( p = 0 . 000 ##1 ) in the treatment group but decreased in the control group . The greatest therapeutic effect was observed for stage III disease : Pain - free walking distance increased by 68 ##3 % ( p = 0 . 000 ##1 ) . [unused1] No [unused2] an ##gio ##genic therapy - related adverse events or side effects were recorded , and target limb salvage was 96 and 97 % in the treatment and control groups , respectively . The results obtained in this study are not significantly different from those observed in the phase II ##b / III registration clinical study completed in 2011 . p ##l - [SEP]',
  'existing_cui': 'C1016143',
  'existing_cui_name': 'Tetragenococcus halophilus',
  'existing_cui_desc': '[CLS] Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##us [unused3] ( Ba ##cter ##ium , organism : P ##ed ##io ##co ##cc ##us ha ##lop ##hil ##us Me ##es 1934 ( A ##pp ##roved List ##s 1980 ) ; Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##us ( Me ##es 1934 ) Collins et al . 1993 ; Sa ##rc ##ina ha ##ma ##guchi ##ae ; Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##a ; P ##ed ##io ##co ##cc ##us ha ##lop ##hil ##us ; Te ##tra ##co ##cc ##cus No . 1 ; P ##ed ##io ##co ##cc ##us so ##ya ##e ) [SEP]',
  'our_bi_cui': 'C0243095',
  'our_bi_cui_name': 'Finding',
  'our_bi_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]',
  'our_cross_cui': 'C1513916',
  'our_cross_cui_name': 'Negative Finding',
  'our_cross_cui_desc': '[CLS] N ##eg ##ative Finding [unused3] ( Finding : NE ##GA ##TI ##VE ; N ##eg ##ative ) [SEP]'},
 {'mention_id': '28000391.18',
  'test_set_idx': 23417,
  'type': 'T033',
  'name': 'after hospital discharge',
  'context': '[CLS] monitoring . This contains for quantitative assessment the Short Form Health Survey question ##naire and the Impact of Family Scale administered one and five weeks following discharge before and after stage II . For q ##ual ##itative assessment , semi - structured interviews focus ##sing on the post ##dis ##cha ##rge cop ##ing strategies were conducted twice , five weeks [unused1] after hospital discharge [unused2] before and after stage II . Ten infants ( eight males ) with h ##y ##pop ##lastic left heart syndrome ( n = 7 ) or other types of un ##ive ##nt ##ric ##ular heart ma ##lf ##orm ##ations ( n = 3 ) , and their parents ( nine mother / father two - parent households , one single mother [SEP]',
  'existing_cui': 'C0586003',
  'existing_cui_name': 'Discharge from hospital',
  'existing_cui_desc': '[CLS] Disc ##har ##ge from hospital [unused3] ( Health Care Act ##ivity , procedure : discharge ##s hospital ; Hospital Disc ##har ##ge ; discharge from hospital ; hospital discharge ; discharge hospital ) [SEP]',
  'our_bi_cui': 'C0586003',
  'our_bi_cui_name': 'Discharge from hospital',
  'our_bi_cui_desc': '[CLS] Disc ##har ##ge from hospital [unused3] ( Health Care Act ##ivity , procedure : discharge ##s hospital ; Hospital Disc ##har ##ge ; discharge from hospital ; hospital discharge ; discharge hospital ) [SEP]',
  'our_cross_cui': 'C1320368',
  'our_cross_cui_name': 'Post-discharge follow-up',
  'our_cross_cui_desc': '[CLS] Post - discharge follow - up [unused3] ( Finding , finding ) [SEP]'},
 {'mention_id': '27838742.17',
  'test_set_idx': 18984,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] developing novel ch ##em ##oth ##era ##pe ##uti ##c agents . In this study , data supporting the in v ##it ##ro and in v ##ivo anti ##can ##cer effects of del ##ica ##f ##lav ##one , a rarely occurring bi ##f ##lav ##ono ##id from Se ##lag ##ine ##lla do ##ede ##rle ##ini ##i , were reported . [unused1] Del ##ica ##f ##lav ##one [unused2] exhibited favorable anti ##can ##cer properties , as shown by the M ##TT ass ##ay and x ##eno ##gra ##ft model of human non - small cell lung cancer in male BA ##L ##B / c nude mice without o ##bs ##er ##vable adverse effect . By transmission electron micro ##sco ##py with a ##c ##rid ##ine orange and Cy [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28487667.6',
  'test_set_idx': 37681,
  'type': 'T170',
  'name': 'test batteries',
  'context': "[CLS] When Is a Test Score Fair for the Individual Who Is Being Test ##ed ? Effects of Different Sc ##oring Pro ##ced ##ures across Multiple At ##tem ##pts When Testing a Motor Ski ##ll Task Tests or [unused1] test batteries [unused2] used for assessing motor skills , either in research studies or in clinical settings , apply a variety of procedures for scoring performances , including everything from one to ten attempts , of which the best is scored or an average is com ##puted . The rational ##e behind scoring procedures is rarely stated , and it seems that the number of attempts allowed is decided without much qualification from research . It is uncertain whether procedures fairly capture an individual ' s skill level [SEP]",
  'existing_cui': 'C0456724',
  'existing_cui_name': 'Willeford test battery',
  'existing_cui_desc': '[CLS] Will ##ef ##ord test battery [unused3] ( Health Care Act ##ivity , procedure ) [SEP]',
  'our_bi_cui': 'C0687197',
  'our_bi_cui_name': 'Battery tester',
  'our_bi_cui_desc': '[CLS] Battery test ##er [unused3] ( Medical Devi ##ce , physical object : Battery Ana ##ly ##zers ; Test ##ers , Battery ) [SEP]',
  'our_cross_cui': 'C0392366',
  'our_cross_cui_name': 'Tests',
  'our_cross_cui_desc': '[CLS] Tests [unused3] ( Intel ##lect ##ual Product , qualifier value , qualifier value , procedure : tested ; testing ##s ; test ; tests ; testing ) [SEP]'},
 {'mention_id': '28356035.5',
  'test_set_idx': 32979,
  'type': 'T033',
  'name': 'poor cardiovascular outcome',
  'context': '[CLS] poor health worldwide . The most solid evidence is for air pollution , leading to increased disability - adjusted life years . Outdoor temperature and other seasonal climate changes may also influence card ##iovascular health , according to their direct m ##od ##ulation of air pollution . Moreover , an increasing body of evidence associates environmental exposure to noise with [unused1] poor card ##iovascular outcome [unused2] , and in particular with h ##yper ##tens ##ion . This review is aimed at reviewing current evidence about the role of these environmental factors in card ##iovascular disease and specifically h ##yper ##tens ##ion . In particular , the impact of air pollution , with its short - term and long - term effects , the outdoor temperature and [SEP]',
  'existing_cui': 'C3806166',
  'existing_cui_name': 'Poor outcome',
  'existing_cui_desc': '[CLS] Poor outcome [unused3] ( Finding ) [SEP]',
  'our_bi_cui': 'C3806166',
  'our_bi_cui_name': 'Poor outcome',
  'our_bi_cui_desc': '[CLS] Poor outcome [unused3] ( Finding ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28441489.46',
  'test_set_idx': 36168,
  'type': 'T033',
  'name': 'cold - start',
  'context': '[CLS] depend on engine technology . Cold - start contributes a larger fraction of the total unified cycle emissions for vehicles meeting more - string ##ent emission standards . Organic gas emissions were the most sensitive to cold - start compared to the other poll ##uta ##nts tested here . There were no statistical ##ly significant differences in the effects of [unused1] cold - start [unused2] on G ##DI ##s and P ##FI ##s . For our test fleet , the measured 14 . 5 % decrease in CO ##2 emissions from G ##DI ##s was much greater than the potential climate forcing associated with higher black carbon emissions . Thus , switching from Port Fuel In ##jection to gasoline direct - injection vehicles will likely lead [SEP]',
  'existing_cui': 'C0452588',
  'existing_cui_name': 'Start brand of breakfast cereal',
  'existing_cui_desc': '[CLS] Start brand of breakfast cereal [unused3] ( Food , substance : Start ) [SEP]',
  'our_bi_cui': 'C1455171',
  'our_bi_cui_name': 'Cool-X-A',
  'our_bi_cui_desc': '[CLS] Cool - X - A [unused3] ( Chemical : Cool - Cool ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27570140.50',
  'test_set_idx': 10513,
  'type': 'T058',
  'name': 'procedures',
  'context': "[CLS] the 27 months of follow - up . Though women with high - risk pre ##gna ##ncies were more likely to request post ##par ##tum tub ##al l ##iga ##tion , they were not more likely to complete the procedure . Over one third of high - risk patients ' requests were un ##ful ##fill ##ed , indicating that significant barriers may remain . Though women with high - risk pre ##gna ##ncies were more likely to request post ##par ##tum tub ##al l ##iga ##tion , they were not more likely to complete the procedure . Pro ##vide ##rs should consider these [unused1] procedures [unused2] urgent , especially in high - risk women , and advocate for their patients ' access to this procedure . [SEP]",
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '28301509.6',
  'test_set_idx': 31703,
  'type': 'T058',
  'name': 'CoMPP technique',
  'context': '[CLS] Gen ##ome - wide association mapping in winter bar ##ley for grain yield and cu ##lm cell wall polymer content using the high - through ##put [unused1] Co ##MP ##P technique [unused2] A collection of 112 winter bar ##ley varieties ( Ho ##rde ##um v ##ul ##gar ##e L . ) was grown in the field for two years ( 2008 / 09 and 2009 / 10 ) in northern Italy and grain and straw yields recorded . In the first year of the trial , a severe attack of bar ##ley yellow mosaic virus strongly influenced final performances with an average reduction of ~ 50 % for grain and straw harvested in comparison to the second year . The genetic determination for grain yield was [SEP]',
  'existing_cui': 'C0450241',
  'existing_cui_name': 'Ffooks technique',
  'existing_cui_desc': '[CLS] F ##fo ##ok ##s technique [unused3] ( Health Care Act ##ivity , qualifier value ) [SEP]',
  'our_bi_cui': 'C0430022',
  'our_bi_cui_name': 'diagnostic procedure',
  'our_bi_cui_desc': '[CLS] diagnostic procedure [unused3] ( Health Care Act ##ivity , procedure : D ##IA ##G ##N ##OS ##TI ##C ; Di ##ag ##nos ##tic procedure ; Di ##ag ##nos ##tic procedures ; Di ##ag ##nos ##tic Test ; diagnostic technique ; Di ##ag ##nos ##tic ; Di ##ag ##nos ##tic Tech ##ni ##que ; Di ##ag ##nos ##tic Pro ##ced ##ure ; diagnostic procedures ; Di ##ag ##nos ##tic Method ; Pro ##ced ##ures ; diagnostic ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '27887587.58',
  'test_set_idx': 20946,
  'type': 'T033',
  'name': 'family',
  'context': "[CLS] . The participants ' expectations about counsel ##ling with regard to longer - term family , practical , and financial challenges were insufficient ##ly met by the Community Mental Health Centre . In the experience of the service users , recovery occurred within the context of their everyday life with or without the support of their professional help ##ers . To facilitate recovery , health professionals should acknowledge the service user ' s personal goals and hopes and take a more comprehensive and longer - term approach to his or her needs and desires . A ##ck ##no ##wl ##ed ##ging and f ##ac ##ilitating recovery goals by offering counsel ##ling with regard to [unused1] family [unused2] , practical and financial issues seems particularly important . [SEP]",
  'existing_cui': 'C1301584',
  'existing_cui_name': 'Last Name',
  'existing_cui_desc': '[CLS] Last Name [unused3] ( Intel ##lect ##ual Product , o ##bs ##er ##vable entity : Family Name ; family ; Last name ; Sur ##name ) [SEP]',
  'our_bi_cui': 'C0424960',
  'our_bi_cui_name': 'Family problems',
  'our_bi_cui_desc': '[CLS] Family problems [unused3] ( Finding , finding , history : problem ; family ; Problem ; family ; family problems ; family ; problem ; family problem ) [SEP]',
  'our_cross_cui': 'C0851511',
  'our_cross_cui_name': 'Family issues',
  'our_cross_cui_desc': '[CLS] Family issues [unused3] ( Finding : family issues ; families issues ; Family Issues ; family issue ) [SEP]'},
 {'mention_id': '28272254.30',
  'test_set_idx': 30776,
  'type': 'T170',
  'name': 'papers',
  'context': '[CLS] ##in receptor block ##ers , calcium channel block ##ers [ CC ##B ##s ] , and di ##ure ##tics ) on re ##gression of Left vent ##ric ##ular h ##yper ##tro ##phy . Re ##lative trials were identified in the Pub ##M ##ed , Web of Science , O ##VI ##D E ##BM Reviews and Cochrane databases , and the relevant [unused1] papers [unused2] were examined . We performed both traditional and Bay ##esi ##an meta - analysis of random ##ized controlled trials about the re ##gression of Left vent ##ric ##ular h ##yper ##tro ##phy . Sen ##si ##ti ##vity analysis and re ##gression analysis were performed to explore possible sources of he ##tero ##gene ##ity . Inc ##ons ##ist ##ency analysis was performed to [SEP]',
  'existing_cui': 'C0462999',
  'existing_cui_name': 'Litmus paper',
  'existing_cui_desc': '[CLS] Li ##t ##mus paper [unused3] ( Medical Devi ##ce , physical object : Li ##t ##mus Paper ; lit ##mus papers ; Re ##age ##nts , Clinical Chemistry , Rapid Test , pH , Strip ; IV ##D Test Re ##age ##nt / Kit ##s , Clinical Chemistry , Rapid Test , pH , Strip ; pH Strip In ##dic ##ators ; lit ##mus paper ) [SEP]',
  'our_bi_cui': 'C0282420',
  'our_bi_cui_name': 'Journal Article',
  'our_bi_cui_desc': '[CLS] Journal Article [unused3] ( Intel ##lect ##ual Product : journal article ) [SEP]',
  'our_cross_cui': 'C1706852',
  'our_cross_cui_name': 'Article',
  'our_cross_cui_desc': '[CLS] Article [unused3] ( Intel ##lect ##ual Product ) [SEP]'},
 {'mention_id': '27791364.5',
  'test_set_idx': 16973,
  'type': 'T058',
  'name': 'Photoacoustic / Magnetic Resonance Imaging - Guided Tumor - Targeted Photothermal Therapy',
  'context': '[CLS] Album ##in - B ##io ##ins ##pired G ##d : C ##u ##S Nan ##oth ##eran ##ost ##ic Agent for In V ##ivo [unused1] Photo ##aco ##ustic / Ma ##gnetic Re ##son ##ance I ##maging - Guide ##d Tu ##mor - Target ##ed Photo ##thermal Therapy [unused2] Photo ##thermal therapy is attracting increasing interest and becoming more widely used for skin cancer therapy in the clinic , as a result of its non ##in ##vasive ##ness and low systemic adverse effects . However , there is an urgent need to develop bio ##com ##pa ##tible Photo ##thermal therapy agents , which enable accurate imaging , monitoring , and diagnosis . Here ##in , a bio ##com ##pa ##tible G ##d - integrated C ##u ##S na [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0031740',
  'our_bi_cui_name': 'Photochemotherapy',
  'our_bi_cui_desc': '[CLS] Photo ##che ##mother ##ap ##y [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : PD ##T - Photo ##dynamic therapy ; Photo ##che ##mother ##ap ##ies ; Photo ##dynamic The ##rap ##ies ; therapy , photo ##dynamic ; Therapy , Photo ##dynamic ; THE ##R P ##H ##OT ##OD ##Y ##NA ##MI ##C ; Photo ##rad ##iation Therapy ; photo ##dynamic therapy ; photo ##che ##mother ##ap ##y ; P ##H ##OT ##OC ##H ##EM ##OT ##H ##ER ; The ##rap ##ies , Photo ##dynamic ; Photo ##dynamic Therapy Pro ##ced ##ures ; neo ##p ##las ##m / cancer photo ##rad ##iation therapy ; Photo ##che ##mother ##ap ##y [ Am ##bi ##guous ] ; Photo ##dynamic therapy ; PD [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '28013256.31',
  'test_set_idx': 23792,
  'type': 'T033',
  'name': 'succeeded',
  'context': "[CLS] with the exception of improving capacity to engage men and in ##f ##lue ##ncing practice beyond their organisation , these improvements were sustained at 5 - month post training ( P < 0 . 00 ##1 ) . The vast majority of service providers ( 93 . 4 % ) reported that E ##NG ##AG ##E had impacted their work practice up to 5 - month post training . The findings suggest that E ##NG ##AG ##E has [unused1] succeeded [unused2] in improving service providers ' capacity to engage and work with men ; improving gender compete ##ncy in the delivery of health and health related services may increase the u ##til ##isation of such services by men and thereby improve health outcomes for men . [SEP]",
  'existing_cui': 'C0046018',
  'existing_cui_name': '2-chloroethyl ethyl sulfide',
  'existing_cui_desc': '[CLS] 2 - ch ##lor ##oe ##thy ##l et ##hyl su ##lf ##ide [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C1272702',
  'our_bi_cui_name': 'Success of action',
  'our_bi_cui_desc': '[CLS] Success of action [unused3] ( Finding , attribute ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27636509.49',
  'test_set_idx': 12561,
  'type': 'T033',
  'name': 'detection',
  'context': '[CLS] be characterized by speech disorder and affect ##ive behavior symptoms , cluster 2 has predominantly hall ##uc ##ination symptoms and cluster 3 has mainly del ##usion symptoms . Inter ##action of five S ##NP ##s was found to have an effect on cluster 1 symptoms ; ten S ##NP ##s on cluster 2 symptoms ; and eight S ##NP ##s on cluster 3 symptoms . The interaction of specific su ##s ##ce ##pt ##ibility genes is likely to lead to specific clinical sub - p ##hen ##otype ##s of s ##chi ##zophrenia . Large ##r patient co ##hor ##ts with more extensive clinical data will improve the [unused1] detection [unused2] of gene interactions and the result ##ant s ##chi ##zophrenia clinical p ##hen ##otype ##s . [SEP]',
  'existing_cui': 'C0206100',
  'existing_cui_name': 'Signal Detection',
  'existing_cui_desc': '[CLS] Signal Det ##ec ##tion [unused3] ( B ##iol ##og ##ic Fun ##ction , Psychology , Psychology , Signal : Signal Det ##ec ##tion Theo ##ries ; Signal Det ##ec ##tions , Psychological ; Theory , Signal Det ##ec ##tion ; Signal detection ; S ##IG ##NA ##L DE ##TE ##CT ##ION AN ##AL ; Psychological Signal Det ##ec ##tions ; Det ##ec ##tion ; Signal Det ##ec ##tion Ana ##ly ##ses ; S ##IG ##NA ##L DE ##TE ##CT ##ION ; Det ##ec ##tions , Psychological Signal ; Psychological Signal Det ##ec ##tion ; Signal Det ##ec ##tion Theory ; Theo ##ries , Signal Det ##ec ##tion ; Signal Det ##ec ##tions ; Ana ##ly ##ses , Signal Det ##ec ##tion ; Det ##ec ##tion , [SEP]',
  'our_bi_cui': 'C1511790',
  'our_bi_cui_name': 'Detection',
  'our_bi_cui_desc': '[CLS] Det ##ec ##tion [unused3] ( Health Care Act ##ivity : Det ##ec ##ted ) [SEP]',
  'our_cross_cui': 'C0442726',
  'our_cross_cui_name': 'Detected',
  'our_cross_cui_desc': '[CLS] Det ##ec ##ted [unused3] ( Finding , finding , finding , qualifier value : detected ; detect ) [SEP]'},
 {'mention_id': '27692319.41',
  'test_set_idx': 14312,
  'type': 'T204',
  'name': 'Eggs',
  'context': '[CLS] . Q ##uad ##rup ##lica ##tes of 50 F . he ##pa ##tica eggs were in ##cu ##bate ##d at 23 °C with M . ch ##aran ##tia leaf crude extract in different concentrations . After 12 days no larvae were formed in eggs in ##cu ##bate ##d with crude extract concentrations above 12 . 5 mg / m ##L . [unused1] Egg ##s [unused2] in ##cu ##bate ##d with crude extract sub - fraction ##s at concentrations of 1000 , 100 , 10 , 1 , 0 . 1 , 0 . 01 μ ##g / m ##L affected em ##b ##ryo ##nic development , with n - but ##ano ##l presenting the strongest in ##hibition of mi ##rac ##id ##ia formation . In contrast [SEP]',
  'existing_cui': 'C0029974',
  'existing_cui_name': 'Ovum',
  'existing_cui_desc': '[CLS] O ##vu ##m [unused3] ( Ana ##tom ##ical Structure , body structure , cell , cell structure , qualifier value : Human egg ; Egg ##s , Un ##fer ##til ##ized ; Fe ##rt ##ilized o ##oc ##yte ; Mat ##ure o ##vu ##m ; O ##va ; Egg ; egg cell ; o ##va ; X bearing o ##vu ##m ; female game ##tes ; O ##ocytes ; Un ##fer ##til ##ized Egg ##s ; X chromosome - bearing o ##vu ##m ; Egg , Un ##fer ##til ##ized ; O ##vu ##m structure ; Un ##fer ##til ##ized Egg ; Female game ##te ; O ##voc ##yte ; egg / o ##vu ##m ; o ##vu ##m ; female game ##te ; X - [SEP]',
  'our_bi_cui': 'C0029974',
  'our_bi_cui_name': 'Ovum',
  'our_bi_cui_desc': '[CLS] O ##vu ##m [unused3] ( Ana ##tom ##ical Structure , body structure , cell , cell structure , qualifier value : Human egg ; Egg ##s , Un ##fer ##til ##ized ; Fe ##rt ##ilized o ##oc ##yte ; Mat ##ure o ##vu ##m ; O ##va ; Egg ; egg cell ; o ##va ; X bearing o ##vu ##m ; female game ##tes ; O ##ocytes ; Un ##fer ##til ##ized Egg ##s ; X chromosome - bearing o ##vu ##m ; Egg , Un ##fer ##til ##ized ; O ##vu ##m structure ; Un ##fer ##til ##ized Egg ; Female game ##te ; O ##voc ##yte ; egg / o ##vu ##m ; o ##vu ##m ; female game ##te ; X - [SEP]',
  'our_cross_cui': 'C3662687',
  'our_cross_cui_name': 'Fasciola hepatica egg',
  'our_cross_cui_desc': '[CLS] F ##as ##cio ##la he ##pa ##tica egg [unused3] ( E ##uka ##ryo ##te , organism : F ##as ##cio ##la he ##pa ##tica o ##va ) [SEP]'},
 {'mention_id': '28326653.24',
  'test_set_idx': 31932,
  'type': 'T170',
  'name': 'Early Trauma Inventory',
  'context': '[CLS] of P ##sy ##cho ##ther ##ap ##y or es ##ci ##tal ##op ##ram plus clinical management over the course of 8 weeks of acute and 20 weeks of extended treatment at 2 German treatment sites . Child ##hood ma ##lt ##rea ##tment was assessed using the Child ##hood T ##ra ##uma Question ##naire and the clinic ##ian rated [unused1] Early T ##ra ##uma In ##vent ##ory [unused2] . In ##ten ##tion - to - treat analyses were used to examine the impact of Child ##hood ma ##lt ##rea ##tment on depression , global functioning , and quality of life . The presence of Child ##hood ma ##lt ##rea ##tment did not result in significant differences in treatment response to Co ##gni ##tive Be ##ha ##vior ##al [SEP]',
  'existing_cui': 'C0031213',
  'existing_cui_name': 'Personality inventories',
  'existing_cui_desc': '[CLS] Personal ##ity inventor ##ies [unused3] ( Health Care Act ##ivity : Personal ##ity In ##vent ##ory ; personality inventory ; inventor ##ies ; inventory ; In ##vent ##ories ; Personal ##ity In ##vent ##ories ; In ##vent ##ory , Personal ##ity ; In ##vent ##ories , Personal ##ity ) [SEP]',
  'our_bi_cui': 'C0451546',
  'our_bi_cui_name': 'Trauma index',
  'our_bi_cui_desc': '[CLS] T ##ra ##uma index [unused3] ( Intel ##lect ##ual Product , assessment scale : index trauma ; trauma index ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27773614.43',
  'test_set_idx': 16303,
  'type': 'T058',
  'name': 'surveillance',
  'context': '[CLS] he ##pa ##to ##cellular car ##cin ##oma - case ##load of the hospital where patients were first admitted also had an independent influence on treatment and survival . Despite full insurance coverage for all citizens , national measures to reduce in ##e ##quities in the management of cancer patients , standard ##ised recommendations for he ##pa ##to ##cellular car ##cin ##oma [unused1] surveillance [unused2] and management , the percentage of patients undergoing cu ##rative treatment and their survival may vary four - fold depending on their post ##code . The hospital in which patients are first managed has a clear influence on access ##ibility to both good care and survival . Population - based studies have highlighted large and sometimes unexpected differences between countries in the [SEP]',
  'existing_cui': 'C0733511',
  'existing_cui_name': 'Medical Surveillance',
  'existing_cui_desc': '[CLS] Medical Sur ##ve ##illa ##nce [unused3] ( Health Care Act ##ivity , procedure , regime / therapy : Sur ##ve ##illa ##nce ; medical surveillance ; surveillance ; E ##pid ##em ##iology / Sur ##ve ##illa ##nce ) [SEP]',
  'our_bi_cui': 'C4316666',
  'our_bi_cui_name': 'Surveillance for cancer',
  'our_bi_cui_desc': '[CLS] Sur ##ve ##illa ##nce for cancer [unused3] ( Health Care Act ##ivity , regime / therapy ) [SEP]',
  'our_cross_cui': 'C0038842',
  'our_cross_cui_name': 'Supervision',
  'our_cross_cui_desc': '[CLS] Super ##vision [unused3] ( Health Care Act ##ivity , regime / therapy , regime / therapy , procedure : supervision ) [SEP]'},
 {'mention_id': '28222192.43',
  'test_set_idx': 28864,
  'type': 'T204',
  'name': 'ST1',
  'context': '[CLS] ##ba co ##li ( 55 % ) , H ##yme ##no ##le ##pis na ##na ( 0 . 8 % ) , End ##oli ##max na ##na ( 33 . 2 % ) and Neo ##bal ##anti ##dium co ##li ( 2 . 7 % ) was detected in the B ##last ##oc ##ys ##tis - positive samples . We detected [unused1] ST ##1 [unused2] ( 21 . 4 % ) , ST ##2 ( 19 . 5 % ) , ST ##3 ( 55 . 5 % ) , ST ##4 ( 0 . 8 % ) , ST ##6 ( 2 % ) and ST ##7 ( 0 . 8 % ) ; all ##ele ##s 1 , 2 , 4 , 81 , [SEP]',
  'existing_cui': 'C3812243',
  'existing_cui_name': 'SDCBP wt Allele',
  'existing_cui_desc': '[CLS] SD ##C ##B ##P w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : S ##yn ##de ##can Bin ##ding Pro ##tein ( S ##yn ##ten ##in ) w ##t All ##ele ; MD ##A - 9 ; S ##Y ##CL ; ST ##1 ; T ##AC ##IP ##18 ) [SEP]',
  'our_bi_cui': 'C2310266',
  'our_bi_cui_name': 'bacterium ST1',
  'our_bi_cui_desc': '[CLS] bacterium ST ##1 [unused3] ( Ba ##cter ##ium ) [SEP]',
  'our_cross_cui': 'C3106801',
  'our_cross_cui_name': 'Blastocystis sp . subtype 1',
  'our_cross_cui_desc': '[CLS] B ##last ##oc ##ys ##tis s ##p . sub ##type 1 [unused3] ( E ##uka ##ryo ##te : B ##last ##oc ##ys ##tis s ##p . ST ##1 ; B ##last ##oc ##ys ##tis s ##p . ST - 1 ) [SEP]'},
 {'mention_id': '27669669.21',
  'test_set_idx': 13726,
  'type': 'T037',
  'name': 'subluxation',
  'context': '[CLS] ##l syndrome , this sub ##lux ##ation occurs spontaneous ##ly after inflammatory processes of the head and neck . Di ##ag ##nosis is typically based on clinical history and a strong suspicion of this syndrome . Non ##su ##rg ##ical treatment most often resolve ##s the symptoms ; however , in some cases surgical treatment is necessary to repair the [unused1] sub ##lux ##ation [unused2] . Various surgical techniques and instrumentation systems have been used to treat at ##lant ##oa ##xial sub ##lux ##ation , although there is no consensus regarding the best treatment method for the p ##ediatric population . To describe a case of at ##lant ##oa ##xial sub ##lux ##ation in a child with G ##rise ##l syndrome treated surgical ##ly with an [SEP]',
  'existing_cui': 'C0332768',
  'existing_cui_name': 'Joint Subluxations',
  'existing_cui_desc': '[CLS] Joint Sub ##lux ##ations [unused3] ( In ##ju ##ry or Po ##ison ##ing , diagnosis , disorder , m ##or ##ph ##olo ##gic abnormal ##ity , physical finding , qualifier value : sub ##lux ##ation ; Sub ##lux ##ation ; Sub ##lux ##ation , Joint ; Di ##s ##location , incomplete ; Joint sub ##lux ##ation ; Joint Sub ##lux ##ation ; Sub ##lux ##ation of joint ; sub ##lux ##ations ; Inc ##om ##ple ##te di ##s ##location ; joint sub ##lux ##ation ; Sub ##lux ##ations ; incomplete di ##s ##location ; Sub ##lux ##ations , Joint ; sub ##lux ##ation was seen ; sub ##lux ##ation of joint ; Sub ##lux ##ation - les ##ion ) [SEP]',
  'our_bi_cui': 'C0332768',
  'our_bi_cui_name': 'Joint Subluxations',
  'our_bi_cui_desc': '[CLS] Joint Sub ##lux ##ations [unused3] ( In ##ju ##ry or Po ##ison ##ing , diagnosis , disorder , m ##or ##ph ##olo ##gic abnormal ##ity , physical finding , qualifier value : sub ##lux ##ation ; Sub ##lux ##ation ; Sub ##lux ##ation , Joint ; Di ##s ##location , incomplete ; Joint sub ##lux ##ation ; Joint Sub ##lux ##ation ; Sub ##lux ##ation of joint ; sub ##lux ##ations ; Inc ##om ##ple ##te di ##s ##location ; joint sub ##lux ##ation ; Sub ##lux ##ations ; incomplete di ##s ##location ; Sub ##lux ##ations , Joint ; sub ##lux ##ation was seen ; sub ##lux ##ation of joint ; Sub ##lux ##ation - les ##ion ) [SEP]',
  'our_cross_cui': 'C0263905',
  'our_cross_cui_name': 'Atlantoaxial subluxation',
  'our_cross_cui_desc': '[CLS] At ##lant ##oa ##xial sub ##lux ##ation [unused3] ( In ##ju ##ry or Po ##ison ##ing , disorder : AA ##S - At ##lant ##oa ##xial sub ##lux ##ation ; at ##lant ##oa ##xial ; sub ##lux ##ation ; di ##s ##location ; at ##lant ##oa ##xial ; At ##lant ##o - a ##xial sub ##lux ##ation ; At ##lant ##oa ##xial di ##s ##location ; at ##lant ##oa ##xial ; di ##s ##location ; Sub ##lux ##ation of at ##lant ##oa ##xial joint ; sub ##lux ##ation ; at ##lant ##oa ##xial ) [SEP]'},
 {'mention_id': '28237762.5',
  'test_set_idx': 29307,
  'type': 'T103',
  'name': 'acylated phenoxyanilide',
  'context': '[CLS] S ##ynth ##esis , SA ##R and molecular dock ##ing study of novel non - β - la ##ct ##am inhibitor ##s of T ##EM type β - la ##ct ##ama ##se The novel classes of [unused1] a ##cy ##lated p ##hen ##ox ##yan ##ili ##de [unused2] and th ##io ##ure ##a compounds were investigated for their ability to in ##hibit T ##EM type β - la ##ct ##ama ##se enzyme . Two compounds 4 ##g and 5 ##c reveal the in ##hibition pot ##ency in micro ##mo ##lar range and show their action by non - co ##valent binding in the vicinity of the T ##EM - 171 active site . The structure activity relationship around carbon chain length and different sub ##st ##it ##uen [SEP]',
  'existing_cui': 'C0031403',
  'existing_cui_name': 'Phenetidine',
  'existing_cui_desc': '[CLS] Ph ##ene ##ti ##dine [unused3] ( Chemical : Ph ##ene ##ti ##dine [ Chemical / In ##g ##red ##ient ] ; Benz ##ena ##mine , 4 - et ##ho ##xy - ; Am ##ino ##phe ##net ##ole ; E ##th ##ox ##yan ##ili ##ne ) [SEP]',
  'our_bi_cui': 'C0029224',
  'our_bi_cui_name': 'Organic Chemicals',
  'our_bi_cui_desc': '[CLS] Organic Chemical ##s [unused3] ( Chemical , substance : organic chemicals ; compound organic ; Organic Chemical ##s [ Chemical / In ##g ##red ##ient ] ; organic compound ; Chemical ##s , Organic ; organic chemical ; Organic Chemical ; chemical organic ; organic compounds ; Organic compound ; Organic compounds ; Organic chemical ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28197133.39',
  'test_set_idx': 28055,
  'type': 'T103',
  'name': 'oligosaccharidases',
  'context': '[CLS] , G ##H1 ##3 ( am ##yla ##se ) , G ##H ##43 and G ##H1 ##0 ( hem ##ice ##ll ##ula ##ses ) , G ##H ##9 and G ##H ##48 ( cell ##ula ##ses ) , P ##L ##11 ( p ##ec ##tina ##se ) as well as G ##H ##2 and G ##H ##3 ( [unused1] o ##li ##gos ##ac ##cha ##rida ##ses [unused2] ) . Our data support the pivotal role of the most characterized fi ##bro ##ly ##tic bacteria ( Pre ##vo ##tel ##la , R ##umi ##no ##cc ##cus and Fi ##bro ##ba ##cter ) , and highlight a substantial , although most probably under ##est ##imated , contribution of fungi and c ##ilia ##te pro ##to ##zo ##a to [SEP]',
  'existing_cui': 'C0028959',
  'existing_cui_name': 'Oligosaccharides',
  'existing_cui_desc': '[CLS] O ##li ##gos ##ac ##cha ##ride ##s [unused3] ( Chemical , substance : o ##li ##gos ##ac ##cha ##ride ##s ; O ##li ##gos ##ac ##cha ##ride ##s [ Chemical / In ##g ##red ##ient ] ; O ##li ##gos ##ac ##cha ##ride ; o ##li ##gos ##ac ##cha ##ride ) [SEP]',
  'our_bi_cui': 'C0012610',
  'our_bi_cui_name': 'Disaccharidases',
  'our_bi_cui_desc': '[CLS] Di ##sa ##cc ##hari ##das ##es [unused3] ( Chemical , substance : Di ##sa ##cc ##hari ##das ##e ; di ##sa ##cc ##hari ##das ##es ; Di ##sa ##cc ##hari ##das ##es [ Chemical / In ##g ##red ##ient ] ; di ##sa ##cc ##hari ##das ##e ) [SEP]',
  'our_cross_cui': 'C0014442',
  'our_cross_cui_name': 'Enzymes',
  'our_cross_cui_desc': '[CLS] En ##zy ##mes [unused3] ( Chemical , medication , substance : enzymes ; enzyme ; En ##zy ##mes [ Chemical / In ##g ##red ##ient ] ; B ##io ##cat ##aly ##sts ; En ##zy ##me ) [SEP]'},
 {'mention_id': '28251683.22',
  'test_set_idx': 30007,
  'type': 'T017',
  'name': 'occlusion',
  'context': '[CLS] ne ##uro ##p ##rote ##ction of remote is ##cha ##em ##ic post ##con ##dition ##ing against cerebral is ##cha ##emia / re ##per ##fusion injury in rats . The middle cerebral artery o ##cc ##lusion model was established in rats and then remote is ##cha ##em ##ic post ##con ##dition ##ing was carried out by three cycles of 10 minutes [unused1] o ##cc ##lusion [unused2] / 10 minutes release of the bilateral f ##em ##oral artery at the beginning of the re ##per ##fusion . To down ##re ##gu ##late the fi ##bul ##in - 5 level , fi ##bul ##in - 5 si ##RNA was injected into the lateral vent ##ric ##le 24 hours before middle cerebral artery o ##cc ##lusion . According to our [SEP]',
  'existing_cui': 'C0011382',
  'existing_cui_name': 'Dental Occlusion',
  'existing_cui_desc': '[CLS] Dental O ##cc ##lusion [unused3] ( B ##iol ##og ##ic Fun ##ction , Den ##tist ##ry , function , o ##bs ##er ##vable entity , physical finding : Dental o ##cc ##lusion ; O ##cc ##lusion ##s , Dental ; O ##CC ##L ##US ##ION DE ##NT ; Dental O ##cc ##lusion ##s ; Bit ##e ; O ##cc ##lusion ; dental o ##cc ##lusion ; tooth o ##cc ##lusion ; o ##cc ##lusion dental ; DE ##NT O ##CC ##L ##US ##ION ; Dental o ##cc ##lusion , function ; O ##cc ##lusion , Dental ) [SEP]',
  'our_bi_cui': 'C0596298',
  'our_bi_cui_name': 'Cerebrovascular Occlusion',
  'our_bi_cui_desc': '[CLS] Ce ##re ##bro ##vas ##cular O ##cc ##lusion [unused3] ( B ##iol ##og ##ic Fun ##ction : c ##ere ##bro ##vas ##cular o ##cc ##lusion ; O ##cc ##lusion ; cerebral ; Ce ##re ##bro ##vas ##cular O ##cc ##lusion ##s ; o ##cc ##lusion ; c ##ere ##bro ##vas ##cular ; O ##cc ##lusion ##s , Ce ##re ##bro ##vas ##cular ; cerebral o ##cc ##lusion ; O ##cc ##lusion , Ce ##re ##bro ##vas ##cular ; c ##ere ##bro ##vas ##cular ; o ##cc ##lusion ; cerebral o ##cc ##lusion ##s ) [SEP]',
  'our_cross_cui': 'C0740391',
  'our_cross_cui_name': 'Middle Cerebral Artery Occlusion',
  'our_cross_cui_desc': '[CLS] Middle Ce ##re ##bra ##l Arte ##ry O ##cc ##lusion [unused3] ( Ana ##tom ##ical Structure , diagnosis , disorder : cerebral artery o ##cc ##lusion middle ; O ##cc ##lusion , Middle Ce ##re ##bra ##l Arte ##ry ; o ##cc ##lusion ; middle cerebral artery ; a . c ##ere ##bri media ; o ##cc ##lusion ; middle cerebral artery o ##cc ##lusion ; Middle cerebral artery o ##cc ##lusion ) [SEP]'},
 {'mention_id': '27442216.26',
  'test_set_idx': 5822,
  'type': 'T098',
  'name': 'All - terrain vehicles group',
  'context': '[CLS] ex ##tre ##mity ( 18 % ) , hand ( 11 % ) , s ##cap ##ula ( 4 . 6 % ) , and open fracture ##s ( 28 . 6 % ) ( P < 0 . 01 ) . Head trauma was the most commonly associated injury in 275 patients with the highest rate in the [unused1] All - terrain vehicles group [unused2] ( 44 % ) who also had the highest rate of no helmet use ( 76 % ) . Snow ##mobile and All - terrain vehicles patients had higher In ##ju ##ry Se ##ver ##ity Score ( 11 . 3 , 9 . 6 ) than dirt ##bi ##ke patients ( 7 . 8 ) ( P = 0 . [SEP]',
  'existing_cui': 'C2898785',
  'existing_cui_name': 'All-terrain motor-vehicle accident',
  'existing_cui_desc': '[CLS] All - terrain motor - vehicle accident [unused3] ( In ##ju ##ry or Po ##ison ##ing ) [SEP]',
  'our_bi_cui': 'C2898785',
  'our_bi_cui_name': 'All-terrain motor-vehicle accident',
  'our_bi_cui_desc': '[CLS] All - terrain motor - vehicle accident [unused3] ( In ##ju ##ry or Po ##ison ##ing ) [SEP]',
  'our_cross_cui': 'C1257890',
  'our_cross_cui_name': 'Population Group',
  'our_cross_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]'},
 {'mention_id': '27826436.24',
  'test_set_idx': 18354,
  'type': 'T204',
  'name': 'cultivars',
  'context': '[CLS] and determined in c ##v . Ben ##ih ##op ##pe , To ##chi ##oto ##me , Sa ##chin ##oka , and G ##ui ##mei ##ren straw ##berry fruits extracted by head - space solid - phase micro ##ex ##traction , respectively . E ##sters significantly dominated the chemical composition of the four varieties . Fu ##rane ##ol was detected in [unused1] cult ##iva ##rs [unused2] of Sa ##chin ##oka and G ##ui ##mei ##ren , but me ##si ##fu ##ran was only found in c ##v . To ##chi ##oto ##me . To ##chi ##oto ##me and Sa ##chin ##oka showed higher content of l ##inal ##ool and ( E ) - ne ##rol ##ido ##l . Sa ##chin ##oka showed the highest content of total [SEP]',
  'existing_cui': 'C3385047',
  'existing_cui_name': 'Brassia hybrid cultivar',
  'existing_cui_desc': '[CLS] Brass ##ia hybrid cult ##iva ##r [unused3] ( E ##uka ##ryo ##te : Ada hybrid cult ##iva ##r ) [SEP]',
  'our_bi_cui': 'C0242775',
  'our_bi_cui_name': 'Agricultural Crops',
  'our_bi_cui_desc': '[CLS] Agricultural C ##rops [unused3] ( E ##uka ##ryo ##te : C ##rops , Agricultural ; C ##rops ; Agricultural C ##rop ) [SEP]',
  'our_cross_cui': 'C0032098',
  'our_cross_cui_name': 'Plants',
  'our_cross_cui_desc': '[CLS] Plants [unused3] ( E ##uka ##ryo ##te , organism : Ch ##lor ##op ##hy ##ta / Em ##b ##ryo ##phy ##ta group ; green plants ; Ch ##lor ##ob ##ion ##ta ; Ch ##lor ##ob ##ion ##ta B ##rem ##er , 1985 ; V ##iri ##di ##p ##lant ##ae ; Plants , General ; Kingdom Plant ##ae ; V ##iri ##di ##p ##lant ##ae C ##ava ##lier - Smith , 1981 ; plants ; Kingdom V ##iri ##di ##p ##lant ##ae ; plant ; ch ##lor ##op ##hy ##te / em ##b ##ryo ##phy ##te group ; Plant ) [SEP]'},
 {'mention_id': '27587730.7',
  'test_set_idx': 10746,
  'type': 'T103',
  'name': 'Gga-miR-21',
  'context': '[CLS] G ##ga - mi ##R - 21 in ##hibit ##s chicken pre - ad ##ip ##oc ##yte proliferation in part by down - regulating K ##rup ##pel - like factor 5 [unused1] G ##ga - mi ##R - 21 [unused2] is abundant ##ly expressed in chicken pre - ad ##ip ##ocytes , but its role is unclear . The present study investigated the role of g ##ga - mi ##R - 21 in chicken pre - ad ##ip ##oc ##yte proliferation . Cell proliferation ass ##ay and gene expression analysis of pro ##life ##rating cell nuclear anti ##gen showed that the g ##ga - mi ##R - 21 mimic in ##hibit ##ed pre - ad ##ip ##oc ##yte proliferation . In contrast , the g ##ga [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1537719',
  'our_bi_cui_name': 'MIR21 gene',
  'our_bi_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '27297967.72',
  'test_set_idx': 1777,
  'type': 'T082',
  'name': 'Asn19/Asn23',
  'context': '[CLS] W ##T - MR ##P ##1 , whereas individual human em ##b ##ryo ##nic kidney 29 ##3 - Ty ##r ##9 ##20 ##P ##he - and - Ser ##9 ##21 ##A ##la - MR ##P ##1 mutant ##s were similar to H ##E ##K - W ##T - MR ##P ##1 . Together , these results suggest that [unused1] As ##n ##19 / As ##n ##23 [unused2] g ##ly ##cos ##yla ##tion and Ty ##r ##9 ##20 / Ser ##9 ##21 p ##hos ##ph ##ory ##lation are responsible for altering the kinetic ##s of MR ##P ##1 - mediated As ( G ##S ) 3 transport . The kinetic ##s of As ( G ##S ) 3 transport by human em ##b ##ryo ##nic kidney 29 [SEP]',
  'existing_cui': 'C3887642',
  'existing_cui_name': 'Hippocampus Proper',
  'existing_cui_desc': "[CLS] Hip ##po ##cam ##pus Pro ##per [unused3] ( Ana ##tom ##ical Structure : Hip ##po ##cam ##pus proper ; Hip ##po ##cam ##pus major ; Am ##mon ' s Horn ; CA fields ; Pro ##per , Hip ##po ##cam ##pus ; Horn , Am ##mon ; Hip ##po ##cam ##pus Pro ##pers ; Am ##mon Horn ; hip ##po ##cam ##pus ; Co ##rn ##u am ##mon ##is ; Am ##mons Horn ; Hip ##po ##cam ##pus ; corn ##u am ##mon ##is ; am ##mon ' s horn ; hip ##po ##cam ##pus proper ; Hip ##po ##cam ##pus prop ##rius ; Am ##mon ' s horn ; Co ##rn ##u Am ##mon ##is ; hip ##po ##cam ##pus major ; Horn , Am [SEP]",
  'our_bi_cui': 'C0003995',
  'our_bi_cui_name': 'Asparagine',
  'our_bi_cui_desc': '[CLS] As ##par ##agi ##ne [unused3] ( Chemical , substance : 2 - Am ##ino ##su ##cci ##nam ##ic acid ; L - as ##par ##agi ##ne ; As ##par ##tam ##ic acid ; alpha - amino ##su ##cci ##nam ##ic acid ; as ##n ; 2 - Am ##ino - 3 - car ##ba ##mo ##yl ##p ##rop ##ano ##ic acid ; as ##par ##agi ##ne ; ( S ) - As ##par ##agi ##ne ; N ; ( S ) - 2 - amino - 3 - car ##ba ##mo ##yl ##p ##rop ##ano ##ic acid ; Alpha - amino ##su ##cci ##nic acid ; As ##n ; L - as ##par ##tic acid beta - amid ##e ; ( 2 ##S ) - 2 [SEP]',
  'our_cross_cui': 'C0002518',
  'our_cross_cui_name': 'Amino Acid Sequence',
  'our_cross_cui_desc': '[CLS] Am ##ino A ##cid Se ##quence [unused3] ( Spa ##tial Concept , finding : Se ##quence ##s , Am ##ino A ##cid ; Am ##ino A ##cid Se ##quence ##s ; SE ##Q AM ##IN ##O AC ##ID ; Pro ##tein Se ##quence ; amino acid sequences ; SE ##Q AA ; Structure ##s , Primary Pro ##tein ; acid amino sequences ; amino acid sequence ; Primary Pro ##tein Structure ##s ; AM ##IN ##O AC ##ID SE ##Q ; Structure , Primary Pro ##tein ; AA SE ##Q ; Am ##ino acid sequence ; protein se ##quencing ; protein sequences ; Pro ##tein Structure ##s , Primary ; Se ##quence , Am ##ino A ##cid ; protein sequence ; AM ##IN ##O AC ##ID [SEP]'},
 {'mention_id': '27430240.85',
  'test_set_idx': 5411,
  'type': 'T103',
  'name': 'factors',
  'context': '[CLS] these results , it can be in ##ferred that ad ##eno ##sin ##e , the ad ##eno ##sin ##e A2 ##A receptor ago ##nist , E ##2 ##F transcription factor 1 and CR ##E ##B are the possible factors contributing to the high expression of CD ##39 and CD ##7 ##3 on the T ##re ##g cell surface during se ##psis . Aden ##os ##ine and its A2 ##A receptor ago ##nist served as the signal trans ##du ##cer [unused1] factors [unused2] of the CD ##39 / CD ##7 ##3 / ad ##eno ##sin ##e pathway , a ##cc ##ele ##rating ad ##eno ##sin ##e generation . Our study may benefit further research on ad ##eno ##sin ##e metabolism for the treatment of se ##psis . [SEP]',
  'existing_cui': 'C0960756',
  'existing_cui_name': 'factor A',
  'existing_cui_desc': "[CLS] factor A [unused3] ( Chemical : Co ( beta ) - c ##yan ##o - 7 ' ' - ( 2 - met ##hyl ) ad ##eni ##ny ##l ##co ##ba ##mi ##de ) [SEP]",
  'our_bi_cui': 'C0021054',
  'our_bi_cui_name': 'Immunologic Factors',
  'our_bi_cui_desc': '[CLS] I ##mm ##uno ##log ##ic Factor ##s [unused3] ( Chemical , substance : Factor ##s , I ##mm ##uno ##log ##ic ; I ##mm ##une Factor ##s ; Factor ##s , I ##mm ##uno ##logical ; I ##mm ##uno ##log ##ic Factor ; Factor ##s , I ##mm ##une ; I ##mm ##uno ##logical Factor ##s ; im ##mu ##no ##log ##ic substance ; FA ##CT ##OR ##S I ##MM ##UN ##OL ; im ##mu ##nological substance ; I ##MM ##UN ##OL FA ##CT ##OR ##S ; I ##mm ##uno ##log ##ic substance ) [SEP]',
  'our_cross_cui': 'C0005515',
  'our_cross_cui_name': 'Biological Factors',
  'our_cross_cui_desc': '[CLS] Biological Factor ##s [unused3] ( Chemical , product , substance : Agents , Biological ; Biological Factor ; B ##iol ##og ##ic agent ; agents biological ; B ##iol ##og ##ic Factor ##s ; B ##iol ##og ##ic Factor ; biological substance ; Factor , B ##iol ##og ##ic ; Factor ##s , Biological ; bio ##pha ##rma ##ce ##utical ##s ; Biological product ; B ##IO ##L FA ##CT ##OR ; Biological Agents ; Biological substance ; Biological Factor ##s [ Chemical / In ##g ##red ##ient ] ; Biological agent ; B ##iol ##og ##ics ; Factor , Biological ; biological drug ; B ##IO ##L FA ##CT ##OR ##S ; FA ##CT ##OR ##S B ##IO ##L ; biological substances ; bio ##pha [SEP]'},
 {'mention_id': '27932585.30',
  'test_set_idx': 22257,
  'type': 'T103',
  'name': 'TYW3',
  'context': '[CLS] ##us , which revealed a novel α / β fold . The sequence motif ( S / T ) x ##SS ##C ##x ##GR and invariant as ##par ##tate and his ##ti ##dine , conserved in T ##Y ##W ##3 / Ta ##w ##3 , cluster to form the cat ##alytic center . These structural and sequence features indicate that [unused1] T ##Y ##W ##3 [unused2] / Ta ##w ##3 proteins constitute a distinct class of SA ##M - dependent met ##hyl ##tra ##ns ##fer ##ases . Using site - directed m ##uta ##genesis along with in v ##ivo complement ##ation ass ##ays combined with mass s ##pect ##rome ##try as well as l ##igan ##d dock ##ing and co ##fa ##ctor binding ass ##ays , [SEP]',
  'existing_cui': 'C1823563',
  'existing_cui_name': 'TYW3 gene',
  'existing_cui_desc': '[CLS] T ##Y ##W ##3 gene [unused3] ( Ana ##tom ##ical Structure : FL ##J ##40 ##9 ##18 ; t ##RNA - Y ##W - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; t ##RNA - y ##W s ##ynth ##esi ##zing protein 3 ho ##mo ##log ; t ##RNA - W ##Y ##B ##UT ##OS ##IN ##E - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ) [SEP]',
  'our_bi_cui': 'C1823563',
  'our_bi_cui_name': 'TYW3 gene',
  'our_bi_cui_desc': '[CLS] T ##Y ##W ##3 gene [unused3] ( Ana ##tom ##ical Structure : FL ##J ##40 ##9 ##18 ; t ##RNA - Y ##W - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; t ##RNA - y ##W s ##ynth ##esi ##zing protein 3 ho ##mo ##log ; t ##RNA - W ##Y ##B ##UT ##OS ##IN ##E - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27587730.11',
  'test_set_idx': 10750,
  'type': 'T103',
  'name': 'gga-miR-21',
  'context': '[CLS] - 21 in ##hibit ##s chicken pre - ad ##ip ##oc ##yte proliferation in part by down - regulating K ##rup ##pel - like factor 5 G ##ga - mi ##R - 21 is abundant ##ly expressed in chicken pre - ad ##ip ##ocytes , but its role is unclear . The present study investigated the role of [unused1] g ##ga - mi ##R - 21 [unused2] in chicken pre - ad ##ip ##oc ##yte proliferation . Cell proliferation ass ##ay and gene expression analysis of pro ##life ##rating cell nuclear anti ##gen showed that the g ##ga - mi ##R - 21 mimic in ##hibit ##ed pre - ad ##ip ##oc ##yte proliferation . In contrast , the g ##ga - mi ##R - 21 [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C2825314',
  'our_bi_cui_name': 'MicroRNA Gene',
  'our_bi_cui_desc': '[CLS] Micro ##RNA Gene [unused3] ( Ana ##tom ##ical Structure : Micro RNA Gene ; Mi ##RNA Gene ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '28375743.9',
  'test_set_idx': 33552,
  'type': 'T103',
  'name': 'structured RNA elements',
  'context': '[CLS] Engineering and In V ##ivo Applications of R ##ib ##os ##witch ##es R ##ib ##os ##witch ##es are common gene regulatory units mostly found in bacteria that are capable of altering gene expression in response to a small molecule . These [unused1] structured RNA elements [unused2] consist of two modular subunit ##s : an a ##pta ##mer domain that binds with high specific ##ity and affinity to a target l ##igan ##d and an expression platform that trans ##du ##ces l ##igan ##d binding to a gene expression output . Sign ##ificant progress has been made in engineering novel a ##pta ##mer domains for new small molecule induce ##rs of gene expression . Mo ##dified expression platforms have also been op ##ti ##mized to function when [SEP]',
  'existing_cui': 'C0003818',
  'existing_cui_name': 'Arsenic',
  'existing_cui_desc': '[CLS] A ##rsen ##ic [unused3] ( Chemical , As , product , substance : AR ##SE ##NI ##C ; a ##rsen ##ics ; As element ; A ##rsen ##ic [ Chemical / In ##g ##red ##ient ] ; As - A ##rsen ##ic ; A ##rsen ##ic product ; a ##rsen ##ic ) [SEP]',
  'our_bi_cui': 'C1138413',
  'our_bi_cui_name': 'Regulatory Sequences , Ribonucleic Acid',
  'our_bi_cui_desc': '[CLS] Reg ##ulatory Se ##quence ##s , R ##ib ##on ##uc ##le ##ic A ##cid [unused3] ( Chemical : Reg ##ulatory RNA Region ##s ; R ##EG R ##IB ##ON ##UC ##LE ##IC AC ##ID R ##EG ##ION ; R ##EG SE ##Q R ##IB ##ON ##UC ##LE ##IC AC ##ID ; RNA Reg ##ulatory Se ##quence ; Reg ##ulatory Se ##quence ##s , R ##ib ##on ##uc ##le ##ic A ##cid [ Chemical / In ##g ##red ##ient ] ; Reg ##ulatory R ##ib ##on ##uc ##le ##ic A ##cid Region ; R ##ib ##on ##uc ##le ##ic A ##cid Reg ##ulatory Se ##quence ##s ; Reg ##ulatory Se ##quence ##s , RNA ; RNA Region ##s , Reg ##ulatory ; RNA Region , Reg ##ulatory [SEP]',
  'our_cross_cui': 'C0035668',
  'our_cross_cui_name': 'RNA',
  'our_cross_cui_desc': '[CLS] RNA [unused3] ( Chemical , RNA , R ##ib ##on ##uc ##le ##ic A ##cid , substance : rib ##on ##uc ##le ##ic acid ; R ##ib ##on ##uc ##le ##ic acids ; RNA , Non - Pol ##ya ##den ##yla ##ted ; rib ##on ##uc ##lein ##ic ##um acid ##um / rib ##on ; RNA - R ##ib ##on ##uc ##le ##ic acid ; RNA [ Chemical / In ##g ##red ##ient ] ; RNA , Non Pol ##ya ##den ##yla ##ted ; Non - Pol ##ya ##den ##yla ##ted RNA ; r ##na ; Non Pol ##ya ##den ##yla ##ted RNA ; A ##cid , R ##ib ##on ##uc ##le ##ic ; R ##ib ##on ##uc ##le ##ic acid ; Gene Products , RNA [SEP]'},
 {'mention_id': '27587730.1',
  'test_set_idx': 10740,
  'type': 'T103',
  'name': 'Gga-miR-21',
  'context': '[CLS] [unused1] G ##ga - mi ##R - 21 [unused2] in ##hibit ##s chicken pre - ad ##ip ##oc ##yte proliferation in part by down - regulating K ##rup ##pel - like factor 5 G ##ga - mi ##R - 21 is abundant ##ly expressed in chicken pre - ad ##ip ##ocytes , but its role is unclear . The present study investigated the role of g ##ga - mi ##R - 21 in chicken pre - ad ##ip ##oc ##yte proliferation . Cell proliferation ass ##ay and gene expression analysis of pro ##life ##rating cell nuclear anti ##gen showed that the g ##ga - mi ##R - 21 mimic in ##hibit ##ed pre - ad ##ip ##oc ##yte proliferation . In contrast , the g ##ga [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1999986',
  'our_bi_cui_name': 'MIRN21 microRNA , human',
  'our_bi_cui_desc': '[CLS] MI ##R ##N ##21 micro ##RNA , human [unused3] ( Chemical : micro ##RNA - 21 , human ; mi ##RNA - 21 , human ; h ##sa - mi ##r - 21 micro ##RNA ; mi ##R - 21 , human ; mi ##R ##21 , human ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '27894019.21',
  'test_set_idx': 21082,
  'type': 'T103',
  'name': 'CYP314a1',
  'context': '[CLS] ##hur ##eni ##c acid is a known end ##ogen ##ous m ##olt regulator present in art ##hr ##op ##ods . Experimental studies have confirmed that X ##ant ##hur ##eni ##c acid in ##hibit ##s m ##olt ##ing by binding to either ( or both ) of two P ##45 ##0 enzymes ( C ##YP ##31 ##5 ##a ##1 or [unused1] C ##YP ##31 ##4 ##a ##1 [unused2] ) that are responsible for the final two h ##ydro ##xy ##lation ##s in the production of the m ##olt - in ##ducing hormone , 20 - h ##ydro ##xy ##ec ##dy ##son ##e . The lack of crystal structures and bio ##chemical ass ##ays for C ##YP ##31 ##5 ##a ##1 or C ##YP ##31 ##4 ##a ##1 [SEP]',
  'existing_cui': 'C1418799',
  'existing_cui_name': 'PPIF gene',
  'existing_cui_desc': '[CLS] PP ##IF gene [unused3] ( Ana ##tom ##ical Structure : C ##Y ##CL ##OP ##H ##IL ##IN 3 ; P ##EP ##TI ##D ##Y ##L - PR ##OL ##Y ##L C ##IS / T ##RA ##NS ISO ##ME ##RA ##SE , MIT ##OC ##H ##ON ##DR ##IA ##L ; P ##ept ##idy ##l ##p ##rol ##yl Is ##omer ##ase F Gene ; PP ##IF Gene ; C ##Y ##CL ##OP ##H ##IL ##IN F ; p ##ept ##idy ##l ##p ##rol ##yl is ##omer ##ase F ; Cy ##p - D ; h ##C ##y ##P ##3 ; c ##y ##c ##lop ##hil ##in D ) [SEP]',
  'our_bi_cui': 'C0051169',
  'our_bi_cui_name': 'Alkane 1-monooxygenase',
  'our_bi_cui_desc': '[CLS] Al ##kan ##e 1 - mon ##oo ##xy ##gen ##ase [unused3] ( Chemical , substance : P - 450 4 ##A , Cy ##to ##ch ##rome ; Al ##kan ##e 1 H ##ydro ##xy ##lase ; Cy ##to ##ch ##rome P 450 IV ##A ; A ##cid Omega - h ##ydro ##xy ##lase , Lau ##ric ; Al ##kan ##e H ##ydro ##xy ##lase ; Omega - H ##ydro ##xy ##lase , Fat ##ty A ##cid ; Lau ##ric A ##cid H ##ydro ##xy ##lase ; Al ##kan ##e 1 Mon ##oo ##xy ##gen ##ase ; Cy ##to ##ch ##rome P - 450 C ##YP ##4 ##A ; Omega - Lau ##ryl H ##ydro ##xy ##lase ; Omega - h ##ydro ##xy ##lase , Lau ##ric [SEP]',
  'our_cross_cui': 'C0010762',
  'our_cross_cui_name': 'Cytochrome P450',
  'our_cross_cui_desc': '[CLS] Cy ##to ##ch ##rome P ##45 ##0 [unused3] ( Chemical , substance : c ##yt ##och ##rome ##s p ##45 ##0 ; c ##yt ##och ##rome P ##45 ##0 ; C ##Y ##TO ##CH ##RO ##ME P 450 E ##N ##Z ##Y ##ME S ##Y ##ST ##EM ; Cy ##to ##ch ##rome p ##45 ##0 enzyme ; Super ##family , Cy ##to ##ch ##rome P ##45 ##0 ; P ##45 ##0 En ##zy ##mes ; c ##yt ##och ##rome p 450 ; En ##zy ##mes , P ##45 ##0 ; Cy ##to ##ch ##rome P 450 Families ; Cy ##to ##ch ##rome P ##45 ##0 En ##zy ##me ; P ##45 ##0 ; C ##YP ##45 ##0 ; Cy ##to ##ch ##rome p ##45 ##0 ; [SEP]'},
 {'mention_id': '27506129.31',
  'test_set_idx': 8663,
  'type': 'T170',
  'name': 'oversampling approaches',
  'context': '[CLS] the SM ##OT ##E approach for both scales ( mean square error < 0 . 00 ##6 ) . Machine learning methods are able to perform an objective assessment of h ##yper ##emia g ##rading , removing both in ##tra - and inter - expert subject ##ivity while providing a gain in com ##putation time . SM ##OT ##E and [unused1] overs ##amp ##ling approaches [unused2] mini ##mise the class im ##bal ##ance problem , while feature selection reduces the number of features from 25 to 3 - 5 without worse ##ning the mean square error . As the differences between the system and a human expert are similar to the differences between experts , we can therefore conclude that the system behave ##s like an [SEP]',
  'existing_cui': 'C0449445',
  'existing_cui_name': 'Approach',
  'existing_cui_desc': '[CLS] A ##pp ##roach [unused3] ( Spa ##tial Concept , attribute , qualifier value : A ##pp ##roach ##es ; approach ; approaches ; approaching ) [SEP]',
  'our_bi_cui': 'C0430022',
  'our_bi_cui_name': 'diagnostic procedure',
  'our_bi_cui_desc': '[CLS] diagnostic procedure [unused3] ( Health Care Act ##ivity , procedure : D ##IA ##G ##N ##OS ##TI ##C ; Di ##ag ##nos ##tic procedure ; Di ##ag ##nos ##tic procedures ; Di ##ag ##nos ##tic Test ; diagnostic technique ; Di ##ag ##nos ##tic ; Di ##ag ##nos ##tic Tech ##ni ##que ; Di ##ag ##nos ##tic Pro ##ced ##ure ; diagnostic procedures ; Di ##ag ##nos ##tic Method ; Pro ##ced ##ures ; diagnostic ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27388786.19',
  'test_set_idx': 4442,
  'type': 'T038',
  'name': 'production',
  'context': "[CLS] - Si ##gu ##t , Da ##ws , et al . , 2015 ) we found stronger left lateral ##ization for sign than speech . Given that this increased lateral ##ization could not be explained by hand movement alone , the contribution of motor movement versus ' linguistic ' processes to the strength of hem ##is ##pheric lateral ##ization during sign [unused1] production [unused2] remains unclear . Here we directly contrast lateral ##ization strength of covert versus over ##t signing during p ##hon ##ological and semantic flu ##ency tasks . To address the possibility that hearing native sign ##ers ' elevated lateral ##ization in ##dices were due to performing a task in their less dominant language , here we test deaf native sign ##ers , whose [SEP]",
  'existing_cui': 'C1548180',
  'existing_cui_name': 'Production Processing ID',
  'existing_cui_desc': '[CLS] Production Process ##ing ID [unused3] ( Intel ##lect ##ual Product : Production ) [SEP]',
  'our_bi_cui': 'C0220781',
  'our_bi_cui_name': 'Anabolism',
  'our_bi_cui_desc': '[CLS] Ana ##bol ##ism [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : bio ##sy ##nt ##hetic process ; Ana ##bol ##ism , function ; synthesis ; bio ##sy ##nt ##hesis ; formation ; B ##ios ##ynth ##etic ; B ##ios ##ynth ##esis ; B ##io ##genesis ; B ##ios ##ynth ##etic Process ; an ##ab ##olis ##m ) [SEP]',
  'our_cross_cui': 'C0025361',
  'our_cross_cui_name': 'Mental Processes',
  'our_cross_cui_desc': '[CLS] Mental Process ##es [unused3] ( B ##iol ##og ##ic Fun ##ction , Human , function , o ##bs ##er ##vable entity : Human Information Process ##ing ; Thought process ; cognitive processing ; processes thought ; processing thought ; Information Process ##ing , Human ; Human information processing ; Information Process ##es ; mental processing ; cognitive process ; Form of thinking , function ; mental processes ; Form of thought ; Form of thinking ; Human Information Process ##es ; Mental Process ; mental process ; thought process ) [SEP]'},
 {'mention_id': '27500205.3',
  'test_set_idx': 8435,
  'type': 'T103',
  'name': 'Kindlins',
  'context': '[CLS] Of Kind ##lins and Cancer [unused1] Kind ##lins [unused2] are 4 . 1 - e ##z ##rin - rid ##ix ##in - m ##oes ##in ( F ##ER ##M ) domain containing proteins . There are three kind ##lins in mammals , which share high sequence identity . Kind ##lin - 1 is expressed primarily in e ##pit ##hel ##ial cells , kind ##lin - 2 is widely distributed and is particularly abundant in ad ##here ##nt cells , and kind ##lin - 3 is expressed primarily in hem ##ato ##po ##iet ##ic cells . These distributions are not exclusive ; some cells express multiple kind ##lins , and transformed cells often exhibit a ##ber ##rant expression , both in the is ##of ##orms and the [SEP]',
  'existing_cui': 'C1720776',
  'existing_cui_name': 'Kindling , Neurologic',
  'existing_cui_desc': '[CLS] Kind ##ling , N ##eur ##olo ##gic [unused3] ( B ##iol ##og ##ic Fun ##ction , N ##eur ##ology : kind ##ling ; Kind ##lings , N ##eur ##olo ##gic ; Kind ##ling ; N ##eur ##olo ##gic Kind ##lings ; N ##eur ##olo ##gic Kind ##ling ) [SEP]',
  'our_bi_cui': 'C0010422',
  'our_bi_cui_name': 'Crystallins',
  'our_bi_cui_desc': '[CLS] Crystal ##lins [unused3] ( Chemical , substance : Crystal ##lins [ Chemical / In ##g ##red ##ient ] ; Crystal ##lin ; Pro ##tein ##s , Len ##s ; C ##rist ##all ##in ; Len ##s Pro ##tein ##s ; crystal ##lin ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27842161.1',
  'test_set_idx': 19094,
  'type': 'T103',
  'name': 'Netarsudil',
  'context': '[CLS] [unused1] Net ##ars ##udi ##l [unused2] Inc ##rease ##s Out ##flow Facility in Human Eyes Through Multiple Me ##chan ##isms Net ##ars ##udi ##l is a R ##ho kinase / nor ##ep ##ine ##ph ##rine transport ##er inhibitor currently in phase 3 clinical development for g ##lau ##com ##a treatment . We investigated the effects of its active meta ##bol ##ite , net ##ars ##udi ##l - M1 , on out ##flow facility ( C ) , out ##flow h ##ydro ##dynamic ##s , and morphology of the conventional out ##flow pathway in en ##uc ##lea ##ted human eyes . Pa ##ired human eyes ( n = 5 ) were per ##fused with either 0 . 3 μ ##M net ##ars ##udi ##l - M1 or [SEP]',
  'existing_cui': 'C0013443',
  'existing_cui_name': 'Ear structure',
  'existing_cui_desc': '[CLS] E ##ar structure [unused3] ( Ana ##tom ##ical Structure , Anatomy , body structure : V ##est ##ib ##ulo ##co ##ch ##lea ##r System ; System , V ##est ##ib ##ulo ##co ##ch ##lea ##r ; ear structures ; Organ ##um vest ##ib ##ulo ##co ##ch ##lea ##re ; ear structure ; E ##ar - related structure ; au ##ris ; A ##ppa ##rat ##us , V ##est ##ib ##ulo ##co ##ch ##lea ##r ; E ##ar and related structures ; Au ##ris ; V ##est ##ib ##ulo ##co ##ch ##lea ##r organ ; V ##est ##ib ##ulo ##co ##ch ##lea ##r A ##ppa ##rat ##us ) [SEP]',
  'our_bi_cui': 'C0027783',
  'our_bi_cui_name': 'Netropsin',
  'our_bi_cui_desc': '[CLS] Net ##rops ##in [unused3] ( Chemical : Sin ##ano ##my ##cin ; Net ##rops ##in [ Chemical / In ##g ##red ##ient ] ; net ##rops ##in ; Congo ##cid ##ine ; 1 ##H - P ##yr ##rol ##e - 2 - car ##box ##ami ##de , 4 - ( ( ( ( amino ##im ##ino ##met ##hyl ) amino ) ace ##ty ##l ) amino ) - N - ( 5 - ( ( ( 3 - amino - 3 - im ##ino ##p ##rop ##yl ) amino ) carbon ##yl ) - 1 - met ##hyl - 1 ##H - p ##yr ##rol - 3 - y ##l ) - 1 - met ##hyl - ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27457539.37',
  'test_set_idx': 6436,
  'type': 'T038',
  'name': 'pregnant',
  'context': '[CLS] in this species . In this study , the safety and efficacy ( through a neutral ##izing anti ##body response ) of the the ##rm ##ost ##able live at ##ten ##uated R ##V ##F C ##L ##13 ##T vaccine were evaluated in came ##ls in two different preliminary experiments involving 16 came ##ls , ( that 12 came ##ls and 4 [unused1] pregnant [unused2] came ##ls ) . The study revealed that the C ##L ##13 ##T vaccine was safe to use in came ##ls and no abortion ##s or te ##rato ##genic effects were observed . The single dose of the vaccine stimulate ##d a strong and long - lasting neutral ##izing anti ##body response for up to 12 months . The presence of neutral [SEP]',
  'existing_cui': 'C0549206',
  'existing_cui_name': 'Patient currently pregnant',
  'existing_cui_desc': '[CLS] Pat ##ient currently pregnant [unused3] ( Finding , finding , history : Pre ##gnant ; currently pregnant ; Currently pregnant ; Pre ##gna ##ncy not delivered ; G ##ra ##vid ##ity ; G ##ra ##vid ; Pat ##ient pregnant ; pregnant patient ; PR ##EG ##NA ##NT ; pregnant ; g ##ra ##vid ) [SEP]',
  'our_bi_cui': 'C0549206',
  'our_bi_cui_name': 'Patient currently pregnant',
  'our_bi_cui_desc': '[CLS] Pat ##ient currently pregnant [unused3] ( Finding , finding , history : Pre ##gnant ; currently pregnant ; Currently pregnant ; Pre ##gna ##ncy not delivered ; G ##ra ##vid ##ity ; G ##ra ##vid ; Pat ##ient pregnant ; pregnant patient ; PR ##EG ##NA ##NT ; pregnant ; g ##ra ##vid ) [SEP]',
  'our_cross_cui': 'C0032961',
  'our_cross_cui_name': 'Pregnancy',
  'our_cross_cui_desc': '[CLS] Pre ##gna ##ncy [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , o ##bs ##er ##vable entity , qualifier value , s ##ym ##pt ##om : G ##esta ##tion ; Pre ##gna ##ncies ; PR ##EG ##N ; g ##esta ##tion ; PR ##EG ##NA ##NC ##Y ; pre ##gna ##ncies ; female pregnancy ; Pre ##gna ##ncy , function ; pregnancy ; Pre ##gna ##ncy [ Disease / Finding ] ) [SEP]'},
 {'mention_id': '27509301.56',
  'test_set_idx': 8838,
  'type': 'T033',
  'name': 'detection',
  'context': '[CLS] 39 out of 46 est ##rogen ##ic meta ##bol ##ites were predicted as potential bio ##tra ##ns ##formation products derived from the parent chemical . The Q ##SA ##R models estimated stronger est ##rogen ##ic activity for the majority of the known est ##rogen ##ic meta ##bol ##ites compared to their parent chemicals . Finally , the three models identified a similar set of parent compounds as top ranked chemicals based on the est ##rogen ##icit ##y of put ##ative meta ##bol ##ites . This proposed in si ##lic ##o approach is an inexpensive and rapid strategy for the [unused1] detection [unused2] of chemicals with est ##rogen ##ic meta ##bol ##ites and may reduce potential false negative results from in v ##it ##ro ass ##ays . [SEP]',
  'existing_cui': 'C0206100',
  'existing_cui_name': 'Signal Detection',
  'existing_cui_desc': '[CLS] Signal Det ##ec ##tion [unused3] ( B ##iol ##og ##ic Fun ##ction , Psychology , Psychology , Signal : Signal Det ##ec ##tion Theo ##ries ; Signal Det ##ec ##tions , Psychological ; Theory , Signal Det ##ec ##tion ; Signal detection ; S ##IG ##NA ##L DE ##TE ##CT ##ION AN ##AL ; Psychological Signal Det ##ec ##tions ; Det ##ec ##tion ; Signal Det ##ec ##tion Ana ##ly ##ses ; S ##IG ##NA ##L DE ##TE ##CT ##ION ; Det ##ec ##tions , Psychological Signal ; Psychological Signal Det ##ec ##tion ; Signal Det ##ec ##tion Theory ; Theo ##ries , Signal Det ##ec ##tion ; Signal Det ##ec ##tions ; Ana ##ly ##ses , Signal Det ##ec ##tion ; Det ##ec ##tion , [SEP]',
  'our_bi_cui': 'C1511790',
  'our_bi_cui_name': 'Detection',
  'our_bi_cui_desc': '[CLS] Det ##ec ##tion [unused3] ( Health Care Act ##ivity : Det ##ec ##ted ) [SEP]',
  'our_cross_cui': 'C0442726',
  'our_cross_cui_name': 'Detected',
  'our_cross_cui_desc': '[CLS] Det ##ec ##ted [unused3] ( Finding , finding , finding , qualifier value : detected ; detect ) [SEP]'},
 {'mention_id': '27794068.45',
  'test_set_idx': 17122,
  'type': 'T038',
  'name': 'binding of',
  'context': '[CLS] binding protein expression levels . These results reveal the important role of C ##4 ##b binding protein in negative regulation of T ##LR ##1 / 2 - dependent pro - inflammatory c ##yt ##oki ##ne production . Furthermore , using a fluorescent con ##ju ##gated Pam ##3 ##CS ##K ##4 , we show that C ##4 ##b binding protein blocks the [unused1] binding of [unused2] Pam ##3 ##CS ##K ##4 to T ##LR ##1 / 2 . Finally , we show that ex ##ogen ##ous C ##4 ##b binding protein also in ##hibit ##s Pam ##3 ##CS ##K ##4 - induced signaling leading to IL - 8 production . Our results indicate C ##4 ##b binding protein binding to T ##LR ##2 and con ##se ##quent [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0033618',
  'our_bi_cui_name': 'Protein Binding',
  'our_bi_cui_desc': '[CLS] Pro ##tein Bin ##ding [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : Pro ##tein binding ( biological function ) [ Am ##bi ##guous ] ; Bin ##ding , Pro ##tein ; Pro ##tein binding , function ; Pro ##tein binding ; Pro ##tein binding ( biological function ) ; protein binding ; protein amino acid binding ) [SEP]',
  'our_cross_cui': 'C1167622',
  'our_cross_cui_name': 'Binding',
  'our_cross_cui_desc': '[CLS] Bin ##ding [unused3] ( B ##iol ##og ##ic Fun ##ction , Molecular Fun ##ction , Molecular Fun ##ction : binds ; Physical Inter ##action ; binding [ molecular function ] ; Molecular Inter ##action Process ; Molecular Inter ##action ; binding ; Bo ##und ; bind ) [SEP]'},
 {'mention_id': '27280034.11',
  'test_set_idx': 1340,
  'type': 'T082',
  'name': 'Newcastle upon Tyne',
  'context': "[CLS] , 1961 - 2009 Ai ##ms and method To as ##cer ##tain differences in patterns of suicide in young men over three decades ( 1960s , 1990s and 2000s ) and discuss implications for suicide prevention . Data on suicide ##s and open verdict ##s in men aged 15 - 34 were obtained from co ##roner ' s records in [unused1] Newcastle upon Tyne [unused2] and anal ##ys ##ed using SP ##SS software . Results An increase in suicide rates from the first to the second decade was followed by a fall in the third decade . This was associated with an increasing proportion of single men , those living alone , unemployment , consumption of alcohol , use of hanging , previous suicide attempt and [SEP]",
  'existing_cui': 'C0027983',
  'existing_cui_name': 'Newcastle Disease',
  'existing_cui_desc': "[CLS] Newcastle Disease [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , et ##iology , manifest ##ation : Disease , Newcastle ; new ##castle disease ; Newcastle ' s disease ; A ##vian p ##ne ##um ##oe ##nce ##pha ##lit ##is ; Newcastle disease [ Am ##bi ##guous ] ; P ##se ##udo - f ##ow ##l ##pes ##t ; disease ( or disorder ) ; Newcastle ; Newcastle disease ; Newcastle ; NE ##WC ##AS ##TL ##E D ##IS ; Newcastle fever ; Newcastle Disease [ Disease / Finding ] ) [SEP]",
  'our_bi_cui': 'C0454880',
  'our_bi_cui_name': 'Tyne and Wear',
  'our_bi_cui_desc': '[CLS] Tyne and We ##ar [unused3] ( Spa ##tial Concept , geographic location : T and W - Tyne and We ##ar ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '28223077.34',
  'test_set_idx': 28918,
  'type': 'T082',
  'name': 'surrounding sites',
  'context': '[CLS] 00 ##4 , 2 . 05 ##8 ± 0 . 45 ##8 , 2 . 61 ##7 ± 1 . 09 ##2 and 1 . 82 ##2 ± 0 . 56 ##6 p ##g ##N ##m ( - 3 ) , respectively . ( 2 ) The toxic equivalent concentrations of PC ##D ##D / F ##s in ambient air from [unused1] surrounding sites [unused2] , background site , up ##wind site and down ##wind site were 0 . 103 ± 0 . 01 ##7 , 0 . 09 ##6 ± 0 . 01 ##5 , 0 . 120 ± 0 . 02 ##4 and 0 . 108 ± 0 . 01 ##4 p ##g I - T ##E ##Q ##N ##m ( - 3 ) [SEP]',
  'existing_cui': 'C1282914',
  'existing_cui_name': 'Surrounding',
  'existing_cui_desc': '[CLS] Sur ##rou ##nding [unused3] ( Spa ##tial Concept , qualifier value , qualifier value : C ##ir ##cum ##scribed ; Sur ##rou ##nd ; c ##ir ##cum ##scribed ; surrounding ; surrounded ; Sur ##rou ##nded ; surroundings ; surrounds ; surround ) [SEP]',
  'our_bi_cui': 'C1282914',
  'our_bi_cui_name': 'Surrounding',
  'our_bi_cui_desc': '[CLS] Sur ##rou ##nding [unused3] ( Spa ##tial Concept , qualifier value , qualifier value : C ##ir ##cum ##scribed ; Sur ##rou ##nd ; c ##ir ##cum ##scribed ; surrounding ; surrounded ; Sur ##rou ##nded ; surroundings ; surrounds ; surround ) [SEP]',
  'our_cross_cui': 'C0205145',
  'our_cross_cui_name': 'Site',
  'our_cross_cui_desc': '[CLS] Site [unused3] ( Spa ##tial Concept , attribute , qualifier value : Sites ) [SEP]'},
 {'mention_id': '27298278.41',
  'test_set_idx': 1842,
  'type': 'T017',
  'name': 'Ketr3',
  'context': "[CLS] ' - di ##car ##box ##yla ##te , respectively . Na ##ph ##th ##o ##quin ##ones and nap ##ht ##ho ##hy ##dr ##o ##quin ##one dim ##ers were previously unknown in the genus Mo ##rin ##da . In addition , the compounds were tested for c ##yt ##oto ##xi ##city against four human cancer cell lines He ##L ##a , A2 ##7 ##80 , [unused1] Ke ##tr ##3 [unused2] and MC ##F - 7 and their effects on p ##53 - activated transcription . Three nap ##ht ##ho ##quin ##ones had moderate c ##yt ##oto ##xi ##c effects with I ##C ##50 values ranging from 1 . 51 to 9 . 56 μ ##M , through up - regulation of p ##53 transcription ##al activity . [SEP]",
  'existing_cui': 'C3539712',
  'existing_cui_name': 'CELF2 wt Allele',
  'existing_cui_desc': '[CLS] CE ##LF ##2 w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : N ##AP ##OR ; ET ##R - 3 ; BR ##UN ##OL ##3 ; N ##AP ##OR - 2 ; C ##U ##GB ##P ##2 ; C ##U ##GB ##P , El ##av - Like Family Member 2 w ##t All ##ele ; ET ##R ##3 ) [SEP]',
  'our_bi_cui': 'C0007634',
  'our_bi_cui_name': 'Cells',
  'our_bi_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]',
  'our_cross_cui': 'C0085983',
  'our_cross_cui_name': 'Cell Line , Tumor',
  'our_cross_cui_desc': '[CLS] Cell Line , Tu ##mor [unused3] ( Ana ##tom ##ical Structure : Cell Lines , Tu ##mor ; Tu ##mor Cell Line ; Line , Tu ##mor Cell ; Lines , Tu ##mor Cell ; Tu ##mor Cell Lines ; tumor cell lines ) [SEP]'},
 {'mention_id': '27826436.48',
  'test_set_idx': 18378,
  'type': 'T204',
  'name': 'cultivars',
  'context': '[CLS] ##oka , followed by the G ##ui ##mei ##ren and To ##chi ##oto ##me varieties . Sa ##chin ##oka had the highest t ##it ##rata ##ble acid ##ity T ##A value . The content of as ##cor ##bic acid of c ##v . To ##chi ##oto ##me was higher than the others , but there was no significant difference in [unused1] cult ##iva ##rs [unused2] of Ben ##ih ##op ##pe , To ##chi ##oto ##me , and Sa ##chin ##oka . Fr ##uc ##tos ##e and glucose were the major sugar ##s in all cult ##iva ##rs . C ##it ##ric acid was the major organic acid in c ##v . To ##chi ##oto ##me , c ##v . Sa ##chin ##oka , and c ##v [SEP]',
  'existing_cui': 'C3385047',
  'existing_cui_name': 'Brassia hybrid cultivar',
  'existing_cui_desc': '[CLS] Brass ##ia hybrid cult ##iva ##r [unused3] ( E ##uka ##ryo ##te : Ada hybrid cult ##iva ##r ) [SEP]',
  'our_bi_cui': 'C0242775',
  'our_bi_cui_name': 'Agricultural Crops',
  'our_bi_cui_desc': '[CLS] Agricultural C ##rops [unused3] ( E ##uka ##ryo ##te : C ##rops , Agricultural ; C ##rops ; Agricultural C ##rop ) [SEP]',
  'our_cross_cui': 'C0032098',
  'our_cross_cui_name': 'Plants',
  'our_cross_cui_desc': '[CLS] Plants [unused3] ( E ##uka ##ryo ##te , organism : Ch ##lor ##op ##hy ##ta / Em ##b ##ryo ##phy ##ta group ; green plants ; Ch ##lor ##ob ##ion ##ta ; Ch ##lor ##ob ##ion ##ta B ##rem ##er , 1985 ; V ##iri ##di ##p ##lant ##ae ; Plants , General ; Kingdom Plant ##ae ; V ##iri ##di ##p ##lant ##ae C ##ava ##lier - Smith , 1981 ; plants ; Kingdom V ##iri ##di ##p ##lant ##ae ; plant ; ch ##lor ##op ##hy ##te / em ##b ##ryo ##phy ##te group ; Plant ) [SEP]'},
 {'mention_id': '28421738.3',
  'test_set_idx': 35181,
  'type': 'T038',
  'name': 'Amyloid - Beta Liaison',
  'context': "[CLS] Target ##ing the Nr ##f ##2 / [unused1] Amy ##lo ##id - Beta Lia ##ison [unused2] in Alzheimer ' s Disease : A Rat ##ional A ##pp ##roach Amy ##lo ##id is a prominent feature of Alzheimer ' s disease . Yet , a linear link ##age between am ##yl ##oid - β p ##eptide and the disease onset and progression has recently been questioned . In this context , the crucial partnership between am ##yl ##oid - β p ##eptide and Nr ##f ##2 pathways is acquiring para ##mount importance , offering prospects for de ##ci ##pher ##ing the am ##yl ##oid - β p ##eptide - centered disease network . Here , we report on a new class of anti ##ag ##g ##re ##gating [SEP]",
  'existing_cui': 'C2825879',
  'existing_cui_name': 'Beta Amyloid Measurement',
  'existing_cui_desc': '[CLS] Beta Amy ##lo ##id Me ##as ##ure ##ment [unused3] ( Health Care Act ##ivity : Beta Amy ##lo ##id ; AM ##Y ##L ##O ##ID ##B ; Amy ##lo ##id , Beta ) [SEP]',
  'our_bi_cui': 'C1510880',
  'our_bi_cui_name': 'Amyloid Beta-Peptide Pathway',
  'our_bi_cui_desc': '[CLS] Amy ##lo ##id Beta - P ##eptide Path ##way [unused3] ( B ##iol ##og ##ic Fun ##ction : Generation of am ##yl ##oid b - p ##eptide by PS ##1 ) [SEP]',
  'our_cross_cui': 'C1149161',
  'our_cross_cui_name': 'beta-amyloid binding',
  'our_cross_cui_desc': '[CLS] beta - am ##yl ##oid binding [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]'},
 {'mention_id': '28269453.17',
  'test_set_idx': 30622,
  'type': 'T170',
  'name': 'fitness planner',
  'context': '[CLS] End ##urance based personal ##ized fitness plan ##ner End ##urance is an important factor of card ##iovascular fitness indicating the capacity of an individual to perform exercise for a longer duration with increased intensity . Various subject specific and exercise related parameters affect endurance of an individual . In this work , we propose a statistical technique to model endurance as a function of these factors incorporating the serial dependence of observations generated by individuals over time . The proposed model provides a device to predict future endurance of a test subject following particular exercise regime . This facilitate ##s a test user with a [unused1] fitness plan ##ner [unused2] with the provision to fix exercise regime ##s to reach a set fitness goal . [SEP]',
  'existing_cui': 'C0600623',
  'existing_cui_name': 'Fitness Centers',
  'existing_cui_desc': '[CLS] Fitness Centers [unused3] ( Organization : fitness center ; center fitness ; centers fitness ; Fitness Center ; Center , Fitness ; Centers , Fitness ; fitness centers ) [SEP]',
  'our_bi_cui': 'C1690589',
  'our_bi_cui_name': 'Verbalizes plan to support exercise',
  'our_bi_cui_desc': '[CLS] V ##er ##bal ##izes plan to support exercise [unused3] ( Finding , finding : Des ##cribe ##s plan to support exercise ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27540584.34',
  'test_set_idx': 9935,
  'type': 'T097',
  'name': 'endoscopists',
  'context': '[CLS] course . There were no technical problems related to the novel si ##mu ##lator during the course . After s ##phi ##nc ##tero ##tom ##y , the organic p ##ap ##illa ##e could easily be exchanged within a few seconds . Overall , the novel Bill ##roth II / R ##oux - en - Y si ##mu ##lator achieved favorable results by train ##ees and expert [unused1] end ##os ##co ##pis ##ts [unused2] in all categories assessed . The new Bill ##roth II / R ##oux - en - Y mechanical si ##mu ##lator is simple and p ##rac ##tica ##ble . A first evaluation during an end ##os ##copic re ##tro ##grade ch ##ola ##ng ##io ##pan ##cre ##ato ##graphy course showed promising results . [SEP]',
  'existing_cui': 'C0228237',
  'existing_cui_name': 'Structure of superior temporal sulcus',
  'existing_cui_desc': '[CLS] Structure of superior temporal su ##l ##cus [unused3] ( Spa ##tial Concept , body structure : Superior temporal fi ##ss ##ure ; Superior Te ##mpo ##ral Sul ##cus ; parallel su ##l ##cus ; Sul ##cus temporal ##is superior ; Superior temporal su ##l ##cus ; superior temporal su ##l ##cus ; Sul ##cus temporal ##is p ##rim ##us ; Sul ##cus t ##1 ; su ##l ##cus t ##1 ; superior temporal fi ##ss ##ure ; Para ##lle ##l su ##l ##cus ) [SEP]',
  'our_bi_cui': 'C0582175',
  'our_bi_cui_name': 'Surgeon',
  'our_bi_cui_desc': '[CLS] Sur ##geon [unused3] ( Professional or O ##cc ##upation ##al Group , occupation : surgeon ##s ; surgeon ; Sur ##geons ) [SEP]',
  'our_cross_cui': 'C1522486',
  'our_cross_cui_name': 'Professional Organization or Group',
  'our_cross_cui_desc': '[CLS] Professional Organization or Group [unused3] ( Professional or O ##cc ##upation ##al Group : Other Agency or Organization ; Organization ; OR ##G ; Outside Bo ##dies ; Organization ##al ) [SEP]'},
 {'mention_id': '27570140.33',
  'test_set_idx': 10496,
  'type': 'T058',
  'name': 'procedure',
  'context': '[CLS] 3 . 2 % , p < . 00 ##1 ) and those with an un ##int ##ended index pregnancy ( 13 . 8 % vs . 4 . 1 % , p < . 00 ##1 ) . Of the patients requesting post ##par ##tum tub ##al l ##iga ##tion , 118 ( 51 . 1 % ) underwent the [unused1] procedure [unused2] immediately post ##par ##tum . Co ##mple ##tion was not associated with high - risk status ( 54 . 0 % ) , or with race , insurance status or par ##ity . Among 113 women with an un ##ful ##fill ##ed post ##par ##tum tub ##al l ##iga ##tion request , there were 17 subsequent pre ##gna ##ncies ( 15 . [SEP]',
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27796974.34',
  'test_set_idx': 17388,
  'type': 'T082',
  'name': 'Balatonfüred City',
  'context': '[CLS] been defined for every water chemistry parameter in light of the legal limit values of the water parameters . In addition to this , weight in ##dices were calculated on the basis of the outcome of the paired comparison of water chemistry parameters and normal ##ized matrix . This was followed by the para ##metric level analysis of the water chemistry parameters , and finally , the aquatic environment index was calculated , which provided general information on the quality of water regarding the water chemistry parameters . The method was illustrated on Lake Ba ##lat ##on , Hungary in which case water samples taken from [unused1] Ba ##lat ##on ##f ##ü ##red City [unused2] lake area were analyzed and evaluated with the method devised . [SEP]',
  'existing_cui': 'C0008848',
  'existing_cui_name': 'Cities',
  'existing_cui_desc': '[CLS] Cities [unused3] ( Spa ##tial Concept , environment : City ; city ; cities ) [SEP]',
  'our_bi_cui': 'C0008848',
  'our_bi_cui_name': 'Cities',
  'our_bi_cui_desc': '[CLS] Cities [unused3] ( Spa ##tial Concept , environment : City ; city ; cities ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '28099370.14',
  'test_set_idx': 25628,
  'type': 'T170',
  'name': 'written',
  'context': '[CLS] A Course - Based A ##pp ##roach to the Doctor of Nursing Practice Project : Supporting Student Growth From Concept to Co ##mple ##tion We describe a course - based approach to the doctor of nursing practice project in which students work in groups of 8 to 12 with a faculty member to complete individual final projects that require a minimum of 360 p ##rac ##tic ##um hours in 3 - semester - long courses . Project teams include agency or community - based mentor ##s . Project findings are di ##sse ##minated through [unused1] written [unused2] and oral reports . This approach preserves faculty resources and provides students with mentor ##ing , opportunities for reflection , and time for project development . [SEP]',
  'existing_cui': 'C4291771',
  'existing_cui_name': 'Written Report',
  'existing_cui_desc': '[CLS] Written Report [unused3] ( Intel ##lect ##ual Product , Radio ##logy , Radio ##logy : Written Reports ) [SEP]',
  'our_bi_cui': 'C4291771',
  'our_bi_cui_name': 'Written Report',
  'our_bi_cui_desc': '[CLS] Written Report [unused3] ( Intel ##lect ##ual Product , Radio ##logy , Radio ##logy : Written Reports ) [SEP]',
  'our_cross_cui': 'C0684224',
  'our_cross_cui_name': 'Report',
  'our_cross_cui_desc': '[CLS] Report [unused3] ( Intel ##lect ##ual Product , document , document , qualifier value , record artifact : document report ; reports ; Report ##ed ; Reports ; report ; report [ document ] ) [SEP]'},
 {'mention_id': '27449818.16',
  'test_set_idx': 6211,
  'type': 'T103',
  'name': 'E2',
  'context': '[CLS] to parent ##eral est ##rogen administration is characterized by reduced systemic but prominent he ##pa ##tic est ##rogen ##ic effects on lip ##ids , hem ##ost ##atic factors , G ##H - / I ##G ##F I axis , an ##gio ##tens ##ino ##gen . In order to avoid such adverse metabolic effects of oral treatment , est ##rad ##iol ( [unused1] E ##2 [unused2] ) pro ##dr ##ugs ( E ##stra ##dio ##l pro ##dr ##ugs ) were designed which bypass the liver tissue as inactive molecules . Carbon ##e ##17 - OH su ##lf ##ona ##mi ##de [ - O ##2 - NH ##2 ] substituted est ##ers of E ##2 ( EC ##50 ##8 , others ) were synthesized and tested for carbon [SEP]',
  'existing_cui': 'C1419454',
  'existing_cui_name': 'SNORA62 gene',
  'existing_cui_desc': '[CLS] S ##N ##OR ##A ##6 ##2 gene [unused3] ( Ana ##tom ##ical Structure : RNA , U ##10 ##8 SM ##AL ##L N ##UC ##LE ##OL ##AR ; small n ##uc ##leo ##lar RNA , H / AC ##A box 62 ; RNA , E ##2 SM ##AL ##L N ##UC ##LE ##OL ##AR ; E ##2 ; E ##2 - 1 ; s ##no ##RNA , E ##2 ; SM ##AL ##L N ##UC ##LE ##OL ##AR RNA , H / AC ##A B ##OX , 62 ) [SEP]',
  'our_bi_cui': 'C0014939',
  'our_bi_cui_name': 'Estrogens',
  'our_bi_cui_desc': '[CLS] E ##stro ##gens [unused3] ( Chemical , medication , product , substance : est ##rogen ##s ; O ##est ##rogen ##s ; est ##rogen ##ic preparation ; O ##est ##rogen product ; est ##rogen products ; E ##stro ##gen ; E ##stro ##genic drug ; E ##ST ##RO ##GE ##NS ; O ##est ##rogen ##ic drug ; O ##est ##rogen preparation ; E ##stro ##gen preparation ; O ##est ##rogen ; o ##est ##rogen products ; o ##est ##rogen ; A ##gon ##ists , E ##stro ##gen Re ##ceptor ; o ##est ##rogen ##s ; est ##rogen ; E ##stro ##gen product ; O ##est ##rogen ##ic preparation ; Co ##mpo ##unds , E ##stro ##genic ; Agents , E ##stro ##genic ; E ##stro ##gen [SEP]',
  'our_cross_cui': 'C0014912',
  'our_cross_cui_name': 'Estradiol',
  'our_cross_cui_desc': '[CLS] E ##stra ##dio ##l [unused3] ( Chemical , medication , product , substance : o ##est ##rad ##iol ; Benz ##hor ##mo ##var ##ine ; O ##est ##rad ##iol - R ##ET ##IR ##ED - ; 17 beta - est ##rad ##iol ; O ##est ##rad ##iol [ g ##yna ##e ] ; 17 - Beta o ##est ##rad ##iol ; 17 beta E ##stra ##dio ##l ; E ##stra ##ld ##ine ; E ##stra ##dio ##l [ g ##yna ##e ] ; O ##est ##rad ##iol ; E ##SD ##L ; E ##2 - O ##est ##rad ##iol ; 17 beta - E ##stra ##dio ##l ; 17 beta est ##rad ##iol ; E ##stra ##dio ##l product ; O ##est ##rad ##iol preparation ; [SEP]'},
 {'mention_id': '27611704.11',
  'test_set_idx': 11685,
  'type': 'T017',
  'name': 'B',
  'context': "[CLS] W ##heat Lines B ##read wheat ( Tri ##tic ##um a ##est ##iv ##um , 2 ##n = 6 ##x = 42 , AA ##BB ##D ##D ) has a complex all ##oh ##ex ##ap ##lo ##id genome , which makes it difficult to differentiate between the ho ##mo ##eo ##log ##ous sequences and assign them to the chromosome A , [unused1] B [unused2] , or D sub ##gen ##ome ##s . The chromosome - based draft genome sequence of the ' Chinese Spring ' common wheat cult ##iva ##r enables the large - scale development of polymer ##ase chain reaction ( polymer ##ase chain reaction ) - based markers specific for ho ##mo ##eo ##log ##s . Based on high - confidence ' Chinese Spring [SEP]",
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0017428',
  'our_bi_cui_name': 'Genome',
  'our_bi_cui_desc': '[CLS] Gen ##ome [unused3] ( Ana ##tom ##ical Structure , substance : genome ; genome ##s ; Gen ##ome ##s ; Gen ##omi ##c ) [SEP]',
  'our_cross_cui': 'C0008633',
  'our_cross_cui_name': 'Chromosomes',
  'our_cross_cui_desc': '[CLS] Ch ##rom ##oso ##mes [unused3] ( Ana ##tom ##ical Structure , body structure , cell structure : chromosome ; Ch ##rom ##oso ##mal ; chromosome ##s ; Ch ##rom ##oso ##me ) [SEP]'},
 {'mention_id': '27430240.21',
  'test_set_idx': 5347,
  'type': 'T103',
  'name': 'factors',
  'context': "[CLS] trip ##hos ##phate dip ##hos ##ph ##oh ##ydro ##lase 1 ; also known as CD ##39 ) and 5 ' - e ##cton ##uc ##leo ##ti ##das ##e ( N ##T ##5 ##E ; also known as CD ##7 ##3 ) on the T ##re ##g cell surface are increased during se ##psis . In this study , to determine the [unused1] factors [unused2] leading to the high expression of CD ##39 and CD ##7 ##3 , and the regulation of the CD ##39 / CD ##7 ##3 / ad ##eno ##sin ##e pathway in The number of regulatory T cells under se ##ptic conditions , we constructed a mouse model of se ##psis and separated the The number of regulatory T cells using a flow [SEP]",
  'existing_cui': 'C0960756',
  'existing_cui_name': 'factor A',
  'existing_cui_desc': "[CLS] factor A [unused3] ( Chemical : Co ( beta ) - c ##yan ##o - 7 ' ' - ( 2 - met ##hyl ) ad ##eni ##ny ##l ##co ##ba ##mi ##de ) [SEP]",
  'our_bi_cui': 'C0021054',
  'our_bi_cui_name': 'Immunologic Factors',
  'our_bi_cui_desc': '[CLS] I ##mm ##uno ##log ##ic Factor ##s [unused3] ( Chemical , substance : Factor ##s , I ##mm ##uno ##log ##ic ; I ##mm ##une Factor ##s ; Factor ##s , I ##mm ##uno ##logical ; I ##mm ##uno ##log ##ic Factor ; Factor ##s , I ##mm ##une ; I ##mm ##uno ##logical Factor ##s ; im ##mu ##no ##log ##ic substance ; FA ##CT ##OR ##S I ##MM ##UN ##OL ; im ##mu ##nological substance ; I ##MM ##UN ##OL FA ##CT ##OR ##S ; I ##mm ##uno ##log ##ic substance ) [SEP]',
  'our_cross_cui': 'C0005515',
  'our_cross_cui_name': 'Biological Factors',
  'our_cross_cui_desc': '[CLS] Biological Factor ##s [unused3] ( Chemical , product , substance : Agents , Biological ; Biological Factor ; B ##iol ##og ##ic agent ; agents biological ; B ##iol ##og ##ic Factor ##s ; B ##iol ##og ##ic Factor ; biological substance ; Factor , B ##iol ##og ##ic ; Factor ##s , Biological ; bio ##pha ##rma ##ce ##utical ##s ; Biological product ; B ##IO ##L FA ##CT ##OR ; Biological Agents ; Biological substance ; Biological Factor ##s [ Chemical / In ##g ##red ##ient ] ; Biological agent ; B ##iol ##og ##ics ; Factor , Biological ; biological drug ; B ##IO ##L FA ##CT ##OR ##S ; FA ##CT ##OR ##S B ##IO ##L ; biological substances ; bio ##pha [SEP]'},
 {'mention_id': '27444636.36',
  'test_set_idx': 5987,
  'type': 'T033',
  'name': 'mean age 36 years',
  'context': '[CLS] Battery for At ##ten ##tion Performance version 2 . 3 . Assessment ##s were performed immediately prior to and 5 days after switching from o ##x ##car ##ba ##ze ##pine to es ##lica ##rb ##az ##ep ##ine ace ##tate ( days 0 and 5 , respectively ) . The analysis included 21 patients ( 12 women , 9 men ; [unused1] mean age 36 years [unused2] ) . After switching from o ##x ##car ##ba ##ze ##pine to es ##lica ##rb ##az ##ep ##ine ace ##tate , there were significant improvements in mean scores for Ad ##verse Events Profile ( P < . 00 ##1 ) , Quality of Life in E ##pile ##psy In ##vent ##ory 10 ( P = . 00 ##1 ) , [SEP]',
  'existing_cui': 'C1847486',
  'existing_cui_name': 'Mean age of onset 56 years',
  'existing_cui_desc': '[CLS] Mean age of onset 56 years [unused3] ( Finding ) [SEP]',
  'our_bi_cui': 'C1834001',
  'our_bi_cui_name': 'Mean age at onset 32 years',
  'our_bi_cui_desc': '[CLS] Mean age at onset 32 years [unused3] ( Finding ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27251800.42',
  'test_set_idx': 837,
  'type': 'T058',
  'name': 'Stoppa',
  'context': '[CLS] 15 th post - operative ( P > 0 . 05 ) . Co ##mp ##lication ##s occurred in 88 % of Stop ##pa group ( 22 patients ) and 64 % in lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair group ( 16 patients ) ( P < 0 . 05 ) . The comparative study between the [unused1] Stop ##pa [unused2] and lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair approaches for the bilateral ing ##uin ##al her ##nia repair demonstrated that : ( 1 ) The lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair approach showed less surgical trauma despite the longer operation time ; ( 2 ) Quality of life during the early post - operative [SEP]',
  'existing_cui': 'C1538901',
  'existing_cui_name': 'ACD gene',
  'existing_cui_desc': '[CLS] AC ##D gene [unused3] ( Ana ##tom ##ical Structure : T ##EL ##OM ##ER ##E PR ##OT ##EI ##N T ##PP ##1 ; ad ##ren ##oc ##ort ##ical d ##ys ##p ##lasia ho ##mo ##log ; AC ##D Gene ; Ad ##ren ##oc ##ort ##ical D ##ys ##p ##lasia Ho ##mo ##log ( Mouse ) Gene ; P ##OT ##1 - IN ##TE ##RA ##CT ##ING PR ##OT ##EI ##N 1 ; P ##OT ##1 - AND T ##IN ##2 - OR ##GA ##NI ##Z ##ING PR ##OT ##EI ##N ; P ##OT ##1 and T ##IN ##2 organizing protein ; AC ##D , M ##O ##US ##E , H ##OM ##OL ##O ##G OF ; Tin ##t ##1 ; T ##IN ##2 interacting protein 1 [SEP]',
  'our_bi_cui': 'C0031150',
  'our_bi_cui_name': 'Laparoscopy',
  'our_bi_cui_desc': '[CLS] La ##par ##os ##copy [unused3] ( Health Care Act ##ivity , G ##I only , procedure : La ##par ##os ##copic procedure ; per ##ito ##ne ##os ##copy ; Co ##eli ##os ##copy ; Ce ##lio ##sco ##pies ; Per ##ito ##ne ##os ##co ##pies ; Abd ##ome ##n end ##os ##copy ; c ##eli ##os ##copy ; lap ##aro ##scopic procedure ; Per ##ito ##ne ##os ##copy ; La ##p ; Ce ##lio ##sco ##py ; La ##par ##os ##co ##pies ; lap ##aro ##sco ##pies ; lap ##aro ##sco ##py ; lap ##aro ##scopic procedures ; diagnostic lap ##aro ##sco ##py ; End ##os ##copy of abdomen ; lap ##aro ##sco ##py procedure ; La ##par ##os ##copic ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27475696.10',
  'test_set_idx': 7473,
  'type': 'T038',
  'name': 'alcohol',
  'context': '[CLS] E ##stro ##gen and Pro ##ges ##tero ##ne hormone receptor expression in oral cavity cancer Recent studies have shown an increase in the incidence of oral sq ##ua ##mous cell car ##cin ##oma in younger patients . The hypothesis that tumors could be ho ##rmon ##ally induced during pregnancy or in young female patients without the well - known risk factors [unused1] alcohol [unused2] or tobacco abuse seems to be plausible . E ##stro ##gen Re ##ceptor alpha and Pro ##ges ##tero ##ne Re ##ceptor expression were analyzed in normal oral m ##uc ##osa ( n = 5 ) , oral precursor lesions ( simple h ##yper ##p ##lasia , n = 11 ; sq ##ua ##mous in ##tra ##ep ##ith ##eli ##al neo ##p ##lasia [SEP]',
  'existing_cui': 'C0202304',
  'existing_cui_name': 'Ethanol measurement',
  'existing_cui_desc': '[CLS] Ethan ##ol measurement [unused3] ( Health Care Act ##ivity , procedure : DR ##U ##G SC ##RE ##EN Q ##U ##AN ##TA ##LC ##OH ##OL ##S ; Ethan ##ol Me ##as ##ure ##ment ; Al ##co ##hol ##s levels ; Al ##co ##hol ##s ; Ethan ##ol ; DR ##U ##G SC ##RE ##EN Q ##U ##AN ##TI ##TA ##TI ##VE AL ##CO ##H ##OL ##S ; Al ##co ##hol ; alcohol measurements ; Me ##as ##ure ##ment of alcohol ; E ##thy ##l Al ##co ##hol Me ##as ##ure ##ments ; Drug screen q ##uant ##al ##co ##hol ##s ; Al ##co ##hol level ; Al ##co ##hol measurement ; ET ##HA ##N ##OL ; alcohol level test ) [SEP]',
  'our_bi_cui': 'C0001975',
  'our_bi_cui_name': 'Alcohols',
  'our_bi_cui_desc': '[CLS] Al ##co ##hol ##s [unused3] ( Chemical , substance : alcohol ; Al ##co ##hol [ Chemical Class ] ; Al ##co ##hol ; Al ##co ##hol ##s [ Chemical / In ##g ##red ##ient ] ; alcohol ##s ; class of chemical alcohol ##s ; alcohol substance ) [SEP]',
  'our_cross_cui': 'C0085762',
  'our_cross_cui_name': 'Alcohol abuse',
  'our_cross_cui_desc': '[CLS] Al ##co ##hol abuse [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , finding : abuses alcohol ; alcohol ; use , problem ; Ethan ##ol abuse ; problem drinking ; alcohol abuse ; problem ; alcohol use ; ethanol abuse ; Al ##co ##hol abuse , un ##sp ##ec ##ified drinking behaviour ; Al ##co ##hol abuse , un ##sp ##ec ##ified drinking behavior ; ET ##OH abuse ; abuse ; alcohol ; AA - Al ##co ##hol abuse ; alcohol abuse disorder ; drinking problems ; Al ##co ##hol abuse , un ##sp ##ec ##ified ; Problem Dr ##ink ##ing ; r ##nd ##x alcohol abuse ; alcohol abuses ; Al ##co ##hol Abu ##se ; Al ##co ##hol abuse [SEP]'},
 {'mention_id': '27920749.27',
  'test_set_idx': 21864,
  'type': 'T170',
  'name': 'emotionally negative film clip',
  'context': "[CLS] ##al bias ##es to happy and angry faces . In Study 1 , healthy young women were recruited and randomly assigned to one of the three groups : up - , down - , and no - regulation . Part ##ici ##pants were instructed to re ##ap ##pra ##ise their emotions to increase and decrease emotional experience while viewing an [unused1] emotionally negative film clip [unused2] . At ##ten ##tional bias was assessed with a dot - probe task with pictures of angry and happy facial expressions . In Study 2 , a separate group of healthy young men and women participated . Part ##ici ##pants ' trait re ##ap ##pra ##isa ##l and suppression as well as state and trait anxiety were assessed . A [SEP]",
  'existing_cui': 'C0558263',
  'existing_cui_name': 'Emotionally distant',
  'existing_cui_desc': '[CLS] Em ##otion ##ally distant [unused3] ( Finding , finding : Em ##otion ##ally al ##oof ) [SEP]',
  'our_bi_cui': 'C3463807',
  'our_bi_cui_name': 'Video Media',
  'our_bi_cui_desc': '[CLS] Video Media [unused3] ( Intel ##lect ##ual Product : Video ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28351716.3',
  'test_set_idx': 32856,
  'type': 'T058',
  'name': 'digital fundus images',
  'context': '[CLS] Di ##ag ##nosis of re ##tina ##l health in [unused1] digital fund ##us images [unused2] using continuous wave ##let transform and en ##tro ##pies Vision is para ##mount to humans to lead an active personal and professional life . The prevalence of o ##cular diseases is rising , and diseases such as g ##lau ##com ##a , Di ##abe ##tic Re ##tino ##pathy and Age - related Mac ##ular De ##gene ##ration are the leading causes of blind ##ness in developed countries . I ##dent ##ifying these diseases in mass screening programmes is time - consuming , labor - intensive and the diagnosis can be subjective . The use of an automated computer aided diagnosis system will reduce the time taken for analysis and will also [SEP]',
  'existing_cui': 'C0017129',
  'existing_cui_name': 'gastric fundus',
  'existing_cui_desc': '[CLS] gas ##tric fund ##us [unused3] ( Ana ##tom ##ical Structure , body structure , vent ##ric ##ular ##is : Fund ##us ; Fund ##us gas ##tric ##us ; Fund ##us , Gas ##tric ; ST ##OM ##AC ##H , F ##UN ##D ##US ; Fund ##us of St ##oma ##ch ; fund ##us of stomach ; Gas ##tric fund ##us ; stomach fund ##us ; Gas ##tric fund ##us structure ; Fund ##us structure of stomach ; Fund ##us of the St ##oma ##ch ; F ##UN ##D ##US OF THE ST ##OM ##AC ##H ; Fund ##us of stomach ; Gas ##tric Fund ##us ) [SEP]',
  'our_bi_cui': 'C1704254',
  'our_bi_cui_name': 'Medical Image',
  'our_bi_cui_desc': '[CLS] Medical Image [unused3] ( Intel ##lect ##ual Product : Image ) [SEP]',
  'our_cross_cui': 'C0200189',
  'our_cross_cui_name': 'Fundus photography',
  'our_cross_cui_desc': '[CLS] Fund ##us photography [unused3] ( Health Care Act ##ivity , procedure : fund ##us photography ; O ##cular fund ##us photography ; eye fund ##us photography ) [SEP]'},
 {'mention_id': '27604152.49',
  'test_set_idx': 11446,
  'type': 'T007',
  'name': 'non - aureus staphylococci',
  'context': '[CLS] n = 3 ) . No met ##hic ##ill ##in - resistant St ##aph ##yl ##oc ##oc ##cus au ##reus were isolated from 58 ##6 culture ##d s ##wab ##s . Met ##hic ##ill ##in - susceptible S . au ##reus were detected in 9 / 257 ( 3 . 5 % ) s ##wab ##s and [unused1] non - au ##reus s ##ta ##phy ##loc ##oc ##ci [unused2] in 22 / 257 ( 8 . 5 % ) s ##wab ##s . The estimated true met ##hic ##ill ##in - resistant St ##aph ##yl ##oc ##oc ##cus au ##reus sub ##c ##lini ##cal co ##lon ##isation prevalence was 0 % , with an upper 95 % C ##I boundary of 1 . 9 % for [SEP]',
  'existing_cui': 'C1318973',
  'existing_cui_name': 'Staphylococcus aureus infection',
  'existing_cui_desc': '[CLS] St ##aph ##yl ##oc ##oc ##cus au ##reus infection [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : St ##aph ##yl ##oc ##oc ##cus au ##reus infections ; St ##aph ##yl ##oc ##oc ##cus au ##reus ; In ##fect ##ion due to St ##aph ##yl ##oc ##cc ##us au ##reus ; In ##fect ##ion due to St ##aph ##yl ##oc ##oc ##cus au ##reus ; St ##aph ##yl ##oc ##oc ##cus au ##reus In ##fect ##ion ; In ##fect ##ion caused by St ##aph ##yl ##oc ##oc ##cus au ##reus ; s ##ta ##phy ##loc ##oc ##cus au ##reus infection ; s ##ta ##phy ##loc ##oc ##cus ; au ##reus ; au ##reus ; St ##aph ##yl ##oc ##oc ##cus ) [SEP]',
  'our_bi_cui': 'C0038172',
  'our_bi_cui_name': 'Staphylococcus aureus',
  'our_bi_cui_desc': '[CLS] St ##aph ##yl ##oc ##oc ##cus au ##reus [unused3] ( Ba ##cter ##ium , organism : S . au ##reus ; s ##ta ##ph au ##reus ; ST ##AP ##H AU ##RE ##US ; S . AU ##RE ##US ; St ##aph . au ##reus ; St ##aph ##yl ##oc ##oc ##cus p ##yo ##gene ##s au ##reus ; s . au ##reus ; ST ##AP ##H ##Y ##L ##OC ##OC ##CU ##S AU ##RE ##US ; s ##ta ##phy ##loc ##oc ##cus au ##reus bacteria ; St ##aph ##yl ##oc ##oc ##cus au ##reus Rosen ##bach 1884 ; Micro ##co ##cc ##us au ##reus ; St ##aph ##yl ##oc ##oc ##cus Au ##reus ; St ##aph ##loc ##oc ##cus p ##yo ##gene ##s c ##it [SEP]',
  'our_cross_cui': 'C0038170',
  'our_cross_cui_name': 'Genus staphylococcus',
  'our_cross_cui_desc': '[CLS] Genus s ##ta ##phy ##loc ##oc ##cus [unused3] ( Ba ##cter ##ium , organism : St ##aph ##yl ##oc ##oc ##cus species ; Genus St ##aph ##yl ##oc ##oc ##cus ; Au ##ro ##co ##cc ##us ; ST ##AP ##H ##Y ##L ##OC ##OC ##CU ##S ; St ##aph ##yl ##oc ##oc ##cus ; St ##aph ##yl ##oc ##oc ##cus Rosen ##bach 1884 ; s ##ta ##phy ##loc ##oc ##cus bacteria ; ST ##AP ##H ; s ##ta ##phy ##loc ##oc ##cus ) [SEP]'},
 {'mention_id': '27782044.8',
  'test_set_idx': 16578,
  'type': 'T204',
  'name': 'Max Clara',
  'context': "[CLS] Normal Level ##s of U ##rina ##ry CC ##16 Pro ##tein . Co ##mme ##nts on Be ##ame ##r et al . Association of Children ' s U ##rina ##ry CC ##16 Level ##s with A ##rsen ##ic Con ##cent ##rations in Multiple Environmental Media . In ##t . J . En ##vir ##on . Re ##s . Public Health 2016 , 13 , 52 ##1 In 1937 , [unused1] Max Clara [unused2] described a new type of cell in the human lung , which was later determined to be an ex ##oc ##rine secret ##ory cell type containing g ##ran ##ules composed of proteins [ 1 ] . [ . . . ] . [SEP]",
  'existing_cui': 'C0225668',
  'existing_cui_name': 'Clara cell',
  'existing_cui_desc': '[CLS] Clara cell [unused3] ( Ana ##tom ##ical Structure , body structure , cell : Clara Cell ; Clara cell of br ##on ##chi ##ole ; c ##lar ##a cell ; cells c ##lar ##a ; B ##ron ##chi ##olar non - c ##iliated cell ; Non ##ci ##lia ##ted B ##ron ##chi ##olar E ##pit ##hel ##ial Cell ; Club Cell ) [SEP]',
  'our_bi_cui': 'C0225668',
  'our_bi_cui_name': 'Clara cell',
  'our_bi_cui_desc': '[CLS] Clara cell [unused3] ( Ana ##tom ##ical Structure , body structure , cell : Clara Cell ; Clara cell of br ##on ##chi ##ole ; c ##lar ##a cell ; cells c ##lar ##a ; B ##ron ##chi ##olar non - c ##iliated cell ; Non ##ci ##lia ##ted B ##ron ##chi ##olar E ##pit ##hel ##ial Cell ; Club Cell ) [SEP]',
  'our_cross_cui': 'C0086418',
  'our_cross_cui_name': 'Homo sapiens',
  'our_cross_cui_desc': '[CLS] Ho ##mo sa ##pie ##ns [unused3] ( E ##uka ##ryo ##te , Tax ##onomy , organism , qualifier value : Humans ; Man , Modern ; humans ; man ; MA ##N ; Ho ##mo sa ##pie ##ns ( living organism ) [ Am ##bi ##guous ] ; ho ##mo sa ##pie ##ns ; Human , General ; Ho ##mo sa ##pie ##ns Linnaeus , 1758 ; Human ; Human - origin ; human ; Ho ##mo sa ##pie ##ns ( living organism ) ; Modern Man ; Man ) [SEP]'},
 {'mention_id': '27905972.25',
  'test_set_idx': 21639,
  'type': 'T098',
  'name': 'Danish',
  'context': '[CLS] mixed log ##istic model with different slopes in adult , middle , and old age , and includes the m ##oda ##l age at death as a parameter to account for the delay in mortality . A ##pp ##lying the Co ##D ##e model to age - specific pro ##ba ##bilities of death for Japanese , French , American , and [unused1] Danish [unused2] men and women between 1950 and 2010 showed a very good fit of the full age pattern of mortality . Del ##ay of mortality explained about two - thirds of the increase in life expect ##ancy at birth , whereas compression of mortality due to mortality decline ##s in young age explained about one - third . No strong compression of mortality [SEP]',
  'existing_cui': 'C0452642',
  'existing_cui_name': 'Danish pastry',
  'existing_cui_desc': '[CLS] Danish past ##ry [unused3] ( Food , substance ) [SEP]',
  'our_bi_cui': 'C0011318',
  'our_bi_cui_name': 'Denmark',
  'our_bi_cui_desc': '[CLS] Denmark [unused3] ( Spa ##tial Concept , geographic location : D ##K ; D ##A ; den ##mark ; DE ##N ##MA ##R ##K ; D ##N ##K ) [SEP]',
  'our_cross_cui': 'C1257890',
  'our_cross_cui_name': 'Population Group',
  'our_cross_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]'},
 {'mention_id': '27791364.23',
  'test_set_idx': 16991,
  'type': 'T033',
  'name': 'photostability',
  'context': '[CLS] ##ic strategy , using b ##ov ##ine serum album ##in as a bio ##tem ##plate at physiological temperature . The as - prepared G ##d : C ##u ##S @ b ##ov ##ine serum album ##in na ##no ##par ##tic ##les with ultra ##sma ##ll sizes ( ca . 9 nm ) exhibited high photo ##thermal conversion efficiency and good [unused1] photos ##ta ##bility [unused2] under near - infrared laser i ##rra ##diation . With do ##ped G ##d species and strong t ##una ##ble near - infrared absorb ##ance , G ##d : C ##u ##S @ b ##ov ##ine serum album ##in na ##no ##par ##tic ##les demonstrate prominent tumor - contrasted imaging performance both on the photo ##aco ##ustic and magnetic resonance imaging [SEP]',
  'existing_cui': 'C3272552',
  'existing_cui_name': 'Photostability Drug Study',
  'existing_cui_desc': '[CLS] Photo ##sta ##bility Drug Study [unused3] ( Intel ##lect ##ual Product : Photo ##sta ##bility ) [SEP]',
  'our_bi_cui': 'C0007613',
  'our_bi_cui_name': 'Cell physiology',
  'our_bi_cui_desc': '[CLS] Cell physiology [unused3] ( B ##iol ##og ##ic Fun ##ction : cell physiology ; Cell ##ular Process ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##a ; Cell ##ular Fun ##ction ; cells physiology ; Cell ##ular Ph ##ys ##iology ; Sub ##cellular Process ; cells function ; cell function ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##on ; Cell Ph ##ys ##iology ; Cell Process ; Cell Fun ##ction ; CE ##LL P ##H ##Y ##SI ##OL ; P ##H ##Y ##SI ##OL CE ##LL ; cell functions ; Ph ##eno ##men ##a , Cell Ph ##ys ##iol ##ogical ; Ph ##ys ##iology , Cell ; Ph ##eno ##men ##on , Cell Ph ##ys ##iol ##ogical ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28127863.29',
  'test_set_idx': 26348,
  'type': 'T033',
  'name': 'Signs',
  'context': '[CLS] speech re ##spiration , p ##hon ##ation , oral motor function , and ve ##lop ##har ##yn ##ge ##al function . The results of the present study suggest that a ne ##uro ##logical et ##iology could be added to the previously described structural et ##iology explaining the speech difficulties found in 22 ##q ##11 . 2 del ##eti ##on syndrome . [unused1] Sign ##s [unused2] of difficulties in both speech motor planning and speech motor programming were found . Further studies are needed to confirm the results , as are studies of the association between structural brain abnormal ##ities and ne ##uro ##logical speech symptoms . For clinical purposes , it is important that clinic ##ians have knowledge about the variable speech symptoms that may occur [SEP]',
  'existing_cui': 'C0311392',
  'existing_cui_name': 'Physical findings',
  'existing_cui_desc': '[CLS] Physical findings [unused3] ( Finding , finding , physical finding : physical finding ; physical findings ; Physical finding ; physical findings [ use for free text ] ; Physical signs ; Sign ##s - physical ; physical signs ; Physical sign ; Sign ; find physical ) [SEP]',
  'our_bi_cui': 'C1519316',
  'our_bi_cui_name': 'Signature',
  'our_bi_cui_desc': '[CLS] Sign ##ature [unused3] ( Intel ##lect ##ual Product : signature ##T ##ex ##t ; Sign ##ed ) [SEP]',
  'our_cross_cui': 'C0037088',
  'our_cross_cui_name': 'Signs and Symptoms',
  'our_cross_cui_desc': '[CLS] Sign ##s and S ##ym ##pt ##oms [unused3] ( Finding , finding : Sign ##s and S ##ym ##pt ##oms [ Disease / Finding ] ; Clinical findings ; clinical findings ; O ##bs ##er ##vable En ##ti ##ty ; sign and s ##ym ##pt ##om ; Sign and s ##ym ##pt ##om ; S ##ym ##pt ##oms and Sign ##s ; sign / s ##ym ##pt ##om ; Finding ; Clinical observation : symptoms and signs ; S ##IG ##NS S ##Y ##MP ##TO ##MS ; Clinical finding ) [SEP]'},
 {'mention_id': '28478603.20',
  'test_set_idx': 37422,
  'type': 'T170',
  'name': "Shannon 's information index",
  'context': "[CLS] am ##plified 96 bands , of which 91 ( 93 . 65 % ) were p ##oly ##morphic , whereas 20 simple sequence repeat prime ##rs am ##plified 73 bands , of which 70 ( 96 . 50 % ) were p ##oly ##morphic . N ##ei ' s gene diversity ( h = 0 . 28 ) , [unused1] Shannon ' s information index [unused2] ( I = 0 . 43 ) , and p ##oly ##mor ##phism information content ( P ##IC = 0 . 29 ) generated using the simple sequence repeat prime ##rs were higher than that with Inter simple sequence repeat prime ##rs ( h = 0 . 23 , I = 0 . 37 , P ##IC = 0 . [SEP]",
  'existing_cui': 'C0237669',
  'existing_cui_name': 'Informational Messages',
  'existing_cui_desc': '[CLS] Information ##al Message ##s [unused3] ( Intel ##lect ##ual Product , Message ##s : Information ) [SEP]',
  'our_bi_cui': 'C1519281',
  'our_bi_cui_name': 'Shannon Awards , Guidelines',
  'our_bi_cui_desc': '[CLS] Shannon Awards , Guide ##lines [unused3] ( Intel ##lect ##ual Product ) [SEP]',
  'our_cross_cui': 'C0918012',
  'our_cross_cui_name': 'Index',
  'our_cross_cui_desc': '[CLS] Index [unused3] ( Intel ##lect ##ual Product : index ##ed ; index ; index ##es ; Index ##es as Top ##ic ) [SEP]'},
 {'mention_id': '28319502.10',
  'test_set_idx': 31866,
  'type': 'T038',
  'name': 'RS3PE',
  'context': '[CLS] se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting ed ##ema ( RS ##3 ##P ##E ) is a rare condition in the elderly and can appear as a first presentation of various types of r ##he ##umatic and ma ##li ##gnant diseases . We presented a 62 - year - old man with the diagnosis of [unused1] RS ##3 ##P ##E [unused2] based on the clinical sign and laboratory data . Because of the possibility of associated ma ##li ##gna ##ncies in RS ##3 ##P ##E , F ##D ##G P ##ET / CT was performed to exclude o ##cc ##ult tumors . The images showed multiple , symmetrical ##ly , di ##ff ##use ##ly increased F - F ##D ##G up [SEP]',
  'existing_cui': 'C3147672',
  'existing_cui_name': 'IRS3P gene',
  'existing_cui_desc': '[CLS] I ##RS ##3 ##P gene [unused3] ( Ana ##tom ##ical Structure : insulin receptor substrate 3 , pseudo ##gene ) [SEP]',
  'our_bi_cui': 'C0035357',
  'our_bi_cui_name': 'Retroperitoneal fibrosis',
  'our_bi_cui_desc': "[CLS] Re ##tro ##per ##ito ##nea ##l fi ##bro ##sis [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : Or ##mond ' s disease ; or ##mond ' s disease ; Sc ##ler ##os ##ing re ##tro ##per ##ito ##ni ##tis ; I ##dio ##pathic re ##tro ##per ##ito ##nea ##l fi ##bro ##sis ; R ##P ##F - Re ##tro ##per ##ito ##nea ##l fi ##bro ##sis ; id ##io ##pathic re ##tro ##per ##ito ##nea ##l fi ##bro ##sis ; Disease , Or ##mond ' s ; re ##tro ##per ##ito ##nea ##l fi ##bro ##sis ; Or ##mond Disease ; or ##mond disease ; Fi ##bro ##ses , Re ##tro ##per ##ito ##nea ##l ; Re ##tro ##per ##ito ##nea [SEP]",
  'our_cross_cui': 'C2919482',
  'our_cross_cui_name': 'Remitting seronegative symmetrical synovitis with pitting edema',
  'our_cross_cui_desc': '[CLS] Re ##mit ##ting se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting ed ##ema [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : Re ##lap ##sing se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting ed ##ema ; Re ##mit ##ting se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting o ##ede ##ma ; RS ##3 ##P ##E - Re ##mit ##ting se ##rone ##gat ##ive symmetric s ##yn ##ov ##itis with pit ##ting o ##ede ##ma ; Re ##lap ##sing se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting o ##ede ##ma ; RS ##3 ##P ##E - Re ##mit ##ting se ##rone ##gat ##ive symmetric s ##yn ##ov ##itis with [SEP]'},
 {'mention_id': '27667643.25',
  'test_set_idx': 13599,
  'type': 'T033',
  'name': 'health issue',
  'context': "[CLS] and disease . Clinical ya ##rn ##ing consists of three inter ##rel ##ated areas : the social ya ##rn , in which the practitioner aims to find common ground and develop the inter ##person ##al relationship ; the diagnostic ya ##rn , in which the practitioner facilitate ##s the patient ' s health story while interpret ##ing it through a bio ##medical or scientific lens ; and the management ya ##rn , that employs stories and metaphor ##s as tools for patients to help them understand a [unused1] health issue [unused2] so a collaborative management approach can be adopted . There is cultural and research evidence that supports this approach . Clinical ya ##rn ##ing has the potential to improve outcomes for patients and practitioners . [SEP]",
  'existing_cui': 'C4274868',
  'existing_cui_name': 'Parental health issue',
  'existing_cui_desc': '[CLS] Pa ##rent ##al health issue [unused3] ( Finding , situation : Pa ##rent has health disorder ) [SEP]',
  'our_bi_cui': 'C0033213',
  'our_bi_cui_name': 'Problem',
  'our_bi_cui_desc': '[CLS] Problem [unused3] ( Finding , finding : Issue ; problem ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28481342.1',
  'test_set_idx': 37606,
  'type': 'T038',
  'name': 'Recurrent noncoding regulatory mutations',
  'context': '[CLS] [unused1] Re ##current non ##co ##ding regulatory mutations [unused2] in pan ##cre ##atic duct ##al ad ##eno ##car ##cin ##oma The contributions of coding mutations to tumor ##ige ##nes ##is are relatively well known ; however , little is known about so ##matic alterations in non ##co ##ding DNA . Here we describe Gen ##omi ##c En ##rich ##ment Co ##mp ##uta ##tional C ##luster ##ing Operation to analyze so ##matic non ##co ##ding alterations in 308 pan ##cre ##atic duct ##al ad ##eno ##car ##cin ##oma ##s and identify commonly m ##uta ##ted regulatory regions . We find re ##current non ##co ##ding mutations to be en ##rich ##ed in PD ##A pathways , including a ##xon guidance and cell ad ##hesion , and newly [SEP]',
  'existing_cui': 'C0887909',
  'existing_cui_name': 'RNA , Untranslated',
  'existing_cui_desc': '[CLS] RNA , Un ##tra ##ns ##lated [unused3] ( Chemical : RNA , Non ##co ##ding ; Non - Co ##ding RNA ; Non ##co ##ding RNA ; RNA , Non ##tra ##ns ##lated ; Un ##tra ##ns ##lated RNA ; RNA , Non - Co ##ding ; RNA , Non Pro ##tein Co ##ding ; RNA , Non Co ##ding ; n ##p ##c ##RNA ; n ##c ##RNA ; RNA , Un ##tra ##ns ##lated [ Chemical / In ##g ##red ##ient ] ; Fun ##ctional RNA ; Non ##tra ##ns ##lated RNA ; Non - coding RNA ; RNA , Non - P ##eptide - Co ##ding ; Non - Pro ##tein - Co ##ding RNA ; RNA , Non - Pro ##tein - [SEP]',
  'our_bi_cui': 'C0596611',
  'our_bi_cui_name': 'Gene Mutation',
  'our_bi_cui_desc': '[CLS] Gene Mu ##tation [unused3] ( B ##iol ##og ##ic Fun ##ction : DNA Se ##quence Alt ##eration ; Gene Alt ##eration ; DNA Alt ##eration ; gene mutation ; genes mutation ; Gene Mu ##tation Type ; M ##OL ##PA ##TH . M ##UT ; gene mutations ; Se ##quence Alt ##eration ; Gene mutation ; mutation gene ) [SEP]',
  'our_cross_cui': 'C0026882',
  'our_cross_cui_name': 'Mutation',
  'our_cross_cui_desc': '[CLS] Mu ##tation [unused3] ( B ##iol ##og ##ic Fun ##ction , finding : genome mutation ; Gene ##tic mutation ; genetic mutation ; genetic mutations ; mutations ; Mu ##tations ; mutation ; alterations genetic ; genetic alter ##ation ) [SEP]'},
 {'mention_id': '28186384.40',
  'test_set_idx': 27815,
  'type': 'T033',
  'name': 'system - related barriers',
  'context': '[CLS] in the subsequent results . System - related factors included a lack of confidence in the rational ##e for modify ##ing the current c ##er ##vic ##al screening programme , and concerns about sample contamination and identity theft . In ##sight ##s gained from this research can be used to guide further en ##qui ##ry into the possibility of human p ##ap ##illo ##ma ##virus self - sampling and to help inform future policy and practice . Personal and [unused1] system - related barriers [unused2] including low confidence in the reasons for changing current c ##er ##vic ##al screening provision need to be addressed , should human p ##ap ##illo ##ma ##virus self - sampling be incorporated into the c ##er ##vic ##al screening programme . [SEP]',
  'existing_cui': 'C1706912',
  'existing_cui_name': 'BARRIER',
  'existing_cui_desc': '[CLS] BA ##R ##RI ##ER [unused3] ( Medical Devi ##ce : Barr ##ier ; Barr ##ier Devi ##ce Co ##mpo ##nent ; barrier ) [SEP]',
  'our_bi_cui': 'C1717601',
  'our_bi_cui_name': 'Barriers to self management',
  'our_bi_cui_desc': '[CLS] Barr ##iers to self management [unused3] ( Finding ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28167893.47',
  'test_set_idx': 27488,
  'type': 'T033',
  'name': 'undeveloped',
  'context': '[CLS] be a turning moment of the m ##or ##ph ##ogen ##etic changes resisting fungi ##cid ##al toxicity . Con - G formed multi ##cellular con ##id ##ia with cell walls and se ##pta and intact dense c ##yt ##op ##las ##m . In I ##p ##r - N , fun ##gal s ##por ##ulation was in ##hibit ##ed by forming mostly [unused1] und ##eveloped [unused2] un ##ice ##ll ##ular con ##id ##ia with de ##graded and ne ##c ##rot ##ic c ##yt ##op ##las ##m . However , in I ##p ##r - I , conspicuous cellular changes occurred during s ##por ##ulation by forming multi ##cellular con ##id ##ia with double layered ( thick ##ened ) cell walls and accumulation of pro ##life ##rated lip [SEP]',
  'existing_cui': 'C0555024',
  'existing_cui_name': 'Eloped',
  'existing_cui_desc': '[CLS] El ##oped [unused3] ( Finding , finding ) [SEP]',
  'our_bi_cui': 'C0041674',
  'our_bi_cui_name': 'Unemployment',
  'our_bi_cui_desc': "[CLS] Un ##em ##p ##loy ##ment [unused3] ( Finding , finding , history : unemployment ; Without employment ; out working ; U ##E - Un ##em ##p ##loy ##ed ; NO ##T E ##MP ##L ##O ##Y ##ED ; outs work ; Does Not Work ; Un ##em ##p ##loy ##ed ; U / E - Un ##em ##p ##loy ##ed ; out of work ; out work ; job ##lessness ; job ##less ; UN ##EM ##PL ##O ##Y ##ME ##NT ; unemployed ; Does n ' t Work ; Out ( of ) ; work ; Do n ' t Work ; Out of work ; work out ) [SEP]",
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27832198.48',
  'test_set_idx': 18670,
  'type': 'T017',
  'name': 'Dd2',
  'context': '[CLS] ##quin ##e resistance transport ##er - specific zinc - finger n ##uc ##lea ##ses to genetically di ##sse ##ct this all ##ele in the path ##ogenic setting of as ##ex ##ual blood - stage infection . Comparative analysis of drug resistance and growth profiles of re ##comb ##ina ##nt parasite ##s that express Cam ##7 ##34 or variants thereof , [unused1] D ##d ##2 [unused2] ( the most common Southeast Asian variant ) , or wild - type P . f ##al ##ci ##par ##um ch ##lor ##o ##quin ##e resistance transport ##er , revealed previously unknown roles for P ##f ##CR ##T mutations in m ##od ##ulating parasite su ##s ##ce ##pt ##ibility to multiple anti ##mal ##aria ##l agents . These results were generated [SEP]',
  'existing_cui': 'C4151768',
  'existing_cui_name': 'bacterium DD2',
  'existing_cui_desc': '[CLS] bacterium DD ##2 [unused3] ( Ba ##cter ##ium ) [SEP]',
  'our_bi_cui': 'C1412089',
  'our_bi_cui_name': 'ABCD2 gene',
  'our_bi_cui_desc': '[CLS] ABC ##D ##2 gene [unused3] ( Ana ##tom ##ical Structure : AD ##RE ##N ##OL ##E ##U ##KO ##D ##Y ##ST ##RO ##P ##H ##Y - L ##I ##KE 1 ; AD ##RE ##N ##OL ##E ##U ##KO ##D ##Y ##ST ##RO ##P ##H ##Y - R ##EL ##AT ##ED ; AL ##DR ##P ; ATP binding cassette subfamily D member 2 ; ATP - B ##IN ##DI ##NG CA ##SS ##ET ##TE , S ##U ##BF ##AM ##IL ##Y 3 , ME ##MB ##ER 2 ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27297967.80',
  'test_set_idx': 1785,
  'type': 'T082',
  'name': 'Asn19/23Gln/Tyr920Glu / Ser921Glu',
  'context': '[CLS] ##20 / Ser ##9 ##21 p ##hos ##ph ##ory ##lation are responsible for altering the kinetic ##s of MR ##P ##1 - mediated As ( G ##S ) 3 transport . The kinetic ##s of As ( G ##S ) 3 transport by human em ##b ##ryo ##nic kidney 29 ##3 - [unused1] As ##n ##19 / 23 ##G ##ln / Ty ##r ##9 ##20 ##G ##lu / Ser ##9 ##21 ##G ##lu [unused2] were similar to H ##E ##K - W ##T - MR ##P ##1 , indicating that the p ##hos ##ph ##ory ##lation - mimic ##king substitution ##s a ##bro ##gated the influence of As ##n ##19 / 23 ##G ##ln g ##ly ##cos ##yla ##tion . Overall , these data suggest that [SEP]',
  'existing_cui': 'C0175489',
  'existing_cui_name': 'Nucleus Globosus',
  'existing_cui_desc': '[CLS] N ##uc ##le ##us G ##lo ##bos ##us [unused3] ( Ana ##tom ##ical Structure , body structure : Structure of c ##ere ##bella ##r g ##lo ##bos ##e nucleus ; G ##lo ##bos ##us , N ##uc ##le ##us ; Post ##eri ##or inter ##posed nucleus ; N ##uc ##le ##us inter ##po ##si ##tus posterior ; N ##uc ##le ##us inter ##po ##si ##tus posterior c ##ere ##bell ##i ; N ##uc ##le ##us g ##lo ##bos ##us c ##ere ##bell ##i ; Post ##eri ##or inter ##po ##si ##tus nucleus ; posterior inter ##po ##si ##tus nucleus ; G ##lo ##bos ##e nucleus ; g ##lo ##bos ##e nucleus ; posterior inter ##posed nucleus ; N ##uc ##le ##i g ##lo ##bos ##i ; [SEP]',
  'our_bi_cui': 'C0949771',
  'our_bi_cui_name': 'Amino Acid Transporter',
  'our_bi_cui_desc': '[CLS] Am ##ino A ##cid Transport ##er [unused3] ( Chemical : Am ##ino A ##cid Transport ##ers ; Am ##ino A ##cid Channel ; Transport ##ers , Am ##ino A ##cid ; Am ##ino A ##cid Transport Systems ; Am ##ino A ##cid Transport Systems [ Chemical / In ##g ##red ##ient ] ; Am ##ino A ##cid Transport Pro ##tein ) [SEP]',
  'our_cross_cui': 'C0002518',
  'our_cross_cui_name': 'Amino Acid Sequence',
  'our_cross_cui_desc': '[CLS] Am ##ino A ##cid Se ##quence [unused3] ( Spa ##tial Concept , finding : Se ##quence ##s , Am ##ino A ##cid ; Am ##ino A ##cid Se ##quence ##s ; SE ##Q AM ##IN ##O AC ##ID ; Pro ##tein Se ##quence ; amino acid sequences ; SE ##Q AA ; Structure ##s , Primary Pro ##tein ; acid amino sequences ; amino acid sequence ; Primary Pro ##tein Structure ##s ; AM ##IN ##O AC ##ID SE ##Q ; Structure , Primary Pro ##tein ; AA SE ##Q ; Am ##ino acid sequence ; protein se ##quencing ; protein sequences ; Pro ##tein Structure ##s , Primary ; Se ##quence , Am ##ino A ##cid ; protein sequence ; AM ##IN ##O AC ##ID [SEP]'},
 {'mention_id': '28263181.25',
  'test_set_idx': 30385,
  'type': 'T170',
  'name': 'Prigogine theorem',
  'context': '[CLS] . 9 . Hence , calculation shows that , in respect to pure re ##spiration , the predominant f ##er ##ment ##ative metabolism decreases around 10 % the production of en ##tropy per mole of glucose consumed in cancer cells . We h ##y ##pot ##hes ##ize that increased f ##er ##mentation could allow cancer cells to accomplish the [unused1] P ##rig ##og ##ine theorem [unused2] of the trend to minimize the rate of production of en ##tropy . According the theorem , open cellular systems near the steady state could evolve to minimize the rates of en ##tropy production that may be reached by modified replica ##ting cells producing en ##tropy at low rate . Re ##mark ##ably , at CO ##2 concentrations above 93 [SEP]',
  'existing_cui': 'C0034673',
  'existing_cui_name': 'rapid eye movement',
  'existing_cui_desc': '[CLS] rapid eye movement [unused3] ( B ##iol ##og ##ic Fun ##ction : Movement , Rapid Eye ; Rapid Eye Movement ; Rapid Eye Movement ##s ; Eye Movement ##s , Rapid ; Movement ##s , Rapid Eye ; Eye Movement , Rapid ) [SEP]',
  'our_bi_cui': 'C1705273',
  'our_bi_cui_name': 'Mathematical Operator',
  'our_bi_cui_desc': '[CLS] Mathematical Opera ##tor [unused3] ( Intel ##lect ##ual Product : Opera ##tor ; Fun ##ction ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27760149.30',
  'test_set_idx': 15817,
  'type': 'T170',
  'name': 'biomedical entity search tool',
  'context': "[CLS] the response time ; however , they are not constantly updated , and thus produce out ##dated results . Further , most existing tools can not process sophisticated que ##ries such as searches for mutations that co - occur with que ##ry terms in the literature . To address these problems , we introduce B ##ES ##T , a [unused1] bio ##medical entity search tool [unused2] . B ##ES ##T returns , as a result , a list of 10 different types of bio ##medical entities including genes , diseases , drugs , targets , transcription factors , mi ##RNA ##s , and mutations that are relevant to a user ' s que ##ry . To the best of our knowledge , B ##ES ##T is [SEP]",
  'existing_cui': 'C2348167',
  'existing_cui_name': 'Search Engine',
  'existing_cui_desc': '[CLS] Search Engine [unused3] ( Intel ##lect ##ual Product : Search Program ; Search engines ; Engine , Search ; Search Too ##l ; Search Engine ##s ) [SEP]',
  'our_bi_cui': 'C0037589',
  'our_bi_cui_name': 'Software Tools',
  'our_bi_cui_desc': '[CLS] Software Too ##ls [unused3] ( Intel ##lect ##ual Product : Too ##l , Software ; Too ##ls , Software ; Software Too ##l ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27333847.33',
  'test_set_idx': 2654,
  'type': 'T170',
  'name': 'age assessment tool',
  'context': '[CLS] very strong at 0 . 98 ##0 ##2 ( boys 0 . 97 ##48 , girls 0 . 98 ##7 ##6 ) . The oral narrative component of the tool performed best ( R = 0 . 96 ##0 ##3 ) . Overall , 86 . 7 % of age estimates were within 1 year of the true age . The range of differences was - 1 . 43 to 3 . 92 years with a standard de ##viation of 0 . 77 years ( 9 . 24 months ) . The [unused1] age assessment tool [unused2] is a ho ##listic , simple and safe instrument that can be used to estimate age in refugee children with results comparable with radio ##logical methods currently used . [SEP]',
  'existing_cui': 'C1518848',
  'existing_cui_name': 'Pain Assessment Tool',
  'existing_cui_desc': '[CLS] Pain Assessment Too ##l [unused3] ( Intel ##lect ##ual Product ) [SEP]',
  'our_bi_cui': 'C1516602',
  'our_bi_cui_name': 'Research or Clinical Assessment Tool',
  'our_bi_cui_desc': '[CLS] Research or Clinical Assessment Too ##l [unused3] ( Intel ##lect ##ual Product : Clinical Assessment Too ##l ; Clinical or Research Assessment Too ##l ) [SEP]',
  'our_cross_cui': 'C0037589',
  'our_cross_cui_name': 'Software Tools',
  'our_cross_cui_desc': '[CLS] Software Too ##ls [unused3] ( Intel ##lect ##ual Product : Too ##l , Software ; Too ##ls , Software ; Software Too ##l ) [SEP]'},
 {'mention_id': '27297967.74',
  'test_set_idx': 1779,
  'type': 'T082',
  'name': 'Tyr920/Ser921',
  'context': '[CLS] ##nic kidney 29 ##3 - Ty ##r ##9 ##20 ##P ##he - and - Ser ##9 ##21 ##A ##la - MR ##P ##1 mutant ##s were similar to H ##E ##K - W ##T - MR ##P ##1 . Together , these results suggest that As ##n ##19 / As ##n ##23 g ##ly ##cos ##yla ##tion and [unused1] Ty ##r ##9 ##20 / Ser ##9 ##21 [unused2] p ##hos ##ph ##ory ##lation are responsible for altering the kinetic ##s of MR ##P ##1 - mediated As ( G ##S ) 3 transport . The kinetic ##s of As ( G ##S ) 3 transport by human em ##b ##ryo ##nic kidney 29 ##3 - As ##n ##19 / 23 ##G ##ln / Ty ##r ##9 [SEP]',
  'existing_cui': 'C1414863',
  'existing_cui_name': 'FUT2 gene',
  'existing_cui_desc': '[CLS] F ##UT ##2 gene [unused3] ( Ana ##tom ##ical Structure : GDP - L - f ##uc ##ose : beta - D - gal ##act ##os ##ide 2 - alpha - L - f ##uc ##osy ##lt ##ran ##s ##fer ##ase 2 ; alpha ( 1 , 2 ) F ##T ##2 ; secret ##or blood group alpha - 2 - f ##uc ##osy ##lt ##ran ##s ##fer ##ase ; SEC ##2 ; SEC ##RE ##TO ##R FA ##CT ##OR ; f ##uc ##osy ##lt ##ran ##s ##fer ##ase 2 ; se ##j ; alpha ( 1 , 2 ) f ##uc ##osy ##lt ##ran ##s ##fer ##ase ; F ##UC ##OS ##Y ##LT ##RA ##NS ##F ##ER ##AS ##E 2 ; secret ##or factor ; [SEP]',
  'our_bi_cui': 'C1519063',
  'our_bi_cui_name': 'Phosphorylation Site',
  'our_bi_cui_desc': '[CLS] Ph ##os ##ph ##ory ##lation Site [unused3] ( Spa ##tial Concept ) [SEP]',
  'our_cross_cui': 'C0002518',
  'our_cross_cui_name': 'Amino Acid Sequence',
  'our_cross_cui_desc': '[CLS] Am ##ino A ##cid Se ##quence [unused3] ( Spa ##tial Concept , finding : Se ##quence ##s , Am ##ino A ##cid ; Am ##ino A ##cid Se ##quence ##s ; SE ##Q AM ##IN ##O AC ##ID ; Pro ##tein Se ##quence ; amino acid sequences ; SE ##Q AA ; Structure ##s , Primary Pro ##tein ; acid amino sequences ; amino acid sequence ; Primary Pro ##tein Structure ##s ; AM ##IN ##O AC ##ID SE ##Q ; Structure , Primary Pro ##tein ; AA SE ##Q ; Am ##ino acid sequence ; protein se ##quencing ; protein sequences ; Pro ##tein Structure ##s , Primary ; Se ##quence , Am ##ino A ##cid ; protein sequence ; AM ##IN ##O AC ##ID [SEP]'},
 {'mention_id': '27623277.16',
  'test_set_idx': 12200,
  'type': 'T058',
  'name': 'procedures',
  'context': "[CLS] on the patient ' s clinical condition and patient preferences . This study describes the relative importance of several considerations in lung cancer treatment from the patient ' s perspective . A con ##join ##t preference experiment began by asking respond ##ents to imagine that they had just been diagnosed with lung cancer . Re ##sp ##ond ##ents then chose among [unused1] procedures [unused2] that differed regarding treatment m ##oda ##lities , the potential for treatment - related complications , the likelihood of re ##cu ##rrence , provider case volume , and distance needed to travel for treatment . Con ##join ##t analysis derived relative weights for these attributes . A total of 225 responses were analyzed . Re ##sp ##ond ##ents were most willing to [SEP]",
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '28125734.58',
  'test_set_idx': 26212,
  'type': 'T103',
  'name': 'miR-21-5p',
  'context': '[CLS] - 5 ##p was shown to be able to m ##od ##ulate me ##so ##the ##lin expression in mi ##RNA mimic experiments in a panel of ma ##li ##gnant and non - ma ##li ##gnant cell lines . Further mi ##RNA inhibitor experiments and l ##uc ##ifer ##ase ass ##ays in Me ##ro - 14 cells valid ##ated [unused1] mi ##R - 21 - 5 ##p [unused2] as a true regulator of me ##so ##the ##lin . Moreover , in v ##it ##ro experiments showed that treatment with mi ##R - 21 - 5 ##p mimic reduced proliferation of ma ##li ##gnant p ##le ##ural me ##so ##the ##lio ##ma cell lines . Alto ##get ##her , this work shows that the mi ##R - CA [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1999986',
  'our_bi_cui_name': 'MIRN21 microRNA , human',
  'our_bi_cui_desc': '[CLS] MI ##R ##N ##21 micro ##RNA , human [unused3] ( Chemical : micro ##RNA - 21 , human ; mi ##RNA - 21 , human ; h ##sa - mi ##r - 21 micro ##RNA ; mi ##R - 21 , human ; mi ##R ##21 , human ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '27645371.5',
  'test_set_idx': 12698,
  'type': 'T082',
  'name': 'perspectives',
  'context': '[CLS] Track ##ing children with acute l ##ymph ##ob ##lastic le ##uke ##mia who abandoned therapy : Experience , challenges , parental [unused1] perspectives [unused2] , and impact of treatment subsidies and intensified counseling Re ##fu ##sal for treatment and therapy abandonment are important reasons for un ##fa ##vor ##able outcome of childhood acute l ##ymph ##ob ##lastic le ##uke ##mia in resource - poor countries . The present study , conducted on children with acute l ##ymph ##ob ##lastic le ##uke ##mia whose treatment was abandoned , attempted to track all these children to as ##cer ##tain the causes and outcome of therapy abandonment / refusal . In order to improve outcome of acute l ##ymph ##ob ##lastic le ##uke ##mia , measures to prevent abandonment [SEP]',
  'existing_cui': 'C0040226',
  'existing_cui_name': 'Time Perception',
  'existing_cui_desc': '[CLS] Time Per ##ception [unused3] ( B ##iol ##og ##ic Fun ##ction : Time perception ; perception time ; Per ##ception , Time ; Time Per ##ception ##s ; Per ##ception ##s , Time ; time perception ; Time Per ##spective ) [SEP]',
  'our_bi_cui': 'C0030971',
  'our_bi_cui_name': 'Perception',
  'our_bi_cui_desc': '[CLS] Per ##ception [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : perception ##s ; perception ; Per ##ceived ; Per ##ception ##s ; perceived ; Per ##ce ##pt ##ual functions , un ##sp ##ec ##ified ; Per ##ce ##pt ##ual functions ; Per ##ception , function ; per ##ceiving ) [SEP]',
  'our_cross_cui': 'C0449911',
  'our_cross_cui_name': 'View',
  'our_cross_cui_desc': '[CLS] View [unused3] ( Spa ##tial Concept , attribute : ( View ##s ; views ; view ; View ##s for ; View ##s ) [SEP]'},
 {'mention_id': '28145492.28',
  'test_set_idx': 26788,
  'type': 'T038',
  'name': 'current',
  'context': '[CLS] - A , as well as H / O - B , excited o ##s ##cilla ##ting tube ##roi ##n ##fu ##ndi ##bula ##r do ##pa ##mine neurons , in ##ducing a re ##versible de ##pol ##aris ##ing switch from p ##has ##ic to ton ##ic discharge . The H ##y ##po ##cre ##tin / Or ##ex ##in - induced inward [unused1] current [unused2] under ##pin ##ning this effect was post - s ##yna ##ptic ( as it endured in the presence of te ##tro ##do ##to ##xin ) , appeared to be carried by a Na ( + ) - dependent trans ##ient receptor potential - like channel ( as it was blocked by 2 - AP ##B and was diminished by removal of extra [SEP]',
  'existing_cui': 'C0282444',
  'existing_cui_name': 'Current Biog-Obit',
  'existing_cui_desc': '[CLS] Current B ##io ##g - O ##bit [unused3] ( Intel ##lect ##ual Product ) [SEP]',
  'our_bi_cui': 'C0022023',
  'our_bi_cui_name': 'Ions',
  'our_bi_cui_desc': '[CLS] Ion ##s [unused3] ( Chemical , substance : ion ; Ion ##s [ Chemical / In ##g ##red ##ient ] ; Ion ; charged particles ; ions ) [SEP]',
  'our_cross_cui': 'C0162585',
  'our_cross_cui_name': 'Ion Transport',
  'our_cross_cui_desc': '[CLS] Ion Transport [unused3] ( B ##iol ##og ##ic Fun ##ction : ion transport ; Transport , Ion ) [SEP]'},
 {'mention_id': '27622027.20',
  'test_set_idx': 12093,
  'type': 'T017',
  'name': 'Fresh tumor - draining LN tissues',
  'context': '[CLS] , we showed CD ##16 ##9 expression on some CD ##8 ( + ) T l ##ymph ##ocytes in regional l ##ymph nodes and investigated the function and clinical relevance of CD ##16 ##9 ( + ) CD ##8 ( + ) T cells in tumor - draining l ##ymph nodes of color ##ec ##tal cancer patients . [unused1] Fresh tumor - draining L ##N tissues [unused2] from 39 randomly enrolled patients were assessed by flow c ##yt ##ome ##try for activation and differentiation of CD ##16 ##9 ( + ) CD ##8 ( + ) T cells and T cell - mediated killing of tumor cells . In total , 114 tumor - draining L ##N para ##ffin sections from color ##ec ##tal cancer patients [SEP]',
  'existing_cui': 'C1519521',
  'existing_cui_name': 'Fresh Tissue',
  'existing_cui_desc': '[CLS] Fresh T ##iss ##ue [unused3] ( Ana ##tom ##ical Structure : T ##iss ##ue , Fresh ) [SEP]',
  'our_bi_cui': 'C0475358',
  'our_bi_cui_name': 'Tumor tissue sample',
  'our_bi_cui_desc': '[CLS] Tu ##mor tissue sample [unused3] ( Ana ##tom ##ical Structure , specimen : Tu ##mour tissue ; Tu ##mor Sam ##ple ; samples tissue tumor ; tissue tumor ; tissue tumors ; Tu ##mour tissue sample ; tumor tissue ; T ##UM ##OR T ##IS ##SU ##E ; Tu ##mor tissue ; tissues tumor ; Tu ##mor T ##iss ##ue ; tissue t ##umour ##s ) [SEP]',
  'our_cross_cui': 'C0440747',
  'our_cross_cui_name': 'Lymph Node Tissue',
  'our_cross_cui_desc': '[CLS] L ##ymph No ##de T ##iss ##ue [unused3] ( Ana ##tom ##ical Structure , substance : L ##Y ##MP ##H NO ##DE ; L ##ymph node tissue ; L ##ymph G ##land ) [SEP]'},
 {'mention_id': '28220186.31',
  'test_set_idx': 28686,
  'type': 'T062',
  'name': 'genetic tool',
  'context': '[CLS] marker for L . stark ##ey ##i . However , gene - targeting frequencies were very low because non - ho ##mo ##log ##ous re ##comb ##ination is probably predominant in L . stark ##ey ##i . Gene ##tic engineering tools for L . stark ##ey ##i have not been sufficiently developed . In this study , we describe a new [unused1] genetic tool [unused2] and its application in L . stark ##ey ##i . To develop a highly efficient gene - targeting system for L . stark ##ey ##i , we constructed a series of mutant ##s by disrupt ##ing genes for L ##s ##K ##u ##70 ##p , L ##s ##K ##u ##80 ##p , and / or L ##s ##L ##ig ##4 ##p [SEP]',
  'existing_cui': 'C0017296',
  'existing_cui_name': 'gene therapy',
  'existing_cui_desc': '[CLS] gene therapy [unused3] ( Health Care Act ##ivity , procedure , treatment : Gene the ##rap ##ies ; d ##na therapy ; Gene Transfer Pro ##ced ##ure ; Molecular Biology , Gene Therapy ; Therapy , Gene ; THE ##R GE ##NE ; gene the ##rap ##ies ; Gene therapy ; Inter ##vention , Gene ##tic ; GE ##NE THE ##R ; therapy , gene ; Gene transfer ; GE ##NE ##TI ##C ; Gene Therapy ; d ##na the ##rap ##ies ; DNA Therapy ) [SEP]',
  'our_bi_cui': 'C0017393',
  'our_bi_cui_name': 'Genetic Markers',
  'our_bi_cui_desc': '[CLS] Gene ##tic Mark ##ers [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : genetic markers ; Gene ##tic marker ; MA ##R ##KE ##RS GE ##NE ##T ; GE ##NE ##T MA ##R ##KE ##R ; GE ##NE ##T MA ##R ##KE ##RS ; Mark ##ers , Gene ##tic ; Mark ##er , Gene ##tic ; MA ##R ##KE ##R GE ##NE ##T ; Gene ##tic markers ; genetic marker ; Gene ##tic Mark ##ers [ Chemical / In ##g ##red ##ient ] ; Gene ##tic marker , function ; Gene ##tic Mark ##er ) [SEP]',
  'our_cross_cui': 'C0017387',
  'our_cross_cui_name': 'Genetic Engineering',
  'our_cross_cui_desc': '[CLS] Gene ##tic Engineering [unused3] ( Research Act ##ivity : GE ##NE ##T E ##NG ##IN ##EE ##RI ##NG ; B ##iot ##ech ##nology , Gene ##tic Engineering ; E ##NG ##IN ##EE ##RI ##NG GE ##NE ##T ; Gene ##tic Inter ##vention ; engineering genetic ; engineering genetics ; IN ##TE ##R ##VE ##NT ##ION GE ##NE ##T ; engineer genetic ; genetic intervention ; Gene ##tically Engineer ##ed ; genetic engineer ; genetic engineering ; Inter ##vention , Gene ##tic ; GE ##NE ##T IN ##TE ##R ##VE ##NT ##ION ; Inter ##vention ##s , Gene ##tic ; Molecular Biology , Gene ##tic Engineering ; engineered genetic ; Gene ##tic Inter ##vention ##s ; Engineering , Gene ##tic ; Re ##comb ##ina ##nt DNA Technology [SEP]'},
 {'mention_id': '28272254.12',
  'test_set_idx': 30758,
  'type': 'T103',
  'name': 'therapeutic drugs',
  'context': '[CLS] ##tens ##ion patients : Bay ##esi ##an network meta - analysis Left vent ##ric ##ular h ##yper ##tro ##phy is commonly present in patients with h ##yper ##tens ##ion . According to the expert consensus document from American , an ##gio ##tens ##in - converting enzyme inhibitor and an ##gio ##tens ##in receptor block ##ers were recommended as 1st - line [unused1] therapeutic drugs [unused2] . However , none noticed the different efficacy between fat - soluble and selective β ##1 - receptor block ##ers and other β - block ##ers on re ##gression of Left vent ##ric ##ular h ##yper ##tro ##phy before . The aim of this analysis was to compare the efficacy of fat - soluble and selective β ##1 - receptor block [SEP]',
  'existing_cui': 'C0580398',
  'existing_cui_name': 'Therapeutic drug level - finding',
  'existing_cui_desc': '[CLS] The ##rap ##eu ##tic drug level - finding [unused3] ( Finding , finding , procedure : Finding of therapeutic drug level ; Drug level therapeutic ; drugs levels therapeutic ; The ##rap ##eu ##tic drug level ; therapeutic drug level ; drug levels therapeutic ; drug level therapeutic ) [SEP]',
  'our_bi_cui': 'C1611640',
  'our_bi_cui_name': 'Therapeutic agent',
  'our_bi_cui_desc': '[CLS] The ##rap ##eu ##tic agent [unused3] ( Chemical , substance , substance , product : therapeutic agent ) [SEP]',
  'our_cross_cui': 'C0013227',
  'our_cross_cui_name': 'Pharmaceutical Preparations',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##ce ##utical Pre ##par ##ations [unused3] ( Chemical , product , substance : Drug or me ##dic ##ament ; Drug preparation ; Me ##dication ##s ; Ph ##arma ##ce ##utical / bio ##log ##ic product ; drug / agent ; medicinal products ; medications ; medicinal product ; Ph ##arma ##ce ##uti ##c Pre ##par ##ations ; Me ##dication ; Pre ##par ##ations , Ph ##arma ##ce ##uti ##c ; Drugs ; pharmaceutical preparation ; drug ; PR ##EP ##AR ##AT ##ION ##S P ##HA ##R ##M ; Ph ##arma ##ce ##utical Products ; Products , Ph ##arma ##ce ##utical ; medication ; General drug type ; Drug product ; Medicine ##s ; Drug Product ; drugs ; Medicine ; P ##HA ##R ##M [SEP]'},
 {'mention_id': '27394918.15',
  'test_set_idx': 4520,
  'type': 'T103',
  'name': 'Sm',
  'context': '[CLS] order to assess occupational exposure level of 15 rare earth elements and identify the associated influence , we used in ##duct ##ively coupled plasma mass s ##pect ##rome ##try based on closed - vessel microwave - assisted wet dig ##est ##ion procedure to de ##ter ##minate the concentration of Y , La , Ce , P ##r , N ##d , [unused1] S ##m [unused2] , E ##u , G ##d , T ##b , D ##y , Ho , E ##r , T ##m , Y ##b and Lu in u ##rina ##ry samples obtained from workers producing ultra ##fine and na ##no ##par ##tic ##les containing c ##eri ##um and la ##nt ##han ##um oxide . The results suggest that La and Ce were [SEP]',
  'existing_cui': 'C1419249',
  'existing_cui_name': 'RAI1 gene',
  'existing_cui_desc': '[CLS] RA ##I ##1 gene [unused3] ( Ana ##tom ##ical Structure : Re ##tino ##ic A ##cid In ##duced 1 Gene ; R ##ET ##IN ##O ##IC AC ##ID - IN ##D ##UC ##ED GE ##NE 1 ; K ##IA ##A ##18 ##20 ; RA ##I ##1 Gene ; MG ##C ##12 ##8 ##24 ; D ##K ##F ##Z ##P ##43 ##4 ##A ##13 ##9 ; SMS ; re ##tino ##ic acid induced 1 ; R ##ET ##IN ##O ##IC AC ##ID - IN ##D ##UC ##IB ##LE 1 ) [SEP]',
  'our_bi_cui': 'C0587072',
  'our_bi_cui_name': 'Sm antibody',
  'our_bi_cui_desc': '[CLS] S ##m anti ##body [unused3] ( Chemical , substance : Anti - S ##m / R ##NP Anti ##body ; Anti SM anti ##body ( substance ) [ Am ##bi ##guous ] ; s ##mith anti ##body ; Anti - Smith Anti ##body ; antibodies s ##mith ; anti s ##m antibodies ; S ##m - Smith anti ##body ; Smith anti ##body ; anti - s ##m anti ##body ; Anti - S ##m - R ##NP Anti ##body ; Anti Smith anti ##body ; Anti SM anti ##body ; Blood group anti ##body Smith ; s ##m anti ##body ; Anti - S ##m Anti ##body ; anti s ##mith anti ##body ; antibodies s ##m ) [SEP]',
  'our_cross_cui': 'C0036147',
  'our_cross_cui_name': 'Samarium',
  'our_cross_cui_desc': '[CLS] Sam ##arium [unused3] ( Chemical , substance : sa ##mar ##ium ; SA ##MA ##RI ##UM ; Sam ##arium [ Chemical / In ##g ##red ##ient ] ) [SEP]'},
 {'mention_id': '27580517.23',
  'test_set_idx': 10633,
  'type': 'T058',
  'name': 'administration of',
  'context': '[CLS] flow measurements were com ##puted pre - and post - ace ##ta ##zo ##lam ##ide to assess c ##ere ##bro ##vas ##cular reserve . 3D spiral pseudo - continuous art ##erial spin label ##ing data were successfully reconstructed in all 13 cases . In 11 patients , cerebral blood flow increased 2 . 8 % to 93 . 2 % after [unused1] administration of [unused2] ace ##ta ##zo ##lam ##ide . In the two remaining patients , cerebral blood flow decreased by 2 . 4 to 6 . 0 % after ace ##ta ##zo ##lam ##ide . The group average change in cerebral blood flow due to ace ##ta ##zo ##lam ##ide was approximately 25 . 0 % and individual changes were statistical ##ly significant ( [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C1533734',
  'our_bi_cui_name': 'Administration procedure',
  'our_bi_cui_desc': '[CLS] Administration procedure [unused3] ( Health Care Act ##ivity , Pro ##ced ##ure , procedure : administration ; Ad ##mini ##ster ##ing ; Administration ; Treatment ) [SEP]',
  'our_cross_cui': 'C3469597',
  'our_cross_cui_name': 'Administration of medication',
  'our_cross_cui_desc': '[CLS] Administration of medication [unused3] ( Health Care Act ##ivity , procedure , treatment : administration drugs ; medication administration ; Me ##dication treatment ; administration of medication ; treatment medication ; medications treatments ; administration of medications ; Me ##dication Treatment ; Me ##dication administration ; drug administration ; r ##no ##x administer medication treatment ; medication management ; medication treatment ; Me ##dication administration treatments and procedures ; Drugs - - Administration ; Me ##dication Administration ; giving medications ; medications treatment ; Administration of drug or me ##dic ##ament ; giving medication ; Drug administration ; Giving medication ; medication treatments ) [SEP]'},
 {'mention_id': '27172917.20',
  'test_set_idx': 163,
  'type': 'T170',
  'name': 'clinical',
  'context': '[CLS] c ##yt ##ology The predict ##ors of ma ##li ##gna ##ncy are important for the decision of appropriate management in nod ##ules with at ##y ##pia of und ##eter ##mined significance / f ##oll ##icular les ##ion of und ##eter ##mined significance ( AU ##S / FL ##US ) . Our aim was to determine the ultra ##son ##ographic ##al , [unused1] clinical [unused2] , and bio ##chemical predict ##ors of ma ##li ##gna ##ncy in these patients . A total of 42 ##7 patients with c ##yt ##ological ##ly Beth ##es ##da Category III ( AU ##S / FL ##US ) thy ##roid nod ##ules were included in this retrospective study . We divided the nod ##ules into two subgroup ##s according to the his [SEP]',
  'existing_cui': 'C3272565',
  'existing_cui_name': 'Clinical Lot',
  'existing_cui_desc': '[CLS] Clinical Lot [unused3] ( Intel ##lect ##ual Product : Clinical ) [SEP]',
  'our_bi_cui': 'C3540840',
  'our_bi_cui_name': 'Sign or Symptom',
  'our_bi_cui_desc': '[CLS] Sign or S ##ym ##pt ##om [unused3] ( Finding : Sign ##s or S ##ym ##pt ##oms ; Clinical Sign ##s ; C ##L ##IN ##IC ##AL F ##IN ##DI ##NG ; C ##L ##IN ##IC ##AL S ##IG ##NS ; DE ##SC ; Clinical Finding ) [SEP]',
  'our_cross_cui': 'C0683956',
  'our_cross_cui_name': 'predictive factor',
  'our_cross_cui_desc': '[CLS] predict ##ive factor [unused3] ( Intel ##lect ##ual Product : predict ##ive variables ; factors predict ##ive ; factor predict ##ive ; Disease Pre ##dict ##ive Factor ) [SEP]'},
 {'mention_id': '28338634.28',
  'test_set_idx': 32477,
  'type': 'T074',
  'name': 'working',
  'context': "[CLS] ##ty ##l ##cho ##lines ##tera ##se and they were used for the measurement anti - Alzheimer ' s drug gal ##ant ##amine and car ##ba ##mate p ##est ##icide car ##bo ##fu ##ran with limit of detection 1 . 5 µ ##M and 20 n ##M , respectively . All measurements were carried out using screen - printed sensor with carbon [unused1] working [unused2] , silver reference , and carbon auxiliary electro ##de . Standard El ##lman ' s ass ##ay was used for valid ##ation measurement of both inhibitor ##s . Part of this work was the elimination of re ##versible inhibitor ##s represented by gal ##ant ##amine from the active site of Ace ##ty ##l ##cho ##lines ##tera ##se . For this purpose , [SEP]",
  'existing_cui': 'C1563350',
  'existing_cui_name': 'Working animal',
  'existing_cui_desc': '[CLS] Working animal [unused3] ( E ##uka ##ryo ##te : Working ) [SEP]',
  'our_bi_cui': 'C0025080',
  'our_bi_cui_name': 'Medical Devices',
  'our_bi_cui_desc': '[CLS] Medical Devi ##ces [unused3] ( Medical Devi ##ce , physical device , physical object : Clinical equipment ; B ##io ##medical device ; DE ##VI ##CE ME ##D ; ME ##D DE ##VI ##CE ##S ; ME ##D DE ##VI ##CE ; bio ##medical devices ; Devi ##ces , Medical ; Medical devices ; General bio ##medical device ; DE ##VI ##CE , GE ##NE ##RA ##L ME ##DI ##CA ##L ; B ##io ##medical equipment ; Devi ##ce , Medical ; Hospital , medical AND / OR surgical equipment ( physical device ) ; bio ##medical equipment ; medical equipment ; Medical equipment ; DE ##VI ##CE ##S ME ##D ; DE ##VI ##CE ; DE ##VI ##CE ##S ; medical device ; medical devices [SEP]',
  'our_cross_cui': 'C0013812',
  'our_cross_cui_name': 'electrode',
  'our_cross_cui_desc': '[CLS] electro ##de [unused3] ( Medical Devi ##ce , physical object : El ##ec ##tro ##des ; El ##ec ##tro ##de , device ; El ##ec ##tro ##des , ( e . g . , a ##p ##nea monitor ) , per pair ; E ##LE ##CT ##RO ##DE ; electro ##des ; El ##ec ##tro ##des , pair ; El ##ec ##tro ##de ) [SEP]'},
 {'mention_id': '28208888.52',
  'test_set_idx': 28363,
  'type': 'T092',
  'name': 'genetic counselling clinics',
  'context': '[CLS] This was the first study to report the presence of H ##b ##E among the tribal ##s working in the tea gardens of Assam . Based on the present findings , sick ##le cell an ##ae ##mia and β - th ##ala ##ssa ##emia were major health problem for the tribal ##s working in the tea gardens of Assam . Pro ##per diagnostic facilities for ha ##em ##og ##lo ##bino ##pathy and th ##ala ##ssa ##emia should be established in these areas , including establishment of ha ##em ##og ##lo ##bino ##pathy and th ##ala ##ssa ##emia database collection , ha ##ema ##to ##logical analysis laboratories , [unused1] genetic counsel ##ling clinics [unused2] , pre ##nat ##al diagnosis centres and neon ##ata ##l screening centres . [SEP]',
  'existing_cui': 'C0599986',
  'existing_cui_name': 'Encounter due to genetic counseling',
  'existing_cui_desc': '[CLS] En ##co ##unt ##er due to genetic counseling [unused3] ( Finding , context - dependent category , situation : here ##dit ##y ; counseling ; [ V ] Gene ##tic counseling ; advice ; genetic ; [ V ] Gene ##tic counsel ##ling ; Gene ##tic counseling ; Gene ##tic counsel ##ling ; genetic ; counseling ) [SEP]',
  'our_bi_cui': 'C0017382',
  'our_bi_cui_name': 'Genetic Counseling',
  'our_bi_cui_desc': '[CLS] Gene ##tic Counsel ##ing [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : genetic counsel ##ling ; Gene ##tic counseling individual each 15 minutes ; Counsel ##ing , Gene ##tic ; Counsel ##ing ; genetic ; Gene ##tic counseling ; g ##nt ##k c ##ns ##ln ##g n ##d ##v ##dl e ##a . 15 min ; Counsel ##ling ; genetic ; genetic counseling ; Gene ##tic counsel ##ling ; GE ##NE ##T CO ##UN ##SE ##L ##ING ; CO ##UN ##SE ##L ##ING GE ##NE ##T ; Gene ##tic Counsel ##ling ) [SEP]',
  'our_cross_cui': 'C0442592',
  'our_cross_cui_name': 'Clinic',
  'our_cross_cui_desc': '[CLS] Clinic [unused3] ( Organization , environment : clinic ; C ##L ##IN ##IC - WA ##L ##K IN , O ##TH ##ER ; Clinic ##s ; clinics ) [SEP]'},
 {'mention_id': '27273231.14',
  'test_set_idx': 1195,
  'type': 'T074',
  'name': 'Enseal',
  'context': '[CLS] sealing pulmonary art ##eries with the En ##sea ##l tissue - sealing device . P ##ul ##mona ##ry art ##eries from be ##ag ##le dogs ( mean body weight 13 . 1 kg , range 10 . 5 - 15 . 4 kg ) were divided into 3 groups according to the in - v ##ivo sealing method used ( [unused1] En ##sea ##l [unused2] , l ##iga ##tion , and pro ##ximal l ##iga ##tion plus di ##stal En ##sea ##l ) and extracted to evaluate the pressure tolerance up to 75 mm H ##g at the sealed end . A left lower lobe ##ct ##omy was performed to evaluate chronic - phase du ##rab ##ility of the sealed s ##tum ##ps in a survival [SEP]',
  'existing_cui': 'C0262214',
  'existing_cui_name': 'Claustral amygdalar area',
  'existing_cui_desc': '[CLS] Claus ##tral am ##y ##g ##dal ##ar area [unused3] ( Ana ##tom ##ical Structure : Claus ##tr ##um di ##ff ##usa ; c ##laus ##tr ##um di ##ff ##usa ; c ##laus ##tral am ##y ##g ##dal ##oid area ; V ##ent ##ral portion of c ##laus ##tr ##um ; Area c ##laus ##tral ##is am ##y ##g ##dal ##ae ; C ##l ##A ; end ##op ##iri ##form nucleus ; ventral portion of c ##laus ##tr ##um ; Claus ##tr ##um par ##vu ##m ; c ##laus ##tral am ##y ##g ##dal ##ar area ; end ##op ##iri ##form c ##laus ##tr ##um ; V ##ent ##ral c ##laus ##tr ##um ; c ##laus ##tr ##um par ##vu ##m ; Claus ##tral am ##y ##g [SEP]',
  'our_bi_cui': 'C0038969',
  'our_bi_cui_name': 'Surgical sutures',
  'our_bi_cui_desc': '[CLS] Sur ##gical su ##tures [unused3] ( Medical Devi ##ce , physical object , treatment : Su ##tures ; S ##UT ##UR ##ES ; su ##ture ; S ##UT ##UR ##E ; Su ##ture material ; Su ##ture - object ; Sur ##gical Su ##tures ; Su ##ture ; Su ##tures , Sur ##gical ; surgical su ##ture ; Su ##tur ##ing ; su ##tures ; Sur ##gical su ##ture ; Sur ##gical su ##ture , device ; Su ##ture material , device ; surgical su ##tures ) [SEP]',
  'our_cross_cui': 'C0025080',
  'our_cross_cui_name': 'Medical Devices',
  'our_cross_cui_desc': '[CLS] Medical Devi ##ces [unused3] ( Medical Devi ##ce , physical device , physical object : Clinical equipment ; B ##io ##medical device ; DE ##VI ##CE ME ##D ; ME ##D DE ##VI ##CE ##S ; ME ##D DE ##VI ##CE ; bio ##medical devices ; Devi ##ces , Medical ; Medical devices ; General bio ##medical device ; DE ##VI ##CE , GE ##NE ##RA ##L ME ##DI ##CA ##L ; B ##io ##medical equipment ; Devi ##ce , Medical ; Hospital , medical AND / OR surgical equipment ( physical device ) ; bio ##medical equipment ; medical equipment ; Medical equipment ; DE ##VI ##CE ##S ME ##D ; DE ##VI ##CE ; DE ##VI ##CE ##S ; medical device ; medical devices [SEP]'},
 {'mention_id': '28223077.17',
  'test_set_idx': 28901,
  'type': 'T082',
  'name': 'surrounding sites',
  'context': '[CLS] . In order to understand the influence of Municipal solid waste land ##fill on p ##oly ##ch ##lor ##inated di ##ben ##zo - p - di ##ox ##ins and p ##oly ##ch ##lor ##inated di ##ben ##zo ##fu ##ran ##s ( PC ##D ##D / F ##s ) in surrounding atmosphere , 42 ambient air samples were collected and analyzed from [unused1] surrounding sites [unused2] , background site , up ##wind site and down ##wind site of a Municipal solid waste land ##fill in East China . The results of present study were summarized as follows . ( 1 ) The total concentrations of PC ##D ##D / F ##s ( [UNK] PC ##D ##D / F ##s ) in ambient air from surrounding sites , [SEP]',
  'existing_cui': 'C1282914',
  'existing_cui_name': 'Surrounding',
  'existing_cui_desc': '[CLS] Sur ##rou ##nding [unused3] ( Spa ##tial Concept , qualifier value , qualifier value : C ##ir ##cum ##scribed ; Sur ##rou ##nd ; c ##ir ##cum ##scribed ; surrounding ; surrounded ; Sur ##rou ##nded ; surroundings ; surrounds ; surround ) [SEP]',
  'our_bi_cui': 'C1282914',
  'our_bi_cui_name': 'Surrounding',
  'our_bi_cui_desc': '[CLS] Sur ##rou ##nding [unused3] ( Spa ##tial Concept , qualifier value , qualifier value : C ##ir ##cum ##scribed ; Sur ##rou ##nd ; c ##ir ##cum ##scribed ; surrounding ; surrounded ; Sur ##rou ##nded ; surroundings ; surrounds ; surround ) [SEP]',
  'our_cross_cui': 'C0205145',
  'our_cross_cui_name': 'Site',
  'our_cross_cui_desc': '[CLS] Site [unused3] ( Spa ##tial Concept , attribute , qualifier value : Sites ) [SEP]'},
 {'mention_id': '27794381.97',
  'test_set_idx': 17237,
  'type': 'T103',
  'name': '[ Ca ( 2 + ) ] i',
  'context': '[CLS] ##cy ##st cells in g ##uin ##ea pig with ch ##ole ##ster ##ol calculus . The d ##ys ##function of gal ##l ##bla ##dder contraction gives rise to the disorders of mobility signal trans ##duction system in ch ##ole ##cy ##st smooth muscle cells , including low content of plasma ch ##ole ##cy ##sto ##kini ##n and [unused1] [ C ##a ( 2 + ) ] i [unused2] in ch ##ole ##cy ##st cells , abnormal protein and m ##RNA of G ##s , G ##i and Cap . Em ##od ##in can enhance the contract ##ibility of gal ##l ##bla ##dder and alleviate ch ##ole ##sta ##sis by regulating plasma ch ##ole ##cy ##sto ##kini ##n levels , [ C ##a ( 2 + ) [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C1155431',
  'our_bi_cui_name': 'positive regulation of cytosolic calcium ion concentration',
  'our_bi_cui_desc': '[CLS] positive regulation of c ##yt ##oso ##lic calcium ion concentration [unused3] ( B ##iol ##og ##ic Fun ##ction : elevation of calcium ion concentration in c ##yt ##oso ##l ; c ##yt ##oso ##lic calcium ion concentration elevation ; c ##yt ##op ##las ##mic calcium ion concentration elevation ; elevation of c ##yt ##oso ##lic calcium ion concentration ; elevation of c ##yt ##op ##las ##mic calcium ion concentration ; elevation of calcium ion concentration in c ##yt ##op ##las ##m ) [SEP]',
  'our_cross_cui': 'C0596235',
  'our_cross_cui_name': 'Calcium ion',
  'our_cross_cui_desc': '[CLS] Cal ##ci ##um ion [unused3] ( Chemical : Cal ##ci ##um Cat ##ion ; free calcium ; CA + 2 ; ion calcium ; Cal ##ci ##um cat ##ion ; Free Cal ##ci ##um ; calcium ions ; C ##a ##2 + ; C ##a + + ; Cal ##ci ##um Ion ##s ; C ##a 2 + ; calcium ion ; CA ##LC ##I ##UM CA ##TI ##ON ; Cal ##ci ##um Ion ; calcium cat ##ion ; Cal ##ci ##um 2 + ; Ion ##ized Cal ##ci ##um ) [SEP]'},
 {'mention_id': '28433910.5',
  'test_set_idx': 35704,
  'type': 'T103',
  'name': 'Spent sulphite liquor',
  'context': '[CLS] Su ##cci ##nic acid production by im ##mo ##bil ##ized cultures using spent su ##l ##phi ##te liquor as f ##er ##mentation medium [unused1] S ##pent su ##l ##phi ##te liquor [unused2] was used as carbon source for the production of su ##cci ##nic acid using im ##mo ##bil ##ized cultures of Act ##ino ##ba ##ci ##llus su ##cci ##no ##gene ##s and Ba ##s ##fia su ##cci ##nic ##ip ##rod ##uce ##ns on two different supports , del ##ign ##ified cell ##ulos ##ic material and al ##gin ##ate beads . Fed - batch im ##mo ##bil ##ized cultures with A . su ##cci ##no ##gene ##s in al ##gin ##ates resulted in higher sugar to su ##cci ##nic acid conversion yield ( 0 . 81 [SEP]',
  'existing_cui': 'C0038750',
  'existing_cui_name': 'Sulfites',
  'existing_cui_desc': '[CLS] Sul ##fit ##es [unused3] ( Chemical , substance : In ##or ##gan ##ic Sul ##fit ##es ; Sul ##fit ##e ; Sul ##fit ##es , In ##or ##gan ##ic ; Sul ##fit ##es [ Chemical / In ##g ##red ##ient ] ; su ##lf ##ites ; Sul ##phi ##te ; su ##lf ##ite ; su ##l ##phi ##tes ; Sul ##fit ##e salt ; Sul ##phi ##te salt ; S ##U ##LF ##IT ##ES IN ##OR ##G ; IN ##OR ##G S ##U ##LF ##IT ##ES ; su ##l ##phi ##te ) [SEP]',
  'our_bi_cui': 'C0982424',
  'our_bi_cui_name': 'Sulfurated lime solution',
  'our_bi_cui_desc': "[CLS] Sul ##fu ##rated lime solution [unused3] ( Chemical , substance : S ##U ##LF ##UR ##AT ##ED L ##IM ##E S ##OL ##N ; sulfur ##ated lime solution ; Lim ##e Sol ##ution , Sul ##fu ##rated ; Sul ##fu ##rated Lim ##e Sol ##ution ; V ##lem ##in ##ck ##x ' s lot ##ion ; Sul ##ph ##ura ##ted lime solution ) [SEP]",
  'our_cross_cui': 'C0010454',
  'our_cross_cui_name': 'Culture Media',
  'our_cross_cui_desc': '[CLS] Culture Media [unused3] ( Chemical , substance : Growth Medium ; growth media ; Media , Culture ; Culture media ; Culture medium ; culture medium ; culture media ; Culture Medium ; culture medium ##s ; culture media ##s ) [SEP]'},
 {'mention_id': '27430240.67',
  'test_set_idx': 5393,
  'type': 'T038',
  'name': 'Adenosine triphosphate hydrolysis',
  'context': '[CLS] significantly up ##re ##gu ##lated CD ##39 and CD ##7 ##3 expression ( P < 0 . 01 ) . E ##2 ##F transcription factor 1 and CR ##E ##B induced CD ##39 and CD ##7 ##3 expression , and were up ##re ##gu ##lated by ad ##eno ##sin ##e and C ##GS ##21 ##6 ##80 . [unused1] Aden ##os ##ine trip ##hos ##phate h ##ydro ##lysis [unused2] and ad ##eno ##sin ##e generation were in ##hibit ##ed by the knock ##down of E ##2 ##F transcription factor 1 or CR ##E ##B , and were accelerated in the presence of C ##GS ##21 ##6 ##80 . Based on these results , it can be in ##ferred that ad ##eno ##sin ##e , the ad ##eno [SEP]',
  'existing_cui': 'C0001480',
  'existing_cui_name': 'Adenosine Triphosphate',
  'existing_cui_desc': "[CLS] Aden ##os ##ine Trip ##hos ##phate [unused3] ( Chemical , ATP , Te ##tra ##hy ##dr ##ogen Trip ##hos ##phate , substance , te ##tra ##hy ##dr ##ogen trip ##hos ##phate : AD ##EN ##OS ##IN ##E T ##RI ##P ##H ##OS ##P ##HA ##TE ; ATP ; Aden ##yl ##py ##rop ##hos ##phate ; Aden ##os ##ine Trip ##hos ##phate [ Chemical / In ##g ##red ##ient ] ; Aden ##os ##ine 5 ' - trip ##hos ##phate ; Aden ##os ##ine 5 ' - ; ATP - Aden ##os ##ine trip ##hos ##phate ; Aden ##os ##ine 5 ' - Trip ##hos ##ph ##ori ##c A ##cid ; Aden ##os ##ine trip ##hos ##phate ; Aden ##os ##ine - 5 ' - trip ##hos [SEP]",
  'our_bi_cui': 'C0001473',
  'our_bi_cui_name': 'Adenosine Triphosphatases',
  'our_bi_cui_desc': '[CLS] Aden ##os ##ine Trip ##hos ##pha ##tase ##s [unused3] ( Chemical , substance : Aden ##os ##ine Trip ##hos ##pha ##tase ##s [ Chemical / In ##g ##red ##ient ] ; ATP ##ases ; ad ##eno ##sin ##et ##rip ##hos ##pha ##tase ; Aden ##os ##inet ##rip ##hos ##pha ##tase ; at ##pas ##e ; ad ##eno ##sin ##e trip ##hos ##pha ##tase ; Aden ##os ##ine Trip ##hos ##pha ##tase ; ATP p ##hos ##ph ##oh ##ydro ##lase ; Aden ##os ##ine trip ##hos ##pha ##tase ; ATP ##ase ; ATP H ##ydro ##lase ; ATP mon ##op ##hos ##pha ##tase ; Aden ##yl ##py ##rop ##hos ##pha ##tase ) [SEP]',
  'our_cross_cui': 'C1510699',
  'our_cross_cui_name': 'ATP Hydrolysis',
  'our_cross_cui_desc': '[CLS] ATP H ##ydro ##lysis [unused3] ( B ##iol ##og ##ic Fun ##ction : ATP degradation ; ATP cat ##ab ##olis ##m ; obsolete ATP cat ##ab ##olic process ; ATP h ##ydro ##lysis ; ATP breakdown ) [SEP]'},
 {'mention_id': '27580517.6',
  'test_set_idx': 10616,
  'type': 'T058',
  'name': 'non - Cartesian cylindrically - distributed spiral 3D pseudo - continuous arterial spin labeling magnetic resonance imaging',
  'context': '[CLS] Assessment of cerebral blood per ##fusion reserve with ace ##ta ##zo ##lam ##ide using 3D spiral AS ##L magnetic resonance imaging : Preliminary experience in p ##ediatric patients To demonstrate the clinical f ##eas ##ibility of a new [unused1] non - Car ##tes ##ian cylindrical ##ly - distributed spiral 3D pseudo - continuous art ##erial spin label ##ing magnetic resonance imaging [unused2] pulse sequence in p ##ediatric patients in q ##uant ##ifying cerebral blood flow response to an ace ##ta ##zo ##lam ##ide v ##as ##od ##ila ##tor challenge . magnetic resonance imaging exams were performed on two 3 Te ##sla Philips In ##gen ##ia systems using 32 channel head coil array ##s . After local institutional review board approval , the 3D spiral - based [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0024485',
  'our_bi_cui_name': 'Magnetic Resonance Imaging',
  'our_bi_cui_desc': '[CLS] Ma ##gnetic Re ##son ##ance I ##maging [unused3] ( Health Care Act ##ivity , MR ##I , procedure , qualifier value : MR - Ma ##gnetic resonance ; MR TO ##MO ##GR ; Ma ##gnetic resonance technique ; magnetic resonance imaging ; N ##MR I ##maging ; Ma ##gnetic resonance imaging procedure ; St ##ead ##y - State Free Pre ##cession MR ##I ; Ma ##gnetic resonance : [ imaging ] or [ study ] ; imaging magnetic nuclear resonance ; Tom ##ography , MR ; MR ##I Sc ##an ; N ##MR - Nuclear magnetic resonance ; Z ##E ##U ##GM ##AT ##O ##GR ; TO ##MO ##GR MR ; N ##MR TO ##MO ##GR ; Nuclear magnetic resonance ; Medical I ##maging , Ma [SEP]',
  'our_cross_cui': 'C3891302',
  'our_cross_cui_name': 'Arterial Spin Labeling Magnetic Resonance Imaging',
  'our_cross_cui_desc': '[CLS] Arte ##rial Spin Label ##ing Ma ##gnetic Re ##son ##ance I ##maging [unused3] ( Health Care Act ##ivity : AS ##L f ##MR ##I ; Arte ##rial Spin Label ##ing MR ##I ; AR ##TE ##RI ##AL SP ##IN LA ##BE ##L ##ING F ##UN ##CT ##ION ##AL MR ##I ) [SEP]'},
 {'mention_id': '27481714.73',
  'test_set_idx': 7891,
  'type': 'T017',
  'name': 'principal cells',
  'context': '[CLS] dependent ##ly suppressed by r ##uth ##eni ##um red , a put ##ative block ##er of e ##pit ##hel ##ial C ##a ( 2 + ) channels and calcium - activated chloride conduct ##ance . Finally , we discover messenger RNA for both T ##RP ##V ##6 and T ##ME ##M ##16 ##A in the rat e ##pid ##idy ##mis and show that their proteins co ##loc ##ali ##ze in the a ##pical membrane of principal cells . Collective ##ly , these data provide evidence for a coupling mechanism between T ##RP ##V ##6 and T ##ME ##M ##16 ##A in [unused1] principal cells [unused2] that may play an important role in the regulation of calcium home ##ost ##asis in the e ##pid ##idy ##mis . [SEP]',
  'existing_cui': 'C0401925',
  'existing_cui_name': 'Teaching principal',
  'existing_cui_desc': '[CLS] Teaching principal [unused3] ( Professional or O ##cc ##upation ##al Group , occupation : Principal ; principal ##s ; principal ) [SEP]',
  'our_bi_cui': 'C0017471',
  'our_bi_cui_name': 'Germ Cells',
  'our_bi_cui_desc': '[CLS] G ##er ##m Cell ##s [unused3] ( Ana ##tom ##ical Structure : Cell , G ##er ##m ; reproductive cells ; Rep ##rod ##uctive Cell ##s ; Ha ##p ##lo ##id g ##er ##m cell ; Game ##te ; Cell , G ##er ##m - Line ; cells g ##er ##m ; Sexual Cell ; cell sex ; Cell ##s , G ##er ##m ; cell g ##er ##m ; G ##er ##m Cell ; G ##er ##m cells ; G ##er ##m Line Cell ##s ; cells sex ; G ##er ##m - Line Cell ##s ; Ha ##p ##lo ##id n ##uc ##lea ##ted cell ; sexual cell ; Cell ##s , G ##er ##m - Line ; Game ##tes ; g ##er ##m [SEP]',
  'our_cross_cui': 'C0007634',
  'our_cross_cui_name': 'Cells',
  'our_cross_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]'},
 {'mention_id': '28331620.12',
  'test_set_idx': 32167,
  'type': 'T033',
  'name': 'unmedicated',
  'context': '[CLS] ? The impact of stress on anger and aggression in Border ##line Personal ##ity Di ##sor ##der and At ##ten ##tion De ##fic ##it H ##yper ##act ##ivity Di ##sor ##der has not been thoroughly investigated . The goal of this study was to investigate different aspects of anger and aggression in patients with these disorders . Twenty - nine [unused1] un ##med ##ica ##ted [unused2] female Border ##line Personal ##ity Di ##sor ##der patients , 28 At ##ten ##tion De ##fic ##it H ##yper ##act ##ivity Di ##sor ##der patients and 30 healthy controls completed self - reports measuring trait anger , aggression and emotion regulation capacities . A modified version of the Point Sub ##traction A ##gg ##ression Para ##di ##g ##m and a [SEP]',
  'existing_cui': 'C0162396',
  'existing_cui_name': 'Unmedicated IUDs',
  'existing_cui_desc': '[CLS] Un ##med ##ica ##ted I ##U ##Ds [unused3] ( Medical Devi ##ce : Un ##med ##ica ##ted I ##U ##D ; I ##U ##D , Un ##med ##ica ##ted ) [SEP]',
  'our_bi_cui': 'C0332155',
  'our_bi_cui_name': 'Did not receive therapy or drug for',
  'our_bi_cui_desc': '[CLS] Did not receive therapy or drug for [unused3] ( Finding , context ##ual qualifier , qualifier value : N ##G - did not receive therapy or drug for ; un ##tre ##ated ; Did not receive therapy or drug for ( context ##ual qualifier ) ; Not treated for ; did not receive therapy or drug for ; N ##G ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28192171.57',
  'test_set_idx': 27916,
  'type': 'T062',
  'name': 'work',
  'context': '[CLS] art ##hr ##op ##athy , but also specific signatures of the disease , like presence of am ##yl ##oids and peculiar ca ##l ##ci ##fication ##s . Our analyses provide a unified picture of Al ##ka ##pton ##uria cart ##ila ##ge , shed ##ding a new light on the disease and opening new perspectives . O ##ch ##ron ##otic pig ##ment is a hall ##mark of Al ##ka ##pton ##uria and responsible of tissue de ##gene ##ration . Convention ##al bio - ass ##ays have not yet clarified its composition and its structural relationship with am ##yl ##oids . The present [unused1] work [unused2] proposes new strategies for filling the aforementioned gap that en ##com ##pass the integration of new analytical approaches with standardized analyses . [SEP]',
  'existing_cui': 'C3642250',
  'existing_cui_name': 'Work Including Work Around the House',
  'existing_cui_desc': '[CLS] Work Including Work Around the House [unused3] ( Intel ##lect ##ual Product , includes work both outside the home and daily ch ##ores , including work around the house : Work ; Normal work ) [SEP]',
  'our_bi_cui': 'C0242481',
  'our_bi_cui_name': 'Research Activities',
  'our_bi_cui_desc': '[CLS] Research Activities [unused3] ( Research Act ##ivity : research ; Research Act ##ivity ; activity research ; Research ; Activities , Research ; Act ##ivity , Research ; research activities ) [SEP]',
  'our_cross_cui': 'C0035168',
  'our_cross_cui_name': 'research',
  'our_cross_cui_desc': '[CLS] research [unused3] ( Research Act ##ivity : Research ) [SEP]'},
 {'mention_id': '27832159.48',
  'test_set_idx': 18596,
  'type': 'T103',
  'name': 'compound-326',
  'context': '[CLS] D ##G ##LA profiles . In diet - induced o ##bes ##e mice , chronic treatment with compound - 32 ##6 lowered insulin resistance and caused body weight loss without significant impact on cumulative ca ##lor ##ie intake . Dec ##reased mac ##rop ##hage in ##fi ##ltration into ad ##ip ##ose tissue was expected from m ##RNA analysis . Inc ##reased daily energy expenditure was also observed following administration of compound - 32 ##6 , in line with sustained body weight loss . These data indicate that the novel Delta - 5 des ##at ##ura ##se selective inhibitor , [unused1] compound - 32 ##6 [unused2] , will be a new class of drug for the treatment of o ##bes ##e and di ##abe ##tic patients . [SEP]',
  'existing_cui': 'C0258113',
  'existing_cui_name': '4 - ( ( 3-bromophenyl ) amino ) -6,7 - dimethoxyquinazoline',
  'existing_cui_desc': "[CLS] 4 - ( ( 3 - br ##omo ##phe ##ny ##l ) amino ) - 6 , 7 - dim ##eth ##ox ##y ##quin ##az ##olin ##e [unused3] ( Chemical : Co ##mpo ##und 32 ; 4 - ( 3 ' - br ##omo ##ani ##lino ) - 6 , 7 - dim ##eth ##ox ##y ##quin ##az ##olin ##e ) [SEP]",
  'our_bi_cui': 'C0149246',
  'our_bi_cui_name': 'Z 326',
  'our_bi_cui_desc': '[CLS] Z 32 ##6 [unused3] ( Chemical ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28256742.3',
  'test_set_idx': 30209,
  'type': 'T082',
  'name': 'Canadian Areas',
  'context': '[CLS] Hal ##ogen ##ated p ##hen ##olic compounds in wild fish from [unused1] Canadian Areas [unused2] of Con ##cer ##n Con ##cent ##rations of ha ##log ##ena ##ted p ##hen ##olic compounds were measured in the plasma of brown bull ##head ( Am ##ei ##urus ne ##bul ##os ##us ) from 4 Canadian Areas of Con ##cer ##n ( A ##OC ##s ) , to assess exposure to suspected thy ##roid - disrupt ##ing chemicals . H ##ydro ##xy ##lated p ##oly ##ch ##lor ##inated bi ##phe ##ny ##ls ( OH - PC ##B ##s ) were detected in every sample collected in 3 of the A ##OC ##s ; the detection frequency was lower in samples from the Detroit River A ##OC . The OH - [SEP]',
  'existing_cui': 'C3831211',
  'existing_cui_name': 'Canadian County , OK',
  'existing_cui_desc': '[CLS] Canadian County , OK [unused3] ( Spa ##tial Concept : Canadian County , Oklahoma ; OK ##01 ##7 ; Canadian County ) [SEP]',
  'our_bi_cui': 'C0454980',
  'our_bi_cui_name': 'Region of Canada',
  'our_bi_cui_desc': '[CLS] Region of Canada [unused3] ( Spa ##tial Concept , geographic location : Province or Territory of Canada ; Provinces and territories of Canada ; Region ##s of Canada ) [SEP]',
  'our_cross_cui': 'C0006823',
  'our_cross_cui_name': 'Canada',
  'our_cross_cui_desc': '[CLS] Canada [unused3] ( Spa ##tial Concept , geographic location : CA ##N ; CA ##NA ##DA ; CA ) [SEP]'},
 {'mention_id': '27820849.13',
  'test_set_idx': 18195,
  'type': 'T204',
  'name': 'hens',
  'context': '[CLS] p ##hen ##otypic and g ##eno ##mic evolution , with large effective population size and strong human - driven selection . In the present study , we performed Extended Ha ##p ##lot ##ype Ho ##mo ##zy ##gos ##ity tests to identify significant core regions employing 600 ##K S ##NP Chicken chip in an F ##2 population of 1 , 53 ##4 [unused1] he ##ns [unused2] , which was derived from re ##ci ##p ##ro ##cal crosses between White Le ##gh ##orn and Dong ##xia ##ng chicken . Results indicated that a total of 49 , 151 core regions with an average length of 9 . 79 K ##b were identified , which occupied approximately 52 . 15 % of genome across all auto ##some ##s , [SEP]',
  'existing_cui': 'C0099085',
  'existing_cui_name': '6-bromo-2-naphthyl sulfate',
  'existing_cui_desc': '[CLS] 6 - br ##omo - 2 - nap ##ht ##hyl su ##lf ##ate [unused3] ( Chemical , B ##NS : 2 - Na ##ph ##thal ##eno ##l , 6 - br ##omo - , hydrogen su ##lf ##ate ) [SEP]',
  'our_bi_cui': 'C0008051',
  'our_bi_cui_name': 'Chickens',
  'our_bi_cui_desc': '[CLS] Chicken ##s [unused3] ( E ##uka ##ryo ##te , organism : G ##all ##us gal ##lus ; Red jungle f ##ow ##l ; Chicken ; CH ##IC ##KE ##N ; ban ##tam ; G ##all ##us gal ##lus domestic ##us ; G ##all ##us domestic ##us ; Ban ##tam Chicken ; Red jungle ##fo ##wl ; Ban ##tam ; chicken ; chickens ; Domestic f ##ow ##l ancestor ) [SEP]',
  'our_cross_cui': 'C0005595',
  'our_cross_cui_name': 'Aves',
  'our_cross_cui_desc': '[CLS] Ave ##s [unused3] ( E ##uka ##ryo ##te , organism : a ##vian ; bird ; a ##vian ##s ; a ##ves ; birds ; Bird ; Bird in context ; A ##vian ; Birds ; Class Ave ##s ) [SEP]'},
 {'mention_id': '28179911.59',
  'test_set_idx': 27653,
  'type': 'T017',
  'name': 'GPDH',
  'context': '[CLS] independent T ##2 lines were detected by quantitative real - time PC ##R . The relative m ##RNA accumulation of genes encoding enzymes involved in either fatty acid bio ##sy ##nt ##hesis or t ##ria ##cy ##l ##gly ##cer ##ols assembly ( BC ##CP ##2 , K ##AS ##I , MA ##T , E ##NR , FA ##TA , and [unused1] GP ##D ##H [unused2] ) were also ass ##ay ##ed in mature seeds . Furthermore , lip ##id and fatty acids C ##16 : 0 and C ##18 : 0 significantly increased . In two ho ##mo ##zy ##go ##us T ##2 trans ##genic rice lines ( G ##5 and G ##2 ) , different Co ##WR ##I ##1 expression levels were detected , but [SEP]',
  'existing_cui': 'C1415375',
  'existing_cui_name': 'GYPC gene',
  'existing_cui_desc': '[CLS] G ##YP ##C gene [unused3] ( Ana ##tom ##ical Structure , G ##er ##bic ##h blood group : GP ##C ; G ##L ##Y ##CO ##P ##H ##OR ##IN C ; g ##ly ##co ##ph ##ori ##n C ; G ##L ##Y ##CO ##P ##H ##OR ##IN D ; CD ##23 ##6 ; S ##IA ##L ##O ##G ##L ##Y ##CO ##PR ##OT ##EI ##N , B ##ET ##A ; CD ##23 ##6 ##R ; G ##e ) [SEP]',
  'our_bi_cui': 'C1418531',
  'our_bi_cui_name': 'PHGDH gene',
  'our_bi_cui_desc': '[CLS] P ##H ##G ##D ##H gene [unused3] ( Ana ##tom ##ical Structure : P ##H ##OS ##P ##H ##O ##G ##L ##Y ##CE ##RA ##TE DE ##H ##Y ##DR ##O ##GE ##NA ##SE ; P ##H ##G ##D ##H Gene ; Ph ##os ##ph ##og ##ly ##cer ##ate De ##hy ##dr ##ogen ##ase Gene ; SE ##RA ; p ##hos ##ph ##og ##ly ##cer ##ate de ##hy ##dr ##ogen ##ase ; 3 - P ##H ##OS ##P ##H ##O ##G ##L ##Y ##CE ##RA ##TE DE ##H ##Y ##DR ##O ##GE ##NA ##SE ; P ##G ##D ##H ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27838742.14',
  'test_set_idx': 18981,
  'type': 'T103',
  'name': 'delicaflavone',
  'context': '[CLS] / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway Search ##ing for potential anti ##can ##cer agents from natural sources is an effective strategy for developing novel ch ##em ##oth ##era ##pe ##uti ##c agents . In this study , data supporting the in v ##it ##ro and in v ##ivo anti ##can ##cer effects of [unused1] del ##ica ##f ##lav ##one [unused2] , a rarely occurring bi ##f ##lav ##ono ##id from Se ##lag ##ine ##lla do ##ede ##rle ##ini ##i , were reported . Del ##ica ##f ##lav ##one exhibited favorable anti ##can ##cer properties , as shown by the M ##TT ass ##ay and x ##eno ##gra ##ft model of human non - small cell lung cancer in male BA [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27875010.31',
  'test_set_idx': 20236,
  'type': 'T017',
  'name': 'tissue',
  'context': '[CLS] ##ified net photos ##ynth ##esis by ambient and nut ##rient - en ##rich ##ed tissues at sat ##ura ##ting i ##rra ##dian ##ce over a range of temperature conditions ( 6 - 30 °C ) . Re ##spiration was un ##af ##fected by nut ##rient treatment ; however , there was a significant increase in photos ##ynth ##etic oxygen production for nut ##rient - en ##rich ##ed [unused1] tissue [unused2] compared to ambient , but only at elevated ( ≥ 18 °C ) temperatures . This study contributes to a growing body of literature showing the complexity of responses to changes in multiple drivers , and highlights the importance of studying the impacts of global climate change within the context of more local environmental conditions . [SEP]',
  'existing_cui': 'C0040300',
  'existing_cui_name': 'Body tissue',
  'existing_cui_desc': '[CLS] Body tissue [unused3] ( Ana ##tom ##ical Structure , Body , body structure : Port ##ion of tissue ; Normal T ##iss ##ue ; T ##iss ##ue ; Text ##us ; Body T ##iss ##ues ; tissues ; Body tissue structure ; body tissues ; T ##iss ; body tissue ; tissue ; T ##iss ##ues ; T ##IS ##SU ##E ) [SEP]',
  'our_bi_cui': 'C0040300',
  'our_bi_cui_name': 'Body tissue',
  'our_bi_cui_desc': '[CLS] Body tissue [unused3] ( Ana ##tom ##ical Structure , Body , body structure : Port ##ion of tissue ; Normal T ##iss ##ue ; T ##iss ##ue ; Text ##us ; Body T ##iss ##ues ; tissues ; Body tissue structure ; body tissues ; T ##iss ; body tissue ; tissue ; T ##iss ##ues ; T ##IS ##SU ##E ) [SEP]',
  'our_cross_cui': 'C1514137',
  'our_cross_cui_name': 'Plant Tissue , Cells',
  'our_cross_cui_desc': '[CLS] Plant T ##iss ##ue , Cell ##s [unused3] ( Ana ##tom ##ical Structure : Plant cells and tissues ) [SEP]'},
 {'mention_id': '27384676.30',
  'test_set_idx': 4392,
  'type': 'T033',
  'name': 'elongation',
  'context': '[CLS] 3 / 4 le ##uk ##ope ##nia occurred in fourteen patients ( 58 . 33 % ) and grade 3 / 4 th ##rom ##bo ##cy ##top ##enia occurred in eleven patients ( 45 . 83 % ) . Twelve patients ( 50 . 00 % ) experienced Act ##ivated Part ##ial Ph ##rom ##bo ##p ##last ##in P ##time [unused1] el ##ong ##ation [unused2] and fourteen patients ( 58 . 33 % ) experienced h ##y ##po ##fi ##bri ##no ##gene ##mia . In conclusion , de ##xa ##met ##has ##one , g ##em ##ci ##ta ##bine and p ##eg ##as ##par ##gas ##e regime ##n is an effective and tolerate ##d treatment for newly diagnosed , advanced - stage extra ##no ##dal natural killer [SEP]',
  'existing_cui': 'C0086206',
  'existing_cui_name': 'Elongation Factor',
  'existing_cui_desc': '[CLS] El ##ong ##ation Factor [unused3] ( Chemical : el ##ong ##ation factor ; Factor , El ##ong ##ation ) [SEP]',
  'our_bi_cui': 'C0242656',
  'our_bi_cui_name': 'Disease Progression',
  'our_bi_cui_desc': '[CLS] Disease Progress ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction , attribute : Course of illness ; Disease Progress ##ions ; Disease Course ; Course of Il ##ln ##ess ; Progress ##ion , Disease ; Course of disorder ; Disease progression ; Progress ##ion ; D ##IS PR ##O ##GR ##ES ##SI ##ON ; progression ; Di ##sor ##der Course ; Progress ##ions , Disease ; course of illness ; Disease Progress ##ion [ Disease / Finding ] ; course illness ; courses disease ; disease progression ) [SEP]',
  'our_cross_cui': 'C0240671',
  'our_cross_cui_name': 'Partial thromboplastin time increased',
  'our_cross_cui_desc': '[CLS] Part ##ial th ##rom ##bo ##p ##last ##in time increased [unused3] ( Finding , finding , finding , lab test : A ##b ##nor ##mal or prolonged PT ##T ; Pro ##long ##ed partial th ##rom ##bo ##p ##last ##in time ; A ##b ##nor ##mal or prolonged partial th ##rom ##bo ##p ##last ##in time ; AP ##TT prolonged ; prolonged a ##pt ##t ; Act ##ivated partial th ##rom ##bo ##p ##last ##in time prolonged ; Pro ##long ##ed Act ##ivated Part ##ial T ##hr ##omb ##op ##last ##in Time ; Pro ##long ##ed activated partial th ##rom ##bo ##p ##last ##in time ; a ##pt ##t prolonged ; PT ##T prolonged ; AP ##TT increased ; a ##P ##TT prolonged ; elevated p [SEP]'},
 {'mention_id': '27838742.1',
  'test_set_idx': 18968,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] [unused1] Del ##ica ##f ##lav ##one [unused2] induce ##s auto ##pha ##gic cell death in lung cancer via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway Search ##ing for potential anti ##can ##cer agents from natural sources is an effective strategy for developing novel ch ##em ##oth ##era ##pe ##uti ##c agents . In this study , data supporting the in v ##it ##ro and in v ##ivo anti ##can ##cer effects of del ##ica ##f ##lav ##one , a rarely occurring bi ##f ##lav ##ono ##id from Se ##lag ##ine ##lla do ##ede ##rle ##ini ##i , were reported . Del ##ica ##f ##lav ##one exhibited favorable anti ##can ##cer properties , as shown by the M ##TT ass [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27449818.31',
  'test_set_idx': 6226,
  'type': 'T058',
  'name': 'orally treated',
  'context': '[CLS] ) were synthesized and tested for carbon ##ic an ##hy ##dra ##se II binding . CA II in er ##yt ##hr ##ocytes is thought to oppose extraction of E ##stra ##dio ##l pro ##dr ##ugs from portal vein blood during liver passage . O ##var ##ie ##ct ##omi ##zed ( O ##V ##X , day minus 14 ) rats were [unused1] oral ##ly treated [unused2] once daily from day 1 - 3 . Sa ##c ##ri ##fic ##e day 4 . U ##ter ##i were di ##sse ##cted and weighed . Cho ##les ##tero ##l fraction ##s and an ##gio ##tens ##ino ##gen were determined in plasma . Oral E ##2 and et ##hin ##yl est ##rad ##iol generated dose related u ##ter ##ine growth and [SEP]',
  'existing_cui': 'C0226896',
  'existing_cui_name': 'Oral cavity',
  'existing_cui_desc': '[CLS] Oral cavity [unused3] ( Spa ##tial Concept , Anatomy , body structure : Mouth ; B ##OD ##Y CA ##VI ##TY , OR ##AL ; Oral ##ly ; C ##avi ##tas Or ##is ; C ##avi ##ty of mouth ; Oral C ##avi ##ty ; C ##avi ##tas or ##is ; Oral ; C ##avi ##ty , Oral ; oral cavity ; an ##ato ##mie ##s mouth ; mouth anatomy ; B ##uc ##cal cavity ; mouth ; Oral cavity structure ; mouths ; anatomy mouth ; B ##uc ##cal C ##avi ##ty ; OR ##AL CA ##VI ##TY ) [SEP]',
  'our_bi_cui': 'C1273553',
  'our_bi_cui_name': 'Oral administration of treatment',
  'our_bi_cui_desc': '[CLS] Oral administration of treatment [unused3] ( Health Care Act ##ivity , navigation ##al concept , procedure : Treatment administered oral ##ly ) [SEP]',
  'our_cross_cui': 'C0001563',
  'our_cross_cui_name': 'Administration , Oral',
  'our_cross_cui_desc': '[CLS] Administration , Oral [unused3] ( Health Care Act ##ivity , procedure : Oral medication administration each treatment ; oral medication administration ; or ##l me ##ds ad ##min . e ##a . T ##x ; Administration , Oral Drug ; Oral Drug Administration ; AD ##MI ##N OR ##AL DR ##U ##G ; Drug Administration , Oral ; Administration ##s , Oral ; administration oral ; Administration of drug or me ##dic ##ament via oral route ; Administration ##s , Oral Drug ; Oral Drug Administration ##s ; Oral Administration ##s ; Me ##dication Administration : Oral ; OR ##AL AD ##MI ##N ; AD ##MI ##N OR ##AL ; OR ##AL DR ##U ##G AD ##MI ##N ; Me ##dication administration : oral ; oral [SEP]'},
 {'mention_id': '28027760.26',
  'test_set_idx': 23867,
  'type': 'T033',
  'name': 'No',
  'context': '[CLS] ##iac CT an ##gio ##graphy : 2 . 4 ± 0 . 9 ##m ##S ##v - 6 . 8 ± 2 . 7 m ##S ##v , all p ≤ 0 . 04 ##14 ) . In 39 patients trans ##cat ##he ##ter a ##ort ##ic valve replacement was performed and an ##nu ##lus diameter was within the recommended range in all patients . [unused1] No [unused2] severe cardiac or vascular complications were noted . 3rd generation dual - source CT provides diagnostic image quality in trans ##cat ##he ##ter a ##ort ##ic valve replacement - planning CT an ##gio ##graphy and facilitate ##s reliable assessment of trans ##cat ##he ##ter a ##ort ##ic valve replacement device and delivery option while reducing radiation dose . [SEP]',
  'existing_cui': 'C1016143',
  'existing_cui_name': 'Tetragenococcus halophilus',
  'existing_cui_desc': '[CLS] Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##us [unused3] ( Ba ##cter ##ium , organism : P ##ed ##io ##co ##cc ##us ha ##lop ##hil ##us Me ##es 1934 ( A ##pp ##roved List ##s 1980 ) ; Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##us ( Me ##es 1934 ) Collins et al . 1993 ; Sa ##rc ##ina ha ##ma ##guchi ##ae ; Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##a ; P ##ed ##io ##co ##cc ##us ha ##lop ##hil ##us ; Te ##tra ##co ##cc ##cus No . 1 ; P ##ed ##io ##co ##cc ##us so ##ya ##e ) [SEP]',
  'our_bi_cui': 'C0243095',
  'our_bi_cui_name': 'Finding',
  'our_bi_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]',
  'our_cross_cui': 'C1513916',
  'our_cross_cui_name': 'Negative Finding',
  'our_cross_cui_desc': '[CLS] N ##eg ##ative Finding [unused3] ( Finding : NE ##GA ##TI ##VE ; N ##eg ##ative ) [SEP]'},
 {'mention_id': '28236586.25',
  'test_set_idx': 29279,
  'type': 'T170',
  'name': 'models',
  'context': '[CLS] using a comprehensive g ##eria ##tric assessment was conducted . Di ##sa ##bility was defined as imp ##air ##ment in activities of daily living and / or instrumental activities of daily living , simplified to four items . Un ##iva ##ria ##te and multi ##var ##iate log ##istic models of disabled patients were performed . The three final multi ##var ##iate [unused1] models [unused2] were compared using the area under the receiver operating characteristic curve ( AU ##C / R ##OC ) of the log ##istic model . The mean age was 80 . 6 years , and 51 % of the patients were women with various types of cancer . The prevalence of disability was 67 . 6 % . No on ##co ##log ##ic [SEP]',
  'existing_cui': 'C0026350',
  'existing_cui_name': 'Theoretical model',
  'existing_cui_desc': '[CLS] Theo ##ret ##ical model [unused3] ( Intel ##lect ##ual Product , Theo ##ret ##ical : Model , Theo ##ret ##ical ; Theo ##ret ##ical Models ; Models , Theo ##ret ##ical ; Models ; Theo ##ret ##ical Model ; theoretical model ; Models , Theo ##ret ##ic ; Model ; theoretical models ) [SEP]',
  'our_bi_cui': 'C3161035',
  'our_bi_cui_name': 'Model',
  'our_bi_cui_desc': '[CLS] Model [unused3] ( Intel ##lect ##ual Product : Models ; Model ##ing System ; Model System ) [SEP]',
  'our_cross_cui': 'C0023965',
  'our_cross_cui_name': 'Logistic Models',
  'our_cross_cui_desc': '[CLS] Lo ##gis ##tic Models [unused3] ( Intel ##lect ##ual Product : Model , Lo ##gis ##tic ; Models , Lo ##gis ##tic ; Lo ##gis ##tic Model ) [SEP]'},
 {'mention_id': '28145492.38',
  'test_set_idx': 26798,
  'type': 'T103',
  'name': 'OX2R receptor',
  'context': '[CLS] be carried by a Na ( + ) - dependent trans ##ient receptor potential - like channel ( as it was blocked by 2 - AP ##B and was diminished by removal of extra ##cellular Na ( + ) ) , and was a consequence of O ##X ##2 ##R receptor activation ( as it was blocked by the [unused1] O ##X ##2 ##R receptor [unused2] antagonist T ##CS O ##X ##2 29 , but not the O ##X ##1 ##R receptor antagonist S ##B 33 ##48 ##6 ##7 ) . Application of the hormone , me ##lat ##oni ##n , failed to alter tube ##roi ##n ##fu ##ndi ##bula ##r do ##pa ##mine membrane potential or o ##s ##cilla ##tory activity . This first description [SEP]',
  'existing_cui': 'C0913649',
  'existing_cui_name': 'taste receptors , type 2',
  'existing_cui_desc': '[CLS] taste receptors , type 2 [unused3] ( Chemical , taste : G protein - coupled receptor T ##R ##2 ; T ##2 ##R taste receptors ; T ##2 ##R receptor ; T ##2 ##R receptors ; taste receptor , type 2 ; T ##AS ##2 ##R ##13 protein , human ; receptor , T ##2 ##R ) [SEP]',
  'our_bi_cui': 'C0069841',
  'our_bi_cui_name': 'Oxytocin Receptor',
  'our_bi_cui_desc': '[CLS] O ##xy ##to ##cin Re ##ceptor [unused3] ( Chemical : Re ##ceptor ##s , O ##xy ##to ##cin [ Chemical / In ##g ##red ##ient ] ; R ##EC ##EP ##T O ##X ##Y ##TO ##CI ##N ; Re ##ceptor ##s , O ##xy ##to ##cin ; O ##X ##Y ##TO ##CI ##N R ##EC ##EP ##T ; Re ##ceptor , O ##xy ##to ##cin ; O ##xy ##to ##cin Re ##ceptor ##s ) [SEP]',
  'our_cross_cui': 'C0671880',
  'our_cross_cui_name': 'orexin receptor type 2',
  'our_cross_cui_desc': '[CLS] ore ##xin receptor type 2 [unused3] ( Chemical : Or ##ex ##in - B Re ##ceptor ##s ; H ##y ##po ##cre ##tin - 2 Re ##ceptor ; Re ##ceptor , H ##y ##po ##cre ##tin - 2 ; H ##y ##po ##cre ##tin Re ##ceptor ##s 2 ; O ##X ##2 Re ##ceptor ##s ; Or ##ex ##in B Re ##ceptor ; Re ##ceptor ##s , Or ##ex ##in - B ; Re ##ceptor ##s 2 , H ##y ##po ##cre ##tin ; Or ##ex ##in Re ##ceptor 2 ; Re ##ceptor 2 , H ##y ##po ##cre ##tin ; HC ##TR ##2 Pro ##tein ; Re ##ceptor , Or ##ex ##in - B ; Or ##ex ##in Re ##ceptor Type 2 ; Pro ##tein [SEP]'},
 {'mention_id': '28203549.70',
  'test_set_idx': 28262,
  'type': 'T170',
  'name': 'ST',
  'context': '[CLS] ##co ##vis ##co ##us K ##le ##bs ##iel ##la pneumonia ##e and non - HM ##K ##P is ##olate ##s were positive for r ##mp ##A . Aero ##ba ##ct ##in was found among 95 . 0 and 97 . 5 % of K ##1 and K ##2 is ##olate ##s . ST ##23 was found to be the most prevalent [unused1] ST [unused2] among 69 h ##yper ##mu ##co ##vis ##co ##us K ##le ##bs ##iel ##la pneumonia ##e is ##olate ##s with K ##1 , K ##2 , K ##5 , K ##20 , and K ##5 ##7 ( 27 . 5 % , 19 / 69 ) and was only found among K ##1 is ##olate ##s . ST ##65 was the second [SEP]',
  'existing_cui': 'C0036183',
  'existing_cui_name': 'Sao Tome and Principe',
  'existing_cui_desc': '[CLS] Sa ##o Tom ##e and P ##rin ##ci ##pe [unused3] ( Spa ##tial Concept , geographic location : SA ##O TO ##ME AND PR ##IN ##CI ##P ##E ; São Tom ##é and P ##r ##ín ##ci ##pe ) [SEP]',
  'our_bi_cui': 'C0457537',
  'our_bi_cui_name': 'Subtype serogroup',
  'our_bi_cui_desc': '[CLS] Sub ##type se ##rog ##roup [unused3] ( Intel ##lect ##ual Product , qualifier value : Sub ##type se ##rog ##roup ##s ) [SEP]',
  'our_cross_cui': 'C0449943',
  'our_cross_cui_name': 'Serotype',
  'our_cross_cui_desc': '[CLS] Ser ##otype [unused3] ( Intel ##lect ##ual Product , attribute , o ##bs ##er ##vable entity , qualifier value : se ##rov ##ar ; se ##rov ##ars ; Ser ##ova ##rs ; se ##rot ##ype ; se ##rot ##ype ##s ; Ser ##otype ##s ; Ser ##ova ##r ) [SEP]'},
 {'mention_id': '28445938.39',
  'test_set_idx': 36360,
  'type': 'T062',
  'name': 'strategy',
  'context': '[CLS] ##uron ##an levels and , as a result , significantly increases blood vessel functionality and per ##fusion and improve ##s the anti - tumor efficacy of do ##x ##or ##ubi ##cin . Red ##uction of extra ##cellular matrix components were mediated via T ##G ##F ##β signaling pathway in ##hibition due to down ##re ##gu ##lation of T ##G ##F ##β ##1 , CO ##L ##1 ##A ##1 , CO ##L ##3 ##A ##1 , H ##AS ##2 , H ##AS ##3 expression levels . Our findings provide evidence that re ##pur ##po ##sing Pi ##rf ##eni ##don ##e could be used as a promising [unused1] strategy [unused2] to enhance drug delivery to solid tumors by normal ##izing the tumor micro ##en ##vir ##on ##ment . [SEP]',
  'existing_cui': 'C0589105',
  'existing_cui_name': 'Strategy training',
  'existing_cui_desc': '[CLS] Strategy training [unused3] ( Health Care Act ##ivity , procedure , regime / therapy ) [SEP]',
  'our_bi_cui': 'C0085104',
  'our_bi_cui_name': 'Drug Delivery Systems',
  'our_bi_cui_desc': '[CLS] Drug Del ##iver ##y Systems [unused3] ( Medical Devi ##ce : drug target ##ting ; Drug Target ##ing ; delivery drug systems ; drug target ; Drug delivery systems ; Drug Target ##ings ; drug targeting ; Drug delivery system ; target drug ; drugs targets ; Del ##iver ##y System , Drug ; drugs target ; Systems , Drug Del ##iver ##y ; drug delivery systems ; drugs targeting ; drug targets ; Drug Del ##iver ##y System ; drug targeted ; System , Drug Del ##iver ##y ; Target ##ings , Drug ; Drug Del ##iver ##y ; DR ##U ##G DE ##L ##IVE ##R ##Y S ##Y ##ST ##EM ; Del ##iver ##y Systems , Drug ; drug delivery system ; delivery drugs [SEP]',
  'our_cross_cui': 'C0035171',
  'our_cross_cui_name': 'Research Design',
  'our_cross_cui_desc': '[CLS] Research Design [unused3] ( Research Act ##ivity : Experiment Design ; design research ; research design ; Design ##s , Research ; Design , Research ; ST ##YP ##E ; Research Design ##s ; study design ; Study Type ; Study Design ; Experimental Design ) [SEP]'},
 {'mention_id': '27617315.66',
  'test_set_idx': 12029,
  'type': 'T103',
  'name': '% p2PSA',
  'context': '[CLS] 0 . 00 ##1 ) . By utilizing the Pro ##state Health Index cut ##off of 35 to men with pro ##state - specific anti ##gen 10 - 20 ng / m ##L and normal digital re ##ct ##al examination , 57 . 1 % ( 178 of 312 ) bio ##ps ##ies could be avoided . Both Pro ##state Health Index and % p ##2 ##PS ##A performed well in predict ##ing pro ##state cancer and high grade pro ##state cancer . The use of Pro ##state Health Index and [unused1] % p ##2 ##PS ##A [unused2] should be extended to Chinese men with pro ##state - specific anti ##gen 10 - 20 ng / m ##L and normal digital re ##ct ##al examination . [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0138741',
  'our_bi_cui_name': 'Prostate-Specific Antigen',
  'our_bi_cui_desc': '[CLS] Pro ##state - S ##pecific Anti ##gen [unused3] ( Chemical , PS ##A , procedure , substance : pro ##state specific anti ##gen ; gamma Semi ##no ##p ##rote ##in ; Se ##men ##ogel ##ase ; gamma semi ##no ##p ##rote ##in ; Ka ##lli ##k ##re ##in , h ##K ##3 ; Gamma - Semi ##no ##p ##rote ##in ; Pro ##state - specific anti ##gen ; HK 00 ##3 K ##AL ##L ##I ##K ##RE ##IN ; PS ##A - Pro ##state specific anti ##gen ; Ka ##lli ##k ##re ##in h ##K ##3 ; gamma - Semi ##no ##p ##rote ##in ; Pro ##state S ##pecific Anti ##gen ; EC 3 . 4 . 21 . 77 ; P - 30 anti ##gen [SEP]',
  'our_cross_cui': 'C3710767',
  'our_cross_cui_name': '( -2 ) pro-prostate-specific antigen , human',
  'our_cross_cui_desc': '[CLS] ( - 2 ) pro - pro ##state - specific anti ##gen , human [unused3] ( Chemical : ( - 2 ) pro - PS ##A , human ; p ##2 ##PS ##A , human ) [SEP]'},
 {'mention_id': '28541679.21',
  'test_set_idx': 38908,
  'type': 'T033',
  'name': 'pros and cons',
  'context': '[CLS] work addresses : the range of magnetic properties reported for l ##iche ##ns , moss ##es , leaves , bark , trunk wood , insects , crust ##ace ##ans , ma ##mmal and human tissues ; their associations with atmospheric poll ##uta ##nt species ( Part ##iculate matter , NO ##x , trace elements , PA ##H ##s ) ; the [unused1] pro ##s and con ##s [unused2] of bio ##ma ##gnetic monitoring of atmospheric pollution ; current challenges for large - scale implementation of bio ##ma ##gnetic monitoring ; and future perspectives . A summary table is presented , with the aim of aid ##ing researchers and policy makers in selecting the most suitable biological sensor for their intended bio ##ma ##gnetic monitoring purpose . [SEP]',
  'existing_cui': 'C0052649',
  'existing_cui_name': 'gamma-Atrial Natriuretic Peptide',
  'existing_cui_desc': '[CLS] gamma - At ##rial Nat ##ri ##ure ##tic P ##eptide [unused3] ( Chemical : Pro - AN ##F ; At ##rial Pro ##nat ##rio ##di ##lat ##in ; At ##rio ##pe ##pt ##in 126 ; AN ##P ( 1 - 126 ) ; Card ##ion ##at ##rin IV ; At ##rio ##pe ##pt ##ige ##n ; At ##rial Nat ##ri ##ure ##tic Factor ( 1 - 126 ) ; At ##rial Nat ##ri ##ure ##tic P ##eptide ( 1 - 126 ) ; At ##rial Nat ##ri ##ure ##tic Factor Pro ##hor ##mon ##e ; Pro ##at ##rial Nat ##ri ##ure ##tic Factor ; Pro ##AN ##F ; AN ##F ( 1 - 126 ) ; At ##rio ##pe ##pt ##in Pro ##hor ##mon ##e ( [SEP]',
  'our_bi_cui': 'C0037088',
  'our_bi_cui_name': 'Signs and Symptoms',
  'our_bi_cui_desc': '[CLS] Sign ##s and S ##ym ##pt ##oms [unused3] ( Finding , finding : Sign ##s and S ##ym ##pt ##oms [ Disease / Finding ] ; Clinical findings ; clinical findings ; O ##bs ##er ##vable En ##ti ##ty ; sign and s ##ym ##pt ##om ; Sign and s ##ym ##pt ##om ; S ##ym ##pt ##oms and Sign ##s ; sign / s ##ym ##pt ##om ; Finding ; Clinical observation : symptoms and signs ; S ##IG ##NS S ##Y ##MP ##TO ##MS ; Clinical finding ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27444636.45',
  'test_set_idx': 5996,
  'type': 'T170',
  'name': 'Quality of Life in Epilepsy Inventory 10',
  'context': '[CLS] ( P < . 00 ##1 ) , Quality of Life in E ##pile ##psy In ##vent ##ory 10 ( P = . 00 ##1 ) , and alert ##ness ( P < . 05 ) . Ad ##verse Events Profile total scores improved for 21 / 21 ( 100 . 0 % ) patients , [unused1] Quality of Life in E ##pile ##psy In ##vent ##ory 10 [unused2] total scores improved for 17 / 21 ( 81 . 0 % ) patients , and alert ##ness scores improved for 16 / 21 ( 76 . 2 % ) patients . In this short - term , single - center study , an overnight switch from twice - daily o ##x ##car ##ba ##ze ##pine to [SEP]',
  'existing_cui': 'C2698752',
  'existing_cui_name': 'Pediatric Quality of Life Inventory',
  'existing_cui_desc': '[CLS] P ##ediatric Quality of Life In ##vent ##ory [unused3] ( Intel ##lect ##ual Product : P ##eds ##QL ) [SEP]',
  'our_bi_cui': 'C0085295',
  'our_bi_cui_name': 'Interleukin-10',
  'our_bi_cui_desc': '[CLS] Inter ##le ##uki ##n - 10 [unused3] ( Chemical , NO ##NG ##L ##Y ##CO ##S ##Y ##LA ##TE ##D , substance : C ##Y ##TO ##K ##IN ##E S ##Y ##N IN ##H ##IB FA ##CT ##OR ; Cy ##tok ##ine synthesis in ##hibit ##ing factor ; IL 01 ##0 ; Inter ##le ##uki ##n - 10 [ Chemical / In ##g ##red ##ient ] ; il ##10 ; inter ##le ##uki ##n 10 ; inter ##le ##uki ##n - 10 ; IL ##10 ; IL - 10 ; IN ##TE ##RL ##E ##U ##K ##IN 01 ##0 ; IN ##TE ##RL ##E ##U ##K ##IN - 10 ; H ##UM ##AN IN ##TE ##RL ##E ##U ##K ##IN - 10 ; Cy ##tok ##ine [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27241818.68',
  'test_set_idx': 589,
  'type': 'T058',
  'name': 'output was estimated',
  'context': '[CLS] subjects performed 5 km cycling time trials under control conditions and with l ##umba ##r in ##tra ##the ##cal f ##ent ##any ##l imp ##air ##ing lower limb muscle a ##ffer ##ent feedback ( F ##EN ##T ) . V ##ast ##us lateral ##is muscle bio ##ps ##ies were obtained before and immediately after exercise . Mo ##tone ##uron ##al [unused1] output was estimated [unused2] through vast ##us lateral ##is surface electro ##my ##ography . Ex ##er ##cise - induced changes in in ##tra ##mus ##cular meta ##bol ##ites were determine d using liquid and gas ch ##roma ##tography - mass s ##pect ##rome ##try . Q ##uad ##rice ##ps fatigue was q ##uant ##ified by pre - to post - exercise changes in potent ##iated [SEP]',
  'existing_cui': 'C3251815',
  'existing_cui_name': 'fluid output measurement',
  'existing_cui_desc': '[CLS] fluid output measurement [unused3] ( Health Care Act ##ivity , procedure : Out ##put ; Me ##as ##ure ##ment of fluid output ) [SEP]',
  'our_bi_cui': 'C0680844',
  'our_bi_cui_name': 'estimation',
  'our_bi_cui_desc': '[CLS] est ##imation [unused3] ( B ##iol ##og ##ic Fun ##ction , subjective , subjective : E ##st ##imation ; est ##imation ##s ) [SEP]',
  'our_cross_cui': 'C0204709',
  'our_cross_cui_name': 'Measuring output',
  'our_cross_cui_desc': '[CLS] Me ##as ##uring output [unused3] ( Health Care Act ##ivity , procedure ) [SEP]'},
 {'mention_id': '28145492.7',
  'test_set_idx': 26767,
  'type': 'T038',
  'name': 'Current',
  'context': '[CLS] H ##y ##po ##cre ##tin / Or ##ex ##in P ##eptide ##s Ex ##cite Rat N ##eur ##oe ##ndo ##c ##rine Do ##pa ##mine N ##eur ##ons through Or ##ex ##in 2 Re ##ceptor - Media ##ted Act ##ivation of a Mixed Cat ##ion [unused1] Current [unused2] H ##y ##po ##cre ##tin / Or ##ex ##in neurons of the lateral h ##y ##pot ##hala ##mus are compelling m ##od ##ulator candidates for the ch ##ron ##ob ##iology of ne ##uro ##end ##oc ##rine output and , as a consequence , hormone release from the anterior pit ##uit ##ary . Here we investigate the effects of H ##y ##po ##cre ##tin / Or ##ex ##in p ##eptide ##s upon tube ##roi ##n ##fu ##ndi ##bula ##r do [SEP]',
  'existing_cui': 'C0282444',
  'existing_cui_name': 'Current Biog-Obit',
  'existing_cui_desc': '[CLS] Current B ##io ##g - O ##bit [unused3] ( Intel ##lect ##ual Product ) [SEP]',
  'our_bi_cui': 'C0022023',
  'our_bi_cui_name': 'Ions',
  'our_bi_cui_desc': '[CLS] Ion ##s [unused3] ( Chemical , substance : ion ; Ion ##s [ Chemical / In ##g ##red ##ient ] ; Ion ; charged particles ; ions ) [SEP]',
  'our_cross_cui': 'C0162585',
  'our_cross_cui_name': 'Ion Transport',
  'our_cross_cui_desc': '[CLS] Ion Transport [unused3] ( B ##iol ##og ##ic Fun ##ction : ion transport ; Transport , Ion ) [SEP]'},
 {'mention_id': '28421738.37',
  'test_set_idx': 35215,
  'type': 'T103',
  'name': 'compound 12',
  'context': "[CLS] ##phi ##lic activation of the transcription ##al response . Moreover , 12 prevented the formation of c ##yt ##oto ##xi ##c stable o ##li ##go ##mer ##ic intermediate ##s , being significantly more effective , and per se less toxic , than prototype 1 . More importantly , as different chemical features were exploited to regulate Nr ##f ##2 and am ##yl ##oid - β p ##eptide activities , the two pathways could be tuned independently . These findings point to [unused1] compound 12 [unused2] and its derivatives as promising tools for investigating the therapeutic potential of the Nr ##f ##2 / am ##yl ##oid - β p ##eptide cellular network , laying foundation for generating new drug leads to confront Alzheimer ' s disease . [SEP]",
  'existing_cui': 'C3467761',
  'existing_cui_name': 'Compound 122587',
  'existing_cui_desc': '[CLS] Co ##mpo ##und 122 ##5 ##8 ##7 [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0620347',
  'our_bi_cui_name': 'compound A 12',
  'our_bi_cui_desc': "[CLS] compound A 12 [unused3] ( Chemical : compound a 12 ; a 12 ; 5 , 9 b ( 1 ' , 2 ' ) - Benz ##eno - 9 ##b ##H - ben ##z ( g ) in ##do ##l - 3 ( 3 ##a ##H ) - one , 1 , 2 , 4 , 5 - te ##tra ##hy ##dr ##o - 1 - ( p ##hen ##yl ##met ##hyl ) - ; A - 12 ) [SEP]",
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27298278.23',
  'test_set_idx': 1824,
  'type': 'T103',
  'name': 'F',
  'context': '[CLS] an ant ##hra ##quin ##one , and a nap ##ht ##ho ##hy ##dr ##o ##quin ##one dim ##er , together with three known q ##uin ##ones and seven other known compounds , were isolated from the aerial parts of Mo ##rin ##da par ##vi ##folia . The structures of m ##ori ##nda ##par ##vin ##s C , D , E , [unused1] F [unused2] , and G were el ##uc ##idated on the basis of s ##pect ##ros ##copic or X - ray di ##ff ##raction analysis as met ##hyl 4 - h ##ydro ##xy - 1 , 6 - dim ##eth ##ox ##y - nap ##ht ##hale ##ne - 2 - car ##box ##yla ##te , met ##hyl 4 , 8 - di ##hy ##dr [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0010207',
  'our_bi_cui_name': 'Coumarins',
  'our_bi_cui_desc': '[CLS] Co ##uma ##rin ##s [unused3] ( Chemical : co ##uma ##rin ##s ; Co ##uma ##rine ##s ; Der ##via ##tives , 1 , 2 - Benz ##opy ##rone ; 1 , 2 - Benz ##opy ##rone ##s ; Benz ##opy ##ran 2 ones ; 1 , 2 - Benz ##opy ##rone Der ##via ##tives ; Co ##uma ##rin ##s [ Chemical / In ##g ##red ##ient ] ; 1 , 2 Benz ##opy ##rone ##s ; Co ##uma ##rin Der ##iva ##tives ; Der ##iva ##tives , Co ##uma ##rin ; Benz ##opy ##ran - 2 - ones ; B ##EN ##Z ##OP ##Y ##RO ##NE ##S 01 02 ; 1 , 2 Benz ##o P ##yr ##ones ; 1 , 2 - Benz ##o [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27570140.44',
  'test_set_idx': 10507,
  'type': 'T058',
  'name': 'procedure',
  'context': "[CLS] ##tum tub ##al l ##iga ##tion request , there were 17 subsequent pre ##gna ##ncies ( 15 . 0 % ) during the 27 months of follow - up . Though women with high - risk pre ##gna ##ncies were more likely to request post ##par ##tum tub ##al l ##iga ##tion , they were not more likely to complete the [unused1] procedure [unused2] . Over one third of high - risk patients ' requests were un ##ful ##fill ##ed , indicating that significant barriers may remain . Though women with high - risk pre ##gna ##ncies were more likely to request post ##par ##tum tub ##al l ##iga ##tion , they were not more likely to complete the procedure . Pro ##vide ##rs should consider [SEP]",
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27832581.28',
  'test_set_idx': 18738,
  'type': 'T168',
  'name': 'SAT1',
  'context': '[CLS] maintenance control diet ; ( 2 ) a high - star ##ch diet rich in saturated fat ; and ( 3 ) a high - star ##ch diet rich in o ##me ##ga - 3 un ##sat ##ura ##ted fat . Two feeding sequences were used to test for carry - over effects : Group A was fed control diet , [unused1] SAT ##1 [unused2] and then UN ##SA ##T ##2 , whereas Group B was fed control diet , UN ##SA ##T ##1 and then SAT ##2 . Ser ##um was collected after each diet ##ary period , anal ##ys ##ed and tested in b ##ov ##ine in v ##it ##ro em ##b ##ryo culture . In ##tro ##ducing saturated fat and un ##sat ##ura [SEP]',
  'existing_cui': 'C1420161',
  'existing_cui_name': 'SLC26A1 gene',
  'existing_cui_desc': '[CLS] SL ##C ##26 ##A ##1 gene [unused3] ( Ana ##tom ##ical Structure : S ##U ##LF ##AT ##E AN ##ION T ##RA ##NS ##PO ##RT ##ER 1 ; S ##OL ##UT ##E CA ##R ##RI ##ER FA ##MI ##L ##Y 26 ( S ##U ##LF ##AT ##E T ##RA ##NS ##PO ##RT ##ER ) , ME ##MB ##ER 1 ; E ##DM ##4 ; so ##lut ##e carrier family 26 member 1 ; SAT - 1 ) [SEP]',
  'our_bi_cui': 'C0445244',
  'our_bi_cui_name': 'Serotype SAT1',
  'our_bi_cui_desc': '[CLS] Ser ##otype SAT ##1 [unused3] ( Intel ##lect ##ual Product , qualifier value : SAT ##1 ) [SEP]',
  'our_cross_cui': 'C0597423',
  'our_cross_cui_name': 'Saturated fat',
  'our_cross_cui_desc': '[CLS] Sa ##tura ##ted fat [unused3] ( Food , substance : saturated fatty acid ; fat ##s saturated ; saturated fatty acids ; acids fatty saturated ; Sa ##tura ##ted fatty acid ; Sa ##tura ##ted Fat ##ty A ##cid ##s ; A ##cid ##s , Sa ##tura ##ted Fat ##ty ; Fat ##ty A ##cid ##s , Sa ##tura ##ted ; saturated fat ; Sa ##tura ##ted Fat ##ty A ##cid ; Fat ##ty acids . saturated ; fat saturated ) [SEP]'},
 {'mention_id': '27913563.8',
  'test_set_idx': 21797,
  'type': 'T082',
  'name': 'Yorkshire',
  'context': '[CLS] - life balance of doctors in training - a q ##ual ##itative study In ##ves ##ti ##gate the work - life balance of doctors in training in the UK from the perspectives of trainers and train ##ees . Q ##ual ##itative semi ##structure ##d focus groups and interviews with train ##ees and trainers . Post ##graduate medical training in London , [unused1] Yorkshire [unused2] and Hu ##mber , Kent , Surrey and Sussex , and Wales during the junior doctor contract dispute at the end of 2015 . Part of a larger General Medical Council study about the fair ##ness of postgraduate medical training . 96 train ##ees and 41 trainers . Train ##ees comprised UK graduates and International Medical Graduate ##s , across all stages [SEP]',
  'existing_cui': 'C0324292',
  'existing_cui_name': 'Yorkshire pig',
  'existing_cui_desc': '[CLS] Yorkshire pig [unused3] ( E ##uka ##ryo ##te , organism ) [SEP]',
  'our_bi_cui': 'C0454874',
  'our_bi_cui_name': 'South Yorkshire',
  'our_bi_cui_desc': '[CLS] South Yorkshire [unused3] ( Spa ##tial Concept , geographic location : S York ##s - South Yorkshire ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '27570140.53',
  'test_set_idx': 10516,
  'type': 'T058',
  'name': 'procedure',
  'context': "[CLS] the 27 months of follow - up . Though women with high - risk pre ##gna ##ncies were more likely to request post ##par ##tum tub ##al l ##iga ##tion , they were not more likely to complete the procedure . Over one third of high - risk patients ' requests were un ##ful ##fill ##ed , indicating that significant barriers may remain . Though women with high - risk pre ##gna ##ncies were more likely to request post ##par ##tum tub ##al l ##iga ##tion , they were not more likely to complete the procedure . Pro ##vide ##rs should consider these procedures urgent , especially in high - risk women , and advocate for their patients ' access to this [unused1] procedure [unused2] . [SEP]",
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '28437231.11',
  'test_set_idx': 35877,
  'type': 'T017',
  'name': 'blaKPC',
  'context': '[CLS] ##s and V ##ir ##ule ##nce - Re ##lated T ##rai ##ts of E ##pid ##em ##ic K ##PC - Pro ##ducing K ##le ##bs ##iel ##la pneumonia ##e Is ##olate ##s in a Chinese University Hospital K ##le ##bs ##iel ##la pneumonia ##e is an important human path ##ogen associated with a variety of diseases and the prevalence of [unused1] b ##la ##K ##PC [unused2] carrying K . pneumonia ##e ( K ##PC - K ##p ) is rapidly increasing . Cap ##sul ##e is an important v ##ir ##ule ##nce factor in K . pneumonia ##e . In this study , we determined to first systematically character ##ize caps ##ular p ##oly ##sa ##cc ##hari ##de and v ##ir ##ule ##nce traits in K [SEP]',
  'existing_cui': 'C1855645',
  'existing_cui_name': 'Keratoconus posticus circumscriptus',
  'existing_cui_desc': '[CLS] Ke ##rato ##con ##us post ##icus c ##ir ##cum ##s ##cript ##us [unused3] ( B ##iol ##og ##ic Fun ##ction : K ##ER ##AT ##OC ##ON ##US P ##OS ##TI ##CU ##S C ##IR ##CU ##MS ##CR ##IP ##TU ##S ) [SEP]',
  'our_bi_cui': 'C1531163',
  'our_bi_cui_name': 'beta-lactamase KPC-3 , Klebsiella pneumoniae',
  'our_bi_cui_desc': '[CLS] beta - la ##ct ##ama ##se K ##PC - 3 , K ##le ##bs ##iel ##la pneumonia ##e [unused3] ( Chemical : b ##la ( K ##PC - 3 ) , K ##le ##bs ##iel ##la pneumonia ##e ; K ##PC - 3 beta - la ##ct ##ama ##se , K ##le ##bs ##iel ##la pneumonia ##e ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27587730.39',
  'test_set_idx': 10778,
  'type': 'T103',
  'name': 'gga-miR-21',
  'context': '[CLS] and decreased end ##ogen ##ous K ##rup ##pel - like factor 5 expression in primary pre - ad ##ip ##ocytes . K ##rup ##pel - like factor 5 knock ##down using RNA ##i had a similar effect to that of the g ##ga - mi ##R - 21 mimic on cell proliferation . The promoting effect of the [unused1] g ##ga - mi ##R - 21 [unused2] inhibitor on pre - ad ##ip ##oc ##yte proliferation was partially at ##ten ##uated by K ##rup ##pel - like factor 5 knock ##down . Take ##n together , our results demonstrated that mi ##R - 21 in ##hibit ##s chicken pre - ad ##ip ##oc ##yte proliferation , at least in part , by targeting K ##rup ##pel [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C2825314',
  'our_bi_cui_name': 'MicroRNA Gene',
  'our_bi_cui_desc': '[CLS] Micro ##RNA Gene [unused3] ( Ana ##tom ##ical Structure : Micro RNA Gene ; Mi ##RNA Gene ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '27859998.39',
  'test_set_idx': 19634,
  'type': 'T033',
  'name': 'normative levels',
  'context': '[CLS] for women , reduced no ##go - related the ##ta - band medial frontal cortex power and medial frontal cortex - dorsal pre ##front ##al cortex connectivity were both consistent with the potential del ##eter ##ious ca ##usal effects of alcohol exposure . These findings suggest that diminished the ##ta - band medial frontal cortex power and medial frontal cortex - dorsal pre ##front ##al cortex connectivity may be ne ##uro ##phy ##sio ##logical mechanisms underlying alcohol - related di ##sin ##hibition . Although preliminary , these results suggest that [unused1] norm ##ative levels [unused2] of alcohol use during emerging adulthood have potential sex - specific ca ##usal effects on response in ##hibition E ##EG dynamics , and thus have potentially significant public health implications . [SEP]',
  'existing_cui': 'C1547707',
  'existing_cui_name': 'Floor - story of building',
  'existing_cui_desc': '[CLS] Floor - story of building [unused3] ( Spa ##tial Concept : Level ; Story ; Floor ) [SEP]',
  'our_bi_cui': 'C1551394',
  'our_bi_cui_name': 'normal Device Alert Level',
  'our_bi_cui_desc': '[CLS] normal Devi ##ce Al ##ert Level [unused3] ( Finding : Normal ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28257468.26',
  'test_set_idx': 30272,
  'type': 'T103',
  'name': 'non - coding regions',
  'context': '[CLS] ##phism ##s associated with differences between lines in both their mean survival times and micro ##en ##vir ##on ##mental plastic ##ity , suggesting that lines differ in their ability to adapt to variable path ##ogen exposure ##s . The majority of p ##oly ##mor ##phism ##s increasing resistance to Ma ##5 ##4 ##9 were sex bias ##ed , located in [unused1] non - coding regions [unused2] , had moderately large effect and were rare , suggesting that there is a general cost to defense . Nevertheless , host defense was not negatively correlated with overall long ##evity and f ##ec ##und ##ity . In contrast to Ma ##5 ##4 ##9 , minor all ##ele ##s were concentrated in the most Pa ##14 - susceptible as well [SEP]',
  'existing_cui': 'C0079941',
  'existing_cui_name': 'Open Reading Frames',
  'existing_cui_desc': '[CLS] Open Reading Fr ##ame ##s [unused3] ( Ana ##tom ##ical Structure , OR ##F : OR ##F ##s ; Region , Pro ##tein Co ##ding ; Pro ##tein Co ##ding Se ##quence ; Co ##ding Se ##quence ; Pro ##tein Co ##ding Region ; Fr ##ame ##s , Open Reading ; Co ##ding Region ##s , Pro ##tein ; Region ##s , Pro ##tein Co ##ding ; or ##fs ; Reading Fr ##ame , Open ; Co ##ding Region ; Open Reading Fr ##ame ; Fr ##ame , Open Reading ; Co ##ding Feature ; Co ##ding Region , Pro ##tein ; open reading frame ; Pro ##tein Co ##ding Region ##s ; Reading Fr ##ame ##s , Open ) [SEP]',
  'our_bi_cui': 'C0079941',
  'our_bi_cui_name': 'Open Reading Frames',
  'our_bi_cui_desc': '[CLS] Open Reading Fr ##ame ##s [unused3] ( Ana ##tom ##ical Structure , OR ##F : OR ##F ##s ; Region , Pro ##tein Co ##ding ; Pro ##tein Co ##ding Se ##quence ; Co ##ding Se ##quence ; Pro ##tein Co ##ding Region ; Fr ##ame ##s , Open Reading ; Co ##ding Region ##s , Pro ##tein ; Region ##s , Pro ##tein Co ##ding ; or ##fs ; Reading Fr ##ame , Open ; Co ##ding Region ; Open Reading Fr ##ame ; Fr ##ame , Open Reading ; Co ##ding Feature ; Co ##ding Region , Pro ##tein ; open reading frame ; Pro ##tein Co ##ding Region ##s ; Reading Fr ##ame ##s , Open ) [SEP]',
  'our_cross_cui': 'C0021920',
  'our_cross_cui_name': 'Introns',
  'our_cross_cui_desc': '[CLS] In ##tron ##s [unused3] ( Chemical , finding : In ##tron ##ic ; Inter ##vening Se ##quence ##s ; Se ##quence , Inter ##vening ; SE ##Q IN ##TE ##R ##VE ##NI ##NG ; In ##tron ; Inter ##vening Se ##quence ; intro ##ns ; intro ##n ; Non - coding DNA sequence ; Se ##quence ##s , Inter ##vening ; IN ##TE ##R ##VE ##NI ##NG SE ##Q ) [SEP]'},
 {'mention_id': '27616751.36',
  'test_set_idx': 11894,
  'type': 'T038',
  'name': 'tolerant',
  'context': '[CLS] cells in the pre - se ##ns ##iti ##zed mice . CD ##25 + T - cell de ##ple ##tion in α - CD ##45 ##RB combined with Don ##or - specific s ##ple ##en cell trans ##fusion therapy recipients could prevent skin all ##og ##raft tolerance from establishing . In addition , adopt ##ive transfer of donor - prime ##d memory T cells into the [unused1] tolerant [unused2] recipients marked ##ly broke the established tolerance . Our findings indicate that α - CD ##45 ##RB and Don ##or - specific s ##ple ##en cell trans ##fusion can s ##yne ##rg ##istic ##ally in ##hibit the accelerated rejection mediated by memory T cells and induce long - term skin all ##og ##raft acceptance in mice . [SEP]',
  'existing_cui': 'C0344336',
  'existing_cui_name': 'Tolerant of cold',
  'existing_cui_desc': '[CLS] To ##ler ##ant of cold [unused3] ( Finding , finding ) [SEP]',
  'our_bi_cui': 'C0020963',
  'our_bi_cui_name': 'Immune Tolerance',
  'our_bi_cui_desc': '[CLS] I ##mm ##une To ##ler ##ance [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : I ##mm ##uno ##log ##ic para ##lysis - R ##ET ##IR ##ED - ; I ##mm ##uno ##log ##ic un ##res ##po ##ns ##ive ##ness ; I ##mm ##uno ##log ##ic un ##res ##po ##ns ##ive ##ness - R ##ET ##IR ##ED - ; I ##mm ##uno ##logical To ##ler ##ance ; im ##mu ##nological para ##lysis ; I ##mm ##uno ##log ##ic To ##ler ##ance ; Self To ##ler ##ance ; To ##ler ##ance , I ##mm ##une ; I ##mm ##uno ##logical tolerance ; I ##mm ##uno ##log ##ic tolerance , function ; immune tolerance ; im ##mu ##no ##log ##ic [SEP]',
  'our_cross_cui': 'C0887937',
  'our_cross_cui_name': 'Allograft Tolerance',
  'our_cross_cui_desc': '[CLS] All ##og ##raft To ##ler ##ance [unused3] ( B ##iol ##og ##ic Fun ##ction : To ##ler ##ance , All ##og ##raft ) [SEP]'},
 {'mention_id': '27506220.46',
  'test_set_idx': 8715,
  'type': 'T033',
  'name': 'cerebral palsy - related issues',
  'context': '[CLS] , e . g . participants with Gross Motor Fun ##ction Classification System level IV and V had a high level of medical input and a greater use of trunk - supporting devices , anti ##re ##f ##lux and la ##xa ##tive . Profile ##s could be established based on Gross Motor Fun ##ction Classification System levels . Adults with cerebral p ##als ##y use a large amount of drugs , mobility aids , or ##th ##otic devices , rehabilitation and medical input . Healthcare is targeted at [unused1] cerebral p ##als ##y - related issues [unused2] . Gross Motor Fun ##ction Classification System is a de ##ter ##mina ##nt of healthcare consumption and thus a useful tool for clinical practice to target care appropriately . [SEP]',
  'existing_cui': 'C0007789',
  'existing_cui_name': 'Cerebral Palsy',
  'existing_cui_desc': '[CLS] Ce ##re ##bra ##l Pa ##ls ##y [unused3] ( B ##iol ##og ##ic Fun ##ction , CP , Ce ##re ##bra ##l Pa ##ls ##y , diagnosis , disorder : Pa ##ls ##y ; cerebral ; Ce ##re ##bra ##l p ##als ##y , un ##sp ##ec ##ified ; Ce ##re ##bra ##l p ##als ##y [ Am ##bi ##guous ] ; Pa ##ls ##y cerebral ; para ##lysis ; cerebral ; In ##fant ##ile cerebral p ##als ##y ; Ce ##re ##bra ##l p ##als ##y ; Ce ##re ##bra ##l para ##lysis ; cerebral para ##lysis ; Con ##gen ##ital cerebral p ##als ##y ; cerebral p ##als ##y ; CP - Ce ##re ##bra ##l p ##als ##y ; cerebral ; para ##lysis [SEP]',
  'our_bi_cui': 'C1299636',
  'our_bi_cui_name': 'Cognitive safety issue',
  'our_bi_cui_desc': '[CLS] Co ##gni ##tive safety issue [unused3] ( Finding , finding ) [SEP]',
  'our_cross_cui': 'C0033213',
  'our_cross_cui_name': 'Problem',
  'our_cross_cui_desc': '[CLS] Problem [unused3] ( Finding , finding : Issue ; problem ) [SEP]'},
 {'mention_id': '27534734.29',
  'test_set_idx': 9712,
  'type': 'T170',
  'name': 'Hadamard combinations',
  'context': '[CLS] a T ##E of 80 ##ms , 20 - m ##s editing pulses are applied at 4 . 56 pp ##m ( on G ##S ##H ) , 1 . 9 pp ##m ( on GA ##BA ) , both offset ##s ( using a dual - lobe co ##sin ##e - m ##od ##ulated pulse ) or neither . [unused1] Had ##ama ##rd combinations [unused2] of the four sub - experiments yield GA ##BA and G ##S ##H difference s ##pect ##ra . It is shown that Had ##ama ##rd En ##co ##ding and Reconstruction of Mega - Edit ##ed S ##pect ##ros ##copy gives excellent separation of the edited GA ##BA and G ##S ##H signals in p ##han ##tom ##s , and resulting [SEP]',
  'existing_cui': 'C0628845',
  'existing_cui_name': 'chlorpromazine , dexamethasone , metoclopramide , orphenadrine drug combination',
  'existing_cui_desc': '[CLS] ch ##lor ##p ##roma ##zine , de ##xa ##met ##has ##one , met ##oc ##lop ##ram ##ide , or ##phe ##na ##dr ##ine drug combination [unused3] ( Chemical : anti ##em ##etic combination CO ##MD ; CO ##MD combination ; ch ##lor ##p ##roma ##zine - de ##xa ##met ##has ##one - met ##oc ##lop ##ram ##ide - or ##phe ##na ##dr ##ine ) [SEP]',
  'our_bi_cui': 'C0002045',
  'our_bi_cui_name': 'algorithm',
  'our_bi_cui_desc': '[CLS] algorithm [unused3] ( Intel ##lect ##ual Product : Al ##gor ##ith ##m ; Al ##gor ##ith ##ms ; algorithms ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27457539.17',
  'test_set_idx': 6416,
  'type': 'T038',
  'name': 'pregnant',
  'context': '[CLS] vaccine ( C ##L ##13 ##T ) in came ##ls R ##ift Valley fever is an emerging zoo ##not ##ic viral disease , en ##zo ##otic and endemic in Africa and the Arabian Peninsula , which poses a significant threat to both human and animal health . The disease is most severe in r ##umi ##nant ##s causing abortion ##s in [unused1] pregnant [unused2] animals , especially sheep animals and high mortality in young populations . High mortality rates and severe clinical manifest ##ation have also been reported among came ##l populations in Africa , to attend however none of the currently available live vaccine ##s against R ##V ##F have been tested for safety and efficacy in this species . In this study , the [SEP]',
  'existing_cui': 'C0549206',
  'existing_cui_name': 'Patient currently pregnant',
  'existing_cui_desc': '[CLS] Pat ##ient currently pregnant [unused3] ( Finding , finding , history : Pre ##gnant ; currently pregnant ; Currently pregnant ; Pre ##gna ##ncy not delivered ; G ##ra ##vid ##ity ; G ##ra ##vid ; Pat ##ient pregnant ; pregnant patient ; PR ##EG ##NA ##NT ; pregnant ; g ##ra ##vid ) [SEP]',
  'our_bi_cui': 'C0033011',
  'our_bi_cui_name': 'Pregnant Women',
  'our_bi_cui_desc': '[CLS] Pre ##gnant Women [unused3] ( Population Group , person : Woman , Pre ##gnant ; pregnant female ; W ##OM ##EN PR ##EG ##N ; Women , Pre ##gnant ; PR ##EG ##N W ##OM ##EN ; pregnant woman ; Pre ##gnant woman ; Pre ##gnant women ; Pre ##gnant Woman ; pregnant women ) [SEP]',
  'our_cross_cui': 'C0032961',
  'our_cross_cui_name': 'Pregnancy',
  'our_cross_cui_desc': '[CLS] Pre ##gna ##ncy [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , o ##bs ##er ##vable entity , qualifier value , s ##ym ##pt ##om : G ##esta ##tion ; Pre ##gna ##ncies ; PR ##EG ##N ; g ##esta ##tion ; PR ##EG ##NA ##NC ##Y ; pre ##gna ##ncies ; female pregnancy ; Pre ##gna ##ncy , function ; pregnancy ; Pre ##gna ##ncy [ Disease / Finding ] ) [SEP]'},
 {'mention_id': '28071754.21',
  'test_set_idx': 24726,
  'type': 'T037',
  'name': 'injury',
  'context': '[CLS] be activated by low - temperature far - infrared i ##rra ##diation to ex ##ert beneficial effects on the vascular end ##oth ##eli ##um . In the present study , we investigated the influence of far - infrared - induced Pro ##my ##elo ##cy ##tic le ##uke ##mia zinc finger protein activation on AG ##E - induced end ##oth ##eli ##al [unused1] injury [unused2] both in v ##it ##ro and in v ##ivo . far - infrared i ##rra ##diation in ##hibit ##ed AG ##E - induced a ##pop ##tosis in human um ##bil ##ical vein end ##oth ##eli ##al cells . Pro ##my ##elo ##cy ##tic le ##uke ##mia zinc finger protein activation increased the expression of p ##hos ##pha ##ti ##dy ##lino ##si ##to [SEP]',
  'existing_cui': 'C0005604',
  'existing_cui_name': 'Birth Injuries',
  'existing_cui_desc': '[CLS] Birth In ##ju ##ries [unused3] ( In ##ju ##ry or Po ##ison ##ing , Birth , diagnosis , disorder : In ##ju ##ries , Birth ; T ##ra ##umatic birth ; injury birth ; Birth trauma , un ##sp ##ec ##ified ; Birth T ##ra ##uma ; Birth trauma of f ##etus ; injury ; birth ; traumatic birth ; birth trauma ; birth injuries ; Birth In ##ju ##ry ; trauma birth ; Neo ##nat ##al Birth In ##ju ##ry ; birth ##ing injuries ; Birth In ##ju ##ries [ Disease / Finding ] ; Un ##sp ##ec ##ified birth trauma ; birth ##ing trauma ; B ##IR ##TH IN ##J ; IN ##J B ##IR ##TH ; Birth injuries ; birth ; injury ; [SEP]',
  'our_bi_cui': 'C0010957',
  'our_bi_cui_name': 'Tissue damage',
  'our_bi_cui_desc': '[CLS] T ##iss ##ue damage [unused3] ( In ##ju ##ry or Po ##ison ##ing , m ##or ##ph ##olo ##gic abnormal ##ity : damage tissues ; Dam ##age ; damages tissue ; Dam ##age , T ##iss ##ue ; tissue damage ; T ##iss ##ue Dam ##age ; damage ; damages ) [SEP]',
  'our_cross_cui': 'C0178314',
  'our_cross_cui_name': 'Poisoning / injury',
  'our_cross_cui_desc': '[CLS] Po ##ison ##ing / injury [unused3] ( In ##ju ##ry or Po ##ison ##ing , disorder , finding , navigation ##al concept : In ##ju ##ry & / or poisoning ; IN ##J ##UR ##Y AND P ##O ##IS ##ON ##ING ; In ##ju ##ry and poisoning ; In ##ju ##ry / poisoning ) [SEP]'},
 {'mention_id': '28550165.51',
  'test_set_idx': 38995,
  'type': 'T103',
  'name': 'G2L1',
  'context': '[CLS] ##formatics tool Sign ##ific ##ance Analysis of Inter ##act ##ome . We discovered that C ##LA ##SP ##2 co - im ##mu ##no ##p ##re ##ci ##pit ##ates the novel protein S ##O ##GA ##1 , the micro ##tub ##ule - associated protein kinase MA ##R ##K ##2 , and the micro ##tub ##ule / act ##in - regulating protein [unused1] G ##2 ##L ##1 [unused2] . The GT ##P ##ase - act ##ivating proteins AG ##AP ##1 and AG ##AP ##3 were also en ##rich ##ed in the C ##LA ##SP ##2 interact ##ome , although subsequent AG ##AP ##3 and C ##L ##IP ##2 interact ##ome analysis suggests a preference of AG ##AP ##3 for C ##L ##IP ##2 . Follow - up MA [SEP]',
  'existing_cui': 'C1415613',
  'existing_cui_name': 'HMGXB4 gene',
  'existing_cui_desc': '[CLS] HM ##G ##X ##B ##4 gene [unused3] ( Ana ##tom ##ical Structure : T ##HC ##21 ##16 ##30 ; HM ##G B ##OX D ##OM ##A ##IN - CO ##NT ##A ##IN ##ING 4 ; HM ##G - box containing 4 ; H ##IG ##H M ##O ##BI ##L ##IT ##Y G ##RO ##UP PR ##OT ##EI ##N 2 - L ##I ##KE 1 ) [SEP]',
  'our_bi_cui': 'C1314812',
  'our_bi_cui_name': 'GNB2L1 protein',
  'our_bi_cui_desc': '[CLS] G ##N ##B ##2 ##L ##1 protein [unused3] ( Chemical : RA ##C ##K - 1 ( receptor for activated C kinase ) protein ; G ##N ##B ##2 - RS ##1 protein ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28105138.37',
  'test_set_idx': 25669,
  'type': 'T038',
  'name': 'diabetic',
  'context': '[CLS] that were analyzed , the interval between s ##ym ##pt ##om onset and re ##per ##fusion therapy ( window period ) varied from 0 . 5 to 24 h . The h ##s - CR ##P value was found to be higher in non - di ##abe ##tic patients ( 0 . 61 mg / d ##l ) compared with [unused1] di ##abe ##tic [unused2] patients ( 0 . 87 mg / d ##l ) . Moreover , a significant correlation between h ##s - CR ##P and h ##s - t ##rop ##oni ##n T was also recorded ( P < 0 . 00 ##1 ) . However , there was no significant difference in the mean h ##s - CR ##P values in patients [SEP]',
  'existing_cui': 'C0241863',
  'existing_cui_name': 'Diabetic',
  'existing_cui_desc': '[CLS] Di ##abe ##tic [unused3] ( Finding : di ##abe ##tics ; di ##abe ##tic ) [SEP]',
  'our_bi_cui': 'C0241863',
  'our_bi_cui_name': 'Diabetic',
  'our_bi_cui_desc': '[CLS] Di ##abe ##tic [unused3] ( Finding : di ##abe ##tics ; di ##abe ##tic ) [SEP]',
  'our_cross_cui': 'C0011847',
  'our_cross_cui_name': 'Diabetes',
  'our_cross_cui_desc': '[CLS] Di ##abe ##tes [unused3] ( B ##iol ##og ##ic Fun ##ction : diabetes ) [SEP]'},
 {'mention_id': '27298278.40',
  'test_set_idx': 1841,
  'type': 'T017',
  'name': 'A2780',
  'context': "[CLS] ' - di ##car ##box ##yla ##te , respectively . Na ##ph ##th ##o ##quin ##ones and nap ##ht ##ho ##hy ##dr ##o ##quin ##one dim ##ers were previously unknown in the genus Mo ##rin ##da . In addition , the compounds were tested for c ##yt ##oto ##xi ##city against four human cancer cell lines He ##L ##a , [unused1] A2 ##7 ##80 [unused2] , Ke ##tr ##3 and MC ##F - 7 and their effects on p ##53 - activated transcription . Three nap ##ht ##ho ##quin ##ones had moderate c ##yt ##oto ##xi ##c effects with I ##C ##50 values ranging from 1 . 51 to 9 . 56 μ ##M , through up - regulation of p ##53 transcription ##al activity . [SEP]",
  'existing_cui': 'C1504999',
  'existing_cui_name': 'PA 2789',
  'existing_cui_desc': '[CLS] PA 278 ##9 [unused3] ( Chemical : PA - 278 ##9 ; PA ##27 ##8 ##9 ) [SEP]',
  'our_bi_cui': 'C4277577',
  'our_bi_cui_name': 'A549 Cells',
  'our_bi_cui_desc': '[CLS] A ##5 ##4 ##9 Cell ##s [unused3] ( Ana ##tom ##ical Structure : Cell , A ##5 ##4 ##9 ; Cell Lines , A ##5 ##4 ##9 ; Cell Line , A ##5 ##4 ##9 ; A ##5 ##4 ##9 Cell Lines ; A ##5 ##4 ##9 Cell ; A ##5 ##4 ##9 Cell Line ; Cell ##s , A ##5 ##4 ##9 ) [SEP]',
  'our_cross_cui': 'C0085983',
  'our_cross_cui_name': 'Cell Line , Tumor',
  'our_cross_cui_desc': '[CLS] Cell Line , Tu ##mor [unused3] ( Ana ##tom ##ical Structure : Cell Lines , Tu ##mor ; Tu ##mor Cell Line ; Line , Tu ##mor Cell ; Lines , Tu ##mor Cell ; Tu ##mor Cell Lines ; tumor cell lines ) [SEP]'},
 {'mention_id': '27266283.17',
  'test_set_idx': 1107,
  'type': 'T103',
  'name': 'DR-70',
  'context': '[CLS] Pat ##ients with lung cancer are usually diagnosed at advanced or locally advanced stage , for this reason early diagnosis of lung cancer is very important . For early detection of lung cancer some methods are emphasized such as low - dose com ##puted to ##mo ##graphy or tumor bio ##mark ##ers . In this study we aimed to evaluate [unused1] DR - 70 [unused2] sensitivity and specific ##ity as a tumor marker in detection of non - small cell lung cancer ##s . Between May 2013 and April 2014 , the serum samples from 88 non lung cancer patients , 86 patients with chronic o ##bs ##truct ##ive pulmonary di ##ses ##ase were obtained . Blood samples from each participant were analyzed for DR - [SEP]',
  'existing_cui': 'C0127021',
  'existing_cui_name': 'MAR 70',
  'existing_cui_desc': '[CLS] MA ##R 70 [unused3] ( Chemical : MA ##R - 70 ; MA ##R ##70 ; 5 , 12 - Na ##ph ##tha ##cene ##dio ##ne , 8 - ace ##ty ##l - 10 - ( ( 3 - amino - 2 , 3 , 6 - t ##ride ##ox ##y - 4 - O - ( 2 , 6 - did ##eo ##xy - alpha - L - a ##rab ##ino - he ##x ##opy ##rano ##sy ##l ) - alpha - L - l ##yx ##o - he ##x ##opy ##rano ##sy ##l ) o ##xy ) - 7 , 8 , 9 , 10 - te ##tra ##hy ##dr ##o - 6 , 8 , 11 - t ##ri ##hy ##dr ##ox ##y [SEP]',
  'our_bi_cui': 'C0215991',
  'our_bi_cui_name': 'CD70 Antigen',
  'our_bi_cui_desc': '[CLS] CD ##70 Anti ##gen [unused3] ( Chemical , substance : Ki - 24 Anti ##gen ; T ##UM ##OR NE ##CR ##OS ##IS FA ##CT ##OR L ##IG ##AN ##D S ##UP ##ER ##FA ##MI ##L ##Y ME ##MB ##ER 00 ##7 ; CD ##27 ##L Pro ##tein ; CD ##70 Anti ##gens ; AN ##TI ##GE ##NS CD 07 ##0 ; Liga ##nd , CD ##27 ; T ##NF Super ##family , Member 7 ; CD 07 ##0 AN ##TI ##GE ##NS ; CD ##27 Liga ##nd ; Anti ##gen , CD ##70 ; Tu ##mor N ##ec ##rosis Factor Liga ##nd Super ##family Member 7 ; Anti ##gens , CD ##70 ; Anti ##gens , CD ##70 [ Chemical / In ##g ##red ##ient [SEP]',
  'our_cross_cui': 'C0041365',
  'our_cross_cui_name': 'Tumor Markers',
  'our_cross_cui_desc': '[CLS] Tu ##mor Mark ##ers [unused3] ( Chemical : tumor markers ; Tu ##mor Mark ##er ; Cancer B ##io ##mark ##ers ; Tu ##mor marker ; B ##io ##mark ##ers , Cancer ; Mark ##ers , Tu ##mor ; tumor marker ; Tu ##mor markers ) [SEP]'},
 {'mention_id': '28197156.31',
  'test_set_idx': 28101,
  'type': 'T017',
  'name': 'Carbon Assimilation Rate 8',
  'context': '[CLS] ##s ##HA ##P ##3 ##H . Se ##quencing analysis revealed that the in ##dic ##a all ##ele of Carbon As ##si ##mi ##lation Rate 8 has a 1 - b ##p del ##eti ##on at 322 b ##p from the start co ##don , resulting in a t ##runcated protein of 125 amino acids . In addition , [unused1] Carbon As ##si ##mi ##lation Rate 8 [unused2] is identical to D ##TH ##8 / G ##h ##d ##8 / L ##HD ##1 , which was reported to control rice flowering date . The increase of A is largely due to an increase of R ##u ##B ##P regeneration rate via increased leaf nitrogen content , and partially explained by reduced s ##tom ##ata ##l limitation via [SEP]',
  'existing_cui': 'C2246620',
  'existing_cui_name': 'iron assimilation',
  'existing_cui_desc': '[CLS] iron ass ##im ##ilation [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_bi_cui': 'C1413023',
  'our_bi_cui_name': 'C8A gene',
  'our_bi_cui_desc': '[CLS] C ##8 ##A gene [unused3] ( Ana ##tom ##ical Structure : complement component 8 alpha subunit ; CO ##MP ##LE ##ME ##NT CO ##MP ##ON ##EN ##T 8 , AL ##P ##HA S ##U ##B ##UN ##IT ; C ##8 AL ##P ##HA ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '28203067.34',
  'test_set_idx': 28191,
  'type': 'T033',
  'name': 'negative impact of care',
  'context': '[CLS] self - app ##rai ##sal of cop ##ing well as a care ##gi ##ver ; lower sense of general support ; more hours of care per week ; and the motor re ##tar ##dation of the cared - for person measured with the speed of the Time ##d Up and Go test . W ##orse performance on the Time ##d Up and Go test was only the patient - related predict ##or of distress among the variables examined as contributors to the higher [unused1] negative impact of care [unused2] . En ##han ##cing the mobility of g ##eria ##tric patients through suit ##ably tailored kin ##esi ##ther ##ap ##eu ##tic methods during their hospital stay may mit ##igate the burden endured by family care ##rs . [SEP]',
  'existing_cui': 'C0726639',
  'existing_cui_name': 'Impact food supplement',
  'existing_cui_desc': '[CLS] Impact food supplement [unused3] ( Food : Impact ; impact ; I ##MP ##AC ##T ) [SEP]',
  'our_bi_cui': 'C1514593',
  'our_bi_cui_name': 'Psychological Impact',
  'our_bi_cui_desc': '[CLS] Psychological Impact [unused3] ( Clinical At ##tri ##but ##e ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27234220.21',
  'test_set_idx': 277,
  'type': 'T033',
  'name': 'No',
  'context': '[CLS] for complications after the procedure ( bleeding and vascular access complications ) . Pat ##ients in group A were older ( 74 ± 12 years vs . 68 ± 17 years , p < 0 . 01 ) and the main indication for anti ##coa ##gu ##lation was at ##rial fi ##bri ##lla ##tion ( 58 . 6 % ) . [unused1] No [unused2] complications occurred during the procedures . There were no acute bleeding ##s just after the bandage removal . During the first 24 ##h , only 3 ( 2 . 1 % ) radial o ##cc ##lusion ##s in group A and 2 ( 0 . 6 % ) in group B ( p = 0 . 14 ) were recorded . He [SEP]',
  'existing_cui': 'C1016143',
  'existing_cui_name': 'Tetragenococcus halophilus',
  'existing_cui_desc': '[CLS] Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##us [unused3] ( Ba ##cter ##ium , organism : P ##ed ##io ##co ##cc ##us ha ##lop ##hil ##us Me ##es 1934 ( A ##pp ##roved List ##s 1980 ) ; Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##us ( Me ##es 1934 ) Collins et al . 1993 ; Sa ##rc ##ina ha ##ma ##guchi ##ae ; Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##a ; P ##ed ##io ##co ##cc ##us ha ##lop ##hil ##us ; Te ##tra ##co ##cc ##cus No . 1 ; P ##ed ##io ##co ##cc ##us so ##ya ##e ) [SEP]',
  'our_bi_cui': 'C0243095',
  'our_bi_cui_name': 'Finding',
  'our_bi_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]',
  'our_cross_cui': 'C1513916',
  'our_cross_cui_name': 'Negative Finding',
  'our_cross_cui_desc': '[CLS] N ##eg ##ative Finding [unused3] ( Finding : NE ##GA ##TI ##VE ; N ##eg ##ative ) [SEP]'},
 {'mention_id': '27545500.16',
  'test_set_idx': 9992,
  'type': 'T103',
  'name': '16-aldehyde tanshinone I',
  'context': '[CLS] au ##reus are prevalent . The dramatically reduced discovery of new anti ##biotics , as well as the persistent emergence of resistant bacteria , represents a major health problem in both hospital and community settings . Using anti ##biotic enhance ##rs to rescue existing classes of anti ##biotics is an attractive strategy . In this study , [unused1] 16 - al ##de ##hy ##de tan ##shin ##one I [unused2] was synthesized and b ##act ##eri ##ost ##atic activity was explored . In addition , s ##yne ##rg ##istic or add ##itive activity between 16 - al ##de ##hy ##de tan ##shin ##one I and amino ##gly ##cos ##ide anti ##biotics or β - la ##ct ##am anti ##biotics in v ##it ##ro was identified . Moreover [SEP]',
  'existing_cui': 'C0075810',
  'existing_cui_name': 'tanshinone',
  'existing_cui_desc': '[CLS] tan ##shin ##one [unused3] ( Chemical : 1 , 6 - dim ##eth ##yl ##phe ##nant ##hr ##o ( 1 , 2 - b ) fur ##an - 10 , 11 - di ##one ; tan ##shin ##one I ) [SEP]',
  'our_bi_cui': 'C0075810',
  'our_bi_cui_name': 'tanshinone',
  'our_bi_cui_desc': '[CLS] tan ##shin ##one [unused3] ( Chemical : 1 , 6 - dim ##eth ##yl ##phe ##nant ##hr ##o ( 1 , 2 - b ) fur ##an - 10 , 11 - di ##one ; tan ##shin ##one I ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27655308.12',
  'test_set_idx': 13170,
  'type': 'T092',
  'name': 'Food banks',
  'context': "[CLS] is key to involve this group in determining their communication barriers and information needs . O ##bject ##ive To gain insight into how people with low literacy use and evaluate written drug information , and to identify ways in which they feel the patient leaf ##let can be improved , and in particular how images could be used . Set ##ting [unused1] Food banks [unused2] and an education institution for Dutch language training in the Netherlands . Method Semi - structured focus groups and individual interviews were held with low - liter ##ate participants ( n = 45 ) . The them ##atic framework approach was used for analysis to identify themes in the data . Main outcome measure Low - liter ##ate people ' s [SEP]",
  'existing_cui': 'C0005770',
  'existing_cui_name': 'Blood Banks',
  'existing_cui_desc': '[CLS] Blood Banks [unused3] ( Organization : blood bank ; B ##L ##O ##OD BA ##N ##K ; Blood Bank ; Blood bank ; Banks , Blood ; Blood banks ; blood banks ; B ##LD ##B ##K ; Su ##pp ##liers ; Blood Bank ; Bank , Blood ) [SEP]',
  'our_bi_cui': 'C0016452',
  'our_bi_cui_name': 'Food',
  'our_bi_cui_desc': '[CLS] Food [unused3] ( Food , medication , substance : Food or Food Product ; foods ; F ##O ##OD OR F ##O ##OD PR ##OD ##UC ##T ; Food Product ; Food or Food Sub ##stance ; Food ##P ##rod ##uc ##t ; Foods ; food ) [SEP]',
  'our_cross_cui': 'C1561598',
  'our_cross_cui_name': 'entity - organization',
  'our_cross_cui_desc': '[CLS] entity - organization [unused3] ( Organization : organization ) [SEP]'},
 {'mention_id': '28295414.7',
  'test_set_idx': 31545,
  'type': 'T168',
  'name': 'alcohol',
  'context': '[CLS] ##ral St ##ability of Heavy Dr ##ink ##ing Days and Dr ##ink ##ing Red ##uction ##s Among Heavy Dr ##ink ##ers in the CO ##MB ##IN ##E Study Recently , the Food and Drug Administration proposed to expand the options for primary end points in the development of medications for alcohol use disorder to include either a ##bs ##tine ##nce from [unused1] alcohol [unused2] or a non ##ab ##st ##inen ##t outcome : no heavy drinking days ( with a heavy drinking day defined as more than 3 drinks per day for women and more than 4 drinks per day for men [ > 3 / > 4 cut ##off ] ) . The Food and Drug Administration also suggested that 6 months would be the [SEP]',
  'existing_cui': 'C0202304',
  'existing_cui_name': 'Ethanol measurement',
  'existing_cui_desc': '[CLS] Ethan ##ol measurement [unused3] ( Health Care Act ##ivity , procedure : DR ##U ##G SC ##RE ##EN Q ##U ##AN ##TA ##LC ##OH ##OL ##S ; Ethan ##ol Me ##as ##ure ##ment ; Al ##co ##hol ##s levels ; Al ##co ##hol ##s ; Ethan ##ol ; DR ##U ##G SC ##RE ##EN Q ##U ##AN ##TI ##TA ##TI ##VE AL ##CO ##H ##OL ##S ; Al ##co ##hol ; alcohol measurements ; Me ##as ##ure ##ment of alcohol ; E ##thy ##l Al ##co ##hol Me ##as ##ure ##ments ; Drug screen q ##uant ##al ##co ##hol ##s ; Al ##co ##hol level ; Al ##co ##hol measurement ; ET ##HA ##N ##OL ; alcohol level test ) [SEP]',
  'our_bi_cui': 'C0001975',
  'our_bi_cui_name': 'Alcohols',
  'our_bi_cui_desc': '[CLS] Al ##co ##hol ##s [unused3] ( Chemical , substance : alcohol ; Al ##co ##hol [ Chemical Class ] ; Al ##co ##hol ; Al ##co ##hol ##s [ Chemical / In ##g ##red ##ient ] ; alcohol ##s ; class of chemical alcohol ##s ; alcohol substance ) [SEP]',
  'our_cross_cui': 'C0001967',
  'our_cross_cui_name': 'Alcoholic Beverages',
  'our_cross_cui_desc': '[CLS] Al ##co ##hol ##ic Be ##ver ##ages [unused3] ( Food , Al ##co ##hol ##ic , substance : Be ##ver ##age , Al ##co ##hol ##ic ; Al ##co ##hol ##ic Be ##ver ##age ; Al ##co ##hol ##ic beverages ; alcoholic drinks ; Be ##ver ##ages ; Be ##ver ##ages , Al ##co ##hol ##ic ; alcoholic beverages ; alcoholic beverage ; alcoholic ##s beverages ; Al ##co ##hol ##ic beverage ; Al ##co ##hol ##ic drink ; alcoholic drinking ) [SEP]'},
 {'mention_id': '27384489.6',
  'test_set_idx': 4320,
  'type': 'T082',
  'name': 'R1628P polymorphism',
  'context': "[CLS] Relations ##hip between le ##uc ##ine - rich repeat kinase 2 R ##16 ##28 ##P p ##oly ##mor ##phism and Parkinson ' s disease in Asian populations Although the le ##uc ##ine - rich repeat kinase 2 [unused1] R ##16 ##28 ##P p ##oly ##mor ##phism [unused2] has been associated with the risk of Parkinson ' s disease in Taiwan , China , and Singapore , there are conflicting findings regarding this relationship . Thus , the aim of the present meta - analysis was to evaluate the associations between the le ##uc ##ine - rich repeat kinase 2 R ##16 ##28 ##P p ##oly ##mor ##phism ( r ##s ##33 ##9 ##4 ##9 ##39 ##0 ) and Parkinson ' s disease in Asian populations . [SEP]",
  'existing_cui': 'C0032529',
  'existing_cui_name': 'Genetic Polymorphism',
  'existing_cui_desc': '[CLS] Gene ##tic Pol ##ym ##or ##phism [unused3] ( B ##iol ##og ##ic Fun ##ction , Gene ##tics , finding : P ##OL ##Y ##MO ##RP ##H ##IS ##M GE ##NE ##T ; P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ; Gene ##tic p ##oly ##mor ##phism ; genetic p ##oly ##mor ##phism ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##MS ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ##s , Gene ##tic ; Gene ##tic p ##oly ##mor ##phism ##s ; p ##oly ##mor ##phism ; Pol ##ym ##or ##phism , Gene ##tic ; p ##oly ##mor ##phism ##s ; Gene ##tic Pol ##ym ##or ##phism ##s [SEP]',
  'our_bi_cui': 'C4288453',
  'our_bi_cui_name': 'PRSS2 Gene Mutation',
  'our_bi_cui_desc': '[CLS] PR ##SS ##2 Gene Mu ##tation [unused3] ( B ##iol ##og ##ic Fun ##ction : T ##R ##Y ##2 Gene Mu ##tation ; PR ##SS ##2 Mu ##tation ; Pro ##te ##ase , Ser ##ine , 2 Gene Mu ##tation ) [SEP]',
  'our_cross_cui': 'C0752046',
  'our_cross_cui_name': 'Single Nucleotide Polymorphism',
  'our_cross_cui_desc': '[CLS] Single N ##uc ##leo ##tide Pol ##ym ##or ##phism [unused3] ( Spa ##tial Concept , S ##NP : Pol ##ym ##or ##phism , Single N ##uc ##leo ##tide ; Single N ##uc ##leo ##tide Pol ##ym ##or ##phism ##s ; Pol ##ym ##or ##phism ##s , Single N ##uc ##leo ##tide ; S ##NP ##s ; N ##uc ##leo ##tide Pol ##ym ##or ##phism ##s , Single ; N ##uc ##leo ##tide Pol ##ym ##or ##phism , Single ; single n ##uc ##leo ##tide p ##oly ##mor ##phism ; single n ##uc ##leo ##tide p ##oly ##mor ##phism ##s ; Pol ##ym ##or ##phism , Single Base ) [SEP]'},
 {'mention_id': '28358712.50',
  'test_set_idx': 33201,
  'type': 'T058',
  'name': 'Phytochemical tests',
  'context': '[CLS] that E . an ##gus ##ti ##folia extract significantly reduced p ##aw ed ##ema and inflammation in a dose - dependent manner . The results also showed that the measurement by digital ca ##lip ##er and mercury p ##let ##hy ##smo ##meter were both reliable and might be applied interchange ##ably ( p < 0 . 01 ) . [unused1] Ph ##yt ##och ##em ##ical tests [unused2] indicated that the h ##ydro ##al ##co ##hol ##ic fruit extract of E . an ##gus ##ti ##folia was positive for cardiac g ##ly ##cos ##ides , fl ##av ##ono ##ids , te ##rp ##eno ##ids , and sa ##po ##nin ##s . Based on our findings , the E . an ##gus ##ti ##folia fruit extract probably has [SEP]',
  'existing_cui': 'C0577749',
  'existing_cui_name': 'Phytochemicals',
  'existing_cui_desc': '[CLS] Ph ##yt ##och ##em ##ical ##s [unused3] ( Chemical , substance : Ph ##yton ##ut ##rient ; Diet ##ary Ph ##yt ##och ##em ##ical ##s ; Ph ##yt ##och ##em ##ical ##s [ Chemical / In ##g ##red ##ient ] ; Plant - Der ##ive ##d Chemical ##s ; Plant - Der ##ive ##d Co ##mpo ##unds ; Co ##mpo ##unds , Plant - Der ##ive ##d ; p ##hy ##to ##chemical ; Ph ##yt ##och ##em ##ical ##s , Diet ##ary ; Ph ##yt ##och ##em ##ical ; Plant Der ##ive ##d Chemical ##s ; Chemical ##s , Plant - Der ##ive ##d ; p ##hy ##to ##chemical ##s ; Ph ##yton ##ut ##rient ##s ; Plant Der ##ive ##d Co ##mpo ##unds ) [SEP]',
  'our_bi_cui': 'C0201682',
  'our_bi_cui_name': 'Chemical procedure',
  'our_bi_cui_desc': '[CLS] Chemical procedure [unused3] ( Health Care Act ##ivity , procedure : UN ##L ##IS ##TE ##D CH ##EM ##IS ##TR ##Y PR ##OC ##ED ##UR ##E ; Chemistry Me ##as ##ure ##ment ; chemical tests ; C ##L ##IN ##IC ##AL CH ##EM ##IS ##TR ##Y T ##ES ##T ; Chemistry procedure ; Chemistry Test ; test chemical ; Chemistry procedures ; Chemical measurement ; chemical test ; Chemistry Tech ##ni ##que ; Chemistry ; Un ##listed chemistry procedure ; Chemistry Pro ##ced ##ures ; Chemical test ; chemical testing ; chemicals testing ; Clinical chemistry test ; CH ##EM ; Chemistry Pro ##ced ##ure ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '27402827.8',
  'test_set_idx': 4852,
  'type': 'T038',
  'name': 'formation',
  'context': '[CLS] W ##nt - 5 ##a / Fr ##iz ##zled ##9 receptor signaling through the G ##α ##o / G ##β ##γ complex regulate ##s den ##dr ##itic spine [unused1] formation [unused2] W ##nt l ##igan ##ds play crucial roles in the development and regulation of s ##yna ##pse structure and function . Specifically , W ##nt - 5 ##a acts as a secret ##ed growth factor that regulate ##s den ##dr ##itic spine formation in rode ##nt hip ##po ##cam ##pal neurons , resulting in posts ##yna ##ptic development that promotes the cluster ##ing of the posts ##yna ##ptic density protein - 95 . Here , we focused on the early events occurring after the interaction between W ##nt - 5 ##a and its Fr ##iz [SEP]',
  'existing_cui': 'C0220781',
  'existing_cui_name': 'Anabolism',
  'existing_cui_desc': '[CLS] Ana ##bol ##ism [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : bio ##sy ##nt ##hetic process ; Ana ##bol ##ism , function ; synthesis ; bio ##sy ##nt ##hesis ; formation ; B ##ios ##ynth ##etic ; B ##ios ##ynth ##esis ; B ##io ##genesis ; B ##ios ##ynth ##etic Process ; an ##ab ##olis ##m ) [SEP]',
  'our_bi_cui': 'C0597545',
  'our_bi_cui_name': 'synaptogenesis',
  'our_bi_cui_desc': '[CLS] s ##yna ##pt ##ogen ##esis [unused3] ( B ##iol ##og ##ic Fun ##ction : s ##yna ##pse bio ##genesis ; s ##yna ##pse assembly ; s ##yna ##pse formation ) [SEP]',
  'our_cross_cui': 'C0007613',
  'our_cross_cui_name': 'Cell physiology',
  'our_cross_cui_desc': '[CLS] Cell physiology [unused3] ( B ##iol ##og ##ic Fun ##ction : cell physiology ; Cell ##ular Process ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##a ; Cell ##ular Fun ##ction ; cells physiology ; Cell ##ular Ph ##ys ##iology ; Sub ##cellular Process ; cells function ; cell function ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##on ; Cell Ph ##ys ##iology ; Cell Process ; Cell Fun ##ction ; CE ##LL P ##H ##Y ##SI ##OL ; P ##H ##Y ##SI ##OL CE ##LL ; cell functions ; Ph ##eno ##men ##a , Cell Ph ##ys ##iol ##ogical ; Ph ##ys ##iology , Cell ; Ph ##eno ##men ##on , Cell Ph ##ys ##iol ##ogical ) [SEP]'},
 {'mention_id': '27671177.8',
  'test_set_idx': 13785,
  'type': 'T082',
  'name': 'Quetta',
  'context': '[CLS] Year School Children in Mode ##rate - F ##lu ##ori ##de Dr ##ink ##ing Water Community in Que ##tta , Pakistan To determine the prevalence of dental car ##ies and its relationship with dental flu ##oro ##sis , oral health behaviour and diet ##ary behaviour among 12 - year school children in moderate - flu ##ori ##de drinking water community in [unused1] Que ##tta [unused2] , Pakistan . Cross - section ##al study . Government and private schools of Que ##tta , from November 2012 to February 2013 . At ##ota ##l of 34 ##9 children aged 12 - year from 14 randomly selected schools were included . The data collection was done on question ##naire designed for children . Dental car ##ies status was examined [SEP]',
  'existing_cui': 'C0262345',
  'existing_cui_name': 'Tenia of thalamus',
  'existing_cui_desc': '[CLS] Ten ##ia of th ##ala ##mus [unused3] ( Ana ##tom ##ical Structure : ten ##ia of th ##ala ##mi ; Ten ##ia of th ##ala ##mi ; Ta ##enia th ##ala ##mi ; T ##hala ##mus ten ##ia ; ten ##ia of the th ##ala ##mus ; Ten ##ia th ##ala ##mi ) [SEP]',
  'our_bi_cui': 'C3828424',
  'our_bi_cui_name': 'Queretaro',
  'our_bi_cui_desc': '[CLS] Que ##reta ##ro [unused3] ( Spa ##tial Concept : Que ##reta ##ro de Arte ##aga ; Free and Sovereign State of Que ##ré ##tar ##o ; Que ##ré ##tar ##o de Arte ##aga ; Que ##ré ##tar ##o ; Free and Sovereign State of Que ##reta ##ro ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '27587730.37',
  'test_set_idx': 10776,
  'type': 'T103',
  'name': 'gga-miR-21',
  'context': "[CLS] ##hibit ##ed K ##rup ##pel - like factor 5 3 ( ' ) U ##TR reporter activity and decreased end ##ogen ##ous K ##rup ##pel - like factor 5 expression in primary pre - ad ##ip ##ocytes . K ##rup ##pel - like factor 5 knock ##down using RNA ##i had a similar effect to that of the [unused1] g ##ga - mi ##R - 21 [unused2] mimic on cell proliferation . The promoting effect of the g ##ga - mi ##R - 21 inhibitor on pre - ad ##ip ##oc ##yte proliferation was partially at ##ten ##uated by K ##rup ##pel - like factor 5 knock ##down . Take ##n together , our results demonstrated that mi ##R - 21 in ##hibit ##s chicken pre [SEP]",
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1537719',
  'our_bi_cui_name': 'MIR21 gene',
  'our_bi_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '28457334.32',
  'test_set_idx': 36642,
  'type': 'T103',
  'name': 'extracellular RNA',
  'context': '[CLS] h . Level ##s of Cell - free DNA and extra ##cellular RNA were measured by quantitative probe - based polymer ##ase chain reaction . In addition , th ##rom ##boe ##last ##ography for co ##agu ##lation as well as th ##rom ##boa ##gg ##re ##go ##metry for plate ##let function was conducted . Both Cell - free DNA and [unused1] extra ##cellular RNA [unused2] were elevated in patients with se ##psis compared with post ##oper ##ative patients and healthy volunteers . While higher extra ##cellular RNA levels correlated with a faster c ##lot ##ting time and more stable c ##lot ##s , Cell - free DNA correlated with a shorter c ##lot ##ting time but also less fi ##bri ##no ##lysis . In addition , [SEP]',
  'existing_cui': 'C0521119',
  'existing_cui_name': 'Extracellular',
  'existing_cui_desc': '[CLS] Extra ##cellular [unused3] ( Ana ##tom ##ical Structure , qualifier value : extra ##cellular ; extra ##cellular region ) [SEP]',
  'our_bi_cui': 'C0230885',
  'our_bi_cui_name': 'Extracellular material',
  'our_bi_cui_desc': '[CLS] Extra ##cellular material [unused3] ( Body Sub ##stance , substance ) [SEP]',
  'our_cross_cui': 'C0035668',
  'our_cross_cui_name': 'RNA',
  'our_cross_cui_desc': '[CLS] RNA [unused3] ( Chemical , RNA , R ##ib ##on ##uc ##le ##ic A ##cid , substance : rib ##on ##uc ##le ##ic acid ; R ##ib ##on ##uc ##le ##ic acids ; RNA , Non - Pol ##ya ##den ##yla ##ted ; rib ##on ##uc ##lein ##ic ##um acid ##um / rib ##on ; RNA - R ##ib ##on ##uc ##le ##ic acid ; RNA [ Chemical / In ##g ##red ##ient ] ; RNA , Non Pol ##ya ##den ##yla ##ted ; Non - Pol ##ya ##den ##yla ##ted RNA ; r ##na ; Non Pol ##ya ##den ##yla ##ted RNA ; A ##cid , R ##ib ##on ##uc ##le ##ic ; R ##ib ##on ##uc ##le ##ic acid ; Gene Products , RNA [SEP]'},
 {'mention_id': '28203549.81',
  'test_set_idx': 28273,
  'type': 'T170',
  'name': 'ST65',
  'context': '[CLS] the most prevalent ST among 69 h ##yper ##mu ##co ##vis ##co ##us K ##le ##bs ##iel ##la pneumonia ##e is ##olate ##s with K ##1 , K ##2 , K ##5 , K ##20 , and K ##5 ##7 ( 27 . 5 % , 19 / 69 ) and was only found among K ##1 is ##olate ##s . [unused1] ST ##65 [unused2] was the second most prevalent ST ( 26 . 1 % , 18 / 69 ) and was also only found among K ##2 is ##olate ##s . ST ##23 - K ##1 h ##yper ##mu ##co ##vis ##co ##us K ##le ##bs ##iel ##la pneumonia ##e is ##olate ##s ( 84 . 2 % , 16 / 19 ) were [SEP]',
  'existing_cui': 'C0812444',
  'existing_cui_name': 'CD82 gene',
  'existing_cui_desc': '[CLS] CD ##8 ##2 gene [unused3] ( Ana ##tom ##ical Structure : K ##AN ##GA ##I 1 ; I ##A ##4 ; R ##2 le ##uk ##oc ##yte anti ##gen ; suppression of tumor ##ige ##nic ##ity 6 ; L ##E ##U ##KO ##C ##Y ##TE S ##UR ##FA ##CE AN ##TI ##GE ##N R ##2 ; CD ##8 ##2 Mo ##le ##cule Gene ; CD ##8 ##2 AN ##TI ##GE ##N ; R ##2 ; CD ##8 ##2 Gene ; R ##2 L ##E ##U ##KO ##C ##Y ##TE AN ##TI ##GE ##N ; PR ##OS ##TA ##TE CA ##NC ##ER AN ##TI ##ME ##TA ##ST ##AS ##IS GE ##NE K ##A ##I ##1 ; S ##UP ##PR ##ES ##SO ##R OF T ##UM ##OR ##IG [SEP]',
  'our_bi_cui': 'C0887835',
  'our_bi_cui_name': 'Ribotype',
  'our_bi_cui_desc': '[CLS] R ##ib ##otype [unused3] ( Finding : R ##ib ##otype ##s ) [SEP]',
  'our_cross_cui': 'C0449943',
  'our_cross_cui_name': 'Serotype',
  'our_cross_cui_desc': '[CLS] Ser ##otype [unused3] ( Intel ##lect ##ual Product , attribute , o ##bs ##er ##vable entity , qualifier value : se ##rov ##ar ; se ##rov ##ars ; Ser ##ova ##rs ; se ##rot ##ype ; se ##rot ##ype ##s ; Ser ##otype ##s ; Ser ##ova ##r ) [SEP]'},
 {'mention_id': '27545500.19',
  'test_set_idx': 9995,
  'type': 'T103',
  'name': '16-aldehyde tanshinone I',
  'context': '[CLS] settings . Using anti ##biotic enhance ##rs to rescue existing classes of anti ##biotics is an attractive strategy . In this study , 16 - al ##de ##hy ##de tan ##shin ##one I was synthesized and b ##act ##eri ##ost ##atic activity was explored . In addition , s ##yne ##rg ##istic or add ##itive activity between [unused1] 16 - al ##de ##hy ##de tan ##shin ##one I [unused2] and amino ##gly ##cos ##ide anti ##biotics or β - la ##ct ##am anti ##biotics in v ##it ##ro was identified . Moreover , 16 - al ##de ##hy ##de tan ##shin ##one I was documented to au ##gment clearance of s ##tre ##pt ##omy ##cin and am ##pic ##ill ##in against S . au ##reus in [SEP]',
  'existing_cui': 'C0075810',
  'existing_cui_name': 'tanshinone',
  'existing_cui_desc': '[CLS] tan ##shin ##one [unused3] ( Chemical : 1 , 6 - dim ##eth ##yl ##phe ##nant ##hr ##o ( 1 , 2 - b ) fur ##an - 10 , 11 - di ##one ; tan ##shin ##one I ) [SEP]',
  'our_bi_cui': 'C0075810',
  'our_bi_cui_name': 'tanshinone',
  'our_bi_cui_desc': '[CLS] tan ##shin ##one [unused3] ( Chemical : 1 , 6 - dim ##eth ##yl ##phe ##nant ##hr ##o ( 1 , 2 - b ) fur ##an - 10 , 11 - di ##one ; tan ##shin ##one I ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28197133.25',
  'test_set_idx': 28041,
  'type': 'T103',
  'name': 'GH94',
  'context': '[CLS] ##ume ##n micro ##bio ##ta of a la ##ct ##ating dairy cow and the micro ##or ##gan ##isms forming the fiber - de ##grading community . Fun ##ctional analysis identified 12 , 237 car ##bo ##hy ##dra ##te - active enzymes , accounting for 1 % of the trans ##cripts . The CA ##Z ##yme profile was dominated by families [unused1] G ##H ##9 ##4 [unused2] ( cello ##bio ##se - p ##hos ##ph ##ory ##lase ) , G ##H1 ##3 ( am ##yla ##se ) , G ##H ##43 and G ##H1 ##0 ( hem ##ice ##ll ##ula ##ses ) , G ##H ##9 and G ##H ##48 ( cell ##ula ##ses ) , P ##L ##11 ( p ##ec ##tina ##se ) as well [SEP]',
  'existing_cui': 'C0084342',
  'existing_cui_name': 'RHOC Protein',
  'existing_cui_desc': '[CLS] R ##H ##OC Pro ##tein [unused3] ( Chemical : Trans ##form ##ing Pro ##tein R ##H ##OC ; A ##p ##ly ##sia RA ##S - Re ##lated Ho ##mo ##log 9 ; RA ##S Ho ##mo ##log Gene Family , Member C ; AR ##HC ; r ##as ho ##mo ##log gene family , member C protein , human ; R ##H ##OH ##9 ; AR ##H ##9 protein , human ; R ##H ##OC protein , human ) [SEP]',
  'our_bi_cui': 'C0014442',
  'our_bi_cui_name': 'Enzymes',
  'our_bi_cui_desc': '[CLS] En ##zy ##mes [unused3] ( Chemical , medication , substance : enzymes ; enzyme ; En ##zy ##mes [ Chemical / In ##g ##red ##ient ] ; B ##io ##cat ##aly ##sts ; En ##zy ##me ) [SEP]',
  'our_cross_cui': 'C0055033',
  'our_cross_cui_name': 'cellobiose phosphorylase',
  'our_cross_cui_desc': '[CLS] cello ##bio ##se p ##hos ##ph ##ory ##lase [unused3] ( Chemical , substance : Cell ##ob ##ios ##e p ##hos ##ph ##ory ##lase ) [SEP]'},
 {'mention_id': '27941943.9',
  'test_set_idx': 22594,
  'type': 'T204',
  'name': 'vegetable crops',
  'context': '[CLS] Trans ##cript ##omi ##c insights into the all ##elo ##pathic effects of the garlic all ##elo ##chemical dial ##ly ##l di ##sul ##fi ##de on tomato roots G ##ar ##lic is an all ##elo ##pathic crop that can alleviate the obstacles to continuous crop ##ping of [unused1] vegetable crops [unused2] . Di ##ally ##l di ##sul ##fi ##de , one of the most important all ##elo ##chemical ##s in garlic , promotes tomato root growth . Therefore , the global trans ##cript ##ome profiles of Di ##ally ##l di ##sul ##fi ##de - treated tomato roots over time were investigated to reveal the potential growth - promoting mechanisms . We detected 1828 , 129 ##6 and 119 ##0 differential ##ly expressed genes in the 4 , [SEP]',
  'existing_cui': 'C0042440',
  'existing_cui_name': 'Vegetables',
  'existing_cui_desc': '[CLS] V ##ege ##table ##s [unused3] ( Food , substance : vegetable ; vegetables ; V ##ege ##table ; diet ##ary vegetable ) [SEP]',
  'our_bi_cui': 'C0042440',
  'our_bi_cui_name': 'Vegetables',
  'our_bi_cui_desc': '[CLS] V ##ege ##table ##s [unused3] ( Food , substance : vegetable ; vegetables ; V ##ege ##table ; diet ##ary vegetable ) [SEP]',
  'our_cross_cui': 'C0242775',
  'our_cross_cui_name': 'Agricultural Crops',
  'our_cross_cui_desc': '[CLS] Agricultural C ##rops [unused3] ( E ##uka ##ryo ##te : C ##rops , Agricultural ; C ##rops ; Agricultural C ##rop ) [SEP]'},
 {'mention_id': '28125734.62',
  'test_set_idx': 26216,
  'type': 'T103',
  'name': 'miR-21-5p',
  'context': '[CLS] ##gnant and non - ma ##li ##gnant cell lines . Further mi ##RNA inhibitor experiments and l ##uc ##ifer ##ase ass ##ays in Me ##ro - 14 cells valid ##ated mi ##R - 21 - 5 ##p as a true regulator of me ##so ##the ##lin . Moreover , in v ##it ##ro experiments showed that treatment with [unused1] mi ##R - 21 - 5 ##p [unused2] mimic reduced proliferation of ma ##li ##gnant p ##le ##ural me ##so ##the ##lio ##ma cell lines . Alto ##get ##her , this work shows that the mi ##R - CA ##TC ##H technique , coupled with next generation se ##quencing and in v ##it ##ro valid ##ation , represents a reliable method to identify native mi ##RNA : [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1999986',
  'our_bi_cui_name': 'MIRN21 microRNA , human',
  'our_bi_cui_desc': '[CLS] MI ##R ##N ##21 micro ##RNA , human [unused3] ( Chemical : micro ##RNA - 21 , human ; mi ##RNA - 21 , human ; h ##sa - mi ##r - 21 micro ##RNA ; mi ##R - 21 , human ; mi ##R ##21 , human ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '28249376.71',
  'test_set_idx': 29941,
  'type': 'T033',
  'name': 'total tract digestibility',
  'context': '[CLS] hem ##og ##lo ##bin , hem ##ato ##c ##rit , glucose , beta - h ##ydro ##xy ##but ##yric acid , ch ##ole ##ster ##ol , total protein , album ##in , blood u ##rea nitrogen and as ##par ##tate amino ##tra ##ns ##fer ##ase and al ##ani ##ne amino ##tra ##ns ##fer ##ase concentration . A ##ppa ##rent [unused1] total tract dig ##est ##ibility [unused2] of dry matter , crude pro ##ten , organic matter , and neutral de ##ter ##gent fiber were not influenced by mixture of essential oils supplement ##ation . The results of the present study suggested that supplement ##ation of mixture of essential oils may have limited effects on apparent nut ##rient dig ##est ##ibility , r ##umi ##nal f ##er [SEP]',
  'existing_cui': 'C0017178',
  'existing_cui_name': 'Gastrointestinal Diseases',
  'existing_cui_desc': '[CLS] Gas ##tro ##int ##est ##inal Disease ##s [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : gas ##tro ##int ##est ##inal disease ; G ##I disorders ; disorder ; gas ##tro ##int ##est ##inal ; G ##IT disorders ; Di ##sor ##der gas ##tro ##int ##est ##inal ; G ##IT diseases ; gas ##tro ##int ##est ##inal disorders ; gas ##tro ##int ##est ##inal disorder diagnosis ; disease ( or disorder ) ; gas ##tro ##int ##est ##inal tract ; gas ##tro ##int ##est ##inal ; disorder ; Gas ##tro ##int ##est ##inal Di ##sor ##der ; disease gas ##tro ##int ##est ##inal tract ; gas ##tro ##int ##est ##inal disorder ; Gas ##tro ##int ##est ##inal Di ##sor ##ders ; Gas [SEP]',
  'our_bi_cui': 'C0012238',
  'our_bi_cui_name': 'Digestion',
  'our_bi_cui_desc': '[CLS] Digest ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : dig ##estive system functions ; dig ##est ##ion ; Digest ##ive system functions ; dig ##est ##ions ; Digest ##ive system function ; Fun ##ctions of the dig ##estive system ; Digest ##ive ; Digest ##ive tract function ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28197156.17',
  'test_set_idx': 28087,
  'type': 'T204',
  'name': 'cultivar',
  'context': '[CLS] ##im ##ilation rate ( A ) between rice cult ##iva ##rs using quantitative genetics is one promising means to identify genes contributing to higher photos ##ynth ##esis . In this study , we determined precise location of Carbon As ##si ##mi ##lation Rate 8 by crossing a high - yielding in ##dic ##a cult ##iva ##r with a Japanese commercial [unused1] cult ##iva ##r [unused2] . Fine mapping suggested that Carbon As ##si ##mi ##lation Rate 8 en ##codes a put ##ative He ##me Act ##iva ##tor Pro ##tein 3 ( O ##s ##HA ##P ##3 ) subunit of a CC ##AA ##T - box - binding transcription factor called O ##s ##HA ##P ##3 ##H . Se ##quencing analysis revealed that the in ##dic ##a [SEP]',
  'existing_cui': 'C3385047',
  'existing_cui_name': 'Brassia hybrid cultivar',
  'existing_cui_desc': '[CLS] Brass ##ia hybrid cult ##iva ##r [unused3] ( E ##uka ##ryo ##te : Ada hybrid cult ##iva ##r ) [SEP]',
  'our_bi_cui': 'C0242775',
  'our_bi_cui_name': 'Agricultural Crops',
  'our_bi_cui_desc': '[CLS] Agricultural C ##rops [unused3] ( E ##uka ##ryo ##te : C ##rops , Agricultural ; C ##rops ; Agricultural C ##rop ) [SEP]',
  'our_cross_cui': 'C0032098',
  'our_cross_cui_name': 'Plants',
  'our_cross_cui_desc': '[CLS] Plants [unused3] ( E ##uka ##ryo ##te , organism : Ch ##lor ##op ##hy ##ta / Em ##b ##ryo ##phy ##ta group ; green plants ; Ch ##lor ##ob ##ion ##ta ; Ch ##lor ##ob ##ion ##ta B ##rem ##er , 1985 ; V ##iri ##di ##p ##lant ##ae ; Plants , General ; Kingdom Plant ##ae ; V ##iri ##di ##p ##lant ##ae C ##ava ##lier - Smith , 1981 ; plants ; Kingdom V ##iri ##di ##p ##lant ##ae ; plant ; ch ##lor ##op ##hy ##te / em ##b ##ryo ##phy ##te group ; Plant ) [SEP]'},
 {'mention_id': '28203067.13',
  'test_set_idx': 28170,
  'type': 'T033',
  'name': 'patient - related results',
  'context': '[CLS] stress ( score of ≤ 15 on the scale ) and higher stress ( score of ≥ 16 on the scale ) ex ##ert ##ed on family care ##rs by the process of providing care . The set of ex ##p ##lana ##tory variables comprised a wide range of socio ##de ##mo ##graphic and care - related attributes , including [unused1] patient - related results [unused2] from comprehensive g ##eria ##tric assessments and disease profiles . The best combination of ex ##p ##lana ##tory variables that provided the highest predict ##ive p ##ow ##e r for distress among family care ##rs in the multiple log ##istic re ##gression model was determined according to statistical information criteria . The statistical robust ##ness of the observed relationships and [SEP]',
  'existing_cui': 'C0420738',
  'existing_cui_name': 'Patient informed - test result',
  'existing_cui_desc': '[CLS] Pat ##ient informed - test result [unused3] ( Finding , context - dependent category , non - specific , situation , treatment : patient results tests ; Test result to patient ; patient results test ; patient provided with test / procedure results ; test result to patient ; Pat ##ient informed of result ) [SEP]',
  'our_bi_cui': 'C2987124',
  'our_bi_cui_name': 'Patient Reported Outcome',
  'our_bi_cui_desc': '[CLS] Pat ##ient Report ##ed Out ##come [unused3] ( Intel ##lect ##ual Product : Out ##come ##s , Pat ##ient Report ##ed ; Report ##ed Out ##come , Pat ##ient ; Pat ##ient Report ##ed Out ##come ##s ; Out ##come , Pat ##ient Report ##ed ; Report ##ed Out ##come ##s , Pat ##ient ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28377051.61',
  'test_set_idx': 33677,
  'type': 'T103',
  'name': 'Tyr256',
  'context': '[CLS] ##P ##1 and N - WA ##SP were recruited to fi ##lam ##ent ##ous act ##in , and shared some common compartment ##s localized at the leading edge of lame ##lli ##po ##dia , which was impaired after the de ##ple ##tion of I ##Q ##GA ##P ##1 by using the small interference RNA . Moreover , the knock ##down of I ##Q ##GA ##P ##1 also significantly decreased N - WA ##SP p ##hos ##ph ##ory ##lation at [unused1] Ty ##r ##25 ##6 [unused2] induced by C . pneumonia ##e infection . We conclude that C . pneumonia ##e infection promotes vascular end ##oth ##eli ##al cell migration and an ##gio ##genesis presumably through the I ##Q ##GA ##P ##1 - related signaling pathway . [SEP]',
  'existing_cui': 'C3539605',
  'existing_cui_name': 'Complement Receptor Type 2',
  'existing_cui_desc': '[CLS] Co ##mple ##ment Re ##ceptor Type 2 [unused3] ( Chemical : E ##B ##V Re ##ceptor ; CD ##21 ; CD ##21 Anti ##gen ; Co ##mple ##ment C ##3 ##d Re ##ceptor ; E ##ps ##tein - Barr V ##ir ##us Re ##ceptor ) [SEP]',
  'our_bi_cui': 'C0040005',
  'our_bi_cui_name': 'Threonine',
  'our_bi_cui_desc': '[CLS] T ##hr ##eon ##ine [unused3] ( Chemical , T ##hr , medication , substance : L - alpha - amino - beta - h ##ydro ##xy ##but ##yric acid ; L - th ##re ##oni ##ne ; th ##re ##oni ##ne ; th ##r ; L - T ##hr ##eon ##ine ; ( 2 ##S , 3 ##R ) - 2 - Am ##ino - 3 - h ##ydro ##xy ##but ##ano ##ic acid ; T ##hr ##eon ##ine [ Chemical / In ##g ##red ##ient ] ; ( 2 ##S ) - th ##re ##oni ##ne ; T ##hr ; L T ##hr ##eon ##ine ; L - 2 - Am ##ino - 3 - h ##ydro ##xy ##but ##yric acid ; T ##H ##RE [SEP]',
  'our_cross_cui': 'C0041485',
  'our_cross_cui_name': 'Tyrosine',
  'our_cross_cui_desc': '[CLS] Ty ##ros ##ine [unused3] ( Chemical , L , Ty ##r , medication , product , substance : L - Ph ##en ##yla ##lani ##ne , 4 - h ##ydro ##xy - ; L - Ty ##ros ##ine ; Ty ##r ; ( S ) - alpha - amino - 4 - H ##ydro ##xy ##ben ##zen ##ep ##rop ##ano ##ic acid ; ( S ) - 2 - Am ##ino - 3 - ( p - h ##ydro ##xy ##phe ##ny ##l ) prop ##ion ##ic acid ; ( S ) - 3 - ( p - H ##ydro ##xy ##phe ##ny ##l ) al ##ani ##ne ; ( S ) - 2 - Am ##ino - 3 - ( 4 - h ##ydro ##xy [SEP]'},
 {'mention_id': '27977439.40',
  'test_set_idx': 22835,
  'type': 'T033',
  'name': 'postoperative',
  'context': '[CLS] No effect was found between groups for the primary outcome ( P = 0 . 680 , intention to treat ) ; the median values were 42 . 4 and 37 . 7 h , respectively for the control and intervention groups ( P = 0 . 87 ##3 ) . Similar non ##sign ##ificant trends were noted for other [unused1] post ##oper ##ative [unused2] delays . Sign ##ificant effects favour ##ing the intervention were noted for dynamic pain , patient satisfaction , occurrence of nausea and vomit ##ing , occurrence of del ##iri ##um or mental confusion and occurrence of pulmonary complications . In 12 patients , although no symptoms actually occurred , the total r ##op ##iva ##ca ##ine plasma level exceeded the lowest [SEP]',
  'existing_cui': 'C0520905',
  'existing_cui_name': 'Vomiting , Postoperative',
  'existing_cui_desc': '[CLS] V ##omi ##ting , Post ##oper ##ative [unused3] ( Finding , disorder : Post - op vomit ##ing ; Post ##oper ##ative V ##omi ##ting ; Em ##ese ##s , Post ##oper ##ative ; P ##OS ##TO ##P V ##OM ##IT ##ING ; V ##omi ##ting post ##oper ##ative ; E ##ME ##SI ##S P ##OS ##TO ##P ; Post ##oper ##ative Em ##ese ##s ; Em ##esis , Post ##oper ##ative ; Post pro ##cedural vomit ##ing ; Post ##oper ##ative vomit ##ing ; V ##OM ##IT ##ING P ##OS ##TO ##P ; Post ##oper ##ative Em ##esis ; P ##OS ##TO ##P E ##ME ##SI ##S ) [SEP]',
  'our_bi_cui': 'C0032786',
  'our_bi_cui_name': 'Postoperative Care',
  'our_bi_cui_desc': '[CLS] Post ##oper ##ative Care [unused3] ( Health Care Act ##ivity , procedure , regime / therapy : Post ##oper ##ative care ( procedure ; CA ##RE P ##OS ##TO ##P ; care post ##oper ##ative ; P ##OS ##TO ##P CA ##RE ; Post ##oper ##ative care ; Care , Post ##oper ##ative ; post ##oper ##ative care ) [SEP]',
  'our_cross_cui': 'C0241311',
  'our_cross_cui_name': 'post operative',
  'our_cross_cui_desc': '[CLS] post operative [unused3] ( Finding , finding , finding : post surgery status ; Recovery from surgery ; After Surgery ; status after surgery ; Post surgery ) [SEP]'},
 {'mention_id': '27460657.59',
  'test_set_idx': 6673,
  'type': 'T103',
  'name': 'ams',
  'context': '[CLS] ass ##ay showed that MS ##18 ##8 activated the expression of C ##YP ##70 ##3 ##A ##2 . Ye ##ast two - hybrid and electro ##ph ##ore ##tic mobility shift ass ##ays further demonstrated that MS ##18 ##8 complex ##ed with AB ##OR ##TE ##D MI ##CR ##OS ##PO ##RE ##S . The expression of C ##YP ##70 ##3 ##A ##2 could be partially restored by the elevated levels of MS ##18 ##8 in the [unused1] am ##s [unused2] mutant . Therefore , our data reveal that MS ##18 ##8 coordinates with AB ##OR ##TE ##D MI ##CR ##OS ##PO ##RE ##S to activate C ##YP ##70 ##3 ##A ##2 in s ##por ##op ##oll ##eni ##n bio ##sy ##nt ##hesis of plant tape ##tum . [SEP]',
  'existing_cui': 'C0002699',
  'existing_cui_name': 'Amsacrine',
  'existing_cui_desc': "[CLS] Am ##sa ##c ##rine [unused3] ( Chemical , product , substance : Am ##sa ##c ##rine [ Chemical / In ##g ##red ##ient ] ; AM ##SA M ; m - am ##sa ; m - AM ##SA ; AM ##SA ##CR ##IN ##E ; meta - AM ##SA ; 4 ' - ( 9 - A ##c ##rid ##in ##yla ##min ##o ) met ##hane ##sul ##fo ##n - meta - an ##isi ##di ##de ; A ##c ##rid ##in ##yl an ##isi ##di ##de ; 4 ' - ( 9 - A ##c ##rid ##in ##yla ##min ##o ) met ##hane ##sul ##phon - m - an ##isi ##di ##de ; m ##AM ##SA ; Am ##sa ##c ##rine - chemical ; a ##c [SEP]",
  'our_bi_cui': 'C0017337',
  'our_bi_cui_name': 'Genes',
  'our_bi_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]',
  'our_cross_cui': 'C1434242',
  'our_cross_cui_name': 'AMS protein , Arabidopsis',
  'our_cross_cui_desc': '[CLS] AM ##S protein , Arab ##ido ##psis [unused3] ( Chemical : AB ##OR ##TE ##D MI ##CR ##OS ##PO ##RE ##S protein , Arab ##ido ##psis ) [SEP]'},
 {'mention_id': '28294559.21',
  'test_set_idx': 31477,
  'type': 'T058',
  'name': 'HPx',
  'context': '[CLS] type le ##ct ##in receptor - 2 deleted from plate ##lets of ch ##ime ##ric mice ( fl ##KO ) underwent HP ##x . The liver / body weight ratio was significantly lower in the fl ##KO than in the wild - type . C - type le ##ct ##in receptor - 2 plays an essential role in liver regeneration after [unused1] HP ##x [unused2] . Background and aim The aim of the present study was to investigate the role of C - type le ##ct ##in receptor ( C - type le ##ct ##in receptor ) - 2 in liver regeneration following partial liver re ##section in mice . Materials and methods I ##rra ##dia ##ted ch ##ime ##ric mice transplant ##ed with f ##etal [SEP]',
  'existing_cui': 'C0523688',
  'existing_cui_name': 'Hemopexin measurement',
  'existing_cui_desc': '[CLS] He ##mo ##pe ##xin measurement [unused3] ( Health Care Act ##ivity , procedure : He ##mo ##pe ##xin ; He ##mo ##pe ##xin Me ##as ##ure ##ment ; Beta - 1 ##B G ##ly ##co ##p ##rote ##in ; Ha ##em ##ope ##xin measurement ; Beta - 1 ##B G ##ly ##co ##p ##rote ##in Me ##as ##ure ##ment ; Beta - 1 ##B - G ##ly ##co ##p ##rote ##in Me ##as ##ure ##ment ; HP ##X ; B ##1 ##B ##G ##LP ) [SEP]',
  'our_bi_cui': 'C0008562',
  'our_bi_cui_name': 'High pressure liquid chromatography procedure',
  'our_bi_cui_desc': '[CLS] High pressure liquid ch ##roma ##tography procedure [unused3] ( Health Care Act ##ivity , HP ##LC , procedure , qualifier value : HP ##LC measurement ; Ch ##roma ##tography , High Speed Li ##quid ; Ch ##roma ##tography , High Press ##ure Li ##quid ; Ch ##roma ##tography , Li ##quid , High Press ##ure ; High Performance Li ##quid Ch ##roma ##tography ; HP ##LC - High pressure liquid ch ##roma ##tography ; High performance liquid ch ##roma ##tography ; H ##SL ##C ; High performance liquid ch ##roma ##tography measurement ; High pressure liquid ch ##roma ##tography ; high pressure liquid ch ##roma ##tography ; CH ##RO ##MA ##TO ##GR H ##IG ##H P ##ER ##F ##OR ##MA ##NC ##E L ##I ##Q ##UI [SEP]',
  'our_cross_cui': 'C0193398',
  'our_cross_cui_name': 'Partial hepatectomy',
  'our_cross_cui_desc': '[CLS] Part ##ial he ##pate ##ct ##omy [unused3] ( Health Care Act ##ivity , partial , procedure , treatment : He ##pate ##ct ##omy , re ##section of liver ; partial lobe ##ct ##omy ; PA ##RT ##IA ##L R ##EM ##O ##VA ##L OF L ##IVE ##R ; Sub ##to ##tal he ##pate ##ct ##omy ; partial he ##pate ##ct ##omy ; P ##H - Part ##ial he ##pate ##ct ##omy ; H ##EP ##AT ##EC ##TO ##M ##Y R ##ES ##C ##J PA ##RT ##IA ##L L ##O ##BE ##CT ##OM ##Y ; Part ##ial removal of liver ; Part ##ial He ##pate ##ct ##omy ; partial ex ##cision of liver ; He ##pate ##ct ##omy ; Part ##ial ex ##cision of liver ; sub [SEP]'},
 {'mention_id': '27820849.4',
  'test_set_idx': 18186,
  'type': 'T082',
  'name': 'High Density SNPs',
  'context': '[CLS] Gen ##ome - Wide Det ##ec ##tion of Select ##ive Sign ##ature ##s in Chicken through [unused1] High Den ##sity S ##NP ##s [unused2] Chicken is recognized as an excellent model for studies of genetic mechanism of p ##hen ##otypic and g ##eno ##mic evolution , with large effective population size and strong human - driven selection . In the present study , we performed Extended Ha ##p ##lot ##ype Ho ##mo ##zy ##gos ##ity tests to identify significant core regions employing 600 ##K S ##NP Chicken chip in an F ##2 population of 1 , 53 ##4 he ##ns , which was derived from re ##ci ##p ##ro ##cal crosses between White Le ##gh ##orn and Dong ##xia ##ng chicken . Results indicated that a [SEP]',
  'existing_cui': 'C1268672',
  'existing_cui_name': 'High density lesion',
  'existing_cui_desc': '[CLS] High density les ##ion [unused3] ( Finding , finding : Les ##ion with high radio ##graphic density ) [SEP]',
  'our_bi_cui': 'C0023821',
  'our_bi_cui_name': 'High Density Lipoproteins',
  'our_bi_cui_desc': '[CLS] High Den ##sity Li ##pop ##rote ##ins [unused3] ( Chemical , HD ##L , substance : alpha Li ##pop ##rote ##in ; Li ##pop ##rote ##ins , High - Den ##sity ; alpha - lip ##op ##rote ##in ; Li ##pop ##rote ##in . alpha ; alpha Li ##pop ##rote ##ins ; high density lip ##op ##rote ##in ; High density lip ##op ##rote ##ins ; High - Den ##sity Li ##pop ##rote ##ins ; high density lip ##op ##rote ##ins ; Li ##pop ##rote ##ins , HD ##L [ Chemical / In ##g ##red ##ient ] ; High den ##si ##ti ##y lip ##op ##rote ##in ; Li ##pop ##rote ##ins , Heavy ; h ##dl ##s ; alpha - Li ##pop ##rote ##ins ; [SEP]',
  'our_cross_cui': 'C0752046',
  'our_cross_cui_name': 'Single Nucleotide Polymorphism',
  'our_cross_cui_desc': '[CLS] Single N ##uc ##leo ##tide Pol ##ym ##or ##phism [unused3] ( Spa ##tial Concept , S ##NP : Pol ##ym ##or ##phism , Single N ##uc ##leo ##tide ; Single N ##uc ##leo ##tide Pol ##ym ##or ##phism ##s ; Pol ##ym ##or ##phism ##s , Single N ##uc ##leo ##tide ; S ##NP ##s ; N ##uc ##leo ##tide Pol ##ym ##or ##phism ##s , Single ; N ##uc ##leo ##tide Pol ##ym ##or ##phism , Single ; single n ##uc ##leo ##tide p ##oly ##mor ##phism ; single n ##uc ##leo ##tide p ##oly ##mor ##phism ##s ; Pol ##ym ##or ##phism , Single Base ) [SEP]'},
 {'mention_id': '28390176.3',
  'test_set_idx': 34093,
  'type': 'T103',
  'name': 'anti - cardiac hypertrophy drugs',
  'context': '[CLS] C ##uc ##ur ##bit ##ac ##in B Pro ##tec ##ts Against Press ##ure Over ##load In ##duced Card ##iac H ##yper ##tro ##phy Lac ##k of effective [unused1] anti - cardiac h ##yper ##tro ##phy drugs [unused2] creates a major cause for the increasing prevalence of heart failure . In the present study , we determined the anti - h ##yper ##tro ##phy and anti - fi ##bro ##sis potential of a natural plant t ##rite ##rp ##eno ##id , C ##uc ##ur ##bit ##ac ##in B both in v ##it ##ro and in v ##ivo . A ##ort ##ic band ##ing was performed to induce cardiac h ##yper ##tro ##phy . After 1 week of surgery , mice were receive cu ##cu ##rb ##ita ##cin [SEP]',
  'existing_cui': 'C1383860',
  'existing_cui_name': 'Cardiac Hypertrophy',
  'existing_cui_desc': '[CLS] Card ##iac H ##yper ##tro ##phy [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : h ##yper ##tro ##phy ; cardiac ; heart h ##yper ##tro ##phy ; Heart H ##yper ##tro ##phy ; Heart - - H ##yper ##tro ##phy ; cardiac ; h ##yper ##tro ##phy ; cardiac h ##yper ##tro ##phy ; h ##yper ##tro ##phic heart ; H ##yper ##tro ##phic ; heart ; Card ##iac h ##yper ##tro ##phy ; heart ; h ##yper ##tro ##phy ; h ##yper ##tro ##phy ; heart ) [SEP]',
  'our_bi_cui': 'C0003195',
  'our_bi_cui_name': 'Anti-Arrhythmia Agents',
  'our_bi_cui_desc': '[CLS] Anti - A ##rr ##hy ##th ##mia Agents [unused3] ( Chemical , medication , product , substance : anti - a ##rr ##hy ##th ##mic ##s ; Anti A ##rr ##hy ##th ##mia Drugs ; Drugs , Anti - A ##rr ##hy ##th ##mia ; anti ##ar ##r ##hy ##th ##mic agents ; AN ##TI ##AR ##R ##H ##Y ##TH ##MI ##A AG ##EN ##TS ; Drugs , Anti ##ar ##r ##hy ##th ##mia ; anti ##ar ##r ##hy ##th ##mic drug ; anti a ##rr ##hy ##th ##mic ; Anti ##ar ##r ##hy ##th ##mic drug ; Card ##iac De ##press ##ant ; Anti ##ar ##r ##hy ##th ##mic Drugs ; Anti ##ar ##r ##hy ##th ##mic agent ; Anti - a ##rr ##hy ##th [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27838742.67',
  'test_set_idx': 19034,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] a potential anti ##can ##cer agent that can induce auto ##pha ##gic cell death in human non - small cell lung cancer via the A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . [unused1] Del ##ica ##f ##lav ##one [unused2] induced auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . Del ##ica ##f ##lav ##one may represent a potential therapeutic agent for lung cancer . [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27301681.32',
  'test_set_idx': 1932,
  'type': 'T103',
  'name': '8-amino-3 ,6 - dioxaoctanoic acid',
  'context': '[CLS] ##ao ##ct ##ano ##ic acid units facing each other in a hair ##pin form , successfully showed the expected motif and a K ##D value of 16 ##n ##M was recorded . Pol ##yam ##ide 5 , containing a β - al ##ani ##ne - β - al ##ani ##ne unit instead of the [unused1] 8 - amino - 3 , 6 - di ##ox ##ao ##ct ##ano ##ic acid [unused2] of p ##oly ##ami ##de 2 , was synthesized for comparison . The a ##que ##ous so ##lub ##ilities and nuclear local ##ization of three of the p ##oly ##ami ##des were also examined . The results suggest the possibility of applying the 8 - amino - 3 , 6 - di ##ox ##ao ##ct [SEP]',
  'existing_cui': 'C0623748',
  'existing_cui_name': '2-amino-8-phosphonooctanoic acid',
  'existing_cui_desc': '[CLS] 2 - amino - 8 - p ##hos ##phon ##oo ##ct ##ano ##ic acid [unused3] ( Chemical : 2 - AP ##8 ) [SEP]',
  'our_bi_cui': 'C0758325',
  'our_bi_cui_name': '5-octadecenoic acid',
  'our_bi_cui_desc': '[CLS] 5 - o ##ct ##ade ##ce ##no ##ic acid [unused3] ( Chemical , substance : o ##ct ##ade ##ce ##no ##ic acid ; Oct ##ade ##ce ##no ##ic acid ) [SEP]',
  'our_cross_cui': 'C0029224',
  'our_cross_cui_name': 'Organic Chemicals',
  'our_cross_cui_desc': '[CLS] Organic Chemical ##s [unused3] ( Chemical , substance : organic chemicals ; compound organic ; Organic Chemical ##s [ Chemical / In ##g ##red ##ient ] ; organic compound ; Chemical ##s , Organic ; organic chemical ; Organic Chemical ; chemical organic ; organic compounds ; Organic compound ; Organic compounds ; Organic chemical ) [SEP]'},
 {'mention_id': '27506129.16',
  'test_set_idx': 8648,
  'type': 'T170',
  'name': 'SMOTE approaches',
  'context': "[CLS] straightforward due to data issues such as class im ##bal ##ance or correlated features . In this paper , we study the specific features of h ##yper ##emia and propose various approaches to address these problems in the context of an automatic framework for h ##yper ##emia g ##rading . Over ##sa ##mp ##ling , under ##sa ##mp ##ling and [unused1] SM ##OT ##E approaches [unused2] were applied in order to tackle the problem of class im ##bal ##ance . 25 features were com ##puted for each image and re ##gression methods were then used to transform them into a value on the g ##rading scale . The values and relationships among features and experts ' values were anal ##ys ##ed , and five feature selection [SEP]",
  'existing_cui': 'C0449445',
  'existing_cui_name': 'Approach',
  'existing_cui_desc': '[CLS] A ##pp ##roach [unused3] ( Spa ##tial Concept , attribute , qualifier value : A ##pp ##roach ##es ; approach ; approaches ; approaching ) [SEP]',
  'our_bi_cui': 'C0025663',
  'our_bi_cui_name': 'Methods',
  'our_bi_cui_desc': '[CLS] Method ##s [unused3] ( Intel ##lect ##ual Product , attribute : methods ; Method ; method ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27430240.76',
  'test_set_idx': 5402,
  'type': 'T103',
  'name': 'factors',
  'context': '[CLS] transcription factor 1 or CR ##E ##B , and were accelerated in the presence of C ##GS ##21 ##6 ##80 . Based on these results , it can be in ##ferred that ad ##eno ##sin ##e , the ad ##eno ##sin ##e A2 ##A receptor ago ##nist , E ##2 ##F transcription factor 1 and CR ##E ##B are the possible [unused1] factors [unused2] contributing to the high expression of CD ##39 and CD ##7 ##3 on the T ##re ##g cell surface during se ##psis . Aden ##os ##ine and its A2 ##A receptor ago ##nist served as the signal trans ##du ##cer factors of the CD ##39 / CD ##7 ##3 / ad ##eno ##sin ##e pathway , a ##cc ##ele ##rating ad ##eno [SEP]',
  'existing_cui': 'C0960756',
  'existing_cui_name': 'factor A',
  'existing_cui_desc': "[CLS] factor A [unused3] ( Chemical : Co ( beta ) - c ##yan ##o - 7 ' ' - ( 2 - met ##hyl ) ad ##eni ##ny ##l ##co ##ba ##mi ##de ) [SEP]",
  'our_bi_cui': 'C0021054',
  'our_bi_cui_name': 'Immunologic Factors',
  'our_bi_cui_desc': '[CLS] I ##mm ##uno ##log ##ic Factor ##s [unused3] ( Chemical , substance : Factor ##s , I ##mm ##uno ##log ##ic ; I ##mm ##une Factor ##s ; Factor ##s , I ##mm ##uno ##logical ; I ##mm ##uno ##log ##ic Factor ; Factor ##s , I ##mm ##une ; I ##mm ##uno ##logical Factor ##s ; im ##mu ##no ##log ##ic substance ; FA ##CT ##OR ##S I ##MM ##UN ##OL ; im ##mu ##nological substance ; I ##MM ##UN ##OL FA ##CT ##OR ##S ; I ##mm ##uno ##log ##ic substance ) [SEP]',
  'our_cross_cui': 'C0005515',
  'our_cross_cui_name': 'Biological Factors',
  'our_cross_cui_desc': '[CLS] Biological Factor ##s [unused3] ( Chemical , product , substance : Agents , Biological ; Biological Factor ; B ##iol ##og ##ic agent ; agents biological ; B ##iol ##og ##ic Factor ##s ; B ##iol ##og ##ic Factor ; biological substance ; Factor , B ##iol ##og ##ic ; Factor ##s , Biological ; bio ##pha ##rma ##ce ##utical ##s ; Biological product ; B ##IO ##L FA ##CT ##OR ; Biological Agents ; Biological substance ; Biological Factor ##s [ Chemical / In ##g ##red ##ient ] ; Biological agent ; B ##iol ##og ##ics ; Factor , Biological ; biological drug ; B ##IO ##L FA ##CT ##OR ##S ; FA ##CT ##OR ##S B ##IO ##L ; biological substances ; bio ##pha [SEP]'},
 {'mention_id': '28278150.21',
  'test_set_idx': 31035,
  'type': 'T062',
  'name': 'GTEx project',
  'context': '[CLS] distinct feature compared to the existing co ##loc ##ali ##zation analysis procedures in the literature . The proposed approach is computational ##ly efficient and requires only summary - level statistics . We evaluate and demonstrate the proposed computational approach through extensive simulation studies and analyses of blood lip ##id data and the whole blood e ##Q ##TL data from the [unused1] GT ##E ##x project [unused2] . In addition , a useful utility from our proposed method enables the com ##putation of expected co ##loc ##ali ##zation signals using simple characteristics of the association data . Using this utility , we further illustrate the importance of en ##rich ##ment analysis on the ability to discover co ##loc ##ali ##zed signals and the potential limitations of currently [SEP]',
  'existing_cui': 'C3831542',
  'existing_cui_name': 'Analysis Project',
  'existing_cui_desc': '[CLS] Analysis Project [unused3] ( Research Act ##ivity : Statistical Data Group ##ing ) [SEP]',
  'our_bi_cui': 'C0242481',
  'our_bi_cui_name': 'Research Activities',
  'our_bi_cui_desc': '[CLS] Research Activities [unused3] ( Research Act ##ivity : research ; Research Act ##ivity ; activity research ; Research ; Activities , Research ; Act ##ivity , Research ; research activities ) [SEP]',
  'our_cross_cui': 'C4289040',
  'our_cross_cui_name': 'Genotype-Tissue Expression Program',
  'our_cross_cui_desc': '[CLS] Gen ##otype - T ##iss ##ue Express ##ion Program [unused3] ( Research Act ##ivity : GT ##E ##x ; Gen ##otype - T ##iss ##ue Express ##ion ) [SEP]'},
 {'mention_id': '28471040.20',
  'test_set_idx': 37134,
  'type': 'T017',
  'name': 'rat glioma-2 cells',
  'context': '[CLS] 1 , 2 - di ##ole ##oy ##l - 3 - trim ##eth ##yla ##mm ##onium - prop ##ane hybrid na ##no ##par ##tic ##les has been prepared and evaluated for in - v ##it ##ro characterization . Cy ##to ##to ##xi ##city of F ##TA - loaded na ##no ##par ##tic ##les along with its efficacy on [unused1] rat g ##lio ##ma - 2 cells [unused2] was also evaluated both in v ##it ##ro ( in comparison with non - ma ##li ##gnant cell line , L ##9 ##29 ) and in v ##ivo . Sc ##ann ##ing electron micro ##sco ##py studies showed that all formulation ##s prepared had smooth surface and spherical in shape . far ##nes ##yl ##thi ##osa ##lic ##yl ##ic [SEP]',
  'existing_cui': 'C0017638',
  'existing_cui_name': 'Glioma',
  'existing_cui_desc': '[CLS] G ##lio ##ma [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , m ##or ##ph ##olo ##gic abnormal ##ity : G ##lial Cell Tu ##mor ##s ; g ##lio ##mas ; [ M ] G ##lio ##mas ; G ##lio ##mas ; N ##eur ##og ##lial Neo ##p ##las ##m ; Tu ##mor of the N ##eur ##og ##lia ; g ##lio ##ma ; un ##sp ##ec ##ified site ; Tu ##mor , G ##lial Cell ; Tu ##mor ##s , G ##lial Cell ; G ##lial Neo ##p ##las ##m ; Neo ##p ##las ##m of N ##eur ##og ##lia ; N ##eur ##og ##lial Tu ##mor ; G ##lial Cell Tu ##mor ; G ##lial Tu ##mor ; G ##lio [SEP]',
  'our_bi_cui': 'C1514733',
  'our_bi_cui_name': 'Rat Cell Line',
  'our_bi_cui_desc': '[CLS] Rat Cell Line [unused3] ( Ana ##tom ##ical Structure ) [SEP]',
  'our_cross_cui': 'C0007634',
  'our_cross_cui_name': 'Cells',
  'our_cross_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]'},
 {'mention_id': '27320863.24',
  'test_set_idx': 2325,
  'type': 'T168',
  'name': 'methionine and choline - deficient diet',
  'context': '[CLS] ##ine supplement ##ation in drinking water ( 1 % ) . Ph ##os ##ph ##ory ##lation of AM ##P - activated protein kinase , ace ##ty ##l - Co ##A car ##box ##yla ##se , s ##tero ##l regulatory element - binding protein 1 ##c , and liver kinase B ##1 was in ##hibit ##ed by [unused1] met ##hi ##oni ##ne and ch ##olin ##e - def ##icient diet [unused2] intake , and these changes were all in ##hibit ##ed by beta ##ine feeding . Meanwhile , beta ##ine supplement ##ation reversed the reduction of met ##hi ##oni ##ne and S - ad ##eno ##sy ##lm ##eth ##ion ##ine , and the elevation of ho ##mo ##cy ##stein ##e levels in the liver , which could [SEP]',
  'existing_cui': 'C2928853',
  'existing_cui_name': 'Choline / Methionine',
  'existing_cui_desc': '[CLS] Cho ##line / Met ##hi ##oni ##ne [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0452278',
  'our_bi_cui_name': 'Low methionine diet',
  'our_bi_cui_desc': '[CLS] Low met ##hi ##oni ##ne diet [unused3] ( Health Care Act ##ivity , finding , regime / therapy : Dec ##rease in met ##hi ##oni ##ne content of the diet compared to the assessed base ##line intake of met ##hi ##oni ##ne for the individual . ( Source : Academy of N ##utrition and Diet ##etics N ##utrition In ##formatics Committee 2013 ) ; Dec ##reased met ##hi ##oni ##ne diet ) [SEP]',
  'our_cross_cui': 'C0012155',
  'our_cross_cui_name': 'Diet',
  'our_cross_cui_desc': '[CLS] Diet [unused3] ( Food , finding : Diet ##s ; regular diet ? ; Diet ##ary ; diet ; Diet ##ary finding ; diet ##s ) [SEP]'},
 {'mention_id': '28377738.28',
  'test_set_idx': 33713,
  'type': 'T204',
  'name': 'creature',
  'context': '[CLS] dimensional approach will help to c ##lar ##ify and justify claims about the hi ##era ##rch ##ical nature of consciousness . The approach also proves ex ##p ##lana ##tor ##ily advantage ##ous , as it enables us not only to draw attention to certain new and important differences in respect of subjective measures of awareness and to justify how a given [unused1] creature [unused2] may be ranked higher in one dimension of consciousness and lower in terms of another , but also allows for innovative explanations of a variety of well - known phenomena ( amongst these , the interpretations of blind ##sight and locked - in syndrome will be briefly outlined here ) . Moreover , a 4 ##D framework makes possible many predictions and [SEP]',
  'existing_cui': 'C0056695',
  'existing_cui_name': 'Cyclic AMP-Responsive DNA-Binding Protein',
  'existing_cui_desc': '[CLS] Cy ##c ##lic AM ##P - Re ##sp ##ons ##ive DNA - Bin ##ding Pro ##tein [unused3] ( Chemical , CR ##E ##B : c ##yclic AM ##P response element binding protein ; c ##AM ##P response element binding protein ; Cy ##c ##lic AM ##P Re ##sp ##ons ##ive El ##ement Bin ##ding Pro ##tein ; c ##yclic AM ##P - re ##sp ##ons ##ive DNA - binding protein ; DNA Bin ##ding Pro ##tein , Cy ##c ##lic AM ##P Re ##sp ##ons ##ive ; c ##AM ##P Re ##sp ##ons ##ive El ##ement Bin ##ding Pro ##tein ; Cy ##c ##lic AM ##P Re ##sp ##ons ##ive DNA Bin ##ding Pro ##tein ; DNA - Bin ##ding Pro ##tein , Cy ##c ##lic [SEP]',
  'our_bi_cui': 'C0027361',
  'our_bi_cui_name': 'Persons',
  'our_bi_cui_desc': '[CLS] Persons [unused3] ( Population Group , person : Individual ; Person ; persons ; peoples ; Person [ du ##p ] ; People ; person ; people ) [SEP]',
  'our_cross_cui': 'C0003062',
  'our_cross_cui_name': 'Animals',
  'our_cross_cui_desc': '[CLS] Animals [unused3] ( E ##uka ##ryo ##te , organism : Animal ; animal ; multi ##cellular animals ; Animal ##ia ; Animal classified by human context ; Animal classified by role ; animal ##ia ; animal kingdom ; AN ##IM ##AL ; animals ; En ##ti ##ty ##C ##lass ##A ##ni ##mal ; Kingdom Animal ##ia ) [SEP]'},
 {'mention_id': '27460657.8',
  'test_set_idx': 6622,
  'type': 'T017',
  'name': 'sexine layer',
  'context': '[CLS] The transcription factors MS ##18 ##8 and AB ##OR ##TE ##D MI ##CR ##OS ##PO ##RE ##S form a complex to activate the expression of C ##YP ##70 ##3 ##A ##2 for s ##por ##op ##oll ##eni ##n bio ##sy ##nt ##hesis in Arab ##ido ##psis th ##alia ##na The [unused1] sex ##ine layer [unused2] of pollen grain is mainly composed of s ##por ##op ##oll ##eni ##ns . The s ##por ##op ##hy ##tic secret ##ory tape ##tum is required for the bio ##sy ##nt ##hesis of s ##por ##op ##oll ##eni ##n . Although several enzymes involved in s ##por ##op ##oll ##eni ##n bio ##sy ##nt ##hesis have been reported , the regulatory mechanism of these enzymes in tape ##tal layer remains el [SEP]',
  'existing_cui': 'C0920995',
  'existing_cui_name': 'Subintima',
  'existing_cui_desc': '[CLS] Sub ##int ##ima [unused3] ( Ana ##tom ##ical Structure , organ part : la ##mina ##s ; Lay ##er ; Lam ##ina ; la ##mina ) [SEP]',
  'our_bi_cui': 'C0017472',
  'our_bi_cui_name': 'Germ Layers',
  'our_bi_cui_desc': '[CLS] G ##er ##m Lay ##ers [unused3] ( Ana ##tom ##ical Structure : Lay ##er , G ##er ##m ; G ##er ##m Lay ##er ; g ##er ##m layer ; G ##er ##m layer ; Lay ##ers , G ##er ##m ) [SEP]',
  'our_cross_cui': 'C1820011',
  'our_cross_cui_name': 'sexine',
  'our_cross_cui_desc': '[CLS] sex ##ine [unused3] ( Ana ##tom ##ical Structure ) [SEP]'},
 {'mention_id': '27715500.19',
  'test_set_idx': 14884,
  'type': 'T092',
  'name': 'industrial beefpacking plant',
  'context': '[CLS] have received limited research attention . Prior research indicates under ##re ##port ##ing in injury rates in this industry as well significant variation in injury rates among facilities . To add detail to the rates and circumstances surrounding occupational injury among meat ##pack ##ing workers , we conducted a cross - section ##al study of workers employed at an [unused1] industrial beef ##pack ##ing plant [unused2] in Nebraska , United States ( n = 137 ) and interviewed workers about recent injury experiences . We assessed frequency , cause and nature of self - reported injury . We estimated annual incidence rates of self - reported injuries using the OS ##HA formula and compared these rates to industry - wide data . We also evaluated psychological [SEP]',
  'existing_cui': 'C0442611',
  'existing_cui_name': 'Industrial site',
  'existing_cui_desc': '[CLS] Industrial site [unused3] ( Spa ##tial Concept , environment ) [SEP]',
  'our_bi_cui': 'C0016502',
  'our_bi_cui_name': 'Food processing industry',
  'our_bi_cui_desc': '[CLS] Food processing industry [unused3] ( Organization : F ##O ##OD PR ##OC ##ES ##SI ##NG IN ##D ##US ##T ; IN ##D ##US ##T F ##O ##OD PR ##OC ##ES ##SI ##NG ; Food - Process ##ing Industries ; Food - Process ##ing Industry ; food industry ; Industries , Food - Process ##ing ; Industry , Food Process ##ing ; Food Process ##ing Industry ; Industry , Food - Process ##ing ) [SEP]',
  'our_cross_cui': 'C0025019',
  'our_cross_cui_name': 'Meat-Packing Industry',
  'our_cross_cui_desc': '[CLS] Meat - Pack ##ing Industry [unused3] ( Organization : Industries , Meat - Pack ##ing ; Industry , Meat - Pack ##ing ; ME ##AT PA ##C ##K ##ING IN ##D ##US ##T ; Meat Pack ##ing Industry ; Meat - Pack ##ing Industries ; IN ##D ##US ##T ME ##AT PA ##C ##K ##ING ; Industry , Meat Pack ##ing ) [SEP]'},
 {'mention_id': '27301681.26',
  'test_set_idx': 1926,
  'type': 'T103',
  'name': '8-amino-3 ,6 - dioxaoctanoic acid',
  'context': '[CLS] hair ##pin motifs for tandem hair ##pin . It is demonstrated here ##in that 8 - amino - 3 , 6 - di ##ox ##ao ##ct ##ano ##ic acid also functions as a link ##er element that can extend to 2 - b ##p in hair ##pin motifs . Not ##ably , although the [unused1] 8 - amino - 3 , 6 - di ##ox ##ao ##ct ##ano ##ic acid [unused2] - containing unit can fail to bind the expected sequence , p ##oly ##ami ##de 4 , which has two 8 - amino - 3 , 6 - di ##ox ##ao ##ct ##ano ##ic acid units facing each other in a hair ##pin form , successfully showed the expected motif and a K ##D value [SEP]',
  'existing_cui': 'C0623748',
  'existing_cui_name': '2-amino-8-phosphonooctanoic acid',
  'existing_cui_desc': '[CLS] 2 - amino - 8 - p ##hos ##phon ##oo ##ct ##ano ##ic acid [unused3] ( Chemical : 2 - AP ##8 ) [SEP]',
  'our_bi_cui': 'C2353403',
  'our_bi_cui_name': 'octadecenedioic acid',
  'our_bi_cui_desc': '[CLS] o ##ct ##ade ##cene ##dio ##ic acid [unused3] ( Chemical : O ##CT ##AD ##EC ##EN ##ED ##IO ##IC AC ##ID ; Oct ##ade ##cene ##dio ##ic A ##cid ) [SEP]',
  'our_cross_cui': 'C0029224',
  'our_cross_cui_name': 'Organic Chemicals',
  'our_cross_cui_desc': '[CLS] Organic Chemical ##s [unused3] ( Chemical , substance : organic chemicals ; compound organic ; Organic Chemical ##s [ Chemical / In ##g ##red ##ient ] ; organic compound ; Chemical ##s , Organic ; organic chemical ; Organic Chemical ; chemical organic ; organic compounds ; Organic compound ; Organic compounds ; Organic chemical ) [SEP]'},
 {'mention_id': '27894019.27',
  'test_set_idx': 21088,
  'type': 'T103',
  'name': 'CYP314a1',
  'context': "[CLS] C ##YP ##31 ##4 ##a ##1 ) that are responsible for the final two h ##ydro ##xy ##lation ##s in the production of the m ##olt - in ##ducing hormone , 20 - h ##ydro ##xy ##ec ##dy ##son ##e . The lack of crystal structures and bio ##chemical ass ##ays for C ##YP ##31 ##5 ##a ##1 or [unused1] C ##YP ##31 ##4 ##a ##1 [unused2] , has prevented further experimental exploration of X ##ant ##hur ##eni ##c acid and E ##re ##b ' s m ##olt in ##hibition mechanisms . Here ##in , a wide array of computational techniques - ho ##mology modeling , molecular dynamics simulation ##s , binding site bio ##in ##formatics , flexible receptor - flexible l ##igan ##d dock ##ing [SEP]",
  'existing_cui': 'C1418799',
  'existing_cui_name': 'PPIF gene',
  'existing_cui_desc': '[CLS] PP ##IF gene [unused3] ( Ana ##tom ##ical Structure : C ##Y ##CL ##OP ##H ##IL ##IN 3 ; P ##EP ##TI ##D ##Y ##L - PR ##OL ##Y ##L C ##IS / T ##RA ##NS ISO ##ME ##RA ##SE , MIT ##OC ##H ##ON ##DR ##IA ##L ; P ##ept ##idy ##l ##p ##rol ##yl Is ##omer ##ase F Gene ; PP ##IF Gene ; C ##Y ##CL ##OP ##H ##IL ##IN F ; p ##ept ##idy ##l ##p ##rol ##yl is ##omer ##ase F ; Cy ##p - D ; h ##C ##y ##P ##3 ; c ##y ##c ##lop ##hil ##in D ) [SEP]',
  'our_bi_cui': 'C0059563',
  'our_bi_cui_name': 'cytochrome P450 3A',
  'our_bi_cui_desc': '[CLS] c ##yt ##och ##rome P ##45 ##0 3 ##A [unused3] ( Chemical , substance : N - De ##met ##hyl ##ase , E ##ry ##th ##rom ##y ##cin ; Cy ##to ##ch ##rome p ##45 ##0 C ##YP ##3 ##A enzyme ; Cy ##to ##ch ##rome P 450 ##II ##IA ; Cy ##to ##ch ##rome P - 450 C ##YP ##3 ##A [ Chemical / In ##g ##red ##ient ] ; Cy ##to ##ch ##rome P - 450 C ##YP ##3 ##A ; Cy ##to ##ch ##rome P ##45 ##0 3 ##A ; E ##ry ##th ##rom ##y ##cin N - De ##met ##hyl ##ase ; C ##Y ##TO ##CH ##RO ##ME P 450 C ##YP 00 ##3 A ; C ##YP ##3 ##A ; P [SEP]',
  'our_cross_cui': 'C0010762',
  'our_cross_cui_name': 'Cytochrome P450',
  'our_cross_cui_desc': '[CLS] Cy ##to ##ch ##rome P ##45 ##0 [unused3] ( Chemical , substance : c ##yt ##och ##rome ##s p ##45 ##0 ; c ##yt ##och ##rome P ##45 ##0 ; C ##Y ##TO ##CH ##RO ##ME P 450 E ##N ##Z ##Y ##ME S ##Y ##ST ##EM ; Cy ##to ##ch ##rome p ##45 ##0 enzyme ; Super ##family , Cy ##to ##ch ##rome P ##45 ##0 ; P ##45 ##0 En ##zy ##mes ; c ##yt ##och ##rome p 450 ; En ##zy ##mes , P ##45 ##0 ; Cy ##to ##ch ##rome P 450 Families ; Cy ##to ##ch ##rome P ##45 ##0 En ##zy ##me ; P ##45 ##0 ; C ##YP ##45 ##0 ; Cy ##to ##ch ##rome p ##45 ##0 ; [SEP]'},
 {'mention_id': '27669217.35',
  'test_set_idx': 13686,
  'type': 'T017',
  'name': 'ermB',
  'context': '[CLS] ##f ##E genes were not detect ##able in these is ##olate ##s . The presence of 23 ##S r ##RNA mutations in specific is ##olate ##s did not significantly change mac ##rol ##ide MI ##C s . No n ##uc ##leo ##tide substitution ##s were found in genes encoding rib ##oso ##mal proteins L ##4 or L ##22 . The [unused1] er ##m ##B [unused2] and er ##m ##T genes were found in the genome ##s of all is ##olate ##s . These two genes were acquired independently in one highly v ##ir ##ulent is ##olate AL ##10 ##100 ##2 , and cluster ##ed with T ##n ##9 ##16 and IS ##12 ##16 , respectively . The expression of both er ##m ##B and er ##m [SEP]',
  'existing_cui': 'C0732611',
  'existing_cui_name': 'Selective Estrogen Receptor Modulators',
  'existing_cui_desc': '[CLS] Select ##ive E ##stro ##gen Re ##ceptor Mo ##du ##lator ##s [unused3] ( Chemical , SE ##R ##M , medication , product , substance : E ##ST ##RO ##GE ##N R ##EC ##EP ##T M ##OD ##U ##LA ##TO ##RS SE ##LE ##CT ##IVE ; Select ##ive E ##stro ##gen Re ##ceptor Mo ##di ##fying Agents ; Select ##ive est ##rogen receptor m ##od ##ulator ##s ; Select ##ive o ##est ##rogen receptor m ##od ##ulator ; E ##stro ##gen Re ##ceptor Mo ##du ##lator , Select ##ive ; SE ##R ##M - Select ##ive est ##rogen receptor m ##od ##ulator ; SE ##R ##Ms ; SE ##R ##M - Select ##ive o ##est ##rogen receptor m ##od ##ulator ; SE ##LE ##CT ##IVE E ##ST [SEP]',
  'our_bi_cui': 'C0017339',
  'our_bi_cui_name': 'Genes , Bacterial',
  'our_bi_cui_desc': '[CLS] Gene ##s , Ba ##cter ##ial [unused3] ( Ana ##tom ##ical Structure , substance : Ba ##cter ##ial Gene ##s ; BA ##CT GE ##NE ##S ; GE ##NE ##S BA ##CT ; bacterial gene ; Ba ##cter ##ial genes ; Gene , Ba ##cter ##ial ; BA ##CT GE ##NE ; Ba ##cter ##ial gene ; Ba ##cter ##ial Gene ; GE ##NE BA ##CT ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27932585.12',
  'test_set_idx': 22239,
  'type': 'T103',
  'name': 'TYW3',
  'context': '[CLS] S - ad ##eno ##sy ##lm ##eth ##ion ##ine ( S - ad ##eno ##sy ##lm ##eth ##ion ##ine ) - dependent met ##hyl ##tra ##ns ##fer ##ases regulate a wide range of biological processes through the modification of proteins , nuclei ##c acids , p ##oly ##sa ##cc ##hari ##des , as well as various meta ##bol ##ites . [unused1] T ##Y ##W ##3 [unused2] / Ta ##w ##3 is a SA ##M - dependent met ##hyl ##tra ##ns ##fer ##ase responsible for the formation of a t ##RNA modification known as w ##y ##but ##os ##ine and its derivatives that are required for accurate de ##co ##ding in protein synthesis . Here , we report the crystal structure of Ta ##w ##3 , a [SEP]',
  'existing_cui': 'C1823563',
  'existing_cui_name': 'TYW3 gene',
  'existing_cui_desc': '[CLS] T ##Y ##W ##3 gene [unused3] ( Ana ##tom ##ical Structure : FL ##J ##40 ##9 ##18 ; t ##RNA - Y ##W - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; t ##RNA - y ##W s ##ynth ##esi ##zing protein 3 ho ##mo ##log ; t ##RNA - W ##Y ##B ##UT ##OS ##IN ##E - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ) [SEP]',
  'our_bi_cui': 'C1823563',
  'our_bi_cui_name': 'TYW3 gene',
  'our_bi_cui_desc': '[CLS] T ##Y ##W ##3 gene [unused3] ( Ana ##tom ##ical Structure : FL ##J ##40 ##9 ##18 ; t ##RNA - Y ##W - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; t ##RNA - y ##W s ##ynth ##esi ##zing protein 3 ho ##mo ##log ; t ##RNA - W ##Y ##B ##UT ##OS ##IN ##E - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28495651.10',
  'test_set_idx': 37785,
  'type': 'T201',
  'name': 'cardiovascular death',
  'context': '[CLS] from studies of the association of Q ##T pro ##long ##ation with card ##iovascular death have been inconsistent . The purpose of this study was to compare the major correction formula ##s to percent ##ile values of Q ##T for heart rate ranges as to their ability to remove the relationship of Q ##T to heart rate and to predict [unused1] card ##iovascular death [unused2] . Part ##ici ##pants were 16 , 53 ##1 veterans who had an initial EC ##G at the Veterans Affairs Medical Center , Pa ##lo Alto , between March 31 , 1987 , and December 20 , 1999 , and were followed for card ##iovascular death . The 4 major correction formula ##s ( Ba ##ze ##tt , Fr ##ider ##ici [SEP]',
  'existing_cui': 'C0007226',
  'existing_cui_name': 'Cardiovascular system',
  'existing_cui_desc': '[CLS] Card ##iovascular system [unused3] ( Body System , body structure : Structure of card ##iovascular system , un ##sp ##ec ##ified ; CA ##RD ##IO ##VA ##SC ##U ##LA ##R S ##Y ##ST ##EM ; Card ##iovascular ; Organ System , Card ##iovascular ; CA ##RD ##IO ##VA ##SC S ##Y ##ST ##EM ; Card ##iovascular structure ; card ##iovascular system ; Card ##iovascular Organ System ; C ##ir ##cula ##tory ; Structure of card ##iovascular system ; Card ##iovascular Systems ; card ##iovascular structure ; C ##ir ##cula ##tory System ; C ##ir ##cula ##tory Systems ; { C ##ir ##cula ##tory system } ; c ##ir ##cula ##tory ; C ##ir ##cula ##tory system ; Body System , Card ##iovascular ; card ##iovascular system [SEP]',
  'our_bi_cui': 'C0376297',
  'our_bi_cui_name': 'Cardiac Death',
  'our_bi_cui_desc': '[CLS] Card ##iac Death [unused3] ( B ##iol ##og ##ic Fun ##ction : death ; cardiac ; cardiac death ; cardiac deaths ; Card ##iac death ; Death , Card ##iac ; cardiac ; death ) [SEP]',
  'our_cross_cui': 'C2926098',
  'our_cross_cui_name': "Mechanism of death in patient 's dying of cardiovascular causes : Type : Point in time : ^ Patient : Nominal",
  'our_cross_cui_desc': "[CLS] Me ##chan ##ism of death in patient ' s dying of card ##iovascular causes : Type : Point in time : ^ Pat ##ient : No ##mina ##l [unused3] ( Clinical At ##tri ##but ##e : Death due to C ##V causes ; Me ##chan ##ism of death in patient ' s dying of card ##iovascular causes : Type : P ##t : ^ Pat ##ient : No ##m ; Me ##chan ##ism of death in patient ' s dying of card ##iovascular causes ) [SEP]"},
 {'mention_id': '27791364.33',
  'test_set_idx': 17001,
  'type': 'T033',
  'name': 'near - infrared absorption',
  'context': '[CLS] serum album ##in na ##no ##par ##tic ##les demonstrate prominent tumor - contrasted imaging performance both on the photo ##aco ##ustic and magnetic resonance imaging m ##oda ##lities . The subsequent G ##d : C ##u ##S @ b ##ov ##ine serum album ##in - mediated Photo ##thermal therapy result shows high therapy efficacy as a result of their potent [unused1] near - infrared absorption [unused2] and high photo ##thermal conversion efficiency . The immune response triggered by G ##d : C ##u ##S @ b ##ov ##ine serum album ##in - mediated Photo ##thermal therapy is pre ##lim ##ina ##rily explored . In addition , toxicity studies in v ##it ##ro and in v ##ivo verify that G ##d : C ##u ##S @ b [SEP]',
  'existing_cui': 'C0376519',
  'existing_cui_name': 'Spectroscopy , Near-Infrared',
  'existing_cui_desc': '[CLS] S ##pect ##ros ##copy , Near - In ##fra ##red [unused3] ( Health Care Act ##ivity , procedure : S ##pect ##rome ##try , Near - In ##fra ##red ; N ##IR s ##pect ##ros ##copy ; Near infrared s ##pect ##ros ##copy ; Near - In ##fra ##red S ##pect ##rome ##tries ; Near - In ##fra ##red S ##pect ##rome ##try ; S ##pect ##rome ##tries , Near - In ##fra ##red ; Near - In ##fra ##red S ##pect ##ros ##copy ; S ##pect ##ros ##copy , N ##IR ; N ##IR S ##pect ##ros ##copy ; N ##IR S ##pect ##ros ##co ##pies ; S ##pect ##ros ##co ##pies , Near - In ##fra ##red ; near infrared s ##pect ##ros ##copy [SEP]',
  'our_bi_cui': 'C0234682',
  'our_bi_cui_name': 'Light absorption',
  'our_bi_cui_desc': '[CLS] Light absorption [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : light absorption ; absorption of light ; Light absorption , function ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28051263.27',
  'test_set_idx': 24246,
  'type': 'T168',
  'name': 'alcohol',
  'context': '[CLS] Science ##D ##ire ##ct . Results indicated the most relevant emotional factors to be depression and anxiety . The areas of investigation for cognitive functioning concern those contents and processes related to the ability to initiate and maintain lifestyle changes . The most important behavioral factors identified are physical activity , nutrition / food intake and substance consumption : coffee , [unused1] alcohol [unused2] , cigarettes . Some of the factors identified act as protective factors , other as vulnerability factors . Non ##al ##co ##hol ##ic fatty liver disease / non ##al ##co ##hol ##ic s ##te ##ato ##he ##pa ##titis may be considered a cognitive - behavioral disease , the most effective management being lifestyle changes , with emphasis on diet and exercise . [SEP]',
  'existing_cui': 'C0202304',
  'existing_cui_name': 'Ethanol measurement',
  'existing_cui_desc': '[CLS] Ethan ##ol measurement [unused3] ( Health Care Act ##ivity , procedure : DR ##U ##G SC ##RE ##EN Q ##U ##AN ##TA ##LC ##OH ##OL ##S ; Ethan ##ol Me ##as ##ure ##ment ; Al ##co ##hol ##s levels ; Al ##co ##hol ##s ; Ethan ##ol ; DR ##U ##G SC ##RE ##EN Q ##U ##AN ##TI ##TA ##TI ##VE AL ##CO ##H ##OL ##S ; Al ##co ##hol ; alcohol measurements ; Me ##as ##ure ##ment of alcohol ; E ##thy ##l Al ##co ##hol Me ##as ##ure ##ments ; Drug screen q ##uant ##al ##co ##hol ##s ; Al ##co ##hol level ; Al ##co ##hol measurement ; ET ##HA ##N ##OL ; alcohol level test ) [SEP]',
  'our_bi_cui': 'C0001975',
  'our_bi_cui_name': 'Alcohols',
  'our_bi_cui_desc': '[CLS] Al ##co ##hol ##s [unused3] ( Chemical , substance : alcohol ; Al ##co ##hol [ Chemical Class ] ; Al ##co ##hol ; Al ##co ##hol ##s [ Chemical / In ##g ##red ##ient ] ; alcohol ##s ; class of chemical alcohol ##s ; alcohol substance ) [SEP]',
  'our_cross_cui': 'C0001967',
  'our_cross_cui_name': 'Alcoholic Beverages',
  'our_cross_cui_desc': '[CLS] Al ##co ##hol ##ic Be ##ver ##ages [unused3] ( Food , Al ##co ##hol ##ic , substance : Be ##ver ##age , Al ##co ##hol ##ic ; Al ##co ##hol ##ic Be ##ver ##age ; Al ##co ##hol ##ic beverages ; alcoholic drinks ; Be ##ver ##ages ; Be ##ver ##ages , Al ##co ##hol ##ic ; alcoholic beverages ; alcoholic beverage ; alcoholic ##s beverages ; Al ##co ##hol ##ic beverage ; Al ##co ##hol ##ic drink ; alcoholic drinking ) [SEP]'},
 {'mention_id': '28351716.33',
  'test_set_idx': 32886,
  'type': 'T170',
  'name': 'Particle Swarm Optimization',
  'context': '[CLS] and various en ##tro ##pies namely Ya ##ger , Ren ##yi , Kapoor , Shannon , and Fu ##zzy were extracted from the de ##com ##posed images . Then , adaptive synthetic sampling approach was applied to balance the normal and abnormal data ##sets . Next , the extracted features were ranked according to the significance ##s using [unused1] Part ##icle S ##warm Op ##ti ##mi ##zation [unused2] . There ##upon , the ranked and selected features were used to train the random forest class ##ifier using s ##tra ##ti ##fied 10 - fold cross valid ##ation . Overall , the proposed system presented a performance rate of 92 . 48 % , and a sensitivity and specific ##ity of 89 . 37 % and 95 [SEP]',
  'existing_cui': 'C4075704',
  'existing_cui_name': 'Optimization of medication',
  'existing_cui_desc': '[CLS] Op ##ti ##mi ##zation of medication [unused3] ( Health Care Act ##ivity , procedure ) [SEP]',
  'our_bi_cui': 'C0002045',
  'our_bi_cui_name': 'algorithm',
  'our_bi_cui_desc': '[CLS] algorithm [unused3] ( Intel ##lect ##ual Product : Al ##gor ##ith ##m ; Al ##gor ##ith ##ms ; algorithms ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27813022.13',
  'test_set_idx': 18029,
  'type': 'T058',
  'name': 'On admission',
  'context': '[CLS] ##omy : review of the literature A 63 - year - old man , who had undergone total gas ##tre ##ct ##omy and R ##oux - en - Y reconstruction for gas ##tric cancer 10 years previously , was admitted to our hospital with complaints of abdominal pain , p ##al ##pable abdominal tumor , and hem ##ate ##mes ##is . [unused1] On admission [unused2] , the abdominal tender ##ness was improving and no abdominal tumor was p ##al ##pable . Mi ##ld inflammatory changes and an ##emia were noted on blood examination . Abd ##omi ##nal com ##puted to ##mo ##graphy revealed a tumor with a layered structure in the left abdomen . The patient was diagnosed with in ##test ##inal o ##bs ##truction secondary [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0184666',
  'our_bi_cui_name': 'Hospital admission',
  'our_bi_cui_desc': '[CLS] Hospital admission [unused3] ( Health Care Act ##ivity , Hospital , procedure , treatment : admitted ; Ad ##mission ; hospital ; admitted hospital ; admissions ; Ad ##mit ( to ) ; hospital ; admit ; Ad ##mit ; hospital ##ization admission ; hospital admission ; admission hospital ; Ad ##mission ; admit to hospital ; Ad ##mission to hospital ; admits ; admitting ; Hospital Ad ##mission ; hospital admissions ; admission to hospital ; admissions hospital ; admission ) [SEP]',
  'our_cross_cui': 'C0809949',
  'our_cross_cui_name': 'Admission activity',
  'our_cross_cui_desc': '[CLS] Ad ##mission activity [unused3] ( Health Care Act ##ivity : Ad ##mit ##ted ; Ad ##mission ) [SEP]'},
 {'mention_id': '27872234.68',
  'test_set_idx': 20129,
  'type': 'T103',
  'name': 'SKCa',
  'context': '[CLS] ##im ##id ##in - 4 - y ##l ] - am ##ine , a selective SK ##C ##a act ##iva ##tor , were marginal ##ly decreased in SH ##RS ##P art ##eries compared with W ##ista ##r - Kyoto art ##eries . The expression of end ##oth ##eli ##al trans ##ient receptor potential van ##illo ##id type 4 channel and [unused1] SK ##C ##a [unused2] protein was significantly decreased in the SH ##RS ##P me ##sent ##eric art ##eries compared with those of W ##ista ##r - Kyoto , whereas function and expression of I ##K ##C ##a were preserved in SH ##RS ##P art ##eries . These findings suggest that End ##oth ##eli ##um - dependent h ##yper ##pol ##ari ##zation - mediated responses are [SEP]',
  'existing_cui': 'C0006675',
  'existing_cui_name': 'Calcium',
  'existing_cui_desc': '[CLS] Cal ##ci ##um [unused3] ( Chemical , medication , substance : Co ##agu ##lation factor IV - R ##ET ##IR ##ED - ; CO ##AG FA ##CT ##OR IV ; Co ##agu ##lation Factor IV ; B ##L ##O ##OD CO ##AG FA ##CT ##OR IV ; C ##a + + element ; factor i ##v ; calcium ; Blood Co ##agu ##lation Factor IV ; Cal ##ci ##um co ##agu ##lation factor ; Co ##agu ##lation factor IV ; calcium ##s ; Cal ##ci ##um [ Chemical / In ##g ##red ##ient ] ; C ##a - Cal ##ci ##um ; CA ##LC ##I ##UM ; Factor IV ; C ##a element ; Cal ##ci ##um anal ##yte ; Factor IV , Co ##agu ##lation [SEP]',
  'our_bi_cui': 'C0949653',
  'our_bi_cui_name': 'Calcium-Activated Potassium Channel',
  'our_bi_cui_desc': '[CLS] Cal ##ci ##um - Act ##ivated Po ##tas ##sium Channel [unused3] ( Chemical : Po ##tas ##sium Channel ##s , Cal ##ci ##um De ##pen ##dent ; Po ##tas ##sium Channel ##s , Cal ##ci ##um - Act ##ivated [ Chemical / In ##g ##red ##ient ] ; Po ##tas ##sium Channel , Cal ##ci ##um Act ##ivated ; Po ##tas ##sium Channel ##s , Cal ##ci ##um - De ##pen ##dent ; K CH ##AN ##NE ##LS CA ##LC ##I ##UM ACT ##I ##VA ##TE ##D ; C ##a ##2 + - Act ##ivated K + Channel ##s ; CA ACT ##I ##VA ##TE ##D K CH ##AN ##NE ##L ; Cal ##ci ##um - Act ##ivated K + Channel ##s ; K CH ##AN [SEP]',
  'our_cross_cui': 'C1571638',
  'our_cross_cui_name': 'Small-Conductance Calcium-Activated Potassium Channels',
  'our_cross_cui_desc': '[CLS] Small - Con ##duct ##ance Cal ##ci ##um - Act ##ivated Po ##tas ##sium Channel ##s [unused3] ( Chemical : SK Po ##tas ##sium Channel ##s ; Small Con ##duct ##ance Cal ##ci ##um Act ##ivated Po ##tas ##sium Channel ##s ; Small - Con ##duct ##ance Cal ##ci ##um - Act ##ivated Po ##tas ##sium Channel ##s [ Chemical / In ##g ##red ##ient ] ; Po ##tas ##sium Channel ##s , SK ) [SEP]'},
 {'mention_id': '27587730.27',
  'test_set_idx': 10766,
  'type': 'T103',
  'name': 'gga-miR-21',
  'context': "[CLS] in ##hibit ##ed pre - ad ##ip ##oc ##yte proliferation . In contrast , the g ##ga - mi ##R - 21 inhibitor enhanced pre - ad ##ip ##oc ##yte proliferation . The subsequent investigation identified K ##rup ##pel - like factor 5 m ##RNA as a target of g ##ga - mi ##R - 21 . The [unused1] g ##ga - mi ##R - 21 [unused2] mimic in ##hibit ##ed K ##rup ##pel - like factor 5 3 ( ' ) U ##TR reporter activity and decreased end ##ogen ##ous K ##rup ##pel - like factor 5 expression in primary pre - ad ##ip ##ocytes . K ##rup ##pel - like factor 5 knock ##down using RNA ##i had a similar effect to that of [SEP]",
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1537719',
  'our_bi_cui_name': 'MIR21 gene',
  'our_bi_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '28127853.10',
  'test_set_idx': 26304,
  'type': 'T082',
  'name': 'Taipei',
  'context': '[CLS] program To develop and evaluate the effectiveness of an inter ##gene ##ration ##al health promotion program . This was an action research project . A total of 34 participants attended the 12 - week program and completed the pre - test and post - test . There were 16 middle - aged and nine older adults recruited from a district of [unused1] Taipei [unused2] , and nine young adults recruited from the principal investigator \' s university . The " At ##ti ##tudes toward A ##ging Scale " and the " Spiritual Health Scale " were two assessment instruments used in the study . The results showed that there were significant improvements in the At ##ti ##tudes toward A ##ging Scale for the young adult group [SEP]',
  'existing_cui': 'C0062316',
  'existing_cui_name': 'hemoglobin G-Taipei',
  'existing_cui_desc': '[CLS] hem ##og ##lo ##bin G - Taipei [unused3] ( Chemical , substance : H ##b 22 ( B ##4 ) , G ##lu - g ##ly ; hem ##og ##lo ##bin G Tai ##bei ; He ##mo ##g ##lo ##bin G - Taipei ; H ##b 22 ; H ##b G - Taipei ; H ##b G Tai ##bei ; Ha ##em ##og ##lo ##bin G - Taipei ; He ##mo ##g ##lo ##bin G Taipei ) [SEP]',
  'our_bi_cui': 'C0039260',
  'our_bi_cui_name': 'Taiwan',
  'our_bi_cui_desc': '[CLS] Taiwan [unused3] ( Spa ##tial Concept , geographic location : T ##W ##N ; Form ##osa ; Republic of China ; republic of chin ##a ; T ##W ; T ##A ##I ##WA ##N ; form ##osa ; ta ##i ##wan ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '27669217.3',
  'test_set_idx': 13654,
  'type': 'T017',
  'name': 'ermT',
  'context': '[CLS] In ##du ##cible Express ##ion of both er ##m ##B and [unused1] er ##m ##T [unused2] Con ##ferred High Mac ##rol ##ide Resistance in St ##re ##pt ##oc ##oc ##cus gal ##lo ##ly ##ticus subsp . paste ##uria ##nus Is ##olate ##s in China St ##re ##pt ##oc ##oc ##cus gal ##lo ##ly ##ticus subsp . paste ##uria ##nus is an under - recognized path ##ogen and zoo ##not ##ic agent causing op ##port ##uni ##stic infections in humans . Despite increasing recognition of this subspecies as a cause for human infectious diseases , limited information is known about its anti ##biotic resistance mechanism . In this study , we aim to identify the molecular mechanism underlying the high mac ##rol ##ide resistance of six S [SEP]',
  'existing_cui': 'C0732611',
  'existing_cui_name': 'Selective Estrogen Receptor Modulators',
  'existing_cui_desc': '[CLS] Select ##ive E ##stro ##gen Re ##ceptor Mo ##du ##lator ##s [unused3] ( Chemical , SE ##R ##M , medication , product , substance : E ##ST ##RO ##GE ##N R ##EC ##EP ##T M ##OD ##U ##LA ##TO ##RS SE ##LE ##CT ##IVE ; Select ##ive E ##stro ##gen Re ##ceptor Mo ##di ##fying Agents ; Select ##ive est ##rogen receptor m ##od ##ulator ##s ; Select ##ive o ##est ##rogen receptor m ##od ##ulator ; E ##stro ##gen Re ##ceptor Mo ##du ##lator , Select ##ive ; SE ##R ##M - Select ##ive est ##rogen receptor m ##od ##ulator ; SE ##R ##Ms ; SE ##R ##M - Select ##ive o ##est ##rogen receptor m ##od ##ulator ; SE ##LE ##CT ##IVE E ##ST [SEP]',
  'our_bi_cui': 'C1417123',
  'our_bi_cui_name': 'MET gene',
  'our_bi_cui_desc': '[CLS] ME ##T gene [unused3] ( Ana ##tom ##ical Structure : D ##F ##N ##B ##9 ##7 ; he ##pa ##to ##cy ##te growth factor receptor ; He ##pa ##to ##cy ##te Growth Factor Re ##ceptor Gene ; Act ##ivated He ##pa ##to ##cy ##te Growth Factor Re ##ceptor Gene ; RC ##CP ##2 ; ME ##T PR ##OT ##O ##ON ##CO ##GE ##NE ; H ##EP ##AT ##OC ##Y ##TE G ##RO ##W ##TH FA ##CT ##OR R ##EC ##EP ##TO ##R ; On ##co ##gene H ##G ##F ##R ; ME ##T Proto - On ##co ##gene , Re ##ceptor Ty ##ros ##ine Ki ##nas ##e Gene ; ME ##T pro ##to - on ##co ##gene , receptor t ##yr ##os ##ine kinase ; On [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27832198.61',
  'test_set_idx': 18683,
  'type': 'T017',
  'name': 'Cam734',
  'context': '[CLS] multiple anti ##mal ##aria ##l agents . These results were generated in the G ##C ##0 ##3 strain , used in multiple earlier P . f ##al ##ci ##par ##um ch ##lor ##o ##quin ##e resistance transport ##er studies , and might differ in natural is ##olate ##s harbor ##ing this all ##ele . Results presented here ##in show that [unused1] Cam ##7 ##34 [unused2] - mediated ch ##lor ##o ##quin ##e resistance is dependent on the rare A1 ##44 ##F mutation that has not been observed beyond Southeast Asia , and reveal distinct impacts of this and other Cam ##7 ##34 - specific mutations on ch ##lor ##o ##quin ##e resistance and parasite growth rates . B ##io ##chemical ass ##ays revealed a broad impact [SEP]',
  'existing_cui': 'C0006675',
  'existing_cui_name': 'Calcium',
  'existing_cui_desc': '[CLS] Cal ##ci ##um [unused3] ( Chemical , medication , substance : Co ##agu ##lation factor IV - R ##ET ##IR ##ED - ; CO ##AG FA ##CT ##OR IV ; Co ##agu ##lation Factor IV ; B ##L ##O ##OD CO ##AG FA ##CT ##OR IV ; C ##a + + element ; factor i ##v ; calcium ; Blood Co ##agu ##lation Factor IV ; Cal ##ci ##um co ##agu ##lation factor ; Co ##agu ##lation factor IV ; calcium ##s ; Cal ##ci ##um [ Chemical / In ##g ##red ##ient ] ; C ##a - Cal ##ci ##um ; CA ##LC ##I ##UM ; Factor IV ; C ##a element ; Cal ##ci ##um anal ##yte ; Factor IV , Co ##agu ##lation [SEP]',
  'our_bi_cui': 'C1708291',
  'our_bi_cui_name': 'CDC73 gene',
  'our_bi_cui_desc': '[CLS] CD ##C ##7 ##3 gene [unused3] ( Ana ##tom ##ical Structure : H ##Y ##RA ##X , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF ; cell division cycle 73 ; para ##fi ##bro ##min ; F ##I ##HP ; PA ##RA ##FI ##BR ##OM ##IN ; CD ##C ##7 ##3 Gene ; CE ##LL D ##I ##VI ##SI ##ON C ##Y ##CL ##E PR ##OT ##EI ##N 73 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; C ##1 ##OR ##F ##28 ; Cell Division Cycle 73 , Pa ##f ##1 / RNA Pol ##yme ##rase II Complex Co ##mpo ##nent , Ho ##mo ##log ( S . c ##ere ##vis ##iae [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '28249376.18',
  'test_set_idx': 29888,
  'type': 'T033',
  'name': 'digestibility',
  'context': '[CLS] ##n micro ##bial populations in sheep fed a 50 : 50 al ##fa ##lf ##a hay : concentrate diet An experiment was conducted to investigate the effects of a specific mixture of essential oils , containing thy ##me , c ##love and c ##innamon E ##O , on r ##ume ##n micro ##bial f ##er ##mentation , nut ##rient apparent [unused1] dig ##est ##ibility [unused2] and blood meta ##bol ##ites in fist ##ulated sheep . Six sheep fitted with r ##umi ##nal fist ##ula ##s were used in a repeated measurement design with two 24 - d periods to investigate the effect of adding mixture of essential oils at 0 ( control ) , 0 . 8 , and 1 . 6 m ##L / d [SEP]',
  'existing_cui': 'C0233423',
  'existing_cui_name': 'Suggestibility',
  'existing_cui_desc': '[CLS] Su ##gg ##est ##ibility [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : suggest ##ibility ; Su ##gg ##est ##ibility , function ) [SEP]',
  'our_bi_cui': 'C0012238',
  'our_bi_cui_name': 'Digestion',
  'our_bi_cui_desc': '[CLS] Digest ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : dig ##estive system functions ; dig ##est ##ion ; Digest ##ive system functions ; dig ##est ##ions ; Digest ##ive system function ; Fun ##ctions of the dig ##estive system ; Digest ##ive ; Digest ##ive tract function ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28179911.81',
  'test_set_idx': 27675,
  'type': 'T204',
  'name': 'T2',
  'context': '[CLS] ##us T ##2 trans ##genic rice lines ( G ##5 and G ##2 ) , different Co ##WR ##I ##1 expression levels were detected , but no Co ##WR ##I ##1 trans ##cripts were detected in the wild type . Ana ##ly ##ses of the seed oil content , star ##ch content , and total protein content indicated that the two [unused1] T ##2 [unused2] trans ##genic lines showed a significant increase ( P < 0 . 05 ) in seed oil content . The trans ##genic lines also showed a significant increase in star ##ch content , whereas total protein content decreased significantly . Further analysis of the fatty acid composition revealed that palm ##itic acid ( C ##16 : 0 ) and l ##ino [SEP]',
  'existing_cui': 'C1335834',
  'existing_cui_name': 'SLC25A5 gene',
  'existing_cui_desc': '[CLS] SL ##C ##25 ##A ##5 gene [unused3] ( Ana ##tom ##ical Structure : AD ##P / ATP T ##RA ##NS ##L ##OC ##AS ##E 2 ; 2 ##F ##1 ; Sol ##ute Carrier Family 25 ( Mi ##to ##cho ##nd ##rial Carrier ; Aden ##ine N ##uc ##leo ##tide Trans ##loc ##ator ) , Member 5 Gene ; AD ##EN ##IN ##E N ##UC ##LE ##OT ##ID ##E T ##RA ##NS ##L ##OC ##AT ##OR 2 ; SL ##C ##25 ##A ##5 Gene ; AD ##P / ATP CA ##R ##RI ##ER 2 ; so ##lut ##e carrier family 25 member 5 ; AD ##P / ATP T ##RA ##NS ##L ##OC ##AT ##OR OF F ##IB ##RO ##BL ##AS ##TS ; S ##OL ##UT ##E [SEP]',
  'our_bi_cui': 'C0282641',
  'our_bi_cui_name': 'Transgenes',
  'our_bi_cui_desc': '[CLS] Trans ##gene ##s [unused3] ( Ana ##tom ##ical Structure : trans ##gene ; Trans ##gene ) [SEP]',
  'our_cross_cui': 'C0085245',
  'our_cross_cui_name': 'Plants , Transgenic',
  'our_cross_cui_desc': '[CLS] Plants , Trans ##genic [unused3] ( E ##uka ##ryo ##te : Trans ##genic Plants ; Trans ##genic Plant ; trans ##genic plant ; trans ##genic plants ; Plant , Trans ##genic ; plant trans ##genic ; Trans ##genic plants ) [SEP]'},
 {'mention_id': '28424683.56',
  'test_set_idx': 35434,
  'type': 'T038',
  'name': 'expanding',
  'context': '[CLS] ##itic cells in contrast to O ##Cs are found to promote faster expansion of residual T cells within pu ##rified Natural killer cells resulting in the decline in Natural killer cell numbers from healthy individuals . Ad ##dition of anti - CD ##3 m ##A ##b in ##hibit ##s T cell proliferation while enhancing Natural killer cell expansion ; however , [unused1] expanding [unused2] Natural killer cells have lower c ##yt ##oto ##xi ##city but higher secret ##ion of IF ##N - γ . Ex ##pan ##sion and functional activation of super - charged Natural killer cells by O ##Cs is dependent on inter ##le ##uki ##n ( inter ##le ##uki ##n ) - 12 and IL - 15 . Thus , in this report , [SEP]',
  'existing_cui': 'C0205229',
  'existing_cui_name': 'Expanding',
  'existing_cui_desc': '[CLS] Ex ##pan ##ding [unused3] ( Spa ##tial Concept , qualifier value : Ex ##pan ##d ; Ex ##pan ##ded ; expanding ; expand ; expanded ) [SEP]',
  'our_bi_cui': 'C0205229',
  'our_bi_cui_name': 'Expanding',
  'our_bi_cui_desc': '[CLS] Ex ##pan ##ding [unused3] ( Spa ##tial Concept , qualifier value : Ex ##pan ##d ; Ex ##pan ##ded ; expanding ; expand ; expanded ) [SEP]',
  'our_cross_cui': 'C0007595',
  'our_cross_cui_name': 'cell growth',
  'our_cross_cui_desc': '[CLS] cell growth [unused3] ( B ##iol ##og ##ic Fun ##ction : cell expansion ; cellular growth ; Cell ##s - - Growth ; Cell Growth ; cell growth ##s ; cells growth ; growth cell ; growth of cell ; Cell ##ular Growth ; Cell ##ular Ex ##pan ##sion ) [SEP]'},
 {'mention_id': '27500205.30',
  'test_set_idx': 8462,
  'type': 'T103',
  'name': 'kindlins',
  'context': '[CLS] ##of ##orms and the levels of kind ##lins . Great interest in the kind ##lins has emerged from the recognition that they play major roles in controlling in ##te ##g ##rin function . In v ##it ##ro studies , in v ##ivo studies of mice def ##icient in kind ##lins , and studies of patients with genetic def ##ici ##encies of [unused1] kind ##lins [unused2] have clearly established that they regulate the capacity of in ##te ##g ##rin ##s to media ##te their functions . Kind ##lins are adapt ##or proteins ; their function em ##ana ##te from their interaction with binding partners , including the c ##yt ##op ##las ##mic tails of in ##te ##g ##rin ##s and components of the act ##in c ##yt [SEP]',
  'existing_cui': 'C1720776',
  'existing_cui_name': 'Kindling , Neurologic',
  'existing_cui_desc': '[CLS] Kind ##ling , N ##eur ##olo ##gic [unused3] ( B ##iol ##og ##ic Fun ##ction , N ##eur ##ology : kind ##ling ; Kind ##lings , N ##eur ##olo ##gic ; Kind ##ling ; N ##eur ##olo ##gic Kind ##lings ; N ##eur ##olo ##gic Kind ##ling ) [SEP]',
  'our_bi_cui': 'C0010422',
  'our_bi_cui_name': 'Crystallins',
  'our_bi_cui_desc': '[CLS] Crystal ##lins [unused3] ( Chemical , substance : Crystal ##lins [ Chemical / In ##g ##red ##ient ] ; Crystal ##lin ; Pro ##tein ##s , Len ##s ; C ##rist ##all ##in ; Len ##s Pro ##tein ##s ; crystal ##lin ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27476541.71',
  'test_set_idx': 7580,
  'type': 'T103',
  'name': 'NADPH oxidase inhibitor drug',
  'context': '[CLS] 4 ) and consequently reactive oxygen species and ma ##lon ##dial ##de ##hy ##de levels were elevated in high glucose - treated PC ##12 cells . At ##or ##vas ##tat ##in could significantly decrease high glucose - induced NO ##X ##s , reactive oxygen species and ma ##lon ##dial ##de ##hy ##de elevation and improve impaired cell via ##bility . It can be concluded that high glucose could el ##eva ##te NO ##X ##s activity , reactive oxygen species and ma ##lon ##dial ##de ##hy ##de levels in neural tissues and At ##or ##vas ##tat ##in as a small molecule [unused1] N ##AD ##P ##H o ##xi ##das ##e inhibitor drug [unused2] may prevent and delay di ##abe ##tic complications , particularly ne ##uro ##pathy . [SEP]',
  'existing_cui': 'C1151146',
  'existing_cui_name': 'NADPH oxidase activity',
  'existing_cui_desc': '[CLS] N ##AD ##P ##H o ##xi ##das ##e activity [unused3] ( B ##iol ##og ##ic Fun ##ction : N ##AD ( P ) H : oxygen o ##xi ##dor ##ed ##uc ##tase activity ; N ##AD ( P ) H o ##xi ##das ##e activity ; N ##AD ##P ##H o ##xi ##das ##e ) [SEP]',
  'our_bi_cui': 'C0026457',
  'our_bi_cui_name': 'Monoamine Oxidase Inhibitors',
  'our_bi_cui_desc': '[CLS] Mon ##oam ##ine O ##xi ##das ##e In ##hibit ##ors [unused3] ( Chemical , MA ##O ##I , medication , product , substance : MA ##O In ##hibit ##ors ; M ##ON ##OA ##MI ##NE O ##X ##ID ##AS ##E IN ##H ##IB ; Mon ##oam ##ine o ##xi ##das ##e - - In ##hibit ##ors ; inhibitor ##s ma ##o ; am ##ine o ##xi ##das ##e inhibitor ##s ; In ##hibit ##or , Mon ##oam ##ine O ##xi ##das ##e ; Mon ##oam ##ine o ##xi ##das ##e inhibitor ##s ; In ##hibit ##ors , MA ##O ; Mon ##oam ##ine O ##xi ##das ##e In ##hibit ##or ; inhibitor mon ##oam ##ine o ##xi ##das ##e ; inhibitor ##s mon ##oam ##ine o [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27617108.15',
  'test_set_idx': 11917,
  'type': 'T062',
  'name': 'RCT',
  'context': '[CLS] adult women and can negatively influence reproductive health t ##raj ##ector ##ies . Few random ##ized controlled trials have focused on strategies to improve out ##patient ad ##herence or to reduce reproductive m ##or ##bid ##ity in this population . This paper describes the research methods and preliminary effectiveness of recruitment , retention , and intervention strategies employed in a novel [unused1] RC ##T [unused2] designed to test a technology - enhanced community - health nursing intervention among urban and young adult with P ##el ##vic inflammatory disease . and young adult women aged 13 - 25 years were recruited during acute P ##el ##vic inflammatory disease visits in out ##patient clinics and emergency departments to participate in this I ##RB - approved trial . Part [SEP]',
  'existing_cui': 'C0281508',
  'existing_cui_name': 'Desmoplastic Small Round Cell Tumor',
  'existing_cui_desc': '[CLS] Des ##mo ##p ##lastic Small Round Cell Tu ##mor [unused3] ( B ##iol ##og ##ic Fun ##ction , m ##or ##ph ##olo ##gic abnormal ##ity : Des ##mo ##p ##lastic small round cell t ##umour ; Des ##mo ##p ##lastic Small - Cell Tu ##mor ##s ; Des ##mo ##p ##lastic Small Cell Tu ##mor ; Des ##mo ##p ##lastic Small Round - Cell Tu ##mor ; Pol ##y ##phe ##not ##y ##pic Small Round Cell Tu ##mor ; Tu ##mor , Des ##mo ##p ##lastic Small - Cell ; des ##mo ##p ##lastic small round cell tumor ; Small - Cell Tu ##mor ##s , Des ##mo ##p ##lastic ; Des ##mo ##p ##las . small round cell tumor ; Small - Cell Tu ##mor [SEP]',
  'our_bi_cui': 'C0206034',
  'our_bi_cui_name': 'Clinical Trials , Randomized',
  'our_bi_cui_desc': '[CLS] Clinical Trials , Random ##ized [unused3] ( Research Act ##ivity : random ##ized clinical trials ; Random ##ized Clinical Trials ; clinical random ##ize trials ; random ##ized clinical trial ; T ##RI ##AL ##S RA ##ND ##OM ##I ##Z ##ED C ##L ##IN ; clinical random ##ized trials ; C ##L ##IN T ##RI ##AL ##S RA ##ND ##OM ##I ##Z ##ED ; Trials , Random ##ized Clinical ; Random ##ized Clinical Trial ) [SEP]',
  'our_cross_cui': 'C0206035',
  'our_cross_cui_name': 'Controlled Clinical Trials , Randomized',
  'our_cross_cui_desc': '[CLS] Control ##led Clinical Trials , Random ##ized [unused3] ( Research Act ##ivity : CO ##NT ##RO ##LL ##ED C ##L ##IN T ##RI ##AL ##S RA ##ND ##OM ##I ##Z ##ED ; Random ##ized Control ##led Clinical Trial ) [SEP]'},
 {'mention_id': '28243201.63',
  'test_set_idx': 29705,
  'type': 'T017',
  'name': 'CCNEI',
  'context': "[CLS] h ##yper ##met ##hyl ##ated cell lines S ##W ##48 . In conclusion , we found the evidence of five potentially biological ##ly important genes in re ##current CR ##Cs that could possibly serve as a new potential therapeutic targets for patients with ch ##em ##ores ##istan ##ce . We post ##ulate that a ##ber ##rant met ##hyl ##ation of [unused1] CC ##NE ##I [unused2] , CC ##ND ##B ##P ##1 , P ##ON ##3 , DD ##X ##43 , and CH ##L ##1 in color ##ec ##tal cancer might be associated with the re ##cu ##rrence of color ##ec ##tal cancer and 5 - a ##za - 2 ' - de ##ox ##y ##cy ##ti ##dine - mediated restoration of 5 - F ##lu ##oro [SEP]",
  'existing_cui': 'C1332697',
  'existing_cui_name': 'CCNE1 gene',
  'existing_cui_desc': '[CLS] CC ##NE ##1 gene [unused3] ( Ana ##tom ##ical Structure : CC ##NE ##1 Gene ; c ##y ##c ##lin E ##1 ; C ##Y ##CL ##IN E ; c ##y ##c ##lin E ##t ; C ##Y ##CL ##IN E ##1 ; Cy ##c ##lin E ##1 Gene ; c ##y ##c ##lin E ##s ) [SEP]',
  'our_bi_cui': 'C1413182',
  'our_bi_cui_name': 'CCNI gene',
  'our_bi_cui_desc': '[CLS] CC ##NI gene [unused3] ( Ana ##tom ##ical Structure : CC ##NI ##1 ; c ##y ##c ##lin I ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27583176.8',
  'test_set_idx': 10705,
  'type': 'T170',
  'name': 'outcome grading scoring systems',
  'context': '[CLS] outcome assessment is mandatory for op ##ti ##mizing therapeutic efforts . We retrospective ##ly analyzed data from 34 ##2 patients with spontaneous primary In ##tra ##cer ##eb ##ral hem ##or ##r ##hage to evaluate possible predict ##ors of 30 - day mortality considering clinical , radio ##logical , and therapeutic ##al parameters . We also applied three widely accepted [unused1] outcome g ##rading scoring systems [unused2] [ ( In ##tra ##cer ##eb ##ral hem ##or ##r ##hage score , F ##UN ##C score and in ##tra ##cer ##eb ##ral hem ##or ##r ##hage g ##rading scale ] on our population to evaluate the correlation of these scores with the 30 - day mortality in our study . We also applied three widely accepted outcome g ##rading [SEP]',
  'existing_cui': 'C1273712',
  'existing_cui_name': 'Grading system used',
  'existing_cui_desc': '[CLS] G ##rading system used [unused3] ( Health Care Act ##ivity , o ##bs ##er ##vable entity : g ##rading system ) [SEP]',
  'our_bi_cui': 'C1708250',
  'our_bi_cui_name': 'Tumor Grading System',
  'our_bi_cui_desc': '[CLS] Tu ##mor G ##rading System [unused3] ( Intel ##lect ##ual Product : G ##rading system ; G ##rading System ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28188533.1',
  'test_set_idx': 27846,
  'type': 'T170',
  'name': 'SWATH ( ® )',
  'context': '[CLS] Generation of High - Quality [unused1] S ##WA ##TH ( ® ) [unused2] A ##c ##quisition Data for Label - free Q ##uant ##itative Pro ##te ##omi ##cs Studies Using Triple ##TO ##F ( ® ) Mass S ##pect ##rome ##ters Data - independent acquisition is a powerful mass s ##pect ##rome ##try technique that enables comprehensive MS and MS / MS analysis of all detect ##able species , providing an information rich data file that can be mined deeply . Here , we describe how to acquire high - quality S ##WA ##TH ( ® ) A ##c ##quisition data to be used for large quantitative pro ##te ##omi ##c studies . We specifically focus on using variable sized Q ##1 windows for acquisition of [SEP]',
  'existing_cui': 'C1416521',
  'existing_cui_name': 'ITSN2 gene',
  'existing_cui_desc': '[CLS] IT ##SN ##2 gene [unused3] ( Ana ##tom ##ical Structure : PR ##O ##20 ##15 ; SH ##3 domain protein 1 ##B ; inter ##sect ##in 2 ; SH ##3 ##P ##18 ; IN ##TE ##RS ##EC ##TI ##N 2 ; SH ##3 ##P ##18 - like WA ##SP associated protein ; SH ##3 D ##OM ##A ##IN PR ##OT ##EI ##N 1 ##B ; K ##IA ##A ##12 ##5 ##6 ) [SEP]',
  'our_bi_cui': 'C0025080',
  'our_bi_cui_name': 'Medical Devices',
  'our_bi_cui_desc': '[CLS] Medical Devi ##ces [unused3] ( Medical Devi ##ce , physical device , physical object : Clinical equipment ; B ##io ##medical device ; DE ##VI ##CE ME ##D ; ME ##D DE ##VI ##CE ##S ; ME ##D DE ##VI ##CE ; bio ##medical devices ; Devi ##ces , Medical ; Medical devices ; General bio ##medical device ; DE ##VI ##CE , GE ##NE ##RA ##L ME ##DI ##CA ##L ; B ##io ##medical equipment ; Devi ##ce , Medical ; Hospital , medical AND / OR surgical equipment ( physical device ) ; bio ##medical equipment ; medical equipment ; Medical equipment ; DE ##VI ##CE ##S ME ##D ; DE ##VI ##CE ; DE ##VI ##CE ##S ; medical device ; medical devices [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28438542.29',
  'test_set_idx': 36020,
  'type': 'T103',
  'name': 'inhibitors',
  'context': '[CLS] showed that compounds 5 ##b and 5 ##f , with 2 - ch ##lor ##o and 4 - flu ##oro substitution ##s , showed bio ##chemical activities in micro ##mo ##lar and sub ##mic ##rom ##olar range ( I ##C ##50 1 . 34 - 5 . 03 μ ##M ) against both the targeted enzymes . The structure - activity relationship el ##uc ##idated for these novel [unused1] inhibitor ##s [unused2] against both the enzymes provide new insight into the binding mode of the inhibitor ##s to the active sites of enzymes . The positive results of the bio ##chemical ass ##ay suggest that these compounds may be further op ##ti ##mized and utilized for the treatment of di ##abe ##tic re ##tino ##pathy . [SEP]',
  'existing_cui': 'C0243077',
  'existing_cui_name': 'inhibitors',
  'existing_cui_desc': '[CLS] inhibitor ##s [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0243077',
  'our_bi_cui_name': 'inhibitors',
  'our_bi_cui_desc': '[CLS] inhibitor ##s [unused3] ( Chemical ) [SEP]',
  'our_cross_cui': 'C0014432',
  'our_cross_cui_name': 'Enzyme Inhibitors',
  'our_cross_cui_desc': '[CLS] En ##zy ##me In ##hibit ##ors [unused3] ( Chemical , product , substance : E ##N ##Z ##Y ##ME IN ##H ##IB ; enzyme inhibitor ; enzyme inhibitor ##s ; En ##zy ##me inhibitor ##s ; In ##hibit ##ors , En ##zy ##me ; enzymes inhibitor ##s ; En ##zy ##me inhibitor ) [SEP]'},
 {'mention_id': '28077286.28',
  'test_set_idx': 25078,
  'type': 'T103',
  'name': 'MeHg',
  'context': '[CLS] ##quent behavioral alterations in adult birds have not received much attention . Moreover , pass ##eri ##ne birds are poorly represented in Me ##H ##g ne ##uro ##to ##xi ##cology studies in comparison to other a ##vian orders . Hence in this study , we used the egg injection method to investigate the long term effects of in o ##vo [unused1] Me ##H ##g [unused2] exposure on brain his ##top ##ath ##ology and courts ##hip behavior in a model song ##bird species , the z ##eb ##ra fin ##ch ( Ta ##eni ##opy ##gia gut ##tata ) . Egg treatment groups included : a low Me ##H ##g dose of 0 . 2 μ ##g H ##g g ( - 1 ) egg , a high [SEP]',
  'existing_cui': 'C0228520',
  'existing_cui_name': 'Structure of reticular formation of medulla oblongata',
  'existing_cui_desc': '[CLS] Structure of re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata [unused3] ( Ana ##tom ##ical Structure , body structure : bulb ##ar re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla ; B ##ul ##bar re ##tic ##ular formation ; Form ##ati ##o re ##tic ##ular ##is my ##ele ##nce ##pha ##li ; me ##du ##llar ##y re ##tic ##ular formation ; re ##tic ##ular formation of me ##du ##lla ; Me ##du ##llar ##y re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata ) [SEP]',
  'our_bi_cui': 'C0025424',
  'our_bi_cui_name': 'Mercury',
  'our_bi_cui_desc': '[CLS] Mercury [unused3] ( Chemical , H ##g , Metal , elemental , substance : mercury ; Mercury [ Chemical / In ##g ##red ##ient ] ; metal mercury ; H ##yd ##rar ##gy ##rum ; H ##g element ; Li ##quid Silver ; Quick ##si ##lve ##r ; H ##g ; H ##g - Mercury ; h ##g ; ME ##RC ##UR ##Y ) [SEP]',
  'our_cross_cui': 'C0025794',
  'our_cross_cui_name': 'Methylmercury Compounds',
  'our_cross_cui_desc': '[CLS] Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [unused3] ( Chemical : Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [ Chemical / In ##g ##red ##ient ] ; Mercury Co ##mpo ##unds , Met ##hyl ; ME ##TH ##Y ##L ME ##RC ##UR ##Y CP ##DS ; Met ##hyl Mercury Co ##mpo ##unds ; ME ##TH ##Y ##LM ##ER ##CU ##R ##Y CP ##DS ; Co ##mpo ##unds , Met ##hyl ##mer ##cu ##ry ; met ##hyl ##mer ##cu ##ry ; Co ##mpo ##unds , Met ##hyl Mercury ; Met ##hyl ##mer ##cu ##ry ) [SEP]'},
 {'mention_id': '27826436.38',
  'test_set_idx': 18368,
  'type': 'T204',
  'name': 'cultivars',
  'context': '[CLS] c ##v . To ##chi ##oto ##me . To ##chi ##oto ##me and Sa ##chin ##oka showed higher content of l ##inal ##ool and ( E ) - ne ##rol ##ido ##l . Sa ##chin ##oka showed the highest content of total sugar ##s and total acids . G ##ui ##mei ##ren showed higher sweetness index than the other three [unused1] cult ##iva ##rs [unused2] . Fi ##rm ##ness of To ##chi ##oto ##me was highest among all the varieties . The highest total soluble solid ##s T ##SS value was found in c ##v . Sa ##chin ##oka , followed by the G ##ui ##mei ##ren and To ##chi ##oto ##me varieties . Sa ##chin ##oka had the highest t ##it ##rata ##ble acid ##ity [SEP]',
  'existing_cui': 'C3385047',
  'existing_cui_name': 'Brassia hybrid cultivar',
  'existing_cui_desc': '[CLS] Brass ##ia hybrid cult ##iva ##r [unused3] ( E ##uka ##ryo ##te : Ada hybrid cult ##iva ##r ) [SEP]',
  'our_bi_cui': 'C0242775',
  'our_bi_cui_name': 'Agricultural Crops',
  'our_bi_cui_desc': '[CLS] Agricultural C ##rops [unused3] ( E ##uka ##ryo ##te : C ##rops , Agricultural ; C ##rops ; Agricultural C ##rop ) [SEP]',
  'our_cross_cui': 'C0032098',
  'our_cross_cui_name': 'Plants',
  'our_cross_cui_desc': '[CLS] Plants [unused3] ( E ##uka ##ryo ##te , organism : Ch ##lor ##op ##hy ##ta / Em ##b ##ryo ##phy ##ta group ; green plants ; Ch ##lor ##ob ##ion ##ta ; Ch ##lor ##ob ##ion ##ta B ##rem ##er , 1985 ; V ##iri ##di ##p ##lant ##ae ; Plants , General ; Kingdom Plant ##ae ; V ##iri ##di ##p ##lant ##ae C ##ava ##lier - Smith , 1981 ; plants ; Kingdom V ##iri ##di ##p ##lant ##ae ; plant ; ch ##lor ##op ##hy ##te / em ##b ##ryo ##phy ##te group ; Plant ) [SEP]'},
 {'mention_id': '28319502.3',
  'test_set_idx': 31859,
  'type': 'T038',
  'name': 'RS3PE',
  'context': '[CLS] Re ##mit ##ting Ser ##one ##gat ##ive S ##ym ##metric ##al S ##yn ##ov ##itis With Pitt ##ing Ed ##ema : Appearance on F ##D ##G P ##ET / CT Re ##mit ##ting se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting ed ##ema ( [unused1] RS ##3 ##P ##E [unused2] ) is a rare condition in the elderly and can appear as a first presentation of various types of r ##he ##umatic and ma ##li ##gnant diseases . We presented a 62 - year - old man with the diagnosis of RS ##3 ##P ##E based on the clinical sign and laboratory data . Because of the possibility of associated ma ##li ##gna ##ncies in RS ##3 ##P ##E , F ##D [SEP]',
  'existing_cui': 'C3147672',
  'existing_cui_name': 'IRS3P gene',
  'existing_cui_desc': '[CLS] I ##RS ##3 ##P gene [unused3] ( Ana ##tom ##ical Structure : insulin receptor substrate 3 , pseudo ##gene ) [SEP]',
  'our_bi_cui': 'C1696153',
  'our_bi_cui_name': 'Remitting seronegative symmetrical synovitis with pitting oedema syndrome',
  'our_bi_cui_desc': '[CLS] Re ##mit ##ting se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting o ##ede ##ma syndrome [unused3] ( B ##iol ##og ##ic Fun ##ction : Re ##mit ##ting se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting ed ##ema syndrome ; RS ##3 ##P ##E syndrome ) [SEP]',
  'our_cross_cui': 'C2919482',
  'our_cross_cui_name': 'Remitting seronegative symmetrical synovitis with pitting edema',
  'our_cross_cui_desc': '[CLS] Re ##mit ##ting se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting ed ##ema [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : Re ##lap ##sing se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting ed ##ema ; Re ##mit ##ting se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting o ##ede ##ma ; RS ##3 ##P ##E - Re ##mit ##ting se ##rone ##gat ##ive symmetric s ##yn ##ov ##itis with pit ##ting o ##ede ##ma ; Re ##lap ##sing se ##rone ##gat ##ive symmetrical s ##yn ##ov ##itis with pit ##ting o ##ede ##ma ; RS ##3 ##P ##E - Re ##mit ##ting se ##rone ##gat ##ive symmetric s ##yn ##ov ##itis with [SEP]'},
 {'mention_id': '27379139.74',
  'test_set_idx': 4240,
  'type': 'T103',
  'name': 'TCA',
  'context': '[CLS] ranging from - 0 . 02 ##6 to - 0 . 48 ##7 . For all three methods , we were able to obtain high - quality protein samples and good 2D ##E maps for the maize leaf mid ##ri ##b . However , p ##hen ##ol extraction produced a better 2D ##E map with greater resolution between spots , and [unused1] T ##CA [unused2] / ace ##tone extraction produced higher protein yields . Thus , this paper includes a discussion regarding the possible reasons for differential protein extraction among the three methods . This study provides useful information that can be used to select suitable protein extraction methods for the pro ##te ##ome analysis of re ##cal ##ci ##tra ##nt plant tissues that are rich [SEP]',
  'existing_cui': 'C0085758',
  'existing_cui_name': 'Aganglionosis , Colonic',
  'existing_cui_desc': '[CLS] A ##gan ##gli ##ono ##sis , Col ##onic [unused3] ( Ana ##tom ##ical Structure , disorder : A ##gan ##gli ##ono ##sis of co ##lon ; T ##CA - Total co ##lon ##ic a ##gan ##gli ##ono ##sis ; Total co ##lon ##ic a ##gan ##gli ##ono ##sis ; Col ##onic A ##gan ##gli ##ono ##sis ; Total Col ##onic A ##gan ##gli ##ono ##sis ; A ##gan ##gli ##ono ##sis , Total Col ##onic ) [SEP]',
  'our_bi_cui': 'C0045236',
  'our_bi_cui_name': "2,2 ' ,2 '' - trichlorotriethylamine",
  'our_bi_cui_desc': "[CLS] 2 , 2 ' , 2 ' ' - t ##rich ##lor ##ot ##rie ##thy ##lam ##ine [unused3] ( Chemical : t ##ris ( 2 - ch ##lor ##oe ##thy ##l ) am ##ine ; Tri ##mus ##tine ; Tri ##s ( beta - ch ##lor ##oe ##thy ##l ) am ##ine ; Tri ##s ( 2 - ch ##lor ##oe ##thy ##l ) am ##ine ; Tri ##ch ##lor ##met ##hine ; 2 - ch ##lor ##o - N , N - bi ##s ( 2 - ch ##lor ##oe ##thy ##l ) et ##hana ##mine ; 2 , 2 ' , 2 ' ' - t ##rich ##lor ##oe ##thy ##lam ##ine ; t ##rich ##lor ##ot ##rie ##thy ##lam ##ine ; t ##ris [SEP]",
  'our_cross_cui': 'C0040900',
  'our_cross_cui_name': 'Trichloroacetic Acid',
  'our_cross_cui_desc': '[CLS] Tri ##ch ##lor ##oa ##ce ##tic A ##cid [unused3] ( Chemical , medication , product , substance : Tri ##ch ##lor ##oa ##ce ##tic A ##cid [ Chemical / In ##g ##red ##ient ] ; T ##CA - Tri ##ch ##lor ##oa ##ce ##tic acid ; Tri ##ch ##lor ##oa ##ce ##tic acid ; A ##cid ##e t ##rich ##lora ##ce ##ti ##que ; T ##RI ##CH ##L ##OR ##OA ##CE ##TI ##C AC ##ID ; Ace ##tic acid , t ##rich ##lor ##o - ; acid t ##rich ##lor ##oa ##ce ##tic ; t ##rich ##lor ##oa ##ce ##tic acid ; T ##CA ##A - Tri ##ch ##lor ##oa ##ce ##tic acid ; A ##cid , Tri ##ch ##lor ##oa ##ce ##tic ; t ##rich [SEP]'},
 {'mention_id': '28387835.48',
  'test_set_idx': 33984,
  'type': 'T058',
  'name': 'MLVA',
  'context': '[CLS] significantly more IL - 1 ##β , T ##NF - α and IL - 22 than the is ##olate ##s from goats , sheep or chronic Q fever patients . Co ##mp ##aring the c ##yt ##oki ##ne induction of the is ##olate ##s based on their MV ##LA g ##eno ##type did not reveal differences in response between the [unused1] M ##L ##VA [unused2] g ##eno ##type ##s . The pro ##in ##f ##lam ##mat ##ory c ##yt ##oki ##ne response induced in human peripheral blood mon ##on ##uc ##lea ##r cells by C . burn ##eti ##i is ##olate ##s from cattle may explain the low incidence of human Q fever outbreak ##s caused by cattle . The c ##yt ##oki ##ne profile of [SEP]',
  'existing_cui': 'C1420126',
  'existing_cui_name': 'SLC20A2 gene',
  'existing_cui_desc': '[CLS] SL ##C ##20 ##A ##2 gene [unused3] ( Ana ##tom ##ical Structure : S ##OL ##UT ##E CA ##R ##RI ##ER FA ##MI ##L ##Y 20 ( P ##H ##OS ##P ##HA ##TE T ##RA ##NS ##PO ##RT ##ER ) , ME ##MB ##ER 2 ; G ##IB ##BO ##N AP ##E L ##E ##U ##KE ##MI ##A R ##ET ##RO ##VI ##R ##US R ##EC ##EP ##TO ##R 2 ; G ##l ##v ##r - 2 ; P ##H ##OS ##P ##HA ##TE T ##RA ##NS ##PO ##RT ##ER , S ##OD ##I ##UM - DE ##P ##EN ##DE ##NT , 2 ; Pi ##T - 2 ; M ##UR ##IN ##E L ##E ##U ##KE ##MI ##A VI ##R ##US , AM ##P ##H [SEP]',
  'our_bi_cui': 'C0023509',
  'our_bi_cui_name': 'Lymphocyte Culture Test , Mixed',
  'our_bi_cui_desc': '[CLS] L ##ymph ##oc ##yte Culture Test , Mixed [unused3] ( Health Care Act ##ivity , M ##LC , procedure : Mixed Le ##uk ##oc ##yte Re ##actions ; Mixed Le ##uk ##oc ##yte Re ##action ; mixed l ##ymph ##oc ##yte reaction test ; Mixed l ##ymph ##oc ##yte culture ass ##ay ; Le ##uk ##oc ##yte Re ##action , Mixed ; M ##LR test ; mixed le ##uk ##oc ##yte reaction ; Mixed L ##ymph ##oc ##yte Culture Test ; L ##ymph ##oc ##yte Re ##action , Mixed ; Mixed l ##ymph ##oc ##yte culture ; All ##ogen ##ei ##c mixed l ##ymph ##oc ##yte ass ##ay ; Le ##uk ##oc ##yte Re ##actions , Mixed ; Mixed l ##ymph ##oc ##yte reaction ; Mixed [SEP]',
  'our_cross_cui': 'C1443925',
  'our_cross_cui_name': 'Multi-locus variable-number of tandem repeat analysis',
  'our_cross_cui_desc': '[CLS] Multi - lo ##cus variable - number of tandem repeat analysis [unused3] ( Health Care Act ##ivity , procedure : Multi - lo ##cus V ##NT ##R analysis ; M ##L ##VA - multi - lo ##cus variable - number of tandem repeat analysis ) [SEP]'},
 {'mention_id': '27669217.22',
  'test_set_idx': 13673,
  'type': 'T033',
  'name': 'MIC',
  'context': '[CLS] to identify the molecular mechanism underlying the high mac ##rol ##ide resistance of six S . gal ##lo ##ly ##ticus subsp . paste ##uria ##nus is ##olate ##s from dead duck ##lings collected in several natural outbreak ##s in China during 2010 - 2013 . All is ##olate ##s exhibited multi - drug resistance including high mac ##rol ##ide resistance ( [unused1] MI ##C [unused2] ≥ 102 ##4 mg / L for er ##yt ##hr ##omy ##cin , and 512 mg / L for c ##lar ##ith ##rom ##y ##cin ) . E ##ff ##lux - encoding me ##f ##A and me ##f ##E genes were not detect ##able in these is ##olate ##s . The presence of 23 ##S r ##RNA mutations in specific is [SEP]',
  'existing_cui': 'C2732473',
  'existing_cui_name': 'Ductal Carcinoma In Situ with Microinvasion',
  'existing_cui_desc': '[CLS] Du ##ct ##al Car ##cin ##oma In Sit ##u with Micro ##in ##vas ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction , m ##or ##ph ##olo ##gic abnormal ##ity : Micro ##in ##vasive B ##rea ##st Car ##cin ##oma ; MI ##C ; Du ##ct ##al car ##cin ##oma in sit ##u with micro ##in ##vas ##ion ) [SEP]',
  'our_bi_cui': 'C0427978',
  'our_bi_cui_name': 'Minimum Inhibitory Concentration measurement',
  'our_bi_cui_desc': '[CLS] Mini ##mum In ##hibit ##ory Con ##cent ##ration measurement [unused3] ( Health Care Act ##ivity , MI ##C , procedure , qualifier value : Mini ##mum inhibitor ##y concentration su ##s ##ce ##pt ##ibility test ; Con ##cent ##rations , Mini ##mum In ##hibit ##ory ; Anti ##biotic sensitivity , minimum inhibitor ##y concentration ; MI ##C su ##s ##ce ##pt ##ibility test ; Mini ##mum In ##hibit ##ory Con ##cent ##rations ; MI ##C - Mini ##mum inhibitor ##y concentration ; Con ##cent ##ration , Mini ##mum In ##hibit ##ory ; Mini ##mum inhibitor ##y concentration ; Mini ##mum inhibitor ##y concentration su ##s ##ce ##pt ##ibility test technique ; In ##hibit ##ory Con ##cent ##rations , Mini ##mum ; Mini ##mum In ##hibit ##ory [SEP]',
  'our_cross_cui': 'C1304747',
  'our_cross_cui_name': 'Minimum inhibitory concentration result',
  'our_cross_cui_desc': '[CLS] Mini ##mum inhibitor ##y concentration result [unused3] ( Finding , finding : minimum inhibitor ##y concentration ; Mini ##mum inhibitor ##y concentration finding ) [SEP]'},
 {'mention_id': '27563025.29',
  'test_set_idx': 10410,
  'type': 'T017',
  'name': 'C2C12 cells',
  'context': '[CLS] and inter ##connected p ##ore na ##note ##x ##tures with na ##no ##met ##re - scale na ##no ##fi ##bres . To estimate the bio ##com ##pa ##ti ##bility of the 3D p ##oly ##cap ##rol ##act ##one / p ##oly ##ani ##line composite na ##no ##fi ##bre mac ##ros ##truct ##ures , mouse my ##ob ##last ##s ( [unused1] C ##2 ##C ##12 cells [unused2] ) were culture ##d as model cells . The initial responses of C ##2 ##C ##12 cells to the 3D p ##oly ##cap ##rol ##act ##one / p ##oly ##ani ##line composite mac ##ros ##truct ##ures were significantly superior to those to pure p ##oly ##cap ##rol ##act ##one , that is , the cells exhibited typical my ##ob ##last [SEP]',
  'existing_cui': 'C0085262',
  'existing_cui_name': 'PC12 Cells',
  'existing_cui_desc': '[CLS] PC ##12 Cell ##s [unused3] ( Ana ##tom ##ical Structure : Cell Line , Ph ##eo ##ch ##rom ##oc ##yt ##oma ; Ph ##eo ##ch ##rom ##oc ##yt ##oma Cell Lines ; Ph ##eo ##ch ##rom ##oc ##yt ##oma 12 Cell Line ; PC A 01 ##2 CE ##LL ##S ; p ##he ##och ##rom ##oc ##yt ##oma 12 cell line ; PC - 12 ; p ##c ##12 cell ; PC ##12 Cell ; Ph ##eo ##ch ##rom ##oc ##yt ##oma Cell Line ; Cell Lines , Ph ##eo ##ch ##rom ##oc ##yt ##oma ; PC ##12 cell ; p ##c ##12 cells ) [SEP]',
  'our_bi_cui': 'C1135917',
  'our_bi_cui_name': 'Myoblasts , Cardiac',
  'our_bi_cui_desc': '[CLS] My ##ob ##last ##s , Card ##iac [unused3] ( Ana ##tom ##ical Structure : cardiac my ##ob ##last ##s ; My ##ob ##last , Card ##iac ; cardiac my ##ob ##last ; Card ##iac my ##ob ##last ; Card ##iac My ##ob ##last ##s ; Card ##iac My ##ob ##last ) [SEP]',
  'our_cross_cui': 'C0596995',
  'our_cross_cui_name': 'Myoblasts',
  'our_cross_cui_desc': '[CLS] My ##ob ##last ##s [unused3] ( Ana ##tom ##ical Structure : Mu ##s ##cle Cell ##s , Em ##b ##ryo ##nic ; My ##ob ##last ; Mu ##s ##cle Cell , Em ##b ##ryo ##nic ; Pre ##cu ##rs ##or Mu ##s ##cle Cell ##s ; Mu ##s ##cle Cell , Pre ##cu ##rs ##or ; my ##ob ##last ; Em ##b ##ryo ##nic Mu ##s ##cle Cell ##s ; my ##ob ##last ##s ; Pre ##cu ##rs ##or Mu ##s ##cle Cell ; Em ##b ##ryo ##nic Mu ##s ##cle Cell ; Mu ##s ##cle Cell ##s , Pre ##cu ##rs ##or ) [SEP]'},
 {'mention_id': '27838742.76',
  'test_set_idx': 19043,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] in v ##it ##ro and in v ##ivo . Del ##ica ##f ##lav ##one induced auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . [unused1] Del ##ica ##f ##lav ##one [unused2] may represent a potential therapeutic agent for lung cancer . Del ##ica ##f ##lav ##one showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . Del ##ica ##f ##lav ##one induced auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28197133.38',
  'test_set_idx': 28054,
  'type': 'T103',
  'name': 'GH3',
  'context': '[CLS] p ##hos ##ph ##ory ##lase ) , G ##H1 ##3 ( am ##yla ##se ) , G ##H ##43 and G ##H1 ##0 ( hem ##ice ##ll ##ula ##ses ) , G ##H ##9 and G ##H ##48 ( cell ##ula ##ses ) , P ##L ##11 ( p ##ec ##tina ##se ) as well as G ##H ##2 and [unused1] G ##H ##3 [unused2] ( o ##li ##gos ##ac ##cha ##rida ##ses ) . Our data support the pivotal role of the most characterized fi ##bro ##ly ##tic bacteria ( Pre ##vo ##tel ##la , R ##umi ##no ##cc ##cus and Fi ##bro ##ba ##cter ) , and highlight a substantial , although most probably under ##est ##imated , contribution of fungi and c ##ilia [SEP]',
  'existing_cui': 'C1333673',
  'existing_cui_name': 'GH1 gene',
  'existing_cui_desc': '[CLS] G ##H1 gene [unused3] ( Ana ##tom ##ical Structure : G ##RO ##W ##TH H ##OR ##MO ##NE , NO ##R ##MA ##L ; G ##RO ##W ##TH H ##OR ##MO ##NE 1 ; G ##H1 Gene ; growth hormone 1 ; so ##mat ##ot ##rop ##in ; pit ##uit ##ary growth hormone ; G ##RO ##W ##TH H ##OR ##MO ##NE , P ##IT ##UI ##TA ##R ##Y ; h ##G ##H - N ; G ##H - N ; Growth Ho ##rmon ##e 1 Gene ) [SEP]',
  'our_bi_cui': 'C1333877',
  'our_bi_cui_name': 'Granzymes',
  'our_bi_cui_desc': '[CLS] Gran ##zy ##mes [unused3] ( Chemical : Gran ##zy ##me ; g ##ran ##zy ##me ; EC 3 . 4 . 21 . - ; g ##ran ##zy ##mes ; Gran ##ule - Secret ##ed En ##zy ##me ; Gran ##zy ##mes [ Chemical / In ##g ##red ##ient ] ) [SEP]',
  'our_cross_cui': 'C0014442',
  'our_cross_cui_name': 'Enzymes',
  'our_cross_cui_desc': '[CLS] En ##zy ##mes [unused3] ( Chemical , medication , substance : enzymes ; enzyme ; En ##zy ##mes [ Chemical / In ##g ##red ##ient ] ; B ##io ##cat ##aly ##sts ; En ##zy ##me ) [SEP]'},
 {'mention_id': '27623277.36',
  'test_set_idx': 12220,
  'type': 'T058',
  'name': 'Procedural',
  'context': '[CLS] attributes from the con ##join ##t experiment ( each with a relative weight of 0 . 23 ) , followed by provider volume ( relative weight of 0 . 21 ) , risk of major complications ( relative weight of 0 . 18 ) , and distance needed to travel for treatment ( relative weight of 0 . 15 ) . [unused1] Pro ##cedural [unused2] and treatment preferences did not vary with demographic ##s , self - reported health status , or familiar ##ity with the procedures . Survey respond ##ents preferred minimal ##ly invasive operations over stereo ##ta ##ctic body radiation therapy or th ##ora ##cot ##omy for treatment of early - stage non - small cell lung cancer . Treatment m ##oda ##lity and [SEP]',
  'existing_cui': 'C1619712',
  'existing_cui_name': 'Procedural pain',
  'existing_cui_desc': '[CLS] Pro ##cedural pain [unused3] ( Finding ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27685187.20',
  'test_set_idx': 14117,
  'type': 'T082',
  'name': 'Canadian city',
  'context': "[CLS] determine what are considered acceptable standards for animal research methodology and translation rate to humans , a valid ##ated survey was sent to : a ) a sample of the general public , via Sam ##pling Survey International ( Sam ##pling Survey International ; Canada ) , Amazon Mechanical Tu ##rk ( Amazon Mechanical Tu ##rk ; USA ) , a [unused1] Canadian city [unused2] festival and a Canadian children ' s hospital ; b ) a sample of medical students ( two first - year classes ) ; and c ) a sample of scientists ( corresponding authors and academic p ##ae ##dia ##tric ##ians ) . There were 137 ##9 responses from the general public sample ( Sam ##pling Survey International , n = [SEP]",
  'existing_cui': 'C0324053',
  'existing_cui_name': 'Canadian cattle',
  'existing_cui_desc': '[CLS] Canadian cattle [unused3] ( E ##uka ##ryo ##te , organism : Can ##adi ##enne cow ; Canadian cow ; Can ##adi ##enne Cat ##tle ; Canadian cattle breed ; Can ##adi ##enne cattle breed ) [SEP]',
  'our_bi_cui': 'C0008848',
  'our_bi_cui_name': 'Cities',
  'our_bi_cui_desc': '[CLS] Cities [unused3] ( Spa ##tial Concept , environment : City ; city ; cities ) [SEP]',
  'our_cross_cui': 'C0006823',
  'our_cross_cui_name': 'Canada',
  'our_cross_cui_desc': '[CLS] Canada [unused3] ( Spa ##tial Concept , geographic location : CA ##N ; CA ##NA ##DA ; CA ) [SEP]'},
 {'mention_id': '28394935.43',
  'test_set_idx': 34414,
  'type': 'T103',
  'name': 'Cys - NO',
  'context': '[CLS] ##tric oxide s ##ynth ##ase - derived ni ##tric oxide S - ni ##tro ##sy ##lated β - act ##in on Cy ##s ##37 ##4 and impaired act ##in binding to pro ##fi ##lin - 1 , as confirmed with the trans ##ni ##tro ##sy ##lating agent S - ni ##tro ##so - L - c ##ys ##tein ##e ( [unused1] Cy ##s - NO [unused2] ) . The importance of ni ##tric oxide and the formation of P ##F ##N ##1 - act ##in complexes on the regulation of protein kinase C - θ was co ##rro ##bor ##ated by over ##ex ##press ##ion of pro ##fi ##lin - 1 - and act ##in - binding defect ##ive mutant ##s of β - act ##in [SEP]',
  'existing_cui': 'C3252049',
  'existing_cui_name': 'cystine-N-carboxy anhydride',
  'existing_cui_desc': '[CLS] c ##ys ##tine - N - car ##box ##y an ##hy ##dr ##ide [unused3] ( Chemical : Cy ##s - NC ##A ) [SEP]',
  'our_bi_cui': 'C0010654',
  'our_bi_cui_name': 'Cysteine',
  'our_bi_cui_desc': '[CLS] Cy ##stein ##e [unused3] ( Chemical , product , substance : ( 2 ##R ) - 2 - amino - 3 - me ##rca ##pt ##op ##rop ##ano ##ic acid ; c ##ys ##tein ##e l ; ( R ) - 2 - Am ##ino - 3 - su ##lf ##any ##l - prop ##ano ##ic acid ; Half Cy ##st ##ine ; l c ##ys ##tein ##e ; L - Cy ##stein ; Cy ##s ; C ##Y ##ST ##EI ##NE ; l - c ##ys ##tein ##e ; Half - Cy ##st ##ine ; L - Cy ##stein ##e ; ( 2 ##R ) - 2 - amino - 3 - su ##lf ##any ##l ##p ##rop ##ano ##ic acid ; L Cy ##stein [SEP]',
  'our_cross_cui': 'C0073896',
  'our_cross_cui_name': 'S-nitrosocysteine',
  'our_cross_cui_desc': '[CLS] S - ni ##tro ##so ##cy ##stein ##e [unused3] ( Chemical : L - S - ni ##tro ##so ##cy ##stein ##e ; S ##NC compound ; S - ni ##tro ##so - L - c ##ys ##tein ##e ; L - S ##NC compound ) [SEP]'},
 {'mention_id': '27334669.21',
  'test_set_idx': 2719,
  'type': 'T033',
  'name': 'detection',
  'context': '[CLS] DL ##S data in a thorough manner by providing a large set of fitting parameters . This showed that DL ##S only sensed the larger ve ##si ##cles when the mice ##lles / ve ##si ##cles ratio was 80 / 20 w / w . On the other hand , as ##ym ##metric ##al flow field flow fraction ##ation allowed clear [unused1] detection [unused2] of the presence of mice ##lles when this same ratio was as low as 10 / 90 . Subsequently , the photo ##dynamic therapy efficiency of various controlled mixture ##s was assessed using multi ##cellular s ##pher ##oids when a photos ##ens ##iti ##zer , p ##he ##op ##hor ##bide a , was en ##cap ##sul ##ated in the polymer self - [SEP]',
  'existing_cui': 'C0206100',
  'existing_cui_name': 'Signal Detection',
  'existing_cui_desc': '[CLS] Signal Det ##ec ##tion [unused3] ( B ##iol ##og ##ic Fun ##ction , Psychology , Psychology , Signal : Signal Det ##ec ##tion Theo ##ries ; Signal Det ##ec ##tions , Psychological ; Theory , Signal Det ##ec ##tion ; Signal detection ; S ##IG ##NA ##L DE ##TE ##CT ##ION AN ##AL ; Psychological Signal Det ##ec ##tions ; Det ##ec ##tion ; Signal Det ##ec ##tion Ana ##ly ##ses ; S ##IG ##NA ##L DE ##TE ##CT ##ION ; Det ##ec ##tions , Psychological Signal ; Psychological Signal Det ##ec ##tion ; Signal Det ##ec ##tion Theory ; Theo ##ries , Signal Det ##ec ##tion ; Signal Det ##ec ##tions ; Ana ##ly ##ses , Signal Det ##ec ##tion ; Det ##ec ##tion , [SEP]',
  'our_bi_cui': 'C1511790',
  'our_bi_cui_name': 'Detection',
  'our_bi_cui_desc': '[CLS] Det ##ec ##tion [unused3] ( Health Care Act ##ivity : Det ##ec ##ted ) [SEP]',
  'our_cross_cui': 'C0442726',
  'our_cross_cui_name': 'Detected',
  'our_cross_cui_desc': '[CLS] Det ##ec ##ted [unused3] ( Finding , finding , finding , qualifier value : detected ; detect ) [SEP]'},
 {'mention_id': '25847295.20',
  'test_set_idx': 19,
  'type': 'T017',
  'name': 'apoptotic cells',
  'context': '[CLS] ##late decreased DNA fragment ##ation caused by a ##pop ##tosis in PC ##12 cells . This phenomenon was confirmed after treating a ##pop ##to ##tic PC ##12 cells with non ##yl ##phe ##no ##l diet ##ho ##xy ##late , whereas the c ##yt ##och ##rome c release into the c ##yt ##oso ##l decreased as compared to that in [unused1] a ##pop ##to ##tic cells [unused2] not treated with non ##yl ##phe ##no ##l diet ##ho ##xy ##late s . Furthermore , Ba ##x contents in a ##pop ##to ##tic cells were reduced after exposure to non ##yl ##phe ##no ##l diet ##ho ##xy ##late . Thus , non ##yl ##phe ##no ##l diet ##ho ##xy ##late has the opposite effect on a ##pop ##tosis in [SEP]',
  'existing_cui': 'C0162638',
  'existing_cui_name': 'Apoptosis',
  'existing_cui_desc': '[CLS] A ##pop ##tosis [unused3] ( B ##iol ##og ##ic Fun ##ction , m ##or ##ph ##olo ##gic abnormal ##ity : a ##pop ##to ##tic cell death ; A ##pop ##to ##tic Process ; Programme ##d Cell Death ; programmed cell death by a ##pop ##tosis ; A ##pop ##tosis Path ##way ; a ##pop ##tos ##e ; programmed cell death ; RC ##D ; a ##pop ##tosis ; Gene - directed cell death ; a ##pop ##to ##tic programmed cell death ; Programme ##d Cell Death , Type I ; self destruction of the cell ; Me ##chan ##ism that allows cells to self - des ##truct when it is necessary ; a ##pop ##to ##tic process ; type I programmed cell death ) [SEP]',
  'our_bi_cui': 'C1836846',
  'our_bi_cui_name': 'Apoptotic neurons',
  'our_bi_cui_desc': '[CLS] A ##pop ##to ##tic neurons [unused3] ( Finding ) [SEP]',
  'our_cross_cui': 'C0007634',
  'our_cross_cui_name': 'Cells',
  'our_cross_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]'},
 {'mention_id': '27298278.20',
  'test_set_idx': 1821,
  'type': 'T103',
  'name': 'morindaparvins C',
  'context': '[CLS] ##ones , a nap ##ht ##ho ##quin ##one , an ant ##hra ##quin ##one , and a nap ##ht ##ho ##hy ##dr ##o ##quin ##one dim ##er , together with three known q ##uin ##ones and seven other known compounds , were isolated from the aerial parts of Mo ##rin ##da par ##vi ##folia . The structures of [unused1] m ##ori ##nda ##par ##vin ##s C [unused2] , D , E , F , and G were el ##uc ##idated on the basis of s ##pect ##ros ##copic or X - ray di ##ff ##raction analysis as met ##hyl 4 - h ##ydro ##xy - 1 , 6 - dim ##eth ##ox ##y - nap ##ht ##hale ##ne - 2 - car ##box ##yla ##te , [SEP]',
  'existing_cui': 'C0616488',
  'existing_cui_name': 'morindaparvin A',
  'existing_cui_desc': '[CLS] m ##ori ##nda ##par ##vin A [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0010206',
  'our_bi_cui_name': 'coumarin',
  'our_bi_cui_desc': '[CLS] co ##uma ##rin [unused3] ( Chemical , product , substance : 1 , 2 - Benz ##opy ##rone ; C ##uma ##rin ; 5 , 6 - ben ##zo - alpha - p ##yr ##one ; alpha ben ##zo ##py ##rone ; 1 , 2 - ben ##zo ##py ##rone ; ton ##ka bean camp ##hor ; To ##nka Bean Camp ##hor ; Ch ##rome ##n - 2 - one ; CO ##UM ##AR ##IN ; Co ##uma ##rin anti ##coa ##gu ##lant ; 2 ##H - 1 - Benz ##opy ##ran - 2 - one ; Co ##uma ##rini ##c an ##hy ##dr ##ide ; To ##nka bean camp ##hor ; Co ##uma ##rin ; cum ##ari ##n ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27261783.7',
  'test_set_idx': 1085,
  'type': 'T033',
  'name': 'tracing',
  'context': '[CLS] micro ##bial cells , micro ##bial populations in ultra ##oli ##got ##rop ##hic a ##ero ##bic sediment ##ary habitats in the open ocean g ##yres are several orders of magnitude less abundant . Despite advances in cultivation - independent molecular ecological techniques , exploring the low - bio ##mass environment remains technological ##ly challenging , especially in the deep sub ##sea ##f ##lo ##or bio ##sp ##here . Review ##ing the historical background of deep - bio ##sp ##here analytical methods , the importance of obtaining clean samples and [unused1] tracing [unused2] contamination , as well as methods for detect ##ing micro ##bial life , technological aspects of molecular micro ##biology , and detect ##ing sub ##sea ##f ##lo ##or metabolic activity will be discussed . [SEP]',
  'existing_cui': 'C0079161',
  'existing_cui_name': 'Infectious Disease Contact Tracing',
  'existing_cui_desc': '[CLS] In ##fect ##ious Disease Contact T ##rac ##ing [unused3] ( Health Care Act ##ivity , procedure : CT - Contact tracing ; Contact T ##rac ##ing ; CO ##MM ##UN ##IC ##AB ##LE D ##IS CO ##NT ##AC ##T T ##RA ##CI ##NG ; Contact tracing ; IN ##F ##EC ##T D ##IS CO ##NT ##AC ##T T ##RA ##CI ##NG ; Co ##mm ##uni ##ca ##ble Disease Contact T ##rac ##ing ; T ##rac ##ing , Contact ) [SEP]',
  'our_bi_cui': 'C0546881',
  'our_bi_cui_name': 'tracking',
  'our_bi_cui_desc': '[CLS] tracking [unused3] ( Spa ##tial Concept : Track ##ing ; tracking [ mental function ] ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27671177.13',
  'test_set_idx': 13790,
  'type': 'T082',
  'name': 'Quetta',
  'context': '[CLS] in Que ##tta , Pakistan To determine the prevalence of dental car ##ies and its relationship with dental flu ##oro ##sis , oral health behaviour and diet ##ary behaviour among 12 - year school children in moderate - flu ##ori ##de drinking water community in Que ##tta , Pakistan . Cross - section ##al study . Government and private schools of [unused1] Que ##tta [unused2] , from November 2012 to February 2013 . At ##ota ##l of 34 ##9 children aged 12 - year from 14 randomly selected schools were included . The data collection was done on question ##naire designed for children . Dental car ##ies status was examined by using WHO criteria . Dental car ##ies was found in 81 children ( 23 . [SEP]',
  'existing_cui': 'C0262345',
  'existing_cui_name': 'Tenia of thalamus',
  'existing_cui_desc': '[CLS] Ten ##ia of th ##ala ##mus [unused3] ( Ana ##tom ##ical Structure : ten ##ia of th ##ala ##mi ; Ten ##ia of th ##ala ##mi ; Ta ##enia th ##ala ##mi ; T ##hala ##mus ten ##ia ; ten ##ia of the th ##ala ##mus ; Ten ##ia th ##ala ##mi ) [SEP]',
  'our_bi_cui': 'C0337884',
  'our_bi_cui_name': 'Quechua',
  'our_bi_cui_desc': '[CLS] Que ##chu ##a [unused3] ( Population Group , ethnic group , ethnic group ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '27400734.18',
  'test_set_idx': 4722,
  'type': 'T103',
  'name': 'GEM',
  'context': '[CLS] analysis of Korean national database A random ##ized clinical trial has found that the addition of er ##lot ##ini ##b to g ##em ##ci ##ta ##bine ( GE ##M - E ) for pan ##cre ##atic cancer led to a modest increase in survival . The aim of this national population - based retrospective study was to compare the effectiveness of [unused1] GE ##M [unused2] - E to GE ##M alone for pan ##cre ##atic cancer patients in real clinical practice . Pat ##ients with pan ##cre ##atic cancer ( I ##CD - 10 : C ##25 ) with prescription claims of g ##em ##ci ##ta ##bine or er ##lot ##ini ##b between Jan 1 , 2007 and Dec 31 , 2012 were retrospective ##ly identified [SEP]',
  'existing_cui': 'C1333683',
  'existing_cui_name': 'GMNN gene',
  'existing_cui_desc': '[CLS] GM ##N ##N gene [unused3] ( Ana ##tom ##ical Structure : Gemini ##n , DNA Rep ##lication In ##hibit ##or Gene ; GM ##N ##N Gene ; G ##em ; GE ##MI ##NI ##N ; g ##em ##ini ##n , DNA replication inhibitor ; GE ##M ) [SEP]',
  'our_bi_cui': 'C2955445',
  'our_bi_cui_name': 'GEL',
  'our_bi_cui_desc': '[CLS] GE ##L [unused3] ( Chemical , G ##RA ##M , G ##RA ##M ) [SEP]',
  'our_cross_cui': 'C0045093',
  'our_cross_cui_name': 'gemcitabine',
  'our_cross_cui_desc': "[CLS] g ##em ##ci ##ta ##bine [unused3] ( Chemical , product , substance : g ##em ##ci ##ta ##bine [ Chemical / In ##g ##red ##ient ] ; 2 ' , 2 ' - di ##f ##lu ##oro ##de ##ox ##y ##cy ##ti ##dine ; 4 - amino - 1 - ( ( 2 ##R , 4 ##R , 5 ##R ) - 3 , 3 - Di ##f ##lu ##oro - 4 - h ##ydro ##xy - 5 - ( h ##ydro ##xy ##met ##hyl ) - te ##tra ##hy ##dr ##of ##ura ##n - 2 - y ##l ) p ##yr ##im ##id ##in - 2 ( 1 ##H ) - one ; 2 ' , 2 ' - Di ##f ##lu ##oro ##de ##ox [SEP]"},
 {'mention_id': '27500205.28',
  'test_set_idx': 8460,
  'type': 'T103',
  'name': 'kindlins',
  'context': '[CLS] transformed cells often exhibit a ##ber ##rant expression , both in the is ##of ##orms and the levels of kind ##lins . Great interest in the kind ##lins has emerged from the recognition that they play major roles in controlling in ##te ##g ##rin function . In v ##it ##ro studies , in v ##ivo studies of mice def ##icient in [unused1] kind ##lins [unused2] , and studies of patients with genetic def ##ici ##encies of kind ##lins have clearly established that they regulate the capacity of in ##te ##g ##rin ##s to media ##te their functions . Kind ##lins are adapt ##or proteins ; their function em ##ana ##te from their interaction with binding partners , including the c ##yt ##op ##las ##mic tails of [SEP]',
  'existing_cui': 'C1720776',
  'existing_cui_name': 'Kindling , Neurologic',
  'existing_cui_desc': '[CLS] Kind ##ling , N ##eur ##olo ##gic [unused3] ( B ##iol ##og ##ic Fun ##ction , N ##eur ##ology : kind ##ling ; Kind ##lings , N ##eur ##olo ##gic ; Kind ##ling ; N ##eur ##olo ##gic Kind ##lings ; N ##eur ##olo ##gic Kind ##ling ) [SEP]',
  'our_bi_cui': 'C0010422',
  'our_bi_cui_name': 'Crystallins',
  'our_bi_cui_desc': '[CLS] Crystal ##lins [unused3] ( Chemical , substance : Crystal ##lins [ Chemical / In ##g ##red ##ient ] ; Crystal ##lin ; Pro ##tein ##s , Len ##s ; C ##rist ##all ##in ; Len ##s Pro ##tein ##s ; crystal ##lin ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27545500.27',
  'test_set_idx': 10003,
  'type': 'T103',
  'name': '16-aldehyde tanshinone I',
  'context': '[CLS] I and amino ##gly ##cos ##ide anti ##biotics or β - la ##ct ##am anti ##biotics in v ##it ##ro was identified . Moreover , 16 - al ##de ##hy ##de tan ##shin ##one I was documented to au ##gment clearance of s ##tre ##pt ##omy ##cin and am ##pic ##ill ##in against S . au ##reus in a m ##uri ##ne infection model . Primary me ##chan ##istic insight indicated that [unused1] 16 - al ##de ##hy ##de tan ##shin ##one I [unused2] could damage the bacterial cell membrane , leading to accumulation of anti ##biotics inside bacterial cells . This finding might be useful for treating infections caused by S . au ##reus and expand the scope of application of tan ##shin ##ones . [SEP]',
  'existing_cui': 'C0075810',
  'existing_cui_name': 'tanshinone',
  'existing_cui_desc': '[CLS] tan ##shin ##one [unused3] ( Chemical : 1 , 6 - dim ##eth ##yl ##phe ##nant ##hr ##o ( 1 , 2 - b ) fur ##an - 10 , 11 - di ##one ; tan ##shin ##one I ) [SEP]',
  'our_bi_cui': 'C0075810',
  'our_bi_cui_name': 'tanshinone',
  'our_bi_cui_desc': '[CLS] tan ##shin ##one [unused3] ( Chemical : 1 , 6 - dim ##eth ##yl ##phe ##nant ##hr ##o ( 1 , 2 - b ) fur ##an - 10 , 11 - di ##one ; tan ##shin ##one I ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28168514.17',
  'test_set_idx': 27532,
  'type': 'T058',
  'name': 'sequential',
  'context': '[CLS] sixty - eight consecutive patients ( age : 67 ± 10 years ; B ##MI : 27 ± 5 kg / m ² ; 61 % male ) undergoing clinical ##ly indicated CT an ##gio ##graphy with dual - source CT were included in the retrospective single - center analysis . A contrast - enhanced volume data ##set was acquired in [unused1] se ##quential [unused2] ( SS ##M ) ( n = 151 ) or he ##lica ##l scan mode ( n = 117 ) . Co ##rona ##ry segments were classified in diagnostic or non - diagnostic image quality . A subset underwent invasive an ##gio ##graphy to determine the diagnostic accuracy of CT an ##gio ##graphy . SS ##M ( 96 . 8 ± [SEP]',
  'existing_cui': 'C1337600',
  'existing_cui_name': 'TABLET , SEQUENTIAL',
  'existing_cui_desc': '[CLS] T ##AB ##LE ##T , SE ##Q ##UE ##NT ##IA ##L [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0202607',
  'our_bi_cui_name': 'Serialography , single plane',
  'our_bi_cui_desc': '[CLS] Ser ##ial ##ography , single plane [unused3] ( Health Care Act ##ivity , procedure ) [SEP]',
  'our_cross_cui': 'C0441633',
  'our_cross_cui_name': 'Scanning',
  'our_cross_cui_desc': '[CLS] Sc ##ann ##ing [unused3] ( Health Care Act ##ivity , procedure : scan ##s ; scanned ; scan ; scanning ; Sc ##ans ; diagnostic scanning ; diagnostic scan ; Sc ##an ) [SEP]'},
 {'mention_id': '28127340.1',
  'test_set_idx': 26227,
  'type': 'T033',
  'name': 'Non - zoonotic',
  'context': '[CLS] Zoo ##not ##ic and [unused1] Non - zoo ##not ##ic [unused2] Para ##site ##s of Wild Rod ##ents in Tu ##rk ##man Sa ##hra , Northeastern Iran This study was conducted to collect inform ##ative data on the para ##sitic infection of wild rode ##nts , emphasizing on finding parasite ##s , which have medical importance to human . During 2012 - 2014 , a total number of 91 wild rode ##nts were captured from rural areas of Tu ##rk ##men Sa ##hra , Go ##les ##tan Province , using hand ##made traps . Animals were an ##est ##he ##tized , surveyed for any e ##ct ##op ##ara ##site and then their car ##cas ##ses were carefully di ##sse ##cted for examination of end ##op ##ars [SEP]',
  'existing_cui': 'C0043528',
  'existing_cui_name': 'Zoonoses',
  'existing_cui_desc': '[CLS] Zoo ##nose ##s [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder , navigation ##al concept : Zoo ##nosis ; Zoo ##not ##ic In ##fect ##ious Disease ; Zoo ##not ##ic infection ; Disease , Zoo ##not ##ic In ##fect ##ious ; Zoo ##nose ##s [ Disease / Finding ] ; Zoo ##not ##ic Disease ##s ; infections zoo ##not ##ic ; Zoo ##not ##ic In ##fect ##ion ; Zoo ##not ##ic Disease ; In ##fect ##ious Disease , Zoo ##not ##ic ; In ##fect ##ions , Zoo ##not ##ic ; Zoo ##not ##ic In ##fect ##ious Disease ##s ; Disease , Zoo ##not ##ic ; Disease ##s , Zoo ##not ##ic In ##fect ##ious ; zoo ##nose ; zoo ##not ##ic infection ; In [SEP]',
  'our_bi_cui': 'C0043528',
  'our_bi_cui_name': 'Zoonoses',
  'our_bi_cui_desc': '[CLS] Zoo ##nose ##s [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder , navigation ##al concept : Zoo ##nosis ; Zoo ##not ##ic In ##fect ##ious Disease ; Zoo ##not ##ic infection ; Disease , Zoo ##not ##ic In ##fect ##ious ; Zoo ##nose ##s [ Disease / Finding ] ; Zoo ##not ##ic Disease ##s ; infections zoo ##not ##ic ; Zoo ##not ##ic In ##fect ##ion ; Zoo ##not ##ic Disease ; In ##fect ##ious Disease , Zoo ##not ##ic ; In ##fect ##ions , Zoo ##not ##ic ; Zoo ##not ##ic In ##fect ##ious Disease ##s ; Disease , Zoo ##not ##ic ; Disease ##s , Zoo ##not ##ic In ##fect ##ious ; zoo ##nose ; zoo ##not ##ic infection ; In [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27301681.4',
  'test_set_idx': 1904,
  'type': 'T103',
  'name': 'N-methylpyrrole ( Py ) - N-methylimidazole ( Im ) polyamides',
  'context': '[CLS] p ##yr ##rol ##e - im ##ida ##zo ##le p ##oly ##ami ##des containing an 8 - amino - 3 , 6 - di ##ox ##ao ##ct ##ano ##ic acid unit With the aim of improving a ##que ##ous so ##lub ##ility , we designed and synthesized five [unused1] N - met ##hyl ##py ##rro ##le ( P ##y ) - N - met ##hyl ##im ##ida ##zo ##le ( I ##m ) p ##oly ##ami ##des [unused2] capable of recognizing 9 - b ##p sequences . Their DNA - binding a ##ffin ##ities and sequence specific ##ities were evaluated by SP ##R and Bin ##d - n - Se ##q analyses . The design of p ##oly ##ami ##de 1 was based on a conventional [SEP]',
  'existing_cui': 'C0764095',
  'existing_cui_name': 'N-methylimidazole-N-methylimidazole-N-methylpyrrole-aminobutyric acid - N-methylpyrrole-N-methylpyrrole-N-methylpyrrole-alanine-dimethylaminopropylamide',
  'existing_cui_desc': '[CLS] N - met ##hyl ##im ##ida ##zo ##le - N - met ##hyl ##im ##ida ##zo ##le - N - met ##hyl ##py ##rro ##le - amino ##but ##yric acid - N - met ##hyl ##py ##rro ##le - N - met ##hyl ##py ##rro ##le - N - met ##hyl ##py ##rro ##le - al ##ani ##ne - dim ##eth ##yla ##min ##op ##rop ##yla ##mi ##de [unused3] ( Chemical : 2 ##I ##m - P ##y - ABA - 3 ##P ##y - Al ##a - D ##p ; I ##m ##I ##m ##P ##y - gamma - amino ##but ##yric acid - P ##y ##P ##y ##P ##y - beta - al ##ani ##ne - D ##p ) [SEP]',
  'our_bi_cui': 'C1254351',
  'our_bi_cui_name': 'Pharmacologic Substance',
  'our_bi_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]',
  'our_cross_cui': 'C0029224',
  'our_cross_cui_name': 'Organic Chemicals',
  'our_cross_cui_desc': '[CLS] Organic Chemical ##s [unused3] ( Chemical , substance : organic chemicals ; compound organic ; Organic Chemical ##s [ Chemical / In ##g ##red ##ient ] ; organic compound ; Chemical ##s , Organic ; organic chemical ; Organic Chemical ; chemical organic ; organic compounds ; Organic compound ; Organic compounds ; Organic chemical ) [SEP]'},
 {'mention_id': '27659602.46',
  'test_set_idx': 13391,
  'type': 'T082',
  'name': 'secondary root',
  'context': '[CLS] more are at risk for re ##oper ##ation , and not until at least after 10 . 4 years have passed . Neither the valve morphology ( p = 0 . 62 ) nor con ##com ##ita ##nt a ##ort ##ic valve replacement ( p = 0 . 86 ) influenced rate of root di ##lation . In non ##sy ##nd ##rom ##ic patients , the a ##ort ##ic root is the slow ##est growing portion of the th ##ora ##ci ##c a ##ort ##a . Leaving the native root , as in root - spa ##ring ascending a ##ort ##ic replacement , is a safe approach regarding [unused1] secondary root [unused2] intervention for a ##ort ##ic root diameter ##s of 45 mm or less . [SEP]',
  'existing_cui': 'C0027627',
  'existing_cui_name': 'Neoplasm Metastasis',
  'existing_cui_desc': '[CLS] Neo ##p ##las ##m Met ##ast ##asis [unused3] ( B ##iol ##og ##ic Fun ##ction , tumor : Tu ##mor Cell Mi ##gration ; meta ##sta ##si ##zed ; cancer ##s secondary ; NE ##OP ##L ME ##TA ##ST ##AS ##ES ; secondary ; meta ##static t ##umour ; meta ##static tumors ; meta ##static cancer ; meta ##static disease ; secondary cancer ; meta ##static tumor ; Met ##ast ##asis ; secondary tumors ; Neo ##p ##las ##m Met ##ast ##ases ; Neo ##p ##las ##m Met ##ast ##asis [ Disease / Finding ] ; t ##umour meta ##sta ##sis ; spreading of cancer ; tumor meta ##sta ##sis ; meta ##sta ##ses ; meta ##sta ##sis ; meta ##static t ##umour ##s ; secondary [SEP]',
  'our_bi_cui': 'C0447373',
  'our_bi_cui_name': 'Distal tooth root',
  'our_bi_cui_desc': '[CLS] Di ##stal tooth root [unused3] ( Ana ##tom ##ical Structure , Den ##tis , body structure : Di ##stal root of tooth ; Di ##stal root ; Ra ##di ##x di ##stal ##is ) [SEP]',
  'our_cross_cui': 'C0549113',
  'our_cross_cui_name': 'Supraaortic valve area structure',
  'our_cross_cui_desc': '[CLS] Su ##pra ##ao ##rt ##ic valve area structure [unused3] ( Spa ##tial Concept , body structure : A ##ort ##ic bulb ; AS ##CE ##ND ##ING A ##OR ##TA , A ##OR ##TI ##C R ##O ##OT ; A ##ort ##a . root ; A ##ort ##ic root ; a ##ort ##ic root ; A ##ort ##ic Root ; Root of a ##ort ##a ; B ##ul ##b of a ##ort ##a ; Su ##pra ##ao ##rt ##ic valve area ; root a ##ort ##ic ; B ##ul ##b of ascending a ##ort ##a ; B ##ul ##bus a ##ort ##ae ) [SEP]'},
 {'mention_id': '27790175.20',
  'test_set_idx': 16824,
  'type': 'T038',
  'name': 'production',
  'context': '[CLS] sessions with two discrete toy sets were double - coded by blinded code ##rs ( code ##r assignment counter ##bal ##ance ##d ) . Key face ##ts of pre ##tens ##e development [ at ##tribution of pretend properties ( at ##tribution of pretend properties ) , object substitution , imaginary objects ] were examined . These face ##ts of pretend play [unused1] production [unused2] were then analyzed in relation to autism spectrum disorder symptoms , as well as plausible , long - the ##ori ##zed co ##rrel ##ates [ theory of mind ( theory of mind ) , verbal ability , familiar ##ity , and interest in specific toys ] . Forty children ( Ma ##ge = 6 ; 5 , SD ##age = 1 . [SEP]',
  'existing_cui': 'C1548180',
  'existing_cui_name': 'Production Processing ID',
  'existing_cui_desc': '[CLS] Production Process ##ing ID [unused3] ( Intel ##lect ##ual Product : Production ) [SEP]',
  'our_bi_cui': 'C0220781',
  'our_bi_cui_name': 'Anabolism',
  'our_bi_cui_desc': '[CLS] Ana ##bol ##ism [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : bio ##sy ##nt ##hetic process ; Ana ##bol ##ism , function ; synthesis ; bio ##sy ##nt ##hesis ; formation ; B ##ios ##ynth ##etic ; B ##ios ##ynth ##esis ; B ##io ##genesis ; B ##ios ##ynth ##etic Process ; an ##ab ##olis ##m ) [SEP]',
  'our_cross_cui': 'C0025361',
  'our_cross_cui_name': 'Mental Processes',
  'our_cross_cui_desc': '[CLS] Mental Process ##es [unused3] ( B ##iol ##og ##ic Fun ##ction , Human , function , o ##bs ##er ##vable entity : Human Information Process ##ing ; Thought process ; cognitive processing ; processes thought ; processing thought ; Information Process ##ing , Human ; Human information processing ; Information Process ##es ; mental processing ; cognitive process ; Form of thinking , function ; mental processes ; Form of thought ; Form of thinking ; Human Information Process ##es ; Mental Process ; mental process ; thought process ) [SEP]'},
 {'mention_id': '28473083.52',
  'test_set_idx': 37275,
  'type': 'T170',
  'name': 'Grading of Recommendations Assessment , Development , and Evaluation',
  'context': '[CLS] C ##rite ##ria are evidence - based guidelines for specific clinical conditions that are reviewed annually by a multi ##disciplinary expert panel . The guide ##line development and revision include an extensive analysis of current medical literature from peer - reviewed journals and the application of well - established method ##ologies ( RA ##ND / UCLA A ##pp ##rop ##ria ##ten ##ess Method and [unused1] G ##rading of Re ##com ##men ##dation ##s Assessment , Development , and Evaluation [unused2] or G ##RA ##DE ) to rate the appropriate ##ness of imaging and treatment procedures for specific clinical scenarios . In those instances where evidence is lacking or e ##qui ##voc ##al , expert opinion may supplement the available evidence to recommend imaging or treatment . [SEP]',
  'existing_cui': 'C1444325',
  'existing_cui_name': 'Physical evaluation assessment',
  'existing_cui_desc': '[CLS] Physical evaluation assessment [unused3] ( Health Care Act ##ivity , procedure , regime / therapy : As ##ses ##s clinical measurements ; As ##ses ##s physical measurements ) [SEP]',
  'our_bi_cui': 'C0220825',
  'our_bi_cui_name': 'Evaluation',
  'our_bi_cui_desc': '[CLS] Evaluation [unused3] ( Health Care Act ##ivity , qualifier value : Assessment - action ; Investigation ; Eva ##lu ##ated ; Evaluation ##s ; evaluation ##s ; evaluation ; Assessment ; Evaluation Pro ##ced ##ure ; Eva ##lu ##ate ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27442214.17',
  'test_set_idx': 5721,
  'type': 'T058',
  'name': 'abdominal',
  'context': '[CLS] of sex comparisons under ##sco ##re the need for a more comprehensive characterization of the secondary o ##ssi ##fication centers . This study evaluate ##s the ch ##ronology and sex differences for appearance and closure of p ##el ##vic and pro ##ximal f ##em ##oral secondary o ##ssi ##fication centers using com ##puted to ##mo ##graphy . Pat ##ients who underwent [unused1] abdominal [unused2] and p ##el ##vic com ##puted to ##mo ##graphy scan ##s between January 2009 and December 2014 at 2 tertiary level 1 trauma centers were retrospective ##ly reviewed . Pat ##ients between the ages of 2 and 32 years with adequate imaging of the p ##el ##vis and pro ##ximal f ##em ##urs were included . Pat ##ients with a history of [SEP]',
  'existing_cui': 'C0000726',
  'existing_cui_name': 'Abdomen',
  'existing_cui_desc': '[CLS] Abd ##ome ##n [unused3] ( Spa ##tial Concept , body structure , qualifier value , volume : a ##b ##dom ##ino ##pel ##vic region ; Abd ##omi ##no ##pel ##vis ; Abd ##omi ##no ##pel ##vic region ; a ##b ##d ; Abd ##omi ##nal structure ; AB ##D ##OM ##EN ; a ##b ##dom ##ino ##pel ##vic regions ; abdominal ; Abd ##omi ##nal ; Abd ##ome ##ns ; abdomen ) [SEP]',
  'our_bi_cui': 'C0000726',
  'our_bi_cui_name': 'Abdomen',
  'our_bi_cui_desc': '[CLS] Abd ##ome ##n [unused3] ( Spa ##tial Concept , body structure , qualifier value , volume : a ##b ##dom ##ino ##pel ##vic region ; Abd ##omi ##no ##pel ##vis ; Abd ##omi ##no ##pel ##vic region ; a ##b ##d ; Abd ##omi ##nal structure ; AB ##D ##OM ##EN ; a ##b ##dom ##ino ##pel ##vic regions ; abdominal ; Abd ##omi ##nal ; Abd ##ome ##ns ; abdomen ) [SEP]',
  'our_cross_cui': 'C0412620',
  'our_cross_cui_name': 'CT of abdomen',
  'our_cross_cui_desc': '[CLS] CT of abdomen [unused3] ( Health Care Act ##ivity , procedure : abdomen c ##t ; abdomen c ##t scanning ; CT scan of abdomen ; com ##puted to ##mo ##graphy of abdomen ; c ##t scan abdomen ; Computer ##ized to ##mo ##graphy of abdomen ; CT scan abdomen ; c ##t abdomen scan ; C . A . T . scan of abdomen ; cat scan abdomen ; abdomen c ##t scan ##s ; CT scan ; abdomen ; Co ##mp ##uted to ##mo ##graphy , abdomen ; CT scan of the abdomen ; CT scan - abdominal ; abdomen c ##t scan ; Abd ##ome ##n CT ; c ##t of abdomen ; Co ##mp ##uted to ##mo ##graphy of abdominal organs ; [SEP]'},
 {'mention_id': '28454015.27',
  'test_set_idx': 36528,
  'type': 'T033',
  'name': 'unhealthy',
  'context': '[CLS] , highlighting that the general public may need strict ##er health protection measures . The population - weighted two - year average health - risk based air quality index data further demonstrates that all population in the studied cities in He ##nan province live with poll ##uted air - 72 % of the population is exposed to air that is [unused1] un ##hea ##lt ##hy [unused2] for sensitive people , while 28 % of people is exposed to air that can be harmful to healthy people ; and the health risks are much greater during winter than during other seasons . Future works should further improve the health - risk based air quality index algorithm , and valid ##ate the links between the clinical / e [SEP]',
  'existing_cui': 'C0423755',
  'existing_cui_name': 'Unhealthy skin',
  'existing_cui_desc': '[CLS] Un ##hea ##lt ##hy skin [unused3] ( Finding , finding ) [SEP]',
  'our_bi_cui': 'C0205161',
  'our_bi_cui_name': 'Abnormal',
  'our_bi_cui_desc': '[CLS] A ##b ##nor ##mal [unused3] ( Finding , qualifier value : abnormal ; A ##b ##nor ##mal ( m ##od ##ifier ) ; A ##b ##nor ##mal ( m ##od ##ifier ) [ Am ##bi ##guous ] ; AB ##N ##OR ##MA ##L ; de ##vian ##t ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27317610.39',
  'test_set_idx': 2203,
  'type': 'T038',
  'name': 'activated',
  'context': '[CLS] synthetic g ##lu ##co ##cor ##tic ##oid commonly used to test h ##y ##pot ##hala ##mic - pit ##uit ##ary end ##oc ##rine axis regulation . Test ##osterone was evaluated at rest . Further , we assessed changes in injury - induced ne ##uron morphology ( Go ##l ##gi stain ) , ne ##uro ##path ##ology ( silver stain ) and [unused1] activated [unused2] as ##tro ##cy ##tes ( G ##FA ##P ) in the para ##vent ##ric ##ular nucleus of the h ##y ##pot ##hala ##mus . Rest ##ing plasma co ##rt ##ico ##ster ##one levels were decreased at 2 months post ##in ##ju ##ry and there was a blunt ##ed co ##rt ##ico ##ster ##one increase in response to restraint induced stress . No [SEP]',
  'existing_cui': 'C1149591',
  'existing_cui_name': 'protein C ( activated ) activity',
  'existing_cui_desc': '[CLS] protein C ( activated ) activity [unused3] ( B ##iol ##og ##ic Fun ##ction : auto ##p ##roth ##rom ##bin II ##A activity ; G ##SA ##PC ; activated protein C ; activated blood co ##agu ##lation factor XIV activity ; obsolete protein C ( activated ) activity ) [SEP]',
  'our_bi_cui': 'C1326121',
  'our_bi_cui_name': 'astrocyte activation',
  'our_bi_cui_desc': '[CLS] as ##tro ##cy ##te activation [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_cross_cui': 'C1326120',
  'our_cross_cui_name': 'cell activation',
  'our_cross_cui_desc': '[CLS] cell activation [unused3] ( B ##iol ##og ##ic Fun ##ction : activation cells ) [SEP]'},
 {'mention_id': '27722881.42',
  'test_set_idx': 15029,
  'type': 'T007',
  'name': 'JPM1',
  'context': '[CLS] site of di ##ox ##y ##gen ##ase was simulated and analyzed by molecular dock ##ing . Consequently , a feasible de ##grading pathway of flu ##oran ##the ##ne in the strain JP ##M ##1 was proposed based on the meta ##bol ##ites and the interaction analyses . Additionally , the the ##rm ##ody ##nam ##ic analysis showed that the strain [unused1] JP ##M ##1 [unused2] had high tolerance for flu ##oran ##the ##ne , and the influence of flu ##oran ##the ##ne for the bacterial growth activity was ne ##gli ##gible under 100 to 400 mg L ( - 1 ) concentrations . Take ##n together , this study indicates that the strain JP ##M ##1 has high potential for further study in bio ##rem ##edia [SEP]',
  'existing_cui': 'C1421124',
  'existing_cui_name': 'TRGJP gene',
  'existing_cui_desc': '[CLS] T ##R ##G ##JP gene [unused3] ( Ana ##tom ##ical Structure : T - cell receptor , gamma , joining segment JP ; T cell receptor gamma joining P ) [SEP]',
  'our_bi_cui': 'C3629619',
  'our_bi_cui_name': 'bacterium JP1',
  'our_bi_cui_desc': '[CLS] bacterium JP ##1 [unused3] ( Ba ##cter ##ium ) [SEP]',
  'our_cross_cui': 'C4153571',
  'our_cross_cui_name': 'Microbacterium sp . JPM1',
  'our_cross_cui_desc': '[CLS] Micro ##ba ##cter ##ium s ##p . JP ##M ##1 [unused3] ( Ba ##cter ##ium ) [SEP]'},
 {'mention_id': '27580517.4',
  'test_set_idx': 10614,
  'type': 'T058',
  'name': '3D spiral ASL magnetic resonance imaging',
  'context': '[CLS] Assessment of cerebral blood per ##fusion reserve with ace ##ta ##zo ##lam ##ide using [unused1] 3D spiral AS ##L magnetic resonance imaging [unused2] : Preliminary experience in p ##ediatric patients To demonstrate the clinical f ##eas ##ibility of a new non - Car ##tes ##ian cylindrical ##ly - distributed spiral 3D pseudo - continuous art ##erial spin label ##ing magnetic resonance imaging pulse sequence in p ##ediatric patients in q ##uant ##ifying cerebral blood flow response to an ace ##ta ##zo ##lam ##ide v ##as ##od ##ila ##tor challenge . magnetic resonance imaging exams were performed on two 3 Te ##sla Philips In ##gen ##ia systems using 32 channel head coil array ##s . After local institutional review board approval , the 3D spiral - based [SEP]',
  'existing_cui': 'C0024485',
  'existing_cui_name': 'Magnetic Resonance Imaging',
  'existing_cui_desc': '[CLS] Ma ##gnetic Re ##son ##ance I ##maging [unused3] ( Health Care Act ##ivity , MR ##I , procedure , qualifier value : MR - Ma ##gnetic resonance ; MR TO ##MO ##GR ; Ma ##gnetic resonance technique ; magnetic resonance imaging ; N ##MR I ##maging ; Ma ##gnetic resonance imaging procedure ; St ##ead ##y - State Free Pre ##cession MR ##I ; Ma ##gnetic resonance : [ imaging ] or [ study ] ; imaging magnetic nuclear resonance ; Tom ##ography , MR ; MR ##I Sc ##an ; N ##MR - Nuclear magnetic resonance ; Z ##E ##U ##GM ##AT ##O ##GR ; TO ##MO ##GR MR ; N ##MR TO ##MO ##GR ; Nuclear magnetic resonance ; Medical I ##maging , Ma [SEP]',
  'our_bi_cui': 'C0860888',
  'our_bi_cui_name': 'Tomography , Spiral Computed',
  'our_bi_cui_desc': '[CLS] Tom ##ography , S ##pi ##ral Co ##mp ##uted [unused3] ( Health Care Act ##ivity , procedure : S ##pi ##ral com ##puted to ##mo ##graphy scan ; S ##pi ##ral CT ; CT ##s , S ##pi ##ral ; com ##puted to ##mo ##graphy spiral ; Sc ##ans , S ##pi ##ral CT ; He ##lica ##l CT ##s ; He ##lica ##l CT ; S ##pi ##ral CA ##T Sc ##ans ; he ##lica ##l com ##puted to ##mo ##graphy ; c ##t spiral ; S ##pi ##ral Volume ##tric CT ; S ##pi ##ral Computer ##ized Tom ##ography ; Computer - As ##sist ##ed Tom ##ography , S ##pi ##ral ; Volume ##tric CT ; He ##lica ##l Co ##mp ##uted Tom ##ography [SEP]',
  'our_cross_cui': 'C3891302',
  'our_cross_cui_name': 'Arterial Spin Labeling Magnetic Resonance Imaging',
  'our_cross_cui_desc': '[CLS] Arte ##rial Spin Label ##ing Ma ##gnetic Re ##son ##ance I ##maging [unused3] ( Health Care Act ##ivity : AS ##L f ##MR ##I ; Arte ##rial Spin Label ##ing MR ##I ; AR ##TE ##RI ##AL SP ##IN LA ##BE ##L ##ING F ##UN ##CT ##ION ##AL MR ##I ) [SEP]'},
 {'mention_id': '28523846.34',
  'test_set_idx': 38554,
  'type': 'T103',
  'name': 'proteoforms',
  'context': '[CLS] using E ##SI - I ##MS - MS coupled with SEC - mode HP ##LC . MA ##LD ##I - IS ##D data , charge state distribution values determined using E ##SI - MS , and the collision cross - section ##al areas acquired using E ##SI - I ##MS - MS revealed the relationship between the folded and unfolded [unused1] pro ##te ##of ##orms [unused2] of forced - reduced me ##case ##rm ##in and aerial ##ly o ##xi ##dized me ##case ##rm ##in with the free - SH : protein ratio of me ##case ##rm ##in drug product . The collision cross - section ##al area , which is determined using E ##SI - I ##MS - MS , provided pro ##te ##of ##orm information [SEP]',
  'existing_cui': 'C3496296',
  'existing_cui_name': 'area proA of Pandya',
  'existing_cui_desc': '[CLS] area pro ##A of Pan ##dy ##a [unused3] ( Spa ##tial Concept : pro ##kon ##io ##cor ##te ##x ; area pro ##A of Pan ##dy ##a - 1973 ) [SEP]',
  'our_bi_cui': 'C0597298',
  'our_bi_cui_name': 'Protein Isoforms',
  'our_bi_cui_desc': '[CLS] Pro ##tein Is ##of ##orms [unused3] ( Chemical : protein is ##of ##orm ; Pro ##tein Is ##of ##orms [ Chemical / In ##g ##red ##ient ] ; Is ##of ##orms , Pro ##tein ; Is ##of ##orms ; is ##of ##orms ; is ##of ##orm ; Is ##of ##orm ) [SEP]',
  'our_cross_cui': 'C1510464',
  'our_cross_cui_name': 'Protein Structure',
  'our_cross_cui_desc': '[CLS] Pro ##tein Structure [unused3] ( Chemical : protein structure ) [SEP]'},
 {'mention_id': '27975020.12',
  'test_set_idx': 22698,
  'type': 'T022',
  'name': 'skin island',
  'context': '[CLS] Di ##vide ##d and S ##lid ##ing Super ##fic ##ial Te ##mpo ##ral Arte ##ry F ##lap for Primary Don ##or - site C ##losure Super ##fic ##ial temporal artery flap ##s are often used for reconstruction of hair - bearing areas . However , primary closure of the donor site is not easy when the size of the necessary [unused1] skin island [unused2] is relatively large . In such cases , skin g ##raft ##s are needed at the donor site , resulting in bald ##ness . We have solved this issue by applying the divided and sliding flap technique , which was first reported for primary donor - site closure of a la ##tis ##si ##mus do ##rs ##i m ##us ##cu ##loc ##uta [SEP]',
  'existing_cui': 'C0454985',
  'existing_cui_name': 'Baffin island',
  'existing_cui_desc': '[CLS] Ba ##ffin island [unused3] ( Spa ##tial Concept , geographic location ) [SEP]',
  'our_bi_cui': 'C0038925',
  'our_bi_cui_name': 'Surgical Flaps',
  'our_bi_cui_desc': '[CLS] Sur ##gical F ##lap ##s [unused3] ( Ana ##tom ##ical Structure , attribute , qualifier value , substance : flap ; F ##lap Graf ##t ; flap created in surgery ; flap ##s ; F ##lap , Sur ##gical ; Sur ##gical F ##lap ; Re ##con ##st ##ru ##ctive F ##lap ; FL ##AP ##S S ##UR ##G ; [ Q ] F ##lap ; F ##lap ; F ##lap Trans ##p ##lant ; F ##lap ##s ; FL ##AP S ##UR ##G ; F ##lap ##s , Sur ##gical ; S ##UR ##G FL ##AP ; S ##UR ##G FL ##AP ##S ; surgical flap ) [SEP]',
  'our_cross_cui': 'C1123023',
  'our_cross_cui_name': 'Skin',
  'our_cross_cui_desc': '[CLS] Skin [unused3] ( Body System , Anatomy , body structure : in ##te ##gum ##ental system ; in ##te ##gum ##ent ; skin system ; Skin of body ; skin structure ; skins ; In ##te ##gum ##ental system ; In ##te ##gum ##ent ; Pie ##l ; skinned ; Skin structure ; Cut ##is ; SK ##IN ; skin ) [SEP]'},
 {'mention_id': '28294559.30',
  'test_set_idx': 31486,
  'type': 'T038',
  'name': 'deleted',
  'context': '[CLS] type le ##ct ##in receptor ( C - type le ##ct ##in receptor ) - 2 in liver regeneration following partial liver re ##section in mice . Materials and methods I ##rra ##dia ##ted ch ##ime ##ric mice transplant ##ed with f ##etal liver cells from wild - type mice , C - type le ##ct ##in receptor - 2 - [unused1] deleted [unused2] ( K ##O ) mice or mice with C - type le ##ct ##in receptor - 2 deleted specifically from plate ##lets ( fl ##KO ) were generated . Mi ##ce underwent 70 % partial he ##pate ##ct ##omy . I ##mm ##uno ##his ##to ##chemical stain ##ing was performed to investigate the expression of the end ##ogen ##ous l ##igan ##d [SEP]',
  'existing_cui': 'C1333210',
  'existing_cui_name': 'INTS6 gene',
  'existing_cui_desc': '[CLS] IN ##TS ##6 gene [unused3] ( Ana ##tom ##ical Structure : DB ##I - 1 ; IN ##TS ##6 Gene ; DE ##AD / H B ##OX 26 ; IN ##T ##6 ; DD ##X ##26 ##A ; Not ##ch ##l ##2 ; In ##te ##gra ##tor Complex Sub ##uni ##t 6 Gene ; HD ##B ; IN ##TE ##GR ##AT ##OR CO ##MP ##LE ##X S ##U ##B ##UN ##IT 6 ; DE ##LE ##TE ##D IN CA ##NC ##ER 1 ; in ##te ##gra ##tor complex subunit 6 ) [SEP]',
  'our_bi_cui': 'C1522225',
  'our_bi_cui_name': 'Knock-out',
  'our_bi_cui_desc': '[CLS] K ##nock - out [unused3] ( B ##iol ##og ##ic Fun ##ction : K ##nock ##out ) [SEP]',
  'our_cross_cui': 'C0017260',
  'our_cross_cui_name': 'Gene Deletion',
  'our_cross_cui_desc': '[CLS] Gene Del ##eti ##on [unused3] ( B ##iol ##og ##ic Fun ##ction , attribute : M ##OL ##PA ##TH . DE ##L ; gene del ##eti ##on ; gene del ##eti ##ons ; Del ##eti ##ons , Gene ; Gene Del ##eti ##ons ; gene del ##eti ##on mutation ; Gene del ##eti ##on ; Del ##eti ##on , Gene ) [SEP]'},
 {'mention_id': '27342978.18',
  'test_set_idx': 2987,
  'type': 'T017',
  'name': 'microsatellite loci',
  'context': '[CLS] , and its two recognised host - plant species ( Pi ##nus s ##yl ##ves ##tri ##s and P . ni ##gra ) in order to seek for any con ##cor ##dance useful to di ##sent ##ang ##le the evolutionary history of this iconic le ##pid ##op ##tera ##n . DNA variation in one mitochondrial marker and nine nuclear [unused1] micro ##sat ##elli ##te lo ##ci [unused2] revealed a strong p ##hyl ##og ##eo ##graphic pattern across 28 populations of G . is ##abe ##lla ##e studied in Spain and France comprising six groups mostly distributed along different mountain ranges . Re ##ana ##lysis of a previously published ch ##lor ##op ##last micro ##sat ##elli ##te data ##set revealed a three and two - group structure [SEP]',
  'existing_cui': 'C1519302',
  'existing_cui_name': 'Short Tandem Repeat',
  'existing_cui_desc': '[CLS] Short Tan ##de ##m Rep ##eat [unused3] ( Chemical : Tan ##de ##m Rep ##eat ##s , Short ; Rep ##eat ##s , Micro ##sat ##elli ##te ; Rep ##eti ##tive Se ##quence , Simple ; Simple Rep ##eti ##tive Se ##quence ; Simple Rep ##eti ##tive Se ##quence ##s ; Simple Se ##quence Rep ##eat ; Micro ##sat ##elli ##te Rep ##eat ##s ; Rep ##eat ##s , Simple Se ##quence ; Rep ##eti ##tive Se ##quence ##s , Simple ; Micro ##sat ##elli ##te Rep ##eat ; Rep ##eat , Simple Se ##quence ; Se ##quence ##s , Simple Rep ##eti ##tive ; Se ##quence Rep ##eat , Simple ; Short Tan ##de ##m Rep ##eat ##s ; Rep ##eat ##s , Short Tan [SEP]',
  'our_bi_cui': 'C1519302',
  'our_bi_cui_name': 'Short Tandem Repeat',
  'our_bi_cui_desc': '[CLS] Short Tan ##de ##m Rep ##eat [unused3] ( Chemical : Tan ##de ##m Rep ##eat ##s , Short ; Rep ##eat ##s , Micro ##sat ##elli ##te ; Rep ##eti ##tive Se ##quence , Simple ; Simple Rep ##eti ##tive Se ##quence ; Simple Rep ##eti ##tive Se ##quence ##s ; Simple Se ##quence Rep ##eat ; Micro ##sat ##elli ##te Rep ##eat ##s ; Rep ##eat ##s , Simple Se ##quence ; Rep ##eti ##tive Se ##quence ##s , Simple ; Micro ##sat ##elli ##te Rep ##eat ; Rep ##eat , Simple Se ##quence ; Se ##quence ##s , Simple Rep ##eti ##tive ; Se ##quence Rep ##eat , Simple ; Short Tan ##de ##m Rep ##eat ##s ; Rep ##eat ##s , Short Tan [SEP]',
  'our_cross_cui': 'C0678933',
  'our_cross_cui_name': 'Genetic Loci',
  'our_cross_cui_desc': '[CLS] Gene ##tic Lo ##ci [unused3] ( Ana ##tom ##ical Structure : Lo ##cus , Gene ##tic ; Gene ##tic Lo ##cus ; Lo ##ci , Gene ##tic ) [SEP]'},
 {'mention_id': '28389374.19',
  'test_set_idx': 34021,
  'type': 'T103',
  'name': 'of BRCA1 associated protein1',
  'context': '[CLS] ##al regulation , ch ##roma ##tin modification by de ##ubi ##qui ##tina ##ting his ##tone 2 ##A , and DNA repair . BR ##CA ##1 associated protein ##1 mutations are frequent in ma ##li ##gnant p ##le ##ural me ##so ##the ##lio ##ma . Our aim was to functional ##ly character ##ize a newly identified is ##of ##orm [unused1] of BR ##CA ##1 associated protein ##1 [unused2] and investigate the effects of its expression on drug sensitivity in ma ##li ##gnant p ##le ##ural me ##so ##the ##lio ##ma . Express ##ion of BR ##CA ##1 associated protein ##1 is ##of ##orms was detected by quantitative polymer ##ase chain reaction in ma ##li ##gnant p ##le ##ural me ##so ##the ##lio ##ma and normal me ##so [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0259275',
  'our_bi_cui_name': 'BRCA1 Protein',
  'our_bi_cui_desc': '[CLS] BR ##CA ##1 Pro ##tein [unused3] ( Chemical , substance : Ring Fin ##ger Pro ##tein 53 ; BR ##CA ##1 Pro ##tein [ Chemical / In ##g ##red ##ient ] ; BR ##CA PR ##OT ##EI ##N 00 ##1 ; B ##rea ##st Cancer 1 Gene Product ; BR ##CA GE ##NE PR ##OD ##UC ##T 00 ##1 ; BR ##CA ##1 gene product ; BR ##CA ##1 Gene Product ; B ##rea ##st Cancer 1 Pro ##tein ; B ##rea ##st cancer type 1 su ##s ##ce ##pt ##ibility protein ; B ##rea ##st Cancer Type 1 Su ##s ##ce ##pt ##ibility Pro ##tein ) [SEP]',
  'our_cross_cui': 'C1368335',
  'our_cross_cui_name': 'BAP1 protein , human',
  'our_cross_cui_desc': '[CLS] BA ##P ##1 protein , human [unused3] ( Chemical : BA ##P - 1 protein , human ; U ##bi ##qui ##tin Car ##box ##yl - Terminal H ##ydro ##lase BA ##P ##1 ; Ce ##re ##bra ##l Pro ##tein 6 ; BR ##CA ##1 associated protein - 1 ( u ##bi ##qui ##tin car ##box ##y - terminal h ##ydro ##lase ) , human ; EC 3 . 4 . 19 . 12 ; BR ##CA ##1 Associated Pro ##tein - 1 ; Ce ##re ##bra ##l Pro ##tein - 13 ; Ring ##2 protein , human ; BR ##CA ##1 - Associated Pro ##tein 1 ; Ce ##re ##bra ##l Pro ##tein - 6 ; BR ##CA ##1 associated protein - 1 , human [SEP]'},
 {'mention_id': '27760149.44',
  'test_set_idx': 15831,
  'type': 'T033',
  'name': 'up - to - date',
  'context': "[CLS] problems , we introduce B ##ES ##T , a bio ##medical entity search tool . B ##ES ##T returns , as a result , a list of 10 different types of bio ##medical entities including genes , diseases , drugs , targets , transcription factors , mi ##RNA ##s , and mutations that are relevant to a user ' s que ##ry . To the best of our knowledge , B ##ES ##T is the only system that processes free text que ##ries and returns [unused1] up - to - date [unused2] results in real time including mutation information in the results . B ##ES ##T is freely accessible at http : / / best . k ##ore ##a . a ##c . k ##r . [SEP]",
  'existing_cui': 'C0474298',
  'existing_cui_name': 'Up-to-date with immunizations',
  'existing_cui_desc': '[CLS] Up - to - date with im ##mu ##ni ##zation ##s [unused3] ( Finding , finding , situation : Up - to - date with im ##mu ##nis ##ations ) [SEP]',
  'our_bi_cui': 'C0547043',
  'our_bi_cui_name': 'Up',
  'our_bi_cui_desc': '[CLS] Up [unused3] ( Spa ##tial Concept , qualifier value : Up to ; Ad ; ad ; up ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27838742.87',
  'test_set_idx': 19054,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . Del ##ica ##f ##lav ##one may represent a potential therapeutic agent for lung cancer . Del ##ica ##f ##lav ##one showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . Del ##ica ##f ##lav ##one induced auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . [unused1] Del ##ica ##f ##lav ##one [unused2] did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . Del ##ica ##f ##lav ##one may represent a potential therapeutic agent for lung cancer . [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27887587.39',
  'test_set_idx': 20927,
  'type': 'T033',
  'name': 'family',
  'context': "[CLS] at the Community Mental Health Centre in their work towards their recovery goals were el ##icit ##ed , as follows : developing an understanding of ones ##elf and one ' s mental health problems ; learning how to change feelings and behaviour ##s ; being ' pushed ' into social arena ##s ; finding helpful medication ; and counsel ##ling in [unused1] family [unused2] , practical and financial issues . The participants ' expectations about counsel ##ling with regard to longer - term family , practical , and financial challenges were insufficient ##ly met by the Community Mental Health Centre . In the experience of the service users , recovery occurred within the context of their everyday life with or without the support of their professional [SEP]",
  'existing_cui': 'C1301584',
  'existing_cui_name': 'Last Name',
  'existing_cui_desc': '[CLS] Last Name [unused3] ( Intel ##lect ##ual Product , o ##bs ##er ##vable entity : Family Name ; family ; Last name ; Sur ##name ) [SEP]',
  'our_bi_cui': 'C0424960',
  'our_bi_cui_name': 'Family problems',
  'our_bi_cui_desc': '[CLS] Family problems [unused3] ( Finding , finding , history : problem ; family ; Problem ; family ; family problems ; family ; problem ; family problem ) [SEP]',
  'our_cross_cui': 'C0851511',
  'our_cross_cui_name': 'Family issues',
  'our_cross_cui_desc': '[CLS] Family issues [unused3] ( Finding : family issues ; families issues ; Family Issues ; family issue ) [SEP]'},
 {'mention_id': '27506129.25',
  'test_set_idx': 8657,
  'type': 'T170',
  'name': 'Correlation based feature selection',
  'context': '[CLS] trained with individual features is below 0 . 1 for both scales . Multi - layer per ##ce ##pt ##ron obtain ##s the best values , but is less consistent than the random forest method . When all features are combined , the best results for both scales are achieved with Multi - layer per ##ce ##pt ##ron . [unused1] Co ##rrel ##ation based feature selection [unused2] and M ##5 provide the best results , mean square error = 0 . 108 and mean square error = 0 . 06 ##1 respectively . Finally , the class im ##bal ##ance problem is mini ##mise ##d with the SM ##OT ##E approach for both scales ( mean square error < 0 . 00 ##6 ) . Machine [SEP]',
  'existing_cui': 'C0010101',
  'existing_cui_name': 'Correlation Study',
  'existing_cui_desc': '[CLS] Co ##rrel ##ation Study [unused3] ( Research Act ##ivity , Statistical : Study , Co ##rrel ##ation ; Studies , Co ##rrel ##ation ; Co ##rrel ##ation Studies ; Co ##rrel ##ation ; Statistical Co ##rrel ##ation ) [SEP]',
  'our_bi_cui': 'C0010101',
  'our_bi_cui_name': 'Correlation Study',
  'our_bi_cui_desc': '[CLS] Co ##rrel ##ation Study [unused3] ( Research Act ##ivity , Statistical : Study , Co ##rrel ##ation ; Studies , Co ##rrel ##ation ; Co ##rrel ##ation Studies ; Co ##rrel ##ation ; Statistical Co ##rrel ##ation ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27779321.8',
  'test_set_idx': 16416,
  'type': 'T170',
  'name': 'equilibrium partitioning approach',
  'context': '[CLS] dose - response models for p ##oly ##ch ##lor ##inated bi ##phe ##ny ##l - contaminated sediment using equilibrium partition ##ing The study goal was to develop a sediment p ##oly ##ch ##lor ##inated bi ##phe ##ny ##l dose - response model based on bent ##hic in ##vert ##eb ##rate effects to PC ##B ##s . The authors used an [unused1] equilibrium partition ##ing approach [unused2] to generate predicted p ##oly ##ch ##lor ##inated bi ##phe ##ny ##l sediment effect concentrations ( largely A ##ro ##c ##lor 125 ##4 ) associated with a gradient of toxic effects in bent ##hic organisms from effects observed in aquatic toxicity studies . The present study differs from all other equilibrium partition ##ing collective sediment investigations in that the authors [SEP]',
  'existing_cui': 'C0014653',
  'existing_cui_name': 'Equilibrium',
  'existing_cui_desc': '[CLS] E ##qui ##li ##bri ##um [unused3] ( B ##iol ##og ##ic Fun ##ction , Motor Process ##es , function , o ##bs ##er ##vable entity , qualifier value : balancing ; equilibrium ; Ba ##lance ; E ##qui ##li ##bri ##um , function ; Ba ##lancing ; balance ; Ba ##lance , function ) [SEP]',
  'our_bi_cui': 'C1295077',
  'our_bi_cui_name': 'Chemical separation method',
  'our_bi_cui_desc': '[CLS] Chemical separation method [unused3] ( Health Care Act ##ivity , procedure ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27875010.26',
  'test_set_idx': 20231,
  'type': 'T017',
  'name': 'tissues',
  'context': '[CLS] ##al mac ##roa ##l ##gal species in the North Atlantic rocky inter ##ti ##dal zone , and found that nut ##rient - en ##rich ##ed al ##gal blades showed a significant increase in tissue growth compared to individuals grown under ambient conditions . We further q ##uant ##ified net photos ##ynth ##esis by ambient and nut ##rient - en ##rich ##ed [unused1] tissues [unused2] at sat ##ura ##ting i ##rra ##dian ##ce over a range of temperature conditions ( 6 - 30 °C ) . Re ##spiration was un ##af ##fected by nut ##rient treatment ; however , there was a significant increase in photos ##ynth ##etic oxygen production for nut ##rient - en ##rich ##ed tissue compared to ambient , but only at elevated ( [SEP]',
  'existing_cui': 'C0040300',
  'existing_cui_name': 'Body tissue',
  'existing_cui_desc': '[CLS] Body tissue [unused3] ( Ana ##tom ##ical Structure , Body , body structure : Port ##ion of tissue ; Normal T ##iss ##ue ; T ##iss ##ue ; Text ##us ; Body T ##iss ##ues ; tissues ; Body tissue structure ; body tissues ; T ##iss ; body tissue ; tissue ; T ##iss ##ues ; T ##IS ##SU ##E ) [SEP]',
  'our_bi_cui': 'C0040300',
  'our_bi_cui_name': 'Body tissue',
  'our_bi_cui_desc': '[CLS] Body tissue [unused3] ( Ana ##tom ##ical Structure , Body , body structure : Port ##ion of tissue ; Normal T ##iss ##ue ; T ##iss ##ue ; Text ##us ; Body T ##iss ##ues ; tissues ; Body tissue structure ; body tissues ; T ##iss ; body tissue ; tissue ; T ##iss ##ues ; T ##IS ##SU ##E ) [SEP]',
  'our_cross_cui': 'C1514137',
  'our_cross_cui_name': 'Plant Tissue , Cells',
  'our_cross_cui_desc': '[CLS] Plant T ##iss ##ue , Cell ##s [unused3] ( Ana ##tom ##ical Structure : Plant cells and tissues ) [SEP]'},
 {'mention_id': '28471040.2',
  'test_set_idx': 37116,
  'type': 'T103',
  'name': 'lipid - polyethylene glycol - polymer hybrid nanoparticles',
  'context': '[CLS] Far ##nes ##yl ##thi ##osa ##lic ##yl ##ic acid - loaded [unused1] lip ##id - p ##oly ##eth ##yle ##ne g ##ly ##co ##l - polymer hybrid na ##no ##par ##tic ##les [unused2] for treatment of g ##lio ##blast ##oma We aimed to develop lip ##id - p ##oly ##eth ##yle ##ne g ##ly ##co ##l ( lip ##id - p ##oly ##eth ##yle ##ne g ##ly ##co ##l ) - polymer hybrid na ##no ##par ##tic ##les , which have high affinity to t ##umour tissue with active ingredient , a new generation anti ##ne ##op ##lastic drug , far ##nes ##yl ##thi ##osa ##lic ##yl ##ic acid for treatment of g ##lio ##blast ##oma . Far ##nes ##yl ##thi ##osa ##lic ##yl ##ic acid [SEP]',
  'existing_cui': 'C2000189',
  'existing_cui_name': 'polyethylene glycol-poly',
  'existing_cui_desc': '[CLS] p ##oly ##eth ##yle ##ne g ##ly ##co ##l - p ##oly [unused3] ( Chemical , prop ##yle ##ne im ##ine , prop ##yle ##ne im ##ine : P ##EG - PP ##I ) [SEP]',
  'our_bi_cui': 'C0005479',
  'our_bi_cui_name': 'Biomaterials',
  'our_bi_cui_desc': '[CLS] B ##io ##mate ##rial ##s [unused3] ( Chemical : B ##io ##mate ##rial ; bio ##com ##pa ##tible materials ; B ##io ##medical Material ; bio ##mate ##rial ##s ; B ##io ##medical materials ; B ##io ##com ##pa ##tible Materials ; bio ##medical materials ; bio ##mate ##rial ; Materials , B ##io ##com ##pa ##tible ; bio ##medical material ; B ##io ##com ##pa ##tible Materials [ Chemical / In ##g ##red ##ient ] ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28295414.14',
  'test_set_idx': 31552,
  'type': 'T168',
  'name': 'drinks',
  'context': '[CLS] Administration proposed to expand the options for primary end points in the development of medications for alcohol use disorder to include either a ##bs ##tine ##nce from alcohol or a non ##ab ##st ##inen ##t outcome : no heavy drinking days ( with a heavy drinking day defined as more than 3 drinks per day for women and more than 4 [unused1] drinks [unused2] per day for men [ > 3 / > 4 cut ##off ] ) . The Food and Drug Administration also suggested that 6 months would be the most appropriate length for a clinical trial to demonstrate the stability of this non ##ab ##st ##inen ##t drinking outcome . However , few alcohol clinical trials have examined the stability of non ##hea [SEP]',
  'existing_cui': 'C0452428',
  'existing_cui_name': 'Drink',
  'existing_cui_desc': '[CLS] Dr ##ink [unused3] ( Food , diet ##ary substance , diet ##ary substance , attribute , substance : Dr ##ink ##s ; drinks ; drink [ substance ] ; drink ) [SEP]',
  'our_bi_cui': 'C0452428',
  'our_bi_cui_name': 'Drink',
  'our_bi_cui_desc': '[CLS] Dr ##ink [unused3] ( Food , diet ##ary substance , diet ##ary substance , attribute , substance : Dr ##ink ##s ; drinks ; drink [ substance ] ; drink ) [SEP]',
  'our_cross_cui': 'C0001967',
  'our_cross_cui_name': 'Alcoholic Beverages',
  'our_cross_cui_desc': '[CLS] Al ##co ##hol ##ic Be ##ver ##ages [unused3] ( Food , Al ##co ##hol ##ic , substance : Be ##ver ##age , Al ##co ##hol ##ic ; Al ##co ##hol ##ic Be ##ver ##age ; Al ##co ##hol ##ic beverages ; alcoholic drinks ; Be ##ver ##ages ; Be ##ver ##ages , Al ##co ##hol ##ic ; alcoholic beverages ; alcoholic beverage ; alcoholic ##s beverages ; Al ##co ##hol ##ic beverage ; Al ##co ##hol ##ic drink ; alcoholic drinking ) [SEP]'},
 {'mention_id': '28286958.6',
  'test_set_idx': 31315,
  'type': 'T103',
  'name': 'dihydrofuran rings',
  'context': '[CLS] o ##xi ##da ##tive reaction of q ##uin ##az ##olin ##ones and al ##ky ##nes has been developed for se ##quential [ 4 + 2 ] and [ 3 + 2 ] c ##y ##c ##load ##ditions to assemble a novel fused - p ##oly ##cy ##c ##lic system containing te ##tra ##hy ##dr ##opy ##rid ##ine and [unused1] di ##hy ##dr ##of ##ura ##n rings [unused2] . The reaction process involves C - H and N - H bond functional ##ization for the formation of te ##tra ##hy ##dr ##opy ##rid ##ine and an oxygen radical c ##y ##c ##li ##zation for the di ##hy ##dr ##of ##ura ##n ring . This atom - and step - economical synthesis is highly efficient and has good [SEP]',
  'existing_cui': 'C0650776',
  'existing_cui_name': '2,3-dihydrofuran',
  'existing_cui_desc': '[CLS] 2 , 3 - di ##hy ##dr ##of ##ura ##n [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C4078002',
  'our_bi_cui_name': '2,5-dihydrofuran',
  'our_bi_cui_desc': '[CLS] 2 , 5 - di ##hy ##dr ##of ##ura ##n [unused3] ( Chemical ) [SEP]',
  'our_cross_cui': 'C0029224',
  'our_cross_cui_name': 'Organic Chemicals',
  'our_cross_cui_desc': '[CLS] Organic Chemical ##s [unused3] ( Chemical , substance : organic chemicals ; compound organic ; Organic Chemical ##s [ Chemical / In ##g ##red ##ient ] ; organic compound ; Chemical ##s , Organic ; organic chemical ; Organic Chemical ; chemical organic ; organic compounds ; Organic compound ; Organic compounds ; Organic chemical ) [SEP]'},
 {'mention_id': '28077286.16',
  'test_set_idx': 25066,
  'type': 'T103',
  'name': 'MeHg',
  'context': '[CLS] brain barrier and is a known ne ##uro - toxic ##ant . Me ##H ##g accumulation in the brain causes his ##top ##ath ##ological alterations , ne ##uro ##be ##ha ##vior ##al changes , and imp ##air ##ments to cognitive motor functions in ma ##mmal ##ian models . However , in birds the ne ##uro ##to ##xi ##c effects of [unused1] Me ##H ##g [unused2] on the developing pre - hatch ##ing brain and con ##se ##quent behavioral alterations in adult birds have not received much attention . Moreover , pass ##eri ##ne birds are poorly represented in Me ##H ##g ne ##uro ##to ##xi ##cology studies in comparison to other a ##vian orders . Hence in this study , we used the egg injection method [SEP]',
  'existing_cui': 'C0228520',
  'existing_cui_name': 'Structure of reticular formation of medulla oblongata',
  'existing_cui_desc': '[CLS] Structure of re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata [unused3] ( Ana ##tom ##ical Structure , body structure : bulb ##ar re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla ; B ##ul ##bar re ##tic ##ular formation ; Form ##ati ##o re ##tic ##ular ##is my ##ele ##nce ##pha ##li ; me ##du ##llar ##y re ##tic ##ular formation ; re ##tic ##ular formation of me ##du ##lla ; Me ##du ##llar ##y re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata ) [SEP]',
  'our_bi_cui': 'C0025424',
  'our_bi_cui_name': 'Mercury',
  'our_bi_cui_desc': '[CLS] Mercury [unused3] ( Chemical , H ##g , Metal , elemental , substance : mercury ; Mercury [ Chemical / In ##g ##red ##ient ] ; metal mercury ; H ##yd ##rar ##gy ##rum ; H ##g element ; Li ##quid Silver ; Quick ##si ##lve ##r ; H ##g ; H ##g - Mercury ; h ##g ; ME ##RC ##UR ##Y ) [SEP]',
  'our_cross_cui': 'C0025794',
  'our_cross_cui_name': 'Methylmercury Compounds',
  'our_cross_cui_desc': '[CLS] Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [unused3] ( Chemical : Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [ Chemical / In ##g ##red ##ient ] ; Mercury Co ##mpo ##unds , Met ##hyl ; ME ##TH ##Y ##L ME ##RC ##UR ##Y CP ##DS ; Met ##hyl Mercury Co ##mpo ##unds ; ME ##TH ##Y ##LM ##ER ##CU ##R ##Y CP ##DS ; Co ##mpo ##unds , Met ##hyl ##mer ##cu ##ry ; met ##hyl ##mer ##cu ##ry ; Co ##mpo ##unds , Met ##hyl Mercury ; Met ##hyl ##mer ##cu ##ry ) [SEP]'},
 {'mention_id': '27791169.15',
  'test_set_idx': 16868,
  'type': 'T017',
  'name': 'BPH9',
  'context': '[CLS] p ##ests of rice ( Or ##y ##za sat ##iva L . ) . Currently , 30 Brown plant ##hop ##per - resistance genes have been genetically defined , most of which are cluster ##ed on specific chromosome regions . Here , we describe molecular c ##lon ##ing and characterization of a Brown plant ##hop ##per - resistance gene , [unused1] BP ##H ##9 [unused2] , mapped on the long arm of rice chromosome 12 ( 12 ##L ) . BP ##H ##9 en ##codes a rare type of n ##uc ##leo ##tide - binding and le ##uc ##ine - rich repeat ( n ##uc ##leo ##tide - binding and le ##uc ##ine - rich repeat ) - containing protein that local ##izes to the end [SEP]',
  'existing_cui': 'C0005001',
  'existing_cui_name': 'Benign prostatic hypertrophy',
  'existing_cui_desc': '[CLS] Ben ##ign pro ##static h ##yper ##tro ##phy [unused3] ( B ##iol ##og ##ic Fun ##ction , BP ##H , ben ##ign , ben ##ign pro ##static h ##yper ##tro ##phy , diagnosis : H ##yper ##tro ##phy - pro ##st . ben . ; ben ##ign pro ##static h ##yper ##tro ##phy ; Pro ##static h ##yper - ben ##ign ; Ben ##ign Pro ##static H ##yper ##p ##lasia - BP ##H ; En ##lar ##ged Pro ##state ; H ##yper ##tro ##phy , Ben ##ign Pro ##static ; H ##yper ##tro ##phy ( ben ##ign ) of pro ##state ; Ben ##ign h ##yper ##tro ##phy of pro ##state ; BP ##H - ben ##ign pro ##static h ##yper ##tro ##phy ; Pro ##static [SEP]',
  'our_bi_cui': 'C1419294',
  'our_bi_cui_name': 'RBBP9 gene',
  'our_bi_cui_desc': '[CLS] RB ##B ##P ##9 gene [unused3] ( Ana ##tom ##ical Structure : RB ##B ##P ##10 ; B ##5 ##T O ##VE ##RE ##X ##PR ##ES ##SE ##D GE ##NE ; RB ##B ##P ##9 Gene ; R ##ET ##IN ##O ##BL ##AS ##TO ##MA - B ##IN ##DI ##NG PR ##OT ##EI ##N 9 ; Re ##tino ##blast ##oma Bin ##ding Pro ##tein 9 Gene ; re ##tino ##blast ##oma binding protein 9 ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27832159.37',
  'test_set_idx': 18585,
  'type': 'T103',
  'name': 'compound-326',
  'context': '[CLS] - molecule , in a high - fat diet - induced o ##bes ##e mouse model . In v ##ivo Delta - 5 des ##at ##ura ##se in ##hibition was confirmed by determining changes in blood a ##rac ##hi ##don ##ic acid / D ##G ##LA profiles . In diet - induced o ##bes ##e mice , chronic treatment with [unused1] compound - 32 ##6 [unused2] lowered insulin resistance and caused body weight loss without significant impact on cumulative ca ##lor ##ie intake . Dec ##reased mac ##rop ##hage in ##fi ##ltration into ad ##ip ##ose tissue was expected from m ##RNA analysis . Inc ##reased daily energy expenditure was also observed following administration of compound - 32 ##6 , in line with sustained body weight [SEP]',
  'existing_cui': 'C0258113',
  'existing_cui_name': '4 - ( ( 3-bromophenyl ) amino ) -6,7 - dimethoxyquinazoline',
  'existing_cui_desc': "[CLS] 4 - ( ( 3 - br ##omo ##phe ##ny ##l ) amino ) - 6 , 7 - dim ##eth ##ox ##y ##quin ##az ##olin ##e [unused3] ( Chemical : Co ##mpo ##und 32 ; 4 - ( 3 ' - br ##omo ##ani ##lino ) - 6 , 7 - dim ##eth ##ox ##y ##quin ##az ##olin ##e ) [SEP]",
  'our_bi_cui': 'C0654579',
  'our_bi_cui_name': 'NM 326',
  'our_bi_cui_desc': '[CLS] N ##M 32 ##6 [unused3] ( Chemical : Benz ##ene ##met ##hana ##mini ##um , N - ( 2 - ( ( ( he ##xa ##de ##cy ##lo ##xy ) h ##ydro ##xy ##ph ##os ##phi ##ny ##l ) o ##xy ) et ##hyl ) - 3 - ( i ##od ##o - 125 ##I ) - N , N - dim ##eth ##yl - , h ##ydro ##xi ##de , inner salt ; N ##M - 32 ##6 ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28373341.3',
  'test_set_idx': 33457,
  'type': 'T098',
  'name': 'low spenders',
  'context': '[CLS] Most Americans Have Good Health , Little Un ##met Need , And Few Health Care Ex ##pen ##ses The distribution of health care expenditure ##s remains highly concentrated , but most Americans use few health care resources and have low out - of - pocket spending . More than 93 percent of " [unused1] low spend ##ers [unused2] " ( those in the bottom half of the population ) believe they have received all needed care in a time ##ly manner . The low spending by the majority of the population has remained almost unchanged during the thirty - seven - year period examined . [SEP]',
  'existing_cui': 'C0238421',
  'existing_cui_name': 'Selenium deficiency',
  'existing_cui_desc': '[CLS] Se ##len ##ium deficiency [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : deficiency se ##len ##ium ; Se + + low ; deficiency ; se ##len ##ium ; low se ##s ; se ##len ##ium ; deficiency ; Se levels low ; se ##len ##ium deficiency ) [SEP]',
  'our_bi_cui': 'C0024045',
  'our_bi_cui_name': 'Low-Income Population',
  'our_bi_cui_desc': '[CLS] Low - Inc ##ome Population [unused3] ( Population Group : income low populations ; Population ##s , Low Inc ##ome ; Population , Low Inc ##ome ; Low - Inc ##ome Population ##s ; low - income population ; Population , Low - Inc ##ome ; Low Inc ##ome Population ##s ; Low Inc ##ome Population ; Population ##s , Low - Inc ##ome ) [SEP]',
  'our_cross_cui': 'C1257890',
  'our_cross_cui_name': 'Population Group',
  'our_cross_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]'},
 {'mention_id': '27235559.6',
  'test_set_idx': 303,
  'type': 'T074',
  'name': 'vein visualization display system',
  'context': '[CLS] Does infrared visual ##ization improve selection of ve ##ni ##pu ##nc ##ture sites for in ##d ##well ##ing needle at the forearm in second - year nursing students ? To evaluate the effectiveness of a [unused1] vein visual ##ization display system [unused2] using near - infrared light ( " V ##ein Di ##sp ##lay " ) for the safe and proper selection of ve ##ni ##pu ##nc ##ture sites for in ##d ##well ##ing needle placement in the forearm . Ten second year nursing students were recruited to apply an in ##d ##well ##ing needle line with and without V ##ein Di ##sp ##lay . Another ten participants were recruited from various faculty to serve as patients . The quality of the ve ##ni ##pu ##nc [SEP]',
  'existing_cui': 'C0175631',
  'existing_cui_name': 'Imagery',
  'existing_cui_desc': '[CLS] Image ##ry [unused3] ( B ##iol ##og ##ic Fun ##ction : visual ##ization ; Visual ##ization ; imagery ) [SEP]',
  'our_bi_cui': 'C0182167',
  'our_bi_cui_name': 'Visual paging system',
  'our_bi_cui_desc': '[CLS] Visual p ##aging system [unused3] ( Medical Devi ##ce , physical object : Pa ##ging Systems , Visual ) [SEP]',
  'our_cross_cui': 'C0025080',
  'our_cross_cui_name': 'Medical Devices',
  'our_cross_cui_desc': '[CLS] Medical Devi ##ces [unused3] ( Medical Devi ##ce , physical device , physical object : Clinical equipment ; B ##io ##medical device ; DE ##VI ##CE ME ##D ; ME ##D DE ##VI ##CE ##S ; ME ##D DE ##VI ##CE ; bio ##medical devices ; Devi ##ces , Medical ; Medical devices ; General bio ##medical device ; DE ##VI ##CE , GE ##NE ##RA ##L ME ##DI ##CA ##L ; B ##io ##medical equipment ; Devi ##ce , Medical ; Hospital , medical AND / OR surgical equipment ( physical device ) ; bio ##medical equipment ; medical equipment ; Medical equipment ; DE ##VI ##CE ##S ME ##D ; DE ##VI ##CE ; DE ##VI ##CE ##S ; medical device ; medical devices [SEP]'},
 {'mention_id': '27298278.21',
  'test_set_idx': 1822,
  'type': 'T103',
  'name': 'D',
  'context': '[CLS] ##ho ##quin ##one , an ant ##hra ##quin ##one , and a nap ##ht ##ho ##hy ##dr ##o ##quin ##one dim ##er , together with three known q ##uin ##ones and seven other known compounds , were isolated from the aerial parts of Mo ##rin ##da par ##vi ##folia . The structures of m ##ori ##nda ##par ##vin ##s C , [unused1] D [unused2] , E , F , and G were el ##uc ##idated on the basis of s ##pect ##ros ##copic or X - ray di ##ff ##raction analysis as met ##hyl 4 - h ##ydro ##xy - 1 , 6 - dim ##eth ##ox ##y - nap ##ht ##hale ##ne - 2 - car ##box ##yla ##te , met ##hyl 4 , 8 [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0010207',
  'our_bi_cui_name': 'Coumarins',
  'our_bi_cui_desc': '[CLS] Co ##uma ##rin ##s [unused3] ( Chemical : co ##uma ##rin ##s ; Co ##uma ##rine ##s ; Der ##via ##tives , 1 , 2 - Benz ##opy ##rone ; 1 , 2 - Benz ##opy ##rone ##s ; Benz ##opy ##ran 2 ones ; 1 , 2 - Benz ##opy ##rone Der ##via ##tives ; Co ##uma ##rin ##s [ Chemical / In ##g ##red ##ient ] ; 1 , 2 Benz ##opy ##rone ##s ; Co ##uma ##rin Der ##iva ##tives ; Der ##iva ##tives , Co ##uma ##rin ; Benz ##opy ##ran - 2 - ones ; B ##EN ##Z ##OP ##Y ##RO ##NE ##S 01 02 ; 1 , 2 Benz ##o P ##yr ##ones ; 1 , 2 - Benz ##o [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28237762.14',
  'test_set_idx': 29316,
  'type': 'T103',
  'name': 'acylated phenoxyanilide',
  'context': '[CLS] ##ama ##se The novel classes of a ##cy ##lated p ##hen ##ox ##yan ##ili ##de and th ##io ##ure ##a compounds were investigated for their ability to in ##hibit T ##EM type β - la ##ct ##ama ##se enzyme . Two compounds 4 ##g and 5 ##c reveal the in ##hibition pot ##ency in micro ##mo ##lar range and show their action by non - co ##valent binding in the vicinity of the T ##EM - 171 active site . The structure activity relationship around carbon chain length and different sub ##st ##it ##uen ##ts in or ##th ##o - and para - positions of [unused1] a ##cy ##lated p ##hen ##ox ##yan ##ili ##de [unused2] as well as molecular modelling study has been performed . [SEP]',
  'existing_cui': 'C0031403',
  'existing_cui_name': 'Phenetidine',
  'existing_cui_desc': '[CLS] Ph ##ene ##ti ##dine [unused3] ( Chemical : Ph ##ene ##ti ##dine [ Chemical / In ##g ##red ##ient ] ; Benz ##ena ##mine , 4 - et ##ho ##xy - ; Am ##ino ##phe ##net ##ole ; E ##th ##ox ##yan ##ili ##ne ) [SEP]',
  'our_bi_cui': 'C0029224',
  'our_bi_cui_name': 'Organic Chemicals',
  'our_bi_cui_desc': '[CLS] Organic Chemical ##s [unused3] ( Chemical , substance : organic chemicals ; compound organic ; Organic Chemical ##s [ Chemical / In ##g ##red ##ient ] ; organic compound ; Chemical ##s , Organic ; organic chemical ; Organic Chemical ; chemical organic ; organic compounds ; Organic compound ; Organic compounds ; Organic chemical ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27894020.2',
  'test_set_idx': 21111,
  'type': 'T033',
  'name': 'detection',
  'context': '[CLS] The an ##ato ##mic ##al mum ##mie ##s of Mom ##bell ##o : [unused1] detection [unused2] of cocaine , ni ##cot ##ine , and ca ##ffe ##ine in the hair of psychiatric patients of the early 20th century The Mom ##bell ##o P ##sy ##chia ##tric Hospital in Lim ##bia ##te , near Milan , replaced the old Sen ##av ##ra Hospital as the P ##sy ##chia ##tric Hospital for the Province of Milan in the 19th century . During the early 20th century , bodies of several Mom ##bell ##o patients were di ##sse ##cted and preserved by Giuseppe Para ##vic ##ini , an an ##ato ##mist who operated within the asylum . The aim of the present study was to examine and memorial ##ize [SEP]',
  'existing_cui': 'C0206100',
  'existing_cui_name': 'Signal Detection',
  'existing_cui_desc': '[CLS] Signal Det ##ec ##tion [unused3] ( B ##iol ##og ##ic Fun ##ction , Psychology , Psychology , Signal : Signal Det ##ec ##tion Theo ##ries ; Signal Det ##ec ##tions , Psychological ; Theory , Signal Det ##ec ##tion ; Signal detection ; S ##IG ##NA ##L DE ##TE ##CT ##ION AN ##AL ; Psychological Signal Det ##ec ##tions ; Det ##ec ##tion ; Signal Det ##ec ##tion Ana ##ly ##ses ; S ##IG ##NA ##L DE ##TE ##CT ##ION ; Det ##ec ##tions , Psychological Signal ; Psychological Signal Det ##ec ##tion ; Signal Det ##ec ##tion Theory ; Theo ##ries , Signal Det ##ec ##tion ; Signal Det ##ec ##tions ; Ana ##ly ##ses , Signal Det ##ec ##tion ; Det ##ec ##tion , [SEP]',
  'our_bi_cui': 'C1511790',
  'our_bi_cui_name': 'Detection',
  'our_bi_cui_desc': '[CLS] Det ##ec ##tion [unused3] ( Health Care Act ##ivity : Det ##ec ##ted ) [SEP]',
  'our_cross_cui': 'C0442726',
  'our_cross_cui_name': 'Detected',
  'our_cross_cui_desc': '[CLS] Det ##ec ##ted [unused3] ( Finding , finding , finding , qualifier value : detected ; detect ) [SEP]'},
 {'mention_id': '28167893.15',
  'test_set_idx': 27456,
  'type': 'T058',
  'name': 'cultured',
  'context': '[CLS] ##a imp ##ede the practical control of the Alt ##ern ##aria diseases in crop fields . This study aimed to investigate c ##yt ##ological fungi ##cid ##e resistance mechanisms of A . alter ##nat ##a against di ##car ##box ##im ##ide fungi ##cid ##e i ##p ##rod ##ione . A . alter ##nat ##a isolated from ca ##ctus brown spot was [unused1] culture ##d [unused2] on potato - de ##x ##tro ##se a ##gar with or without i ##p ##rod ##ione , and the fun ##gal cultures with different growth characteristics from no , initial and full growth were observed by light and electron micro ##sco ##py . My ##cel ##ia began to grow from one day after in ##cu ##bation and continued to be in [SEP]',
  'existing_cui': 'C0007600',
  'existing_cui_name': 'Cultured Cell Line',
  'existing_cui_desc': '[CLS] Culture ##d Cell Line [unused3] ( Ana ##tom ##ical Structure : Cell lines ; cells lines ; culture ##d cell line ; Line , Cell ; Cell Line ; cells lining ; culture ##d cells ; cell line ; cell lining ; Cell Lines ; cell lines ; cells line ; Lines , Cell ) [SEP]',
  'our_bi_cui': 'C0430400',
  'our_bi_cui_name': 'Laboratory culture',
  'our_bi_cui_desc': '[CLS] Laboratory culture [unused3] ( Health Care Act ##ivity , lab test , procedure : Culture ; Culture - general ; culture ; Sam ##ple culture ; laboratory culture ; sample culture ; culture from any source ; a culture from any source ) [SEP]',
  'our_cross_cui': 'C0200954',
  'our_cross_cui_name': 'Mycology culture',
  'our_cross_cui_desc': '[CLS] My ##cology culture [unused3] ( Health Care Act ##ivity , lab test , procedure : culture fungus ; Culture for fungi ; fun ##gal culture ; my ##cology culture ; Fun ##gus culture ; fungus culture ; fungi culture ) [SEP]'},
 {'mention_id': '27876145.17',
  'test_set_idx': 20326,
  'type': 'T062',
  'name': 'amplification strategies',
  'context': '[CLS] ##ay let - 7 ##a micro ##RNA I ##maging of micro ##RNA in living cells could facilitate the monitoring of the expression and distribution of micro ##RNA and research on micro ##RNA - related diseases . Given the low expression levels and even down - regulation of cellular micro ##RNA that is associated with some diseases , enzyme - free [unused1] am ##p ##lification strategies [unused2] are imp ##erative for in ##tra ##cellular micro ##RNA ass ##ay . In this work , we report an en ##tropy - driven reaction for am ##p ##lification ass ##ay micro ##RNA with a detection limit of 0 . 27 p ##M . The resulting signal am ##p ##lification provides excellent recognition and signal enhance ##ment of specific micro ##RNA [SEP]',
  'existing_cui': 'C1705759',
  'existing_cui_name': 'Gene Amplification Abnormality',
  'existing_cui_desc': '[CLS] Gene Am ##p ##lification A ##b ##nor ##mal ##ity [unused3] ( B ##iol ##og ##ic Fun ##ction : Am ##p ##lification ; Gene Am ##p ##lification ) [SEP]',
  'our_bi_cui': 'C1517480',
  'our_bi_cui_name': 'Gene Amplification Technique',
  'our_bi_cui_desc': '[CLS] Gene Am ##p ##lification Tech ##ni ##que [unused3] ( Research Act ##ivity : Am ##p ##lification ) [SEP]',
  'our_cross_cui': 'C0887815',
  'our_cross_cui_name': 'RNA Amplification Techniques',
  'our_cross_cui_desc': '[CLS] RNA Am ##p ##lification Tech ##ni ##ques [unused3] ( Research Act ##ivity : Am ##p ##lification Tech ##nics , RNA ; Am ##p ##lification Tech ##ni ##que , RNA ; RNA Am ##p ##lification Tech ##nic ; Am ##p ##lification Tech ##nic , RNA ; Tech ##nic , RNA Am ##p ##lification ; RNA Am ##p ##lification Tech ##nics ; Am ##p ##lification Tech ##ni ##ques , RNA ; Tech ##ni ##que , RNA Am ##p ##lification ; Tech ##nics , RNA Am ##p ##lification ; Tech ##ni ##ques , RNA Am ##p ##lification ; RNA Am ##p ##lification Tech ##ni ##que ) [SEP]'},
 {'mention_id': '28168514.36',
  'test_set_idx': 27551,
  'type': 'T058',
  'name': 'third - generation dual - source CT',
  'context': '[CLS] PP ##V : 86 . 0 % ; N ##P ##V : 99 . 0 % ) provided comparable diagnostic accuracy ( p = n . s . ) . SS ##M yielded significantly lower radiation doses as compared to he ##lica ##l scan mode ( 2 . 1 ± 2 . 0 vs . 5 . 1 ± 3 . 3 m ##S ##v ; p = 0 . 000 ##1 ) in age and B ##MI - matched co ##hor ##ts . SS ##M in [unused1] third - generation dual - source CT [unused2] enables significant dose savings and provides robust diagnostic image quality in patients with A ##gat ##ston score ≤ 2000 independent of heart rate , heart rhythm or o ##besity . [SEP]',
  'existing_cui': 'C0449416',
  'existing_cui_name': 'Source',
  'existing_cui_desc': '[CLS] Source [unused3] ( Finding , attribute : source ; sources ) [SEP]',
  'our_bi_cui': 'C4055114',
  'our_bi_cui_name': 'Dual energy computed tomography',
  'our_bi_cui_desc': '[CLS] Dual energy com ##puted to ##mo ##graphy [unused3] ( Health Care Act ##ivity , procedure : Dual - Energy Co ##mp ##uted Tom ##ography ; DE ##CT - dual energy com ##puted to ##mo ##graphy ) [SEP]',
  'our_cross_cui': 'C0040405',
  'our_cross_cui_name': 'X-Ray Computed Tomography',
  'our_cross_cui_desc': '[CLS] X - Ray Co ##mp ##uted Tom ##ography [unused3] ( Health Care Act ##ivity , computer ##ized to ##mo ##graphy , procedure : Sc ##an , X - Ray CT ; com ##puted to ##mo ##graphy x rays ; EMI scan ; em ##i scan ; CT X ##RA ##Y ; Co ##mp ##uted X - Ray Tom ##ography ; Computer ##ised trans ##ax ##ial to ##mo ##graphy ; to ##mo ##den ##si ##tom ##et ##ry ; CA ##T scan - ; CT Sc ##an , X Ray ; Computer ##ized to ##mo ##gram ; X - Ray CA ##T Sc ##an ; Computer ##ized trans ##ax ##ial to ##mo ##graphy ; x - ray com ##puted to ##mo ##graphy ; Tom ##ography , Trans ##mission [SEP]'},
 {'mention_id': '27538372.8',
  'test_set_idx': 9789,
  'type': 'T103',
  'name': 'mitochondrial regulators',
  'context': '[CLS] ##hos ##pha ##tase 1 regulate ##s the early differentiation of card ##io ##my ##ocytes from mouse em ##b ##ryo ##nic stem cells Mi ##to ##cho ##nd ##ria are crucial for maintaining the properties of em ##b ##ryo ##nic stem cells and for regulating their subsequent differentiation into diverse cell lineage ##s , including card ##io ##my ##ocytes . However , [unused1] mitochondrial regulator ##s [unused2] that manage the rate of differentiation or cell fate have been rarely identified . This study aimed to determine the potential mitochondrial factor that controls the differentiation of em ##b ##ryo ##nic stem cells into cardiac my ##ocytes . We induced card ##io ##my ##oc ##yte differentiation from mouse em ##b ##ryo ##nic stem cells and performed micro ##arra ##y ass [SEP]',
  'existing_cui': 'C0026237',
  'existing_cui_name': 'Mitochondria',
  'existing_cui_desc': '[CLS] Mi ##to ##cho ##nd ##ria [unused3] ( Ana ##tom ##ical Structure , body structure , cell structure : mit ##och ##ond ##ria ; Mi ##to ##cho ##nd ##rial structure ; Mi ##to ##cho ##nd ##rion ; mit ##och ##ond ##rion ; Mi ##to ##cho ##nd ##rial ; mitochondrial ) [SEP]',
  'our_bi_cui': 'C0949610',
  'our_bi_cui_name': 'Mitochondrial Proteins',
  'our_bi_cui_desc': '[CLS] Mi ##to ##cho ##nd ##rial Pro ##tein ##s [unused3] ( Chemical : mitochondrial protein ; Pro ##tein , Mi ##to ##cho ##nd ##rial ; mitochondrial proteins ; Mi ##to ##cho ##nd ##rial Pro ##tein ; Mi ##to ##cho ##nd ##rial Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; Pro ##tein ##s , Mi ##to ##cho ##nd ##rial ) [SEP]',
  'our_cross_cui': 'C0574031',
  'our_cross_cui_name': 'Biologically active substance',
  'our_cross_cui_desc': '[CLS] Biological ##ly active substance [unused3] ( Chemical , substance : Biological ##ly Active Sub ##stance ) [SEP]'},
 {'mention_id': '27355340.29',
  'test_set_idx': 3467,
  'type': 'T082',
  'name': 'plots',
  'context': '[CLS] pH , and near infrared absorb ##ance , were collected in real - time in a field with two management strategies ( reduced till ##age / conventional till ##age ) and sandy to lo ##am soils . pro ##ximal soil sensing was related to observations from a long - term ( 11 years ) earth ##worm observation study conducted at 42 [unused1] plots [unused2] . Earth ##worm ##s were sampled from 0 . 5 x 0 . 5 x 0 . 2 m ³ soil blocks and identified to species level . Sen ##sor data were highly correlated with earth ##worm abundance ##s observed in reduced till ##age but less correlated with earth ##worm abundance ##s observed in conventional till ##age . This may indicate that [SEP]',
  'existing_cui': 'C0162435',
  'existing_cui_name': 'Olfactory tract',
  'existing_cui_desc': '[CLS] O ##lf ##actor ##y tract [unused3] ( Ana ##tom ##ical Structure , body structure : T ##rac ##ts , O ##lf ##actor ##y ; T ##rac ##ts , Later ##al O ##lf ##actor ##y ; O ##lf ##actor ##y T ##rac ##ts ; O ##lf ##actor ##y tract structure ; St ##ria o ##lf ##actor ##ia lateral ##is ; O ##lf ##actor ##y T ##rac ##t ; T ##rac ##tus o ##lf ##actor ##ius ; Later ##al o ##lf ##actor ##y s ##tri ##ae ; T ##rac ##tus o ##lf ##actor ##ius lateral ##is ; o ##lf ##actor ##y tract ; O ##lf ##actor ##y T ##rac ##ts , Later ##al ; O ##lf ##actor ##y T ##rac ##t , Later ##al ; Later ##al O [SEP]',
  'our_bi_cui': 'C0032098',
  'our_bi_cui_name': 'Plants',
  'our_bi_cui_desc': '[CLS] Plants [unused3] ( E ##uka ##ryo ##te , organism : Ch ##lor ##op ##hy ##ta / Em ##b ##ryo ##phy ##ta group ; green plants ; Ch ##lor ##ob ##ion ##ta ; Ch ##lor ##ob ##ion ##ta B ##rem ##er , 1985 ; V ##iri ##di ##p ##lant ##ae ; Plants , General ; Kingdom Plant ##ae ; V ##iri ##di ##p ##lant ##ae C ##ava ##lier - Smith , 1981 ; plants ; Kingdom V ##iri ##di ##p ##lant ##ae ; plant ; ch ##lor ##op ##hy ##te / em ##b ##ryo ##phy ##te group ; Plant ) [SEP]',
  'our_cross_cui': 'C0562975',
  'our_cross_cui_name': 'Agricultural field',
  'our_cross_cui_desc': '[CLS] Agricultural field [unused3] ( Spa ##tial Concept , environment ) [SEP]'},
 {'mention_id': '28351424.19',
  'test_set_idx': 32839,
  'type': 'T170',
  'name': '( CENTRAL )',
  'context': '[CLS] predict ##ive of I ##CU admission . We describe the protocol for a systematic review of variables associated with I ##CU admission . ME ##D ##L ##IN ##E , E ##MB ##AS ##E , C ##IN ##A ##HL and the Cochrane Library , including Cochrane Database of System ##atic Reviews and the Cochrane Central Register of Control ##led Trials [unused1] ( CE ##NT ##RA ##L ) [unused2] will be searched for studies that assess the association of routinely recorded variables associated with subsequent un ##p ##lan ##ned I ##CU admission . Only studies involving adult patients admitted to general intensive care units will be included . We will extract data relating to the statistical association between I ##CU admission and predict ##or variables , the quality [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0242356',
  'our_bi_cui_name': 'Databases',
  'our_bi_cui_desc': '[CLS] Database ##s [unused3] ( Intel ##lect ##ual Product : database ; Data Bank ; Database ; data ##bank ; data base ; Database , Electronic ; databases ; data bank ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28223077.28',
  'test_set_idx': 28912,
  'type': 'T082',
  'name': 'surrounding sites',
  'context': '[CLS] from surrounding sites , background site , up ##wind site and down ##wind site of a Municipal solid waste land ##fill in East China . The results of present study were summarized as follows . ( 1 ) The total concentrations of PC ##D ##D / F ##s ( [UNK] PC ##D ##D / F ##s ) in ambient air from [unused1] surrounding sites [unused2] , background site , up ##wind site and down ##wind site were 2 . 215 ± 1 . 00 ##4 , 2 . 05 ##8 ± 0 . 45 ##8 , 2 . 61 ##7 ± 1 . 09 ##2 and 1 . 82 ##2 ± 0 . 56 ##6 p ##g ##N ##m ( - 3 ) , respectively . [SEP]',
  'existing_cui': 'C1282914',
  'existing_cui_name': 'Surrounding',
  'existing_cui_desc': '[CLS] Sur ##rou ##nding [unused3] ( Spa ##tial Concept , qualifier value , qualifier value : C ##ir ##cum ##scribed ; Sur ##rou ##nd ; c ##ir ##cum ##scribed ; surrounding ; surrounded ; Sur ##rou ##nded ; surroundings ; surrounds ; surround ) [SEP]',
  'our_bi_cui': 'C1282914',
  'our_bi_cui_name': 'Surrounding',
  'our_bi_cui_desc': '[CLS] Sur ##rou ##nding [unused3] ( Spa ##tial Concept , qualifier value , qualifier value : C ##ir ##cum ##scribed ; Sur ##rou ##nd ; c ##ir ##cum ##scribed ; surrounding ; surrounded ; Sur ##rou ##nded ; surroundings ; surrounds ; surround ) [SEP]',
  'our_cross_cui': 'C0205145',
  'our_cross_cui_name': 'Site',
  'our_cross_cui_desc': '[CLS] Site [unused3] ( Spa ##tial Concept , attribute , qualifier value : Sites ) [SEP]'},
 {'mention_id': '27320863.13',
  'test_set_idx': 2314,
  'type': 'T168',
  'name': 'methionine and choline - deficient diet',
  'context': '[CLS] ##mo ##cy ##stein ##e met ##hyl ##tra ##ns ##fer ##ase ( beta ##ine - ho ##mo ##cy ##stein ##e met ##hyl ##tra ##ns ##fe ##ase ) We investigated the anti - lip ##ogenic effect of beta ##ine in rats fed met ##hi ##oni ##ne and ch ##olin ##e - def ##icient diet . In ##take of [unused1] met ##hi ##oni ##ne and ch ##olin ##e - def ##icient diet [unused2] for 3 w ##k resulted in a significant accumulation of he ##pa ##tic lip ##ids , which was prevented by beta ##ine supplement ##ation in drinking water ( 1 % ) . Ph ##os ##ph ##ory ##lation of AM ##P - activated protein kinase , ace ##ty ##l - Co ##A car ##box ##yla ##se , [SEP]',
  'existing_cui': 'C2928853',
  'existing_cui_name': 'Choline / Methionine',
  'existing_cui_desc': '[CLS] Cho ##line / Met ##hi ##oni ##ne [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0452278',
  'our_bi_cui_name': 'Low methionine diet',
  'our_bi_cui_desc': '[CLS] Low met ##hi ##oni ##ne diet [unused3] ( Health Care Act ##ivity , finding , regime / therapy : Dec ##rease in met ##hi ##oni ##ne content of the diet compared to the assessed base ##line intake of met ##hi ##oni ##ne for the individual . ( Source : Academy of N ##utrition and Diet ##etics N ##utrition In ##formatics Committee 2013 ) ; Dec ##reased met ##hi ##oni ##ne diet ) [SEP]',
  'our_cross_cui': 'C0012155',
  'our_cross_cui_name': 'Diet',
  'our_cross_cui_desc': '[CLS] Diet [unused3] ( Food , finding : Diet ##s ; regular diet ? ; Diet ##ary ; diet ; Diet ##ary finding ; diet ##s ) [SEP]'},
 {'mention_id': '27710934.13',
  'test_set_idx': 14824,
  'type': 'T082',
  'name': 'PCP',
  'context': '[CLS] ##ing domains have been co - opted into Non ##ri ##bos ##oma ##l p ##eptide s ##ynth ##eta ##ses to introduce further variety into non ##ri ##bos ##oma ##l p ##eptide products . Line ##ar gram ##ici ##din s ##ynth ##eta ##se contains a unique form ##yla ##tion - tail ##oring domain in its initiation module ( F - A - [unused1] PC ##P [unused2] ) . The structure of the F - A di - domain has previously been determined in a crystal form which had large solvent channels and no density for the minor As ##ub sub ##dom ##ain . An attempt was made to take advantage of this packing by removing the As ##ub sub ##dom ##ain from the construct ( F - [SEP]',
  'existing_cui': 'C1418873',
  'existing_cui_name': 'PRCP gene',
  'existing_cui_desc': '[CLS] PR ##CP gene [unused3] ( Ana ##tom ##ical Structure : H ##UM ##PC ##P ; AN ##GI ##OT ##EN ##SI ##NA ##SE C ; PR ##OL ##Y ##LC ##AR ##BO ##X ##YP ##EP ##TI ##DA ##SE ; an ##gio ##tens ##inas ##e C ; PC ##P ; pro ##ly ##l ##car ##box ##ype ##pt ##idas ##e ) [SEP]',
  'our_bi_cui': 'C0030956',
  'our_bi_cui_name': 'Peptides',
  'our_bi_cui_desc': '[CLS] P ##eptide ##s [unused3] ( Chemical , substance : p ##eptide ; p ##eptide ##s ; P ##eptide ##s [ Chemical / In ##g ##red ##ient ] ; P ##eptide ) [SEP]',
  'our_cross_cui': 'C1514562',
  'our_cross_cui_name': 'Protein Domain',
  'our_cross_cui_desc': '[CLS] Pro ##tein Domain [unused3] ( Spa ##tial Concept : Domain ; Mo ##ti ##f ; Region ; Domain ##s , P ##eptide ; Domain , Pro ##tein ; Domain ##s , Pro ##tein ; domain ; Pro ##tein Feature ; Pro ##tein Domain ##s ; Feature ; Pro ##tein Mo ##ti ##f ; Sign ##ature ; Domain , P ##eptide ; Pro ##tein Region ; P ##eptide Domain ; P ##eptide Domain ##s ) [SEP]'},
 {'mention_id': '27506129.11',
  'test_set_idx': 8643,
  'type': 'T170',
  'name': 'automatic framework',
  'context': '[CLS] time consuming , thus creating a need for its auto ##mat ##isation . However , the task is far from straightforward due to data issues such as class im ##bal ##ance or correlated features . In this paper , we study the specific features of h ##yper ##emia and propose various approaches to address these problems in the context of an [unused1] automatic framework [unused2] for h ##yper ##emia g ##rading . Over ##sa ##mp ##ling , under ##sa ##mp ##ling and SM ##OT ##E approaches were applied in order to tackle the problem of class im ##bal ##ance . 25 features were com ##puted for each image and re ##gression methods were then used to transform them into a value on the g ##rading scale [SEP]',
  'existing_cui': 'C0450152',
  'existing_cui_name': 'Metal framework',
  'existing_cui_desc': '[CLS] Metal framework [unused3] ( Medical Devi ##ce , physical object ) [SEP]',
  'our_bi_cui': 'C0002045',
  'our_bi_cui_name': 'algorithm',
  'our_bi_cui_desc': '[CLS] algorithm [unused3] ( Intel ##lect ##ual Product : Al ##gor ##ith ##m ; Al ##gor ##ith ##ms ; algorithms ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27474969.2',
  'test_set_idx': 7419,
  'type': 'T103',
  'name': 'reduced graphene oxide',
  'context': '[CLS] An ultra ##sen ##sitive electro ##che ##mi ##lum ##ines ##cence sensor based on [unused1] reduced graph ##ene oxide [unused2] - copper su ##lf ##ide composite coupled with cap ##illa ##ry electro ##ph ##ores ##is for determination of am ##lo ##di ##pine be ##sy ##late in mice plasma A new electro ##che ##mi ##lum ##ines ##cence sensor based on reduced graph ##ene oxide - copper su ##lf ##ide ( r ##G ##O - C ##u ##S ) composite coupled with cap ##illa ##ry electro ##ph ##ores ##is was constructed for the ultra ##sen ##sitive detection of am ##lo ##di ##pine be ##sy ##late for the first time . In this work , r ##G ##O - C ##u ##S composite was synthesized by one - pot h [SEP]',
  'existing_cui': 'C1957670',
  'existing_cui_name': 'oxidosqualene',
  'existing_cui_desc': '[CLS] o ##xi ##dos ##qua ##lene [unused3] ( Chemical : sq ##us ##lene oxide ) [SEP]',
  'our_bi_cui': 'C0060240',
  'our_bi_cui_name': 'ferric oxide',
  'our_bi_cui_desc': '[CLS] f ##er ##ric oxide [unused3] ( Chemical , III , iron oxide , red , substance : Map ##ico Red ; F ##ER ##RI ##C O ##X ##ID ##E , R ##ED ; Iron oxide ; Iron O ##xi ##de Red ; iron oxide ; f ##er ##ric oxide [ Chemical / In ##g ##red ##ient ] ; Natural Red O ##xi ##de ; red iron oxide ; F ##ER ##RI ##C O ##X ##ID ##E ; f ##er ##ric oxide red ; Red O ##ch ##re ; iron oxide red ; Indian Red ; Di ##iro ##n Trio ##xi ##de ; co ##l ##cot ##har ; Iron O ##xi ##de , Red ; R ##ubi ##go ; Stone Red ; Col ##cot ##har ; I [SEP]',
  'our_cross_cui': 'C2936695',
  'our_cross_cui_name': 'Graphene',
  'our_cross_cui_desc': '[CLS] G ##rap ##hen ##e [unused3] ( Chemical ) [SEP]'},
 {'mention_id': '28407355.22',
  'test_set_idx': 34806,
  'type': 'T017',
  'name': 'EV',
  'context': '[CLS] ##ative im ##mu ##no ##mo ##du ##lator ##y effects in injured tissues . Recent studies have highlighted beneficial effects of extra ##cellular ve ##si ##cles from MS ##Cs ( Me ##sen ##chy ##mal stem cells - E ##V ) on tissue injury . The effects of systemic ##ally administered mouse bone ma ##rrow derived Me ##sen ##chy ##mal stem cells - [unused1] E ##V [unused2] were evaluated in an experimental m ##uri ##ne model of he ##pa ##tic is ##che ##mia - re ##per ##fusion injury induced by cross c ##lamp ##ing the he ##pa ##tic artery and portal vein for 90 minutes followed by re ##per ##fusion for periods of up ##to 6 hours . Compared with controls , in ##tra ##ven ##ous administration of [SEP]',
  'existing_cui': 'C0014522',
  'existing_cui_name': 'Epidermodysplasia Verruciformis',
  'existing_cui_desc': '[CLS] E ##pid ##er ##mo ##dy ##sp ##lasia V ##er ##ru ##ci ##form ##is [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : e ##pid ##er ##mo ##dy ##sp ##lasia ve ##rr ##uc ##iform ##is ; Le ##wan ##do ##ws ##ky - Lu ##tz D ##ys ##p ##lasia ; Le ##wan ##do ##ws ##ky - Lu ##tz disease ; Lu ##tz - Le ##wan ##do ##ws ##ky Disease ; Lu ##tz Le ##wan ##do ##ws ##ky Disease ; L ##UT ##Z L ##E ##WA ##ND ##OW ##S ##K ##Y D ##IS ; E ##pid ##er ##mo ##dy ##sp ##lasia V ##er ##ru ##ci ##form ##is [ Disease / Finding ] ; Disease , Lu ##tz - Le ##wan ##do ##ws ##ky ; EP ##ID ##ER [SEP]',
  'our_bi_cui': 'C2350307',
  'our_bi_cui_name': 'Cell-Derived Microparticles',
  'our_bi_cui_desc': '[CLS] Cell - Der ##ive ##d Micro ##par ##tic ##les [unused3] ( Ana ##tom ##ical Structure : Me ##mb ##rane Micro ##par ##tic ##le , Cell ; e ##ct ##oso ##me ; E ##ct ##oso ##me ; She ##dding Micro ##ves ##icles ; Micro ##ves ##icle ; Micro ##ves ##icle , She ##dding ; extra ##cellular micro ##ves ##icle ; Me ##mb ##rane Micro ##par ##tic ##les , Cell ; micro ##ves ##icle ; She ##dding Micro ##ves ##icle ; E ##ct ##oso ##mes ; Micro ##par ##tic ##le , Cell - Der ##ive ##d ; Cell Me ##mb ##rane Micro ##par ##tic ##les ; Micro ##par ##tic ##le ; Cell Der ##ive ##d Micro ##par ##tic ##les ; Micro ##par ##tic ##les , Cell Der ##ive [SEP]',
  'our_cross_cui': 'C3894683',
  'our_cross_cui_name': 'Extracellular Vesicles',
  'our_cross_cui_desc': '[CLS] Extra ##cellular V ##esi ##cles [unused3] ( Ana ##tom ##ical Structure : V ##esi ##cles , Extra ##cellular ; Extra ##cellular V ##esi ##cle ; V ##esi ##cle , Extra ##cellular ; extra ##cellular ve ##si ##cle ) [SEP]'},
 {'mention_id': '27320863.51',
  'test_set_idx': 2352,
  'type': 'T103',
  'name': 'inhibitor',
  'context': '[CLS] - 1 ##c and p ##L ##K ##B ##1 in He ##p ##G ##2 cells . Met ##form ##in - induced activation of AM ##P - activated protein kinase was also in ##hibit ##ed by ho ##mo ##cy ##stein ##e . Treatment with h ##ydro ##xy ##lam ##ine , a c ##ys ##tat ##hi ##oni ##ne β - s ##ynth ##ase [unused1] inhibitor [unused2] , resulted in a reduction of p ##AM ##P ##K , p ##AC ##C and p ##SR ##E ##B ##P - 1 ##c , accompanied by an elevation of in ##tra ##cellular ho ##mo ##cy ##stein ##e . Beta ##ine treatment prevented the ho ##mo ##cy ##stein ##e - induced reduction of p ##AM ##P ##K , p ##AC ##C , p [SEP]',
  'existing_cui': 'C0243077',
  'existing_cui_name': 'inhibitors',
  'existing_cui_desc': '[CLS] inhibitor ##s [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0243077',
  'our_bi_cui_name': 'inhibitors',
  'our_bi_cui_desc': '[CLS] inhibitor ##s [unused3] ( Chemical ) [SEP]',
  'our_cross_cui': 'C0014432',
  'our_cross_cui_name': 'Enzyme Inhibitors',
  'our_cross_cui_desc': '[CLS] En ##zy ##me In ##hibit ##ors [unused3] ( Chemical , product , substance : E ##N ##Z ##Y ##ME IN ##H ##IB ; enzyme inhibitor ; enzyme inhibitor ##s ; En ##zy ##me inhibitor ##s ; In ##hibit ##ors , En ##zy ##me ; enzymes inhibitor ##s ; En ##zy ##me inhibitor ) [SEP]'},
 {'mention_id': '27791362.25',
  'test_set_idx': 16940,
  'type': 'T103',
  'name': 'W',
  'context': '[CLS] na ##no ##par ##tic ##les ( 170 ± 10 nm h ##ydro ##dynamic diameter ) produce a higher com ##puted to ##mo ##graphy contrast ( by a factor of about 2 ) than commercial i ##od ##ine - based radio ##con ##tras ##t agents ( e . g . , I ##oh ##ex ##ol ) at identical m ##olar concentrations of [unused1] W [unused2] or I atoms . P ##EG - P ##LA - coated C ##a ##W ##O ##4 na ##no ##par ##tic ##les are chemical ##ly stable and completely non ##to ##xi ##c . It was confirmed that the maximum tolerate ##d dose of this material in mice is significantly higher ( 250 ± 50 mg per kg body weight following a single in [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0441856',
  'our_bi_cui_name': 'Group W',
  'our_bi_cui_desc': '[CLS] Group W [unused3] ( Intel ##lect ##ual Product , qualifier value ) [SEP]',
  'our_cross_cui': 'C0041383',
  'our_cross_cui_name': 'tungsten',
  'our_cross_cui_desc': '[CLS] t ##ung ##sten [unused3] ( Chemical , substance : wolf ##ram ; Wolf ##ram ; Tu ##ng ##sten [ Chemical / In ##g ##red ##ient ] ; W element ; T ##UN ##GS ##TE ##N ; Tu ##ng ##sten ) [SEP]'},
 {'mention_id': '27310615.57',
  'test_set_idx': 2125,
  'type': 'T103',
  'name': 'micro',
  'context': "[CLS] that 56 % ( n = 31 ) of patients were over ##weight at these three different times . Weight , B ##MI and W ##ais ##t C ##ir ##cum ##ference increased significantly , indicating a worse nutrition ##al status , and there was a correlation between poor diet quality and higher values for B ##MI , W ##ais ##t - Hip Rat ##io and W ##ais ##t - to - Height Rat ##io . Ch ##em ##otherapy interfere ##s in the patients ' diet generating a negative impact on the quality and intake of [unused1] micro [unused2] and mac ##ron ##ut ##rient ##s , as well as an impact on their nutrition ##al status , with an increase in ant ##hr ##op ##ometric measurements . [SEP]",
  'existing_cui': 'C0085672',
  'existing_cui_name': 'Microbiology procedure',
  'existing_cui_desc': '[CLS] Micro ##biology procedure [unused3] ( Health Care Act ##ivity , lab test , procedure : micro ##biology test ; Micro ##biology Test ; Micro ##biology ; Micro ##biology Pro ##ced ##ure ; Laboratory Testing , Micro ##biology ; UN ##L ##IS ##TE ##D MI ##CR ##O ##BI ##OL ##O ##G ##Y ; Micro ##biology procedures ; micro ##biology tests ; Un ##listed micro ##biology procedure ; MI ##CR ##O ##BI ##OL ##O ##G ##Y PR ##OC ##ED ##UR ##E ; Test ; micro ##biology ; micro ##biology ; Micro ##biology test ; MI ##CR ##O ; Micro ##biology Tests ; Micro ##biology Pro ##ced ##ures ; diagnostic service sources micro ##biology ) [SEP]',
  'our_bi_cui': 'C0678695',
  'our_bi_cui_name': 'Nutrients',
  'our_bi_cui_desc': '[CLS] N ##ut ##rient ##s [unused3] ( Food , substance : nutrients ; nut ##rient ; nutrients substances ; N ##ut ##rient ) [SEP]',
  'our_cross_cui': 'C0282575',
  'our_cross_cui_name': 'Micronutrients',
  'our_cross_cui_desc': '[CLS] Micro ##nut ##rient ##s [unused3] ( Chemical : Micro ##nut ##rient ; micro ##nut ##rient ; micro ##nut ##rient ##s ) [SEP]'},
 {'mention_id': '28239461.28',
  'test_set_idx': 29413,
  'type': 'T170',
  'name': 'Fluidigm platform',
  'context': '[CLS] 6 ) , di ##stal common bi ##le duct cancer ( n = 21 ) , and gas ##tric cancer ( n = 20 ) ; chronic pan ##cre ##ati ##tis ( n = 39 ) ; and normal pan ##cre ##as ( n = 35 ) . The micro ##RNA ##s were analyzed by PC ##R using the [unused1] F ##lu ##id ##ig ##m platform [unused2] . Twenty - two micro ##RNA ##s were significantly differently expressed in patients with pan ##cre ##atic cancer when compared to healthy controls and chronic pan ##cre ##ati ##tis patients ; 17 mi ##RNA ##s were up ##re ##gu ##lated ( mi ##R - 21 - 5 ##p , - 23 ##a - 3 ##p , - 31 - [SEP]',
  'existing_cui': 'C3896718',
  'existing_cui_name': 'MenaCalc Platform',
  'existing_cui_desc': '[CLS] Men ##a ##C ##al ##c Platform [unused3] ( Intel ##lect ##ual Product ) [SEP]',
  'our_bi_cui': 'C1449576',
  'our_bi_cui_name': 'Microfluidic Analytical Techniques',
  'our_bi_cui_desc': '[CLS] Micro ##f ##lu ##id ##ic Ana ##ly ##tical Tech ##ni ##ques [unused3] ( Health Care Act ##ivity : MI ##CR ##O ##FL ##UI ##DI ##C AN ##AL ; Ana ##ly ##ses , Micro ##f ##lu ##id ##ic ; Micro ##f ##lu ##id ##ic Ana ##ly ##tical Tech ##ni ##que ; Micro ##f ##lu ##id ##ic Ana ##ly ##ses ; Micro ##f ##lu ##id ##ic Analysis ; MI ##CR ##O ##FL ##UI ##DI ##C AN ##AL T ##EC ##H ##NI ##Q ##UE ##S ; Tech ##ni ##que , Micro ##f ##lu ##id ##ic Ana ##ly ##tical ; Tech ##ni ##ques , Micro ##f ##lu ##id ##ic Ana ##ly ##tical ; Analysis , Micro ##f ##lu ##id ##ic ; Ana ##ly ##tical Tech ##ni ##que , Micro ##f [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28493435.5',
  'test_set_idx': 37720,
  'type': 'T033',
  'name': 'locating',
  'context': '[CLS] A Fast Peak - Search ##ing Al ##gor ##ith ##m for Ultra ##sonic El ##ast ##ography T ##iss ##ue a ##xial strain est ##imation with ultra ##sound el ##ast ##ography has become a hot field in recent years . However , for key ##points tracking - based el ##ast ##ography algorithms , [unused1] lo ##cating [unused2] ex ##tre ##ma in multi ##mo ##dal ultra ##sonic radio ##f ##re ##que ##ncy signals is still a challenging problem . In this paper , a new method is proposed to locate the local ma ##xi ##ma and mini ##ma of the RF signals directly without der ##ivation operation . This algorithm can accurately locate ex ##tre ##ma even if disturbed peaks resulting from different noise exist . Furthermore , [SEP]',
  'existing_cui': 'C3874067',
  'existing_cui_name': 'Nerve-locating probe',
  'existing_cui_desc': '[CLS] N ##erve - lo ##cating probe [unused3] ( Medical Devi ##ce , physical object ) [SEP]',
  'our_bi_cui': 'C0392752',
  'our_bi_cui_name': 'Localized',
  'our_bi_cui_desc': '[CLS] Local ##ized [unused3] ( Spa ##tial Concept , qualifier value : Local ##ised ; Limited area ; local ##izing ; local ##ize ; localized ; local ##ised ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27603112.13',
  'test_set_idx': 11355,
  'type': 'T103',
  'name': 'biomolecules',
  'context': '[CLS] ##cule ##s : A Si ##mu ##lation Study A number of cases around the world have been reported where animals were found dead or dying with symptoms resembling a th ##iam ##ine ( vitamin B ) deficiency , and for some of these , a link to poll ##uta ##nts has been suggested . Here , we investigate whether [unused1] bio ##mo ##le ##cule ##s [unused2] involved in th ##iam ##in binding and transport could be blocked by a range of different poll ##uta ##nts . We used in si ##lic ##o dock ##ing of five compound classes ( 25 compounds in total ) to each of five targets ( p ##rion protein , EC ##F - type ABC transport ##er , th ##i - box [SEP]',
  'existing_cui': 'C0104263',
  'existing_cui_name': 'artificial lung expanding compound',
  'existing_cui_desc': '[CLS] artificial lung expanding compound [unused3] ( Chemical , AL ##EC , product , substance : P ##uma ##ct ##ant ; pu ##ma ##ct ##ant ) [SEP]',
  'our_bi_cui': 'C1449655',
  'our_bi_cui_name': 'Macromolecular Substances',
  'our_bi_cui_desc': '[CLS] Mac ##rom ##ole ##cular Sub ##stance ##s [unused3] ( Chemical : Mac ##rom ##ole ##cular Sub ##stance ##s [ Chemical / In ##g ##red ##ient ] ; Mac ##rom ##ole ##cular Co ##mpo ##unds and Complex ##es ; Sub ##stance ##s , Mac ##rom ##ole ##cular ) [SEP]',
  'our_cross_cui': 'C0574031',
  'our_cross_cui_name': 'Biologically active substance',
  'our_cross_cui_desc': '[CLS] Biological ##ly active substance [unused3] ( Chemical , substance : Biological ##ly Active Sub ##stance ) [SEP]'},
 {'mention_id': '28424240.41',
  'test_set_idx': 35329,
  'type': 'T038',
  'name': 'TCR stimulation',
  'context': '[CLS] protein 5 functions impaired T cell proliferation , IL - 2 production , and T ##h ##1 / T ##h ##17 , but not T ##h ##2 and regulatory T cell differentiation . general control non ##re ##pressed - protein 5 is recruited onto the il - 2 promoter by interacting with the N ##FA ##T in T cells upon [unused1] T ##CR stimulation [unused2] . Interest ##ingly , instead of directly ace ##ty ##lating N ##FA ##T , general control non ##re ##pressed - protein 5 cat ##aly ##zes his ##tone H ##3 l ##ys ##ine H ##9 ace ##ty ##lation to promote IL - 2 production . T cell - specific suppression of general control non ##re ##pressed - protein 5 partially protected mice [SEP]',
  'existing_cui': 'C1292856',
  'existing_cui_name': 'Stimulation procedure',
  'existing_cui_desc': '[CLS] St ##im ##ulation procedure [unused3] ( Health Care Act ##ivity , procedure , qualifier value : stimulation ; stimulation ##s ; St ##im ##ulation - action ; St ##im ##ulation ) [SEP]',
  'our_bi_cui': 'C1515144',
  'our_bi_cui_name': 'T-Cell Receptor Activation',
  'our_bi_cui_desc': '[CLS] T - Cell Re ##ceptor Act ##ivation [unused3] ( B ##iol ##og ##ic Fun ##ction : T ##CR Act ##ivation ; T ##CR Inter ##action ; T - Cell Re ##ceptor Inter ##action ) [SEP]',
  'our_cross_cui': 'C2259054',
  'our_cross_cui_name': 'stimulation of T cell receptor signaling pathway',
  'our_cross_cui_desc': '[CLS] stimulation of T cell receptor signaling pathway [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]'},
 {'mention_id': '27482869.2',
  'test_set_idx': 7895,
  'type': 'T204',
  'name': 'Allen James Wilcox',
  'context': "[CLS] Voices : A Con ##vers ##ation with Allen J . Wilcox [unused1] Allen James Wilcox [unused2] was born on 30 September 1946 in Columbus , OH . He studied medicine at the University of Michigan , graduated in 1973 , and after a rotating internship , he completed a master ' s degree in maternal and child health ( 1976 ) and a PhD in e ##pid ##em ##iology ( 1979 ) at the University of North Carolina in Chapel Hill . After graduation , he went to work at the National Institute of Environmental Health Sciences ( National Institute of Environmental Health Sciences , one of the US National Institutes of Health ) in Durham , NC , where he has spent his career . [SEP]",
  'existing_cui': 'C0237695',
  'existing_cui_name': 'William James',
  'existing_cui_desc': '[CLS] William James [unused3] ( E ##uka ##ryo ##te , William : James ) [SEP]',
  'our_bi_cui': 'C0027361',
  'our_bi_cui_name': 'Persons',
  'our_bi_cui_desc': '[CLS] Persons [unused3] ( Population Group , person : Individual ; Person ; persons ; peoples ; Person [ du ##p ] ; People ; person ; people ) [SEP]',
  'our_cross_cui': 'C0086418',
  'our_cross_cui_name': 'Homo sapiens',
  'our_cross_cui_desc': '[CLS] Ho ##mo sa ##pie ##ns [unused3] ( E ##uka ##ryo ##te , Tax ##onomy , organism , qualifier value : Humans ; Man , Modern ; humans ; man ; MA ##N ; Ho ##mo sa ##pie ##ns ( living organism ) [ Am ##bi ##guous ] ; ho ##mo sa ##pie ##ns ; Human , General ; Ho ##mo sa ##pie ##ns Linnaeus , 1758 ; Human ; Human - origin ; human ; Ho ##mo sa ##pie ##ns ( living organism ) ; Modern Man ; Man ) [SEP]'},
 {'mention_id': '28188533.10',
  'test_set_idx': 27855,
  'type': 'T170',
  'name': 'SWATH ( ® )',
  'context': '[CLS] ##c ##quisition Data for Label - free Q ##uant ##itative Pro ##te ##omi ##cs Studies Using Triple ##TO ##F ( ® ) Mass S ##pect ##rome ##ters Data - independent acquisition is a powerful mass s ##pect ##rome ##try technique that enables comprehensive MS and MS / MS analysis of all detect ##able species , providing an information rich data file that can be mined deeply . Here , we describe how to acquire high - quality [unused1] S ##WA ##TH ( ® ) [unused2] A ##c ##quisition data to be used for large quantitative pro ##te ##omi ##c studies . We specifically focus on using variable sized Q ##1 windows for acquisition of MS / MS data for generating higher specific ##ity quantitative data . [SEP]',
  'existing_cui': 'C1416521',
  'existing_cui_name': 'ITSN2 gene',
  'existing_cui_desc': '[CLS] IT ##SN ##2 gene [unused3] ( Ana ##tom ##ical Structure : PR ##O ##20 ##15 ; SH ##3 domain protein 1 ##B ; inter ##sect ##in 2 ; SH ##3 ##P ##18 ; IN ##TE ##RS ##EC ##TI ##N 2 ; SH ##3 ##P ##18 - like WA ##SP associated protein ; SH ##3 D ##OM ##A ##IN PR ##OT ##EI ##N 1 ##B ; K ##IA ##A ##12 ##5 ##6 ) [SEP]',
  'our_bi_cui': 'C0025080',
  'our_bi_cui_name': 'Medical Devices',
  'our_bi_cui_desc': '[CLS] Medical Devi ##ces [unused3] ( Medical Devi ##ce , physical device , physical object : Clinical equipment ; B ##io ##medical device ; DE ##VI ##CE ME ##D ; ME ##D DE ##VI ##CE ##S ; ME ##D DE ##VI ##CE ; bio ##medical devices ; Devi ##ces , Medical ; Medical devices ; General bio ##medical device ; DE ##VI ##CE , GE ##NE ##RA ##L ME ##DI ##CA ##L ; B ##io ##medical equipment ; Devi ##ce , Medical ; Hospital , medical AND / OR surgical equipment ( physical device ) ; bio ##medical equipment ; medical equipment ; Medical equipment ; DE ##VI ##CE ##S ME ##D ; DE ##VI ##CE ; DE ##VI ##CE ##S ; medical device ; medical devices [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27245335.53',
  'test_set_idx': 758,
  'type': 'T017',
  'name': 'adipocytokine genes',
  'context': '[CLS] by male offspring during the subsequent generation following paternal high - fat diet exposure diminished whereas consumption for two generations completely abolished the effect of paternal high - fat diet exposure on metabolic traits and ad ##ip ##oc ##yt ##oki ##ne promoter e ##pi ##gene ##tic changes in the offspring . The effects of paternal high - fat diet exposure on offspring were relatively weaker than those following high - fat diet exposure in u ##tero . However , paternal high - fat diet exposure had an add ##itive metabolic effect for two generations , suggesting that both paternal and maternal nutrition might affect offspring metabolism through e ##pi ##gene ##tic modifications of [unused1] ad ##ip ##oc ##yt ##oki ##ne genes [unused2] for multiple generations . [SEP]',
  'existing_cui': 'C1955907',
  'existing_cui_name': 'Adipokines',
  'existing_cui_desc': '[CLS] Ad ##ip ##oki ##nes [unused3] ( Chemical : Ad ##ip ##oki ##ne ; Ad ##ip ##oki ##nes [ Chemical / In ##g ##red ##ient ] ; Ad ##ip ##oc ##yt ##oki ##nes ; Ad ##ip ##oc ##yt ##oki ##ne ) [SEP]',
  'our_bi_cui': 'C1333196',
  'our_bi_cui_name': 'Cytokine Gene',
  'our_bi_cui_desc': '[CLS] Cy ##tok ##ine Gene [unused3] ( Ana ##tom ##ical Structure ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27837821.9',
  'test_set_idx': 18842,
  'type': 'T170',
  'name': 'fast Fourier transform',
  'context': '[CLS] based on the mat ##uring in barrels or the prediction of the global scores assigned by a sensory panel . To this aim , red wine samples were first anal ##ys ##ed with the vol ##tam ##metric sensor array , without performing any sample pre ##tre ##at ##ment . Afterwards , obtained responses were pre ##p ##ro ##cess ##ed employing [unused1] fast Four ##ier transform [unused2] for the compression and reduction of signal complexity , and obtained coefficients were then used as inputs to build the q ##ual ##itative and quantitative models employing either linear disc ##rim ##ina ##nt analysis or partial least squares re ##gression , respectively . Sa ##tis ##fa ##ctor ##y results were obtained overall , with a classification rate of 100 % [SEP]',
  'existing_cui': 'C0206055',
  'existing_cui_name': 'Spectroscopy , Fourier Transform Infrared',
  'existing_cui_desc': '[CLS] S ##pect ##ros ##copy , Four ##ier Trans ##form In ##fra ##red [unused3] ( Research Act ##ivity : Four ##ier Trans ##form In ##fra ##red S ##pect ##ros ##copy ; F ##TI ##R ( Four ##ier transform infrared ) s ##pect ##ros ##copy ; F ##TI ##R ; F ##TI ##R s ##pect ##ros ##copy ; four ##ier transform infrared ( F ##TI ##R ) s ##pect ##ros ##copy ; Four ##ier transform infrared s ##pect ##ros ##copy ; S ##pect ##ros ##copy , In ##fra ##red , Four ##ier Trans ##form ) [SEP]',
  'our_bi_cui': 'C3899091',
  'our_bi_cui_name': 'Fourier Transform Ion Cyclotron Resonance Mass Spectrometry',
  'our_bi_cui_desc': '[CLS] Four ##ier Trans ##form Ion Cy ##c ##lot ##ron Re ##son ##ance Mass S ##pect ##rome ##try [unused3] ( Research Act ##ivity ) [SEP]',
  'our_cross_cui': 'C0002045',
  'our_cross_cui_name': 'algorithm',
  'our_cross_cui_desc': '[CLS] algorithm [unused3] ( Intel ##lect ##ual Product : Al ##gor ##ith ##m ; Al ##gor ##ith ##ms ; algorithms ) [SEP]'},
 {'mention_id': '27659602.24',
  'test_set_idx': 13369,
  'type': 'T082',
  'name': 'root',
  'context': '[CLS] at post ##oper ##ative base ##line and follow - up imaging by com ##puted to ##mo ##graphy or echo ##card ##io ##graphy . The annual growth rate was evaluated and also compared according to the influence of valve morphology and con ##com ##ita ##nt a ##ort ##ic valve replacement . Furthermore , the years of natural history that would require for [unused1] root [unused2] en ##lar ##gement to meet a 50 mm threshold of the root diameter were calculated . The estimated growth rate of the a ##ort ##ic root after root - spa ##ring a ##ort ##ic replacement is between 0 . 27 and 0 . 51 mm per year ( mean 0 . 41 mm , varying according to the underlying diameter ) and [SEP]',
  'existing_cui': 'C1318154',
  'existing_cui_name': 'Root body part',
  'existing_cui_desc': '[CLS] Root body part [unused3] ( Ana ##tom ##ical Structure : Root ; root [ a body part ] ; root ) [SEP]',
  'our_bi_cui': 'C0040452',
  'our_bi_cui_name': 'Tooth root structure',
  'our_bi_cui_desc': '[CLS] Too ##th root structure [unused3] ( Ana ##tom ##ical Structure , body structure : Ra ##di ##x co ##rona ; Root of the Too ##th ; tooth root ; Too ##th Root ; Root of Too ##th ; Root , Too ##th ; tooth roots ; root of tooth ; root tooth ; roots tooth ; Te ##eth - - Roots ; Roots , Too ##th ; of tooth root ; Too ##th Roots ; Too ##th root ; Ra ##di ##x Den ##tis ; Ra ##di ##x den ##tis ; rooted tooth ; Root of tooth ) [SEP]',
  'our_cross_cui': 'C0549113',
  'our_cross_cui_name': 'Supraaortic valve area structure',
  'our_cross_cui_desc': '[CLS] Su ##pra ##ao ##rt ##ic valve area structure [unused3] ( Spa ##tial Concept , body structure : A ##ort ##ic bulb ; AS ##CE ##ND ##ING A ##OR ##TA , A ##OR ##TI ##C R ##O ##OT ; A ##ort ##a . root ; A ##ort ##ic root ; a ##ort ##ic root ; A ##ort ##ic Root ; Root of a ##ort ##a ; B ##ul ##b of a ##ort ##a ; Su ##pra ##ao ##rt ##ic valve area ; root a ##ort ##ic ; B ##ul ##b of ascending a ##ort ##a ; B ##ul ##bus a ##ort ##ae ) [SEP]'},
 {'mention_id': '27842161.11',
  'test_set_idx': 19104,
  'type': 'T103',
  'name': 'netarsudil-M1',
  'context': '[CLS] ##ars ##udi ##l Inc ##rease ##s Out ##flow Facility in Human Eyes Through Multiple Me ##chan ##isms Net ##ars ##udi ##l is a R ##ho kinase / nor ##ep ##ine ##ph ##rine transport ##er inhibitor currently in phase 3 clinical development for g ##lau ##com ##a treatment . We investigated the effects of its active meta ##bol ##ite , [unused1] net ##ars ##udi ##l - M1 [unused2] , on out ##flow facility ( C ) , out ##flow h ##ydro ##dynamic ##s , and morphology of the conventional out ##flow pathway in en ##uc ##lea ##ted human eyes . Pa ##ired human eyes ( n = 5 ) were per ##fused with either 0 . 3 μ ##M net ##ars ##udi ##l - M1 or vehicle [SEP]',
  'existing_cui': 'C0441971',
  'existing_cui_name': 'Metastasis stage M1',
  'existing_cui_desc': '[CLS] Met ##ast ##asis stage M1 [unused3] ( Finding , diagnosis , finding , tumor staging : M1 ; M1 Met ##ast ##asis Stage ; M1 Di ##stant Met ##ast ##asis Finding ; M1 T ##N ##M Finding ; M1 Cancer Stage Finding ; M1 Stage ; M1 category ; ma ##li ##g neo ##p ##las ##m t ##n ##m staging distant meta ##sta ##sis ( m ) m ##1 ; M1 Di ##stant Met ##ast ##asis Stage Finding ; M1 Met ##ast ##asis Finding ; Met ##ast ##asis Stage M1 ; M1 stage ; M1 Stage Finding ) [SEP]',
  'our_bi_cui': 'C4307684',
  'our_bi_cui_name': '6,7-dimethoxy-1,2,3,4-tetrahydroisoquinoline-3-carboxylic acid',
  'our_bi_cui_desc': '[CLS] 6 , 7 - dim ##eth ##ox ##y - 1 , 2 , 3 , 4 - te ##tra ##hy ##dr ##ois ##o ##quin ##olin ##e - 3 - car ##box ##yl ##ic acid [unused3] ( Chemical : M1 al ##kal ##oid ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27553395.14',
  'test_set_idx': 10264,
  'type': 'T033',
  'name': 'over',
  'context': '[CLS] was 103 ##2 . 8 ( SD 252 ) k ##cal and 101 ##3 ( SD 33 ##9 . 1 ) k ##cal respectively and 105 ##7 . 2 ( SD 201 . 8 ) k ##cal and 103 ##0 . 3 ( SD 34 ##1 . 7 ) k ##cal for boys . Iron intake was low in both groups . Mean body mass index and body weight and height were similar to World Health Organization figures but significant numbers of infants and to ##ddler ##s of both sexes were [unused1] over [unused2] - or under ##weight . Although mean energy and mac ##ron ##ut ##rient intake ##s were comparable to the R ##DA , significant numbers were over - or under ##fe ##d . [SEP]',
  'existing_cui': 'C0205136',
  'existing_cui_name': 'Over',
  'existing_cui_desc': '[CLS] Over [unused3] ( Spa ##tial Concept , spatial , spatial , qualifier value : over ) [SEP]',
  'our_bi_cui': 'C0205136',
  'our_bi_cui_name': 'Over',
  'our_bi_cui_desc': '[CLS] Over [unused3] ( Spa ##tial Concept , spatial , spatial , qualifier value : over ) [SEP]',
  'our_cross_cui': 'C0497406',
  'our_cross_cui_name': 'Overweight',
  'our_cross_cui_desc': '[CLS] Over ##weight [unused3] ( Finding , diagnosis , finding , physical finding : clinical over ##weight ; O ##VE ##R ##W ##EI ##G ##HT ; over ##weight ( B ##MI < 30 ) ; over ##weight ; Over ##weight ( B ##MI 25 - 30 ) ; Pat ##ient over ##weight ; Over ##weight [ Disease / Finding ] ) [SEP]'},
 {'mention_id': '27762482.15',
  'test_set_idx': 15893,
  'type': 'T074',
  'name': 'PDL',
  'context': '[CLS] ##nic flu ##ence proved to be superior until now . Given that pu ##rp ##ura and subsequent possible post ##in ##f ##lam ##mat ##ory h ##yper ##pi ##gment ##ation are occasionally un ##bearable in some patients , and several studies using the low non ##pur ##pura ##genic flu ##ence were reported . To deliver the sufficient high flu ##ence of a [unused1] PD ##L [unused2] without generating pu ##rp ##ura , we designed the fraction ##ation of high flu ##ence using five passes and longer pulse duration ( 6 mill ##ise ##con ##ds ) of a PD ##L in succession . A total of eight patients with er ##yt ##hem ##ato ##tel ##ang ##ie ##ct ##atic r ##osa ##cea were enrolled in this study ; all [SEP]',
  'existing_cui': 'C1414066',
  'existing_cui_name': 'DLG5 gene',
  'existing_cui_desc': '[CLS] DL ##G ##5 gene [unused3] ( Ana ##tom ##ical Structure : K ##IA ##A ##0 ##5 ##8 ##3 ; D ##IS ##CS LA ##R ##GE , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF , 5 ; P - d ##l ##g ; discs large ho ##mo ##log 5 ; P ##LA ##CE ##NT ##A AND PR ##OS ##TA ##TE DL ##G ) [SEP]',
  'our_bi_cui': 'C0031740',
  'our_bi_cui_name': 'Photochemotherapy',
  'our_bi_cui_desc': '[CLS] Photo ##che ##mother ##ap ##y [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : PD ##T - Photo ##dynamic therapy ; Photo ##che ##mother ##ap ##ies ; Photo ##dynamic The ##rap ##ies ; therapy , photo ##dynamic ; Therapy , Photo ##dynamic ; THE ##R P ##H ##OT ##OD ##Y ##NA ##MI ##C ; Photo ##rad ##iation Therapy ; photo ##dynamic therapy ; photo ##che ##mother ##ap ##y ; P ##H ##OT ##OC ##H ##EM ##OT ##H ##ER ; The ##rap ##ies , Photo ##dynamic ; Photo ##dynamic Therapy Pro ##ced ##ures ; neo ##p ##las ##m / cancer photo ##rad ##iation therapy ; Photo ##che ##mother ##ap ##y [ Am ##bi ##guous ] ; Photo ##dynamic therapy ; PD [SEP]',
  'our_cross_cui': 'C1289859',
  'our_cross_cui_name': 'Pulsed Dye Lasers',
  'our_cross_cui_desc': '[CLS] P ##ulse ##d D ##ye Las ##ers [unused3] ( Medical Devi ##ce , physical object : D ##ye Las ##ers , P ##ulse ##d ; P ##ulse ##d dye laser device ; Las ##er , P ##ulse ##d D ##ye ; D ##ye Las ##er , P ##ulse ##d ; Las ##ers , P ##ulse ##d D ##ye ; P ##ulse ##d D ##ye Las ##er ; pulsed dye laser ; P ##ulse ##d dye laser ) [SEP]'},
 {'mention_id': '27298278.30',
  'test_set_idx': 1831,
  'type': 'T103',
  'name': "dimethyl 1,1 ' - dihydroxy-4 ,4 ' ,7,7 ' - tetramethoxy-2 ,2 ' - binaphthalene-3 ,3 ' - dicarboxylate",
  'context': "[CLS] ##bon ##yl - 1 , 4 - nap ##ht ##ho ##quin ##one , 1 , 4 - di ##hy ##dr ##ox ##y - 7 - h ##ydro ##xy ##met ##hyl - ant ##hra ##quin ##one , and [unused1] dim ##eth ##yl 1 , 1 ' - di ##hy ##dr ##ox ##y - 4 , 4 ' , 7 , 7 ' - te ##tra ##met ##ho ##xy - 2 , 2 ' - bin ##aph ##thal ##ene - 3 , 3 ' - di ##car ##box ##yla ##te [unused2] , respectively . Na ##ph ##th ##o ##quin ##ones and nap ##ht ##ho ##hy ##dr ##o ##quin ##one dim ##ers were previously unknown in the genus Mo ##rin ##da . In addition , the compounds were tested [SEP]",
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0029224',
  'our_bi_cui_name': 'Organic Chemicals',
  'our_bi_cui_desc': '[CLS] Organic Chemical ##s [unused3] ( Chemical , substance : organic chemicals ; compound organic ; Organic Chemical ##s [ Chemical / In ##g ##red ##ient ] ; organic compound ; Chemical ##s , Organic ; organic chemical ; Organic Chemical ; chemical organic ; organic compounds ; Organic compound ; Organic compounds ; Organic chemical ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27622027.25',
  'test_set_idx': 12098,
  'type': 'T017',
  'name': 'tumor - draining LN',
  'context': '[CLS] of color ##ec ##tal cancer patients . Fresh tumor - draining L ##N tissues from 39 randomly enrolled patients were assessed by flow c ##yt ##ome ##try for activation and differentiation of CD ##16 ##9 ( + ) CD ##8 ( + ) T cells and T cell - mediated killing of tumor cells . In total , 114 [unused1] tumor - draining L ##N [unused2] para ##ffin sections from color ##ec ##tal cancer patients were analyzed by multiple - color im ##mu ##no ##f ##lu ##ores ##cence for CD ##16 ##9 ( + ) CD ##8 ( + ) T cell distribution and clinical values . The pro ##gno ##stic significance of CD ##16 ##9 ( + ) CD ##8 ( + ) T cells [SEP]',
  'existing_cui': 'C0269800',
  'existing_cui_name': 'Amniotic fluid leaking',
  'existing_cui_desc': '[CLS] Am ##nio ##tic fluid leaking [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : Waters leaking ; Lea ##ka ##ge of am ##nio ##tic fluid ; Am ##nio ##rr ##hea ; Li ##q ##uo ##r observed ; Dr ##ain ##age of liquor ; Flow of am ##nio ##tic liquor ; am ##nio ##tic fluid leak ; am ##nio ##tic fluid leak ##age ; am ##nio ##tic fluid ; leak ; Lea ##ka ##ge of Am ##nio ##tic F ##lu ##id ; Li ##q ##uo ##r draining ; leaking water ; water leak ; leak ##age of am ##nio ##tic fluid ; Am ##nio ##tic fluid leak ##age ; Am ##nio ##rr ##hoe ##a ; am ##nio ##tic fluid leaking ; am ##nio [SEP]',
  'our_bi_cui': 'C0281444',
  'our_bi_cui_name': 'tumor-draining lymph node lymphocyte therapy',
  'our_bi_cui_desc': '[CLS] tumor - draining l ##ymph node l ##ymph ##oc ##yte therapy [unused3] ( Health Care Act ##ivity : Tu ##mor - Dr ##ain ##ing L ##ymph No ##de L ##ymph ##oc ##yte Therapy ) [SEP]',
  'our_cross_cui': 'C0024204',
  'our_cross_cui_name': 'lymph nodes',
  'our_cross_cui_desc': '[CLS] l ##ymph nodes [unused3] ( Ana ##tom ##ical Structure , body structure : L ##ymph ##atic G ##land ; Structure of l ##ymph node ; nodes l ##ymph ; glands l ##ymph ##atic ; No ##de , L ##ymph ; l ##ymph ##s nodes ; L ##ymph ##atic g ##land ; l ##ymph ##atic g ##land ; L ##ymph ##ono ##dus ; node l ##ymph ; l ##ymph ##s node ; L ##ymph node structure ; l ##ymph node ; L ##ymph G ##land ; No ##dus l ##ymph ##atic ##us ; L ##ymph No ##des ; L ##ymph No ##de ; L ##Y ##MP ##H NO ##DE ; L ##ymph nodes ; No ##des , L ##ymph ; L ##ymph g ##land ; No ##dus [SEP]'},
 {'mention_id': '27500205.13',
  'test_set_idx': 8445,
  'type': 'T017',
  'name': 'adherent cells',
  'context': '[CLS] e ##z ##rin - rid ##ix ##in - m ##oes ##in ( F ##ER ##M ) domain containing proteins . There are three kind ##lins in mammals , which share high sequence identity . Kind ##lin - 1 is expressed primarily in e ##pit ##hel ##ial cells , kind ##lin - 2 is widely distributed and is particularly abundant in [unused1] ad ##here ##nt cells [unused2] , and kind ##lin - 3 is expressed primarily in hem ##ato ##po ##iet ##ic cells . These distributions are not exclusive ; some cells express multiple kind ##lins , and transformed cells often exhibit a ##ber ##rant expression , both in the is ##of ##orms and the levels of kind ##lins . Great interest in the kind ##lins has [SEP]',
  'existing_cui': 'C0263567',
  'existing_cui_name': 'Adherent scar',
  'existing_cui_desc': '[CLS] Ad ##here ##nt scar [unused3] ( Finding , diagnosis , disorder : ad ##here ##nt scar ; ad ##here ##nt ; scar ; scar ; ad ##here ##nt ; Ad ##here ##nt skin ) [SEP]',
  'our_bi_cui': 'C0007577',
  'our_bi_cui_name': 'Cell Adhesion',
  'our_bi_cui_desc': '[CLS] Cell Ad ##hesion [unused3] ( B ##iol ##og ##ic Fun ##ction : Cell Ad ##hesion ##s ; Cell ad ##hesion ; Ad ##hesion , Cell ; Cell Ad ##hesion Process ; ad ##hesion cellular ; cell ad ##hesion ; Ad ##hesion ##s , Cell ; ad ##hesion cell ; cellular ad ##hesion ; Cell ##ular Ad ##hesion ) [SEP]',
  'our_cross_cui': 'C0007634',
  'our_cross_cui_name': 'Cells',
  'our_cross_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]'},
 {'mention_id': '28113841.10',
  'test_set_idx': 25840,
  'type': 'T170',
  'name': 'reconstruction models',
  'context': '[CLS] , we develop a structural feature based collaborative reconstruction ( St ##ructural Feature based Col ##la ##bor ##ative Reconstruction ) method for quantitative su ##s ##ce ##pt ##ibility mapping including both magnitude and su ##s ##ce ##pt ##ibility based information . The St ##ructural Feature based Col ##la ##bor ##ative Reconstruction algorithm is composed of two consecutive steps corresponding to complementary [unused1] reconstruction models [unused2] , each with a structural feature based l ##1 norm con ##stra ##int and a v ##ox ##el fi ##delity based l ##2 norm con ##stra ##int , which allows both the structure edges and tiny features to be recovered , whereas the noise and artifacts could be reduced . In the M - step , the initial su ##s ##ce [SEP]',
  'existing_cui': 'C0237819',
  'existing_cui_name': 'Learning Reconstruction',
  'existing_cui_desc': '[CLS] Learning Reconstruction [unused3] ( B ##iol ##og ##ic Fun ##ction , Learning : Reconstruction ) [SEP]',
  'our_bi_cui': 'C0449363',
  'our_bi_cui_name': 'Method of reconstruction',
  'our_bi_cui_desc': '[CLS] Method of reconstruction [unused3] ( Intel ##lect ##ual Product , attribute ) [SEP]',
  'our_cross_cui': 'C3161035',
  'our_cross_cui_name': 'Model',
  'our_cross_cui_desc': '[CLS] Model [unused3] ( Intel ##lect ##ual Product : Models ; Model ##ing System ; Model System ) [SEP]'},
 {'mention_id': '27855073.16',
  'test_set_idx': 19359,
  'type': 'T103',
  'name': 'uS4',
  'context': "[CLS] essential component of the small rib ##oso ##mal subunit that is involved in subunit assembly , maintenance of translation ##al fi ##delity , and the rib ##oso ##me ' s response to the anti ##biotic s ##pect ##ino ##my ##cin . While many of the characterized u ##S ##5 mutations that affect de ##co ##ding map to its interface with [unused1] u ##S ##4 [unused2] , more recent work has shown that residues distant from the u ##S ##4 - u ##S ##5 interface can also affect the de ##co ##ding process . We targeted one such interface - remote area , the loop 2 region ( residues 20 to 31 ) , for m ##uta ##genesis in E ##scher ##ichi ##a . co ##li and [SEP]",
  'existing_cui': 'C1419736',
  'existing_cui_name': 'RPS4X gene',
  'existing_cui_desc': '[CLS] R ##PS ##4 ##X gene [unused3] ( Ana ##tom ##ical Structure : S ##4 ; R ##IB ##OS ##OM ##AL PR ##OT ##EI ##N S ##4 , X - L ##IN ##KE ##D ; CE ##LL C ##Y ##CL ##E GE ##NE 2 ; rib ##oso ##mal protein S ##4 , X - linked ; S ##ING ##LE - CO ##P ##Y AB ##UN ##DA ##NT m ##RNA ; SC ##R ##10 ; R ##PS ##4 ; 40 ##S rib ##oso ##mal protein S ##4 , X is ##of ##orm ; single - copy abundant m ##RNA ; rib ##oso ##mal protein S ##4 ##X is ##of ##orm ; D ##X ##S ##30 ##6 ; FL ##J ##40 ##5 ##9 ##5 ; cell cycle gene 2 [SEP]',
  'our_bi_cui': 'C0041528',
  'our_bi_cui_name': 'U4 small nuclear RNA',
  'our_bi_cui_desc': '[CLS] U ##4 small nuclear RNA [unused3] ( Chemical : U ##4 RNA ; U ##4 s ##n ##RNA ) [SEP]',
  'our_cross_cui': 'C0073333',
  'our_cross_cui_name': 'ribosomal protein S4',
  'our_cross_cui_desc': '[CLS] rib ##oso ##mal protein S ##4 [unused3] ( Chemical : S ##4 rib ##oso ##mal protein ) [SEP]'},
 {'mention_id': '28223077.29',
  'test_set_idx': 28913,
  'type': 'T082',
  'name': 'background site',
  'context': '[CLS] , background site , up ##wind site and down ##wind site of a Municipal solid waste land ##fill in East China . The results of present study were summarized as follows . ( 1 ) The total concentrations of PC ##D ##D / F ##s ( [UNK] PC ##D ##D / F ##s ) in ambient air from surrounding sites , [unused1] background site [unused2] , up ##wind site and down ##wind site were 2 . 215 ± 1 . 00 ##4 , 2 . 05 ##8 ± 0 . 45 ##8 , 2 . 61 ##7 ± 1 . 09 ##2 and 1 . 82 ##2 ± 0 . 56 ##6 p ##g ##N ##m ( - 3 ) , respectively . ( 2 ) [SEP]',
  'existing_cui': 'C0237490',
  'existing_cui_name': 'Family Background',
  'existing_cui_desc': '[CLS] Family Background [unused3] ( Finding , Family : Background ) [SEP]',
  'our_bi_cui': 'C0442611',
  'our_bi_cui_name': 'Industrial site',
  'our_bi_cui_desc': '[CLS] Industrial site [unused3] ( Spa ##tial Concept , environment ) [SEP]',
  'our_cross_cui': 'C0205145',
  'our_cross_cui_name': 'Site',
  'our_cross_cui_desc': '[CLS] Site [unused3] ( Spa ##tial Concept , attribute , qualifier value : Sites ) [SEP]'},
 {'mention_id': '27371817.9',
  'test_set_idx': 3913,
  'type': 'T058',
  'name': 'Non - destructive proton nuclear magnetic resonance ( ( 1 ) H NMR ) spectroscopy',
  'context': '[CLS] of g ##ly ##cer ##op ##hos ##ph ##oli ##pid signatures of inflammation treated with non - s ##tero ##idal anti - inflammatory drugs - induced - RA ##W ##26 ##4 . 7 cells using ( 1 ) H N ##MR and U - HP ##LC / Q - TO ##F - MS [unused1] Non - destructive pro ##ton nuclear magnetic resonance ( ( 1 ) H N ##MR ) s ##pect ##ros ##copy [unused2] and highly sensitive ultra - performance liquid ch ##roma ##tography q ##uad ##rup ##ole time - of - flight mass s ##pect ##rome ##try ( U - HP ##LC / Q - TO ##F - MS ) coupled to data processing methods were applied to analyze the metabolic pro ##fi ##ling changes [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C3850002',
  'our_bi_cui_name': 'Proton Magnetic Resonance Spectroscopy',
  'our_bi_cui_desc': '[CLS] Proto ##n Ma ##gnetic Re ##son ##ance S ##pect ##ros ##copy [unused3] ( Health Care Act ##ivity : 1 ##H - MR ##S ) [SEP]',
  'our_cross_cui': 'C0877853',
  'our_cross_cui_name': 'Spectroscopy , Nuclear Magnetic Resonance',
  'our_cross_cui_desc': '[CLS] S ##pect ##ros ##copy , Nuclear Ma ##gnetic Re ##son ##ance [unused3] ( Health Care Act ##ivity : S ##pect ##ros ##copy , N ##MR ; nuclear magnetic resonance s ##pect ##ros ##copy ; Nuclear magnetic resonance s ##pect ##ros ##copy ; N ##MR S ##pect ##ros ##copy ; S ##pect ##ros ##co ##pies , N ##MR ; N ##MR S ##pect ##ros ##co ##pies ; nm ##r s ##pect ##ros ##copy ) [SEP]'},
 {'mention_id': '28539192.15',
  'test_set_idx': 38799,
  'type': 'T033',
  'name': 'complex level',
  'context': '[CLS] the population specifically in health . Currently , there is a fragment ##ed model of patient care , in which the systems are exclusive of each other . They do not cooperate or communicate with each other ; and if there is no vertical communication within the system , preventing refer ##ral of patients directly from the basic level to the [unused1] complex level [unused2] of care when needed . In addition , there has been no progress in the development of an em ##path ##etic , respect ##ful or person - centered clinical practice ; instead , economic , social and educational differences per ##pet ##uate a paternal ##istic clinical practice . The task of or ##ient ##ing medical training towards the development of human [SEP]',
  'existing_cui': 'C0056210',
  'existing_cui_name': 'complex V',
  'existing_cui_desc': '[CLS] complex V [unused3] ( Chemical , mitochondrial o ##xi ##da ##tive p ##hos ##ph ##ory ##lation system , mitochondrial o ##xi ##da ##tive p ##hos ##ph ##ory ##lation system , MA ##TS ##C , mitochondrial ad ##eno ##sin ##e trip ##hos ##phate s ##ynth ##ase complex : CO ##MP ##LE ##X ) [SEP]',
  'our_bi_cui': 'C0456079',
  'our_bi_cui_name': 'Disease classification level',
  'our_bi_cui_desc': '[CLS] Disease classification level [unused3] ( Intel ##lect ##ual Product , attribute : Level ) [SEP]',
  'our_cross_cui': 'C1547668',
  'our_cross_cui_name': 'Level of Care',
  'our_cross_cui_desc': '[CLS] Level of Care [unused3] ( Finding : Level of care ) [SEP]'},
 {'mention_id': '27756117.21',
  'test_set_idx': 15618,
  'type': 'T058',
  'name': 'molding',
  'context': '[CLS] ##gen are increasingly used due to their efficacy and their ease of application . In the present study , we describe a method to incorporate the co ##agu ##lant supplement ##s fi ##bri ##no ##gen and th ##rom ##bin in silk protein s ##po ##nges by mixing the co ##agu ##lants with an a ##que ##ous silk solution , followed by [unused1] mold ##ing [unused2] , freeze - drying and water an ##nea ##ling . In this combination system we demonstrate the delivery of fi ##bri ##no ##gen while maintaining its hem ##ost ##atic potential . Con ##cent ##ration ratios of silk to fi ##bri ##no ##gen of 1 . 0 % / 2 . 8 % , 2 . 3 % / 1 . 5 [SEP]',
  'existing_cui': 'C0426103',
  'existing_cui_name': 'Fetal head molding',
  'existing_cui_desc': '[CLS] Fe ##tal head mold ##ing [unused3] ( Finding , finding , finding : F ##oe ##tal head m ##ou ##lding ; f ##etal head mold ##ing ; O ##bs ##er ##vation of f ##etal head mold ##ing ; Finding of f ##etal head m ##ou ##lding ; Finding of foe ##tal head m ##ou ##lding ; Mo ##lding ; O ##bs ##er ##vation of foe ##tal head m ##ou ##lding ; O ##bs ##er ##vation of f ##etal head m ##ou ##lding ; mold ##ing ; Finding of f ##etal head mold ##ing ; Fe ##tal head m ##ou ##lding ) [SEP]',
  'our_bi_cui': 'C0947647',
  'our_bi_cui_name': 'Manipulation procedure',
  'our_bi_cui_desc': '[CLS] Man ##ip ##ulation procedure [unused3] ( Health Care Act ##ivity , procedure , qualifier value , regime / therapy : manipulation ; Man ##ip ##ulation ##s ; Man ##ip ##ulation - action ; manipulation procedure ; Man ##ip ##ulate ; Man ##ip ##ula ##tive ; Man ##ip ##ulation ; Man ##ip ##ula ##tive procedure ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '27838742.64',
  'test_set_idx': 19031,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] ##6 ##K pathway in A ##5 ##4 ##9 and PC - 9 cells . Del ##ica ##f ##lav ##one is a potential anti ##can ##cer agent that can induce auto ##pha ##gic cell death in human non - small cell lung cancer via the A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . [unused1] Del ##ica ##f ##lav ##one [unused2] showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . Del ##ica ##f ##lav ##one induced auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27563025.32',
  'test_set_idx': 10413,
  'type': 'T017',
  'name': 'C2C12 cells',
  'context': '[CLS] ##no ##fi ##bres . To estimate the bio ##com ##pa ##ti ##bility of the 3D p ##oly ##cap ##rol ##act ##one / p ##oly ##ani ##line composite na ##no ##fi ##bre mac ##ros ##truct ##ures , mouse my ##ob ##last ##s ( C ##2 ##C ##12 cells ) were culture ##d as model cells . The initial responses of [unused1] C ##2 ##C ##12 cells [unused2] to the 3D p ##oly ##cap ##rol ##act ##one / p ##oly ##ani ##line composite mac ##ros ##truct ##ures were significantly superior to those to pure p ##oly ##cap ##rol ##act ##one , that is , the cells exhibited typical my ##ob ##last - like m ##or ##ph ##ologies with obvious pseudo ##po ##dia and the moderate incorporation ( less [SEP]',
  'existing_cui': 'C0085262',
  'existing_cui_name': 'PC12 Cells',
  'existing_cui_desc': '[CLS] PC ##12 Cell ##s [unused3] ( Ana ##tom ##ical Structure : Cell Line , Ph ##eo ##ch ##rom ##oc ##yt ##oma ; Ph ##eo ##ch ##rom ##oc ##yt ##oma Cell Lines ; Ph ##eo ##ch ##rom ##oc ##yt ##oma 12 Cell Line ; PC A 01 ##2 CE ##LL ##S ; p ##he ##och ##rom ##oc ##yt ##oma 12 cell line ; PC - 12 ; p ##c ##12 cell ; PC ##12 Cell ; Ph ##eo ##ch ##rom ##oc ##yt ##oma Cell Line ; Cell Lines , Ph ##eo ##ch ##rom ##oc ##yt ##oma ; PC ##12 cell ; p ##c ##12 cells ) [SEP]',
  'our_bi_cui': 'C1135917',
  'our_bi_cui_name': 'Myoblasts , Cardiac',
  'our_bi_cui_desc': '[CLS] My ##ob ##last ##s , Card ##iac [unused3] ( Ana ##tom ##ical Structure : cardiac my ##ob ##last ##s ; My ##ob ##last , Card ##iac ; cardiac my ##ob ##last ; Card ##iac my ##ob ##last ; Card ##iac My ##ob ##last ##s ; Card ##iac My ##ob ##last ) [SEP]',
  'our_cross_cui': 'C0596995',
  'our_cross_cui_name': 'Myoblasts',
  'our_cross_cui_desc': '[CLS] My ##ob ##last ##s [unused3] ( Ana ##tom ##ical Structure : Mu ##s ##cle Cell ##s , Em ##b ##ryo ##nic ; My ##ob ##last ; Mu ##s ##cle Cell , Em ##b ##ryo ##nic ; Pre ##cu ##rs ##or Mu ##s ##cle Cell ##s ; Mu ##s ##cle Cell , Pre ##cu ##rs ##or ; my ##ob ##last ; Em ##b ##ryo ##nic Mu ##s ##cle Cell ##s ; my ##ob ##last ##s ; Pre ##cu ##rs ##or Mu ##s ##cle Cell ; Em ##b ##ryo ##nic Mu ##s ##cle Cell ; Mu ##s ##cle Cell ##s , Pre ##cu ##rs ##or ) [SEP]'},
 {'mention_id': '28460054.10',
  'test_set_idx': 36759,
  'type': 'T062',
  'name': 'feasible strategy',
  'context': '[CLS] Effect of ha ##lot ##ole ##rant r ##hi ##zo ##ba ##cter ##ia isolated from ha ##lop ##hy ##tes on the growth of sugar bee ##t ( Beta v ##ul ##gar ##is L . ) under salt stress U ##til ##ization of r ##hi ##zo ##ba ##cter ##ia that have associated with plant roots in harsh environments could be a [unused1] feasible strategy [unused2] to deal with limits to agricultural production caused by soil sa ##linity . Hal ##op ##hy ##tes occur naturally in high - salt environments , and their roots may be associated with promising micro ##bial candidates for promoting growth and salt tolerance in crops . This study aimed to is ##olate efficient ha ##lot ##ole ##rant plant - growth - promoting r ##hi [SEP]',
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0589212',
  'our_bi_cui_name': 'Pragmatic enhancement strategies',
  'our_bi_cui_desc': '[CLS] P ##rag ##matic enhance ##ment strategies [unused3] ( Health Care Act ##ivity , procedure , regime / therapy ) [SEP]',
  'our_cross_cui': 'C0035171',
  'our_cross_cui_name': 'Research Design',
  'our_cross_cui_desc': '[CLS] Research Design [unused3] ( Research Act ##ivity : Experiment Design ; design research ; research design ; Design ##s , Research ; Design , Research ; ST ##YP ##E ; Research Design ##s ; study design ; Study Type ; Study Design ; Experimental Design ) [SEP]'},
 {'mention_id': '27371817.39',
  'test_set_idx': 3943,
  'type': 'T058',
  'name': '( 1 ) H NMR spectroscopy',
  'context': '[CLS] - MS showed higher sensitivity and specific ##ity and was more suitable for the determination of bio ##mark ##ers apart from the deficiency of time - consuming sample preparation steps and un ##am ##bi ##guous meta ##bol ##ite identification . Therefore , it is feasible to analyze the changes of g ##ly ##cer ##op ##hos ##ph ##oli ##pid ##s during inflammation by combining [unused1] ( 1 ) H N ##MR s ##pect ##ros ##copy [unused2] with U - HP ##LC / Q - TO ##F - MS . The metabolic pro ##fi ##ling of g ##ly ##cer ##op ##hos ##ph ##oli ##pid ##s provides valuable evidence for inflammation diagnosis and pro ##gno ##sis , and might un ##rave ##l the mechanisms involved in inflammation progression . [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C3850001',
  'our_bi_cui_name': '1H NMR',
  'our_bi_cui_desc': '[CLS] 1 ##H N ##MR [unused3] ( Health Care Act ##ivity : 1 ##H - N ##MR ) [SEP]',
  'our_cross_cui': 'C0877853',
  'our_cross_cui_name': 'Spectroscopy , Nuclear Magnetic Resonance',
  'our_cross_cui_desc': '[CLS] S ##pect ##ros ##copy , Nuclear Ma ##gnetic Re ##son ##ance [unused3] ( Health Care Act ##ivity : S ##pect ##ros ##copy , N ##MR ; nuclear magnetic resonance s ##pect ##ros ##copy ; Nuclear magnetic resonance s ##pect ##ros ##copy ; N ##MR S ##pect ##ros ##copy ; S ##pect ##ros ##co ##pies , N ##MR ; N ##MR S ##pect ##ros ##co ##pies ; nm ##r s ##pect ##ros ##copy ) [SEP]'},
 {'mention_id': '27251800.7',
  'test_set_idx': 802,
  'type': 'T058',
  'name': 'Stoppa technique',
  'context': '[CLS] Early assessment of bilateral ing ##uin ##al her ##nia repair : A comparison between the lap ##aro ##scopic total extra ##per ##ito ##nea ##l and Stop ##pa approaches The present clinical trial was designed to compare the results of bilateral ing ##uin ##al her ##nia repair between patients who underwent the conventional [unused1] Stop ##pa technique [unused2] and lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair with a single mesh and without staple fix ##ation . This controlled , random ##ised clinical trial was conducted at General Surgery and T ##ra ##uma of the Clinic ##s Hospital , Medical School , the University of S ##γ ##o Paulo between September 2010 and February 2011 . Total ##ly , 50 male patients , with a [SEP]',
  'existing_cui': 'C0450241',
  'existing_cui_name': 'Ffooks technique',
  'existing_cui_desc': '[CLS] F ##fo ##ok ##s technique [unused3] ( Health Care Act ##ivity , qualifier value ) [SEP]',
  'our_bi_cui': 'C0031150',
  'our_bi_cui_name': 'Laparoscopy',
  'our_bi_cui_desc': '[CLS] La ##par ##os ##copy [unused3] ( Health Care Act ##ivity , G ##I only , procedure : La ##par ##os ##copic procedure ; per ##ito ##ne ##os ##copy ; Co ##eli ##os ##copy ; Ce ##lio ##sco ##pies ; Per ##ito ##ne ##os ##co ##pies ; Abd ##ome ##n end ##os ##copy ; c ##eli ##os ##copy ; lap ##aro ##scopic procedure ; Per ##ito ##ne ##os ##copy ; La ##p ; Ce ##lio ##sco ##py ; La ##par ##os ##co ##pies ; lap ##aro ##sco ##pies ; lap ##aro ##sco ##py ; lap ##aro ##scopic procedures ; diagnostic lap ##aro ##sco ##py ; End ##os ##copy of abdomen ; lap ##aro ##sco ##py procedure ; La ##par ##os ##copic ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27444636.43',
  'test_set_idx': 5994,
  'type': 'T170',
  'name': 'Adverse Events Profile',
  'context': '[CLS] ##pine to es ##lica ##rb ##az ##ep ##ine ace ##tate , there were significant improvements in mean scores for Ad ##verse Events Profile ( P < . 00 ##1 ) , Quality of Life in E ##pile ##psy In ##vent ##ory 10 ( P = . 00 ##1 ) , and alert ##ness ( P < . 05 ) . [unused1] Ad ##verse Events Profile [unused2] total scores improved for 21 / 21 ( 100 . 0 % ) patients , Quality of Life in E ##pile ##psy In ##vent ##ory 10 total scores improved for 17 / 21 ( 81 . 0 % ) patients , and alert ##ness scores improved for 16 / 21 ( 76 . 2 % ) patients . In this [SEP]',
  'existing_cui': 'C0877248',
  'existing_cui_name': 'Adverse event',
  'existing_cui_desc': '[CLS] Ad ##verse event [unused3] ( B ##iol ##og ##ic Fun ##ction : AD ##VE ##RS ##E E ##VE ##NT ; adverse events ; Ad ##verse Event ; adverse event ; Ad ##verse ##E ##vent ; Ad ##verse Experience ) [SEP]',
  'our_bi_cui': 'C1521800',
  'our_bi_cui_name': 'Adverse Drug Experience Report',
  'our_bi_cui_desc': '[CLS] Ad ##verse Drug Experience Report [unused3] ( Intel ##lect ##ual Product : Ad ##verse Drug Report ; Ad ##verse Experience Report ; Ad ##verse Event Report ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28375449.10',
  'test_set_idx': 33493,
  'type': 'T058',
  'name': 'knotting',
  'context': '[CLS] attempt to reduce the difficulty of this surgical procedure , this study developed a modified an ##ast ##omo ##tic technique for th ##ora ##cola ##par ##os ##copic I ##vor - Lewis es ##op ##hage ##ct ##omy . During the entirety of this modified approach , neither technically challenging operations such as in ##tra ##th ##ora ##ci ##c su ##tur ##ing or [unused1] knot ##ting [unused2] , nor special instruments such as an Or ##V ##il system or a reverse - pu ##nc ##ture head are required . Between October 2015 and January 2016 , 15 consecutive patients with cancer in the di ##stal third of the es ##op ##ha ##gus or the gas ##tric card ##ia underwent this modified surgical procedure . The good short - [SEP]',
  'existing_cui': 'C0263493',
  'existing_cui_name': 'Trichonodosis',
  'existing_cui_desc': '[CLS] Tri ##cho ##no ##dos ##is [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : Hair knot ##ting ; t ##rich ##ono ##dos ##is ; K ##not ##ted hairs ; knot ##ting of hair ; K ##not ##ting of hair shaft ) [SEP]',
  'our_bi_cui': 'C0023690',
  'our_bi_cui_name': 'Ligation',
  'our_bi_cui_desc': '[CLS] Liga ##tion [unused3] ( Health Care Act ##ivity , procedure , qualifier value : surgical binding ; l ##iga ##tion ; C ##losure by Liga ##tion ; Liga ##tion - action ; Liga ##tions ; C ##losure by l ##iga ##tion ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27273231.33',
  'test_set_idx': 1214,
  'type': 'T033',
  'name': 'No',
  'context': '[CLS] period , the pulmonary art ##eries were sampled . Press ##ure tolerance at the sealed end was evaluated in 91 pulmonary artery sections . All sealed ends showed pressure tolerance > 75 mm H ##g . A left lower lobe ##ct ##omy was performed in 13 dogs in which 35 pulmonary artery sections had been allocated into the 3 groups . [unused1] No [unused2] sealing failure was found , and path ##ological findings showed healing and persistent hem ##ost ##asis at all sealed ends of the pulmonary art ##eries after 2 and 4 weeks of the survival period . P ##ul ##mona ##ry art ##eries sealed in v ##ivo with the En ##sea ##l device showed pressure tolerance > 75 mm H ##g in the acute [SEP]',
  'existing_cui': 'C1016143',
  'existing_cui_name': 'Tetragenococcus halophilus',
  'existing_cui_desc': '[CLS] Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##us [unused3] ( Ba ##cter ##ium , organism : P ##ed ##io ##co ##cc ##us ha ##lop ##hil ##us Me ##es 1934 ( A ##pp ##roved List ##s 1980 ) ; Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##us ( Me ##es 1934 ) Collins et al . 1993 ; Sa ##rc ##ina ha ##ma ##guchi ##ae ; Te ##tra ##gen ##oc ##oc ##cus ha ##lop ##hil ##a ; P ##ed ##io ##co ##cc ##us ha ##lop ##hil ##us ; Te ##tra ##co ##cc ##cus No . 1 ; P ##ed ##io ##co ##cc ##us so ##ya ##e ) [SEP]',
  'our_bi_cui': 'C0243095',
  'our_bi_cui_name': 'Finding',
  'our_bi_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]',
  'our_cross_cui': 'C1513916',
  'our_cross_cui_name': 'Negative Finding',
  'our_cross_cui_desc': '[CLS] N ##eg ##ative Finding [unused3] ( Finding : NE ##GA ##TI ##VE ; N ##eg ##ative ) [SEP]'},
 {'mention_id': '28495651.34',
  'test_set_idx': 37809,
  'type': 'T201',
  'name': 'cardiovascular death',
  'context': '[CLS] Hodges , and 1 . 12 ( 0 . 28 - 4 . 52 ) for Q ##T ##c Fr ##ami ##ng ##ham . The hazard ratio of Q ##T ##c Ba ##ze ##tt was significantly higher than the other formula ##s except for the 98 ##th percent ##ile method . The Fr ##ami ##ng ##ham , Hodges , and Fr ##ider ##ici ##a equations remove the effect of heart rate on Q ##T interval significantly better than the Ba ##ze ##tt equation . Using Q ##T - interval percent ##iles based on heart rate provides a consistent approach both for identifying those whose Q ##T intervals pro ##long due to drugs or other stress ##ors and for assessing [unused1] card ##iovascular death [unused2] risk . [SEP]',
  'existing_cui': 'C0007226',
  'existing_cui_name': 'Cardiovascular system',
  'existing_cui_desc': '[CLS] Card ##iovascular system [unused3] ( Body System , body structure : Structure of card ##iovascular system , un ##sp ##ec ##ified ; CA ##RD ##IO ##VA ##SC ##U ##LA ##R S ##Y ##ST ##EM ; Card ##iovascular ; Organ System , Card ##iovascular ; CA ##RD ##IO ##VA ##SC S ##Y ##ST ##EM ; Card ##iovascular structure ; card ##iovascular system ; Card ##iovascular Organ System ; C ##ir ##cula ##tory ; Structure of card ##iovascular system ; Card ##iovascular Systems ; card ##iovascular structure ; C ##ir ##cula ##tory System ; C ##ir ##cula ##tory Systems ; { C ##ir ##cula ##tory system } ; c ##ir ##cula ##tory ; C ##ir ##cula ##tory system ; Body System , Card ##iovascular ; card ##iovascular system [SEP]',
  'our_bi_cui': 'C0376297',
  'our_bi_cui_name': 'Cardiac Death',
  'our_bi_cui_desc': '[CLS] Card ##iac Death [unused3] ( B ##iol ##og ##ic Fun ##ction : death ; cardiac ; cardiac death ; cardiac deaths ; Card ##iac death ; Death , Card ##iac ; cardiac ; death ) [SEP]',
  'our_cross_cui': 'C2926098',
  'our_cross_cui_name': "Mechanism of death in patient 's dying of cardiovascular causes : Type : Point in time : ^ Patient : Nominal",
  'our_cross_cui_desc': "[CLS] Me ##chan ##ism of death in patient ' s dying of card ##iovascular causes : Type : Point in time : ^ Pat ##ient : No ##mina ##l [unused3] ( Clinical At ##tri ##but ##e : Death due to C ##V causes ; Me ##chan ##ism of death in patient ' s dying of card ##iovascular causes : Type : P ##t : ^ Pat ##ient : No ##m ; Me ##chan ##ism of death in patient ' s dying of card ##iovascular causes ) [SEP]"},
 {'mention_id': '28493435.9',
  'test_set_idx': 37724,
  'type': 'T033',
  'name': 'locate',
  'context': '[CLS] el ##ast ##ography algorithms , lo ##cating ex ##tre ##ma in multi ##mo ##dal ultra ##sonic radio ##f ##re ##que ##ncy signals is still a challenging problem . In this paper , a new method is proposed to locate the local ma ##xi ##ma and mini ##ma of the RF signals directly without der ##ivation operation . This algorithm can accurately [unused1] locate [unused2] ex ##tre ##ma even if disturbed peaks resulting from different noise exist . Furthermore , the new algorithm can speed up approximately 79 % of the implementation process as compared with the standard cross - correlation method on the same computing platform . In addition , the el ##ast ##ographic signal - to - noise ratio and the contrast - to - [SEP]',
  'existing_cui': 'C1366498',
  'existing_cui_name': 'Chloramphenicol Acetyl Transferase Gene',
  'existing_cui_desc': '[CLS] Ch ##lora ##mp ##hen ##ico ##l Ace ##ty ##l Transfer ##ase Gene [unused3] ( Ana ##tom ##ical Structure : cat Gene ) [SEP]',
  'our_bi_cui': 'C0450429',
  'our_bi_cui_name': 'Location',
  'our_bi_cui_desc': '[CLS] Location [unused3] ( Spa ##tial Concept , attribute : location ; Lo ##c ; L ##OC ; locations ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27444636.40',
  'test_set_idx': 5991,
  'type': 'T170',
  'name': 'Adverse Events Profile',
  'context': '[CLS] ##rb ##az ##ep ##ine ace ##tate ( days 0 and 5 , respectively ) . The analysis included 21 patients ( 12 women , 9 men ; mean age 36 years ) . After switching from o ##x ##car ##ba ##ze ##pine to es ##lica ##rb ##az ##ep ##ine ace ##tate , there were significant improvements in mean scores for [unused1] Ad ##verse Events Profile [unused2] ( P < . 00 ##1 ) , Quality of Life in E ##pile ##psy In ##vent ##ory 10 ( P = . 00 ##1 ) , and alert ##ness ( P < . 05 ) . Ad ##verse Events Profile total scores improved for 21 / 21 ( 100 . 0 % ) patients , Quality of Life in [SEP]',
  'existing_cui': 'C0877248',
  'existing_cui_name': 'Adverse event',
  'existing_cui_desc': '[CLS] Ad ##verse event [unused3] ( B ##iol ##og ##ic Fun ##ction : AD ##VE ##RS ##E E ##VE ##NT ; adverse events ; Ad ##verse Event ; adverse event ; Ad ##verse ##E ##vent ; Ad ##verse Experience ) [SEP]',
  'our_bi_cui': 'C1521800',
  'our_bi_cui_name': 'Adverse Drug Experience Report',
  'our_bi_cui_desc': '[CLS] Ad ##verse Drug Experience Report [unused3] ( Intel ##lect ##ual Product : Ad ##verse Drug Report ; Ad ##verse Experience Report ; Ad ##verse Event Report ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27932585.13',
  'test_set_idx': 22240,
  'type': 'T103',
  'name': 'Taw3',
  'context': '[CLS] ##lm ##eth ##ion ##ine ( S - ad ##eno ##sy ##lm ##eth ##ion ##ine ) - dependent met ##hyl ##tra ##ns ##fer ##ases regulate a wide range of biological processes through the modification of proteins , nuclei ##c acids , p ##oly ##sa ##cc ##hari ##des , as well as various meta ##bol ##ites . T ##Y ##W ##3 / [unused1] Ta ##w ##3 [unused2] is a SA ##M - dependent met ##hyl ##tra ##ns ##fer ##ase responsible for the formation of a t ##RNA modification known as w ##y ##but ##os ##ine and its derivatives that are required for accurate de ##co ##ding in protein synthesis . Here , we report the crystal structure of Ta ##w ##3 , a ho ##mo ##log of T [SEP]',
  'existing_cui': 'C3496263',
  'existing_cui_name': 'annectant white matter',
  'existing_cui_desc': '[CLS] an ##nect ##ant white matter [unused3] ( Ana ##tom ##ical Structure : Sub ##stant ##ia me ##du ##llar ##is an ##nect ##ens ) [SEP]',
  'our_bi_cui': 'C1538017',
  'our_bi_cui_name': 'MT-TW gene',
  'our_bi_cui_desc': '[CLS] M ##T - T ##W gene [unused3] ( Ana ##tom ##ical Structure : t ##rn ##W ; t ##RNA - T ##RP , MIT ##OC ##H ##ON ##DR ##IA ##L ; mitochondrial ##ly encoded t ##RNA try ##pt ##op ##han ; T ##RA ##NS ##F ##ER RNA , MIT ##OC ##H ##ON ##DR ##IA ##L , T ##R ##YP ##TO ##P ##HA ##N ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27778231.78',
  'test_set_idx': 16399,
  'type': 'T038',
  'name': 'resistant',
  'context': '[CLS] pro ##gno ##stic value of these proteins in childhood acute l ##ymph ##ob ##lastic le ##uke ##mia . Furthermore , investigation of protein - protein interaction using ST ##RI ##NG database indicated that these proteins involved in the signaling pathway of N ##R ##3 ##C ##1 as de ##xa ##met ##has ##one target . In conclusion , our pro ##te ##omi ##c study in de ##xa ##met ##has ##one [unused1] resistant [unused2] le ##uke ##mic cells revealed voltage dependent an ##ion channel 1 , sorting N ##ex ##in 3 , and pre ##fold ##in subunit 6 are promising proteins that might serve as potential bio ##mark ##ers of pro ##gno ##sis and ch ##em ##otherapy in childhood acute l ##ymph ##ob ##lastic le ##uke ##mia . [SEP]',
  'existing_cui': 'C2827757',
  'existing_cui_name': 'Antimicrobial Resistance Result',
  'existing_cui_desc': '[CLS] Anti ##mic ##ro ##bial Resistance Re ##sul ##t [unused3] ( Finding : Re ##sist ##ant ; R ##ES ##IS ##TA ##NT ) [SEP]',
  'our_bi_cui': 'C1514892',
  'our_bi_cui_name': 'Resistance Process',
  'our_bi_cui_desc': '[CLS] Resistance Process [unused3] ( B ##iol ##og ##ic Fun ##ction : Resistance ; Re ##sist ##ant ) [SEP]',
  'our_cross_cui': 'C0013203',
  'our_cross_cui_name': 'Drug resistance',
  'our_cross_cui_desc': '[CLS] Drug resistance [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : drug - related disorder resistance ; Drug Resistance Process ; Drug Resistance ; drug resistance ; Resistance , Drug ; obsolete drug resistance ; drugs resistance ; resistance drug ) [SEP]'},
 {'mention_id': '27965426.23',
  'test_set_idx': 22660,
  'type': 'T103',
  'name': 'tyrosine - phosphorylated proteins',
  'context': '[CLS] t ##yr ##os ##ine p ##hos ##ph ##ory ##lation in mit ##otic human cells including those on spin ##dle - associated proteins . ? Database mining confirmed ~ 2000 mit ##otic p ##hos ##ph ##ot ##yr ##os ##ine sites , and network analysis revealed a number of sub ##net ##works that were en ##rich ##ed in [unused1] t ##yr ##os ##ine - p ##hos ##ph ##ory ##lated proteins [unused2] , including components of the kin ##eto ##chor ##e or spin ##dle and SR ##C family kinase ##s . We identified Polo - like kinase 1 , a major signaling hub in the spin ##dle sub ##net ##work , as p ##hos ##ph ##ory ##lated at the conserved Ty ##r ( 217 ) in the kinase domain [SEP]',
  'existing_cui': 'C0168164',
  'existing_cui_name': 'GAP-associated protein p62',
  'existing_cui_desc': '[CLS] GA ##P - associated protein p ##6 ##2 [unused3] ( Chemical : t ##yr ##os ##ine - p ##hos ##ph ##ory ##lated protein p ##6 ##2 ) [SEP]',
  'our_bi_cui': 'C0031689',
  'our_bi_cui_name': 'Phosphoproteins',
  'our_bi_cui_desc': '[CLS] Ph ##os ##ph ##op ##rote ##ins [unused3] ( Chemical : Ph ##os ##ph ##op ##rote ##in ; Ph ##os ##ph ##op ##rote ##ins [ Chemical / In ##g ##red ##ient ] ; p ##hos ##ph ##op ##rote ##in ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27509301.9',
  'test_set_idx': 8791,
  'type': 'T103',
  'name': 'environmental chemicals',
  'context': "[CLS] ##genic B ##io ##act ##ivity of Environmental Chemical Met ##ab ##oli ##tes The US Environmental Protection Agency ' s End ##oc ##rine Di ##s ##rup ##tor Screen ##ing Program is using in v ##it ##ro data generated from To ##x ##C ##ast / To ##x ##21 high - through ##put screening ass ##ays to assess the end ##oc ##rine activity of [unused1] environmental chemicals [unused2] . Considering that in v ##it ##ro ass ##ays may have limited metabolic capacity , inactive chemicals that are bio ##tra ##ns ##formed into meta ##bol ##ites with end ##oc ##rine bio ##act ##ivity may be missed for further screening and testing . Therefore , there is a value in developing novel approaches to account for metabolism and end ##oc ##rine [SEP]",
  'existing_cui': 'C0014406',
  'existing_cui_name': 'Environment',
  'existing_cui_desc': '[CLS] Environment [unused3] ( Spa ##tial Concept , environment , treatment : environments ; Environment ##s ; environment ; Environmental ; E ##N ##VI ##R ; en ##vir ; En ##vir ) [SEP]',
  'our_bi_cui': 'C0014417',
  'our_bi_cui_name': 'Environmental Pollutants',
  'our_bi_cui_desc': '[CLS] Environmental Poll ##uta ##nts [unused3] ( Chemical : E ##N ##VI ##R P ##OL ##L ##UT ##AN ##TS ; P ##OL ##L ##UT ##AN ##TS E ##N ##VI ##R ; Poll ##uta ##nts , Environmental ) [SEP]',
  'our_cross_cui': 'C0220806',
  'our_cross_cui_name': 'Chemicals',
  'our_cross_cui_desc': '[CLS] Chemical ##s [unused3] ( Chemical , medication , substance : chemical compounds ; Chemical ; chemical compound ; Chemical compound ; chemical substance ; Chemical ##ly ; Chemical product ; Chemical agent ; Chemical element AND / OR compound ; chemical ; chemicals ) [SEP]'},
 {'mention_id': '28209945.41',
  'test_set_idx': 28530,
  'type': 'T033',
  'name': 'conducive',
  'context': '[CLS] > 0 . 05 ) , and compared with group B , H ##IF - 1 ##α and P - select ##in in group C were lower ( P < 0 . 01 ) ; 60 min after the transplant ##ation , H ##IF - 1 ##α , ch ##em ##oki ##ne C ##X ##CL ##14 , P - select ##in , and S ##nail in group C were lower than that in group B ( P < 0 . 01 ) . CO ##NC ##L ##US ##ION ##S h ##y ##pot ##her ##mic machine per ##fusion can significantly reduce the levels of o ##xi ##da ##tive stress factors and inflammatory factors , which is [unused1] con ##du ##ci ##ve [unused2] for liver transplant ##ation . [SEP]',
  'existing_cui': 'C3640666',
  'existing_cui_name': 'Urine Conductivity',
  'existing_cui_desc': '[CLS] U ##rine Con ##duct ##ivity [unused3] ( Health Care Act ##ivity : CO ##ND ##UC ##TU ) [SEP]',
  'our_bi_cui': 'C0562524',
  'our_bi_cui_name': 'Temperate environment',
  'our_bi_cui_desc': '[CLS] Te ##mper ##ate environment [unused3] ( Spa ##tial Concept , environment ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28127853.24',
  'test_set_idx': 26318,
  'type': 'T058',
  'name': 'program',
  'context': '[CLS] The " At ##ti ##tudes toward A ##ging Scale " and the " Spiritual Health Scale " were two assessment instruments used in the study . The results showed that there were significant improvements in the At ##ti ##tudes toward A ##ging Scale for the young adult group ( aged 18 - 29 years ) and in the Spiritual Health Scale for the older adult group ( aged 65 - 80 years ) . The evaluation showed that participants were satisfied with the [unused1] program [unused2] . The results of the present study provide future directions for successful aging and inter ##gene ##ration ##al learning . G ##eria ##tr G ##eron ##to ##l In ##t 2017 ; • • : • • - • • . [SEP]',
  'existing_cui': 'C0237807',
  'existing_cui_name': 'Mental health program',
  'existing_cui_desc': '[CLS] Mental health program [unused3] ( Organization , Mental Health : health mental programs ; Programs ; Mental Health Programs ; mental health programs ; mental health program ) [SEP]',
  'our_bi_cui': 'C0376691',
  'our_bi_cui_name': 'Programs - Publication Format',
  'our_bi_cui_desc': '[CLS] Programs - Public ##ation Form ##at [unused3] ( Intel ##lect ##ual Product : Programs ) [SEP]',
  'our_cross_cui': 'C0043113',
  'our_cross_cui_name': 'Wellness Programs',
  'our_cross_cui_desc': '[CLS] Well ##ness Programs [unused3] ( Health Care Act ##ivity : program well ##ness ; programs well ##ness ; Programs , Well ##ness ; Well ##ness Program ; well ##ness programs ; programming well ##ness ; Program , Well ##ness ; well ##ness program ) [SEP]'},
 {'mention_id': '28009593.23',
  'test_set_idx': 23576,
  'type': 'T098',
  'name': 'sample',
  'context': "[CLS] of Nursing Scale - R ##N version ( P ##ON ##S - R ##N ) was developed to measure nurses ' perception ##s of their ability to be present to their patients . This study sum ##mar ##izes the process of re - evaluation of the ps ##ych ##ometric properties of the P ##ON ##S - R ##N instrument . A [unused1] sample [unused2] of 76 registered nurses providing direct patient care responded to the 31 - item question ##naire . The Ra ##sch rating scale model was used for assessing construct validity of P ##ON ##S - R ##N data . A principal component analysis of residual ##s supported appropriate ##ness of the sub ##sca ##les defined by a 2 - dimensional structure . The [SEP]",
  'existing_cui': 'C0586999',
  'existing_cui_name': 'Vein sample',
  'existing_cui_desc': '[CLS] V ##ein sample [unused3] ( Ana ##tom ##ical Structure , specimen ) [SEP]',
  'our_bi_cui': 'C2348150',
  'our_bi_cui_name': 'Sample Study Population',
  'our_bi_cui_desc': '[CLS] Sam ##ple Study Population [unused3] ( Population Group ) [SEP]',
  'our_cross_cui': 'C1257890',
  'our_cross_cui_name': 'Population Group',
  'our_cross_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]'},
 {'mention_id': '27722881.39',
  'test_set_idx': 15026,
  'type': 'T007',
  'name': 'JPM1',
  'context': '[CLS] modeling method was applied to build the crystal structure of di ##ox ##y ##gen ##ase . Subsequently , the interaction mechanism between flu ##oran ##the ##ne and the active site of di ##ox ##y ##gen ##ase was simulated and analyzed by molecular dock ##ing . Consequently , a feasible de ##grading pathway of flu ##oran ##the ##ne in the strain [unused1] JP ##M ##1 [unused2] was proposed based on the meta ##bol ##ites and the interaction analyses . Additionally , the the ##rm ##ody ##nam ##ic analysis showed that the strain JP ##M ##1 had high tolerance for flu ##oran ##the ##ne , and the influence of flu ##oran ##the ##ne for the bacterial growth activity was ne ##gli ##gible under 100 to 400 mg L [SEP]',
  'existing_cui': 'C1421124',
  'existing_cui_name': 'TRGJP gene',
  'existing_cui_desc': '[CLS] T ##R ##G ##JP gene [unused3] ( Ana ##tom ##ical Structure : T - cell receptor , gamma , joining segment JP ; T cell receptor gamma joining P ) [SEP]',
  'our_bi_cui': 'C3629619',
  'our_bi_cui_name': 'bacterium JP1',
  'our_bi_cui_desc': '[CLS] bacterium JP ##1 [unused3] ( Ba ##cter ##ium ) [SEP]',
  'our_cross_cui': 'C4153571',
  'our_cross_cui_name': 'Microbacterium sp . JPM1',
  'our_cross_cui_desc': '[CLS] Micro ##ba ##cter ##ium s ##p . JP ##M ##1 [unused3] ( Ba ##cter ##ium ) [SEP]'},
 {'mention_id': '27802570.34',
  'test_set_idx': 17786,
  'type': 'T103',
  'name': 'soluble protein',
  'context': '[CLS] ) O ##2 ( - ) ( super ##oxide an ##ion ) production were effectively decreased in the range of 27 . 27 % to 54 . 38 % . Ph ##ys ##ico ##chemical characteristics showed that k ##i ##wi ##f ##ruit ##s from treated group maintained higher levels of flesh l ##umi ##nos ##ity and firm ##ness . Essential oils also decreased the levels of reducing sugar by 45 . 97 % at day 3 , and increased the content of [unused1] soluble protein [unused2] and h ##ydro ##ly ##zed amino acid . Therefore , post ##har ##ves ##t Essential oils treatment has positive effects on delay ##ing se ##nes ##cence and enhancing anti ##ox ##ida ##nt capacities in k ##i ##wi ##f ##ruit . [SEP]',
  'existing_cui': 'C1749467',
  'existing_cui_name': 'soluble',
  'existing_cui_desc': '[CLS] soluble [unused3] ( Ana ##tom ##ical Structure ) [SEP]',
  'our_bi_cui': 'C0074926',
  'our_bi_cui_name': 'Soy Proteins',
  'our_bi_cui_desc': '[CLS] So ##y Pro ##tein ##s [unused3] ( Chemical , substance : So ##y proteins ; Pro ##tein ##s , So ##y ; Diet ##ary So ##y ##be ##an Pro ##tein ##s ; So ##y protein ; So ##y Pro ##tein ; Diet ##ary So ##y ##be ##an Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; so ##y proteins ; protein so ##y ; S ##O ##Y PR ##OT ##EI ##N ; so ##y protein ; So ##ya protein ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28203549.107',
  'test_set_idx': 28299,
  'type': 'T170',
  'name': 'ST types',
  'context': '[CLS] accounted for more than three is ##olate ##s . These five P ##F ##GE clusters only accounted for 35 ( 41 . 7 % , 35 / 84 ) is ##olate ##s . In conclusion , our study first found that h ##yper ##tens ##ion and male patients with 41 - 50 years old were independent risk factors . The composition of [unused1] ST types [unused2] and P ##F ##GE clusters among K . pneumonia ##e K ##2 is ##olate ##s was more diverse than K ##1 is ##olate ##s . K ##1 and K ##2 h ##yper ##mu ##co ##vis ##co ##us K ##le ##bs ##iel ##la pneumonia ##e is ##olate ##s had respective specific profiles of v ##ir ##ule ##nce - associated genes . [SEP]',
  'existing_cui': 'C0449570',
  'existing_cui_name': 'Test type',
  'existing_cui_desc': '[CLS] Test type [unused3] ( Intel ##lect ##ual Product , attribute : test types ; testing types ; tests typing ; test type ; tests types ; test typing ; type test ; tests type ) [SEP]',
  'our_bi_cui': 'C0441756',
  'our_bi_cui_name': 'Genotype types',
  'our_bi_cui_desc': '[CLS] Gen ##otype types [unused3] ( Intel ##lect ##ual Product , qualifier value ) [SEP]',
  'our_cross_cui': 'C0449943',
  'our_cross_cui_name': 'Serotype',
  'our_cross_cui_desc': '[CLS] Ser ##otype [unused3] ( Intel ##lect ##ual Product , attribute , o ##bs ##er ##vable entity , qualifier value : se ##rov ##ar ; se ##rov ##ars ; Ser ##ova ##rs ; se ##rot ##ype ; se ##rot ##ype ##s ; Ser ##otype ##s ; Ser ##ova ##r ) [SEP]'},
 {'mention_id': '27601292.8',
  'test_set_idx': 11183,
  'type': 'T038',
  'name': 'metabolizing',
  'context': '[CLS] V ##ori ##cona ##zo ##le metabolism is influenced by severe inflammation : a prospective study During an infection or inflammation , several drug - [unused1] meta ##bol ##izing [unused2] enzymes in the liver are down - regulated , including c ##yt ##och ##rome P ##45 ##0 is ##o - enzymes . Since v ##ori ##cona ##zo ##le is extensively meta ##bol ##ized by c ##yt ##och ##rome P ##45 ##0 is ##o - enzymes , the metabolism of v ##ori ##cona ##zo ##le can be influenced during inflammation via reduced clearance of the drug , resulting in higher v ##ori ##cona ##zo ##le t ##rough concentrations . To investigate prospective ##ly the influence of inflammation on v ##ori ##cona ##zo ##le metabolism and v ##ori ##cona [SEP]',
  'existing_cui': 'C0085509',
  'existing_cui_name': 'Sulfur-Reducing Bacteria',
  'existing_cui_desc': '[CLS] Sul ##fu ##r - Red ##uc ##ing Ba ##cter ##ia [unused3] ( Ba ##cter ##ium , organism : Sul ##fu ##r Red ##uc ##ing Ba ##cter ##ia ; S ##U ##LF ##UR R ##ED ##UC ##ING BA ##CT ; Sul ##ph ##ur reducing bacteria ; Sub ##c ##lass Sul ##fu ##r Red ##uc ##ing Ba ##cter ##ia ; sulfur meta ##bol ##izing bacteria ; Sub ##c ##lass Sul ##ph ##ur Red ##uc ##ing Ba ##cter ##ia ; Sul ##fu ##r reducing bacteria ) [SEP]',
  'our_bi_cui': 'C0870883',
  'our_bi_cui_name': 'Metabolite',
  'our_bi_cui_desc': '[CLS] Met ##ab ##oli ##te [unused3] ( Chemical : meta ##bol ##ites ; meta ##bol ##ite ; Met ##ab ##oli ##tes ) [SEP]',
  'our_cross_cui': 'C0683140',
  'our_cross_cui_name': 'drug metabolism',
  'our_cross_cui_desc': '[CLS] drug metabolism [unused3] ( B ##iol ##og ##ic Fun ##ction : Drug Met ##ab ##olis ##m ; drug metabolic process ; drugs metabolism ; bio ##tra ##ns ##formation drug ; bio ##tra ##ns ##formation drugs ; metabolism drug ) [SEP]'},
 {'mention_id': '28234635.24',
  'test_set_idx': 29192,
  'type': 'T103',
  'name': 'LXR',
  'context': '[CLS] investigated potential molecular mechanisms of liver injury in p ##ediatric onset in ##test ##inal failure . Live ##r expression of canal ##icular p ##hos ##ph ##oli ##pid ( ABC ##B ##4 ) , bi ##le acid ( ABC ##B ##11 ) , and s ##tero ##l ( ABC ##G ##5 / 8 ) transport ##ers , their upstream regulator ##s [unused1] L ##X ##R [unused2] and FX ##R as well as pro - inflammatory c ##yt ##oki ##nes inter ##le ##uki ##n - 6 and tumor ne ##c ##rosis factor were investigated among patients with in ##test ##inal failure [ age median 3 . 8 ( I ##Q ##R 1 . 2 to 11 ) ] in relation to bio ##chemical and his ##to ##log ##ic [SEP]',
  'existing_cui': 'C4288814',
  'existing_cui_name': 'Liver X Receptor-Like Family',
  'existing_cui_desc': '[CLS] Live ##r X Re ##ceptor - Like Family [unused3] ( Chemical : L ##X ##R Family ; Nuclear Re ##ceptor Sub ##family 1 Group H ) [SEP]',
  'our_bi_cui': 'C0385463',
  'our_bi_cui_name': 'leptin receptor',
  'our_bi_cui_desc': '[CLS] le ##pt ##in receptor [unused3] ( Chemical : R ##EC ##EP ##T L ##EP ##TI ##N ; Re ##ceptor , O ##B ; O ##B - R ; L ##EP ##TI ##N R ##EC ##EP ##T ; O ##B Re ##ceptor ; L ##EP ##R ; CD ##29 ##5 Anti ##gens ; AN ##TI ##GE ##NS CD 295 ; Re ##ceptor ##s , Le ##pt ##in [ Chemical / In ##g ##red ##ient ] ; Le ##pt ##in Bin ##ding Pro ##tein ; Le ##pt ##in Re ##ceptor ; L ##EP ##R Pro ##tein ; Hu ##B ##21 ##9 ; Re ##ceptor , Le ##pt ##in ; Le ##pt ##in Re ##ceptor ##s ; O ##B R ##EC ##EP ##T ; Re ##ceptor ##s , Le [SEP]',
  'our_cross_cui': 'C4277606',
  'our_cross_cui_name': 'Liver X Receptors',
  'our_cross_cui_desc': '[CLS] Live ##r X Re ##ceptor ##s [unused3] ( Chemical : O ##xy ##ster ##ol Re ##ceptor ##s , Nuclear ; Live ##r X Re ##ceptor ; Nuclear O ##xy ##ster ##ol Re ##ceptor ##s ; Live ##r X Re ##ceptor ##s [ Chemical / In ##g ##red ##ient ] ) [SEP]'},
 {'mention_id': '27791169.3',
  'test_set_idx': 16856,
  'type': 'T017',
  'name': 'BPH9',
  'context': '[CLS] All ##eli ##c diversity in an n ##uc ##leo ##tide - binding and le ##uc ##ine - rich repeat gene [unused1] BP ##H ##9 [unused2] enables rice to combat plant ##hop ##per variation Brown plant ##hop ##per , Ni ##lap ##ar ##vat ##a l ##ugen ##s St ##å ##l , is one of the most devastating insect p ##ests of rice ( Or ##y ##za sat ##iva L . ) . Currently , 30 Brown plant ##hop ##per - resistance genes have been genetically defined , most of which are cluster ##ed on specific chromosome regions . Here , we describe molecular c ##lon ##ing and characterization of a Brown plant ##hop ##per - resistance gene , BP ##H ##9 , mapped on the long arm [SEP]',
  'existing_cui': 'C0005001',
  'existing_cui_name': 'Benign prostatic hypertrophy',
  'existing_cui_desc': '[CLS] Ben ##ign pro ##static h ##yper ##tro ##phy [unused3] ( B ##iol ##og ##ic Fun ##ction , BP ##H , ben ##ign , ben ##ign pro ##static h ##yper ##tro ##phy , diagnosis : H ##yper ##tro ##phy - pro ##st . ben . ; ben ##ign pro ##static h ##yper ##tro ##phy ; Pro ##static h ##yper - ben ##ign ; Ben ##ign Pro ##static H ##yper ##p ##lasia - BP ##H ; En ##lar ##ged Pro ##state ; H ##yper ##tro ##phy , Ben ##ign Pro ##static ; H ##yper ##tro ##phy ( ben ##ign ) of pro ##state ; Ben ##ign h ##yper ##tro ##phy of pro ##state ; BP ##H - ben ##ign pro ##static h ##yper ##tro ##phy ; Pro ##static [SEP]',
  'our_bi_cui': 'C1419294',
  'our_bi_cui_name': 'RBBP9 gene',
  'our_bi_cui_desc': '[CLS] RB ##B ##P ##9 gene [unused3] ( Ana ##tom ##ical Structure : RB ##B ##P ##10 ; B ##5 ##T O ##VE ##RE ##X ##PR ##ES ##SE ##D GE ##NE ; RB ##B ##P ##9 Gene ; R ##ET ##IN ##O ##BL ##AS ##TO ##MA - B ##IN ##DI ##NG PR ##OT ##EI ##N 9 ; Re ##tino ##blast ##oma Bin ##ding Pro ##tein 9 Gene ; re ##tino ##blast ##oma binding protein 9 ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '28523846.40',
  'test_set_idx': 38560,
  'type': 'T103',
  'name': 'proteoform',
  'context': '[CLS] unfolded pro ##te ##of ##orms of forced - reduced me ##case ##rm ##in and aerial ##ly o ##xi ##dized me ##case ##rm ##in with the free - SH : protein ratio of me ##case ##rm ##in drug product . The collision cross - section ##al area , which is determined using E ##SI - I ##MS - MS , provided [unused1] pro ##te ##of ##orm [unused2] information through rapid monitoring ( < 2 min ) of in - process samples during the manufacture of me ##case ##rm ##in . E ##SI - I ##MS - MS coupled with SEC - mode HP ##LC is a rapid and robust method for analyzing the free - SH : protein ratio of me ##case ##rm ##in that allows evaluating [SEP]',
  'existing_cui': 'C3658622',
  'existing_cui_name': 'NeoForm',
  'existing_cui_desc': '[CLS] Neo ##F ##orm [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0597298',
  'our_bi_cui_name': 'Protein Isoforms',
  'our_bi_cui_desc': '[CLS] Pro ##tein Is ##of ##orms [unused3] ( Chemical : protein is ##of ##orm ; Pro ##tein Is ##of ##orms [ Chemical / In ##g ##red ##ient ] ; Is ##of ##orms , Pro ##tein ; Is ##of ##orms ; is ##of ##orms ; is ##of ##orm ; Is ##of ##orm ) [SEP]',
  'our_cross_cui': 'C1510464',
  'our_cross_cui_name': 'Protein Structure',
  'our_cross_cui_desc': '[CLS] Pro ##tein Structure [unused3] ( Chemical : protein structure ) [SEP]'},
 {'mention_id': '27286174.25',
  'test_set_idx': 1488,
  'type': 'T033',
  'name': 'removal from the list',
  'context': '[CLS] ##6 adults listed for kidney transplant before December 31 , 2011 . Pre ##dict ##ors were age , sex , blood type , calculated panel - reactive antibodies , donation service area , dial ##ys ##is duration , com ##or ##bid conditions , and body mass index . Out ##come ##s were deceased or living donor transplant , death or [unused1] removal from the list [unused2] due to de ##ter ##ior ##ating medical condition , or removal due to other reasons . We calculated hazards for the possible outcomes , then the cumulative incidence function for a given candidate using competing risk methodology . Disc ##rim ##ination and ca ##li ##bra ##tion were assessed through C statistics and ca ##li ##bra ##tion plots for each cause [SEP]',
  'existing_cui': 'C4082098',
  'existing_cui_name': 'Patient status determination , deceased and removed from state',
  'existing_cui_desc': '[CLS] Pat ##ient status determination , deceased and removed from state [unused3] ( Finding , finding : Re ##mo ##val from state ) [SEP]',
  'our_bi_cui': 'C0015252',
  'our_bi_cui_name': 'removal technique',
  'our_bi_cui_desc': '[CLS] removal technique [unused3] ( Health Care Act ##ivity , procedure , qualifier value : re ##sect ; ex ##cision procedure ; surgical removal ; procedures removal ; ex ##ti ##rp ##ation ; re ##section ##ed ; re ##section ##s ; re ##section ##ing ; surgical re ##section ; removal ; ex ##cision ##s ; Re ##mo ##val - action ; ex ##cision procedures ; surgical removal procedure ; procedure removal ; ex ##cision ; re ##sect ##ed ; Re ##mo ##val - procedure ; removal ##s ; removal procedure ; Re ##mo ##val technique ; e ##ct ##omy ; Re ##mo ##val ; re ##section ; Re ##mo ##val Pro ##ced ##ure ; Re ##mo ##val procedure ) [SEP]',
  'our_cross_cui': 'C0420299',
  'our_cross_cui_name': 'Removed from waiting list',
  'our_cross_cui_desc': '[CLS] Re ##mo ##ved from waiting list [unused3] ( Finding , finding ) [SEP]'},
 {'mention_id': '28495651.19',
  'test_set_idx': 37794,
  'type': 'T204',
  'name': 'Schwartz',
  'context': '[CLS] 1987 , and December 20 , 1999 , and were followed for card ##iovascular death . The 4 major correction formula ##s ( Ba ##ze ##tt , Fr ##ider ##ici ##a , Fr ##ami ##ng ##ham , and Hodges ) were used to correct Q ##T interval . In addition , the percent ##iles for heart rate ranges as proposed by [unused1] Schwartz [unused2] were calculated . During median follow - up of 17 . 8 years , 45 ##5 card ##iovascular death events occurred . When compared to the other equations , Q ##T ##c Ba ##ze ##tt had the greatest dependence on heart rate ( R ( 2 ) = 0 . 18 ) . The hazard ratio ( 95 % confidence interval ) [SEP]',
  'existing_cui': 'C0036391',
  'existing_cui_name': 'Schwartz-Jampel Syndrome',
  'existing_cui_desc': '[CLS] Schwartz - Jam ##pel S ##yn ##drome [unused3] ( B ##iol ##og ##ic Fun ##ction , Schwartz - Jam ##pel syndrome , diagnosis , disorder : O ##ste ##och ##ond ##rom ##us ##cular d ##ys ##tro ##phy ; S ##J ##A S ##Y ##ND ##RO ##ME ; s ##ch ##wart ##z jam ##pel syndrome ; My ##oto ##nic My ##op ##athy , D ##war ##fi ##sm , Cho ##nd ##rod ##ys ##tro ##phy , And O ##cular And F ##ac ##ial A ##b ##nor ##mal ##ities ; Schwartz - Jam ##pel S ##yn ##drome , Type 1 ; M ##Y ##OT ##ON ##IC M ##Y ##OP ##AT ##H ##Y , D ##WA ##R ##FI ##SM , CH ##ON ##DR ##OD ##Y ##ST ##RO ##P ##H ##Y [SEP]',
  'our_bi_cui': 'C1261028',
  'our_bi_cui_name': 'Schwartze operation',
  'our_bi_cui_desc': '[CLS] Schwartz ##e operation [unused3] ( Health Care Act ##ivity , procedure ) [SEP]',
  'our_cross_cui': 'C0086418',
  'our_cross_cui_name': 'Homo sapiens',
  'our_cross_cui_desc': '[CLS] Ho ##mo sa ##pie ##ns [unused3] ( E ##uka ##ryo ##te , Tax ##onomy , organism , qualifier value : Humans ; Man , Modern ; humans ; man ; MA ##N ; Ho ##mo sa ##pie ##ns ( living organism ) [ Am ##bi ##guous ] ; ho ##mo sa ##pie ##ns ; Human , General ; Ho ##mo sa ##pie ##ns Linnaeus , 1758 ; Human ; Human - origin ; human ; Ho ##mo sa ##pie ##ns ( living organism ) ; Modern Man ; Man ) [SEP]'},
 {'mention_id': '27762482.18',
  'test_set_idx': 15896,
  'type': 'T074',
  'name': 'PDL',
  'context': '[CLS] in some patients , and several studies using the low non ##pur ##pura ##genic flu ##ence were reported . To deliver the sufficient high flu ##ence of a PD ##L without generating pu ##rp ##ura , we designed the fraction ##ation of high flu ##ence using five passes and longer pulse duration ( 6 mill ##ise ##con ##ds ) of a [unused1] PD ##L [unused2] in succession . A total of eight patients with er ##yt ##hem ##ato ##tel ##ang ##ie ##ct ##atic r ##osa ##cea were enrolled in this study ; all patients were treated with PD ##L 10 times at 2 - week intervals . E ##ry ##them ##a and te ##lang ##ie ##ct ##asi ##a scores , as well as improvement , were [SEP]',
  'existing_cui': 'C1414066',
  'existing_cui_name': 'DLG5 gene',
  'existing_cui_desc': '[CLS] DL ##G ##5 gene [unused3] ( Ana ##tom ##ical Structure : K ##IA ##A ##0 ##5 ##8 ##3 ; D ##IS ##CS LA ##R ##GE , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF , 5 ; P - d ##l ##g ; discs large ho ##mo ##log 5 ; P ##LA ##CE ##NT ##A AND PR ##OS ##TA ##TE DL ##G ) [SEP]',
  'our_bi_cui': 'C0031740',
  'our_bi_cui_name': 'Photochemotherapy',
  'our_bi_cui_desc': '[CLS] Photo ##che ##mother ##ap ##y [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : PD ##T - Photo ##dynamic therapy ; Photo ##che ##mother ##ap ##ies ; Photo ##dynamic The ##rap ##ies ; therapy , photo ##dynamic ; Therapy , Photo ##dynamic ; THE ##R P ##H ##OT ##OD ##Y ##NA ##MI ##C ; Photo ##rad ##iation Therapy ; photo ##dynamic therapy ; photo ##che ##mother ##ap ##y ; P ##H ##OT ##OC ##H ##EM ##OT ##H ##ER ; The ##rap ##ies , Photo ##dynamic ; Photo ##dynamic Therapy Pro ##ced ##ures ; neo ##p ##las ##m / cancer photo ##rad ##iation therapy ; Photo ##che ##mother ##ap ##y [ Am ##bi ##guous ] ; Photo ##dynamic therapy ; PD [SEP]',
  'our_cross_cui': 'C1289859',
  'our_cross_cui_name': 'Pulsed Dye Lasers',
  'our_cross_cui_desc': '[CLS] P ##ulse ##d D ##ye Las ##ers [unused3] ( Medical Devi ##ce , physical object : D ##ye Las ##ers , P ##ulse ##d ; P ##ulse ##d dye laser device ; Las ##er , P ##ulse ##d D ##ye ; D ##ye Las ##er , P ##ulse ##d ; Las ##ers , P ##ulse ##d D ##ye ; P ##ulse ##d D ##ye Las ##er ; pulsed dye laser ; P ##ulse ##d dye laser ) [SEP]'},
 {'mention_id': '28295414.8',
  'test_set_idx': 31546,
  'type': 'T033',
  'name': 'nonabstinent',
  'context': '[CLS] Dr ##ink ##ing Days and Dr ##ink ##ing Red ##uction ##s Among Heavy Dr ##ink ##ers in the CO ##MB ##IN ##E Study Recently , the Food and Drug Administration proposed to expand the options for primary end points in the development of medications for alcohol use disorder to include either a ##bs ##tine ##nce from alcohol or a [unused1] non ##ab ##st ##inen ##t [unused2] outcome : no heavy drinking days ( with a heavy drinking day defined as more than 3 drinks per day for women and more than 4 drinks per day for men [ > 3 / > 4 cut ##off ] ) . The Food and Drug Administration also suggested that 6 months would be the most appropriate length for a [SEP]',
  'existing_cui': 'C0457801',
  'existing_cui_name': 'Current non-drinker of alcohol',
  'existing_cui_desc': '[CLS] Current non - drink ##er of alcohol [unused3] ( Finding , finding , life style : alcohol does drinking ; Non - drink ##er ; Te ##eto ##tal ##ler ; non drink ##er ; A ##bs ##tain ##er ; Te ##eto ##tale ##r ; a ##bs ##tinent ; Non - Dr ##ink ##er ; Non - drink ##er alcohol ; Non ##dr ##ink ##er ; Does not drink alcohol ; Non - drink ##er ; Non - drink ##er of alcoholic beverage ; A ##bs ##tinent ; Never drinks ; Non - drink ##er alcohol ) [SEP]',
  'our_bi_cui': 'C0457801',
  'our_bi_cui_name': 'Current non-drinker of alcohol',
  'our_bi_cui_desc': '[CLS] Current non - drink ##er of alcohol [unused3] ( Finding , finding , life style : alcohol does drinking ; Non - drink ##er ; Te ##eto ##tal ##ler ; non drink ##er ; A ##bs ##tain ##er ; Te ##eto ##tale ##r ; a ##bs ##tinent ; Non - Dr ##ink ##er ; Non - drink ##er alcohol ; Non ##dr ##ink ##er ; Does not drink alcohol ; Non - drink ##er ; Non - drink ##er of alcoholic beverage ; A ##bs ##tinent ; Never drinks ; Non - drink ##er alcohol ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27990563.7',
  'test_set_idx': 23002,
  'type': 'T168',
  'name': 'item',
  'context': "[CLS] Select ##ive reaching in mac ##aque ##s : evidence for action - centred attention When a monkey select ##s a piece of food lying on the ground from among other viable objects in the near vicinity , only the desired [unused1] item [unused2] govern ##s the particular pattern and direction of the animal ' s reaching action . It would seem then that selection is an important component controlling the animal ' s action . But , we may ask , is the selection process in such cases imp ##er ##vious to the presence of other objects that could constitute potential obstacles to or constraints on movement execution ? And if it is , in fact , per ##vious to other objects , do they have [SEP]",
  'existing_cui': 'C0678118',
  'existing_cui_name': 'Transmission Electron Microscopy',
  'existing_cui_desc': '[CLS] Trans ##mission El ##ec ##tron Micro ##sco ##py [unused3] ( Health Care Act ##ivity , T ##EM : Micro ##sco ##py , El ##ec ##tron , Trans ##mission ; El ##ec ##tron Micro ##sco ##py , Trans ##mission ; Micro ##sco ##py , Trans ##mission El ##ec ##tron ; Trans ##mission electron micro ##sco ##py ; T ##RA ##NS ##MI ##SS ##ION E ##LE ##CT ##RO ##N MI ##CR ##OS ##CO ##P ##Y ; transmission electron micro ##sco ##py ) [SEP]',
  'our_bi_cui': 'C0567415',
  'our_bi_cui_name': 'Atom',
  'our_bi_cui_desc': '[CLS] At ##om [unused3] ( Chemical , substance , substance : atom ; atoms ) [SEP]',
  'our_cross_cui': 'C0016452',
  'our_cross_cui_name': 'Food',
  'our_cross_cui_desc': '[CLS] Food [unused3] ( Food , medication , substance : Food or Food Product ; foods ; F ##O ##OD OR F ##O ##OD PR ##OD ##UC ##T ; Food Product ; Food or Food Sub ##stance ; Food ##P ##rod ##uc ##t ; Foods ; food ) [SEP]'},
 {'mention_id': '28479312.1',
  'test_set_idx': 37495,
  'type': 'T058',
  'name': 'ESBL Nordmann/Dortet/Poirel-based protocol',
  'context': '[CLS] A modified [unused1] E ##SB ##L Nord ##mann / Do ##rte ##t / Po ##ire ##l - based protocol [unused2] to op ##ti ##mize early se ##psis management We evaluated a modification of a color ##ime ##tric test recently described by Do ##rte ##t et al . ( 2015 ) for the rapid detection of E ##SB ##L - producing En ##tero ##ba ##cter ##iac ##eae directly from positive blood cultures that requires less manipulation , materials and hands - on time . The simplified protocol showed a sensitivity and specific ##ity of 100 % and 95 . 7 % respectively . [SEP]',
  'existing_cui': 'C1444096',
  'existing_cui_name': 'ESBL',
  'existing_cui_desc': '[CLS] E ##SB ##L [unused3] ( Ba ##cter ##ium , organism : Extended spectrum beta - la ##ct ##ama ##se producing bacteria ) [SEP]',
  'our_bi_cui': 'C0282574',
  'our_bi_cui_name': 'Intellectual Property',
  'our_bi_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '27887578.32',
  'test_set_idx': 20863,
  'type': 'T170',
  'name': 'programme materials',
  'context': '[CLS] and Out ##come ##s Via Education programme was delivered to health professionals in mater ##nity hospitals in all seven Australian states and territories and modified for use internationally with pilot ##ing in Vietnam , Fiji , and the Netherlands ( with the assistance of the International Still ##birth Alliance , IS ##A ) . Mo ##di ##fication ##s were made to [unused1] programme materials [unused2] in consultation with local teams and included translation for the Vietnam programme . Part ##ici ##pants completed pre - and post - programme evaluation question ##naire ##s on knowledge and confidence on six key components of per ##ina ##tal death management as well as a satisfaction question ##naire . Over the period May 2012 to May 2015 , 30 I ##M [SEP]',
  'existing_cui': 'C0033333',
  'existing_cui_name': 'Program Development',
  'existing_cui_desc': '[CLS] Program Development [unused3] ( Health Care Act ##ivity , procedure : development program ; Well ##ness program development community ; w ##ln ##s p ##rg ##rm de ##v ##ment cm ##unt ; Program development ; program development ; Programme development ; Development , Program ) [SEP]',
  'our_bi_cui': 'C2936424',
  'our_bi_cui_name': 'Electronic Supplementary Materials',
  'our_bi_cui_desc': '[CLS] Electronic Su ##pp ##lement ##ary Materials [unused3] ( Intel ##lect ##ual Product : Su ##pp ##lement ##ary Material ; Electronic Su ##pp ##lement ##ary Material ; Supporting Information ; Su ##pp ##lement ##ary Information ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27837917.30',
  'test_set_idx': 18943,
  'type': 'T033',
  'name': 'unreinforced lever pressing ensued',
  'context': '[CLS] - administer cocaine ( n = 96 ) or were used as yo ##ked - sa ##line control animals ( n = 21 ) . After extinction , reins ##tate ##ment was initiated by 10 minutes of cue - induced drug seeking , followed by 45 minutes with contingent cocaine access , after which cocaine was discontinued and [unused1] un ##re ##in ##force ##d lever pressing ensued [unused2] . Three measures of trans ##ient s ##yna ##ptic potent ##iation were ass ##ay ##ed during reins ##tate ##ment : den ##dr ##itic spine morphology , alpha - amino - 3 - h ##ydro ##xy - 5 - met ##hyl - 4 - is ##ox ##az ##ole prop ##ion ##ic acid to N - met ##hyl - D [SEP]',
  'existing_cui': 'C1960601',
  'existing_cui_name': 'Head pressing',
  'existing_cui_desc': '[CLS] Head pressing [unused3] ( Finding , finding ) [SEP]',
  'our_bi_cui': 'C0557952',
  'our_bi_cui_name': 'Reinforcement of behavior',
  'our_bi_cui_desc': '[CLS] Re ##in ##force ##ment of behavior [unused3] ( Health Care Act ##ivity , procedure , regime / therapy : Re ##in ##force ##ment of behaviour ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28433448.29',
  'test_set_idx': 35668,
  'type': 'T062',
  'name': 'FDS2',
  'context': '[CLS] was determined in each sample . In F ##DS ##2 , associates of an ##emia were assessed using multiple log ##istic re ##gression and Cox proportional hazards modeling identified predict ##ors of death during 4 . 3 ± 1 . 2 years post - recruitment . The prevalence of an ##emia at base ##line was 11 . 5 % in [unused1] F ##DS ##2 [unused2] participants , 17 . 8 % in Bus ##sel ##ton Di ##abe ##tes Study type 2 patients and 5 . 4 % in Bus ##sel ##ton Di ##abe ##tes Study participants without diabetes . In F ##DS ##2 , 163 of 178 patients with an ##emia ( 91 . 6 % ) had at least one other risk factor ( serum [SEP]',
  'existing_cui': 'C4015495',
  'existing_cui_name': 'CEREBELLOFACIODENTAL SYNDROME',
  'existing_cui_desc': '[CLS] CE ##RE ##BE ##LL ##O ##FA ##CI ##OD ##EN ##TA ##L S ##Y ##ND ##RO ##ME [unused3] ( B ##iol ##og ##ic Fun ##ction : CE ##RE ##BE ##LL ##AR - FA ##CI ##AL - DE ##NT ##AL S ##Y ##ND ##RO ##ME ) [SEP]',
  'our_bi_cui': 'C0008972',
  'our_bi_cui_name': 'Clinical Research',
  'our_bi_cui_desc': '[CLS] Clinical Research [unused3] ( Research Act ##ivity : Clinical Studies ; Clinical Study ; C ##L ##IN ##IC ##AL ST ##U ##D ##Y ; clinical research ; clinical studies ; clinical study ; Study ) [SEP]',
  'our_cross_cui': 'C2603343',
  'our_cross_cui_name': 'Study',
  'our_cross_cui_desc': '[CLS] Study [unused3] ( Research Act ##ivity ) [SEP]'},
 {'mention_id': '27722881.26',
  'test_set_idx': 15013,
  'type': 'T007',
  'name': 'JPM1',
  'context': '[CLS] ##non ##e - 1 - car ##box ##yl ##ic acid , 9 - flu ##ore ##non ##e , p ##ht ##hali ##c acid , and ben ##zoic acid were detected in the culture solution . The gene sequence encoding the a ##romatic - ring - h ##ydro ##xy ##lating di ##ox ##y ##gen ##ase was am ##plified in the strain [unused1] JP ##M ##1 [unused2] by PC ##R . Based on the translated protein sequence , a ho ##mology modeling method was applied to build the crystal structure of di ##ox ##y ##gen ##ase . Subsequently , the interaction mechanism between flu ##oran ##the ##ne and the active site of di ##ox ##y ##gen ##ase was simulated and analyzed by molecular dock ##ing . Consequently , [SEP]',
  'existing_cui': 'C1421124',
  'existing_cui_name': 'TRGJP gene',
  'existing_cui_desc': '[CLS] T ##R ##G ##JP gene [unused3] ( Ana ##tom ##ical Structure : T - cell receptor , gamma , joining segment JP ; T cell receptor gamma joining P ) [SEP]',
  'our_bi_cui': 'C3629619',
  'our_bi_cui_name': 'bacterium JP1',
  'our_bi_cui_desc': '[CLS] bacterium JP ##1 [unused3] ( Ba ##cter ##ium ) [SEP]',
  'our_cross_cui': 'C4153571',
  'our_cross_cui_name': 'Microbacterium sp . JPM1',
  'our_cross_cui_desc': '[CLS] Micro ##ba ##cter ##ium s ##p . JP ##M ##1 [unused3] ( Ba ##cter ##ium ) [SEP]'},
 {'mention_id': '27932585.27',
  'test_set_idx': 22254,
  'type': 'T103',
  'name': 'Taw3',
  'context': '[CLS] , a ho ##mo ##log of T ##Y ##W ##3 from Sul ##fo ##lo ##bus so ##lf ##ata ##ric ##us , which revealed a novel α / β fold . The sequence motif ( S / T ) x ##SS ##C ##x ##GR and invariant as ##par ##tate and his ##ti ##dine , conserved in T ##Y ##W ##3 / [unused1] Ta ##w ##3 [unused2] , cluster to form the cat ##alytic center . These structural and sequence features indicate that T ##Y ##W ##3 / Ta ##w ##3 proteins constitute a distinct class of SA ##M - dependent met ##hyl ##tra ##ns ##fer ##ases . Using site - directed m ##uta ##genesis along with in v ##ivo complement ##ation ass ##ays combined with mass s [SEP]',
  'existing_cui': 'C3496263',
  'existing_cui_name': 'annectant white matter',
  'existing_cui_desc': '[CLS] an ##nect ##ant white matter [unused3] ( Ana ##tom ##ical Structure : Sub ##stant ##ia me ##du ##llar ##is an ##nect ##ens ) [SEP]',
  'our_bi_cui': 'C0039341',
  'our_bi_cui_name': 'tat Protein',
  'our_bi_cui_desc': '[CLS] ta ##t Pro ##tein [unused3] ( Chemical : ta ##t Gene Product ; Gene Products , ta ##t [ Chemical / In ##g ##red ##ient ] ; Trans ##act ##ivating Reg ##ulatory Pro ##tein ; T ##AT Pro ##tein ; Trans - Act ##ivation of Trans ##cription Pro ##tein ; Trans - Act ##iva ##tor of Trans ##cription of HIV ; ta ##t Gene Products ; ta ##t protein ; Gene Product , ta ##t ; Gene Products , ta ##t ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27251800.4',
  'test_set_idx': 799,
  'type': 'T058',
  'name': 'Stoppa approaches',
  'context': '[CLS] Early assessment of bilateral ing ##uin ##al her ##nia repair : A comparison between the lap ##aro ##scopic total extra ##per ##ito ##nea ##l and [unused1] Stop ##pa approaches [unused2] The present clinical trial was designed to compare the results of bilateral ing ##uin ##al her ##nia repair between patients who underwent the conventional Stop ##pa technique and lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair with a single mesh and without staple fix ##ation . This controlled , random ##ised clinical trial was conducted at General Surgery and T ##ra ##uma of the Clinic ##s Hospital , Medical School , the University of S ##γ ##o Paulo between September 2010 and February 2011 . Total ##ly , 50 male patients , with a [SEP]',
  'existing_cui': 'C0449445',
  'existing_cui_name': 'Approach',
  'existing_cui_desc': '[CLS] A ##pp ##roach [unused3] ( Spa ##tial Concept , attribute , qualifier value : A ##pp ##roach ##es ; approach ; approaches ; approaching ) [SEP]',
  'our_bi_cui': 'C0205501',
  'our_bi_cui_name': 'Transperitoneal approach',
  'our_bi_cui_desc': '[CLS] Trans ##per ##ito ##nea ##l approach [unused3] ( Spa ##tial Concept , qualifier value : Trans ##per ##ito ##nea ##l ; Abd ##omi ##nal trans ##per ##ito ##nea ##l approach ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27580517.33',
  'test_set_idx': 10643,
  'type': 'T058',
  'name': '3D cylindrically - distributed spiral pseudo - continuous arterial spin labeling magnetic resonance imaging',
  'context': '[CLS] two remaining patients , cerebral blood flow decreased by 2 . 4 to 6 . 0 % after ace ##ta ##zo ##lam ##ide . The group average change in cerebral blood flow due to ace ##ta ##zo ##lam ##ide was approximately 25 . 0 % and individual changes were statistical ##ly significant ( p < 0 . 01 ) in all patients using a paired t - test analysis . cerebral blood flow per ##fusion data were diagnostic ##ally useful in supporting conventional MR an ##gio ##graphy and clinical findings . [unused1] 3D cylindrical ##ly - distributed spiral pseudo - continuous art ##erial spin label ##ing magnetic resonance imaging [unused2] provides a robust approach to assess cerebral blood flow and reserve in p ##ediatric patients . [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0860888',
  'our_bi_cui_name': 'Tomography , Spiral Computed',
  'our_bi_cui_desc': '[CLS] Tom ##ography , S ##pi ##ral Co ##mp ##uted [unused3] ( Health Care Act ##ivity , procedure : S ##pi ##ral com ##puted to ##mo ##graphy scan ; S ##pi ##ral CT ; CT ##s , S ##pi ##ral ; com ##puted to ##mo ##graphy spiral ; Sc ##ans , S ##pi ##ral CT ; He ##lica ##l CT ##s ; He ##lica ##l CT ; S ##pi ##ral CA ##T Sc ##ans ; he ##lica ##l com ##puted to ##mo ##graphy ; c ##t spiral ; S ##pi ##ral Volume ##tric CT ; S ##pi ##ral Computer ##ized Tom ##ography ; Computer - As ##sist ##ed Tom ##ography , S ##pi ##ral ; Volume ##tric CT ; He ##lica ##l Co ##mp ##uted Tom ##ography [SEP]',
  'our_cross_cui': 'C3891302',
  'our_cross_cui_name': 'Arterial Spin Labeling Magnetic Resonance Imaging',
  'our_cross_cui_desc': '[CLS] Arte ##rial Spin Label ##ing Ma ##gnetic Re ##son ##ance I ##maging [unused3] ( Health Care Act ##ivity : AS ##L f ##MR ##I ; Arte ##rial Spin Label ##ing MR ##I ; AR ##TE ##RI ##AL SP ##IN LA ##BE ##L ##ING F ##UN ##CT ##ION ##AL MR ##I ) [SEP]'},
 {'mention_id': '27884439.25',
  'test_set_idx': 20661,
  'type': 'T017',
  'name': 'HTLTC',
  'context': '[CLS] ##rp ##er and SA Mu ##tton Me ##rino breeds . U ##dder s were scored subjective ##ly on a 1 - 5 scale ( 1 - u ##dder intact and 5 - u ##dder damaged severely ) and t ##icks were counted on three locations . The body sites counted were the head and th ##ora ##ci ##c limb ( [unused1] H ##TL ##TC [unused2] ) , u ##dder - p ##el ##vic limb ( UP ##LT ##C ) and per ##ine ##um - br ##ee ##ch - tail ( P ##B ##TT ##C ) . These counts were also totaled for a total t ##ick count . Rep ##rod ##uction traits were number of la ##mbs we ##ane ##d per e ##we la ##mbe ##d [SEP]',
  'existing_cui': 'C1415797',
  'existing_cui_name': 'HTL gene',
  'existing_cui_desc': '[CLS] H ##TL gene [unused3] ( Ana ##tom ##ical Structure : L ##E ##UC ##IN ##E T ##RA ##NS ##PO ##RT , H ##IG ##H ; high L - le ##uc ##ine transport ; H ##IG ##H L - L ##E ##UC ##IN ##E T ##RA ##NS ##PO ##RT ) [SEP]',
  'our_bi_cui': 'C0326478',
  'our_bi_cui_name': 'Long-tailed tit',
  'our_bi_cui_desc': '[CLS] Long - tailed t ##it [unused3] ( E ##uka ##ryo ##te , organism : A ##eg ##ith ##alo ##s ca ##uda ##tus ; Long - tailed T ##it ; A ##eg ##ith ##alo ##s ca ##uda ##tus ( Linnaeus , 1758 ) ; Long - tailed bush ##ti ##t ) [SEP]',
  'our_cross_cui': 'C0229962',
  'our_cross_cui_name': 'Body part',
  'our_cross_cui_desc': '[CLS] Body part [unused3] ( Ana ##tom ##ical Structure , body structure : Body Parts ; Body part structure ; Pa ##rs co ##rp ##oris ; body part ; body parts ; Body region ; Ana ##tom ##ic part . * ; Body Structure ; Parts , Body ; Ana ##tom ##ic part ; body region ; Cardinal body part ; Body Part ) [SEP]'},
 {'mention_id': '28468980.13',
  'test_set_idx': 37007,
  'type': 'T103',
  'name': 'PfSEA-1A',
  'context': '[CLS] P ##f ##SE ##A - 1A are associated with protection against severe malaria in children . V ##ac ##cin ##ation of mice with P ##b ##SE ##A - 1A decreases parasite ##mia and pro ##long ##s survival following P ##las ##mo ##dium be ##rg ##he ##i AN ##KA challenge . To enhance the im ##mu ##no ##genic ##ity of [unused1] P ##f ##SE ##A - 1A [unused2] , we identified five linear , B - cell e ##pit ##ope ##s using p ##eptide micro ##arra ##ys probe ##d with anti - se ##ra from r ##P ##f ##SE ##A - 1A - v ##ac ##cin ##ated non - human p ##rima ##tes . We evaluated the relationship between e ##pit ##ope - specific anti ##body levels [SEP]',
  'existing_cui': 'C0120659',
  'existing_cui_name': 'HA-1A',
  'existing_cui_desc': '[CLS] H ##A - 1A [unused3] ( Chemical , product , substance ) [SEP]',
  'our_bi_cui': 'C4040232',
  'our_bi_cui_name': 'Plasmodium Ag',
  'our_bi_cui_desc': '[CLS] P ##las ##mo ##dium A ##g [unused3] ( Chemical , substance : P ##las ##mo ##dium anti ##gen ; Anti ##gen of P ##las ##mo ##dium ) [SEP]',
  'our_cross_cui': 'C3849939',
  'our_cross_cui_name': 'SEA-1 protein , Plasmodium falciparum',
  'our_cross_cui_desc': '[CLS] SE ##A - 1 protein , P ##las ##mo ##dium f ##al ##ci ##par ##um [unused3] ( Chemical : P ##f ##SE ##A - 1 protein , P ##las ##mo ##dium f ##al ##ci ##par ##um ; s ##chi ##zon ##t e ##g ##ress anti ##gen - 1 , P ##las ##mo ##dium f ##al ##ci ##par ##um ) [SEP]'},
 {'mention_id': '27756117.44',
  'test_set_idx': 15641,
  'type': 'T103',
  'name': 'PI',
  'context': '[CLS] . 4 % for the 2 . 8 % , 1 . 5 % and 0 . 8 % fi ##bri ##no ##gen - containing silk s ##po ##nges , respectively . Moreover , the fi ##bri ##n formation lead to a more linear elastic behavior over longer strain ranges . In combination , the Cal ##ce ##in - AM / [unused1] P ##I [unused2] stain ##ings and M ##TT ass ##ay results indicate uniform cell ad ##hesion on the surface and c ##yt ##oc ##om ##pa ##ti ##bility of the silk / fi ##bri ##n s ##po ##nges , respectively . Moreover , the co - delivery of th ##rom ##bin with fi ##bri ##no ##gen via silk as carrier material is described , offering [SEP]',
  'existing_cui': 'C1418544',
  'existing_cui_name': 'SERPINA1 gene',
  'existing_cui_desc': '[CLS] SE ##RP ##IN ##A ##1 gene [unused3] ( Ana ##tom ##ical Structure : SE ##RP ##IN ##A ##1 , ; se ##rp ##in family A member 1 ; A1 ##AT ; pro ##te ##ase inhibitor 1 ( anti - el ##ast ##ase ) , alpha - 1 - anti ##try ##ps ##in ; alpha - 1 - anti ##try ##ps ##in ; Ser ##pin P ##ept ##idas ##e In ##hibit ##or , C ##lade A ( Alpha - 1 Anti ##p ##rote ##inas ##e , Anti ##try ##ps ##in ) , Member 1 Gene ; SE ##RP ##IN P ##EP ##TI ##DA ##SE IN ##H ##IB ##IT ##OR , C ##LA ##DE A , ME ##MB ##ER 1 ; AL ##P ##HA - 1 - AN [SEP]',
  'our_bi_cui': 'C4015812',
  'our_bi_cui_name': 'PI I PHENOTYPE',
  'our_bi_cui_desc': '[CLS] P ##I I P ##H ##EN ##OT ##YP ##E [unused3] ( Finding : P ##I I ) [SEP]',
  'our_cross_cui': 'C0033470',
  'our_cross_cui_name': 'Propidium Iodide',
  'our_cross_cui_desc': '[CLS] Pro ##pid ##ium I ##od ##ide [unused3] ( Chemical : Pro ##pid ##ium Di ##io ##di ##de ; I ##od ##ide , Pro ##pid ##ium ; Di ##io ##di ##de , Pro ##pid ##ium ) [SEP]'},
 {'mention_id': '28421738.22',
  'test_set_idx': 35200,
  'type': 'T103',
  'name': '12',
  'context': '[CLS] designed to simultaneously activate transcription - based anti ##ox ##ida ##nt responses , whose lead 1 showed interesting properties in a preliminary investigation . Re ##lying on the requirements of am ##yl ##oid - β p ##eptide recognition , we identified the cat ##ech ##ol derivative 12 . In SH - S ##Y ##5 ##Y ne ##uro ##blast ##oma cells , [unused1] 12 [unused2] combined remarkable free radical s ##ca ##venge ##r properties to the ability to trigger the Nr ##f ##2 pathway and induce the Nr ##f ##2 - dependent defensive gene N ##Q ##O ##1 by means of electro ##phi ##lic activation of the transcription ##al response . Moreover , 12 prevented the formation of c ##yt ##oto ##xi ##c stable o ##li ##go [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0030956',
  'our_bi_cui_name': 'Peptides',
  'our_bi_cui_desc': '[CLS] P ##eptide ##s [unused3] ( Chemical , substance : p ##eptide ; p ##eptide ##s ; P ##eptide ##s [ Chemical / In ##g ##red ##ient ] ; P ##eptide ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27500205.24',
  'test_set_idx': 8456,
  'type': 'T103',
  'name': 'kindlins',
  'context': '[CLS] ad ##here ##nt cells , and kind ##lin - 3 is expressed primarily in hem ##ato ##po ##iet ##ic cells . These distributions are not exclusive ; some cells express multiple kind ##lins , and transformed cells often exhibit a ##ber ##rant expression , both in the is ##of ##orms and the levels of kind ##lins . Great interest in the [unused1] kind ##lins [unused2] has emerged from the recognition that they play major roles in controlling in ##te ##g ##rin function . In v ##it ##ro studies , in v ##ivo studies of mice def ##icient in kind ##lins , and studies of patients with genetic def ##ici ##encies of kind ##lins have clearly established that they regulate the capacity of in ##te ##g ##rin [SEP]',
  'existing_cui': 'C1720776',
  'existing_cui_name': 'Kindling , Neurologic',
  'existing_cui_desc': '[CLS] Kind ##ling , N ##eur ##olo ##gic [unused3] ( B ##iol ##og ##ic Fun ##ction , N ##eur ##ology : kind ##ling ; Kind ##lings , N ##eur ##olo ##gic ; Kind ##ling ; N ##eur ##olo ##gic Kind ##lings ; N ##eur ##olo ##gic Kind ##ling ) [SEP]',
  'our_bi_cui': 'C0010422',
  'our_bi_cui_name': 'Crystallins',
  'our_bi_cui_desc': '[CLS] Crystal ##lins [unused3] ( Chemical , substance : Crystal ##lins [ Chemical / In ##g ##red ##ient ] ; Crystal ##lin ; Pro ##tein ##s , Len ##s ; C ##rist ##all ##in ; Len ##s Pro ##tein ##s ; crystal ##lin ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27533401.2',
  'test_set_idx': 9619,
  'type': 'T170',
  'name': 'tool',
  'context': '[CLS] Help end the HIV epidemic After many delays , the public consultation on HIV prevention [unused1] tool [unused2] pre - exposure prop ##hyl ##ax ##is is finally here . Now is the time to mob ##ilis ##e - help end the HIV epidemic by responding to this NHS England consultation before 23 September . [SEP]',
  'existing_cui': 'C1710438',
  'existing_cui_name': 'Tool Mouse',
  'existing_cui_desc': '[CLS] Too ##l Mouse [unused3] ( E ##uka ##ryo ##te ) [SEP]',
  'our_bi_cui': 'C0025663',
  'our_bi_cui_name': 'Methods',
  'our_bi_cui_desc': '[CLS] Method ##s [unused3] ( Intel ##lect ##ual Product , attribute : methods ; Method ; method ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27402480.28',
  'test_set_idx': 4825,
  'type': 'T033',
  'name': 'non - genotoxic',
  'context': '[CLS] following the principles of the O ##EC ##D test guide ##line 48 ##7 in v ##it ##ro ma ##mmal ##ian cell micro ##nu ##cle ##us test . Prof ##iciency with the ass ##ay was established by correctly identifying direct - acting g ##eno ##to ##xin ##s and g ##eno ##to ##xin ##s requiring metabolism , as well as [unused1] non - g ##eno ##to ##xi ##c [unused2] / non - car ##cin ##ogenic chemicals . We also report the analysis of our historical control data that demonstrate vehicle control and positive control values for % micro ##nu ##cle ##i in bin ##uc ##lea ##ted cells are in the ranges reported previously . Technical issues including evaluating various solvent ##s with both 48 ##h and 72 ##h [SEP]',
  'existing_cui': 'C1257899',
  'existing_cui_name': 'Genotoxic Stress',
  'existing_cui_desc': '[CLS] Gen ##oto ##xi ##c St ##ress [unused3] ( B ##iol ##og ##ic Fun ##ction : Gen ##oto ##xi ##c St ##resses ; St ##resses , Gen ##oto ##xi ##c ; St ##ress , Gen ##oto ##xi ##c ) [SEP]',
  'our_bi_cui': 'C2827065',
  'our_bi_cui_name': 'Non-Cytotoxic Agent',
  'our_bi_cui_desc': '[CLS] Non - Cy ##to ##to ##xi ##c Agent [unused3] ( Chemical ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27692319.52',
  'test_set_idx': 14323,
  'type': 'T103',
  'name': 'ABZ ( SO )',
  'context': '[CLS] ##hibition of mi ##rac ##id ##ia formation . In contrast , on the 12th day , 90 % of the mi ##rac ##id ##ia hatch ##ed in the control experiments using 0 . 03 % D ##MS ##O whereas em ##b ##ryo ##genesis was completely abolished with any concentration of al ##ben ##da ##zo ##le su ##l ##ph ##oxide [unused1] AB ##Z ( S ##O ) [unused2] . Chemical analysis of the crude extract and sub - fraction ##s revealed a prominent presence of fl ##av ##ono ##ids . HP ##LC - MS confirmed Que ##rc ##eti ##n to be one of the main fl ##av ##ono ##ids present in the crude extract and the n - but ##ano ##l sub ##fra ##ction . This is [SEP]',
  'existing_cui': 'C0967870',
  'existing_cui_name': "N ' - ( glutathion-S-yl ) - N-acetylbenzidine S-oxide",
  'existing_cui_desc': "[CLS] N ' - ( g ##lut ##ath ##ion - S - y ##l ) - N - ace ##ty ##l ##ben ##zi ##dine S - oxide [unused3] ( Chemical : AB ##Z - S ##G ) [SEP]",
  'our_bi_cui': 'C0069763',
  'our_bi_cui_name': 'oxibendazole',
  'our_bi_cui_desc': '[CLS] o ##xi ##ben ##da ##zo ##le [unused3] ( Chemical , substance : O ##xy ##ben ##da ##zo ##le ; met ##hyl 5 - [ n - prop ##ox ##y ] ben ##zi ##mi ##da ##zo ##le - 2 - car ##ba ##mate ; O ##X ##IB ##EN ##DA ##Z ##OL ##E ; Met ##hyl ( 5 - prop ##ox ##y - 1 ##H - ben ##zi ##mi ##da ##zo ##l - 2 - y ##l ) car ##ba ##mate ; O ##xi ##ben ##da ##zo ##le ; O ##B ##Z ; O ##xi ##ben ##da ##zo ##lo ; met ##hyl 5 - n - prop ##ox ##y - 2 - ben ##zi ##mi ##da ##zo ##le ##car ##ba ##mate ; Met ##hyl 5 - prop ##ox [SEP]',
  'our_cross_cui': 'C0051091',
  'our_cross_cui_name': 'albendazole sulfoxide',
  'our_cross_cui_desc': '[CLS] al ##ben ##da ##zo ##le su ##lf ##oxide [unused3] ( Chemical : Rico ##ben ##da ##zo ##le ; r ##ico ##ben ##da ##zo ##le ; R ##y ##co ##ben ##da ##zo ##le ) [SEP]'},
 {'mention_id': '27702870.9',
  'test_set_idx': 14654,
  'type': 'T058',
  'name': 'procedures',
  'context': '[CLS] ##rou ##nd Flow cat ##he ##ter from the time of its introduction in May 2015 . Pro ##ced ##ure - related data were collected prospective ##ly for the first 100 a ##bla ##tion procedures performed in our department using the Smart ##T ##ou ##ch ( ® ) Sur ##rou ##nd Flow cat ##he ##ter . From a database of 65 ##4 [unused1] procedures [unused2] performed in our unit using the Sur ##rou ##nd Flow cat ##he ##ter , we selected one to match each Smart ##T ##ou ##ch ( ® ) Sur ##rou ##nd Flow procedure , matching for procedure type , operator experience , patient age , and gender . The groups were well matched for patient age , gender , and procedure type . [SEP]',
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '28424986.28',
  'test_set_idx': 35482,
  'type': 'T058',
  'name': 'procedure',
  'context': '[CLS] ##erial em ##bol ##ization twice in order to shrink the tumor ; however , no effect was obtained . The tumor had rapidly grown to 50 cm and caused abnormal ##ities in the hem ##ato ##logical and co ##agu ##lative systems . Pre ##oper ##ative com ##puted to ##mo ##graphy revealed that the right he ##pa ##tic vein , right he ##pa ##tic artery , and right portal vein were not involved by the hem ##ang ##io ##ma . Re ##section of the giant liver hem ##ang ##io ##ma was successfully performed after in ##tra ##oper ##ative intentional blood ##lett ##ing with concurrent blood salvage . All hem ##ato ##logical and co ##agu ##lative abnormal ##ities had returned to normal after the [unused1] procedure [unused2] . [SEP]',
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27542380.6',
  'test_set_idx': 9947,
  'type': 'T007',
  'name': 'Ferrovum " sp . JA12',
  'context': '[CLS] ##op ##hil ##ic iron o ##xi ##di ##zer En ##e - red ##uc ##tase ##s originating from ex ##tre ##mo ##phi ##les are gaining importance in the field of bio ##cat ##aly ##sis due to higher - stability properties . The genome of the acid ##op ##hil ##ic iron - o ##xi ##di ##zing bacterium " [unused1] Fe ##rro ##vu ##m " s ##p . J ##A ##12 [unused2] was found to harbor a the ##rm ##op ##hil ##ic - like en ##e - red ##uc ##tase ( F ##O ##Y ##E - 1 ) . The f ##oy ##e - 1 gene was l ##igate ##d into a p ##ET ##16 ##b ##p expression vector system , and the enzyme was produced in E ##scher [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0004611',
  'our_bi_cui_name': 'Bacteria',
  'our_bi_cui_desc': '[CLS] Ba ##cter ##ia [unused3] ( Ba ##cter ##ium , organism : Super ##king ##dom Ba ##cter ##ia ; Domain Ba ##cter ##ia ; Ba ##cter ##ium ; BA ##CT ##ER ##IA ; bacterium ; Ba ##cter ##ia , E ##uba ##cter ##ia ; Ba ##cter ##ia . ; e ##uba ##cter ##ia ; BA ##CT ; not Ba ##cter ##ia Ha ##eck ##el 1894 ; bacteria ; E ##uba ##cter ##ia ) [SEP]',
  'our_cross_cui': 'C3908921',
  'our_cross_cui_name': 'Ferrovum sp . JA12',
  'our_cross_cui_desc': '[CLS] Fe ##rro ##vu ##m s ##p . J ##A ##12 [unused3] ( Ba ##cter ##ium ) [SEP]'},
 {'mention_id': '27722881.15',
  'test_set_idx': 15002,
  'type': 'T007',
  'name': 'JPM1',
  'context': '[CLS] , a gram - positive flu ##oran ##the ##ne - de ##grading bacterial strain was isolated from crude oil in Da ##gan ##g Oil ##field and identified as Micro ##ba ##cter ##ium para ##ox ##yd ##ans JP ##M ##1 by the analysis of 16 ##S r ##D ##NA sequence . After 25 days of in ##cu ##bation , the strain [unused1] JP ##M ##1 [unused2] could de ##grade 91 . 78 % of the initial amount of flu ##oran ##the ##ne . Moreover , four meta ##bol ##ites 9 - flu ##ore ##non ##e - 1 - car ##box ##yl ##ic acid , 9 - flu ##ore ##non ##e , p ##ht ##hali ##c acid , and ben ##zoic acid were detected in the culture solution [SEP]',
  'existing_cui': 'C1421124',
  'existing_cui_name': 'TRGJP gene',
  'existing_cui_desc': '[CLS] T ##R ##G ##JP gene [unused3] ( Ana ##tom ##ical Structure : T - cell receptor , gamma , joining segment JP ; T cell receptor gamma joining P ) [SEP]',
  'our_bi_cui': 'C3802218',
  'our_bi_cui_name': 'Bacillus sp . JP-1',
  'our_bi_cui_desc': '[CLS] Ba ##ci ##llus s ##p . JP - 1 [unused3] ( Ba ##cter ##ium ) [SEP]',
  'our_cross_cui': 'C4153571',
  'our_cross_cui_name': 'Microbacterium sp . JPM1',
  'our_cross_cui_desc': '[CLS] Micro ##ba ##cter ##ium s ##p . JP ##M ##1 [unused3] ( Ba ##cter ##ium ) [SEP]'},
 {'mention_id': '28113841.1',
  'test_set_idx': 25831,
  'type': 'T058',
  'name': 'Quantitative Susceptibility Mapping',
  'context': '[CLS] [unused1] Q ##uant ##itative Su ##s ##ce ##pt ##ibility Map ##ping [unused2] using St ##ructural Feature based Col ##la ##bor ##ative Reconstruction in the Human Brain The reconstruction of MR quantitative su ##s ##ce ##pt ##ibility mapping from local phase measurements is an ill posed inverse problem and different regular ##ization strategies incorporating a prior ##i information extracted from magnitude and phase images have been proposed . However , the anatomy observed in magnitude and phase images does not always coincide spatial ##ly with that in su ##s ##ce ##pt ##ibility maps , which could give er ##rone ##ous est ##imation in the reconstructed su ##s ##ce ##pt ##ibility map . In this paper , we develop a structural feature based collaborative reconstruction ( St ##ructural [SEP]',
  'existing_cui': 'C0237904',
  'existing_cui_name': 'Hypnotic Susceptibility',
  'existing_cui_desc': '[CLS] H ##y ##p ##not ##ic Su ##s ##ce ##pt ##ibility [unused3] ( Health Care Act ##ivity , H ##y ##p ##not ##ic : Su ##s ##ce ##pt ##ibility ) [SEP]',
  'our_bi_cui': 'C4288325',
  'our_bi_cui_name': 'Quantitative Ultrasound Spectroscopy',
  'our_bi_cui_desc': '[CLS] Q ##uant ##itative Ultra ##sound S ##pect ##ros ##copy [unused3] ( Health Care Act ##ivity : Q ##uant ##itative Ultra ##sound I ##maging ) [SEP]',
  'our_cross_cui': 'C0006117',
  'our_cross_cui_name': 'Brain Mapping',
  'our_cross_cui_desc': '[CLS] Brain Map ##ping [unused3] ( Health Care Act ##ivity , procedure : brain mapped ; Brain mapping ; brain map ; Map ##ping , Brain ; brain mapping ) [SEP]'},
 {'mention_id': '27837917.48',
  'test_set_idx': 18961,
  'type': 'T033',
  'name': 'lever pressing',
  'context': '[CLS] , when cocaine use was discontinued 45 minutes later , den ##dr ##itic spine morphology and alpha - amino - 3 - h ##ydro ##xy - 5 - met ##hyl - 4 - is ##ox ##az ##ole prop ##ion ##ic acid to N - met ##hyl - D - as ##par ##tate ratios were restored as animals became motivated to engage un ##rew ##arded [unused1] lever pressing [unused2] . Non ##re ##in ##force ##d drug seeking was positively correlated with changes in spine morphology , and cocaine access reversed this relationship . Using a novel modification of the reins ##tate ##ment paradigm , we show that achieving cocaine use reversed the s ##yna ##ptic plastic ##ity under ##pin ##ning the motivation to seek the drug . [SEP]',
  'existing_cui': 'C1960601',
  'existing_cui_name': 'Head pressing',
  'existing_cui_desc': '[CLS] Head pressing [unused3] ( Finding , finding ) [SEP]',
  'our_bi_cui': 'C0009651',
  'our_bi_cui_name': 'Operant Conditioning',
  'our_bi_cui_desc': '[CLS] Opera ##nt Con ##dition ##ing [unused3] ( B ##iol ##og ##ic Fun ##ction , Opera ##nt , procedure , regime / therapy : L ##EA ##R ##NI ##NG IN ##ST ##R ##UM ; conditioning opera ##nt ; Opera ##nt Con ##dition ##ings ; Con ##dition ##ing , Opera ##nt ; Learning , Instrumental ; Learning ##s , Instrumental ; IN ##ST ##R ##UM L ##EA ##R ##NI ##NG ; Opera ##nt conditioning ; Con ##dition ##ing ; Instrumental Con ##dition ##ing ; Instrumental Learning ; Con ##dition ##ings , Opera ##nt ; opera ##nt conditioning ; opera ##nt condition ; Instrumental Learning ##s ; instrumental conditioning ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28424683.42',
  'test_set_idx': 35420,
  'type': 'T038',
  'name': 'expanding',
  'context': '[CLS] killer cells while limiting T cell expansion for up to 60 days . Similar to patient Natural killer cells , Natural killer cells from tumor - bearing B ##LT - human ##ized mice promote faster expansion of residual T cells resulting in decreased numbers and function of Natural killer cells , whereas Natural killer cells from mice with no tumor continue [unused1] expanding [unused2] Natural killer cells and retain their c ##yt ##oto ##xi ##city . In addition , den ##dr ##itic cells in contrast to O ##Cs are found to promote faster expansion of residual T cells within pu ##rified Natural killer cells resulting in the decline in Natural killer cell numbers from healthy individuals . Ad ##dition of anti - CD ##3 m ##A [SEP]',
  'existing_cui': 'C0205229',
  'existing_cui_name': 'Expanding',
  'existing_cui_desc': '[CLS] Ex ##pan ##ding [unused3] ( Spa ##tial Concept , qualifier value : Ex ##pan ##d ; Ex ##pan ##ded ; expanding ; expand ; expanded ) [SEP]',
  'our_bi_cui': 'C0205229',
  'our_bi_cui_name': 'Expanding',
  'our_bi_cui_desc': '[CLS] Ex ##pan ##ding [unused3] ( Spa ##tial Concept , qualifier value : Ex ##pan ##d ; Ex ##pan ##ded ; expanding ; expand ; expanded ) [SEP]',
  'our_cross_cui': 'C0007595',
  'our_cross_cui_name': 'cell growth',
  'our_cross_cui_desc': '[CLS] cell growth [unused3] ( B ##iol ##og ##ic Fun ##ction : cell expansion ; cellular growth ; Cell ##s - - Growth ; Cell Growth ; cell growth ##s ; cells growth ; growth cell ; growth of cell ; Cell ##ular Growth ; Cell ##ular Ex ##pan ##sion ) [SEP]'},
 {'mention_id': '28418943.15',
  'test_set_idx': 35137,
  'type': 'T204',
  'name': 'rats',
  'context': '[CLS] of alcohol seeking was reduced after 4 months compared with 2 months of intermittent alcohol access . The influence of individual levels of alcohol intake on loss of control over alcohol seeking was subsequently determined by assessing conditioned suppression in subgroup ##s of low and high alcohol drinking rats . Unlike the low alcohol drinking rats , the high alcohol drinking [unused1] rats [unused2] showed a ##version - resistant alcohol seeking after 2 months of intermittent alcohol access , although both groups showed comparable levels of conditioned freezing . These findings show that the development of loss of control over alcohol seeking , a key characteristic of Al ##co ##hol use disorder in humans , is dependent on both the extent of alcohol exposure and the [SEP]',
  'existing_cui': 'C2097385',
  'existing_cui_name': 'allergy testing rat',
  'existing_cui_desc': '[CLS] all ##er ##gy testing rat [unused3] ( Health Care Act ##ivity , lab test : rat ) [SEP]',
  'our_bi_cui': 'C0034721',
  'our_bi_cui_name': 'Rattus',
  'our_bi_cui_desc': '[CLS] Rat ##tus [unused3] ( E ##uka ##ryo ##te , organism : Rat ; Genus rat ##tus ; Genus Rat ##tus ; rats ; rat ; rat ##tus ; Rat ##s ) [SEP]',
  'our_cross_cui': 'C0086893',
  'our_cross_cui_name': 'Rats , Laboratory',
  'our_cross_cui_desc': '[CLS] Rat ##s , Laboratory [unused3] ( E ##uka ##ryo ##te : laboratory rat ; Laboratory Rat ##s ; Rat , Laboratory ; Laboratory Rat ) [SEP]'},
 {'mention_id': '27838742.81',
  'test_set_idx': 19048,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . Del ##ica ##f ##lav ##one may represent a potential therapeutic agent for lung cancer . Del ##ica ##f ##lav ##one showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . [unused1] Del ##ica ##f ##lav ##one [unused2] induced auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . Del ##ica ##f ##lav ##one may represent a potential therapeutic agent for lung cancer . [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28013256.6',
  'test_set_idx': 23767,
  'type': 'T097',
  'name': 'health',
  'context': "[CLS] From training to practice : the impact of E ##NG ##AG ##E , Ireland ' s national men ' s health training programme Ireland ' s National Men ' s Health Policy recommended developing training programmes tailored to the needs of those working in [unused1] health [unused2] and allied health professionals and E ##NG ##AG ##E was developed to meet that recommendation . This study evaluated the impact of E ##NG ##AG ##E on front ##line service providers ' self - reported knowledge , skills , capacity and practice up to 5 - months post training . Between 2012 and 2015 , E ##NG ##AG ##E Train ##ers ( n = 57 ) delivered 62 1 - day training programmes to 81 ##0 participants . This [SEP]",
  'existing_cui': 'C4277525',
  'existing_cui_name': 'One Health Initiative',
  'existing_cui_desc': '[CLS] One Health Initiative [unused3] ( B ##io ##medical O ##cc ##upation or Disc ##ip ##line : Health , One ; One Medicine Initiative ; Health Initiative , One ; Medicine Initiative , One ; Initiative , One Medicine ; Initiative , One Health ; One Health ) [SEP]',
  'our_bi_cui': 'C0018722',
  'our_bi_cui_name': 'Health Occupations',
  'our_bi_cui_desc': '[CLS] Health O ##cc ##upation ##s [unused3] ( B ##io ##medical O ##cc ##upation or Disc ##ip ##line : health occupations ; Health O ##cc ##upation ; Health Prof ##ess ##ion ; O ##cc ##upation ##s , Health ; Prof ##ess ##ions , Health ; Health Prof ##ess ##ions ; health profession ; O ##cc ##upation , Health ; health science profession ; health care field ; health occupation ; Health profession ##s ; Prof ##ess ##ion , Health ; occupation health ; Health careers ; H ##EA ##LT ##H O ##CC ##UP ; health profession ##s ) [SEP]',
  'our_cross_cui': 'C1704312',
  'our_cross_cui_name': 'Health Professional',
  'our_cross_cui_desc': '[CLS] Health Professional [unused3] ( Professional or O ##cc ##upation ##al Group , occupation : Health professionals ; Health Care Professional ##s ; H ##EA ##LT ##H CA ##RE PR ##O ##F ##ES ##SI ##ON ##AL ; Health profession ; Healthcare professional ; Healthcare worker ; Health Care Professional ; Health professional ) [SEP]'},
 {'mention_id': '27838742.43',
  'test_set_idx': 19010,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] - I , which are auto ##pha ##gy - related proteins , and promoting the generation of acidic ve ##si ##cular organ ##elles and auto ##ly ##so ##some ##s in the c ##yt ##op ##las ##m of human lung cancer A ##5 ##4 ##9 and PC - 9 cells in a time - and dose - dependent manner . [unused1] Del ##ica ##f ##lav ##one [unused2] down ##re ##gu ##lated the expression of p ##hos ##ph ##o - A ##kt , p ##hos ##ph ##o - m ##TO ##R , and p ##hos ##ph ##o - p ##70 ##S ##6 ##K in a time - and dose - dependent manner , suggesting that it induced auto ##pha ##gy by in ##hibit ##ing the A ##kt / [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0529351',
  'our_bi_cui_name': 'darifenacin',
  'our_bi_cui_desc': '[CLS] da ##ri ##fen ##ac ##in [unused3] ( Chemical , medication , product , substance : Dar ##ife ##nac ##in ##um ; ( S ) - 1 - ( 2 - ( 2 , 3 - di ##hy ##dr ##o - 5 - Benz ##of ##ura ##ny ##l ) et ##hyl ) - alpha , alpha - dip ##hen ##yl - 3 - p ##yr ##rol ##id ##ine ##ace ##tam ##ide ; Dar ##ife ##nac ##in ; ( S ) - 1 - ( 2 - ( 2 , 3 - di ##hy ##dr ##o - 5 - ben ##zo ##fu ##ran ##yl ) et ##hyl ) - alpha , alpha - dip ##hen ##yl - 3 - p ##yr ##rol ##id ##ine ##ace ##tam ##ide ; [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27500205.19',
  'test_set_idx': 8451,
  'type': 'T103',
  'name': 'kindlins',
  'context': '[CLS] high sequence identity . Kind ##lin - 1 is expressed primarily in e ##pit ##hel ##ial cells , kind ##lin - 2 is widely distributed and is particularly abundant in ad ##here ##nt cells , and kind ##lin - 3 is expressed primarily in hem ##ato ##po ##iet ##ic cells . These distributions are not exclusive ; some cells express multiple [unused1] kind ##lins [unused2] , and transformed cells often exhibit a ##ber ##rant expression , both in the is ##of ##orms and the levels of kind ##lins . Great interest in the kind ##lins has emerged from the recognition that they play major roles in controlling in ##te ##g ##rin function . In v ##it ##ro studies , in v ##ivo studies of mice def [SEP]',
  'existing_cui': 'C1720776',
  'existing_cui_name': 'Kindling , Neurologic',
  'existing_cui_desc': '[CLS] Kind ##ling , N ##eur ##olo ##gic [unused3] ( B ##iol ##og ##ic Fun ##ction , N ##eur ##ology : kind ##ling ; Kind ##lings , N ##eur ##olo ##gic ; Kind ##ling ; N ##eur ##olo ##gic Kind ##lings ; N ##eur ##olo ##gic Kind ##ling ) [SEP]',
  'our_bi_cui': 'C0010422',
  'our_bi_cui_name': 'Crystallins',
  'our_bi_cui_desc': '[CLS] Crystal ##lins [unused3] ( Chemical , substance : Crystal ##lins [ Chemical / In ##g ##red ##ient ] ; Crystal ##lin ; Pro ##tein ##s , Len ##s ; C ##rist ##all ##in ; Len ##s Pro ##tein ##s ; crystal ##lin ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28144577.19',
  'test_set_idx': 26719,
  'type': 'T033',
  'name': 'signs',
  'context': '[CLS] combination with bone g ##raft ##s was used in 84 patients for the treatment of long bone non ##union ##s . All patients were evaluated radio ##graphical ##y for the development of he ##tero ##top ##ic o ##ssi ##fication during the standard assessment for the non ##union healing . In all patients ( 80 . 9 % ) with radio ##graphic [unused1] signs [unused2] of he ##tero ##top ##ic o ##ssi ##fication , a CT scan was performed . Non ##union site p ##al ##pation and ROM evaluation of the adjacent joints were also carried out . Factor ##s related to the patient ( age , gender ) , the non ##union ( location , size , chronic ##ity , number of previous procedures , infection [SEP]',
  'existing_cui': 'C0311392',
  'existing_cui_name': 'Physical findings',
  'existing_cui_desc': '[CLS] Physical findings [unused3] ( Finding , finding , physical finding : physical finding ; physical findings ; Physical finding ; physical findings [ use for free text ] ; Physical signs ; Sign ##s - physical ; physical signs ; Physical sign ; Sign ; find physical ) [SEP]',
  'our_bi_cui': 'C0311392',
  'our_bi_cui_name': 'Physical findings',
  'our_bi_cui_desc': '[CLS] Physical findings [unused3] ( Finding , finding , physical finding : physical finding ; physical findings ; Physical finding ; physical findings [ use for free text ] ; Physical signs ; Sign ##s - physical ; physical signs ; Physical sign ; Sign ; find physical ) [SEP]',
  'our_cross_cui': 'C0037088',
  'our_cross_cui_name': 'Signs and Symptoms',
  'our_cross_cui_desc': '[CLS] Sign ##s and S ##ym ##pt ##oms [unused3] ( Finding , finding : Sign ##s and S ##ym ##pt ##oms [ Disease / Finding ] ; Clinical findings ; clinical findings ; O ##bs ##er ##vable En ##ti ##ty ; sign and s ##ym ##pt ##om ; Sign and s ##ym ##pt ##om ; S ##ym ##pt ##oms and Sign ##s ; sign / s ##ym ##pt ##om ; Finding ; Clinical observation : symptoms and signs ; S ##IG ##NS S ##Y ##MP ##TO ##MS ; Clinical finding ) [SEP]'},
 {'mention_id': '28376393.14',
  'test_set_idx': 33592,
  'type': 'T033',
  'name': 'detection',
  'context': '[CLS] ass ##ay as an approximation for the tertiary and q ##uate ##rna ##ry am ##ine load in waste ##waters and surface water samples , and this approximation was compared to N - ni ##tro ##so ##di ##met ##hyl ##amine formation potential using ch ##lora ##mine ##s . An existing color ##ime ##tric method was examined and op ##ti ##mized for the [unused1] detection [unused2] of am ##ines in environmental water samples . The method consists of liquid - liquid extraction followed by a cat ##aly ##zed reaction to form a yet - und ##ef ##ined product that is known to be both a strong ch ##rom ##op ##hore and flu ##oro ##ph ##ore . Previous work verified that this reaction was effectively cat ##aly ##zed by [SEP]',
  'existing_cui': 'C0206100',
  'existing_cui_name': 'Signal Detection',
  'existing_cui_desc': '[CLS] Signal Det ##ec ##tion [unused3] ( B ##iol ##og ##ic Fun ##ction , Psychology , Psychology , Signal : Signal Det ##ec ##tion Theo ##ries ; Signal Det ##ec ##tions , Psychological ; Theory , Signal Det ##ec ##tion ; Signal detection ; S ##IG ##NA ##L DE ##TE ##CT ##ION AN ##AL ; Psychological Signal Det ##ec ##tions ; Det ##ec ##tion ; Signal Det ##ec ##tion Ana ##ly ##ses ; S ##IG ##NA ##L DE ##TE ##CT ##ION ; Det ##ec ##tions , Psychological Signal ; Psychological Signal Det ##ec ##tion ; Signal Det ##ec ##tion Theory ; Theo ##ries , Signal Det ##ec ##tion ; Signal Det ##ec ##tions ; Ana ##ly ##ses , Signal Det ##ec ##tion ; Det ##ec ##tion , [SEP]',
  'our_bi_cui': 'C1511790',
  'our_bi_cui_name': 'Detection',
  'our_bi_cui_desc': '[CLS] Det ##ec ##tion [unused3] ( Health Care Act ##ivity : Det ##ec ##ted ) [SEP]',
  'our_cross_cui': 'C0442726',
  'our_cross_cui_name': 'Detected',
  'our_cross_cui_desc': '[CLS] Det ##ec ##ted [unused3] ( Finding , finding , finding , qualifier value : detected ; detect ) [SEP]'},
 {'mention_id': '28377738.10',
  'test_set_idx': 33695,
  'type': 'T170',
  'name': 'gradational',
  'context': '[CLS] and the p ##oly ##se ##mous notion of consciousness continue to prove resistant to consistent measurement and un ##am ##bi ##guous definition . This is hardly surprising , given that there is no agreement even as regards the most fundamental issues they involve . One of the basic disagreements present in the continuing debate about consciousness per ##tains to its [unused1] g ##rada ##tional [unused2] nature . The general aim of this article is to show how consciousness might be graded and multi ##di ##men ##sional at the same time . We therefore focus on the question of what it is , exactly , that is or could be graded in cases of consciousness , and how we can measure it . Ultimately , four different [SEP]',
  'existing_cui': 'C0699900',
  'existing_cui_name': 'Catabolism',
  'existing_cui_desc': '[CLS] Cat ##ab ##olis ##m [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : Cat ##ab ##olis ##m , function ; cat ##ab ##olis ##m ; cat ##ab ##olic process ; breakdown ; degradation ) [SEP]',
  'our_bi_cui': 'C0445237',
  'our_bi_cui_name': 'Rotational',
  'our_bi_cui_desc': '[CLS] R ##ota ##tional [unused3] ( Spa ##tial Concept , qualifier value : rotation ##al ) [SEP]',
  'our_cross_cui': 'C0441800',
  'our_cross_cui_name': 'Grade',
  'our_cross_cui_desc': '[CLS] Grade [unused3] ( Intel ##lect ##ual Product , attribute , qualifier value : With grade ; Has grade ; grades ; Grades ; G ##rading ; grade ) [SEP]'},
 {'mention_id': '28301509.57',
  'test_set_idx': 31754,
  'type': 'T017',
  'name': '3H',
  'context': '[CLS] . A single Q ##TL for grain yield containing three M ##TA ##s was found on chromosome 3 ##H located close to the H ##v - e ##IF ##4 ##E gene , which is known to regulate resistance to bar ##ley yellow mosaic virus . Subsequently the Q ##TL was shown to be tightly linked to r ##ym ##4 , a lo ##cus for resistance to the virus . G ##WA ##s on a ##rab ##ina ##ns q ##uant ##ified by L ##M ##6 resulted in the identification of major Q ##TL ##s closely located on [unused1] 3 ##H [unused2] and h ##y ##pot ##heses regarding put ##ative candidate genes were formulated through the study of gene expression levels based on bio ##in ##formatics tools . [SEP]',
  'existing_cui': 'C1098079',
  'existing_cui_name': '( 3H ) azidoneonicotinoid',
  'existing_cui_desc': '[CLS] ( 3 ##H ) a ##zi ##don ##eon ##ico ##tino ##id [unused3] ( Chemical : ( 3 ##H ) A ##z ##N ##N ) [SEP]',
  'our_bi_cui': 'C0008666',
  'our_bi_cui_name': 'Chromosomes , Human , Pair 3',
  'our_bi_cui_desc': '[CLS] Ch ##rom ##oso ##mes , Human , Pa ##ir 3 [unused3] ( Ana ##tom ##ical Structure , body structure , cell structure : Ch ##rom ##oso ##me 3 ; chromosome 3 ; Ch ##rom ##oso ##me pair 3 ; 3 chromosome ##s ; CH ##RO ##MO ##SO ##ME 03 ; CH ##RO ##MO ##SO ##ME ##S H ##UM ##AN PA ##IR 03 ; 3 chromosome ) [SEP]',
  'our_cross_cui': 'C0008633',
  'our_cross_cui_name': 'Chromosomes',
  'our_cross_cui_desc': '[CLS] Ch ##rom ##oso ##mes [unused3] ( Ana ##tom ##ical Structure , body structure , cell structure : chromosome ; Ch ##rom ##oso ##mal ; chromosome ##s ; Ch ##rom ##oso ##me ) [SEP]'},
 {'mention_id': '27234220.24',
  'test_set_idx': 280,
  'type': 'T033',
  'name': 'no',
  'context': '[CLS] access complications ) . Pat ##ients in group A were older ( 74 ± 12 years vs . 68 ± 17 years , p < 0 . 01 ) and the main indication for anti ##coa ##gu ##lation was at ##rial fi ##bri ##lla ##tion ( 58 . 6 % ) . No complications occurred during the procedures . There were [unused1] no [unused2] acute bleeding ##s just after the bandage removal . During the first 24 ##h , only 3 ( 2 . 1 % ) radial o ##cc ##lusion ##s in group A and 2 ( 0 . 6 % ) in group B ( p = 0 . 14 ) were recorded . He ##mat ##oma ##s between 5 and 10 ##c ##m [SEP]',
  'existing_cui': 'C1140150',
  'existing_cui_name': 'NOC - CodeSystem',
  'existing_cui_desc': '[CLS] NO ##C - Code ##S ##ys ##tem [unused3] ( Intel ##lect ##ual Product , NO ##C : nursing outcomes classification ) [SEP]',
  'our_bi_cui': 'C0243095',
  'our_bi_cui_name': 'Finding',
  'our_bi_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]',
  'our_cross_cui': 'C1513916',
  'our_cross_cui_name': 'Negative Finding',
  'our_cross_cui_desc': '[CLS] N ##eg ##ative Finding [unused3] ( Finding : NE ##GA ##TI ##VE ; N ##eg ##ative ) [SEP]'},
 {'mention_id': '27676724.19',
  'test_set_idx': 13878,
  'type': 'T097',
  'name': 'sonologist',
  'context': '[CLS] or ##th ##oped ##ic surgeon , an occupational therapist for upper limb ma ##lf ##orm ##ation , a p ##hy ##sio ##ther ##ap ##ist for lower ma ##lf ##orm ##ation and a psychologist . This multi ##disciplinary consultation is unique because of each pregnancy story , because of each patient life story but also because of the words used by the [unused1] son ##ologist [unused2] when announcing the diagnosis . We deal with couples shocked by the pre ##nat ##al diagnosis . We help them get acceptance of the child to be born and to forget about their imaginary child . Medical information is often perceived as being " a surf ##eit of information " difficult to handle by the couple stunned by the overwhelming diagnosis [SEP]',
  'existing_cui': 'C0586859',
  'existing_cui_name': 'Pulmonologists',
  'existing_cui_desc': '[CLS] P ##ul ##mon ##ologists [unused3] ( Professional or O ##cc ##upation ##al Group , occupation : Re ##sp ##ira ##tory disease specialist ; P ##ul ##mon ##ologist ; pulmonary specialist ; Re ##sp ##ira ##tory physician ; pu ##lm ##ono ##logist ) [SEP]',
  'our_bi_cui': 'C1954848',
  'our_bi_cui_name': 'Sonographer',
  'our_bi_cui_desc': '[CLS] Son ##ographer [unused3] ( Professional or O ##cc ##upation ##al Group ) [SEP]',
  'our_cross_cui': 'C1522486',
  'our_cross_cui_name': 'Professional Organization or Group',
  'our_cross_cui_desc': '[CLS] Professional Organization or Group [unused3] ( Professional or O ##cc ##upation ##al Group : Other Agency or Organization ; Organization ; OR ##G ; Outside Bo ##dies ; Organization ##al ) [SEP]'},
 {'mention_id': '27500205.23',
  'test_set_idx': 8455,
  'type': 'T103',
  'name': 'kindlins',
  'context': '[CLS] widely distributed and is particularly abundant in ad ##here ##nt cells , and kind ##lin - 3 is expressed primarily in hem ##ato ##po ##iet ##ic cells . These distributions are not exclusive ; some cells express multiple kind ##lins , and transformed cells often exhibit a ##ber ##rant expression , both in the is ##of ##orms and the levels of [unused1] kind ##lins [unused2] . Great interest in the kind ##lins has emerged from the recognition that they play major roles in controlling in ##te ##g ##rin function . In v ##it ##ro studies , in v ##ivo studies of mice def ##icient in kind ##lins , and studies of patients with genetic def ##ici ##encies of kind ##lins have clearly established that they regulate [SEP]',
  'existing_cui': 'C1720776',
  'existing_cui_name': 'Kindling , Neurologic',
  'existing_cui_desc': '[CLS] Kind ##ling , N ##eur ##olo ##gic [unused3] ( B ##iol ##og ##ic Fun ##ction , N ##eur ##ology : kind ##ling ; Kind ##lings , N ##eur ##olo ##gic ; Kind ##ling ; N ##eur ##olo ##gic Kind ##lings ; N ##eur ##olo ##gic Kind ##ling ) [SEP]',
  'our_bi_cui': 'C0010422',
  'our_bi_cui_name': 'Crystallins',
  'our_bi_cui_desc': '[CLS] Crystal ##lins [unused3] ( Chemical , substance : Crystal ##lins [ Chemical / In ##g ##red ##ient ] ; Crystal ##lin ; Pro ##tein ##s , Len ##s ; C ##rist ##all ##in ; Len ##s Pro ##tein ##s ; crystal ##lin ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27659602.43',
  'test_set_idx': 13388,
  'type': 'T082',
  'name': 'root',
  'context': '[CLS] more are at risk for re ##oper ##ation , and not until at least after 10 . 4 years have passed . Neither the valve morphology ( p = 0 . 62 ) nor con ##com ##ita ##nt a ##ort ##ic valve replacement ( p = 0 . 86 ) influenced rate of root di ##lation . In non ##sy ##nd ##rom ##ic patients , the a ##ort ##ic root is the slow ##est growing portion of the th ##ora ##ci ##c a ##ort ##a . Leaving the native [unused1] root [unused2] , as in root - spa ##ring ascending a ##ort ##ic replacement , is a safe approach regarding secondary root intervention for a ##ort ##ic root diameter ##s of 45 mm or less . [SEP]',
  'existing_cui': 'C1318154',
  'existing_cui_name': 'Root body part',
  'existing_cui_desc': '[CLS] Root body part [unused3] ( Ana ##tom ##ical Structure : Root ; root [ a body part ] ; root ) [SEP]',
  'our_bi_cui': 'C0040452',
  'our_bi_cui_name': 'Tooth root structure',
  'our_bi_cui_desc': '[CLS] Too ##th root structure [unused3] ( Ana ##tom ##ical Structure , body structure : Ra ##di ##x co ##rona ; Root of the Too ##th ; tooth root ; Too ##th Root ; Root of Too ##th ; Root , Too ##th ; tooth roots ; root of tooth ; root tooth ; roots tooth ; Te ##eth - - Roots ; Roots , Too ##th ; of tooth root ; Too ##th Roots ; Too ##th root ; Ra ##di ##x Den ##tis ; Ra ##di ##x den ##tis ; rooted tooth ; Root of tooth ) [SEP]',
  'our_cross_cui': 'C0549113',
  'our_cross_cui_name': 'Supraaortic valve area structure',
  'our_cross_cui_desc': '[CLS] Su ##pra ##ao ##rt ##ic valve area structure [unused3] ( Spa ##tial Concept , body structure : A ##ort ##ic bulb ; AS ##CE ##ND ##ING A ##OR ##TA , A ##OR ##TI ##C R ##O ##OT ; A ##ort ##a . root ; A ##ort ##ic root ; a ##ort ##ic root ; A ##ort ##ic Root ; Root of a ##ort ##a ; B ##ul ##b of a ##ort ##a ; Su ##pra ##ao ##rt ##ic valve area ; root a ##ort ##ic ; B ##ul ##b of ascending a ##ort ##a ; B ##ul ##bus a ##ort ##ae ) [SEP]'},
 {'mention_id': '27474969.18',
  'test_set_idx': 7435,
  'type': 'T103',
  'name': 'CuS',
  'context': '[CLS] sensor based on reduced graph ##ene oxide - copper su ##lf ##ide ( r ##G ##O - C ##u ##S ) composite coupled with cap ##illa ##ry electro ##ph ##ores ##is was constructed for the ultra ##sen ##sitive detection of am ##lo ##di ##pine be ##sy ##late for the first time . In this work , r ##G ##O - [unused1] C ##u ##S [unused2] composite was synthesized by one - pot h ##ydro ##thermal method and used for electro ##de modification . The electro ##chemical and electro ##che ##mi ##lum ##ines ##cence behaviors of the sensor were investigated . More than 5 - fold enhance in electro ##che ##mi ##lum ##ines ##cence intensity was observed after modified with r ##G ##O - C ##u ##S [SEP]',
  'existing_cui': 'C0056599',
  'existing_cui_name': 'cupric sulfide',
  'existing_cui_desc': '[CLS] cup ##ric su ##lf ##ide [unused3] ( Chemical , C ##u ##S : c ##p ##d with un ##sp ##ec ##ified M ##F of cup ##ric su ##lf ##ide ; co ##vel ##lite ) [SEP]',
  'our_bi_cui': 'C0056301',
  'our_bi_cui_name': 'Copper Sulfate',
  'our_bi_cui_desc': '[CLS] Copper Sul ##fa ##te [unused3] ( Chemical , AS C ##UP ##RI ##C S ##U ##LF ##AT ##E , discontinued , medication , obsolete , product , substance : copper su ##l ##phate ; Copper Sul ##fa ##te [ Chemical / In ##g ##red ##ient ] ; Blue V ##it ##rio ##l ; cup ##ric su ##lf ##ate an ##hy ##dr ##ous ; CO ##PP ##ER S ##U ##LF ##AT ##E ; V ##it ##rio ##l , Blue ; BA ##SI ##C CO ##PP ##ER S ##U ##LF ##AT ##E ; Blue Vic ##king ; C ##UP ##RI ##C S ##U ##LF ##AT ##E , AN ##H ##Y ##DR ##O ##US ; Cup ##ric Sul ##fa ##te An ##hy ##dr ##ous ; Sul ##fa ##te , [SEP]',
  'our_cross_cui': 'C0389357',
  'our_cross_cui_name': 'cuprous sulfide',
  'our_cross_cui_desc': '[CLS] cup ##rous su ##lf ##ide [unused3] ( Chemical : copper ( I ) su ##lf ##ide ; C ##u ##2 ##S c ##p ##d ) [SEP]'},
 {'mention_id': '28167893.31',
  'test_set_idx': 27472,
  'type': 'T038',
  'name': 'Ipr-I',
  'context': '[CLS] on potato - de ##x ##tro ##se a ##gar with i ##p ##rod ##ione , no fun ##gal growth ( i ##p ##rod ##ione - no growth , I ##p ##r - N ) occurred for the first 3 day after in ##cu ##bation , but once the initial growth ( i ##p ##rod ##ione - initial growth , [unused1] I ##p ##r - I [unused2] ) began at 4 - 5 day after in ##cu ##bation , the colonies grew and expanded continuously to be in full growth ( i ##p ##rod ##ione - growth , I ##p ##r - G ) , suggesting I ##p ##r - I may be a turning moment of the m ##or ##ph ##ogen ##etic changes resisting fungi ##cid [SEP]',
  'existing_cui': 'C1418954',
  'existing_cui_name': 'PRPS1 gene',
  'existing_cui_desc': '[CLS] PR ##PS ##1 gene [unused3] ( Ana ##tom ##ical Structure : D ##F ##N ##X ##1 ; C ##MT ##X ##5 ; P ##H ##OS ##P ##H ##OR ##IB ##OS ##Y ##LP ##Y ##RO ##P ##H ##OS ##P ##HA ##TE S ##Y ##NT ##H ##ET ##AS ##E I ; PR ##S I ; rib ##ose - phosphate dip ##hos ##ph ##oki ##nas ##e 1 ; p ##hos ##ph ##ori ##bos ##yl p ##yr ##op ##hos ##phate s ##ynth ##eta ##se 1 ) [SEP]',
  'our_bi_cui': 'C3755286',
  'our_bi_cui_name': 'Idiophantis',
  'our_bi_cui_desc': '[CLS] I ##dio ##pha ##nti ##s [unused3] ( E ##uka ##ryo ##te ) [SEP]',
  'our_cross_cui': 'C0018270',
  'our_cross_cui_name': 'Growth',
  'our_cross_cui_desc': "[CLS] Growth [unused3] ( B ##iol ##og ##ic Fun ##ction : General ##ized Growth ; normal growth ; onto ##geny ; em ##b ##ryo ##nic growth ? ; Physical Growth ; growth ##s ; physical growth ; physical growth process ; growth ; ' growth ' ; G ##RO ##W ##TH ) [SEP]"},
 {'mention_id': '28471040.35',
  'test_set_idx': 37149,
  'type': 'T038',
  'name': 'rat glioma model',
  'context': '[CLS] 2 g ##lio ##ma cell lines in cell culture studies , which further increases with addition of 1 , 2 - di ##ole ##oy ##l - 3 - trim ##eth ##yla ##mm ##onium - prop ##ane . Ma ##gnetic resonance imaging and his ##top ##ath ##olo ##gic evaluation on rat g ##lio ##ma - 2 t ##umour cells in [unused1] rat g ##lio ##ma model [unused2] ( 49 female W ##ista ##r rats , 250 - 300 g ) comparing in ##tra ##ven ##ous and in ##tra ##tum ##oral injection ##s of the drug have been performed and F ##TA - loaded na ##no ##par ##tic ##les reduced t ##umour size significantly in in - v ##ivo studies , with higher efficiency of in ##tra ##tum [SEP]',
  'existing_cui': 'C0017638',
  'existing_cui_name': 'Glioma',
  'existing_cui_desc': '[CLS] G ##lio ##ma [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , m ##or ##ph ##olo ##gic abnormal ##ity : G ##lial Cell Tu ##mor ##s ; g ##lio ##mas ; [ M ] G ##lio ##mas ; G ##lio ##mas ; N ##eur ##og ##lial Neo ##p ##las ##m ; Tu ##mor of the N ##eur ##og ##lia ; g ##lio ##ma ; un ##sp ##ec ##ified site ; Tu ##mor , G ##lial Cell ; Tu ##mor ##s , G ##lial Cell ; G ##lial Neo ##p ##las ##m ; Neo ##p ##las ##m of N ##eur ##og ##lia ; N ##eur ##og ##lial Tu ##mor ; G ##lial Cell Tu ##mor ; G ##lial Tu ##mor ; G ##lio [SEP]',
  'our_bi_cui': 'C1882742',
  'our_bi_cui_name': 'Rat Malignant Glioma',
  'our_bi_cui_desc': '[CLS] Rat Mali ##gnant G ##lio ##ma [unused3] ( B ##iol ##og ##ic Fun ##ction , R ##EN ##I : G ##lio ##ma , ma ##li ##gnant ) [SEP]',
  'our_cross_cui': 'C0012644',
  'our_cross_cui_name': 'Animal Disease Models',
  'our_cross_cui_desc': '[CLS] Animal Disease Models [unused3] ( B ##iol ##og ##ic Fun ##ction : animal model ; Disease Models , Animal ; D ##IS M ##OD ##EL ##S AN ##IM ##AL ; Animal Model of Disease ; Disease Model , Animal ; Animal Disease Model ; Disease ##s - - Animal models ) [SEP]'},
 {'mention_id': '27838742.90',
  'test_set_idx': 19057,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . Del ##ica ##f ##lav ##one may represent a potential therapeutic agent for lung cancer . Del ##ica ##f ##lav ##one showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . Del ##ica ##f ##lav ##one induced auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . [unused1] Del ##ica ##f ##lav ##one [unused2] may represent a potential therapeutic agent for lung cancer . [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27509301.2',
  'test_set_idx': 8784,
  'type': 'T103',
  'name': 'Environmental Chemical',
  'context': "[CLS] Pre ##diction of E ##stro ##genic B ##io ##act ##ivity of [unused1] Environmental Chemical [unused2] Met ##ab ##oli ##tes The US Environmental Protection Agency ' s End ##oc ##rine Di ##s ##rup ##tor Screen ##ing Program is using in v ##it ##ro data generated from To ##x ##C ##ast / To ##x ##21 high - through ##put screening ass ##ays to assess the end ##oc ##rine activity of environmental chemicals . Considering that in v ##it ##ro ass ##ays may have limited metabolic capacity , inactive chemicals that are bio ##tra ##ns ##formed into meta ##bol ##ites with end ##oc ##rine bio ##act ##ivity may be missed for further screening and testing . Therefore , there is a value in developing novel approaches to account for [SEP]",
  'existing_cui': 'C0014406',
  'existing_cui_name': 'Environment',
  'existing_cui_desc': '[CLS] Environment [unused3] ( Spa ##tial Concept , environment , treatment : environments ; Environment ##s ; environment ; Environmental ; E ##N ##VI ##R ; en ##vir ; En ##vir ) [SEP]',
  'our_bi_cui': 'C0014417',
  'our_bi_cui_name': 'Environmental Pollutants',
  'our_bi_cui_desc': '[CLS] Environmental Poll ##uta ##nts [unused3] ( Chemical : E ##N ##VI ##R P ##OL ##L ##UT ##AN ##TS ; P ##OL ##L ##UT ##AN ##TS E ##N ##VI ##R ; Poll ##uta ##nts , Environmental ) [SEP]',
  'our_cross_cui': 'C0220806',
  'our_cross_cui_name': 'Chemicals',
  'our_cross_cui_desc': '[CLS] Chemical ##s [unused3] ( Chemical , medication , substance : chemical compounds ; Chemical ; chemical compound ; Chemical compound ; chemical substance ; Chemical ##ly ; Chemical product ; Chemical agent ; Chemical element AND / OR compound ; chemical ; chemicals ) [SEP]'},
 {'mention_id': '27832198.67',
  'test_set_idx': 18689,
  'type': 'T017',
  'name': 'Cam734',
  'context': '[CLS] ##er studies , and might differ in natural is ##olate ##s harbor ##ing this all ##ele . Results presented here ##in show that Cam ##7 ##34 - mediated ch ##lor ##o ##quin ##e resistance is dependent on the rare A1 ##44 ##F mutation that has not been observed beyond Southeast Asia , and reveal distinct impacts of this and other [unused1] Cam ##7 ##34 [unused2] - specific mutations on ch ##lor ##o ##quin ##e resistance and parasite growth rates . B ##io ##chemical ass ##ays revealed a broad impact of mutant P ##f ##CR ##T is ##of ##orms on parasite metabolism , including n ##uc ##leo ##side trip ##hos ##phate levels , hem ##og ##lo ##bin cat ##ab ##olis ##m and disposition of hem ##e , [SEP]',
  'existing_cui': 'C0006675',
  'existing_cui_name': 'Calcium',
  'existing_cui_desc': '[CLS] Cal ##ci ##um [unused3] ( Chemical , medication , substance : Co ##agu ##lation factor IV - R ##ET ##IR ##ED - ; CO ##AG FA ##CT ##OR IV ; Co ##agu ##lation Factor IV ; B ##L ##O ##OD CO ##AG FA ##CT ##OR IV ; C ##a + + element ; factor i ##v ; calcium ; Blood Co ##agu ##lation Factor IV ; Cal ##ci ##um co ##agu ##lation factor ; Co ##agu ##lation factor IV ; calcium ##s ; Cal ##ci ##um [ Chemical / In ##g ##red ##ient ] ; C ##a - Cal ##ci ##um ; CA ##LC ##I ##UM ; Factor IV ; C ##a element ; Cal ##ci ##um anal ##yte ; Factor IV , Co ##agu ##lation [SEP]',
  'our_bi_cui': 'C0678941',
  'our_bi_cui_name': 'Gene Mutant',
  'our_bi_cui_desc': '[CLS] Gene Mu ##tant [unused3] ( Ana ##tom ##ical Structure : gene mutant ##s ; Gene Mu ##tation ; gene mutant ; Gene V ##arian ##t ; mutant gene ; genes mutant ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27384489.27',
  'test_set_idx': 4341,
  'type': 'T082',
  'name': 'R1628P polymorphism',
  'context': "[CLS] used to evaluate the strength of the association between the R ##16 ##28 ##P p ##oly ##mor ##phism and Parkinson ' s disease . This meta - analysis assessed 19 studies from 14 papers that involved a total of 9 , 92 ##7 Parkinson ' s disease patients and 8 , 60 ##2 controls and found that the [unused1] R ##16 ##28 ##P p ##oly ##mor ##phism [unused2] was significantly associated with the risk of Parkinson ' s disease in Asian populations . Moreover , s ##tra ##ti ##fication analyses indicated that the R ##16 ##28 ##P p ##oly ##mor ##phism was significantly associated with an increased risk of Parkinson ' s disease among Chinese as well as non - Chinese Asian populations and an increased [SEP]",
  'existing_cui': 'C0032529',
  'existing_cui_name': 'Genetic Polymorphism',
  'existing_cui_desc': '[CLS] Gene ##tic Pol ##ym ##or ##phism [unused3] ( B ##iol ##og ##ic Fun ##ction , Gene ##tics , finding : P ##OL ##Y ##MO ##RP ##H ##IS ##M GE ##NE ##T ; P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ; Gene ##tic p ##oly ##mor ##phism ; genetic p ##oly ##mor ##phism ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##MS ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ##s , Gene ##tic ; Gene ##tic p ##oly ##mor ##phism ##s ; p ##oly ##mor ##phism ; Pol ##ym ##or ##phism , Gene ##tic ; p ##oly ##mor ##phism ##s ; Gene ##tic Pol ##ym ##or ##phism ##s [SEP]',
  'our_bi_cui': 'C0032529',
  'our_bi_cui_name': 'Genetic Polymorphism',
  'our_bi_cui_desc': '[CLS] Gene ##tic Pol ##ym ##or ##phism [unused3] ( B ##iol ##og ##ic Fun ##ction , Gene ##tics , finding : P ##OL ##Y ##MO ##RP ##H ##IS ##M GE ##NE ##T ; P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ; Gene ##tic p ##oly ##mor ##phism ; genetic p ##oly ##mor ##phism ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##MS ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ##s , Gene ##tic ; Gene ##tic p ##oly ##mor ##phism ##s ; p ##oly ##mor ##phism ; Pol ##ym ##or ##phism , Gene ##tic ; p ##oly ##mor ##phism ##s ; Gene ##tic Pol ##ym ##or ##phism ##s [SEP]',
  'our_cross_cui': 'C0752046',
  'our_cross_cui_name': 'Single Nucleotide Polymorphism',
  'our_cross_cui_desc': '[CLS] Single N ##uc ##leo ##tide Pol ##ym ##or ##phism [unused3] ( Spa ##tial Concept , S ##NP : Pol ##ym ##or ##phism , Single N ##uc ##leo ##tide ; Single N ##uc ##leo ##tide Pol ##ym ##or ##phism ##s ; Pol ##ym ##or ##phism ##s , Single N ##uc ##leo ##tide ; S ##NP ##s ; N ##uc ##leo ##tide Pol ##ym ##or ##phism ##s , Single ; N ##uc ##leo ##tide Pol ##ym ##or ##phism , Single ; single n ##uc ##leo ##tide p ##oly ##mor ##phism ; single n ##uc ##leo ##tide p ##oly ##mor ##phism ##s ; Pol ##ym ##or ##phism , Single Base ) [SEP]'},
 {'mention_id': '28353042.19',
  'test_set_idx': 32947,
  'type': 'T168',
  'name': 'high - fructose diet',
  'context': '[CLS] incidence of In ##sul ##in resistance . The present study investigate ##s the beneficial effects of a ##que ##ous extract of Ph ##yl ##lant ##hus am ##aru s ( PA ##A ##E ) on In ##sul ##in resistance and o ##xi ##da ##tive stress in high - f ##ru ##ct ##ose fed male W ##ista ##r rats . [unused1] high - f ##ru ##ct ##ose diet [unused2] ( 66 % of f ##ru ##ct ##ose ) and PA ##A ##E ( 200 mg / kg body weight / day ) were given concurrently to the rats for a period of 60 days . Fr ##uc ##tos ##e - fed rats showed weight gain , h ##yper ##gly ##ce ##mia , h ##yper ##ins ##ulin ##emia , [SEP]',
  'existing_cui': 'C0452314',
  'existing_cui_name': 'Low fructose diet',
  'existing_cui_desc': '[CLS] Low f ##ru ##ct ##ose diet [unused3] ( Health Care Act ##ivity , finding ) [SEP]',
  'our_bi_cui': 'C0301568',
  'our_bi_cui_name': 'High residue diet',
  'our_bi_cui_desc': '[CLS] High residue diet [unused3] ( Health Care Act ##ivity , finding , regime / therapy , treatment : High - fiber diet ; diet fiber high ; high fibre diet ; N ##SP - High non - star ##ch p ##oly ##sa ##cc ##hari ##de diet ; High rough ##age diet ; diet high fiber ; H ##F - High fibre diet ; H ##F ##D - High fiber diet ; H ##F - High fiber diet ; High fiber diet ; high residue diet ; Inc ##reased fiber diet ; diet ##s fiber high ; High non - star ##ch p ##oly ##sa ##cc ##hari ##de diet ; High fibre diet ; H ##F ##D - High fibre diet ; high fiber diet ; diet [SEP]',
  'our_cross_cui': 'C0012155',
  'our_cross_cui_name': 'Diet',
  'our_cross_cui_desc': '[CLS] Diet [unused3] ( Food , finding : Diet ##s ; regular diet ? ; Diet ##ary ; diet ; Diet ##ary finding ; diet ##s ) [SEP]'},
 {'mention_id': '27601594.44',
  'test_set_idx': 11275,
  'type': 'T103',
  'name': 'IDO-1',
  'context': '[CLS] + ) plasma cells , CD ##6 ##8 ( + ) mac ##rop ##hage ##s , and M ##HC class I on tumor cells . I ##mm ##uno ##su ##pp ##ress ##ive cell types were also unchanged , including Fox ##P ##3 ( + ) PD - 1 ( + ) cells ( put ##ative regulatory T cells ) , [unused1] ID ##O - 1 [unused2] ( + ) cells , and PD - L ##1 ( + ) cells ( both mac ##rop ##hage ##s and tumor cells ) . Hi ##era ##rch ##ical cluster ##ing revealed three response patterns : ( i ) T ##IL ( high ) tumors showed increases in multiple immune markers after ch ##em ##otherapy ; ( ii ) T [SEP]',
  'existing_cui': 'C1427370',
  'existing_cui_name': 'MYLIP gene',
  'existing_cui_desc': '[CLS] M ##Y ##L ##IP gene [unused3] ( Ana ##tom ##ical Structure : IN ##D ##UC ##IB ##LE DE ##GR ##AD ##ER OF THE L ##OW DE ##NS ##IT ##Y L ##IP ##OP ##RO ##TE ##IN R ##EC ##EP ##TO ##R ; E ##3 u ##bi ##qui ##tin l ##iga ##se - in ##du ##cible de ##grade ##r of the low density lip ##op ##rote ##in receptor ; MI ##R ; M ##Y ##OS ##IN R ##EG ##U ##LA ##TO ##R ##Y L ##IG ##HT CH ##A ##IN - IN ##TE ##RA ##CT ##ING PR ##OT ##EI ##N ; my ##os ##in regulatory light chain interacting protein ) [SEP]',
  'our_bi_cui': 'C1416433',
  'our_bi_cui_name': 'IDO1 gene',
  'our_bi_cui_desc': '[CLS] ID ##O ##1 gene [unused3] ( Ana ##tom ##ical Structure : IN ##D ##OL ##EA ##MI ##NE 2 , 3 - D ##IO ##X ##Y ##GE ##NA ##SE ; IN ##D ##O Gene ; Indo ##leam ##ine - P ##yr ##rol ##e 2 , 3 Di ##ox ##y ##gen ##ase Gene ; ID ##O ; in ##do ##leam ##ine 2 , 3 - di ##ox ##y ##gen ##ase 1 ; IN ##D ##O ) [SEP]',
  'our_cross_cui': 'C3529891',
  'our_cross_cui_name': 'indoleamine 2,3-dioxygenase 1 , human',
  'our_cross_cui_desc': '[CLS] in ##do ##leam ##ine 2 , 3 - di ##ox ##y ##gen ##ase 1 , human [unused3] ( Chemical : ID ##O ##1 protein , human ) [SEP]'},
 {'mention_id': '27898905.54',
  'test_set_idx': 21266,
  'type': 'T033',
  'name': 'no',
  'context': '[CLS] service . On p ##hen ##ot ##y ##ping all training animals for both commencement of l ##ute ##al activity and ca ##lving to first service , accuracy for ca ##lving to first service increased to 0 . 18 ; however , when valid ##ation animals were also p ##hen ##otype ##d for commencement of l ##ute ##al activity , there was [unused1] no [unused2] substantial increase in accuracy . When predict ##ing ca ##lving to first service in bi ##var ##iate analysis with proportion of samples with l ##ute ##al activity , accuracy ranged from 0 . 07 to 0 . 14 . This first study on g ##eno ##mic predictions for fertility using end ##oc ##rine traits suggests some improvement in the accuracy of prediction [SEP]',
  'existing_cui': 'C1140150',
  'existing_cui_name': 'NOC - CodeSystem',
  'existing_cui_desc': '[CLS] NO ##C - Code ##S ##ys ##tem [unused3] ( Intel ##lect ##ual Product , NO ##C : nursing outcomes classification ) [SEP]',
  'our_bi_cui': 'C0205160',
  'our_bi_cui_name': 'Negative',
  'our_bi_cui_desc': '[CLS] N ##eg ##ative [unused3] ( Finding , qualifier value : Rule ##d out ; N ##eg ##ative ( m ##od ##ifier ) [ Am ##bi ##guous ] ; N ##eg ##ative for ; N ##eg ##ative ( m ##od ##ifier ) ; ruled out ; for negative ; out ruled ; negative ; negative ##s ) [SEP]',
  'our_cross_cui': 'C1513916',
  'our_cross_cui_name': 'Negative Finding',
  'our_cross_cui_desc': '[CLS] N ##eg ##ative Finding [unused3] ( Finding : NE ##GA ##TI ##VE ; N ##eg ##ative ) [SEP]'},
 {'mention_id': '27384489.31',
  'test_set_idx': 4345,
  'type': 'T082',
  'name': 'R1628P polymorphism',
  'context': "[CLS] papers that involved a total of 9 , 92 ##7 Parkinson ' s disease patients and 8 , 60 ##2 controls and found that the R ##16 ##28 ##P p ##oly ##mor ##phism was significantly associated with the risk of Parkinson ' s disease in Asian populations . Moreover , s ##tra ##ti ##fication analyses indicated that the [unused1] R ##16 ##28 ##P p ##oly ##mor ##phism [unused2] was significantly associated with an increased risk of Parkinson ' s disease among Chinese as well as non - Chinese Asian populations and an increased risk of Parkinson ' s disease in Chinese patients from China , Taiwan , and Singapore . In a s ##tra ##ti ##fied analysis conducted according to age , significant associations were found [SEP]",
  'existing_cui': 'C0032529',
  'existing_cui_name': 'Genetic Polymorphism',
  'existing_cui_desc': '[CLS] Gene ##tic Pol ##ym ##or ##phism [unused3] ( B ##iol ##og ##ic Fun ##ction , Gene ##tics , finding : P ##OL ##Y ##MO ##RP ##H ##IS ##M GE ##NE ##T ; P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ; Gene ##tic p ##oly ##mor ##phism ; genetic p ##oly ##mor ##phism ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##MS ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ##s , Gene ##tic ; Gene ##tic p ##oly ##mor ##phism ##s ; p ##oly ##mor ##phism ; Pol ##ym ##or ##phism , Gene ##tic ; p ##oly ##mor ##phism ##s ; Gene ##tic Pol ##ym ##or ##phism ##s [SEP]',
  'our_bi_cui': 'C0032529',
  'our_bi_cui_name': 'Genetic Polymorphism',
  'our_bi_cui_desc': '[CLS] Gene ##tic Pol ##ym ##or ##phism [unused3] ( B ##iol ##og ##ic Fun ##ction , Gene ##tics , finding : P ##OL ##Y ##MO ##RP ##H ##IS ##M GE ##NE ##T ; P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ; Gene ##tic p ##oly ##mor ##phism ; genetic p ##oly ##mor ##phism ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##MS ; GE ##NE ##T P ##OL ##Y ##MO ##RP ##H ##IS ##M ; Pol ##ym ##or ##phism ##s , Gene ##tic ; Gene ##tic p ##oly ##mor ##phism ##s ; p ##oly ##mor ##phism ; Pol ##ym ##or ##phism , Gene ##tic ; p ##oly ##mor ##phism ##s ; Gene ##tic Pol ##ym ##or ##phism ##s [SEP]',
  'our_cross_cui': 'C0752046',
  'our_cross_cui_name': 'Single Nucleotide Polymorphism',
  'our_cross_cui_desc': '[CLS] Single N ##uc ##leo ##tide Pol ##ym ##or ##phism [unused3] ( Spa ##tial Concept , S ##NP : Pol ##ym ##or ##phism , Single N ##uc ##leo ##tide ; Single N ##uc ##leo ##tide Pol ##ym ##or ##phism ##s ; Pol ##ym ##or ##phism ##s , Single N ##uc ##leo ##tide ; S ##NP ##s ; N ##uc ##leo ##tide Pol ##ym ##or ##phism ##s , Single ; N ##uc ##leo ##tide Pol ##ym ##or ##phism , Single ; single n ##uc ##leo ##tide p ##oly ##mor ##phism ; single n ##uc ##leo ##tide p ##oly ##mor ##phism ##s ; Pol ##ym ##or ##phism , Single Base ) [SEP]'},
 {'mention_id': '27259647.25',
  'test_set_idx': 956,
  'type': 'T103',
  'name': '9-cis-11-trans-CLA',
  'context': '[CLS] h ##ydro ##xy ##l derivative 10 - h ##ydro ##xi - 12 - c ##is - o ##ct ##ade ##ce ##no ##ic acid to finally form bio ##active con ##ju ##gated l ##ino ##le ##ic acid . B ##io ##chemical optimization studies were carried out to el ##uc ##ida ##te the conditions for maximum production of [unused1] 9 - c ##is - 11 - trans - C ##LA [unused2] and maximum stability of α - en ##ola ##se when cat ##aly ##zing this reaction . Furthermore , through structural analysis of the protein , we propose the binding sites of substrate and product molecules that were characterized as two h ##ydro ##phobic superficial pockets located at opposite ends of the en ##ola ##se connected through a [SEP]',
  'existing_cui': 'C0281666',
  'existing_cui_name': 'alitretinoin',
  'existing_cui_desc': '[CLS] al ##it ##ret ##ino ##in [unused3] ( Chemical , product , substance : 9 - CR ##A ; 9 - c ##is - T ##ret ##ino ##in ; Ali ##tre ##tino ##ín ##a ; 9 - c ##is re ##tino ##ic acid ; 9 - c ##is - Re ##tino ##ic acid ; 9 - c ##is - Re ##tino ##ic A ##cid ; Ali ##tre ##tino ##in ; Ali ##tre ##tino ##in ##um ; 3 , 7 - Di ##met ##hyl - 9 - ( 2 , 6 , 6 - trim ##eth ##yl - 1 - c ##y ##c ##lo ##he ##xen - 1 - y ##l ) 2 - trans - 4 - trans - 6 - c ##is - 8 - trans [SEP]',
  'our_bi_cui': 'C0902483',
  'our_bi_cui_name': 'cis-9-trans-11-octadecadienoic acid',
  'our_bi_cui_desc': '[CLS] c ##is - 9 - trans - 11 - o ##ct ##ade ##ca ##die ##no ##ic acid [unused3] ( Chemical : 9 - c ##is - 11 - trans - o ##ct ##ade ##ca ##die ##no ##ic acid ; c ##9 ##t ##11 C ##LA ) [SEP]',
  'our_cross_cui': 'C1609806',
  'our_cross_cui_name': 'cis-9 , trans-11-conjugated linoleic acid',
  'our_cross_cui_desc': '[CLS] c ##is - 9 , trans - 11 - con ##ju ##gated l ##ino ##le ##ic acid [unused3] ( Chemical : c ##9 - t ##11 - C ##LA ) [SEP]'},
 {'mention_id': '28353042.2',
  'test_set_idx': 32930,
  'type': 'T168',
  'name': 'High Fructose Diet',
  'context': '[CLS] Ben ##ef ##ici ##al Effects of Ph ##yl ##lant ##hus am ##aru ##s Against [unused1] High Fr ##uc ##tos ##e Diet [unused2] In ##duced In ##sul ##in Resistance and He ##pa ##tic O ##xi ##da ##tive St ##ress in Male W ##ista ##r Rat ##s In ##sul ##in resistance is a characteristic feature of o ##besity , type 2 diabetes me ##lli ##tus , and card ##iovascular diseases . Em ##erging evidence suggests that the high - f ##ru ##ct ##ose consumption is a potential and important factor responsible for the rising incidence of In ##sul ##in resistance . The present study investigate ##s the beneficial effects of a ##que ##ous extract of Ph ##yl ##lant ##hus am ##aru s ( PA ##A ##E ) on [SEP]',
  'existing_cui': 'C0452314',
  'existing_cui_name': 'Low fructose diet',
  'existing_cui_desc': '[CLS] Low f ##ru ##ct ##ose diet [unused3] ( Health Care Act ##ivity , finding ) [SEP]',
  'our_bi_cui': 'C0301568',
  'our_bi_cui_name': 'High residue diet',
  'our_bi_cui_desc': '[CLS] High residue diet [unused3] ( Health Care Act ##ivity , finding , regime / therapy , treatment : High - fiber diet ; diet fiber high ; high fibre diet ; N ##SP - High non - star ##ch p ##oly ##sa ##cc ##hari ##de diet ; High rough ##age diet ; diet high fiber ; H ##F - High fibre diet ; H ##F ##D - High fiber diet ; H ##F - High fiber diet ; High fiber diet ; high residue diet ; Inc ##reased fiber diet ; diet ##s fiber high ; High non - star ##ch p ##oly ##sa ##cc ##hari ##de diet ; High fibre diet ; H ##F ##D - High fibre diet ; high fiber diet ; diet [SEP]',
  'our_cross_cui': 'C0012155',
  'our_cross_cui_name': 'Diet',
  'our_cross_cui_desc': '[CLS] Diet [unused3] ( Food , finding : Diet ##s ; regular diet ? ; Diet ##ary ; diet ; Diet ##ary finding ; diet ##s ) [SEP]'},
 {'mention_id': '28493435.7',
  'test_set_idx': 37722,
  'type': 'T033',
  'name': 'locate',
  'context': '[CLS] with ultra ##sound el ##ast ##ography has become a hot field in recent years . However , for key ##points tracking - based el ##ast ##ography algorithms , lo ##cating ex ##tre ##ma in multi ##mo ##dal ultra ##sonic radio ##f ##re ##que ##ncy signals is still a challenging problem . In this paper , a new method is proposed to [unused1] locate [unused2] the local ma ##xi ##ma and mini ##ma of the RF signals directly without der ##ivation operation . This algorithm can accurately locate ex ##tre ##ma even if disturbed peaks resulting from different noise exist . Furthermore , the new algorithm can speed up approximately 79 % of the implementation process as compared with the standard cross - correlation method on the [SEP]',
  'existing_cui': 'C1366498',
  'existing_cui_name': 'Chloramphenicol Acetyl Transferase Gene',
  'existing_cui_desc': '[CLS] Ch ##lora ##mp ##hen ##ico ##l Ace ##ty ##l Transfer ##ase Gene [unused3] ( Ana ##tom ##ical Structure : cat Gene ) [SEP]',
  'our_bi_cui': 'C0450429',
  'our_bi_cui_name': 'Location',
  'our_bi_cui_desc': '[CLS] Location [unused3] ( Spa ##tial Concept , attribute : location ; Lo ##c ; L ##OC ; locations ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27444636.41',
  'test_set_idx': 5992,
  'type': 'T170',
  'name': 'Quality of Life in Epilepsy Inventory 10',
  'context': '[CLS] analysis included 21 patients ( 12 women , 9 men ; mean age 36 years ) . After switching from o ##x ##car ##ba ##ze ##pine to es ##lica ##rb ##az ##ep ##ine ace ##tate , there were significant improvements in mean scores for Ad ##verse Events Profile ( P < . 00 ##1 ) , [unused1] Quality of Life in E ##pile ##psy In ##vent ##ory 10 [unused2] ( P = . 00 ##1 ) , and alert ##ness ( P < . 05 ) . Ad ##verse Events Profile total scores improved for 21 / 21 ( 100 . 0 % ) patients , Quality of Life in E ##pile ##psy In ##vent ##ory 10 total scores improved for 17 / 21 ( 81 [SEP]',
  'existing_cui': 'C2698752',
  'existing_cui_name': 'Pediatric Quality of Life Inventory',
  'existing_cui_desc': '[CLS] P ##ediatric Quality of Life In ##vent ##ory [unused3] ( Intel ##lect ##ual Product : P ##eds ##QL ) [SEP]',
  'our_bi_cui': 'C3897419',
  'our_bi_cui_name': 'Symptom Interference with Life Score 10',
  'our_bi_cui_desc': '[CLS] S ##ym ##pt ##om Inter ##ference with Life Score 10 [unused3] ( Intel ##lect ##ual Product : MD ##AS ##I S ##ym ##pt ##om Inter ##ference with Life Score 10 ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28188533.11',
  'test_set_idx': 27856,
  'type': 'T091',
  'name': 'proteomic studies',
  'context': '[CLS] ##c ##quisition Data for Label - free Q ##uant ##itative Pro ##te ##omi ##cs Studies Using Triple ##TO ##F ( ® ) Mass S ##pect ##rome ##ters Data - independent acquisition is a powerful mass s ##pect ##rome ##try technique that enables comprehensive MS and MS / MS analysis of all detect ##able species , providing an information rich data file that can be mined deeply . Here , we describe how to acquire high - quality S ##WA ##TH ( ® ) A ##c ##quisition data to be used for large quantitative [unused1] pro ##te ##omi ##c studies [unused2] . We specifically focus on using variable sized Q ##1 windows for acquisition of MS / MS data for generating higher specific ##ity quantitative data . [SEP]',
  'existing_cui': 'C1519031',
  'existing_cui_name': 'Pharmacoeconomic Study',
  'existing_cui_desc': '[CLS] Ph ##arma ##coe ##con ##omi ##c Study [unused3] ( Research Act ##ivity : P ##HA ##R ##MA ##CO ##EC ##ON ##OM ##IC ) [SEP]',
  'our_bi_cui': 'C1327760',
  'our_bi_cui_name': 'proteomic profiling',
  'our_bi_cui_desc': '[CLS] pro ##te ##omi ##c pro ##fi ##ling [unused3] ( Health Care Act ##ivity : Pro ##te ##omi ##c Prof ##iling ; Pro ##tein Express ##ion Prof ##iling ) [SEP]',
  'our_cross_cui': 'C0872252',
  'our_cross_cui_name': 'Proteomics',
  'our_cross_cui_desc': '[CLS] Pro ##te ##omi ##cs [unused3] ( B ##io ##medical O ##cc ##upation or Disc ##ip ##line : pro ##te ##omi ##cs ; pro ##te ##omi ##c ) [SEP]'},
 {'mention_id': '28179911.58',
  'test_set_idx': 27652,
  'type': 'T017',
  'name': 'FATA',
  'context': '[CLS] ##WR ##I ##1 in seven independent T ##2 lines were detected by quantitative real - time PC ##R . The relative m ##RNA accumulation of genes encoding enzymes involved in either fatty acid bio ##sy ##nt ##hesis or t ##ria ##cy ##l ##gly ##cer ##ols assembly ( BC ##CP ##2 , K ##AS ##I , MA ##T , E ##NR , [unused1] FA ##TA [unused2] , and GP ##D ##H ) were also ass ##ay ##ed in mature seeds . Furthermore , lip ##id and fatty acids C ##16 : 0 and C ##18 : 0 significantly increased . In two ho ##mo ##zy ##go ##us T ##2 trans ##genic rice lines ( G ##5 and G ##2 ) , different Co ##WR ##I ##1 expression [SEP]',
  'existing_cui': 'C1366645',
  'existing_cui_name': 'CD36 gene',
  'existing_cui_desc': '[CLS] CD ##36 gene [unused3] ( Ana ##tom ##ical Structure : SC ##AR ##B ##3 ; T ##H ##RO ##MB ##OS ##PO ##ND ##IN R ##EC ##EP ##TO ##R ; L ##E ##U ##KO ##C ##Y ##TE D ##IF ##F ##ER ##EN ##TI ##AT ##ION AN ##TI ##GE ##N CD ##36 ; GP ##I ##V ; CD ##36 Gene ; CD ##36 Mo ##le ##cule Gene ; G ##L ##Y ##CO ##PR ##OT ##EI ##N III ##b ; CD ##36 molecule ; GP III ##b ; CO ##LL ##AG ##EN R ##EC ##EP ##TO ##R , P ##LA ##TE ##LE ##T ; P ##LA ##TE ##LE ##T G ##L ##Y ##CO ##PR ##OT ##EI ##N IV ; FA ##TT ##Y AC ##ID T ##RA ##NS ##L ##OC [SEP]',
  'our_bi_cui': 'C0812278',
  'our_bi_cui_name': 'FAT1 gene',
  'our_bi_cui_desc': '[CLS] FA ##T ##1 gene [unused3] ( Ana ##tom ##ical Structure : FA ##T at ##y ##pical ca ##dh ##eri ##n 1 ; CD ##HF ##7 ; FA ##T At ##y ##pical C ##ad ##her ##in 1 Gene ; ca ##dh ##eri ##n - related family member 8 ; FA ##T ; FA ##T T ##UM ##OR S ##UP ##PR ##ES ##SO ##R , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF , 1 ; fat gene ; FA ##T ##1 Gene ; CD ##H ##R ##8 ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '25847295.22',
  'test_set_idx': 21,
  'type': 'T017',
  'name': 'apoptotic cells',
  'context': '[CLS] PC ##12 cells with non ##yl ##phe ##no ##l diet ##ho ##xy ##late , whereas the c ##yt ##och ##rome c release into the c ##yt ##oso ##l decreased as compared to that in a ##pop ##to ##tic cells not treated with non ##yl ##phe ##no ##l diet ##ho ##xy ##late s . Furthermore , Ba ##x contents in [unused1] a ##pop ##to ##tic cells [unused2] were reduced after exposure to non ##yl ##phe ##no ##l diet ##ho ##xy ##late . Thus , non ##yl ##phe ##no ##l diet ##ho ##xy ##late has the opposite effect on a ##pop ##tosis in PC ##12 cells compared to non ##yl ##phe ##no ##l , which enhance ##s a ##pop ##tosis induced by serum de ##p ##ri ##vation . [SEP]',
  'existing_cui': 'C0162638',
  'existing_cui_name': 'Apoptosis',
  'existing_cui_desc': '[CLS] A ##pop ##tosis [unused3] ( B ##iol ##og ##ic Fun ##ction , m ##or ##ph ##olo ##gic abnormal ##ity : a ##pop ##to ##tic cell death ; A ##pop ##to ##tic Process ; Programme ##d Cell Death ; programmed cell death by a ##pop ##tosis ; A ##pop ##tosis Path ##way ; a ##pop ##tos ##e ; programmed cell death ; RC ##D ; a ##pop ##tosis ; Gene - directed cell death ; a ##pop ##to ##tic programmed cell death ; Programme ##d Cell Death , Type I ; self destruction of the cell ; Me ##chan ##ism that allows cells to self - des ##truct when it is necessary ; a ##pop ##to ##tic process ; type I programmed cell death ) [SEP]',
  'our_bi_cui': 'C1836846',
  'our_bi_cui_name': 'Apoptotic neurons',
  'our_bi_cui_desc': '[CLS] A ##pop ##to ##tic neurons [unused3] ( Finding ) [SEP]',
  'our_cross_cui': 'C0007634',
  'our_cross_cui_name': 'Cells',
  'our_cross_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]'},
 {'mention_id': '27965426.19',
  'test_set_idx': 22656,
  'type': 'T103',
  'name': 'spindle - associated proteins',
  'context': '[CLS] not well characterized . Although a few functional ##ly - relevant mit ##otic p ##hos ##ph ##ot ##yr ##os ##ine sites have been characterized , evidence suggests that this modification may be more prevalent than previously appreciated . Here , we examined t ##yr ##os ##ine p ##hos ##ph ##ory ##lation in mit ##otic human cells including those on [unused1] spin ##dle - associated proteins [unused2] . ? Database mining confirmed ~ 2000 mit ##otic p ##hos ##ph ##ot ##yr ##os ##ine sites , and network analysis revealed a number of sub ##net ##works that were en ##rich ##ed in t ##yr ##os ##ine - p ##hos ##ph ##ory ##lated proteins , including components of the kin ##eto ##chor ##e or spin ##dle and SR ##C [SEP]',
  'existing_cui': 'C1425583',
  'existing_cui_name': 'NUSAP1 gene',
  'existing_cui_desc': '[CLS] N ##US ##AP ##1 gene [unused3] ( Ana ##tom ##ical Structure : N ##US ##AP ##1 Gene ; L ##NP ; PR ##O ##0 ##31 ##0 ##p ##1 ; N ##US ##AP ; N ##UC ##LE ##OL ##AR AND SP ##IN ##D ##LE - AS ##SO ##CI ##AT ##ED PR ##OT ##EI ##N 1 ; SA ##PL ; Q ##0 ##31 ##0 ; N ##UC ##LE ##OL ##AR PR ##OT ##EI ##N AN ##K ##T ; N ##u ##SA ##P ##1 ; FL ##J ##13 ##42 ##1 ; N ##uc ##leo ##lar and Spin ##dle Associated Pro ##tein 1 Gene ; n ##uc ##leo ##lar and spin ##dle associated protein 1 ; B ##M ##0 ##37 ) [SEP]',
  'our_bi_cui': 'C0026045',
  'our_bi_cui_name': 'Microtubule-Associated Proteins',
  'our_bi_cui_desc': '[CLS] Micro ##tub ##ule - Associated Pro ##tein ##s [unused3] ( Chemical , MA ##P : Micro ##tub ##ule Associated Pro ##tein ##s ; Micro ##tub ##ule - associated protein ; MI ##CR ##OT ##U ##B ##U ##LE AS ##SO ##C PR ##OT ##EI ##NS ; micro ##tub ##ule - associated proteins ; Micro ##tub ##ule - Associated Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; Micro ##tub ##ule - Associated Pro ##tein ; Micro ##tub ##ule Associated Pro ##tein ; MA ##Ps ; micro ##tub ##ule associated protein ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28454015.24',
  'test_set_idx': 36525,
  'type': 'T170',
  'name': 'health - risk based air quality index',
  'context': '[CLS] to evaluate the health risks . Results show that based on health - risk based air quality index , the current air quality index system likely significantly under ##est ##imate the health risks of air pollution , highlighting that the general public may need strict ##er health protection measures . The population - weighted two - year average [unused1] health - risk based air quality index [unused2] data further demonstrates that all population in the studied cities in He ##nan province live with poll ##uted air - 72 % of the population is exposed to air that is un ##hea ##lt ##hy for sensitive people , while 28 % of people is exposed to air that can be harmful to healthy people ; and the health [SEP]',
  'existing_cui': 'C1268941',
  'existing_cui_name': 'Health risks education',
  'existing_cui_desc': '[CLS] Health risks education [unused3] ( Health Care Act ##ivity , procedure , regime / therapy : Counsel ##ing regarding health risks ; Ad ##vice about health risks ; Counsel ##ling regarding health risks ; Education regarding health risks ) [SEP]',
  'our_bi_cui': 'C0018761',
  'our_bi_cui_name': 'Health Status Indicators',
  'our_bi_cui_desc': '[CLS] Health Status In ##dic ##ators [unused3] ( Intel ##lect ##ual Product , assessment scale : Index ##es , Health Status ; In ##dic ##ators , Health Status ; health status index ; In ##dic ##ator , Health Status ; Health Status Index ; In ##dices , Health Status ; Health status indicators ; Health Status In ##dices ; Health status index ; Health Status In ##dic ##ator ; Index , Health Status ; Health Status Index ##es ; health status indicators ; health in ##dices status ) [SEP]',
  'our_cross_cui': 'C0918012',
  'our_cross_cui_name': 'Index',
  'our_cross_cui_desc': '[CLS] Index [unused3] ( Intel ##lect ##ual Product : index ##ed ; index ; index ##es ; Index ##es as Top ##ic ) [SEP]'},
 {'mention_id': '27500205.6',
  'test_set_idx': 8438,
  'type': 'T103',
  'name': 'kindlins',
  'context': '[CLS] Of Kind ##lins and Cancer Kind ##lins are 4 . 1 - e ##z ##rin - rid ##ix ##in - m ##oes ##in ( F ##ER ##M ) domain containing proteins . There are three [unused1] kind ##lins [unused2] in mammals , which share high sequence identity . Kind ##lin - 1 is expressed primarily in e ##pit ##hel ##ial cells , kind ##lin - 2 is widely distributed and is particularly abundant in ad ##here ##nt cells , and kind ##lin - 3 is expressed primarily in hem ##ato ##po ##iet ##ic cells . These distributions are not exclusive ; some cells express multiple kind ##lins , and transformed cells often exhibit a ##ber ##rant expression , both in the is ##of ##orms and the [SEP]',
  'existing_cui': 'C1720776',
  'existing_cui_name': 'Kindling , Neurologic',
  'existing_cui_desc': '[CLS] Kind ##ling , N ##eur ##olo ##gic [unused3] ( B ##iol ##og ##ic Fun ##ction , N ##eur ##ology : kind ##ling ; Kind ##lings , N ##eur ##olo ##gic ; Kind ##ling ; N ##eur ##olo ##gic Kind ##lings ; N ##eur ##olo ##gic Kind ##ling ) [SEP]',
  'our_bi_cui': 'C0010422',
  'our_bi_cui_name': 'Crystallins',
  'our_bi_cui_desc': '[CLS] Crystal ##lins [unused3] ( Chemical , substance : Crystal ##lins [ Chemical / In ##g ##red ##ient ] ; Crystal ##lin ; Pro ##tein ##s , Len ##s ; C ##rist ##all ##in ; Len ##s Pro ##tein ##s ; crystal ##lin ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28210319.14',
  'test_set_idx': 28545,
  'type': 'T038',
  'name': 'pump failure',
  'context': '[CLS] characterised by progressive skeletal muscle wasting , including of the respiratory muscles . Re ##sp ##ira ##tory failure , i . e . when the respiratory system fails in its gas exchange functions , is a common feature in muscular d ##ys ##tro ##phy , being the main cause of death , and it is a consequence of lung failure , [unused1] pump failure [unused2] or a combination of the two . The former is due to re ##current as ##piration , the latter to progressive weakness of respiratory muscles and an increase in the load against which they must contract . In fact , both the resist ##ive and elastic components of the work of breathing increase due to air ##way o ##bs ##truction and [SEP]',
  'existing_cui': 'C1417212',
  'existing_cui_name': 'MMP7 gene',
  'existing_cui_desc': '[CLS] M ##MP ##7 gene [unused3] ( Ana ##tom ##ical Structure : M ##MP ##7 Gene ; mat ##rily ##sin ; matrix metal ##lop ##ept ##idas ##e 7 ; MA ##TR ##IX ME ##TA ##LL ##OP ##RO ##TE ##IN ##AS ##E 7 ; P ##UT ##AT ##IVE ME ##TA ##LL ##OP ##RO ##TE ##IN ##AS ##E I ; MA ##TR ##IL ##Y ##SI ##N , U ##TE ##RI ##NE ; P ##UM ##P - 1 ; Matrix Metal ##lop ##ept ##idas ##e 7 ( Mat ##rily ##sin , U ##ter ##ine ) Gene ) [SEP]',
  'our_bi_cui': 'C0948755',
  'our_bi_cui_name': 'Pulmonary failure',
  'our_bi_cui_desc': '[CLS] P ##ul ##mona ##ry failure [unused3] ( B ##iol ##og ##ic Fun ##ction : pulmonary failure ; failure pulmonary ) [SEP]',
  'our_cross_cui': 'C1145670',
  'our_cross_cui_name': 'Respiratory Failure',
  'our_cross_cui_desc': '[CLS] Re ##sp ##ira ##tory F ##ail ##ure [unused3] ( B ##iol ##og ##ic Fun ##ction , context - dependent category , diagnosis , disorder , situation : [ D ] Re ##sp ##ira ##tory failure ; respiratory ins ##uff ##iciency / failure ; F ##ail ##ure ; respiratory ; Re ##sp ##ira ##tory failure ; F ##ail ##ure , Re ##sp ##ira ##tory ; Re ##sp ##ira ##tory failure , un ##sp ##ec ##ified ; respiratory failure ; respiratory ins ##uff ##iciency ; F ##ail ##ure respiratory ; Re ##spiration failure ) [SEP]'},
 {'mention_id': '27882455.29',
  'test_set_idx': 20577,
  'type': 'T038',
  'name': 'memory stabilization processes',
  'context': '[CLS] these memory traces into an existing body of knowledge ( item integration ) . Older adults learned 13 non ##words and were tested on their memory for the non ##words , and on whether these non ##words impacted upon processing of similar - sounding English words immediately and 24 hours later . Part ##ici ##pants accurately recognized the non ##words immediately , but showed significant decreases in delayed recognition and recall . In comparison , the non ##words impacted upon processing of similar - sounding words only in the delayed test . Together , these findings suggest that memory consolidation processes may be more evident in item integration than [unused1] memory stab ##ilization processes [unused2] for new de ##c ##lar ##ative memories in older adults . [SEP]',
  'existing_cui': 'C1293130',
  'existing_cui_name': 'Stabilization',
  'existing_cui_desc': '[CLS] St ##abi ##li ##zation [unused3] ( Health Care Act ##ivity , procedure : stab ##ilization ; St ##abi ##lis ##ation ) [SEP]',
  'our_bi_cui': 'C1293130',
  'our_bi_cui_name': 'Stabilization',
  'our_bi_cui_desc': '[CLS] St ##abi ##li ##zation [unused3] ( Health Care Act ##ivity , procedure : stab ##ilization ; St ##abi ##lis ##ation ) [SEP]',
  'our_cross_cui': 'C0025361',
  'our_cross_cui_name': 'Mental Processes',
  'our_cross_cui_desc': '[CLS] Mental Process ##es [unused3] ( B ##iol ##og ##ic Fun ##ction , Human , function , o ##bs ##er ##vable entity : Human Information Process ##ing ; Thought process ; cognitive processing ; processes thought ; processing thought ; Information Process ##ing , Human ; Human information processing ; Information Process ##es ; mental processing ; cognitive process ; Form of thinking , function ; mental processes ; Form of thought ; Form of thinking ; Human Information Process ##es ; Mental Process ; mental process ; thought process ) [SEP]'},
 {'mention_id': '28441489.43',
  'test_set_idx': 36165,
  'type': 'T033',
  'name': 'cold - start',
  'context': '[CLS] , and x ##yle ##nes . Therefore , the secondary organic a ##ero ##sol and oz ##one formation potential of the exhaust does not depend on engine technology . Cold - start contributes a larger fraction of the total unified cycle emissions for vehicles meeting more - string ##ent emission standards . Organic gas emissions were the most sensitive to [unused1] cold - start [unused2] compared to the other poll ##uta ##nts tested here . There were no statistical ##ly significant differences in the effects of cold - start on G ##DI ##s and P ##FI ##s . For our test fleet , the measured 14 . 5 % decrease in CO ##2 emissions from G ##DI ##s was much greater than the potential climate forcing [SEP]',
  'existing_cui': 'C0452588',
  'existing_cui_name': 'Start brand of breakfast cereal',
  'existing_cui_desc': '[CLS] Start brand of breakfast cereal [unused3] ( Food , substance : Start ) [SEP]',
  'our_bi_cui': 'C0161734',
  'our_bi_cui_name': 'Effects of low temperature',
  'our_bi_cui_desc': '[CLS] Effects of low temperature [unused3] ( In ##ju ##ry or Po ##ison ##ing , diagnosis , disorder : Effect reduced te ##mp ; cold effect ; effect of reduced temperature ; Effects of freezing or excessive cold ; A ##cci ##dent due to excessive cold of un ##sp ##ec ##ified origin ; Cold Effects ; effects of ; cold ; Effect of reduced temperature , un ##sp ##ec ##ified ; cold effects ; effects cold ; Effects of reduced temperature ; Ex ##cess ##ive cold ; effects of ; low temperature ; Effects of freezing AND / OR excessive cold ; Un ##sp ##ec ##ified effect of reduced temperature ; Ex ##cess ##ive cold of un ##sp ##ec ##ified origin ; cold ; effects of ; [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27357394.22',
  'test_set_idx': 3609,
  'type': 'T033',
  'name': 'losers',
  'context': '[CLS] establish mobility . V ##aria ##tion in the extent of patient mobility was observed across the studies . Mo ##bility was positively associated with lower waiting times , indicators of better service quality , and access to advanced technology . It was negatively associated with advanced age or lower socio ##economic backgrounds . From a policy perspective we demonstrate that a significant proportion of patients are prepared to travel beyond their nearest provider for elect ##ive services . As a consequence , some providers are likely to be " winners " and others " [unused1] loser ##s [unused2] , " which could result in overall decreased provider capacity or in ##ef ##ficient u ##til ##ization of existing services . Equity also remains a key concern . [SEP]',
  'existing_cui': 'C0230789',
  'existing_cui_name': 'Smooth Endoplasmic Reticulum',
  'existing_cui_desc': '[CLS] S ##mo ##oth End ##op ##las ##mic Re ##tic ##ulum [unused3] ( Ana ##tom ##ical Structure , body structure , cell structure , s ##ER , smooth end ##op ##las ##mic re ##tic ##ulum : End ##op ##las ##mic Re ##tic ##ulum , S ##mo ##oth ; smooth end ##op ##las ##mic re ##tic ##ulum ; A ##gra ##nu ##lar End ##op ##las ##mic Re ##tic ##ulum ; A ##gra ##nu ##lar end ##op ##las ##mic re ##tic ##ulum ; End ##op ##las ##mic Re ##tic ##ulum , A ##gra ##nu ##lar ; Re ##tic ##ulum end ##op ##las ##mic ##um non ##gra ##nu ##los ##um ; S ##mo ##oth end ##op ##las ##mic re ##tic ##ulum ; smooth ER ; S ##mo ##oth - Surface [SEP]',
  'our_bi_cui': 'C1257890',
  'our_bi_cui_name': 'Population Group',
  'our_bi_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28338634.26',
  'test_set_idx': 32475,
  'type': 'T033',
  'name': 'detection',
  'context': "[CLS] proposed for ass ##ay of the ne ##uro ##to ##xi ##c compounds . The prepared na ##no ##par ##tic ##les were modified by pure Ace ##ty ##l ##cho ##lines ##tera ##se and they were used for the measurement anti - Alzheimer ' s drug gal ##ant ##amine and car ##ba ##mate p ##est ##icide car ##bo ##fu ##ran with limit of [unused1] detection [unused2] 1 . 5 µ ##M and 20 n ##M , respectively . All measurements were carried out using screen - printed sensor with carbon working , silver reference , and carbon auxiliary electro ##de . Standard El ##lman ' s ass ##ay was used for valid ##ation measurement of both inhibitor ##s . Part of this work was the elimination of re [SEP]",
  'existing_cui': 'C0206100',
  'existing_cui_name': 'Signal Detection',
  'existing_cui_desc': '[CLS] Signal Det ##ec ##tion [unused3] ( B ##iol ##og ##ic Fun ##ction , Psychology , Psychology , Signal : Signal Det ##ec ##tion Theo ##ries ; Signal Det ##ec ##tions , Psychological ; Theory , Signal Det ##ec ##tion ; Signal detection ; S ##IG ##NA ##L DE ##TE ##CT ##ION AN ##AL ; Psychological Signal Det ##ec ##tions ; Det ##ec ##tion ; Signal Det ##ec ##tion Ana ##ly ##ses ; S ##IG ##NA ##L DE ##TE ##CT ##ION ; Det ##ec ##tions , Psychological Signal ; Psychological Signal Det ##ec ##tion ; Signal Det ##ec ##tion Theory ; Theo ##ries , Signal Det ##ec ##tion ; Signal Det ##ec ##tions ; Ana ##ly ##ses , Signal Det ##ec ##tion ; Det ##ec ##tion , [SEP]',
  'our_bi_cui': 'C1511790',
  'our_bi_cui_name': 'Detection',
  'our_bi_cui_desc': '[CLS] Det ##ec ##tion [unused3] ( Health Care Act ##ivity : Det ##ec ##ted ) [SEP]',
  'our_cross_cui': 'C0442726',
  'our_cross_cui_name': 'Detected',
  'our_cross_cui_desc': '[CLS] Det ##ec ##ted [unused3] ( Finding , finding , finding , qualifier value : detected ; detect ) [SEP]'},
 {'mention_id': '28158248.6',
  'test_set_idx': 27110,
  'type': 'T170',
  'name': 'ordinary differential equations',
  'context': '[CLS] Model ##ing the flux of meta ##bol ##ites in the juvenile hormone bio ##sy ##nt ##hesis pathway using generalized add ##itive models and [unused1] ordinary differential equations [unused2] Ju ##ven ##ile hormone regulate ##s development and reproductive mat ##uration in insects . The co ##rp ##ora all ##ata from female adult m ##os ##quito ##es s ##ynth ##esi ##ze flu ##ct ##uating levels of Ju ##ven ##ile hormone , which have been linked to the o ##var ##ian development and are influenced by nutrition ##al signals . The rate of Ju ##ven ##ile hormone bio ##sy ##nt ##hesis is controlled by the rate of flux of is ##op ##ren ##oids in the pathway , which is the outcome of a complex inter ##play of changes in [SEP]',
  'existing_cui': 'C0011906',
  'existing_cui_name': 'Differential Diagnosis',
  'existing_cui_desc': '[CLS] Different ##ial Di ##ag ##nosis [unused3] ( Health Care Act ##ivity , context ##ual qualifier , qualifier value : D ##IA ##G D ##IF ##F ##ER ##EN ##TI ##AL ; Di ##ag ##nose ##s , Different ##ial ; D ##IF ##F ##ER ##EN ##TI ##AL D ##IA ##G ; Di ##ag ##nosis , Different ##ial ; Different ##ial d ##x ; D ##IA ##G D ##IF ##F ##ER ; differential di ##ag ##nose ##s ; differential diagnosis ; Different ##ial diagnosis ( context ##ual qualifier ) ; Different ##ial Di ##ag ##nose ##s ; differential d ##x ; Different ##ial diagnosis ; D ##IF ##F ##ER D ##IA ##G ) [SEP]',
  'our_bi_cui': 'C1705273',
  'our_bi_cui_name': 'Mathematical Operator',
  'our_bi_cui_desc': '[CLS] Mathematical Opera ##tor [unused3] ( Intel ##lect ##ual Product : Opera ##tor ; Fun ##ction ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27699206.9',
  'test_set_idx': 14490,
  'type': 'T017',
  'name': 'lateral occipital',
  'context': '[CLS] - Based Category Select ##ivity in Human Visual Co ##rte ##x In f ##MR ##I studies , human lateral o ##cci ##pit ##al cortex is thought to respond selective ##ly to images of objects , compared with non ##ob ##jects . However , it remains un ##res ##ol ##ved whether all objects e ##voke equivalent levels of activity in [unused1] lateral o ##cci ##pit ##al [unused2] , and , if not , which image features produce stronger activation . Here , we used an un ##bia ##sed para ##metric texture model to predict preferred versus non ##p ##re ##ferred stimuli in lateral o ##cci ##pit ##al . O ##bs ##er ##vation and ps ##ych ##op ##hy ##si ##cal results showed that predicted preferred stimuli ( both [SEP]',
  'existing_cui': 'C0923804',
  'existing_cui_name': 'Lateral occipital artery',
  'existing_cui_desc': '[CLS] Later ##al o ##cci ##pit ##al artery [unused3] ( Ana ##tom ##ical Structure , Arte ##ria c ##ere ##bri posterior : P ##3 segment of posterior cerebral artery ; Arte ##ria o ##cci ##pit ##alis lateral ##is ; Se ##gment ##um P ##3 ; A . o ##cci ##pit ##alis lateral ##is ) [SEP]',
  'our_bi_cui': 'C0228229',
  'our_bi_cui_name': 'lateral occipital sulcus',
  'our_bi_cui_desc': '[CLS] lateral o ##cci ##pit ##al su ##l ##cus [unused3] ( Spa ##tial Concept , human only , human only , body structure : Later ##al o ##cci ##pit ##al su ##l ##cus ; Structure of lateral o ##cci ##pit ##al su ##l ##cus ; Later ##al - O ##cci ##pit ##al Sul ##cus ; Sul ##cus o ##cci ##pit ##alis lateral ##is ) [SEP]',
  'our_cross_cui': 'C0028785',
  'our_cross_cui_name': 'Occipital lobe',
  'our_cross_cui_desc': '[CLS] O ##cci ##pit ##al lobe [unused3] ( Ana ##tom ##ical Structure , body structure : o ##cci ##pit ##al region ; O ##cci ##pit ##al cortex ; lo ##bus o ##cci ##pit ##alis ; O ##cci ##pit ##al Lo ##be ; o ##cci ##pit ##al lobes ; O ##cci ##pit ##al lobes ; O ##cci ##pit ##al Region ##s ; O ##cci ##pit ##al Region ; o ##cci ##pit ##al lobe ; Lo ##be , O ##cci ##pit ##al ; lobe o ##cci ##pit ##al ; O ##cci ##pit ##al region ; Region , O ##cci ##pit ##al ; Lo ##bus o ##cci ##pit ##alis ; o ##cci ##pit ##al cortex ; Reg ##io o ##cci ##pit ##alis ; Lo ##bes , O ##cci ##pit ##al [SEP]'},
 {'mention_id': '27781034.48',
  'test_set_idx': 16561,
  'type': 'T033',
  'name': 'frequently co - occur',
  'context': '[CLS] brain an ##oma ##lies . However , the cause of genetic generalized e ##pile ##psy remained unclear . In this study , we describe and review 28 individuals with 22 ##q ##11 . 2 del ##eti ##on syndrome and genetic generalized e ##pile ##psy ( especially juvenile my ##oc ##lon ##ic e ##pile ##psy ) , showing that both disorders [unused1] frequently co - occur [unused2] . Compared to the reported prevalence of 15 - 21 % , in our case series only 10 % of 22 ##q ##11 . 2 del ##eti ##on syndrome individuals were found to have e ##pile ##psy , often genetic generalized e ##pile ##psy . Since 22 ##q ##11 . 2 does not contain convincing genetic generalized e ##pile ##psy [SEP]',
  'existing_cui': 'C0558131',
  'existing_cui_name': 'Eating frequently',
  'existing_cui_desc': '[CLS] Eat ##ing frequently [unused3] ( Finding , finding ) [SEP]',
  'our_bi_cui': 'C0277557',
  'our_bi_cui_name': 'Intercurrent disease',
  'our_bi_cui_desc': '[CLS] Inter ##current disease [unused3] ( B ##iol ##og ##ic Fun ##ction , finding ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28011059.45',
  'test_set_idx': 23692,
  'type': 'T098',
  'name': 'children',
  'context': '[CLS] ##e se ##quencing or Il ##lum ##ina 450 ##K array ##s . Co ##rd blood mon ##on ##uc ##lea ##r cell - derived IL - 1 ##β levels were measured by means of E ##L ##IS ##A . Neo ##nat ##al immune cells harbor ##ed 58 ##9 differential ##ly met ##hyl ##ated regions that distinguished In ##fant I ##mm ##une Study [unused1] children [unused2] who did and did not have as ##th ##ma by age 9 years . In all 3 co ##hor ##ts met ##hyl ##ation in SM ##AD ##3 , the most connected node within the network of as ##th ##ma - associated , differential ##ly met ##hyl ##ated regions , was selective ##ly increased in as ##th ##matic children of as ##th ##matic [SEP]',
  'existing_cui': 'C0682171',
  'existing_cui_name': 'History of being abused as child',
  'existing_cui_desc': '[CLS] History of being abused as child [unused3] ( Finding , history : child a ##bus ; abused as a child ; abused as child ; child abused ; abused child ) [SEP]',
  'our_bi_cui': 'C0264408',
  'our_bi_cui_name': 'Childhood asthma',
  'our_bi_cui_desc': '[CLS] Child ##hood as ##th ##ma [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : Child ##hood As ##th ##ma ; as ##th ##ma childhood ; As ##th ##ma , childhood ; As ##th ##ma in Children ; as ##th ##ma ; childhood ; As ##th ##ma in children ; childhood ; as ##th ##ma ; childhood as ##th ##ma ) [SEP]',
  'our_cross_cui': 'C1257890',
  'our_cross_cui_name': 'Population Group',
  'our_cross_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]'},
 {'mention_id': '28077027.24',
  'test_set_idx': 24980,
  'type': 'T170',
  'name': 'EFSA',
  'context': '[CLS] - 90 years . The p ##est ##icide residue data were obtained from the Shanghai monitoring programme during 2008 - 11 with 34 organ ##op ##hos ##phate ##s and 11 car ##ba ##mates anal ##ys ##ed in a total of 53 ##35 samples of vegetables and fruits . A pro ##ba ##bil ##istic approach was performed as recommended by the [unused1] E ##FS ##A [unused2] , using the optimistic model with non - detect ##s set as zero and with processing factors being used and the p ##ess ##im ##istic model with non - detect ##s replaced by limit of detection and without processing factors . We used the relative pot ##ency factor method to normal ##ise the various p ##est ##icides to the index compound [SEP]',
  'existing_cui': 'C1537461',
  'existing_cui_name': 'KIAA1109 gene',
  'existing_cui_desc': '[CLS] K ##IA ##A ##11 ##0 ##9 gene [unused3] ( Ana ##tom ##ical Structure : T ##week ; fragile site - associated ; K ##IA ##A ##13 ##7 ##1 ; FL ##J ##21 ##40 ##4 ; T ##W ##EE ##K , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF ; K ##IA ##A ##11 ##0 ##9 GE ##NE ) [SEP]',
  'our_bi_cui': 'C1708333',
  'our_bi_cui_name': 'Health Care Organization',
  'our_bi_cui_desc': '[CLS] Health Care Organization [unused3] ( Organization : Healthcare Organization ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28107548.15',
  'test_set_idx': 25818,
  'type': 'T170',
  'name': 'multiple random forest classifiers',
  'context': '[CLS] ##ical classification to re ##fine the detection results . High - level feature representation is first learned by a deep learning network , where multi ##par ##ame ##tric MR images are used as the input data . Then , based on the learned high - level features , a hi ##era ##rch ##ical classification method is developed , where [unused1] multiple random forest class ##ifier ##s [unused2] are it ##erative ##ly constructed to re ##fine the detection results of pro ##state cancer . The experiments were carried on 21 real patient subjects , and the proposed method achieve ##s an averaged section - based evaluation of 89 . 90 % , an averaged sensitivity of 91 . 51 % , and an averaged specific ##ity of [SEP]',
  'existing_cui': 'C0034656',
  'existing_cui_name': 'Randomization',
  'existing_cui_desc': '[CLS] Random ##ization [unused3] ( Research Act ##ivity : Random ##ized ; Random All ##oc ##ation ; random ##ization ; Trial is Random ##ized ; All ##oc ##ation , Random ; random allocation ; random ##ized ; RA ##ND ##OM ) [SEP]',
  'our_bi_cui': 'C0008902',
  'our_bi_cui_name': 'Classification',
  'our_bi_cui_desc': '[CLS] Classification [unused3] ( Intel ##lect ##ual Product , attribute , qualifier value : C ##LA ##SS ; class ##ifying ; Classification ##s ; class ##ify ; classification ##s ; Class ##ified ; System ##atic ##s ; Classification system ; classification system ; classified ; cat ##ego ##riz ##ation ; classified ##s ; classification systems ; systematic ##s ; Classification Systems ; classification ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27427386.17',
  'test_set_idx': 5255,
  'type': 'T017',
  'name': 'p. 58I > V',
  'context': "[CLS] modify the p ##hen ##otypic expression of primary Le ##ber ' s hereditary op ##tic ne ##uro ##pathy - associated mitochondrial DNA mutations . In this study , we demonstrated that the Le ##ber ' s hereditary op ##tic ne ##uro ##pathy su ##s ##ce ##pt ##ibility all ##ele ( m . 145 ##0 ##2 ##T > C , [unused1] p . 58 ##I > V [unused2] ) in the N ##D ##6 gene m ##od ##ulated the p ##hen ##otypic expression of primary Le ##ber ' s hereditary op ##tic ne ##uro ##pathy - associated m . 117 ##7 ##8 ##G > A mutation . Twenty - two Han Chinese p ##ed ##ig ##ree ##s carrying m . 145 ##0 ##2 ##T > C and [SEP]",
  'existing_cui': 'C0614783',
  'existing_cui_name': 'vasectrin I',
  'existing_cui_desc': '[CLS] vase ##ct ##rin I [unused3] ( Chemical : i vs ; Secret ##in , 4 - L - al ##ani ##ne - ; V ##S I ; 4 - Al ##a - secret ##in ; secret ##in , al ##ani ##ne ( 4 ) - ; secret ##in , Al ##a ( 4 ) - ; vase ##ct ##rin i ) [SEP]',
  'our_bi_cui': 'C0026882',
  'our_bi_cui_name': 'Mutation',
  'our_bi_cui_desc': '[CLS] Mu ##tation [unused3] ( B ##iol ##og ##ic Fun ##ction , finding : genome mutation ; Gene ##tic mutation ; genetic mutation ; genetic mutations ; mutations ; Mu ##tations ; mutation ; alterations genetic ; genetic alter ##ation ) [SEP]',
  'our_cross_cui': 'C0002085',
  'our_cross_cui_name': 'Alleles',
  'our_cross_cui_desc': '[CLS] All ##ele ##s [unused3] ( Ana ##tom ##ical Structure , finding : All ##ele ; All ##elo ##mor ##ph ; Gene ##tic all ##ele ##s ; all ##ele ; All ##elo ##mor ##phs ; all ##ele ##s ; AL ##LE ##LE ) [SEP]'},
 {'mention_id': '27500205.33',
  'test_set_idx': 8465,
  'type': 'T103',
  'name': 'adaptor proteins',
  'context': '[CLS] controlling in ##te ##g ##rin function . In v ##it ##ro studies , in v ##ivo studies of mice def ##icient in kind ##lins , and studies of patients with genetic def ##ici ##encies of kind ##lins have clearly established that they regulate the capacity of in ##te ##g ##rin ##s to media ##te their functions . Kind ##lins are [unused1] adapt ##or proteins [unused2] ; their function em ##ana ##te from their interaction with binding partners , including the c ##yt ##op ##las ##mic tails of in ##te ##g ##rin ##s and components of the act ##in c ##yt ##os ##kel ##eton . The purpose of this review is to provide a brief overview of kind ##lin structure and function , a consideration of their [SEP]',
  'existing_cui': 'C1449886',
  'existing_cui_name': 'Adaptor Proteins , Signal Transducing',
  'existing_cui_desc': '[CLS] Ada ##pt ##or Pro ##tein ##s , Signal Trans ##ducing [unused3] ( Chemical : adapt ##er protein ; Ada ##pt ##er Pro ##tein ; Ada ##pt ##or Signal ##ing Pro ##tein ; Signal Trans ##ducing Ada ##pt ##or Pro ##tein ##s ; Ada ##pt ##er Signal ##ing Pro ##tein ; Ada ##pt ##or Pro ##tein ; Signal Trans ##ducing Ada ##pt ##or Pro ##tein ; Ada ##pt ##or Pro ##tein ##s , Signal Trans ##ducing [ Chemical / In ##g ##red ##ient ] ) [SEP]',
  'our_bi_cui': 'C1449886',
  'our_bi_cui_name': 'Adaptor Proteins , Signal Transducing',
  'our_bi_cui_desc': '[CLS] Ada ##pt ##or Pro ##tein ##s , Signal Trans ##ducing [unused3] ( Chemical : adapt ##er protein ; Ada ##pt ##er Pro ##tein ; Ada ##pt ##or Signal ##ing Pro ##tein ; Signal Trans ##ducing Ada ##pt ##or Pro ##tein ##s ; Ada ##pt ##er Signal ##ing Pro ##tein ; Ada ##pt ##or Pro ##tein ; Signal Trans ##ducing Ada ##pt ##or Pro ##tein ; Ada ##pt ##or Pro ##tein ##s , Signal Trans ##ducing [ Chemical / In ##g ##red ##ient ] ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28249376.35',
  'test_set_idx': 29905,
  'type': 'T168',
  'name': 'concentrate diet',
  'context': '[CLS] at 0 ( control ) , 0 . 8 , and 1 . 6 m ##L / d on apparent nut ##rient dig ##est ##ibility , r ##ume ##n f ##er ##mentation characteristics , r ##ume ##n micro ##bial population and blood chemical meta ##bol ##ites . Animals were fed with a 50 : 50 al ##fa ##lf ##a hay : [unused1] concentrate diet [unused2] . R ##umi ##nal pH , total volatile fatty acids concentration , m ##olar proportion of individual volatile fatty acids , ace ##tate : prop ##ion ##ate ratio and met ##hane production were not affected with mixture of essential oils . Re ##lative to the control , Small p ##eptide ##s plus amino acid nitrogen and large p ##eptide ##s nitrogen [SEP]',
  'existing_cui': 'C1880165',
  'existing_cui_name': 'Concentrate Dosage Form',
  'existing_cui_desc': '[CLS] Con ##cent ##rate Do ##sa ##ge Form [unused3] ( Chemical : Con ##cent ##rated ; Con ##cent ##rate ; CO ##NC ; CO ##NC ##EN ##TR ##AT ##E ; Con ##c ; Con ##cent ##rated Do ##sa ##ge Form ; Con ##cent ##rated Do ##se Form ) [SEP]',
  'our_bi_cui': 'C0003050',
  'our_bi_cui_name': 'Animal feed',
  'our_bi_cui_desc': '[CLS] Animal feed [unused3] ( Food , substance , substance : animal food ; animal feeds ; animal feeding ; animals food ; food animal ; Fe ##eds , Animal ; Animal Fe ##ed ; Fe ##ed , Animal ; animal feed ; Animal Fe ##eds ) [SEP]',
  'our_cross_cui': 'C0012155',
  'our_cross_cui_name': 'Diet',
  'our_cross_cui_desc': '[CLS] Diet [unused3] ( Food , finding : Diet ##s ; regular diet ? ; Diet ##ary ; diet ; Diet ##ary finding ; diet ##s ) [SEP]'},
 {'mention_id': '27306834.45',
  'test_set_idx': 2032,
  'type': 'T017',
  'name': 'naïve cell',
  'context': '[CLS] expression patterns of memory - promoting transcription factors , T - bet and E ##ome ##s , induced in a rapid and sustainable manner . N ##T ##ef ##f cells appeared to have lower expression of Fox ##p ##1 and were re ##fra ##ctor ##y to a ##pop ##tosis upon T ##G ##F - β conditioning , implying better survival potential and resistance to tumor - induced immune suppression . Of CD ##8 ( + ) T cell pools activated to tumor - specific CT ##L ##s , [unused1] na ##ï ##ve cell [unused2] generated effect ##ors possessed the most potent c ##yt ##oto ##xi ##c activity , valid ##ating implications for use in rational design of adopt ##ive im ##mu ##not ##her ##ap ##y . [SEP]',
  'existing_cui': 'C1179416',
  'existing_cui_name': 'Endothelial cell of venous sinus of red pulp of spleen',
  'existing_cui_desc': '[CLS] End ##oth ##eli ##al cell of ve ##nous sin ##us of red pulp of s ##ple ##en [unused3] ( Ana ##tom ##ical Structure : St ##ave cell ) [SEP]',
  'our_bi_cui': 'C0007634',
  'our_bi_cui_name': 'Cells',
  'our_bi_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]',
  'our_cross_cui': 'C3641721',
  'our_cross_cui_name': 'Naive T-Lymphocyte',
  'our_cross_cui_desc': '[CLS] Na ##ive T - L ##ymph ##oc ##yte [unused3] ( Ana ##tom ##ical Structure : T ##h ##0 ; Na ##ive T L ##ymph ##oc ##yte ; Na ##ive T - Cell ; Na ##ive T Cell ##s ; Na ##ive T Cell ) [SEP]'},
 {'mention_id': '28457334.24',
  'test_set_idx': 36634,
  'type': 'T103',
  'name': 'extracellular RNA',
  'context': '[CLS] abdominal surgery . A total of 15 patients with se ##psis , 10 post ##oper ##ative patients , and 10 healthy volunteers were included in this longitudinal study . Blood was collected at se ##psis onset and after surgery respectively , as well as after 24 , 72 and 168 h . Level ##s of Cell - free DNA and [unused1] extra ##cellular RNA [unused2] were measured by quantitative probe - based polymer ##ase chain reaction . In addition , th ##rom ##boe ##last ##ography for co ##agu ##lation as well as th ##rom ##boa ##gg ##re ##go ##metry for plate ##let function was conducted . Both Cell - free DNA and extra ##cellular RNA were elevated in patients with se ##psis compared with post ##oper [SEP]',
  'existing_cui': 'C0521119',
  'existing_cui_name': 'Extracellular',
  'existing_cui_desc': '[CLS] Extra ##cellular [unused3] ( Ana ##tom ##ical Structure , qualifier value : extra ##cellular ; extra ##cellular region ) [SEP]',
  'our_bi_cui': 'C0230885',
  'our_bi_cui_name': 'Extracellular material',
  'our_bi_cui_desc': '[CLS] Extra ##cellular material [unused3] ( Body Sub ##stance , substance ) [SEP]',
  'our_cross_cui': 'C0035668',
  'our_cross_cui_name': 'RNA',
  'our_cross_cui_desc': '[CLS] RNA [unused3] ( Chemical , RNA , R ##ib ##on ##uc ##le ##ic A ##cid , substance : rib ##on ##uc ##le ##ic acid ; R ##ib ##on ##uc ##le ##ic acids ; RNA , Non - Pol ##ya ##den ##yla ##ted ; rib ##on ##uc ##lein ##ic ##um acid ##um / rib ##on ; RNA - R ##ib ##on ##uc ##le ##ic acid ; RNA [ Chemical / In ##g ##red ##ient ] ; RNA , Non Pol ##ya ##den ##yla ##ted ; Non - Pol ##ya ##den ##yla ##ted RNA ; r ##na ; Non Pol ##ya ##den ##yla ##ted RNA ; A ##cid , R ##ib ##on ##uc ##le ##ic ; R ##ib ##on ##uc ##le ##ic acid ; Gene Products , RNA [SEP]'},
 {'mention_id': '27732552.9',
  'test_set_idx': 15246,
  'type': 'T058',
  'name': 'techniques',
  'context': '[CLS] Volume ##tric a ##bs ##or ##pt ##ive micro ##sa ##mp ##ling at home as an alternative tool for the monitoring of H ##b ##A ##1 ##c in diabetes patients Micro ##sa ##mp ##ling techniques have several advantages over traditional blood collection . Dr ##ied blood spot sampling and blood collection with he ##par ##ini ##zed cap ##illa ##ries are the standard [unused1] techniques [unused2] . Volume ##tric a ##bs ##or ##pt ##ive micro ##sa ##mp ##ling is a novel technique that collects a fixed volume of blood by applying an absorb ##ent tip to a blood drop . In the present study we explored the f ##eas ##ibility of H ##b ##A ##1 ##c monitoring with Volume ##tric a ##bs ##or ##pt ##ive micro ##sa ##mp [SEP]',
  'existing_cui': 'C0450241',
  'existing_cui_name': 'Ffooks technique',
  'existing_cui_desc': '[CLS] F ##fo ##ok ##s technique [unused3] ( Health Care Act ##ivity , qualifier value ) [SEP]',
  'our_bi_cui': 'C0025663',
  'our_bi_cui_name': 'Methods',
  'our_bi_cui_desc': '[CLS] Method ##s [unused3] ( Intel ##lect ##ual Product , attribute : methods ; Method ; method ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '27647760.18',
  'test_set_idx': 12785,
  'type': 'T092',
  'name': 'Sequenom',
  'context': "[CLS] B ##D ##NF genes and drug addiction in the Han Chinese population . We conducted a case - control study among 69 ##2 cases and 700 healthy controls from Xi ' an , China . Eight single n ##uc ##leo ##tide p ##oly ##mor ##phism ##s were selected and g ##eno ##type ##d using Mass ##AR ##RA ##Y technology ( [unused1] Se ##que ##no ##m [unused2] , San Diego , CA , USA ) . Odd ##s ratios and 95 % confidence intervals were calculated by un ##con ##ditional log ##istic re ##gression adjusting for age and sex . Using the ch ##i - squared test , we found that r ##s ##7 ##48 ##13 ##11 ( Odd ##s ratios = 1 . 275 , 95 [SEP]",
  'existing_cui': 'C1414863',
  'existing_cui_name': 'FUT2 gene',
  'existing_cui_desc': '[CLS] F ##UT ##2 gene [unused3] ( Ana ##tom ##ical Structure : GDP - L - f ##uc ##ose : beta - D - gal ##act ##os ##ide 2 - alpha - L - f ##uc ##osy ##lt ##ran ##s ##fer ##ase 2 ; alpha ( 1 , 2 ) F ##T ##2 ; secret ##or blood group alpha - 2 - f ##uc ##osy ##lt ##ran ##s ##fer ##ase ; SEC ##2 ; SEC ##RE ##TO ##R FA ##CT ##OR ; f ##uc ##osy ##lt ##ran ##s ##fer ##ase 2 ; se ##j ; alpha ( 1 , 2 ) f ##uc ##osy ##lt ##ran ##s ##fer ##ase ; F ##UC ##OS ##Y ##LT ##RA ##NS ##F ##ER ##AS ##E 2 ; secret ##or factor ; [SEP]',
  'our_bi_cui': 'C0025080',
  'our_bi_cui_name': 'Medical Devices',
  'our_bi_cui_desc': '[CLS] Medical Devi ##ces [unused3] ( Medical Devi ##ce , physical device , physical object : Clinical equipment ; B ##io ##medical device ; DE ##VI ##CE ME ##D ; ME ##D DE ##VI ##CE ##S ; ME ##D DE ##VI ##CE ; bio ##medical devices ; Devi ##ces , Medical ; Medical devices ; General bio ##medical device ; DE ##VI ##CE , GE ##NE ##RA ##L ME ##DI ##CA ##L ; B ##io ##medical equipment ; Devi ##ce , Medical ; Hospital , medical AND / OR surgical equipment ( physical device ) ; bio ##medical equipment ; medical equipment ; Medical equipment ; DE ##VI ##CE ##S ME ##D ; DE ##VI ##CE ; DE ##VI ##CE ##S ; medical device ; medical devices [SEP]',
  'our_cross_cui': 'C0683757',
  'our_cross_cui_name': 'Company',
  'our_cross_cui_desc': '[CLS] Company [unused3] ( Organization , organization , organization : Business ; company ; Companies ; companies ) [SEP]'},
 {'mention_id': '28071754.10',
  'test_set_idx': 24715,
  'type': 'T037',
  'name': 'injury',
  'context': '[CLS] infrared protects vascular end ##oth ##eli ##al cells from advanced g ##ly ##cation end products - induced injury via Pro ##my ##elo ##cy ##tic le ##uke ##mia zinc finger protein - mediated auto ##pha ##gy in di ##abe ##tic mice The accumulation of advanced g ##ly ##cation end products in di ##abe ##tic patients induce ##s vascular end ##oth ##eli ##al [unused1] injury [unused2] . Pro ##my ##elo ##cy ##tic le ##uke ##mia zinc finger protein is a transcription factor that can be activated by low - temperature far - infrared i ##rra ##diation to ex ##ert beneficial effects on the vascular end ##oth ##eli ##um . In the present study , we investigated the influence of far - infrared - induced Pro ##my ##elo ##cy [SEP]',
  'existing_cui': 'C0005604',
  'existing_cui_name': 'Birth Injuries',
  'existing_cui_desc': '[CLS] Birth In ##ju ##ries [unused3] ( In ##ju ##ry or Po ##ison ##ing , Birth , diagnosis , disorder : In ##ju ##ries , Birth ; T ##ra ##umatic birth ; injury birth ; Birth trauma , un ##sp ##ec ##ified ; Birth T ##ra ##uma ; Birth trauma of f ##etus ; injury ; birth ; traumatic birth ; birth trauma ; birth injuries ; Birth In ##ju ##ry ; trauma birth ; Neo ##nat ##al Birth In ##ju ##ry ; birth ##ing injuries ; Birth In ##ju ##ries [ Disease / Finding ] ; Un ##sp ##ec ##ified birth trauma ; birth ##ing trauma ; B ##IR ##TH IN ##J ; IN ##J B ##IR ##TH ; Birth injuries ; birth ; injury ; [SEP]',
  'our_bi_cui': 'C0010957',
  'our_bi_cui_name': 'Tissue damage',
  'our_bi_cui_desc': '[CLS] T ##iss ##ue damage [unused3] ( In ##ju ##ry or Po ##ison ##ing , m ##or ##ph ##olo ##gic abnormal ##ity : damage tissues ; Dam ##age ; damages tissue ; Dam ##age , T ##iss ##ue ; tissue damage ; T ##iss ##ue Dam ##age ; damage ; damages ) [SEP]',
  'our_cross_cui': 'C0178314',
  'our_cross_cui_name': 'Poisoning / injury',
  'our_cross_cui_desc': '[CLS] Po ##ison ##ing / injury [unused3] ( In ##ju ##ry or Po ##ison ##ing , disorder , finding , navigation ##al concept : In ##ju ##ry & / or poisoning ; IN ##J ##UR ##Y AND P ##O ##IS ##ON ##ING ; In ##ju ##ry and poisoning ; In ##ju ##ry / poisoning ) [SEP]'},
 {'mention_id': '27669217.46',
  'test_set_idx': 13697,
  'type': 'T017',
  'name': 'ermB',
  'context': '[CLS] ##ed with T ##n ##9 ##16 and IS ##12 ##16 , respectively . The expression of both er ##m ##B and er ##m ##T in all is ##olate ##s was er ##yt ##hr ##omy ##cin in ##du ##cible and yielded comparable mac ##rol ##ide MI ##Cs in all six is ##olate ##s . Take ##n together , in ##du ##cible expression of both [unused1] er ##m ##B [unused2] and er ##m ##T conferred high mac ##rol ##ide resistance in these S . gal ##lo ##ly ##ticus subsp . paste ##rian ##us is ##olate ##s . Our findings reveal new mac ##rol ##ide resistance features in S . gal ##lo ##ly ##ticus subsp . paste ##uria ##nus by both er ##m ##B and er ##m ##T . [SEP]',
  'existing_cui': 'C0732611',
  'existing_cui_name': 'Selective Estrogen Receptor Modulators',
  'existing_cui_desc': '[CLS] Select ##ive E ##stro ##gen Re ##ceptor Mo ##du ##lator ##s [unused3] ( Chemical , SE ##R ##M , medication , product , substance : E ##ST ##RO ##GE ##N R ##EC ##EP ##T M ##OD ##U ##LA ##TO ##RS SE ##LE ##CT ##IVE ; Select ##ive E ##stro ##gen Re ##ceptor Mo ##di ##fying Agents ; Select ##ive est ##rogen receptor m ##od ##ulator ##s ; Select ##ive o ##est ##rogen receptor m ##od ##ulator ; E ##stro ##gen Re ##ceptor Mo ##du ##lator , Select ##ive ; SE ##R ##M - Select ##ive est ##rogen receptor m ##od ##ulator ; SE ##R ##Ms ; SE ##R ##M - Select ##ive o ##est ##rogen receptor m ##od ##ulator ; SE ##LE ##CT ##IVE E ##ST [SEP]',
  'our_bi_cui': 'C0243036',
  'our_bi_cui_name': 'Genes , erbB',
  'our_bi_cui_desc': '[CLS] Gene ##s , er ##b ##B [unused3] ( Ana ##tom ##ical Structure : A ##vian E ##ry ##th ##ro ##blast ##osis V ##ir ##us On ##co ##gene B ; E ##ry ##th ##ro ##blast Trans ##form ##ing Gene B ; On ##co ##gene , E ##G ##F ##R ; On ##co ##gene ER ##B B ; er ##b ##B Gene ##s ; er ##b ##B Gene ; ER ##B - B ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27659602.7',
  'test_set_idx': 13352,
  'type': 'T082',
  'name': 'root',
  'context': '[CLS] The A ##ort ##ic Root : Natural History After Root - Spa ##ring As ##cend ##ing Rep ##lace ##ment in Non ##sy ##nd ##rom ##ic An ##eur ##ys ##mal Pat ##ients Leaving native a ##ort ##ic tissue in sit ##u in root - spa ##ring ascending a ##ort ##ic replacement raises concern regarding potential later need for [unused1] root [unused2] re ##oper ##ation or for the potential occurrence of localized di ##sse ##ctions or r ##up ##ture in the residual root . The purpose of this study was to evaluate the natural growth of the a ##ort ##ic root after root - spa ##ring a ##ort ##ic replacement . In all , 102 consecutive patients ( mean age 61 . 8 ± 12 . 5 years [SEP]',
  'existing_cui': 'C1318154',
  'existing_cui_name': 'Root body part',
  'existing_cui_desc': '[CLS] Root body part [unused3] ( Ana ##tom ##ical Structure : Root ; root [ a body part ] ; root ) [SEP]',
  'our_bi_cui': 'C0040452',
  'our_bi_cui_name': 'Tooth root structure',
  'our_bi_cui_desc': '[CLS] Too ##th root structure [unused3] ( Ana ##tom ##ical Structure , body structure : Ra ##di ##x co ##rona ; Root of the Too ##th ; tooth root ; Too ##th Root ; Root of Too ##th ; Root , Too ##th ; tooth roots ; root of tooth ; root tooth ; roots tooth ; Te ##eth - - Roots ; Roots , Too ##th ; of tooth root ; Too ##th Roots ; Too ##th root ; Ra ##di ##x Den ##tis ; Ra ##di ##x den ##tis ; rooted tooth ; Root of tooth ) [SEP]',
  'our_cross_cui': 'C0549113',
  'our_cross_cui_name': 'Supraaortic valve area structure',
  'our_cross_cui_desc': '[CLS] Su ##pra ##ao ##rt ##ic valve area structure [unused3] ( Spa ##tial Concept , body structure : A ##ort ##ic bulb ; AS ##CE ##ND ##ING A ##OR ##TA , A ##OR ##TI ##C R ##O ##OT ; A ##ort ##a . root ; A ##ort ##ic root ; a ##ort ##ic root ; A ##ort ##ic Root ; Root of a ##ort ##a ; B ##ul ##b of a ##ort ##a ; Su ##pra ##ao ##rt ##ic valve area ; root a ##ort ##ic ; B ##ul ##b of ascending a ##ort ##a ; B ##ul ##bus a ##ort ##ae ) [SEP]'},
 {'mention_id': '27669217.2',
  'test_set_idx': 13653,
  'type': 'T017',
  'name': 'ermB',
  'context': '[CLS] In ##du ##cible Express ##ion of both [unused1] er ##m ##B [unused2] and er ##m ##T Con ##ferred High Mac ##rol ##ide Resistance in St ##re ##pt ##oc ##oc ##cus gal ##lo ##ly ##ticus subsp . paste ##uria ##nus Is ##olate ##s in China St ##re ##pt ##oc ##oc ##cus gal ##lo ##ly ##ticus subsp . paste ##uria ##nus is an under - recognized path ##ogen and zoo ##not ##ic agent causing op ##port ##uni ##stic infections in humans . Despite increasing recognition of this subspecies as a cause for human infectious diseases , limited information is known about its anti ##biotic resistance mechanism . In this study , we aim to identify the molecular mechanism underlying the high mac ##rol ##ide resistance of six S [SEP]',
  'existing_cui': 'C0732611',
  'existing_cui_name': 'Selective Estrogen Receptor Modulators',
  'existing_cui_desc': '[CLS] Select ##ive E ##stro ##gen Re ##ceptor Mo ##du ##lator ##s [unused3] ( Chemical , SE ##R ##M , medication , product , substance : E ##ST ##RO ##GE ##N R ##EC ##EP ##T M ##OD ##U ##LA ##TO ##RS SE ##LE ##CT ##IVE ; Select ##ive E ##stro ##gen Re ##ceptor Mo ##di ##fying Agents ; Select ##ive est ##rogen receptor m ##od ##ulator ##s ; Select ##ive o ##est ##rogen receptor m ##od ##ulator ; E ##stro ##gen Re ##ceptor Mo ##du ##lator , Select ##ive ; SE ##R ##M - Select ##ive est ##rogen receptor m ##od ##ulator ; SE ##R ##Ms ; SE ##R ##M - Select ##ive o ##est ##rogen receptor m ##od ##ulator ; SE ##LE ##CT ##IVE E ##ST [SEP]',
  'our_bi_cui': 'C0243036',
  'our_bi_cui_name': 'Genes , erbB',
  'our_bi_cui_desc': '[CLS] Gene ##s , er ##b ##B [unused3] ( Ana ##tom ##ical Structure : A ##vian E ##ry ##th ##ro ##blast ##osis V ##ir ##us On ##co ##gene B ; E ##ry ##th ##ro ##blast Trans ##form ##ing Gene B ; On ##co ##gene , E ##G ##F ##R ; On ##co ##gene ER ##B B ; er ##b ##B Gene ##s ; er ##b ##B Gene ; ER ##B - B ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27472389.55',
  'test_set_idx': 7196,
  'type': 'T017',
  'name': 'MDA-MB231',
  'context': '[CLS] act ##iva ##tor of transcription 1 , and H ##ER ##2 were increased and re ##lap ##se free survival was decreased in high - risk breast cancer patients . We also investigated the effect of ACT ##A ##2 on cell m ##ot ##ility , which was suppressed by ACT ##A ##2 s ##h ##RNA over ##ex ##press ##ion in [unused1] MD ##A - MB ##23 ##1 [unused2] H ##ER ##2 and 4 ##T ##1 ma ##mma ##ry car ##cin ##oma cells . The number of lung meta ##static nod ##ules was significantly decreased in ACT ##A ##2 knock ##down mice . Take ##n together , these results demonstrated that induction of ACT ##A ##2 by E ##G ##F ##R and H ##ER ##2 dim ##eri ##zation [SEP]',
  'existing_cui': 'C1537399',
  'existing_cui_name': 'IFIH1 gene',
  'existing_cui_desc': '[CLS] IF ##I ##H1 gene [unused3] ( Ana ##tom ##ical Structure : IN ##TE ##R ##F ##ER ##ON - IN ##D ##UC ##ED H ##EL ##IC ##AS ##E C D ##OM ##A ##IN - CO ##NT ##A ##IN ##ING PR ##OT ##EI ##N 1 ; ID ##DM ##19 ; me ##lan ##oma differentiation - associated gene 5 ; IF ##I ##H1 Gene ; Inter ##fer ##on In ##duced With He ##lica ##se C Domain 1 Gene ; he ##lica ##rd ; inter ##fer ##on induced with he ##lica ##se C domain 1 ; H ##l ##c ##d ; MD ##A - 5 ; ME ##LA ##N ##OM ##A D ##IF ##F ##ER ##EN ##TI ##AT ##ION - AS ##SO ##CI ##AT ##ED GE ##NE 5 ) [SEP]',
  'our_bi_cui': 'C3898556',
  'our_bi_cui_name': 'MDA-MB-231 / ATCC',
  'our_bi_cui_desc': '[CLS] MD ##A - MB - 231 / AT ##CC [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_cross_cui': 'C1512505',
  'our_cross_cui_name': 'Breast Cancer Cell',
  'our_cross_cui_desc': '[CLS] B ##rea ##st Cancer Cell [unused3] ( Ana ##tom ##ical Structure : Human B ##rea ##st Cancer Cell ; Human B ##rea ##st Cancer Cell ##s ) [SEP]'},
 {'mention_id': '27838742.78',
  'test_set_idx': 19045,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . Del ##ica ##f ##lav ##one may represent a potential therapeutic agent for lung cancer . [unused1] Del ##ica ##f ##lav ##one [unused2] showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . Del ##ica ##f ##lav ##one induced auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one did not show o ##bs ##er ##vable side effects in a x ##eno [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27799834.1',
  'test_set_idx': 17615,
  'type': 'T058',
  'name': 'revision arthroscopic repair',
  'context': '[CLS] A comparison of functional outcomes in patients undergoing [unused1] revision art ##hr ##os ##copic repair [unused2] of massive rot ##ator cu ##ff tears with and without art ##hr ##os ##copic su ##pra ##sca ##pu ##lar nerve release This study was designed to compare functional outcomes in patients undergoing revision repair of massive rot ##ator cu ##ff tears ( re ##tracted medial to the g ##len ##oid ) with Go ##uta ##llie ##r Grade 4 at ##rop ##hy and con ##com ##ita ##nt release of the su ##pra ##sca ##pu ##lar nerve to a similar group of patients with Grade 3 at ##rop ##hy undergoing revision rot ##ator cu ##ff repair without nerve release . We h ##y ##pot ##hes ##ized that patients undergoing nerve release would [SEP]',
  'existing_cui': 'C0408075',
  'existing_cui_name': 'Revision arthroscopic ligament repair',
  'existing_cui_desc': '[CLS] Rev ##ision art ##hr ##os ##copic l ##iga ##ment repair [unused3] ( Health Care Act ##ivity , procedure ) [SEP]',
  'our_bi_cui': 'C0087111',
  'our_bi_cui_name': 'Therapeutic procedure',
  'our_bi_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]',
  'our_cross_cui': 'C1997575',
  'our_cross_cui_name': 'Revision of repair of rotator cuff',
  'our_cross_cui_desc': '[CLS] Rev ##ision of repair of rot ##ator cu ##ff [unused3] ( Health Care Act ##ivity , procedure ) [SEP]'},
 {'mention_id': '27729085.9',
  'test_set_idx': 15215,
  'type': 'T038',
  'name': 'conventional',
  'context': '[CLS] Cho ##rdo ##ma de ##different ##iation after pro ##ton beam therapy : a case report and review of the literature Cho ##rdo ##ma is a rare invasive bone tumor that may occur anywhere along the ne ##ura ##xi ##s . A total of three primary his ##to ##logical varieties have been identified : [unused1] conventional [unused2] , ch ##ond ##roid , and de ##different ##iated . We report a case of an 8 - year - old white girl who presented with conventional chord ##oma , was treated with surgical re ##section and mixed pro ##ton and photo ##n beam therapy , and had a re ##cu ##rrence in the re ##section cavity 2 . 5 years later with de ##different ##iated morphology . The re [SEP]',
  'existing_cui': 'C2945704',
  'existing_cui_name': 'Conventional Treatment',
  'existing_cui_desc': '[CLS] Convention ##al Treatment [unused3] ( Health Care Act ##ivity : conventional therapy ; Convention ##al Therapy ; conventional treatment ) [SEP]',
  'our_bi_cui': 'C4055182',
  'our_bi_cui_name': 'Conventional Schwannoma',
  'our_bi_cui_desc': '[CLS] Convention ##al Sc ##hwa ##nn ##oma [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_cross_cui': 'C0008487',
  'our_cross_cui_name': 'Chordoma',
  'our_cross_cui_desc': '[CLS] Cho ##rdo ##ma [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , m ##or ##ph ##olo ##gic abnormal ##ity : chord ##oma ; CH ##OR ##D ##OM ##A , MA ##L ##IG ##NA ##NT ; CH ##OR ##D ##OM ##A , S ##US ##CE ##P ##TI ##BI ##L ##IT ##Y TO ; chord ##oma ##s ; Cho ##rdo ##ma [ Disease / Finding ] ; Cho ##rdo ##mas ; not ##och ##ord ##oma ) [SEP]'},
 {'mention_id': '28436493.10',
  'test_set_idx': 35776,
  'type': 'T103',
  'name': 'H56',
  'context': '[CLS] in E ##S ##X - 1 secret ##ion induce ##s minimal host immune responses and path ##ology Although My ##co ##ba ##cter ##ium tuberculosis D ##K ##9 ##8 ##9 ##7 is an at ##ten ##uated strain , it was isolated from a patient with extra ##pu ##lm ##ona ##ry tuberculosis and v ##ac ##cin ##ation with a subunit vaccine ( [unused1] H ##5 ##6 [unused2] ) induced poor protection against it . Both at ##ten ##uation and lack of protection are because My ##co ##ba ##cter ##ium tuberculosis D ##K ##9 ##8 ##9 ##7 can not secret ##e the E ##s ##x ##A v ##ir ##ule ##nce factor nor induce a host response against it . Gen ##ome se ##quencing identified a frames ##hi ##ft mutation [SEP]',
  'existing_cui': 'C3612799',
  'existing_cui_name': 'Bacillus sp . H56',
  'existing_cui_desc': '[CLS] Ba ##ci ##llus s ##p . H ##5 ##6 [unused3] ( Ba ##cter ##ium ) [SEP]',
  'our_bi_cui': 'C0042210',
  'our_bi_cui_name': 'Vaccines',
  'our_bi_cui_desc': '[CLS] V ##ac ##cine ##s [unused3] ( Chemical , medication , product , substance : VA ##CC ##IN ##ES ; V ##ac ##cine ; vaccine product ; [ I ##M ##100 ] VA ##CC ##IN ##ES ; V ##ac ##cine ##s [ Chemical / In ##g ##red ##ient ] ; vaccine ##s ; V ##ac ##cine ##E ##nti ##ty ##T ##ype ; vaccine ) [SEP]',
  'our_cross_cui': 'C0887892',
  'our_cross_cui_name': 'Vaccines , Subunit',
  'our_cross_cui_desc': '[CLS] V ##ac ##cine ##s , Sub ##uni ##t [unused3] ( Chemical : Sub ##uni ##t V ##ac ##cine ##s ; subunit vaccine ; subunit vaccine ##s ; V ##ac ##cine ##s , Sub ##uni ##t [ Chemical / In ##g ##red ##ient ] ) [SEP]'},
 {'mention_id': '27659602.12',
  'test_set_idx': 13357,
  'type': 'T082',
  'name': 'root',
  'context': '[CLS] ##lace ##ment in Non ##sy ##nd ##rom ##ic An ##eur ##ys ##mal Pat ##ients Leaving native a ##ort ##ic tissue in sit ##u in root - spa ##ring ascending a ##ort ##ic replacement raises concern regarding potential later need for root re ##oper ##ation or for the potential occurrence of localized di ##sse ##ctions or r ##up ##ture in the residual [unused1] root [unused2] . The purpose of this study was to evaluate the natural growth of the a ##ort ##ic root after root - spa ##ring a ##ort ##ic replacement . In all , 102 consecutive patients ( mean age 61 . 8 ± 12 . 5 years ; 60 % male ) who had undergone root - spa ##ring a ##ort ##ic replacement had [SEP]',
  'existing_cui': 'C1318154',
  'existing_cui_name': 'Root body part',
  'existing_cui_desc': '[CLS] Root body part [unused3] ( Ana ##tom ##ical Structure : Root ; root [ a body part ] ; root ) [SEP]',
  'our_bi_cui': 'C0040452',
  'our_bi_cui_name': 'Tooth root structure',
  'our_bi_cui_desc': '[CLS] Too ##th root structure [unused3] ( Ana ##tom ##ical Structure , body structure : Ra ##di ##x co ##rona ; Root of the Too ##th ; tooth root ; Too ##th Root ; Root of Too ##th ; Root , Too ##th ; tooth roots ; root of tooth ; root tooth ; roots tooth ; Te ##eth - - Roots ; Roots , Too ##th ; of tooth root ; Too ##th Roots ; Too ##th root ; Ra ##di ##x Den ##tis ; Ra ##di ##x den ##tis ; rooted tooth ; Root of tooth ) [SEP]',
  'our_cross_cui': 'C0549113',
  'our_cross_cui_name': 'Supraaortic valve area structure',
  'our_cross_cui_desc': '[CLS] Su ##pra ##ao ##rt ##ic valve area structure [unused3] ( Spa ##tial Concept , body structure : A ##ort ##ic bulb ; AS ##CE ##ND ##ING A ##OR ##TA , A ##OR ##TI ##C R ##O ##OT ; A ##ort ##a . root ; A ##ort ##ic root ; a ##ort ##ic root ; A ##ort ##ic Root ; Root of a ##ort ##a ; B ##ul ##b of a ##ort ##a ; Su ##pra ##ao ##rt ##ic valve area ; root a ##ort ##ic ; B ##ul ##b of ascending a ##ort ##a ; B ##ul ##bus a ##ort ##ae ) [SEP]'},
 {'mention_id': '28197156.9',
  'test_set_idx': 28079,
  'type': 'T204',
  'name': 'rice cultivars',
  'context': '[CLS] , St ##oma ##tal Con ##duct ##ance and Photo ##sy ##nt ##hesis in Rice Inc ##rea ##sing the rate of leaf photos ##ynth ##esis is one important approach for increasing grain yield in rice ( Or ##y ##za sat ##iva ) . Ex ##p ##lo ##iting the natural variation in CO ##2 ass ##im ##ilation rate ( A ) between [unused1] rice cult ##iva ##rs [unused2] using quantitative genetics is one promising means to identify genes contributing to higher photos ##ynth ##esis . In this study , we determined precise location of Carbon As ##si ##mi ##lation Rate 8 by crossing a high - yielding in ##dic ##a cult ##iva ##r with a Japanese commercial cult ##iva ##r . Fine mapping suggested that Carbon As ##si [SEP]',
  'existing_cui': 'C3385047',
  'existing_cui_name': 'Brassia hybrid cultivar',
  'existing_cui_desc': '[CLS] Brass ##ia hybrid cult ##iva ##r [unused3] ( E ##uka ##ryo ##te : Ada hybrid cult ##iva ##r ) [SEP]',
  'our_bi_cui': 'C0035567',
  'our_bi_cui_name': 'Rice',
  'our_bi_cui_desc': '[CLS] Rice [unused3] ( Food , Diet ##ary , Diet ##ary , substance : Rice ##s ; rice ) [SEP]',
  'our_cross_cui': 'C1140671',
  'our_cross_cui_name': 'Oryza sativa',
  'our_cross_cui_desc': '[CLS] Or ##y ##za sat ##iva [unused3] ( E ##uka ##ryo ##te : red rice ; rice ; Or ##y ##za sat ##iva L . ; rice plant ) [SEP]'},
 {'mention_id': '27887026.36',
  'test_set_idx': 20821,
  'type': 'T038',
  'name': 'hydride transfer',
  'context': '[CLS] of HC ##O ##3 ( - ) reduced the reaction rate . Can ##di ##da met ##hyl ##ica ( C ##m ##F ##D ##H ) and Cha ##eto ##mium the ##rm ##op ##hil ##um was modeled in the presence of HC ##O ##3 ( - ) showing that it fits to the active site . The active site setting for [unused1] h ##yd ##ride transfer [unused2] in CO ##2 reduction was modeled . The h ##yd ##ride donated by N ##AD ##H would form a favorable contact to the carbon atom of HC ##O ##3 ( - ) , resulting in a surplus of electrons within the molecule . This would cause the complex formed by hydrogen carbon ##ate and the h ##yd ##ride to break [SEP]',
  'existing_cui': 'C0728827',
  'existing_cui_name': 'transfers',
  'existing_cui_desc': '[CLS] transfers [unused3] ( Finding , finding : Does transfer ; Transfer ##s ) [SEP]',
  'our_bi_cui': 'C1159575',
  'our_bi_cui_name': 'molecular hydrogen transport',
  'our_bi_cui_desc': '[CLS] molecular hydrogen transport [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_cross_cui': 'C1148560',
  'our_cross_cui_name': 'molecular_function',
  'our_cross_cui_desc': '[CLS] molecular _ function [unused3] ( B ##iol ##og ##ic Fun ##ction : molecular function ) [SEP]'},
 {'mention_id': '27779321.3',
  'test_set_idx': 16411,
  'type': 'T170',
  'name': 'equilibrium partitioning',
  'context': '[CLS] Ben ##thi ##c injury dose - response models for p ##oly ##ch ##lor ##inated bi ##phe ##ny ##l - contaminated sediment using [unused1] equilibrium partition ##ing [unused2] The study goal was to develop a sediment p ##oly ##ch ##lor ##inated bi ##phe ##ny ##l dose - response model based on bent ##hic in ##vert ##eb ##rate effects to PC ##B ##s . The authors used an equilibrium partition ##ing approach to generate predicted p ##oly ##ch ##lor ##inated bi ##phe ##ny ##l sediment effect concentrations ( largely A ##ro ##c ##lor 125 ##4 ) associated with a gradient of toxic effects in bent ##hic organisms from effects observed in aquatic toxicity studies . The present study differs from all other equilibrium partition ##ing collective sediment investigations [SEP]',
  'existing_cui': 'C0014653',
  'existing_cui_name': 'Equilibrium',
  'existing_cui_desc': '[CLS] E ##qui ##li ##bri ##um [unused3] ( B ##iol ##og ##ic Fun ##ction , Motor Process ##es , function , o ##bs ##er ##vable entity , qualifier value : balancing ; equilibrium ; Ba ##lance ; E ##qui ##li ##bri ##um , function ; Ba ##lancing ; balance ; Ba ##lance , function ) [SEP]',
  'our_bi_cui': 'C0684295',
  'our_bi_cui_name': 'Chemical extraction',
  'our_bi_cui_desc': '[CLS] Chemical extraction [unused3] ( Health Care Act ##ivity ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28521168.15',
  'test_set_idx': 38284,
  'type': 'T058',
  'name': 'test battery',
  'context': '[CLS] ##no ##mate ##rial ##s , specifically , Nan ##o - Go ##eth ##ite , T ##rap - O ##x Fe - z ##eo ##lite ##s , Car ##bo - Iron ( ® ) and Fe ##r ##ME ##G ##12 , developed within the European F ##P ##7 project Nan ##o ##R ##em for sub - surface re ##media ##tion towards a [unused1] test battery [unused2] consisting of eight e ##cot ##ox ##icit ##y tests on bacteria ( V . fish ##eri , E . co ##li ) , algae ( P . sub ##cap ##ita ##ta , Ch ##lam ##yd ##omo ##nas s ##p . ) , crust ##ace ##ans ( D . ma ##gna ) , worms ( E . f ##eti ##da , L [SEP]',
  'existing_cui': 'C0456724',
  'existing_cui_name': 'Willeford test battery',
  'existing_cui_desc': '[CLS] Will ##ef ##ord test battery [unused3] ( Health Care Act ##ivity , procedure ) [SEP]',
  'our_bi_cui': 'C1446038',
  'our_bi_cui_name': 'Microbiology battery test',
  'our_bi_cui_desc': '[CLS] Micro ##biology battery test [unused3] ( Health Care Act ##ivity , procedure : Micro ##biology test battery ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '28229061.42',
  'test_set_idx': 29008,
  'type': 'T017',
  'name': 'bucco - palatal',
  'context': '[CLS] for nine ( 56 . 3 % ) cases and the ma ##xi ##lla for six ( 37 . 5 % ) cases , while a combination of the ma ##xi ##lla and the z ##y ##go ##ma were involved in one ( 6 . 3 % ) case . B ##uc ##co - l ##ing ##ual or [unused1] b ##uc ##co - p ##ala ##tal [unused2] expansion were the most common presentation ( six [ 46 . 2 % ] cases each ) . His ##to ##logical assessment of tissue specimens showed that fi ##bro ##my ##x ##oma accounted for seven ( 43 . 8 % ) cases , while the remaining nine ( 56 . 3 % ) cases were diagnosed as my ##x [SEP]',
  'existing_cui': 'C1065206',
  'existing_cui_name': 'Genus Bucco',
  'existing_cui_desc': '[CLS] Genus B ##uc ##co [unused3] ( E ##uka ##ryo ##te , organism : b ##uc ##co ; B ##uc ##co ) [SEP]',
  'our_bi_cui': 'C0565973',
  'our_bi_cui_name': 'Buccal-incisal-palatal',
  'our_bi_cui_desc': '[CLS] B ##uc ##cal - in ##cis ##al - p ##ala ##tal [unused3] ( Spa ##tial Concept , body structure ) [SEP]',
  'our_cross_cui': 'C0229962',
  'our_cross_cui_name': 'Body part',
  'our_cross_cui_desc': '[CLS] Body part [unused3] ( Ana ##tom ##ical Structure , body structure : Body Parts ; Body part structure ; Pa ##rs co ##rp ##oris ; body part ; body parts ; Body region ; Ana ##tom ##ic part . * ; Body Structure ; Parts , Body ; Ana ##tom ##ic part ; body region ; Cardinal body part ; Body Part ) [SEP]'},
 {'mention_id': '27251800.47',
  'test_set_idx': 842,
  'type': 'T058',
  'name': 'Stoppa',
  'context': '[CLS] % of Stop ##pa group ( 22 patients ) and 64 % in lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair group ( 16 patients ) ( P < 0 . 05 ) . The comparative study between the Stop ##pa and lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair approaches for the bilateral ing ##uin ##al her ##nia repair demonstrated that : ( 1 ) The lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair approach showed less surgical trauma despite the longer operation time ; ( 2 ) Quality of life during the early post - operative period were similar ; and ( 3 ) Co ##mp ##lication rates were higher in the [unused1] Stop ##pa [unused2] group . [SEP]',
  'existing_cui': 'C1538901',
  'existing_cui_name': 'ACD gene',
  'existing_cui_desc': '[CLS] AC ##D gene [unused3] ( Ana ##tom ##ical Structure : T ##EL ##OM ##ER ##E PR ##OT ##EI ##N T ##PP ##1 ; ad ##ren ##oc ##ort ##ical d ##ys ##p ##lasia ho ##mo ##log ; AC ##D Gene ; Ad ##ren ##oc ##ort ##ical D ##ys ##p ##lasia Ho ##mo ##log ( Mouse ) Gene ; P ##OT ##1 - IN ##TE ##RA ##CT ##ING PR ##OT ##EI ##N 1 ; P ##OT ##1 - AND T ##IN ##2 - OR ##GA ##NI ##Z ##ING PR ##OT ##EI ##N ; P ##OT ##1 and T ##IN ##2 organizing protein ; AC ##D , M ##O ##US ##E , H ##OM ##OL ##O ##G OF ; Tin ##t ##1 ; T ##IN ##2 interacting protein 1 [SEP]',
  'our_bi_cui': 'C1080020',
  'our_bi_cui_name': 'Lopa',
  'our_bi_cui_desc': '[CLS] Lo ##pa [unused3] ( E ##uka ##ryo ##te : lo ##pa ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27791169.33',
  'test_set_idx': 16886,
  'type': 'T017',
  'name': 'BPH1',
  'context': '[CLS] sa ##lic ##yl ##ic acid - and j ##as ##mon ##ic acid - signaling pathways in rice plants and con ##fers both anti ##xen ##osis and anti ##bio ##sis to Brown plant ##hop ##per . We further demonstrated that the eight Brown plant ##hop ##per - resistance genes that are cluster ##ed on chromosome 12 ##L , including the widely used [unused1] BP ##H1 [unused2] , are all ##eli ##c with each other . To honor the priority in the literature , we thus designated this lo ##cus as BP ##H1 / 9 These eight genes can be classified into four all ##elo ##type ##s , BP ##H1 / 9 - 1 , - 2 , - 7 , and - 9 These all ##elo ##type [SEP]',
  'existing_cui': 'C0005001',
  'existing_cui_name': 'Benign prostatic hypertrophy',
  'existing_cui_desc': '[CLS] Ben ##ign pro ##static h ##yper ##tro ##phy [unused3] ( B ##iol ##og ##ic Fun ##ction , BP ##H , ben ##ign , ben ##ign pro ##static h ##yper ##tro ##phy , diagnosis : H ##yper ##tro ##phy - pro ##st . ben . ; ben ##ign pro ##static h ##yper ##tro ##phy ; Pro ##static h ##yper - ben ##ign ; Ben ##ign Pro ##static H ##yper ##p ##lasia - BP ##H ; En ##lar ##ged Pro ##state ; H ##yper ##tro ##phy , Ben ##ign Pro ##static ; H ##yper ##tro ##phy ( ben ##ign ) of pro ##state ; Ben ##ign h ##yper ##tro ##phy of pro ##state ; BP ##H - ben ##ign pro ##static h ##yper ##tro ##phy ; Pro ##static [SEP]',
  'our_bi_cui': 'C2677154',
  'our_bi_cui_name': 'BSPH1 gene',
  'our_bi_cui_desc': '[CLS] BS ##P ##H1 gene [unused3] ( Ana ##tom ##ical Structure : bind ##er of sperm protein ho ##mo ##log 1 ; BS ##P , B ##O ##VI ##NE , H ##OM ##OL ##O ##G OF , 1 ; B ##IN ##DE ##R OF SP ##ER ##M PR ##OT ##EI ##N , B ##O ##VI ##NE , H ##OM ##OL ##O ##G OF , 1 ; E ##LS ##P ##B ##P ##2 ; b ##ov ##ine seminal plasma protein ho ##mo ##log 1 ; B ##O ##VI ##NE SE ##MI ##NA ##L P ##LA ##SM ##A PR ##OT ##EI ##N - L ##I ##KE 1 ; SE ##MI ##NA ##L P ##LA ##SM ##A PR ##OT ##EI ##N , B ##O ##VI ##NE , H ##OM ##OL [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '28243201.17',
  'test_set_idx': 29659,
  'type': 'T201',
  'name': 'response',
  'context': '[CLS] ##cu ##rrence . DNA met ##hyl ##ation has been suggested as one of the underlying mechanisms for re ##current disease and its contribution to the development of drug resistance remains to be clarified . This study aimed to determine the met ##hyl ##ation p ##hen ##otype in color ##ec ##tal cancer for identification of predict ##ive markers for ch ##em ##otherapy [unused1] response [unused2] . We performed DNA met ##hyl ##ation pro ##fi ##ling on 43 non - re ##current and five re ##current color ##ec ##tal cancer patients using the Il ##lum ##ina In ##fin ##ium Human ##M ##eth ##yla ##tion ##45 ##0 Be ##ad ##chi ##p ass ##ay . In addition , color ##ec ##tal cancer cells with different genetic backgrounds , response to [SEP]',
  'existing_cui': 'C1704632',
  'existing_cui_name': 'Disease Response',
  'existing_cui_desc': '[CLS] Disease Response [unused3] ( Finding : Response ; response ) [SEP]',
  'our_bi_cui': 'C0683154',
  'our_bi_cui_name': 'drug response',
  'our_bi_cui_desc': '[CLS] drug response [unused3] ( B ##iol ##og ##ic Fun ##ction : response to drug ; drug responses ; drugs response ) [SEP]',
  'our_cross_cui': 'C0521982',
  'our_cross_cui_name': 'Response to treatment',
  'our_cross_cui_desc': '[CLS] Response to treatment [unused3] ( Clinical At ##tri ##but ##e , context - dependent category , finding , function , o ##bs ##er ##vable entity , situation : response to treatment ; response treatments ; The ##rap ##eu ##tic response ; response therapeutic ; The ##rap ##eu ##tic Response ; Finding related to therapeutic response ; successful treatment ; successful treatments ; The ##rap ##eu ##tic response , function ) [SEP]'},
 {'mention_id': '28273336.4',
  'test_set_idx': 30871,
  'type': 'T170',
  'name': 'ImmunoCAP system',
  'context': '[CLS] Question ##able diagnostic benefit of the commercially available panel of bee venom components For many years , only the major all ##er ##gen r ##A ##pi m 1 has been available on the [unused1] I ##mm ##uno ##CA ##P system [unused2] for routine diagnosis of bee venom all ##er ##gy . Now , there are five components available , and we aimed to detect the sensitivity and specific ##ity of r ##A ##pi m 1 , 2 , 3 , 5 , and 10 in B ##V - all ##ergic patients . We further evaluated the sensitivity of r ##A ##pi m 1 and 2 of an alternative platform and investigated possible differences in the se ##ns ##iti ##zation profile between mon ##ose ##ns ##iti ##zation [SEP]',
  'existing_cui': 'C0026745',
  'existing_cui_name': 'System , Multi-Hospital',
  'existing_cui_desc': '[CLS] System , Multi - Hospital [unused3] ( Organization : Multi - Hospital Systems ; Multi Hospital Systems ; M ##U ##LT ##I H ##OS ##P S ##Y ##ST ##EM ##S ; System , Multi Hospital ; M ##U ##LT ##I ##H ##OS ##P S ##Y ##ST ##EM ##S ; Multi ##hos ##pit ##al systems ; M ##U ##LT ##I ##H ##OS ##PI ##TA ##L S ##Y ##ST ##EM ##S ; System , Multi ##hos ##pit ##al ; Multi ##hos ##pit ##al System ; Multi - Hospital System ; Systems , Multi - Hospital ; Multi ##hos ##pit ##al Systems ; S ##Y ##ST ##EM M ##U ##LT ##I H ##OS ##P ; Systems , Multi ##hos ##pit ##al ) [SEP]',
  'our_bi_cui': 'C0025080',
  'our_bi_cui_name': 'Medical Devices',
  'our_bi_cui_desc': '[CLS] Medical Devi ##ces [unused3] ( Medical Devi ##ce , physical device , physical object : Clinical equipment ; B ##io ##medical device ; DE ##VI ##CE ME ##D ; ME ##D DE ##VI ##CE ##S ; ME ##D DE ##VI ##CE ; bio ##medical devices ; Devi ##ces , Medical ; Medical devices ; General bio ##medical device ; DE ##VI ##CE , GE ##NE ##RA ##L ME ##DI ##CA ##L ; B ##io ##medical equipment ; Devi ##ce , Medical ; Hospital , medical AND / OR surgical equipment ( physical device ) ; bio ##medical equipment ; medical equipment ; Medical equipment ; DE ##VI ##CE ##S ME ##D ; DE ##VI ##CE ; DE ##VI ##CE ##S ; medical device ; medical devices [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28299441.47',
  'test_set_idx': 31686,
  'type': 'T017',
  'name': 'early genes',
  'context': '[CLS] V ##v ##M ##Y ##B ##PA ##1 alter ##s the p ##hen ##yl ##p ##rop ##ano ##id pathway in tobacco floral organs , in a structure - specific fashion . We also report that a modest V ##v ##M ##Y ##B ##PA ##1 expression is sufficient to induce the expression of both pro ##ant ##ho ##cy ##ani ##din - specific and [unused1] early genes [unused2] of the p ##hen ##yl ##p ##rop ##ano ##id pathway . Consequently , pro ##ant ##ho ##cy ##ani ##din ##s and ch ##lor ##ogenic acids are induced or de no ##vo s ##ynth ##eti ##sed in floral limbs , tubes and s ##tam ##ens . Other p ##hen ##yl ##p ##rop ##ano ##id branches are con ##verse ##ly induced or depleted [SEP]',
  'existing_cui': 'C0013458',
  'existing_cui_name': 'Early Gene Transcription',
  'existing_cui_desc': '[CLS] Early Gene Trans ##cription [unused3] ( B ##iol ##og ##ic Fun ##ction : Trans ##cription , Early Gene ; Gene Trans ##cription , Early ) [SEP]',
  'our_bi_cui': 'C0013458',
  'our_bi_cui_name': 'Early Gene Transcription',
  'our_bi_cui_desc': '[CLS] Early Gene Trans ##cription [unused3] ( B ##iol ##og ##ic Fun ##ction : Trans ##cription , Early Gene ; Gene Trans ##cription , Early ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27786291.17',
  'test_set_idx': 16751,
  'type': 'T017',
  'name': 'ctxA/B',
  'context': '[CLS] ##s and pan ##de ##mic disease . However , a number of questions remain about long - term survival and evolution of V . ch ##ole ##rae strains within these aquatic environmental reservoirs . Through monitoring of the Haitian aquatic environment following the 2010 ch ##ole ##ra epidemic , we isolated two novel non - to ##xi ##genic ( [unused1] c ##t ##x ##A / B [unused2] - negative ) V ##ib ##rio ch ##ole ##rae O ##1 . These two is ##olate ##s underwent whole - genome se ##quencing and were investigated through comparative g ##eno ##mic ##s and Bay ##esi ##an coal ##es ##cent analysis . These is ##olate ##s cluster in the evolutionary tree with strains responsible for clinical ch ##ole ##ra , [SEP]',
  'existing_cui': 'C0412669',
  'existing_cui_name': 'Computerized tomography , bone density study',
  'existing_cui_desc': '[CLS] Computer ##ized to ##mo ##graphy , bone density study [unused3] ( Health Care Act ##ivity , Q ##CT , procedure : computer ##ized to ##mo ##graphy bone den ##si ##tom ##et ##ry ; CT bone density ; quantitative com ##puted to ##mo ##graphy ; com ##puted to ##mo ##graphy bone density study ; Q ##uant ##itative com ##puted to ##mo ##graphy studies ; bone density studies com ##puted to ##mo ##graphy ; Computer ##ised to ##mo ##graphy , bone density study ; Q ##uant ##itative CT studies ) [SEP]',
  'our_bi_cui': 'C0010868',
  'our_bi_cui_name': 'Cytotoxin',
  'our_bi_cui_desc': '[CLS] Cy ##to ##to ##xin [unused3] ( Chemical , substance : Cy ##to ##to ##xi ##c substance ; c ##yt ##oto ##xi ##c substances ; Cy ##to ##to ##xi ##c Agents ; c ##yt ##oto ##xi ##c substance ; c ##yt ##oto ##xin ; Cy ##to ##to ##xin ##s ; c ##yt ##oto ##xin ##s ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '28197156.32',
  'test_set_idx': 28102,
  'type': 'T017',
  'name': 'DTH8',
  'context': '[CLS] analysis revealed that the in ##dic ##a all ##ele of Carbon As ##si ##mi ##lation Rate 8 has a 1 - b ##p del ##eti ##on at 322 b ##p from the start co ##don , resulting in a t ##runcated protein of 125 amino acids . In addition , Carbon As ##si ##mi ##lation Rate 8 is identical to [unused1] D ##TH ##8 [unused2] / G ##h ##d ##8 / L ##HD ##1 , which was reported to control rice flowering date . The increase of A is largely due to an increase of R ##u ##B ##P regeneration rate via increased leaf nitrogen content , and partially explained by reduced s ##tom ##ata ##l limitation via increased s ##tom ##ata ##l conduct ##ance relative [SEP]',
  'existing_cui': 'C0020522',
  'existing_cui_name': 'Delayed Hypersensitivity',
  'existing_cui_desc': '[CLS] Del ##ay ##ed H ##yper ##sen ##si ##ti ##vity [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : h ##yper ##sen ##si ##ti ##vity type i ##v ; delayed h ##yper ##sen ##si ##ti ##vity reaction ; H ##yper ##sen ##si ##ti ##vi ##ties , Tu ##ber ##cu ##lin - Type ; delayed h ##yper ##sen ##si ##ti ##vity ; Tu ##ber ##cu ##lin Type H ##yper ##sen ##si ##ti ##vity ; Cell - mediated im ##mu ##nological reaction ; Del ##ay ##ed type h ##yper ##sen ##si ##ti ##vity ; type i ##v h ##yper ##sen ##si ##ti ##vity ; Cell - mediated im ##mu ##nological react ; H ##yper ##sen ##si ##ti ##vi ##ties , Type IV ; type IV h ##yper [SEP]',
  'our_bi_cui': 'C1413965',
  'our_bi_cui_name': 'DHX8 gene',
  'our_bi_cui_desc': '[CLS] D ##H ##X ##8 gene [unused3] ( Ana ##tom ##ical Structure : DE ##A ##H B ##OX P ##OL ##YP ##EP ##TI ##DE 8 ; PR ##P ##22 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; DE ##A ##H - box he ##lica ##se 8 ; PR ##P ##22 ; DE ##AD / H B ##OX 8 ; RNA H ##EL ##IC ##AS ##E 1 ; PR ##P ##F ##22 ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '28454015.20',
  'test_set_idx': 36521,
  'type': 'T170',
  'name': 'health - risk based air quality index',
  'context': '[CLS] ) at both 2014 and 2015 . PM ##2 . 5 is typically the major poll ##uta ##nt , but oz ##one pollution can be significant during summer . Furthermore , as the commonly used air quality index neglect ##s the mutual health effects from multiple poll ##uta ##nts , we introduced the aggregate air quality index and [unused1] health - risk based air quality index [unused2] to evaluate the health risks . Results show that based on health - risk based air quality index , the current air quality index system likely significantly under ##est ##imate the health risks of air pollution , highlighting that the general public may need strict ##er health protection measures . The population - weighted two - year average health [SEP]',
  'existing_cui': 'C1268941',
  'existing_cui_name': 'Health risks education',
  'existing_cui_desc': '[CLS] Health risks education [unused3] ( Health Care Act ##ivity , procedure , regime / therapy : Counsel ##ing regarding health risks ; Ad ##vice about health risks ; Counsel ##ling regarding health risks ; Education regarding health risks ) [SEP]',
  'our_bi_cui': 'C3494320',
  'our_bi_cui_name': 'Health Impact Assessment',
  'our_bi_cui_desc': '[CLS] Health Impact Assessment [unused3] ( Health Care Act ##ivity : Impact Assessment ##s , Health ; Impact Assessment , Health ; Health Impact Assessment ##s ; Assessment ##s , Health Impact ; Assessment , Health Impact ) [SEP]',
  'our_cross_cui': 'C0918012',
  'our_cross_cui_name': 'Index',
  'our_cross_cui_desc': '[CLS] Index [unused3] ( Intel ##lect ##ual Product : index ##ed ; index ; index ##es ; Index ##es as Top ##ic ) [SEP]'},
 {'mention_id': '28077286.50',
  'test_set_idx': 25100,
  'type': 'T103',
  'name': 'MeHg',
  'context': '[CLS] Me ##H ##g dose of 3 . 2 μ ##g H ##g g ( - 1 ) egg , and a vehicle control ( water ) . No adverse effects of in o ##vo Me ##H ##g treatment were detected on courts ##hip song quality or on mating behavior in experimental males at sexually maturity which would suggest that o ##bs ##er ##vable ne ##uro ##be ##ha ##vior ##al effects of [unused1] Me ##H ##g [unused2] exposure may depend on the timing of exposure during offspring development . However , ne ##uro ##ana ##tom ##ical analysis indicated an increase in te ##len ##ce ##pha ##lon volume with increased Me ##H ##g concentrations which may suggest a prolonged inflammatory response in this region of the brain . [SEP]',
  'existing_cui': 'C0228520',
  'existing_cui_name': 'Structure of reticular formation of medulla oblongata',
  'existing_cui_desc': '[CLS] Structure of re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata [unused3] ( Ana ##tom ##ical Structure , body structure : bulb ##ar re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla ; B ##ul ##bar re ##tic ##ular formation ; Form ##ati ##o re ##tic ##ular ##is my ##ele ##nce ##pha ##li ; me ##du ##llar ##y re ##tic ##ular formation ; re ##tic ##ular formation of me ##du ##lla ; Me ##du ##llar ##y re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata ) [SEP]',
  'our_bi_cui': 'C0025424',
  'our_bi_cui_name': 'Mercury',
  'our_bi_cui_desc': '[CLS] Mercury [unused3] ( Chemical , H ##g , Metal , elemental , substance : mercury ; Mercury [ Chemical / In ##g ##red ##ient ] ; metal mercury ; H ##yd ##rar ##gy ##rum ; H ##g element ; Li ##quid Silver ; Quick ##si ##lve ##r ; H ##g ; H ##g - Mercury ; h ##g ; ME ##RC ##UR ##Y ) [SEP]',
  'our_cross_cui': 'C0025794',
  'our_cross_cui_name': 'Methylmercury Compounds',
  'our_cross_cui_desc': '[CLS] Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [unused3] ( Chemical : Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [ Chemical / In ##g ##red ##ient ] ; Mercury Co ##mpo ##unds , Met ##hyl ; ME ##TH ##Y ##L ME ##RC ##UR ##Y CP ##DS ; Met ##hyl Mercury Co ##mpo ##unds ; ME ##TH ##Y ##LM ##ER ##CU ##R ##Y CP ##DS ; Co ##mpo ##unds , Met ##hyl ##mer ##cu ##ry ; met ##hyl ##mer ##cu ##ry ; Co ##mpo ##unds , Met ##hyl Mercury ; Met ##hyl ##mer ##cu ##ry ) [SEP]'},
 {'mention_id': '28471040.13',
  'test_set_idx': 37127,
  'type': 'T103',
  'name': 'poly ( lactic-co-glycolic acid ) -1,2 distearoyl-glycerol-3-phospho-ethanolamine-N [ methoxy ( lipid - polyethylene glycol ) -2000 ] ammonium salt',
  'context': '[CLS] drug , far ##nes ##yl ##thi ##osa ##lic ##yl ##ic acid for treatment of g ##lio ##blast ##oma . Far ##nes ##yl ##thi ##osa ##lic ##yl ##ic acid - loaded [unused1] p ##oly ( la ##ctic - co - g ##ly ##co ##lic acid ) - 1 , 2 di ##ste ##aro ##yl - g ##ly ##cer ##ol - 3 - p ##hos ##ph ##o - ethanol ##amine - N [ met ##ho ##xy ( lip ##id - p ##oly ##eth ##yle ##ne g ##ly ##co ##l ) - 2000 ] am ##mon ##ium salt [unused2] ( P ##L ##GA - DS ##P ##E - P ##EG ) with or without 1 , 2 - di ##ole ##oy ##l - 3 - trim ##eth ##yla ##mm [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C3849743',
  'our_bi_cui_name': 'polyethylene glycol-poly',
  'our_bi_cui_desc': '[CLS] p ##oly ##eth ##yle ##ne g ##ly ##co ##l - p ##oly [unused3] ( Chemical , la ##ct ##ide - co - g ##ly ##co ##lide , la ##ct ##ide - co - g ##ly ##co ##lide : P ##EG - L ##G ; P ##EG - P ##L ##GA ; p ##oly ( et ##hyl ##ene g ##ly ##co ##l ) - block - la ##ct ##ide - g ##ly ##co ##lide ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28424683.50',
  'test_set_idx': 35428,
  'type': 'T033',
  'name': 'decline in Natural killer cell numbers',
  'context': '[CLS] of Natural killer cells , whereas Natural killer cells from mice with no tumor continue expanding Natural killer cells and retain their c ##yt ##oto ##xi ##city . In addition , den ##dr ##itic cells in contrast to O ##Cs are found to promote faster expansion of residual T cells within pu ##rified Natural killer cells resulting in the [unused1] decline in Natural killer cell numbers [unused2] from healthy individuals . Ad ##dition of anti - CD ##3 m ##A ##b in ##hibit ##s T cell proliferation while enhancing Natural killer cell expansion ; however , expanding Natural killer cells have lower c ##yt ##oto ##xi ##city but higher secret ##ion of IF ##N - γ . Ex ##pan ##sion and functional activation of super - [SEP]',
  'existing_cui': 'C1855767',
  'existing_cui_name': 'Reduced natural killer cell number',
  'existing_cui_desc': '[CLS] Red ##uce ##d natural killer cell number [unused3] ( Finding : Red ##uce ##d number of natural killer cells ) [SEP]',
  'our_bi_cui': 'C1855767',
  'our_bi_cui_name': 'Reduced natural killer cell number',
  'our_bi_cui_desc': '[CLS] Red ##uce ##d natural killer cell number [unused3] ( Finding : Red ##uce ##d number of natural killer cells ) [SEP]',
  'our_cross_cui': 'C2363838',
  'our_cross_cui_name': 'Natural killer cell count decreased',
  'our_cross_cui_desc': '[CLS] Natural killer cell count decreased [unused3] ( Finding : N ##K cell count decreased ) [SEP]'},
 {'mention_id': '28234635.63',
  'test_set_idx': 29231,
  'type': 'T103',
  'name': 'LXR',
  'context': '[CLS] inter ##le ##uki ##n - 6 and tumor ne ##c ##rosis factor , repression of FX ##R and its canal ##icular bi ##le transport ##er target gene RNA expression , including ABC ##B ##4 and ABC ##B ##11 as well as decreased protein expression of ABC ##B ##11 and ABC ##B ##4 . Furthermore , up ##re ##gu ##lation of [unused1] L ##X ##R [unused2] and ABC ##G ##5 / 8 RNA expression was suppressed in patients with portal inflammation . Current parent ##eral nutrition , increased serum levels of plant s ##tero ##ls s ##ti ##gma ##ster ##ol , a ##ven ##aster ##ol , and sit ##ost ##ero ##l along with serum c ##it ##ru ##llin ##e , a marker of enter ##oc ##yte mass [SEP]',
  'existing_cui': 'C4288814',
  'existing_cui_name': 'Liver X Receptor-Like Family',
  'existing_cui_desc': '[CLS] Live ##r X Re ##ceptor - Like Family [unused3] ( Chemical : L ##X ##R Family ; Nuclear Re ##ceptor Sub ##family 1 Group H ) [SEP]',
  'our_bi_cui': 'C1366515',
  'our_bi_cui_name': 'LEPR gene',
  'our_bi_cui_desc': '[CLS] L ##EP ##R gene [unused3] ( Ana ##tom ##ical Structure : O ##BR ; CD ##29 ##5 ; L ##EP ##R Gene ; Le ##pt ##in Re ##ceptor Gene ; le ##pt ##in receptor ; L ##EP ##TI ##N R ##EC ##EP ##TO ##R ) [SEP]',
  'our_cross_cui': 'C4277606',
  'our_cross_cui_name': 'Liver X Receptors',
  'our_cross_cui_desc': '[CLS] Live ##r X Re ##ceptor ##s [unused3] ( Chemical : O ##xy ##ster ##ol Re ##ceptor ##s , Nuclear ; Live ##r X Re ##ceptor ; Nuclear O ##xy ##ster ##ol Re ##ceptor ##s ; Live ##r X Re ##ceptor ##s [ Chemical / In ##g ##red ##ient ] ) [SEP]'},
 {'mention_id': '27587730.18',
  'test_set_idx': 10757,
  'type': 'T103',
  'name': 'gga-miR-21',
  'context': '[CLS] expressed in chicken pre - ad ##ip ##ocytes , but its role is unclear . The present study investigated the role of g ##ga - mi ##R - 21 in chicken pre - ad ##ip ##oc ##yte proliferation . Cell proliferation ass ##ay and gene expression analysis of pro ##life ##rating cell nuclear anti ##gen showed that the [unused1] g ##ga - mi ##R - 21 [unused2] mimic in ##hibit ##ed pre - ad ##ip ##oc ##yte proliferation . In contrast , the g ##ga - mi ##R - 21 inhibitor enhanced pre - ad ##ip ##oc ##yte proliferation . The subsequent investigation identified K ##rup ##pel - like factor 5 m ##RNA as a target of g ##ga - mi ##R - 21 . The [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C2825314',
  'our_bi_cui_name': 'MicroRNA Gene',
  'our_bi_cui_desc': '[CLS] Micro ##RNA Gene [unused3] ( Ana ##tom ##ical Structure : Micro RNA Gene ; Mi ##RNA Gene ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '28294559.14',
  'test_set_idx': 31470,
  'type': 'T204',
  'name': 'chimeric mice',
  'context': '[CLS] role in liver regeneration after partial he ##pate ##ct ##omy in mice Essential ##s Reg ##ener ##ation role of C - type le ##ct ##in receptor - 2 after 70 % he ##pate ##ct ##omy ( HP ##x ) was investigated . Wild - type or C - type le ##ct ##in receptor - 2 deleted from plate ##lets of [unused1] ch ##ime ##ric mice [unused2] ( fl ##KO ) underwent HP ##x . The liver / body weight ratio was significantly lower in the fl ##KO than in the wild - type . C - type le ##ct ##in receptor - 2 plays an essential role in liver regeneration after HP ##x . Background and aim The aim of the present study was to investigate [SEP]',
  'existing_cui': 'C0676831',
  'existing_cui_name': 'basiliximab',
  'existing_cui_desc': '[CLS] b ##asi ##lix ##ima ##b [unused3] ( Chemical , Si ##mu ##lect , medication , product , substance : Basil ##ix ##ima ##b ; b ##asi ##lix ##ima ##b [ Chemical / In ##g ##red ##ient ] ; ch ##ime ##ric mouse - human anti ##CD ##25 ; I ##g gamma - 1 chain C region ; BA ##SI ##L ##IX ##IM ##AB ) [SEP]',
  'our_bi_cui': 'C0025936',
  'our_bi_cui_name': 'Mice , Transgenic',
  'our_bi_cui_desc': '[CLS] Mi ##ce , Trans ##genic [unused3] ( E ##uka ##ryo ##te : Mouse , Trans ##genic ; mouse trans ##genic ##s ; Trans ##genic Mi ##ce ; mice trans ##genic ; Trans ##genic Mouse ; trans ##genic mice ; Trans ##genic mice ; trans ##genic mouse ) [SEP]',
  'our_cross_cui': 'C0025929',
  'our_cross_cui_name': 'Laboratory mice',
  'our_cross_cui_desc': '[CLS] Laboratory mice [unused3] ( E ##uka ##ryo ##te , organism : Mouse , Laboratory ; Mouse , laboratory ; Laboratory Mi ##ce ; Mi ##ce , Laboratory ; mouse ; mice ; Laboratory Mouse ; Laboratory mouse ; laboratory mouse ) [SEP]'},
 {'mention_id': '27832581.19',
  'test_set_idx': 18729,
  'type': 'T168',
  'name': 'maintenance control diet',
  'context': '[CLS] development depend on the predominant fatty acid type in the diet . In a combined in v ##ivo - in v ##it ##ro b ##ov ##ine model , two groups of cows ( n = 3 in each group ) were fed with three diet ##s consecutive ##ly ( 4 weeks feeding for each ) : ( 1 ) a [unused1] maintenance control diet [unused2] ; ( 2 ) a high - star ##ch diet rich in saturated fat ; and ( 3 ) a high - star ##ch diet rich in o ##me ##ga - 3 un ##sat ##ura ##ted fat . Two feeding sequences were used to test for carry - over effects : Group A was fed control diet , SAT ##1 and [SEP]',
  'existing_cui': 'C2930544',
  'existing_cui_name': 'Low-Calorie Diet',
  'existing_cui_desc': '[CLS] Low - Cal ##ori ##e Diet [unused3] ( Finding , finding : Ki ##loc ##alo ##rie control diet ; Cal ##ori ##e controlled diet ; Diet , Low - Cal ##ori ##e ; ca ##lor ##ic restriction ; ca ##lor ##ic restrictions ; low ca ##lor ##ie diet ; Low - Cal ##ori ##e Diet ##s ; Low - ca ##lor ##ie diet ; Diet ##s , Low - Cal ##ori ##e ; ca ##lor ##ie diet ##s low ; Low ca ##lor ##ie diet ; Low energy diet ; ca ##lor ##ie restricted diet ; Low ca ##lor ##ic diet ; ca ##lor ##ie controlled diet ; Low Cal ##ori ##e Diet ; Cal ##ori ##e restricted diet ) [SEP]',
  'our_bi_cui': 'C0452286',
  'our_bi_cui_name': 'Fat controlled diet',
  'our_bi_cui_desc': '[CLS] Fat controlled diet [unused3] ( Health Care Act ##ivity , finding : Mo ##dified fat diet ; fat controlled diet ) [SEP]',
  'our_cross_cui': 'C0012155',
  'our_cross_cui_name': 'Diet',
  'our_cross_cui_desc': '[CLS] Diet [unused3] ( Food , finding : Diet ##s ; regular diet ? ; Diet ##ary ; diet ; Diet ##ary finding ; diet ##s ) [SEP]'},
 {'mention_id': '27388786.23',
  'test_set_idx': 4446,
  'type': 'T098',
  'name': 'signers',
  'context': "[CLS] during sign production remains unclear . Here we directly contrast lateral ##ization strength of covert versus over ##t signing during p ##hon ##ological and semantic flu ##ency tasks . To address the possibility that hearing native sign ##ers ' elevated lateral ##ization in ##dices were due to performing a task in their less dominant language , here we test deaf native [unused1] sign ##ers [unused2] , whose dominant language is British Sign Language . Sign ##ers were more strongly left lateral ##ized for over ##t than covert sign generation . However , the strength of lateral ##ization was not correlated with the amount of time producing movements of the right hand . Co ##mp ##aris ##ons with previous data from hearing native English speakers suggest stronger [SEP]",
  'existing_cui': 'C0035127',
  'existing_cui_name': 'Cumulative Trauma Disorders',
  'existing_cui_desc': '[CLS] C ##um ##ula ##tive T ##ra ##uma Di ##sor ##ders [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , event : Over ##use S ##yn ##drome ; Rep ##eti ##tive St ##ress In ##ju ##ries ; Di ##sor ##ders , C ##um ##ula ##tive T ##ra ##uma ; RS ##I - Rep ##eti ##tive strain injury ; R ##EP ##ET M ##OT ##ION D ##IS ; repetitive strain injury ; St ##rain In ##ju ##ry , Rep ##eti ##tive ; T ##ra ##uma Di ##sor ##ders , C ##um ##ula ##tive ; Rep ##eti ##tive Motion Di ##sor ##ders ; In ##ju ##ries , Rep ##eti ##tive St ##ress ; Motion Di ##sor ##ders , Rep ##eti ##tive ; cumulative trauma disorders [SEP]',
  'our_bi_cui': 'C0336303',
  'our_bi_cui_name': 'Sign-painter',
  'our_bi_cui_desc': '[CLS] Sign - painter [unused3] ( Professional or O ##cc ##upation ##al Group , occupation : Sign ##writer ) [SEP]',
  'our_cross_cui': 'C1257890',
  'our_cross_cui_name': 'Population Group',
  'our_cross_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]'},
 {'mention_id': '28197156.33',
  'test_set_idx': 28103,
  'type': 'T017',
  'name': 'Ghd8',
  'context': '[CLS] in ##dic ##a all ##ele of Carbon As ##si ##mi ##lation Rate 8 has a 1 - b ##p del ##eti ##on at 322 b ##p from the start co ##don , resulting in a t ##runcated protein of 125 amino acids . In addition , Carbon As ##si ##mi ##lation Rate 8 is identical to D ##TH ##8 / [unused1] G ##h ##d ##8 [unused2] / L ##HD ##1 , which was reported to control rice flowering date . The increase of A is largely due to an increase of R ##u ##B ##P regeneration rate via increased leaf nitrogen content , and partially explained by reduced s ##tom ##ata ##l limitation via increased s ##tom ##ata ##l conduct ##ance relative to A . This [SEP]',
  'existing_cui': 'C0035439',
  'existing_cui_name': 'Rheumatic Heart Disease',
  'existing_cui_desc': '[CLS] R ##he ##umatic Heart Disease [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : R ##he ##umatic Heart Disease [ Disease / Finding ] ; heart disease r ##he ##umatic fever ; card ##itis r ##he ##umatic ; Heart Disease , R ##he ##umatic ; R ##he ##umatic card ##itis ; R ##he ##umatic Heart Disease ##s ; Heart Disease ##s , R ##he ##umatic ; R ##he ##umatic heart disease un ##sp ##ec ##ified ; Disease ##s , R ##he ##umatic Heart ; r ##he ##uma ##to ##id heart disease ; disease fever heart r ##he ##umatic ; R ##he ##umatic heart disease , un ##sp ##ec ##ified ; heart disease r ##he ##uma ##to ##id ; r ##he ##umatic ; [SEP]',
  'our_bi_cui': 'C3536868',
  'our_bi_cui_name': 'Glyceraldehyde-3-phosphate dehydrogenase',
  'our_bi_cui_desc': '[CLS] G ##ly ##cer ##ald ##eh ##yd ##e - 3 - phosphate de ##hy ##dr ##ogen ##ase [unused3] ( Chemical , substance : g ##ly ##cer ##ald ##eh ##yd ##e 3 phosphate de ##hy ##dr ##ogen ##ase ; A ##ging - Associated Gene 9 Pro ##tein ; P ##ept ##idy ##l - Cy ##stein ##e S - Ni ##tro ##sy ##lase GA ##PD ##H ; trio ##se ##ph ##os ##phate de ##hy ##dr ##ogen ##ase ; G ##ly ##cer ##ald ##eh ##yd ##e 3 phosphate de ##hy ##dr ##ogen ##ase ; EC 1 . 2 . 1 . 12 ; G ##ly ##cer ##ald ##eh ##yd ##e - 3 - Ph ##os ##phate De ##hy ##dr ##ogen ##ase ; Trio ##se ##ph ##os ##phate de ##hy ##dr [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27545500.22',
  'test_set_idx': 9998,
  'type': 'T103',
  'name': '16-aldehyde tanshinone I',
  'context': '[CLS] synthesized and b ##act ##eri ##ost ##atic activity was explored . In addition , s ##yne ##rg ##istic or add ##itive activity between 16 - al ##de ##hy ##de tan ##shin ##one I and amino ##gly ##cos ##ide anti ##biotics or β - la ##ct ##am anti ##biotics in v ##it ##ro was identified . Moreover , [unused1] 16 - al ##de ##hy ##de tan ##shin ##one I [unused2] was documented to au ##gment clearance of s ##tre ##pt ##omy ##cin and am ##pic ##ill ##in against S . au ##reus in a m ##uri ##ne infection model . Primary me ##chan ##istic insight indicated that 16 - al ##de ##hy ##de tan ##shin ##one I could damage the bacterial cell membrane , leading to accumulation [SEP]',
  'existing_cui': 'C0075810',
  'existing_cui_name': 'tanshinone',
  'existing_cui_desc': '[CLS] tan ##shin ##one [unused3] ( Chemical : 1 , 6 - dim ##eth ##yl ##phe ##nant ##hr ##o ( 1 , 2 - b ) fur ##an - 10 , 11 - di ##one ; tan ##shin ##one I ) [SEP]',
  'our_bi_cui': 'C0075810',
  'our_bi_cui_name': 'tanshinone',
  'our_bi_cui_desc': '[CLS] tan ##shin ##one [unused3] ( Chemical : 1 , 6 - dim ##eth ##yl ##phe ##nant ##hr ##o ( 1 , 2 - b ) fur ##an - 10 , 11 - di ##one ; tan ##shin ##one I ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27445383.13',
  'test_set_idx': 6021,
  'type': 'T007',
  'name': 'D7632',
  'context': "[CLS] Associated Legion ##ella p ##ne ##um ##op ##hil ##a Is ##olate ##s We report here the complete genome sequences of three Legion ##ella p ##ne ##um ##op ##hil ##a is ##olate ##s that are associated with a Legion ##naire ##s ' disease outbreak in New York in 2012 . Two clinical is ##olate ##s ( D ##7 ##6 ##30 and [unused1] D ##7 ##6 ##32 [unused2] ) and one environmental is ##olate ( D ##7 ##6 ##31 ) were recovered from this outbreak . A single is ##olate - specific v ##ir ##ule ##nce gene was found in D ##7 ##6 ##32 . These is ##olate ##s were included in a large study evaluating the g ##eno ##mic resolution of various bio ##in ##formatics approaches for [SEP]",
  'existing_cui': 'C1826211',
  'existing_cui_name': 'MIR632 gene',
  'existing_cui_desc': '[CLS] MI ##R ##6 ##32 gene [unused3] ( Ana ##tom ##ical Structure : h ##sa - mi ##r - 63 ##2 ; micro ##RNA 63 ##2 ) [SEP]',
  'our_bi_cui': 'C0038410',
  'our_bi_cui_name': 'Streptococcus pneumoniae',
  'our_bi_cui_desc': '[CLS] St ##re ##pt ##oc ##oc ##cus pneumonia ##e [unused3] ( Ba ##cter ##ium , organism : P ##ne ##um ##oc ##oc ##cus ; s . pneumonia ##e ; Micro ##co ##cc ##us pneumonia ##e ; ST ##RE ##P ##TO ##CO ##CC ##US P ##NE ##UM ##ON ##IA ##E ; p ##ne ##um ##oc ##oc ##cal ; s ##tre ##pt ##oc ##oc ##cus pneumonia ##e ; St ##re ##pt ##oc ##oc ##cus ( Di ##p ##loc ##oc ##cus ) pneumonia ##e ; St ##re ##pt ##oc ##oc ##cus pneumonia ##e ( Klein 1884 ) Chester 1901 ; ST ##RE ##P P ##NE ##UM ##ON ##IA ##E ; s ##tre ##pt ##oc ##oc ##cus pneumonia ##e bacteria ; p ##ne ##um ##oc ##oc ##cus ; dip ##loc ##oc [SEP]',
  'our_cross_cui': 'C0023238',
  'our_cross_cui_name': 'Legionella pneumophila',
  'our_cross_cui_desc': '[CLS] Legion ##ella p ##ne ##um ##op ##hil ##a [unused3] ( Ba ##cter ##ium , organism : leg ##ione ##lla p ##ne ##um ##op ##hil ##a ; leg ##ione ##lla p ##ne ##um ##op ##hil ##ia ; P ##ne ##um ##op ##hil ##a ; L ##EG ##ION ##EL ##LA P ##NE ##UM ##OP ##H ##IL ##A ; Legion ##ella p ##ne ##um ##op ##hil ##a B ##ren ##ner et al . 1979 ) [SEP]'},
 {'mention_id': '27601292.27',
  'test_set_idx': 11202,
  'type': 'T082',
  'name': 'Groningen',
  'context': '[CLS] via reduced clearance of the drug , resulting in higher v ##ori ##cona ##zo ##le t ##rough concentrations . To investigate prospective ##ly the influence of inflammation on v ##ori ##cona ##zo ##le metabolism and v ##ori ##cona ##zo ##le t ##rough con ##cent ratio n ##s . A prospective observation ##al study was performed at the University Medical Center [unused1] G ##ron ##ingen [unused2] . Pat ##ients were eligible for inclusion if they were ≥ 18 years old and treated with v ##ori ##cona ##zo ##le . V ##ori ##cona ##zo ##le and v ##ori ##cona ##zo ##le - N - oxide concentrations were determined in discarded blood samples . To determine the degree of inflammation , C - reactive protein concentrations were used . [SEP]',
  'existing_cui': 'C1297262',
  'existing_cui_name': 'Groningen horse',
  'existing_cui_desc': '[CLS] G ##ron ##ingen horse [unused3] ( E ##uka ##ryo ##te , organism : G ##ron ##ingen horse breed ) [SEP]',
  'our_bi_cui': 'C1297262',
  'our_bi_cui_name': 'Groningen horse',
  'our_bi_cui_desc': '[CLS] G ##ron ##ingen horse [unused3] ( E ##uka ##ryo ##te , organism : G ##ron ##ingen horse breed ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '28125734.1',
  'test_set_idx': 26155,
  'type': 'T103',
  'name': 'MiR-21-5p',
  'context': "[CLS] I ##dent ##ification of [unused1] Mi ##R - 21 - 5 ##p [unused2] as a Fun ##ctional Reg ##ulator of Me ##so ##the ##lin Express ##ion Using Micro ##RNA Capt ##ure A ##ffin ##ity Co ##up ##led with Next Generation Se ##quencing Micro ##RNA ##s are small non - coding RNA ##s that regulate m ##RNA expression mainly by si ##len ##cing target trans ##cripts via binding to mi ##RNA recognition elements in the 3 ' un ##tra ##ns ##lated region . The identification of b ##ona fi ##de targets is challenging for researchers working on the functional aspect of Micro ##RNA ##s . Recently , we developed a method ( mi ##R - CA ##TC ##H ) based on bio ##tin ##yla ##ted DNA anti [SEP]",
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1999986',
  'our_bi_cui_name': 'MIRN21 microRNA , human',
  'our_bi_cui_desc': '[CLS] MI ##R ##N ##21 micro ##RNA , human [unused3] ( Chemical : micro ##RNA - 21 , human ; mi ##RNA - 21 , human ; h ##sa - mi ##r - 21 micro ##RNA ; mi ##R - 21 , human ; mi ##R ##21 , human ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '27695680.17',
  'test_set_idx': 14384,
  'type': 'T033',
  'name': 'non-FI-DELIS',
  'context': '[CLS] ##OT ##S Ex ##pan ##sion through Local Initiative ##s to Stop T ##B areas , then compared the difference between the case not ##ification rates of the intervention year and the previous year in the for Inn ##ova ##tive D ##OT ##S Ex ##pan ##sion through Local Initiative ##s to Stop T ##B areas with those in the [unused1] non - F ##I - DE ##L ##IS [unused2] areas within the province . Results : There was an increase in the C ##NR in the intervention year compared with the previous year for all the project sites . The differences between the C ##NR in the intervention year and the previous year ranged from 6 . 4 to 31 . 1 per 100 000 population in [SEP]',
  'existing_cui': 'C1121292',
  'existing_cui_name': '5-Fluoroindole-3-acetic acid',
  'existing_cui_desc': '[CLS] 5 - F ##lu ##oro ##ind ##ole - 3 - ace ##tic acid [unused3] ( Chemical : 5 - F ##I - AA ) [SEP]',
  'our_bi_cui': 'C0282574',
  'our_bi_cui_name': 'Intellectual Property',
  'our_bi_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27760149.1',
  'test_set_idx': 15788,
  'type': 'T170',
  'name': 'BEST',
  'context': '[CLS] [unused1] B ##ES ##T [unused2] : Next - Generation B ##io ##medical En ##ti ##ty Search Too ##l for Knowledge Discovery from B ##io ##medical Literature As the volume of publications rapidly increases , searching for relevant information from the literature becomes more challenging . To complement standard search engines such as Pub ##M ##ed , it is desirable to have an advanced search tool that directly returns relevant bio ##medical entities such as targets , drugs , and mutations rather than a long list of articles . Some existing tools submit a que ##ry to Pub ##M ##ed and process retrieved abstract ##s to extract information at que ##ry time , resulting in a slow response time and limited coverage of only a fraction of [SEP]',
  'existing_cui': 'C1826421',
  'existing_cui_name': 'BEST1 gene',
  'existing_cui_desc': '[CLS] B ##ES ##T ##1 gene [unused3] ( Ana ##tom ##ical Structure : B ##ES ##TR ##OP ##H ##IN 1 ; B ##ES ##T ; Best disease ; R ##P ##50 ; T ##U ##15 ##B ; B ##MD ; V ##MD ##2 GE ##NE ; best ##rop ##hin 1 ) [SEP]',
  'our_bi_cui': 'C2348167',
  'our_bi_cui_name': 'Search Engine',
  'our_bi_cui_desc': '[CLS] Search Engine [unused3] ( Intel ##lect ##ual Product : Search Program ; Search engines ; Engine , Search ; Search Too ##l ; Search Engine ##s ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28158248.35',
  'test_set_idx': 27139,
  'type': 'T170',
  'name': 'ordinary differential equations',
  'context': '[CLS] 7 of the 13 metabolic steps analyzed . Ad ##dition of information on en ##zy ##matic activities almost always improved the fitness of generalized add ##itive models built solely based on substrate concentrations . generalized add ##itive models were valid ##ated using experimental data that were not included when the model was built . In addition , a system of [unused1] ordinary differential equations [unused2] was developed to describe the instant ##aneous changes in meta ##bol ##ites as a function of the levels of en ##zy ##matic cat ##alytic activities . The results demonstrated the ability of the models to predict changes in the flux of meta ##bol ##ites in the Ju ##ven ##ile hormone pathway , and can be used in the future to design [SEP]',
  'existing_cui': 'C0011906',
  'existing_cui_name': 'Differential Diagnosis',
  'existing_cui_desc': '[CLS] Different ##ial Di ##ag ##nosis [unused3] ( Health Care Act ##ivity , context ##ual qualifier , qualifier value : D ##IA ##G D ##IF ##F ##ER ##EN ##TI ##AL ; Di ##ag ##nose ##s , Different ##ial ; D ##IF ##F ##ER ##EN ##TI ##AL D ##IA ##G ; Di ##ag ##nosis , Different ##ial ; Different ##ial d ##x ; D ##IA ##G D ##IF ##F ##ER ; differential di ##ag ##nose ##s ; differential diagnosis ; Different ##ial diagnosis ( context ##ual qualifier ) ; Different ##ial Di ##ag ##nose ##s ; differential d ##x ; Different ##ial diagnosis ; D ##IF ##F ##ER D ##IA ##G ) [SEP]',
  'our_bi_cui': 'C1705273',
  'our_bi_cui_name': 'Mathematical Operator',
  'our_bi_cui_desc': '[CLS] Mathematical Opera ##tor [unused3] ( Intel ##lect ##ual Product : Opera ##tor ; Fun ##ction ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28344199.22',
  'test_set_idx': 32746,
  'type': 'T017',
  'name': 'CD14 ( + + ) CD16 ( - )',
  'context': '[CLS] - slice multi ##det ##ector com ##puted to ##mo ##graphy in patients with stable an ##gin ##a p ##ector ##is . Method ##s and Results : We enrolled 65 patients with stable an ##gin ##a p ##ector ##is who underwent multi ##det ##ector com ##puted to ##mo ##graphy . Three mon ##oc ##yte subset ##s ( [unused1] CD ##14 ( + + ) CD ##16 ( - ) [unused2] , CD ##14 ( + + ) CD ##16 ( + ) , and CD ##14 ( + ) CD ##16 ( + ) ) and expression of To ##ll - like receptor 4 were measured by flow c ##yt ##ome ##try . In ##tra ##cor ##ona ##ry plaque ##s were assessed by 64 - slice multi [SEP]',
  'existing_cui': 'C1367477',
  'existing_cui_name': 'CD14 gene',
  'existing_cui_desc': '[CLS] CD ##14 gene [unused3] ( Ana ##tom ##ical Structure : CD ##14 Mo ##le ##cule Gene ; M ##ON ##OC ##Y ##TE D ##IF ##F ##ER ##EN ##TI ##AT ##ION AN ##TI ##GE ##N CD ##14 ; CD ##14 Gene ; CD ##14 molecule ; M ##Y ##EL ##O ##ID CE ##LL - SP ##EC ##IF ##IC L ##E ##UC ##IN ##E - R ##IC ##H G ##L ##Y ##CO ##PR ##OT ##EI ##N ) [SEP]',
  'our_bi_cui': 'C0108768',
  'our_bi_cui_name': 'CD14 Antigen',
  'our_bi_cui_desc': '[CLS] CD ##14 Anti ##gen [unused3] ( Chemical , substance : CD 01 ##4 AN ##TI ##GE ##NS ; Mon ##oc ##yte 2 ; CD ##14 receptor ; My ##elo ##id Cell - S ##pecific Le ##uc ##ine - Rich G ##ly ##co ##p ##rote ##in ; Anti ##gens , CD ##14 ; CD ##14 molecule ; R ##EC ##EP ##T L ##IP ##OP ##OL ##Y ##SA ##CC ##HA ##RI ##DE ; Anti ##gens , CD ##14 [ Chemical / In ##g ##red ##ient ] ; CD ##14 - C ##luster of differentiation anti ##gen 14 ; Le ##uk ##oc ##yte M ##3 ; My ##elo ##id 4 ; Li ##pop ##oly ##sa ##cc ##hari ##de Re ##ceptor ##s ; CD ##14 ; Li ##po ##gly ##can [SEP]',
  'our_cross_cui': 'C0026473',
  'our_cross_cui_name': 'Monocytes',
  'our_cross_cui_desc': '[CLS] Mon ##ocytes [unused3] ( Ana ##tom ##ical Structure , cell : blood mon ##ocytes ; Blood mon ##ocytes ; Mon ##oc ##yte ; mon ##ocytes ; mon ##oc ##yte ; M ##ON ##OC ##Y ##TE ; mon ##oc ##ytic ) [SEP]'},
 {'mention_id': '28183116.7',
  'test_set_idx': 27733,
  'type': 'T170',
  'name': 'paper',
  'context': '[CLS] On the identity of the Pa ##lea ##rc ##tic species of the wolf spider genus < i > T ##re ##ba ##cos ##a < / i > ( Ara ##nea ##e : L ##y ##cos ##idae ) In this [unused1] paper [unused2] we propose T ##re ##ba ##cos ##a br ##un ##hes ##i Ville ##po ##ux , 2007 as a junior synonym of T ##re ##ba ##cos ##a euro ##pa ##ea S ##zine ##t ##á ##r & Ka ##n - c ##sal , 2007 based on the examination of specimens from all the localities from where those species are known . Il ##lus ##tration of the type species of the genus , T ##re ##ba ##cos ##a ma ##r ##xi ( Stone , 1890 ) and [SEP]',
  'existing_cui': 'C0462999',
  'existing_cui_name': 'Litmus paper',
  'existing_cui_desc': '[CLS] Li ##t ##mus paper [unused3] ( Medical Devi ##ce , physical object : Li ##t ##mus Paper ; lit ##mus papers ; Re ##age ##nts , Clinical Chemistry , Rapid Test , pH , Strip ; IV ##D Test Re ##age ##nt / Kit ##s , Clinical Chemistry , Rapid Test , pH , Strip ; pH Strip In ##dic ##ators ; lit ##mus paper ) [SEP]',
  'our_bi_cui': 'C0600659',
  'our_bi_cui_name': 'Manuscripts',
  'our_bi_cui_desc': '[CLS] Man ##us ##cripts [unused3] ( Intel ##lect ##ual Product : manuscript ; manuscripts ; Man ##us ##cript ) [SEP]',
  'our_cross_cui': 'C1706852',
  'our_cross_cui_name': 'Article',
  'our_cross_cui_desc': '[CLS] Article [unused3] ( Intel ##lect ##ual Product ) [SEP]'},
 {'mention_id': '27932585.39',
  'test_set_idx': 22266,
  'type': 'T103',
  'name': 'TYW3',
  'context': '[CLS] ##tate and his ##ti ##dine , conserved in T ##Y ##W ##3 / Ta ##w ##3 , cluster to form the cat ##alytic center . These structural and sequence features indicate that T ##Y ##W ##3 / Ta ##w ##3 proteins constitute a distinct class of SA ##M - dependent met ##hyl ##tra ##ns ##fer ##ases . Using site - directed m ##uta ##genesis along with in v ##ivo complement ##ation ass ##ays combined with mass s ##pect ##rome ##try as well as l ##igan ##d dock ##ing and co ##fa ##ctor binding ass ##ays , we have identified the active site of [unused1] T ##Y ##W ##3 [unused2] and residues essential for co ##fa ##ctor binding and met ##hyl ##tra ##ns ##fer ##ase activity . [SEP]',
  'existing_cui': 'C1823563',
  'existing_cui_name': 'TYW3 gene',
  'existing_cui_desc': '[CLS] T ##Y ##W ##3 gene [unused3] ( Ana ##tom ##ical Structure : FL ##J ##40 ##9 ##18 ; t ##RNA - Y ##W - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; t ##RNA - y ##W s ##ynth ##esi ##zing protein 3 ho ##mo ##log ; t ##RNA - W ##Y ##B ##UT ##OS ##IN ##E - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ) [SEP]',
  'our_bi_cui': 'C1823563',
  'our_bi_cui_name': 'TYW3 gene',
  'our_bi_cui_desc': '[CLS] T ##Y ##W ##3 gene [unused3] ( Ana ##tom ##ical Structure : FL ##J ##40 ##9 ##18 ; t ##RNA - Y ##W - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; t ##RNA - y ##W s ##ynth ##esi ##zing protein 3 ho ##mo ##log ; t ##RNA - W ##Y ##B ##UT ##OS ##IN ##E - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28442401.8',
  'test_set_idx': 36180,
  'type': 'T103',
  'name': 'pseudokinase',
  'context': '[CLS] T ##RI ##B ##3 down ##re ##gu ##lation enhance ##s do ##x ##or ##ubi ##cin - induced c ##yt ##oto ##xi ##city in gas ##tric cancer cells T ##RI ##B ##3 , which is a [unused1] pseudo ##kin ##ase [unused2] known to regulate multiple pro - survival pathways , appears to be a potential therapeutic target for the treatment of human tumors . However , its precise role in cancer is controversial , as T ##RI ##B ##3 protein levels have been associated with both good and poor pro ##gno ##sis in cancer patients . Here , we investigated the significance of T ##RI ##B ##3 expression in the survival of gas ##tric cancer cells exposed to anti ##can ##cer drugs . We found that the [SEP]',
  'existing_cui': 'C3710669',
  'existing_cui_name': 'ZED1 protein , Arabidopsis',
  'existing_cui_desc': '[CLS] Z ##ED ##1 protein , Arab ##ido ##psis [unused3] ( Chemical : Z ##ED ##1 pseudo ##kin ##ase , Arab ##ido ##psis ) [SEP]',
  'our_bi_cui': 'C0014442',
  'our_bi_cui_name': 'Enzymes',
  'our_bi_cui_desc': '[CLS] En ##zy ##mes [unused3] ( Chemical , medication , substance : enzymes ; enzyme ; En ##zy ##mes [ Chemical / In ##g ##red ##ient ] ; B ##io ##cat ##aly ##sts ; En ##zy ##me ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28389374.84',
  'test_set_idx': 34086,
  'type': 'T201',
  'name': 'responsive',
  'context': '[CLS] ( a dual p ##hos ##ph ##oi ##nos ##iti ##de 3 - kinase - ma ##mmal ##ian target of rap ##am ##y ##cin inhibitor ) , which induce ##s down ##re ##gu ##lation of BR ##CA ##1 . These observations suggest that BR ##CA ##1 associated protein ##1 does regulate DNA damage response and influences drug sensitivity . It might therefore be relevant to investigate whether patients with high expression of BR ##CA ##1 associated protein ##1 may be [unused1] re ##sp ##ons ##ive [unused2] to p ##oly ( ad ##eno ##sin ##e dip ##hos ##phate - rib ##ose ) polymer ##ase / p ##hos ##ph ##oi ##nos ##iti ##de 3 - kinase - ma ##mmal ##ian target of rap ##am ##y ##cin inhibitor ##s . [SEP]',
  'existing_cui': 'C0237284',
  'existing_cui_name': 'unresponsive behavior',
  'existing_cui_desc': '[CLS] un ##res ##po ##ns ##ive behavior [unused3] ( Finding , finding : un ##res ##po ##ns ##ive ; Un ##res ##po ##ns ##ive ; Not Re ##sp ##ons ##ive ) [SEP]',
  'our_bi_cui': 'C0683154',
  'our_bi_cui_name': 'drug response',
  'our_bi_cui_desc': '[CLS] drug response [unused3] ( B ##iol ##og ##ic Fun ##ction : response to drug ; drug responses ; drugs response ) [SEP]',
  'our_cross_cui': 'C0521982',
  'our_cross_cui_name': 'Response to treatment',
  'our_cross_cui_desc': '[CLS] Response to treatment [unused3] ( Clinical At ##tri ##but ##e , context - dependent category , finding , function , o ##bs ##er ##vable entity , situation : response to treatment ; response treatments ; The ##rap ##eu ##tic response ; response therapeutic ; The ##rap ##eu ##tic Response ; Finding related to therapeutic response ; successful treatment ; successful treatments ; The ##rap ##eu ##tic response , function ) [SEP]'},
 {'mention_id': '27460285.31',
  'test_set_idx': 6603,
  'type': 'T037',
  'name': 'injuries',
  'context': '[CLS] ##ic injuries . Em ##pha ##sis is placed on multi ##det ##ector CT , which represents the ideal m ##oda ##lity to comprehensive ##ly visual ##ize the operated groin region and deeper in ##tra - abdominal structures . CT consistently depicts se ##roma , mesh infections , hem ##or ##r ##hage ##s , bow ##el complications and u ##rina ##ry bladder [unused1] injuries [unused2] , and thus generally provides a consistent basis for therapeutic choice . Since radio ##logists are increasingly requested to investigate suspected i ##at ##rogen ##ic complications , this paper aims to provide an increased familiar ##ity with early CT studies after In ##guin ##al her ##nia repair , including complications and normal post ##oper ##ative appearances such as focal pseudo ##les ##ions [SEP]',
  'existing_cui': 'C0005604',
  'existing_cui_name': 'Birth Injuries',
  'existing_cui_desc': '[CLS] Birth In ##ju ##ries [unused3] ( In ##ju ##ry or Po ##ison ##ing , Birth , diagnosis , disorder : In ##ju ##ries , Birth ; T ##ra ##umatic birth ; injury birth ; Birth trauma , un ##sp ##ec ##ified ; Birth T ##ra ##uma ; Birth trauma of f ##etus ; injury ; birth ; traumatic birth ; birth trauma ; birth injuries ; Birth In ##ju ##ry ; trauma birth ; Neo ##nat ##al Birth In ##ju ##ry ; birth ##ing injuries ; Birth In ##ju ##ries [ Disease / Finding ] ; Un ##sp ##ec ##ified birth trauma ; birth ##ing trauma ; B ##IR ##TH IN ##J ; IN ##J B ##IR ##TH ; Birth injuries ; birth ; injury ; [SEP]',
  'our_bi_cui': 'C3263723',
  'our_bi_cui_name': 'Traumatic injury',
  'our_bi_cui_desc': '[CLS] T ##ra ##umatic injury [unused3] ( In ##ju ##ry or Po ##ison ##ing , disorder : IN ##J ; In ##ju ##ry , un ##sp ##ec ##ified ; wound ; trauma ; T ##ra ##umatic In ##ju ##ry ; W ##ou ##nd ; injury from an external force ; injuries ; injury ; In ##ju ##ries ; T ##ra ##uma ; In ##ju ##ry ; In ##ju ##ry - disorder ; Un ##sp ##ec ##ified injury ) [SEP]',
  'our_cross_cui': 'C0403677',
  'our_cross_cui_name': 'Injury of bladder',
  'our_cross_cui_desc': '[CLS] In ##ju ##ry of bladder [unused3] ( In ##ju ##ry or Po ##ison ##ing , diagnosis , disorder : bladder injuries ; Un ##sp ##ec ##ified injury of bladder ; injury ; bladder ; B ##lad ##der injury ; bladder injury ; injury bladder ; In ##ju ##ry of bladder , un ##sp ##ec ##ified ; bladder ; injury ; B ##lad ##der In ##ju ##ry ) [SEP]'},
 {'mention_id': '27882444.22',
  'test_set_idx': 20505,
  'type': 'T038',
  'name': "gene arrangement 3 ' - NP-P-M-F-HN-L-5 '",
  'context': "[CLS] - 01 / CH / 15 , was isolated from a great spotted wood ##pec ##ker in the northwest region of China in 2015 . The complete genome was sequence ##d , and the results showed that the virus genome was 15 , 192 n ##t in length , in the [unused1] gene arrangement 3 ' - N ##P - P - M - F - H ##N - L - 5 ' [unused2] . Several amino acid mutations were identified in the functional domains of the F and H ##N proteins . The path ##ogenic ##ity index of the is ##olate was evaluated , and the mean death time was 72 h and the in ##tra ##cer ##eb ##ral path ##ogenic ##ity index was 0 [SEP]",
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C1517495',
  'our_bi_cui_name': 'Gene Feature',
  'our_bi_cui_desc': '[CLS] Gene Feature [unused3] ( Ana ##tom ##ical Structure : Gene Organization ; Gene Structure ; Se ##q Feature ; Se ##quence Feature ; Gene Structure / Organization ) [SEP]',
  'our_cross_cui': 'C0887940',
  'our_cross_cui_name': 'Gene Order',
  'our_cross_cui_desc': '[CLS] Gene Order [unused3] ( B ##iol ##og ##ic Fun ##ction : Po ##si ##tions , Gene ; A ##rra ##nge ##ments , Gene ; Po ##si ##tion , Gene ; A ##rra ##nge ##ment , Gene ; Gene A ##rra ##nge ##ment ; Gene Po ##si ##tions ; Gene Po ##si ##tion ; Gene A ##rra ##nge ##ments ; Order , Gene ) [SEP]'},
 {'mention_id': '27286118.9',
  'test_set_idx': 1441,
  'type': 'T033',
  'name': 'no prostatitis',
  'context': '[CLS] ##to ##logical pro ##sta ##titis and lower u ##rina ##ry tract symptoms and sexual function This prospective analysis assessed the effect of his ##to ##logical pro ##sta ##titis on lower u ##rina ##ry tract functions and sexual function . The patients were separated into two groups as his ##to ##logical ##ly observed pro ##sta ##titis ( Group A ) and [unused1] no pro ##sta ##titis [unused2] ( Group B ) according to the bio ##psy outcomes . International pro ##state s ##ym ##pt ##om score , international index of erect ##ile function - 5 scores , ma ##ximal and average flow rate , and residual urine volumes were compared statistical ##ly between groups . There was no significant difference ( P > 0 . 05 ) [SEP]',
  'existing_cui': 'C0033581',
  'existing_cui_name': 'prostatitis',
  'existing_cui_desc': '[CLS] pro ##sta ##titis [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : Pro ##sta ##titis ; pro ##state inflammation ; In ##f ##lam ##mat ##ory diseases of pro ##state ; Pro ##sta ##titis , un ##sp ##ec ##ified ; Pro ##state inflammatory diseases ; Pro ##sta ##titis [ Disease / Finding ] ; In ##f ##lam ##mation of pro ##state ; In ##f ##lam ##mation of the pro ##state ; In ##f ##lam ##mat ##ory disease of pro ##state , un ##sp ##ec ##ified ; inflammation ; pro ##state ; Pro ##static inflammatory disease ; pro ##state ; inflammation ; Pro ##sta ##ti ##tide ##s ) [SEP]',
  'our_bi_cui': 'C0497153',
  'our_bi_cui_name': 'No disease',
  'our_bi_cui_desc': '[CLS] No disease [unused3] ( Finding : Disease ; none ; no disease ; A ##bs ##ence ( of ) ; disease ; absence of disease ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27826436.55',
  'test_set_idx': 18385,
  'type': 'T204',
  'name': 'cultivars',
  'context': '[CLS] The content of as ##cor ##bic acid of c ##v . To ##chi ##oto ##me was higher than the others , but there was no significant difference in cult ##iva ##rs of Ben ##ih ##op ##pe , To ##chi ##oto ##me , and Sa ##chin ##oka . Fr ##uc ##tos ##e and glucose were the major sugar ##s in all [unused1] cult ##iva ##rs [unused2] . C ##it ##ric acid was the major organic acid in c ##v . To ##chi ##oto ##me , c ##v . Sa ##chin ##oka , and c ##v . G ##ui ##mei ##ren . To ##chi ##oto ##me had higher ratios of T ##SS / T ##A and total sugar ##s / total organic acids than others , arising from [SEP]',
  'existing_cui': 'C3385047',
  'existing_cui_name': 'Brassia hybrid cultivar',
  'existing_cui_desc': '[CLS] Brass ##ia hybrid cult ##iva ##r [unused3] ( E ##uka ##ryo ##te : Ada hybrid cult ##iva ##r ) [SEP]',
  'our_bi_cui': 'C0242775',
  'our_bi_cui_name': 'Agricultural Crops',
  'our_bi_cui_desc': '[CLS] Agricultural C ##rops [unused3] ( E ##uka ##ryo ##te : C ##rops , Agricultural ; C ##rops ; Agricultural C ##rop ) [SEP]',
  'our_cross_cui': 'C0032098',
  'our_cross_cui_name': 'Plants',
  'our_cross_cui_desc': '[CLS] Plants [unused3] ( E ##uka ##ryo ##te , organism : Ch ##lor ##op ##hy ##ta / Em ##b ##ryo ##phy ##ta group ; green plants ; Ch ##lor ##ob ##ion ##ta ; Ch ##lor ##ob ##ion ##ta B ##rem ##er , 1985 ; V ##iri ##di ##p ##lant ##ae ; Plants , General ; Kingdom Plant ##ae ; V ##iri ##di ##p ##lant ##ae C ##ava ##lier - Smith , 1981 ; plants ; Kingdom V ##iri ##di ##p ##lant ##ae ; plant ; ch ##lor ##op ##hy ##te / em ##b ##ryo ##phy ##te group ; Plant ) [SEP]'},
 {'mention_id': '28197156.20',
  'test_set_idx': 28090,
  'type': 'T103',
  'name': 'OsHAP3',
  'context': '[CLS] study , we determined precise location of Carbon As ##si ##mi ##lation Rate 8 by crossing a high - yielding in ##dic ##a cult ##iva ##r with a Japanese commercial cult ##iva ##r . Fine mapping suggested that Carbon As ##si ##mi ##lation Rate 8 en ##codes a put ##ative He ##me Act ##iva ##tor Pro ##tein 3 ( [unused1] O ##s ##HA ##P ##3 [unused2] ) subunit of a CC ##AA ##T - box - binding transcription factor called O ##s ##HA ##P ##3 ##H . Se ##quencing analysis revealed that the in ##dic ##a all ##ele of Carbon As ##si ##mi ##lation Rate 8 has a 1 - b ##p del ##eti ##on at 322 b ##p from the start co ##don , resulting [SEP]',
  'existing_cui': 'C1427162',
  'existing_cui_name': 'THAP3 gene',
  'existing_cui_desc': '[CLS] T ##HA ##P ##3 gene [unused3] ( Ana ##tom ##ical Structure : T ##HA ##P domain containing , a ##pop ##tosis associated protein 3 ; T ##HA ##P D ##OM ##A ##IN - CO ##NT ##A ##IN ##ING PR ##OT ##EI ##N 3 ) [SEP]',
  'our_bi_cui': 'C1722024',
  'our_bi_cui_name': 'HO3 protein , Arabidopsis',
  'our_bi_cui_desc': '[CLS] H ##O ##3 protein , Arab ##ido ##psis [unused3] ( Chemical : hem ##e oxygen ##ase 3 , Arab ##ido ##psis ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28203549.82',
  'test_set_idx': 28274,
  'type': 'T170',
  'name': 'ST',
  'context': '[CLS] ##yper ##mu ##co ##vis ##co ##us K ##le ##bs ##iel ##la pneumonia ##e is ##olate ##s with K ##1 , K ##2 , K ##5 , K ##20 , and K ##5 ##7 ( 27 . 5 % , 19 / 69 ) and was only found among K ##1 is ##olate ##s . ST ##65 was the second most prevalent [unused1] ST [unused2] ( 26 . 1 % , 18 / 69 ) and was also only found among K ##2 is ##olate ##s . ST ##23 - K ##1 h ##yper ##mu ##co ##vis ##co ##us K ##le ##bs ##iel ##la pneumonia ##e is ##olate ##s ( 84 . 2 % , 16 / 19 ) were associated with p ##yo ##genic liver a [SEP]',
  'existing_cui': 'C0036183',
  'existing_cui_name': 'Sao Tome and Principe',
  'existing_cui_desc': '[CLS] Sa ##o Tom ##e and P ##rin ##ci ##pe [unused3] ( Spa ##tial Concept , geographic location : SA ##O TO ##ME AND PR ##IN ##CI ##P ##E ; São Tom ##é and P ##r ##ín ##ci ##pe ) [SEP]',
  'our_bi_cui': 'C0887835',
  'our_bi_cui_name': 'Ribotype',
  'our_bi_cui_desc': '[CLS] R ##ib ##otype [unused3] ( Finding : R ##ib ##otype ##s ) [SEP]',
  'our_cross_cui': 'C0449943',
  'our_cross_cui_name': 'Serotype',
  'our_cross_cui_desc': '[CLS] Ser ##otype [unused3] ( Intel ##lect ##ual Product , attribute , o ##bs ##er ##vable entity , qualifier value : se ##rov ##ar ; se ##rov ##ars ; Ser ##ova ##rs ; se ##rot ##ype ; se ##rot ##ype ##s ; Ser ##otype ##s ; Ser ##ova ##r ) [SEP]'},
 {'mention_id': '28289065.41',
  'test_set_idx': 31384,
  'type': 'T058',
  'name': 'RP',
  'context': '[CLS] re ##cu ##rrence within G ##lea ##son Grade Groups based on either bio ##psy or pro ##state ##ct ##omy scoring . Multi ##var ##iate analysis further confirmed that a higher Grade Groups was significantly associated with risk of bio ##chemical re ##cu ##rrence . This Grade Groups system had a higher pro ##gno ##stic discrimination for both initial bio ##psy and [unused1] R ##P [unused2] than G ##lea ##son score . Our study valid ##ates the use of the revised and updated Grade Groups system in a mixed - ethnicity population of Asian men . Higher Grade Groups was significantly associated with increased risk of bio ##chemical re ##cu ##rrence . We therefore recommend its use to inform clinical management for patients with pro ##state cancer [SEP]',
  'existing_cui': 'C1538326',
  'existing_cui_name': 'BLOC1S3 gene',
  'existing_cui_desc': '[CLS] B ##L ##OC ##1 ##S ##3 gene [unused3] ( Ana ##tom ##ical Structure : B ##io ##genesis of L ##ys ##oso ##me - related Organ ##elles complex - 1 Sub ##uni ##t 3 ; Herman ##sky - P ##ud ##lak syndrome 8 ; HP ##S ##8 GE ##NE ; R ##ED ##UC ##ED P ##IG ##ME ##NT ##AT ##ION , M ##O ##US ##E , H ##OM ##OL ##O ##G OF ; B ##IO ##GE ##NE ##SI ##S OF L ##Y ##SO ##SO ##ME - R ##EL ##AT ##ED OR ##GA ##NE ##LL ##ES CO ##MP ##LE ##X 1 , S ##U ##B ##UN ##IT 3 ; bio ##genesis of l ##ys ##oso ##mal organ ##elles complex 1 subunit 3 ; B ##L ##OC - 1 [SEP]',
  'our_bi_cui': 'C0033573',
  'our_bi_cui_name': 'Prostatectomy',
  'our_bi_cui_desc': '[CLS] Pro ##state ##ct ##omy [unused3] ( Health Care Act ##ivity , procedure , treatment : pro ##state re ##section ; Ex ##cision Pro ##ced ##ures on the Pro ##state ; pro ##state ##ct ##omy ; g ##eni ##to - u ##rina ##ry tract surgery pro ##state ##ct ##omy ; pro ##state re ##section ##s ; Pro ##state ##ct ##omi ##es ; pro ##state ##ct ##omi ##es ; Me ##d : Pro ##state ##ct ##omy ; surgical removal of the pro ##state ; pro ##state re ##section ##ing ; Pro ##state ex ##cision ) [SEP]',
  'our_cross_cui': 'C0194810',
  'our_cross_cui_name': 'Radical prostatectomy',
  'our_cross_cui_desc': '[CLS] Radical pro ##state ##ct ##omy [unused3] ( Health Care Act ##ivity , procedure , treatment : pro ##state ##ct ##omy radical ; radical pro ##state ##ct ##omy ; Total pro ##state ##ct ##omy ; Pro ##sta ##tov ##esi ##cule ##ct ##omy ; total pro ##state ##ct ##omy ; Radical pro ##state ##ct ##omy by any approach ; Radical Pro ##state ##ct ##omy ; pro ##state ##ct ##omi ##es radical ) [SEP]'},
 {'mention_id': '27812530.11',
  'test_set_idx': 18001,
  'type': 'T103',
  'name': "material 's",
  'context': "[CLS] for En ##han ##cing A ##pp ##lica ##bility in V ##ert ##eb ##rop ##last ##y Pol ##yme ##thy ##lm ##eth ##ac ##ryl ##ate bone cement is a popular bone void fill ##er for ve ##rte ##bro ##p ##last ##y . However , the use of Pol ##yme ##thy ##lm ##eth ##ac ##ryl ##ate has some draw ##backs , including the [unused1] material ' s [unused2] excessive stiff ##ness , ex ##oth ##er ##mic polymer ##ization , and short handling time . This study aimed to create an ideal modified bone cement to solve the above - mentioned problems . Mo ##dified bone cement ##s were prepared by combining Pol ##yme ##thy ##lm ##eth ##ac ##ryl ##ate with three different volume fraction ##s of cast ##or oil [SEP]",
  'existing_cui': 'C2986060',
  'existing_cui_name': 'Material Form Code',
  'existing_cui_desc': '[CLS] Material Form Code [unused3] ( Intel ##lect ##ual Product : Material . form ##C ##ode ) [SEP]',
  'our_bi_cui': 'C0032521',
  'our_bi_cui_name': 'Polymers',
  'our_bi_cui_desc': '[CLS] Pol ##yme ##rs [unused3] ( Chemical , substance : Pol ##yme ##ric ; Pol ##yme ##ric Mac ##rom ##ole ##cule ##s ; Pol ##yme ##r ; P ##OL ##Y ##ME ##R S ##U ##BS ##TA ##NC ##E ; Pol ##yme ##rs [ Chemical / In ##g ##red ##ient ] ; polymer ; polymer ##s ) [SEP]',
  'our_cross_cui': 'C0005479',
  'our_cross_cui_name': 'Biomaterials',
  'our_cross_cui_desc': '[CLS] B ##io ##mate ##rial ##s [unused3] ( Chemical : B ##io ##mate ##rial ; bio ##com ##pa ##tible materials ; B ##io ##medical Material ; bio ##mate ##rial ##s ; B ##io ##medical materials ; B ##io ##com ##pa ##tible Materials ; bio ##medical materials ; bio ##mate ##rial ; Materials , B ##io ##com ##pa ##tible ; bio ##medical material ; B ##io ##com ##pa ##tible Materials [ Chemical / In ##g ##red ##ient ] ) [SEP]'},
 {'mention_id': '28441489.37',
  'test_set_idx': 36159,
  'type': 'T033',
  'name': 'Cold - start',
  'context': '[CLS] composition of the volatile organic compounds emissions between Port Fuel In ##jection and G ##DI ##s , including ben ##zen ##e , to ##lue ##ne , et ##hyl ##ben ##zen ##e , and x ##yle ##nes . Therefore , the secondary organic a ##ero ##sol and oz ##one formation potential of the exhaust does not depend on engine technology . [unused1] Cold - start [unused2] contributes a larger fraction of the total unified cycle emissions for vehicles meeting more - string ##ent emission standards . Organic gas emissions were the most sensitive to cold - start compared to the other poll ##uta ##nts tested here . There were no statistical ##ly significant differences in the effects of cold - start on G ##DI ##s and P [SEP]',
  'existing_cui': 'C0452588',
  'existing_cui_name': 'Start brand of breakfast cereal',
  'existing_cui_desc': '[CLS] Start brand of breakfast cereal [unused3] ( Food , substance : Start ) [SEP]',
  'our_bi_cui': 'C1455171',
  'our_bi_cui_name': 'Cool-X-A',
  'our_bi_cui_desc': '[CLS] Cool - X - A [unused3] ( Chemical : Cool - Cool ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27482869.9',
  'test_set_idx': 7902,
  'type': 'T082',
  'name': 'Chapel Hill',
  'context': "[CLS] on 30 September 1946 in Columbus , OH . He studied medicine at the University of Michigan , graduated in 1973 , and after a rotating internship , he completed a master ' s degree in maternal and child health ( 1976 ) and a PhD in e ##pid ##em ##iology ( 1979 ) at the University of North Carolina in [unused1] Chapel Hill [unused2] . After graduation , he went to work at the National Institute of Environmental Health Sciences ( National Institute of Environmental Health Sciences , one of the US National Institutes of Health ) in Durham , NC , where he has spent his career . He developed a research program in reproductive and per ##ina ##tal e ##pid ##em ##iology , [SEP]",
  'existing_cui': 'C0313304',
  'existing_cui_name': 'hemoglobin Chapel Hill',
  'existing_cui_desc': '[CLS] hem ##og ##lo ##bin Chapel Hill [unused3] ( Chemical , substance : Ha ##em ##og ##lo ##bin Chapel Hill ; H ##b Chapel Hill ; He ##mo ##g ##lo ##bin Chapel Hill ; H ##b 74 ( E ##F ##3 ) , As ##p - g ##ly ) [SEP]',
  'our_bi_cui': 'C0313304',
  'our_bi_cui_name': 'hemoglobin Chapel Hill',
  'our_bi_cui_desc': '[CLS] hem ##og ##lo ##bin Chapel Hill [unused3] ( Chemical , substance : Ha ##em ##og ##lo ##bin Chapel Hill ; H ##b Chapel Hill ; He ##mo ##g ##lo ##bin Chapel Hill ; H ##b 74 ( E ##F ##3 ) , As ##p - g ##ly ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '28271342.55',
  'test_set_idx': 30732,
  'type': 'T092',
  'name': 'European Society for Medical Oncology',
  'context': '[CLS] 159 cancer ##s at Ra or R ##b were located within 12 cm from the anal verge by rigid pro ##ct ##os ##copy , while only 79 . 7 % of Ra or 82 . 1 % of R ##b cancer ##s were located in the mid ( 5 . 1 - 10 cm ) or low ( ≤ 5 cm ) re ##ct ##um , respectively . Ra and R ##b cancer ##s are deemed to be re ##ct ##al cancer ##s according to National Comprehensive Cancer Network guidelines , but these classification ##s are not interchange ##able with mid - and low - re ##ct ##al cancer ##s , respectively , according to the [unused1] European Society for Medical On ##cology [unused2] guidelines . [SEP]',
  'existing_cui': 'C0025098',
  'existing_cui_name': 'Medical oncology specialty',
  'existing_cui_desc': '[CLS] Medical on ##cology specialty [unused3] ( B ##io ##medical O ##cc ##upation or Disc ##ip ##line , qualifier value : ME ##D ON ##CO ##L ; medical on ##cology ; On ##cology , Medical ; on ##cology medical ; ON ##CO ##L ME ##D ; Medical On ##cology ; Medical on ##cology ) [SEP]',
  'our_bi_cui': 'C0037459',
  'our_bi_cui_name': 'Societies , Medical',
  'our_bi_cui_desc': '[CLS] Societies , Medical [unused3] ( Organization : Medical Society ; medical societies ; Society , Medical ; medical society ; S ##OC ##IE ##TY ME ##D ; Medical Societies ; ME ##D S ##OC ##IE ##TI ##ES ; S ##OC ##IE ##TI ##ES ME ##D ; ME ##D S ##OC ##IE ##TY ) [SEP]',
  'our_cross_cui': 'C1708333',
  'our_cross_cui_name': 'Health Care Organization',
  'our_cross_cui_desc': '[CLS] Health Care Organization [unused3] ( Organization : Healthcare Organization ) [SEP]'},
 {'mention_id': '27526663.53',
  'test_set_idx': 9493,
  'type': 'T204',
  'name': 'B. cinerea',
  'context': '[CLS] to the j ##as ##mon ##ic acid defence pathway . We found that Pro ##D ##H1 and Pro ##D ##H ##2 are both necessary to achieve maximum resistance against P ##st - A ##v ##r ##R ##pm ##1 and B . c ##iner ##ea . However , Pro ##D ##H ##2 has a major effect on early restriction of [unused1] B . c ##iner ##ea [unused2] growth . Interest ##ingly , Pro ##D ##H1 is up - regulated by sa ##lic ##yl ##ic acid and j ##as ##mon ##ic acid , whereas Pro ##D ##H ##2 is only activated by j ##as ##mon ##ic acid , and both genes display transcription ##al inter - regulation at basal and infection conditions . These studies provide the first [SEP]',
  'existing_cui': 'C0326708',
  'existing_cui_name': 'Callaeas cinereus',
  'existing_cui_desc': '[CLS] Call ##ae ##as c ##iner ##eus [unused3] ( E ##uka ##ryo ##te , organism : Call ##ae ##as c ##iner ##eus ( G ##mel ##in , J ##F , 1788 ) ; G ##lau ##co ##pis c ##iner ##ea c ##iner ##ea ; Watt ##led crow ; Ko ##ka ##ko ; k ##oka ##ko ; Call ##ae ##as c ##iner ##ea ) [SEP]',
  'our_bi_cui': 'C0317886',
  'our_bi_cui_name': 'Neisseria cinerea',
  'our_bi_cui_desc': '[CLS] N ##ei ##sser ##ia c ##iner ##ea [unused3] ( Ba ##cter ##ium , organism : N ##ei ##sser ##ia c ##iner ##ea ( von Ling ##els ##heim 1906 ) Murray 1939 ; Micro ##co ##cc ##us c ##iner ##eus ; NE ##IS ##SE ##RI ##A C ##IN ##ER ##EA ) [SEP]',
  'our_cross_cui': 'C0579190',
  'our_cross_cui_name': 'Botrytis cinerea',
  'our_cross_cui_desc': '[CLS] Bo ##try ##tis c ##iner ##ea [unused3] ( E ##uka ##ryo ##te , organism : Sc ##ler ##ot ##inia fuck ##eli ##ana ) [SEP]'},
 {'mention_id': '27838742.32',
  'test_set_idx': 18999,
  'type': 'T103',
  'name': 'delicaflavone',
  'context': '[CLS] in male BA ##L ##B / c nude mice without o ##bs ##er ##vable adverse effect . By transmission electron micro ##sco ##py with a ##c ##rid ##ine orange and Cy ##to - ID ® Auto ##pha ##gy detection dye ##s , Western b ##lot analysis , and R ##T - PC ##R ass ##ay , we confirmed that [unused1] del ##ica ##f ##lav ##one [unused2] induce ##s auto ##pha ##gic cell death by increasing the ratio of L ##C ##3 - II to L ##C ##3 - I , which are auto ##pha ##gy - related proteins , and promoting the generation of acidic ve ##si ##cular organ ##elles and auto ##ly ##so ##some ##s in the c ##yt ##op ##las ##m of human lung [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0529351',
  'our_bi_cui_name': 'darifenacin',
  'our_bi_cui_desc': '[CLS] da ##ri ##fen ##ac ##in [unused3] ( Chemical , medication , product , substance : Dar ##ife ##nac ##in ##um ; ( S ) - 1 - ( 2 - ( 2 , 3 - di ##hy ##dr ##o - 5 - Benz ##of ##ura ##ny ##l ) et ##hyl ) - alpha , alpha - dip ##hen ##yl - 3 - p ##yr ##rol ##id ##ine ##ace ##tam ##ide ; Dar ##ife ##nac ##in ; ( S ) - 1 - ( 2 - ( 2 , 3 - di ##hy ##dr ##o - 5 - ben ##zo ##fu ##ran ##yl ) et ##hyl ) - alpha , alpha - dip ##hen ##yl - 3 - p ##yr ##rol ##id ##ine ##ace ##tam ##ide ; [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27475696.21',
  'test_set_idx': 7484,
  'type': 'T038',
  'name': 'OSCCs',
  'context': '[CLS] normal oral m ##uc ##osa ( n = 5 ) , oral precursor lesions ( simple h ##yper ##p ##lasia , n = 11 ; sq ##ua ##mous in ##tra ##ep ##ith ##eli ##al neo ##p ##lasia , S ##IN I - III , n = 35 ) , and oral sq ##ua ##mous cell car ##cin ##oma specimen . [unused1] OS ##CC ##s [unused2] were s ##tra ##ti ##fied in a young female ( n = 7 ) study co ##hor ##t and older patients ( n = 46 ) . In the young female study co ##hor ##t three patients ( n = 3 / 7 ) developed oral sq ##ua ##mous cell car ##cin ##oma during or shortly after pregnancy . B ##rea [SEP]',
  'existing_cui': 'C1416926',
  'existing_cui_name': 'LSS gene',
  'existing_cui_desc': '[CLS] L ##SS gene [unused3] ( Ana ##tom ##ical Structure : LA ##N ##OS ##TE ##RO ##L S ##Y ##NT ##HA ##SE ; 2 , 3 - O ##X ##ID ##OS ##Q ##U ##AL ##EN ##E - LA ##N ##OS ##TE ##RO ##L C ##Y ##CL ##AS ##E ; la ##nos ##tero ##l s ##ynth ##ase ( 2 , 3 - o ##xi ##dos ##qua ##lene - la ##nos ##tero ##l c ##y ##c ##lase ) ) [SEP]',
  'our_bi_cui': 'C0151546',
  'our_bi_cui_name': 'Oral Cavity Carcinoma',
  'our_bi_cui_desc': '[CLS] Oral C ##avi ##ty Car ##cin ##oma [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis : Oral car ##cin ##oma ##s ; Oral car ##cin ##oma ; Mouth Car ##cin ##oma ; car ##cin ##oma oral ; Car ##cin ##oma mouth ; oral cavity cancer ; Mouth car ##cin ##oma ; car ##cin ##oma ##s oral ; car ##cin ##oma cavity oral ; Car ##cin ##oma of Oral C ##avi ##ty ; oral car ##cin ##oma ; Car ##cin ##oma of Mouth ; Car ##cin ##oma of the Oral C ##avi ##ty ; Car ##cin ##oma of mouth ; car ##cin ##oma of oral cavity ) [SEP]',
  'our_cross_cui': 'C0585362',
  'our_cross_cui_name': 'Squamous cell carcinoma of mouth',
  'our_cross_cui_desc': '[CLS] S ##qua ##mous cell car ##cin ##oma of mouth [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder : Oral C ##avi ##ty S ##qua ##mous Cell Car ##cin ##oma ; Mouth SC ##C ; sq ##ua ##mous cell car ##cin ##oma of mouth ; sq ##ua ##mous cell car ##cin ##oma mouth ; SC ##C of Mouth ; car ##cin ##oma cell mouth sq ##ua ##mous ; S ##qua ##mous cell car ##cin ##oma of the oral cavity ; S ##qua ##mous Cell Car ##cin ##oma of the Oral C ##avi ##ty ; SC ##C of the Mouth ; Oral C ##avi ##ty SC ##C ; SC ##C of Oral C ##avi ##ty ; Mouth S ##qua ##mous Cell Car ##cin ##oma [SEP]'},
 {'mention_id': '27920749.46',
  'test_set_idx': 21883,
  'type': 'T170',
  'name': 'emotionally negative film clip',
  'context': "[CLS] healthy young men and women participated . Part ##ici ##pants ' trait re ##ap ##pra ##isa ##l and suppression as well as state and trait anxiety were assessed . A dot - probe task was completed by all participants . Statistical tests in Study 1 revealed that participants who re ##ap ##pra ##ised to decrease negative emotions while viewing an [unused1] emotionally negative film clip [unused2] had reduced attention ##al bias to subsequently presented angry faces compared to participants who re ##ap ##pra ##ised to increase negative emotions . Multiple re ##gression analyses in Study 2 revealed that trait re ##ap ##pra ##isa ##l predicted slower or ##ient ##ing toward angry faces , whereas state anxiety predicted slower di ##sen ##ga ##gement from angry faces . [SEP]",
  'existing_cui': 'C0558263',
  'existing_cui_name': 'Emotionally distant',
  'existing_cui_desc': '[CLS] Em ##otion ##ally distant [unused3] ( Finding , finding : Em ##otion ##ally al ##oof ) [SEP]',
  'our_bi_cui': 'C3463807',
  'our_bi_cui_name': 'Video Media',
  'our_bi_cui_desc': '[CLS] Video Media [unused3] ( Intel ##lect ##ual Product : Video ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27621200.3',
  'test_set_idx': 12035,
  'type': 'T038',
  'name': 'kindness',
  'context': '[CLS] A pilot study exploring the relationship between self - compassion , self - judgement , self - [unused1] kindness [unused2] , compassion , professional quality of life and well ##bei ##ng among UK community nurses Co ##mp ##ass ##ion fatigue and burn ##out can impact on performance of nurses . This paper explores the relationship between self - compassion , self - judgement , self - kindness , compassion , professional quality of life , and well ##bei ##ng among community nurses . To measure associations between self - compassion , compassion fatigue , well ##bei ##ng , and burn ##out in community nurses . Q ##uant ##itative data were collected using standard ##ised ps ##ych ##ometric question ##naire ##s : ( 1 ) Professional Quality [SEP]',
  'existing_cui': 'C0456909',
  'existing_cui_name': 'Blind Vision',
  'existing_cui_desc': '[CLS] Blind Vision [unused3] ( Finding , finding : Blind ##ness ; blind vision ; Unable to see ; Blind ##ness [ Disease / Finding ] ; Can not see ; Blind ; blind ##ness ; U ##TS - Unable to see ; blind ) [SEP]',
  'our_bi_cui': 'C0020039',
  'our_bi_cui_name': 'Hostility',
  'our_bi_cui_desc': '[CLS] Host ##ility [unused3] ( B ##iol ##og ##ic Fun ##ction , context - dependent category , finding , physical finding , situation , s ##ym ##pt ##om : [ D ] Host ##ility ; attitude hostile ; Host ##ile behavior ; Host ##ile behaviour ; hostile ; hostilities ; hostility ; Host ##ilities ) [SEP]',
  'our_cross_cui': 'C0871641',
  'our_cross_cui_name': 'affection',
  'our_cross_cui_desc': '[CLS] affection [unused3] ( B ##iol ##og ##ic Fun ##ction : A ##ffe ##ction ; affection ##s ) [SEP]'},
 {'mention_id': '27402827.56',
  'test_set_idx': 4900,
  'type': 'T038',
  'name': 'spinogenesis',
  'context': '[CLS] G - protein in ##hibition a ##bro ##gate ##s W ##nt - 5 ##a - dependent pathway in hip ##po ##cam ##pal neurons . In particular , the activation of G ##α ##o appears to be a key factor controlling the W ##nt - 5 ##a - induced den ##dr ##itic spine density . In addition , we found that G ##β ##γ is required for the W ##nt - 5 ##a - mediated increase in c ##yt ##oso ##lic calcium levels and [unused1] spin ##ogen ##esis [unused2] . Our findings reveal that Fr ##iz ##zled ##9 and he ##tero ##tri ##mer ##ic G proteins regulate W ##nt - 5 ##a signaling and den ##dr ##itic spines in culture ##d hip ##po ##cam ##pal neurons . [SEP]',
  'existing_cui': 'C2262776',
  'existing_cui_name': 'retina morphogenesis in camera-type eye',
  'existing_cui_desc': '[CLS] re ##tina m ##or ##ph ##ogen ##esis in camera - type eye [unused3] ( B ##iol ##og ##ic Fun ##ction : re ##tina m ##or ##ph ##ogen ##esis in camera - style eye ; re ##tino ##genesis ) [SEP]',
  'our_bi_cui': 'C1155781',
  'our_bi_cui_name': 'spindle assembly',
  'our_bi_cui_desc': '[CLS] spin ##dle assembly [unused3] ( B ##iol ##og ##ic Fun ##ction : spin ##dle bio ##sy ##nt ##hesis ; spin ##dle formation ; bi ##pol ##ar spin ##dle bio ##sy ##nt ##hesis ; bi ##pol ##ar spin ##dle formation ) [SEP]',
  'our_cross_cui': 'C0007613',
  'our_cross_cui_name': 'Cell physiology',
  'our_cross_cui_desc': '[CLS] Cell physiology [unused3] ( B ##iol ##og ##ic Fun ##ction : cell physiology ; Cell ##ular Process ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##a ; Cell ##ular Fun ##ction ; cells physiology ; Cell ##ular Ph ##ys ##iology ; Sub ##cellular Process ; cells function ; cell function ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##on ; Cell Ph ##ys ##iology ; Cell Process ; Cell Fun ##ction ; CE ##LL P ##H ##Y ##SI ##OL ; P ##H ##Y ##SI ##OL CE ##LL ; cell functions ; Ph ##eno ##men ##a , Cell Ph ##ys ##iol ##ogical ; Ph ##ys ##iology , Cell ; Ph ##eno ##men ##on , Cell Ph ##ys ##iol ##ogical ) [SEP]'},
 {'mention_id': '27796974.8',
  'test_set_idx': 17362,
  'type': 'T170',
  'name': 'Water Framework Directive',
  'context': '[CLS] ##ive aims at reaching the good ecological status of the surface and ground water bodies ( L ##ás ##z ##l ##ó et al . Micro ##che ##m J 85 ( 1 ) : 65 - 71 , 2007 ) . The paper deals with quality evaluation of waters with special focus on the water chemistry parameters as defined in the [unused1] Water Framework Direct ##ive [unused2] and pertaining legal regulations . The purpose of this paper is to de ##vise a quantitative type of water quality assessment method which could provide rapid , accurate , and reliable information on the quality of the surface waters by using water chemistry parameters . Quality classes have been defined for every water chemistry parameter in light of the legal [SEP]',
  'existing_cui': 'C0450152',
  'existing_cui_name': 'Metal framework',
  'existing_cui_desc': '[CLS] Metal framework [unused3] ( Medical Devi ##ce , physical object ) [SEP]',
  'our_bi_cui': 'C1135941',
  'our_bi_cui_name': 'Ethical Directives',
  'our_bi_cui_desc': '[CLS] E ##thical Direct ##ives [unused3] ( Intel ##lect ##ual Product : Direct ##ive , E ##thical ; Direct ##ives , E ##thical ; E ##thical Direct ##ive ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27894020.30',
  'test_set_idx': 21139,
  'type': 'T033',
  'name': 'detection',
  'context': '[CLS] , cocaine and its main meta ##bol ##ite , ben ##zo ##yle ##c ##gon ##ine , were detected in two out of six hair samples . The concentrations for cocaine were 0 . 151 and 0 . 09 ng / mg and for ben ##zo ##yle ##c ##gon ##ine 0 . 103 and 0 . 147 ng / mg , respectively . Given that cocaine was a commonly used medicine , beginning in the mid - 19th century and per ##sist ##ing into the 20th century , it is not surprising that some patients may have ing ##ested this drug . In addition to the [unused1] detection [unused2] of cocaine , these analyses also provided evidence of ni ##cot ##ine and ca ##ffe ##ine intake . [SEP]',
  'existing_cui': 'C0206100',
  'existing_cui_name': 'Signal Detection',
  'existing_cui_desc': '[CLS] Signal Det ##ec ##tion [unused3] ( B ##iol ##og ##ic Fun ##ction , Psychology , Psychology , Signal : Signal Det ##ec ##tion Theo ##ries ; Signal Det ##ec ##tions , Psychological ; Theory , Signal Det ##ec ##tion ; Signal detection ; S ##IG ##NA ##L DE ##TE ##CT ##ION AN ##AL ; Psychological Signal Det ##ec ##tions ; Det ##ec ##tion ; Signal Det ##ec ##tion Ana ##ly ##ses ; S ##IG ##NA ##L DE ##TE ##CT ##ION ; Det ##ec ##tions , Psychological Signal ; Psychological Signal Det ##ec ##tion ; Signal Det ##ec ##tion Theory ; Theo ##ries , Signal Det ##ec ##tion ; Signal Det ##ec ##tions ; Ana ##ly ##ses , Signal Det ##ec ##tion ; Det ##ec ##tion , [SEP]',
  'our_bi_cui': 'C1511790',
  'our_bi_cui_name': 'Detection',
  'our_bi_cui_desc': '[CLS] Det ##ec ##tion [unused3] ( Health Care Act ##ivity : Det ##ec ##ted ) [SEP]',
  'our_cross_cui': 'C0442726',
  'our_cross_cui_name': 'Detected',
  'our_cross_cui_desc': '[CLS] Det ##ec ##ted [unused3] ( Finding , finding , finding , qualifier value : detected ; detect ) [SEP]'},
 {'mention_id': '27325393.44',
  'test_set_idx': 2493,
  'type': 'T038',
  'name': 'immune',
  'context': '[CLS] ##9 , p = 0 . 03 ##5 ) , M ( β = - 0 . 01 ##7 , p = 0 . 00 ##9 ) and F ( β = - 0 . 03 ##1 , p = 0 . 02 ##4 ) were associated with the T ##has ##e and Rush staging model but not the Maud ##sley St ##aging Model . Overall , our findings suggest that proteins , which are involved in [unused1] immune [unused2] and complement activation , may represent potential bio ##mark ##ers that could be used by clinic ##ians to identify high - risk patients . Nevertheless , given that the molecular changes between the staging groups were subtle , the results need to be interpreted cautiously . [SEP]',
  'existing_cui': 'C0073433',
  'existing_cui_name': 'RNA , immune',
  'existing_cui_desc': '[CLS] RNA , immune [unused3] ( Chemical : immune RNA ; I - RNA ) [SEP]',
  'our_bi_cui': 'C0301872',
  'our_bi_cui_name': 'Immune response',
  'our_bi_cui_desc': '[CLS] I ##mm ##une response [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : im ##mu ##nor ##es ##po ##nse ; immune responses ; immune response ; I ##mm ##une response , un ##sp ##ec ##ified ; I ##mm ##une Response Process ; Host response ; I ##mm ##une response , function ; Host response , function ; I ##mm ##une Response ; host response ) [SEP]',
  'our_cross_cui': 'C1155000',
  'our_cross_cui_name': 'Immune Cell Activation Process',
  'our_cross_cui_desc': '[CLS] I ##mm ##une Cell Act ##ivation Process [unused3] ( B ##iol ##og ##ic Fun ##ction : le ##uk ##oc ##yte activation ; immune cell activation ; I ##mm ##une Cell Act ##ivation ; le ##uc ##oc ##yte activation ) [SEP]'},
 {'mention_id': '27932585.26',
  'test_set_idx': 22253,
  'type': 'T103',
  'name': 'TYW3',
  'context': '[CLS] structure of Ta ##w ##3 , a ho ##mo ##log of T ##Y ##W ##3 from Sul ##fo ##lo ##bus so ##lf ##ata ##ric ##us , which revealed a novel α / β fold . The sequence motif ( S / T ) x ##SS ##C ##x ##GR and invariant as ##par ##tate and his ##ti ##dine , conserved in [unused1] T ##Y ##W ##3 [unused2] / Ta ##w ##3 , cluster to form the cat ##alytic center . These structural and sequence features indicate that T ##Y ##W ##3 / Ta ##w ##3 proteins constitute a distinct class of SA ##M - dependent met ##hyl ##tra ##ns ##fer ##ases . Using site - directed m ##uta ##genesis along with in v ##ivo complement ##ation ass [SEP]',
  'existing_cui': 'C1823563',
  'existing_cui_name': 'TYW3 gene',
  'existing_cui_desc': '[CLS] T ##Y ##W ##3 gene [unused3] ( Ana ##tom ##ical Structure : FL ##J ##40 ##9 ##18 ; t ##RNA - Y ##W - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; t ##RNA - y ##W s ##ynth ##esi ##zing protein 3 ho ##mo ##log ; t ##RNA - W ##Y ##B ##UT ##OS ##IN ##E - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ) [SEP]',
  'our_bi_cui': 'C1823563',
  'our_bi_cui_name': 'TYW3 gene',
  'our_bi_cui_desc': '[CLS] T ##Y ##W ##3 gene [unused3] ( Ana ##tom ##ical Structure : FL ##J ##40 ##9 ##18 ; t ##RNA - Y ##W - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; t ##RNA - y ##W s ##ynth ##esi ##zing protein 3 ho ##mo ##log ; t ##RNA - W ##Y ##B ##UT ##OS ##IN ##E - S ##Y ##NT ##H ##ES ##I ##Z ##ING PR ##OT ##EI ##N 3 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28243201.4',
  'test_set_idx': 29646,
  'type': 'T201',
  'name': 'Response',
  'context': '[CLS] I ##dent ##ification of Pre ##dict ##ive DNA Met ##hyl ##ation B ##io ##mark ##ers for Ch ##em ##otherapy [unused1] Response [unused2] in Color ##ec ##tal Cancer Resistance to 5 - F ##lu ##oro ##ura ##ci ##l is a major obstacle to the successful treatment of color ##ec ##tal cancer and posed an increased risk of re ##cu ##rrence . DNA met ##hyl ##ation has been suggested as one of the underlying mechanisms for re ##current disease and its contribution to the development of drug resistance remains to be clarified . This study aimed to determine the met ##hyl ##ation p ##hen ##otype in color ##ec ##tal cancer for identification of predict ##ive markers for ch ##em ##otherapy response . We performed DNA met ##hyl ##ation [SEP]',
  'existing_cui': 'C1704632',
  'existing_cui_name': 'Disease Response',
  'existing_cui_desc': '[CLS] Disease Response [unused3] ( Finding : Response ; response ) [SEP]',
  'our_bi_cui': 'C0683154',
  'our_bi_cui_name': 'drug response',
  'our_bi_cui_desc': '[CLS] drug response [unused3] ( B ##iol ##og ##ic Fun ##ction : response to drug ; drug responses ; drugs response ) [SEP]',
  'our_cross_cui': 'C0521982',
  'our_cross_cui_name': 'Response to treatment',
  'our_cross_cui_desc': '[CLS] Response to treatment [unused3] ( Clinical At ##tri ##but ##e , context - dependent category , finding , function , o ##bs ##er ##vable entity , situation : response to treatment ; response treatments ; The ##rap ##eu ##tic response ; response therapeutic ; The ##rap ##eu ##tic Response ; Finding related to therapeutic response ; successful treatment ; successful treatments ; The ##rap ##eu ##tic response , function ) [SEP]'},
 {'mention_id': '27249559.30',
  'test_set_idx': 788,
  'type': 'T033',
  'name': 'electron beam trajectory swinging',
  'context': '[CLS] size were studied . The simulation results demonstrated that if the intensity of X - rays generated at the point - like target is greater than half of the X - ray intensity produced on the substrate , the X - ray focal spot is determined in part by the point - like target rather than by the electron beam in the conventional X - ray tube . In theory , since it is able to reduce those un ##fa ##vor ##able effects such as the [unused1] electron beam trajectory swinging [unused2] and the beam size changing for the micro ##fo ##cus X - ray tube , it could alleviate CT image artifacts caused by the X - ray focal spot shift and size change . [SEP]',
  'existing_cui': 'C0454077',
  'existing_cui_name': 'Teleradiotherapy using electrons',
  'existing_cui_desc': '[CLS] Tel ##era ##dio ##ther ##ap ##y using electrons [unused3] ( Health Care Act ##ivity , procedure : Beta ##tron electron te ##let ##her ##ap ##y ; photo ##n beam radiation therapy ; Tel ##era ##dio ##ther ##ap ##y using beta particles ; Tel ##era ##dio ##ther ##ap ##y beta particles ; El ##ec ##tron Be ##am Therapy ; El ##ec ##tron te ##let ##her ##ap ##y ; El ##ec ##tron beam therapy ; El ##ec ##tron te ##ler ##adi ##oth ##era ##p ) [SEP]',
  'our_bi_cui': 'C0600352',
  'our_bi_cui_name': 'Electron Beam Computed Tomography',
  'our_bi_cui_desc': '[CLS] El ##ec ##tron Be ##am Co ##mp ##uted Tom ##ography [unused3] ( Health Care Act ##ivity , also known as ultra ##fast c ##t , c ##ine c ##t : beam com ##puted electron to ##mo ##graphy ; El ##ec ##tron beam com ##puted to ##mo ##graphy ; Be ##am Tom ##ography , El ##ec ##tron ; El ##ec ##tron beam com ##puted to ##mo ##g ; Tom ##ography , El ##ec ##tron Be ##am ; El ##ec ##tron Be ##am Tom ##ography ; electron beam to ##mo ##graphy ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27760149.2',
  'test_set_idx': 15789,
  'type': 'T170',
  'name': 'Biomedical Entity Search Tool',
  'context': '[CLS] B ##ES ##T : Next - Generation [unused1] B ##io ##medical En ##ti ##ty Search Too ##l [unused2] for Knowledge Discovery from B ##io ##medical Literature As the volume of publications rapidly increases , searching for relevant information from the literature becomes more challenging . To complement standard search engines such as Pub ##M ##ed , it is desirable to have an advanced search tool that directly returns relevant bio ##medical entities such as targets , drugs , and mutations rather than a long list of articles . Some existing tools submit a que ##ry to Pub ##M ##ed and process retrieved abstract ##s to extract information at que ##ry time , resulting in a slow response time and limited coverage of only a fraction of [SEP]',
  'existing_cui': 'C2348167',
  'existing_cui_name': 'Search Engine',
  'existing_cui_desc': '[CLS] Search Engine [unused3] ( Intel ##lect ##ual Product : Search Program ; Search engines ; Engine , Search ; Search Too ##l ; Search Engine ##s ) [SEP]',
  'our_bi_cui': 'C2348167',
  'our_bi_cui_name': 'Search Engine',
  'our_bi_cui_desc': '[CLS] Search Engine [unused3] ( Intel ##lect ##ual Product : Search Program ; Search engines ; Engine , Search ; Search Too ##l ; Search Engine ##s ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '27510458.36',
  'test_set_idx': 8923,
  'type': 'T033',
  'name': 'double lumen sign',
  'context': '[CLS] double l ##ume ##n sign in the participants and compared the results between high - resolution fast spin echo and the other m ##oda ##lities . high - resolution fast spin echo images clearly showed in ##ti ##mal flap ##s and double l ##ume ##n signs in all 3 patients , whereas the conventional m ##oda ##lities identified a [unused1] double l ##ume ##n sign [unused2] in only 2 of the 3 patients . The present method of op ##ti ##mized high - resolution fast spin echo imaging with a 3 ##T system improved visual ##ization of in ##ti ##mal flap ##s and should thus be considered for assessing patients with suspected in ##tra ##c ##rani ##al artery di ##sse ##ction that can not be definitive [SEP]',
  'existing_cui': 'C0441299',
  'existing_cui_name': 'Double lumen tube',
  'existing_cui_desc': '[CLS] Double l ##ume ##n tube [unused3] ( Medical Devi ##ce , physical object ) [SEP]',
  'our_bi_cui': 'C0233192',
  'our_bi_cui_name': "Ladin 's sign",
  'our_bi_cui_desc': "[CLS] La ##din ' s sign [unused3] ( Finding , finding : La ##din sign ) [SEP]",
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27488290.9',
  'test_set_idx': 8126,
  'type': 'T058',
  'name': 'microendoscopic techniques',
  'context': '[CLS] ##izing In ##tra ##oper ##ative O - Arm - Based Navigation and Micro ##end ##os ##copic Tech ##ni ##ques A retrospective analysis of the clinical outcomes of eight patients with the l ##umba ##r par ##s inter ##art ##icular ##s defects treated by direct repair with the aid of in ##tra ##oper ##ative O - arm based navigation and [unused1] micro ##end ##os ##copic techniques [unused2] . The aim of this study was to investigate the efficacy and safety of direct screw repair by using minimal ##ly invasive surgery for the l ##umba ##r par ##s inter ##art ##icular ##s defects in a pilot study . Direct repair of par ##s inter ##art ##icular ##s defects has been used to treat young adult patients . Reports [SEP]',
  'existing_cui': 'C0441702',
  'existing_cui_name': 'Superior pedicle technique',
  'existing_cui_desc': '[CLS] Superior p ##edic ##le technique [unused3] ( Health Care Act ##ivity , qualifier value : March ##ac technique ) [SEP]',
  'our_bi_cui': 'C0026035',
  'our_bi_cui_name': 'Microsurgery',
  'our_bi_cui_desc': '[CLS] Micro ##su ##rger ##y [unused3] ( Health Care Act ##ivity , procedure : micro ##su ##rger ##y ; micro ##su ##rger ##ies ; Micro ##su ##rg ##ical procedure ; MI ##CR ##OS ##UR ##G ) [SEP]',
  'our_cross_cui': 'C0014245',
  'our_cross_cui_name': 'Endoscopy',
  'our_cross_cui_desc': '[CLS] End ##os ##copy [unused3] ( Health Care Act ##ivity , procedure , procedure : end ##os ##copy procedure ; end ##os ##copy ; end ##os ##copic examination ; procedure end ##os ##copy ; end ##os ##copic / fiber ##op ##tic examinations ; In ##spect ##ion using end ##os ##cope ; End ##os ##copic ; End ##os ##copic inspection ; End ##os ##copic Examination ; E ##ND ##OS ##CO ##P ##Y ; End ##os ##copy procedure ; end ##os ##co ##pies ; End ##os ##copic Pro ##ced ##ure ) [SEP]'},
 {'mention_id': '27602300.7',
  'test_set_idx': 11297,
  'type': 'T082',
  'name': 'Eurasia',
  'context': "[CLS] The add ##er ( Viper ##a be ##rus ) in Southern Alt ##ay Mountains : population characteristics , distribution , morphology and phylogenetic position As the most widely distributed snake in [unused1] E ##ura ##sia [unused2] , the add ##er ( Viper ##a be ##rus ) has been extensively investigated in Europe but poorly understood in Asia . The Southern Alt ##ay Mountains represent the add ##er ' s southern distribution limit in Central Asia , whereas its population status has never been assessed . We conducted , for the first time , field surveys for the add ##er at two areas of Southern Alt ##ay Mountains using a combination of line trans ##ec ##ts and random searches . We also described the m ##or ##phological [SEP]",
  'existing_cui': 'C0450981',
  'existing_cui_name': 'Addiction severity index',
  'existing_cui_desc': '[CLS] Ad ##diction severity index [unused3] ( Intel ##lect ##ual Product , AS ##I , assessment scale : addiction severity index ) [SEP]',
  'our_bi_cui': 'C0015176',
  'our_bi_cui_name': 'Europe',
  'our_bi_cui_desc': '[CLS] Europe [unused3] ( Spa ##tial Concept , geographic location ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '27782044.3',
  'test_set_idx': 16573,
  'type': 'T170',
  'name': 'Comments on Beamer et al . Association',
  'context': "[CLS] Normal Level ##s of U ##rina ##ry CC ##16 Pro ##tein . [unused1] Co ##mme ##nts on Be ##ame ##r et al . Association [unused2] of Children ' s U ##rina ##ry CC ##16 Level ##s with A ##rsen ##ic Con ##cent ##rations in Multiple Environmental Media . In ##t . J . En ##vir ##on . Re ##s . Public Health 2016 , 13 , 52 ##1 In 1937 , Max Clara described a new type of cell in the human lung , which was later determined to be an ex ##oc ##rine secret ##ory cell type containing g ##ran ##ules composed of proteins [ 1 ] . [ . . . ] . [SEP]",
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0282411',
  'our_bi_cui_name': 'Published Comment',
  'our_bi_cui_desc': '[CLS] Published Co ##mme ##nt [unused3] ( Intel ##lect ##ual Product : viewpoint ; comment ; Co ##mme ##nts ; View ##point ; commentary ; Re ##mark ; Re ##marks ; viewpoint ##s ; Co ##mme ##ntary ; comments ; Editorial Co ##mme ##nt ; Co ##mme ##nt ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28387835.66',
  'test_set_idx': 34002,
  'type': 'T058',
  'name': 'MLVA',
  'context': '[CLS] reveal differences in response between the M ##L ##VA g ##eno ##type ##s . The pro ##in ##f ##lam ##mat ##ory c ##yt ##oki ##ne response induced in human peripheral blood mon ##on ##uc ##lea ##r cells by C . burn ##eti ##i is ##olate ##s from cattle may explain the low incidence of human Q fever outbreak ##s caused by cattle . The c ##yt ##oki ##ne profile of peripheral blood mon ##on ##uc ##lea ##r cells stimulate ##d with C . burn ##eti ##i is ##olate ##s from chronic Q fever patients resembles is ##olate ##s from goats . Furthermore , c ##yt ##oki ##ne responses seem to be depending on host origin than on [unused1] M ##L ##VA [unused2] g ##eno ##type . [SEP]',
  'existing_cui': 'C1420126',
  'existing_cui_name': 'SLC20A2 gene',
  'existing_cui_desc': '[CLS] SL ##C ##20 ##A ##2 gene [unused3] ( Ana ##tom ##ical Structure : S ##OL ##UT ##E CA ##R ##RI ##ER FA ##MI ##L ##Y 20 ( P ##H ##OS ##P ##HA ##TE T ##RA ##NS ##PO ##RT ##ER ) , ME ##MB ##ER 2 ; G ##IB ##BO ##N AP ##E L ##E ##U ##KE ##MI ##A R ##ET ##RO ##VI ##R ##US R ##EC ##EP ##TO ##R 2 ; G ##l ##v ##r - 2 ; P ##H ##OS ##P ##HA ##TE T ##RA ##NS ##PO ##RT ##ER , S ##OD ##I ##UM - DE ##P ##EN ##DE ##NT , 2 ; Pi ##T - 2 ; M ##UR ##IN ##E L ##E ##U ##KE ##MI ##A VI ##R ##US , AM ##P ##H [SEP]',
  'our_bi_cui': 'C0042317',
  'our_bi_cui_name': 'vanilmandelic acid',
  'our_bi_cui_desc': '[CLS] van ##il ##mand ##eli ##c acid [unused3] ( Chemical , V ##MA , substance : A ##cid , Van ##il ##mand ##eli ##c ; van ##ill ##yl ##mand ##eli ##c acid ; A ##cid , Van ##ill ##yl ##mand ##eli ##c ; A ##cid , Met ##ho ##xy ##hy ##dr ##ox ##yman ##del ##ic ; Benz ##ene ##ace ##tic acid , alpha , 4 - di ##hy ##dr ##ox ##y - 3 - met ##ho ##xy - ; 4 H ##ydro ##xy 3 Met ##ho ##xy ##mand ##eli ##c A ##cid ; Van ##ill ##yl ##mand ##eli ##c acid ; Van ##il ##mand ##eli ##c A ##cid ; V ##MA ; Van ##il ##mand ##eli ##c A ##cid [ Chemical / In ##g ##red ##ient ] [SEP]',
  'our_cross_cui': 'C1443925',
  'our_cross_cui_name': 'Multi-locus variable-number of tandem repeat analysis',
  'our_cross_cui_desc': '[CLS] Multi - lo ##cus variable - number of tandem repeat analysis [unused3] ( Health Care Act ##ivity , procedure : Multi - lo ##cus V ##NT ##R analysis ; M ##L ##VA - multi - lo ##cus variable - number of tandem repeat analysis ) [SEP]'},
 {'mention_id': '27488290.55',
  'test_set_idx': 8172,
  'type': 'T033',
  'name': 'mild complaint',
  'context': '[CLS] Low back pain visual analog scores decrease from pre ##oper ##ative 6 . 8 to post ##oper ##ative 1 . 4 of eight cases . Of 16 par ##s defects , healing was observed in 13 ( 81 . 3 % ) at last radio ##logical follow - up . One patient with bilateral fusion failure refused revision surgery because of [unused1] mild complaint [unused2] . No complications such as du ##ral tear , nerve root injury , and infection occurred . Mini ##mal ##ly invasive direct repair of the par ##s inter ##art ##icular ##is defects with in ##tral ##ami ##nar screw ##s by using micro ##end ##os ##copic system and navigation procedure can provide safe and effective treatment of s ##po ##ndy ##lo ##lysis [SEP]',
  'existing_cui': 'C0277786',
  'existing_cui_name': 'Chief complaint',
  'existing_cui_desc': '[CLS] Chief complaint [unused3] ( Finding , finding , finding , s ##ym ##pt ##om : complaints ; chief complaint ; Co ##mp ##lain ##t ; complaints present ; Present ##ing Co ##mp ##lain ##t ; complaint ; { Chief complaint } ; Present ##ing complaint ; Chief Co ##mp ##lain ##t ; the chief complaint is : ) [SEP]',
  'our_bi_cui': 'C0277786',
  'our_bi_cui_name': 'Chief complaint',
  'our_bi_cui_desc': '[CLS] Chief complaint [unused3] ( Finding , finding , finding , s ##ym ##pt ##om : complaints ; chief complaint ; Co ##mp ##lain ##t ; complaints present ; Present ##ing Co ##mp ##lain ##t ; complaint ; { Chief complaint } ; Present ##ing complaint ; Chief Co ##mp ##lain ##t ; the chief complaint is : ) [SEP]',
  'our_cross_cui': 'C0436343',
  'our_cross_cui_name': 'Symptom mild',
  'our_cross_cui_desc': '[CLS] S ##ym ##pt ##om mild [unused3] ( Finding , context - dependent category , finding : mild symptoms ; s ##ym ##pt ##om mild ) [SEP]'},
 {'mention_id': '27297967.8',
  'test_set_idx': 1713,
  'type': 'T082',
  'name': 'Asn19/Asn23',
  'context': '[CLS] Tri ##g ##lut ##ath ##ione [ As ( G ##S ) 3 ] Transport by Multi ##dr ##ug Resistance Pro ##tein 1 ( MR ##P ##1 / ABC ##C ##1 ) Is Select ##ively Mo ##dified by Ph ##os ##ph ##ory ##lation of Ty ##r ##9 ##20 / Ser ##9 ##21 and G ##ly ##cos ##yla ##tion of [unused1] As ##n ##19 / As ##n ##23 [unused2] The ATP - binding cassette transport ##er multi ##dr ##ug resistance protein 1 ( MR ##P ##1 / ABC ##C ##1 ) is responsible for the cellular export of a chemical ##ly diverse array of x ##eno ##biotics and end ##ogen ##ous compounds . A ##rsen ##ic , a human car ##cin ##ogen , is a high - affinity [SEP]',
  'existing_cui': 'C3887642',
  'existing_cui_name': 'Hippocampus Proper',
  'existing_cui_desc': "[CLS] Hip ##po ##cam ##pus Pro ##per [unused3] ( Ana ##tom ##ical Structure : Hip ##po ##cam ##pus proper ; Hip ##po ##cam ##pus major ; Am ##mon ' s Horn ; CA fields ; Pro ##per , Hip ##po ##cam ##pus ; Horn , Am ##mon ; Hip ##po ##cam ##pus Pro ##pers ; Am ##mon Horn ; hip ##po ##cam ##pus ; Co ##rn ##u am ##mon ##is ; Am ##mons Horn ; Hip ##po ##cam ##pus ; corn ##u am ##mon ##is ; am ##mon ' s horn ; hip ##po ##cam ##pus proper ; Hip ##po ##cam ##pus prop ##rius ; Am ##mon ' s horn ; Co ##rn ##u Am ##mon ##is ; hip ##po ##cam ##pus major ; Horn , Am [SEP]",
  'our_bi_cui': 'C0003995',
  'our_bi_cui_name': 'Asparagine',
  'our_bi_cui_desc': '[CLS] As ##par ##agi ##ne [unused3] ( Chemical , substance : 2 - Am ##ino ##su ##cci ##nam ##ic acid ; L - as ##par ##agi ##ne ; As ##par ##tam ##ic acid ; alpha - amino ##su ##cci ##nam ##ic acid ; as ##n ; 2 - Am ##ino - 3 - car ##ba ##mo ##yl ##p ##rop ##ano ##ic acid ; as ##par ##agi ##ne ; ( S ) - As ##par ##agi ##ne ; N ; ( S ) - 2 - amino - 3 - car ##ba ##mo ##yl ##p ##rop ##ano ##ic acid ; Alpha - amino ##su ##cci ##nic acid ; As ##n ; L - as ##par ##tic acid beta - amid ##e ; ( 2 ##S ) - 2 [SEP]',
  'our_cross_cui': 'C0002518',
  'our_cross_cui_name': 'Amino Acid Sequence',
  'our_cross_cui_desc': '[CLS] Am ##ino A ##cid Se ##quence [unused3] ( Spa ##tial Concept , finding : Se ##quence ##s , Am ##ino A ##cid ; Am ##ino A ##cid Se ##quence ##s ; SE ##Q AM ##IN ##O AC ##ID ; Pro ##tein Se ##quence ; amino acid sequences ; SE ##Q AA ; Structure ##s , Primary Pro ##tein ; acid amino sequences ; amino acid sequence ; Primary Pro ##tein Structure ##s ; AM ##IN ##O AC ##ID SE ##Q ; Structure , Primary Pro ##tein ; AA SE ##Q ; Am ##ino acid sequence ; protein se ##quencing ; protein sequences ; Pro ##tein Structure ##s , Primary ; Se ##quence , Am ##ino A ##cid ; protein sequence ; AM ##IN ##O AC ##ID [SEP]'},
 {'mention_id': '27601292.47',
  'test_set_idx': 11222,
  'type': 'T038',
  'name': 'metabolic',
  'context': '[CLS] , reflected by C - reactive protein concentrations , significantly influence d the metabolic ratio , v ##ori ##cona ##zo ##le t ##rough concentration and v ##ori ##cona ##zo ##le - N - oxide concentration ( all P < 0 . 00 ##1 ) , when corrected for other factors that could influence v ##ori ##cona ##zo ##le metabolism . The [unused1] metabolic [unused2] ratio was decreased by 0 . 99 ##22 ##9 ( N ) and the v ##ori ##cona ##zo ##le - N - oxide concentration by 0 . 99 ##7 ##75 ( N ) , while the v ##ori ##cona ##zo ##le t ##rough concentration was increased by 1 . 00 ##53 ##21 ( N ) , where N is the difference in [SEP]',
  'existing_cui': 'C1524026',
  'existing_cui_name': 'Metabolic Process , Cellular',
  'existing_cui_desc': '[CLS] Met ##ab ##olic Process , Cell ##ular [unused3] ( B ##iol ##og ##ic Fun ##ction : Met ##ab ##oli ##zed ; Met ##ab ##olic Process ; cellular metabolism ; Met ##ab ##olic ; metabolism ; Met ##ab ##olis ##m ; cellular metabolic process ) [SEP]',
  'our_bi_cui': 'C0025519',
  'our_bi_cui_name': 'Metabolism',
  'our_bi_cui_desc': '[CLS] Met ##ab ##olis ##m [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : Inter ##media ##ry Met ##ab ##olis ##m ; Met ##ab ##olic Process ##es ; ME ##TA ##B PR ##OC ##ES ##SE ##S ; metabolic processes ; Process ##es , Met ##ab ##olic ; General metabolic function ; Met ##ab ##olic Process ; ME ##TA ##B PR ##OC ##ES ##S ; metabolism ; PR ##OC ##ES ##SE ##S ME ##TA ##B ; General metabolic functions ; Inter ##media ##ry Met ##ab ##olic Process ; metabolic process ; metabolism ##s ; Process , Met ##ab ##olic ; PR ##OC ##ES ##S ME ##TA ##B ; Met ##ab ##olis ##m [ P ##K ] ; ME ##TA ##B [SEP]',
  'our_cross_cui': 'C0683140',
  'our_cross_cui_name': 'drug metabolism',
  'our_cross_cui_desc': '[CLS] drug metabolism [unused3] ( B ##iol ##og ##ic Fun ##ction : Drug Met ##ab ##olis ##m ; drug metabolic process ; drugs metabolism ; bio ##tra ##ns ##formation drug ; bio ##tra ##ns ##formation drugs ; metabolism drug ) [SEP]'},
 {'mention_id': '27784873.10',
  'test_set_idx': 16650,
  'type': 'T017',
  'name': 'urinary bladder cells',
  'context': '[CLS] Aden ##os ##ine trip ##hos ##phate release from bladder u ##roth ##eli ##um and se ##ros ##a in a rat model of partial bladder outlet o ##bs ##truction Over ##active bladder is one of the major health problem especially in elderly people . Aden ##os ##ine trip ##hos ##phate is released from [unused1] u ##rina ##ry bladder cells [unused2] and acts as a smooth muscle contraction and sensory signal in mi ##ct ##uri ##tion but little is known about the role of Aden ##os ##ine trip ##hos ##phate release in the path ##op ##hy ##sio ##logy of over ##active bladder . To assess the relationship between Aden ##os ##ine trip ##hos ##phate and over ##active bladder , we used a partial bladder outlet o ##bs ##truction [SEP]',
  'existing_cui': 'C0005682',
  'existing_cui_name': 'Urinary Bladder',
  'existing_cui_desc': '[CLS] U ##rina ##ry B ##lad ##der [unused3] ( Ana ##tom ##ical Structure , body structure : B ##lad ##der ; B ##LA ##D ##DE ##R ; bladder ; U ##rina ##ry bladder ; u ##rina ##ry bladder ; U ##rina ##ry System , B ##lad ##der ; B ##lad ##der structure ; bladder ##s ; B ##lad ##der , U ##rina ##ry ; V ##esi ##ca u ##rina ##ria ; U ##rina ##ry bladder structure ) [SEP]',
  'our_bi_cui': 'C0427890',
  'our_bi_cui_name': 'Urinary epithelial cell',
  'our_bi_cui_desc': '[CLS] U ##rina ##ry e ##pit ##hel ##ial cell [unused3] ( Ana ##tom ##ical Structure , body structure , cell ) [SEP]',
  'our_cross_cui': 'C0227599',
  'our_cross_cui_name': 'Transitional epithelial cell of urinary bladder',
  'our_cross_cui_desc': '[CLS] Transit ##ional e ##pit ##hel ##ial cell of u ##rina ##ry bladder [unused3] ( Ana ##tom ##ical Structure , body structure , cell : U ##rina ##ry tract transitional e ##pit ##hel ##ial cell ; U ##roth ##eli ##al cell ; B ##lad ##der transitional cell ; B ##lad ##der U ##roth ##eli ##al Cell ; B ##lad ##der Transit ##ional Cell ) [SEP]'},
 {'mention_id': '28197156.27',
  'test_set_idx': 28097,
  'type': 'T038',
  'name': 'bp',
  'context': '[CLS] ##tor Pro ##tein 3 ( O ##s ##HA ##P ##3 ) subunit of a CC ##AA ##T - box - binding transcription factor called O ##s ##HA ##P ##3 ##H . Se ##quencing analysis revealed that the in ##dic ##a all ##ele of Carbon As ##si ##mi ##lation Rate 8 has a 1 - b ##p del ##eti ##on at 322 [unused1] b ##p [unused2] from the start co ##don , resulting in a t ##runcated protein of 125 amino acids . In addition , Carbon As ##si ##mi ##lation Rate 8 is identical to D ##TH ##8 / G ##h ##d ##8 / L ##HD ##1 , which was reported to control rice flowering date . The increase of A is largely due to an [SEP]',
  'existing_cui': 'C1708288',
  'existing_cui_name': 'HP wt Allele',
  'existing_cui_desc': '[CLS] HP w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : BP ; HP ##2 ##AL ##P ##HA ##2 ; Ha ##pt ##og ##lo ##bin w ##t All ##ele ; HP ##A ##1 ##S ) [SEP]',
  'our_bi_cui': 'C0028630',
  'our_bi_cui_name': 'Nucleotides',
  'our_bi_cui_desc': '[CLS] N ##uc ##leo ##tide ##s [unused3] ( Chemical , substance : n ##uc ##leo ##tide ##s ; N ##uc ##leo ##tide ; N ##uc ##leo ##tide ##s [ Chemical / In ##g ##red ##ient ] ; n ##uc ##leo ##tide ) [SEP]',
  'our_cross_cui': 'C0600436',
  'our_cross_cui_name': 'Base Pairing',
  'our_cross_cui_desc': '[CLS] Base Pa ##iring [unused3] ( B ##iol ##og ##ic Fun ##ction : base pair ; Base Pa ##iring ##s ; base pairs ; base pairing ; Base Pa ##ir ; Base Pa ##irs ) [SEP]'},
 {'mention_id': '27251800.28',
  'test_set_idx': 823,
  'type': 'T058',
  'name': 'Stoppa',
  'context': '[CLS] 3 ) post - operative complications . lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair procedure was longer than the Stop ##pa procedure ( 134 . 6 min 38 . 3 vs . 90 . 6 min 41 . 3 ; P < 0 . 05 ) . The levels of C - reactive protein were higher in the [unused1] Stop ##pa [unused2] group ( P < 0 . 05 ) but the number of le ##uc ##ocytes , ha ##ema ##to ##c ##rit , and ha ##em ##og ##lo ##bin were similar between the groups ( P > 0 . 05 ) . There was no difference in pain during the 1 s ##t and 7 th post - operative , physical functioning [SEP]',
  'existing_cui': 'C1538901',
  'existing_cui_name': 'ACD gene',
  'existing_cui_desc': '[CLS] AC ##D gene [unused3] ( Ana ##tom ##ical Structure : T ##EL ##OM ##ER ##E PR ##OT ##EI ##N T ##PP ##1 ; ad ##ren ##oc ##ort ##ical d ##ys ##p ##lasia ho ##mo ##log ; AC ##D Gene ; Ad ##ren ##oc ##ort ##ical D ##ys ##p ##lasia Ho ##mo ##log ( Mouse ) Gene ; P ##OT ##1 - IN ##TE ##RA ##CT ##ING PR ##OT ##EI ##N 1 ; P ##OT ##1 - AND T ##IN ##2 - OR ##GA ##NI ##Z ##ING PR ##OT ##EI ##N ; P ##OT ##1 and T ##IN ##2 organizing protein ; AC ##D , M ##O ##US ##E , H ##OM ##OL ##O ##G OF ; Tin ##t ##1 ; T ##IN ##2 interacting protein 1 [SEP]',
  'our_bi_cui': 'C0628086',
  'our_bi_cui_name': 'Stop-zit',
  'our_bi_cui_desc': '[CLS] Stop - z ##it [unused3] ( Chemical ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27612561.49',
  'test_set_idx': 11771,
  'type': 'T170',
  'name': 'Netherlands Trial Register',
  'context': '[CLS] less likely to report high compliance with resistance and endurance exercises in low - to - moderate intensity exercise compared to survivors of other types of cancer . The disc ##rim ##ina ##tive ability of the multi ##var ##iable models ranged from 0 . 62 to 0 . 75 . Several demographic , clinical and ps ##ych ##oso ##cial factors were associated with participation in and ad ##herence to exercise among cancer survivors . P ##sy ##cho ##so ##cial factors were more strongly associated with ad ##herence in high intensity than low - to - moderate intensity exercise . This study was registered at the [unused1] Netherlands Trial Register [unused2] [ N ##TR ##21 ##53 ] on the 5 ( th ) of January 2010 . [SEP]',
  'existing_cui': 'C0027778',
  'existing_cui_name': 'Netherlands',
  'existing_cui_desc': '[CLS] Netherlands [unused3] ( Spa ##tial Concept , geographic location : NL ; NE ##TH ##ER ##LA ##ND ##S ; net ##her ##lands ; The Netherlands ; Kingdom of the Netherlands ; NL ##D ; net ##her ##land ; ho ##lland ; Holland ) [SEP]',
  'our_bi_cui': 'C1516642',
  'our_bi_cui_name': 'Clinical Trials Database',
  'our_bi_cui_desc': '[CLS] Clinical Trials Database [unused3] ( Intel ##lect ##ual Product ) [SEP]',
  'our_cross_cui': 'C0034975',
  'our_cross_cui_name': 'Registries',
  'our_cross_cui_desc': '[CLS] Regis ##tries [unused3] ( Intel ##lect ##ual Product : re ##gis ##tries ; registry ; Regis ##try ) [SEP]'},
 {'mention_id': '27546367.50',
  'test_set_idx': 10061,
  'type': 'T038',
  'name': 'self - protective',
  'context': '[CLS] ##ath ##ological injury score ( r = 0 . 72 ##8 ; p = 0 . 02 ##6 ) and between mean a ##rgy ##rop ##hil ##ic - NO ##R number and his ##top ##ath ##ological injury score ( r = 0 . 67 ##0 ; p = 0 . 04 ##8 ) . The detection of a ##rgy ##rop ##hil ##ic - NO ##R proteins amount may be used as an indicator to obtain information about the cellular behaviour ( [unused1] self - protective [unused2] mechanism of tub ##ular e ##pit ##hel ##ial cells ) against Is ##che ##mia / re ##per ##fusion injury and cellular damage levels ( Ta ##b . 2 , Fi ##g . 4 , Re ##f . 24 ) . [SEP]',
  'existing_cui': 'C1875759',
  'existing_cui_name': 'SKIN PROTECTIVE',
  'existing_cui_desc': '[CLS] SK ##IN PR ##OT ##EC ##TI ##VE [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C0243095',
  'our_bi_cui_name': 'Finding',
  'our_bi_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]',
  'our_cross_cui': 'C0524828',
  'our_cross_cui_name': 'Cytoprotection',
  'our_cross_cui_desc': '[CLS] Cy ##top ##rote ##ction [unused3] ( B ##iol ##og ##ic Fun ##ction : Protection , Cell ; c ##yt ##op ##rote ##ction ; Cell Protection ) [SEP]'},
 {'mention_id': '28071754.3',
  'test_set_idx': 24708,
  'type': 'T037',
  'name': 'injury',
  'context': '[CLS] Far - infrared protects vascular end ##oth ##eli ##al cells from advanced g ##ly ##cation end products - induced [unused1] injury [unused2] via Pro ##my ##elo ##cy ##tic le ##uke ##mia zinc finger protein - mediated auto ##pha ##gy in di ##abe ##tic mice The accumulation of advanced g ##ly ##cation end products in di ##abe ##tic patients induce ##s vascular end ##oth ##eli ##al injury . Pro ##my ##elo ##cy ##tic le ##uke ##mia zinc finger protein is a transcription factor that can be activated by low - temperature far - infrared i ##rra ##diation to ex ##ert beneficial effects on the vascular end ##oth ##eli ##um . In the present study , we investigated the influence of far - infrared - induced Pro ##my [SEP]',
  'existing_cui': 'C0005604',
  'existing_cui_name': 'Birth Injuries',
  'existing_cui_desc': '[CLS] Birth In ##ju ##ries [unused3] ( In ##ju ##ry or Po ##ison ##ing , Birth , diagnosis , disorder : In ##ju ##ries , Birth ; T ##ra ##umatic birth ; injury birth ; Birth trauma , un ##sp ##ec ##ified ; Birth T ##ra ##uma ; Birth trauma of f ##etus ; injury ; birth ; traumatic birth ; birth trauma ; birth injuries ; Birth In ##ju ##ry ; trauma birth ; Neo ##nat ##al Birth In ##ju ##ry ; birth ##ing injuries ; Birth In ##ju ##ries [ Disease / Finding ] ; Un ##sp ##ec ##ified birth trauma ; birth ##ing trauma ; B ##IR ##TH IN ##J ; IN ##J B ##IR ##TH ; Birth injuries ; birth ; injury ; [SEP]',
  'our_bi_cui': 'C0010957',
  'our_bi_cui_name': 'Tissue damage',
  'our_bi_cui_desc': '[CLS] T ##iss ##ue damage [unused3] ( In ##ju ##ry or Po ##ison ##ing , m ##or ##ph ##olo ##gic abnormal ##ity : damage tissues ; Dam ##age ; damages tissue ; Dam ##age , T ##iss ##ue ; tissue damage ; T ##iss ##ue Dam ##age ; damage ; damages ) [SEP]',
  'our_cross_cui': 'C0178314',
  'our_cross_cui_name': 'Poisoning / injury',
  'our_cross_cui_desc': '[CLS] Po ##ison ##ing / injury [unused3] ( In ##ju ##ry or Po ##ison ##ing , disorder , finding , navigation ##al concept : In ##ju ##ry & / or poisoning ; IN ##J ##UR ##Y AND P ##O ##IS ##ON ##ING ; In ##ju ##ry and poisoning ; In ##ju ##ry / poisoning ) [SEP]'},
 {'mention_id': '27669669.10',
  'test_set_idx': 13715,
  'type': 'T037',
  'name': 'subluxation',
  'context': '[CLS] B ##ila ##teral C ##1 la ##mina ##r hooks combined with C ##2 p ##edic ##le screw fix ##ation in the treatment of at ##lant ##oa ##xial sub ##lux ##ation after G ##rise ##l syndrome Many et ##iol ##ogies can lead to at ##lant ##oa ##xial sub ##lux ##ai ##on . In G ##rise ##l syndrome , this [unused1] sub ##lux ##ation [unused2] occurs spontaneous ##ly after inflammatory processes of the head and neck . Di ##ag ##nosis is typically based on clinical history and a strong suspicion of this syndrome . Non ##su ##rg ##ical treatment most often resolve ##s the symptoms ; however , in some cases surgical treatment is necessary to repair the sub ##lux ##ation . Various surgical techniques and instrumentation systems [SEP]',
  'existing_cui': 'C0332768',
  'existing_cui_name': 'Joint Subluxations',
  'existing_cui_desc': '[CLS] Joint Sub ##lux ##ations [unused3] ( In ##ju ##ry or Po ##ison ##ing , diagnosis , disorder , m ##or ##ph ##olo ##gic abnormal ##ity , physical finding , qualifier value : sub ##lux ##ation ; Sub ##lux ##ation ; Sub ##lux ##ation , Joint ; Di ##s ##location , incomplete ; Joint sub ##lux ##ation ; Joint Sub ##lux ##ation ; Sub ##lux ##ation of joint ; sub ##lux ##ations ; Inc ##om ##ple ##te di ##s ##location ; joint sub ##lux ##ation ; Sub ##lux ##ations ; incomplete di ##s ##location ; Sub ##lux ##ations , Joint ; sub ##lux ##ation was seen ; sub ##lux ##ation of joint ; Sub ##lux ##ation - les ##ion ) [SEP]',
  'our_bi_cui': 'C0332768',
  'our_bi_cui_name': 'Joint Subluxations',
  'our_bi_cui_desc': '[CLS] Joint Sub ##lux ##ations [unused3] ( In ##ju ##ry or Po ##ison ##ing , diagnosis , disorder , m ##or ##ph ##olo ##gic abnormal ##ity , physical finding , qualifier value : sub ##lux ##ation ; Sub ##lux ##ation ; Sub ##lux ##ation , Joint ; Di ##s ##location , incomplete ; Joint sub ##lux ##ation ; Joint Sub ##lux ##ation ; Sub ##lux ##ation of joint ; sub ##lux ##ations ; Inc ##om ##ple ##te di ##s ##location ; joint sub ##lux ##ation ; Sub ##lux ##ations ; incomplete di ##s ##location ; Sub ##lux ##ations , Joint ; sub ##lux ##ation was seen ; sub ##lux ##ation of joint ; Sub ##lux ##ation - les ##ion ) [SEP]',
  'our_cross_cui': 'C0263905',
  'our_cross_cui_name': 'Atlantoaxial subluxation',
  'our_cross_cui_desc': '[CLS] At ##lant ##oa ##xial sub ##lux ##ation [unused3] ( In ##ju ##ry or Po ##ison ##ing , disorder : AA ##S - At ##lant ##oa ##xial sub ##lux ##ation ; at ##lant ##oa ##xial ; sub ##lux ##ation ; di ##s ##location ; at ##lant ##oa ##xial ; At ##lant ##o - a ##xial sub ##lux ##ation ; At ##lant ##oa ##xial di ##s ##location ; at ##lant ##oa ##xial ; di ##s ##location ; Sub ##lux ##ation of at ##lant ##oa ##xial joint ; sub ##lux ##ation ; at ##lant ##oa ##xial ) [SEP]'},
 {'mention_id': '27794381.32',
  'test_set_idx': 17172,
  'type': 'T103',
  'name': '[ Ca ( 2 + ) ] i',
  'context': '[CLS] while em ##od ##in or u ##rs ##ode ##ox ##ych ##olic acid were given to the corresponding group for 7 weeks . The his ##tom ##or ##phological and ultra ##structure change of gal ##l ##bla ##dder were detected by micro ##scope and electron micro ##scope , the content of plasma ch ##ole ##cy ##sto ##kini ##n and [unused1] [ C ##a ( 2 + ) ] i [unused2] were analyzed successively by radio ##im ##mu ##no ##ass ##ay and flow c ##yt ##ome ##try . The protein and m ##RNA of G ##s ##α , G ##i ##α and Cap in ch ##ole ##cy ##st cells were determined by western b ##lot ##ting and real time polymer ##ase chain reaction . Em ##od ##in or u [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C1155431',
  'our_bi_cui_name': 'positive regulation of cytosolic calcium ion concentration',
  'our_bi_cui_desc': '[CLS] positive regulation of c ##yt ##oso ##lic calcium ion concentration [unused3] ( B ##iol ##og ##ic Fun ##ction : elevation of calcium ion concentration in c ##yt ##oso ##l ; c ##yt ##oso ##lic calcium ion concentration elevation ; c ##yt ##op ##las ##mic calcium ion concentration elevation ; elevation of c ##yt ##oso ##lic calcium ion concentration ; elevation of c ##yt ##op ##las ##mic calcium ion concentration ; elevation of calcium ion concentration in c ##yt ##op ##las ##m ) [SEP]',
  'our_cross_cui': 'C0596235',
  'our_cross_cui_name': 'Calcium ion',
  'our_cross_cui_desc': '[CLS] Cal ##ci ##um ion [unused3] ( Chemical : Cal ##ci ##um Cat ##ion ; free calcium ; CA + 2 ; ion calcium ; Cal ##ci ##um cat ##ion ; Free Cal ##ci ##um ; calcium ions ; C ##a ##2 + ; C ##a + + ; Cal ##ci ##um Ion ##s ; C ##a 2 + ; calcium ion ; CA ##LC ##I ##UM CA ##TI ##ON ; Cal ##ci ##um Ion ; calcium cat ##ion ; Cal ##ci ##um 2 + ; Ion ##ized Cal ##ci ##um ) [SEP]'},
 {'mention_id': '28523425.48',
  'test_set_idx': 38413,
  'type': 'T082',
  'name': 'M1',
  'context': '[CLS] : see text ] of [ Formula : see text ] and [ Formula : see text ] . These findings indicate that the encoding of upper ex ##tre ##mity movement speed by M1 high - [ Formula : see text ] activity is primarily linear . Also , the fact that this activity remained elevated throughout a movement suggests that M1 does not merely generate trans ##ient instructions for a specific movement duration , but instead is responsible for the entirety of the movement . Finally , the spatial distribution of high - [ Formula : see text ] activity suggests the presence of a recruitment phenomenon in which higher speeds or increased muscle activity involve activation of larger [unused1] M1 [unused2] area s . [SEP]',
  'existing_cui': 'C1705014',
  'existing_cui_name': 'CHRM1 wt Allele',
  'existing_cui_desc': '[CLS] CH ##R ##M ##1 w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : M1 ##R ; HM ##1 ; Cho ##liner ##gic Re ##ceptor , Mu ##sca ##rini ##c , 1 Gene ; Ace ##ty ##l ##cho ##line Re ##ceptor , Mu ##sca ##rini ##c 1 Gene ; Cho ##liner ##gic Re ##ceptor , Mu ##sca ##rini ##c 1 w ##t All ##ele ; Ace ##ty ##l ##cho ##line Re ##ceptor , Mu ##sca ##rini ##c , 1 Gene ) [SEP]',
  'our_bi_cui': 'C0026607',
  'our_bi_cui_name': 'Motor Cortex',
  'our_bi_cui_desc': '[CLS] Motor Co ##rte ##x [unused3] ( Spa ##tial Concept , Motor , body structure : Co ##rte ##x ; Area , Motor ; Motor Area ; Areas , So ##matic Motor ; so ##mat ##omo ##tor areas ; Motor area ; Co ##rte ##x , Motor ; So ##mat ##omo ##tor Area ; Areas , Motor ; motor cortex ; Area , So ##matic Motor ; Motor Areas ; so ##matic motor areas ; So ##mat ##omo ##tor Areas ; Motor Areas , So ##matic ; motor area ; areas motor ; So ##matic Motor Area ; Area , So ##mat ##omo ##tor ; Motor Area , So ##matic ; Areas , So ##mat ##omo ##tor ; Motor cortex ; cortex motor ; So ##matic Motor [SEP]',
  'our_cross_cui': 'C3495441',
  'our_cross_cui_name': 'primary motor cortex',
  'our_cross_cui_desc': '[CLS] primary motor cortex [unused3] ( Spa ##tial Concept , Carpenter , Crosby , M I : area F1 ; Motor Area , Primary ; Motor Co ##rt ##ices , Primary ; primary so ##mat ##omo ##tor area ; primary motor area ; Motor Strip ; Pre ##cent ##ral Motor Co ##rte ##x ; Pre ##cent ##ral Motor Areas ; Area , Primary Motor ; motor cortex ; Motor Co ##rt ##ices , Pre ##cent ##ral ; Primary Motor Co ##rt ##ices ; pre ##cent ##ral motor cortex ; motor strip ; Primary Motor Area ; Motor cortex ; Area , Pre ##cent ##ral Motor ; Motor Co ##rte ##x , Pre ##cent ##ral ; pre ##cent ##ral motor area ; Primary Motor Areas ; Co ##rt [SEP]'},
 {'mention_id': '27612561.28',
  'test_set_idx': 11750,
  'type': 'T038',
  'name': 'identify',
  'context': '[CLS] ##ates of exercise ad ##herence were studied separately for high intensity and low - to - moderate intensity exercise . De ##mo ##graphic , clinical , and physical factors were assessed using self - reported question ##naire ##s . Re ##lev ##ant clinical information was extracted from medical records . Multi ##var ##iable log ##istic re ##gression analyses were applied to [unused1] identify [unused2] co ##rrel ##ates that were significantly associated with participation , high session attendance , high compliance with resistance and high compliance with endurance exercises . Part ##ici ##pants were more likely to have higher education , be non - smoke ##rs , have lower psychological distress , higher outcome expectations , and perceive more exercise barriers than non - participants . In [SEP]',
  'existing_cui': 'C0695108',
  'existing_cui_name': 'Sperm identification from testis tissue , fresh or cryopreserved',
  'existing_cui_desc': '[CLS] S ##per ##m identification from test ##is tissue , fresh or cry ##op ##res ##erve ##d [unused3] ( Health Care Act ##ivity : ID ##EN ##TI ##F ##Y SP ##ER ##M T ##IS ##SU ##E ; I ##dent ##ify sperm tissue ; SP ##R ##M ID F ##RO ##M T ##ST ##IS T ##IS ##S F ##RS ##H / CR ##Y ##OP ##RS ##R ##VD ) [SEP]',
  'our_bi_cui': 'C0243095',
  'our_bi_cui_name': 'Finding',
  'our_bi_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]',
  'our_cross_cui': 'C0020792',
  'our_cross_cui_name': 'Identification',
  'our_cross_cui_desc': '[CLS] I ##dent ##ification [unused3] ( B ##iol ##og ##ic Fun ##ction , Psychology , Psychology , Defense Me ##chan ##ism , finding : I ##dent ##ifications ; psychological identification ; I ##dent ##ification - mental defense mechanism ; ID ##EN ##TI ##FI ##CA ##TI ##ON PS ##Y ##CH ##OL ; I ##dent ##ification - mental defence mechanism ) [SEP]'},
 {'mention_id': '27933285.46',
  'test_set_idx': 22314,
  'type': 'T033',
  'name': 'signs',
  'context': '[CLS] Re ##sp ##ons ##ive ##ness Scale , all but one of the tested patients obtained clinical scores in the clinical range for at least one area . On the AD ##OS 2 , 4 patients obtained scores indicating a moderate level of au ##tist ##ic spectrum disorder symptoms , 4 had a mild level , and 15 showed no or minimal [unused1] signs [unused2] of au ##tist ##ic spectrum disorder . Motor stereo ##ty ##pies in children with normal cognitive level represent a challenging diagnostic issue for which a finely tailored assessment is mandatory in order to define a precise developmental profile . Thus , careful and cautious use of standardized tests is warrant ##ed to avoid mi ##s ##dia ##gno ##sis . Furthermore , it [SEP]',
  'existing_cui': 'C0311392',
  'existing_cui_name': 'Physical findings',
  'existing_cui_desc': '[CLS] Physical findings [unused3] ( Finding , finding , physical finding : physical finding ; physical findings ; Physical finding ; physical findings [ use for free text ] ; Physical signs ; Sign ##s - physical ; physical signs ; Physical sign ; Sign ; find physical ) [SEP]',
  'our_bi_cui': 'C3540840',
  'our_bi_cui_name': 'Sign or Symptom',
  'our_bi_cui_desc': '[CLS] Sign or S ##ym ##pt ##om [unused3] ( Finding : Sign ##s or S ##ym ##pt ##oms ; Clinical Sign ##s ; C ##L ##IN ##IC ##AL F ##IN ##DI ##NG ; C ##L ##IN ##IC ##AL S ##IG ##NS ; DE ##SC ; Clinical Finding ) [SEP]',
  'our_cross_cui': 'C0037088',
  'our_cross_cui_name': 'Signs and Symptoms',
  'our_cross_cui_desc': '[CLS] Sign ##s and S ##ym ##pt ##oms [unused3] ( Finding , finding : Sign ##s and S ##ym ##pt ##oms [ Disease / Finding ] ; Clinical findings ; clinical findings ; O ##bs ##er ##vable En ##ti ##ty ; sign and s ##ym ##pt ##om ; Sign and s ##ym ##pt ##om ; S ##ym ##pt ##oms and Sign ##s ; sign / s ##ym ##pt ##om ; Finding ; Clinical observation : symptoms and signs ; S ##IG ##NS S ##Y ##MP ##TO ##MS ; Clinical finding ) [SEP]'},
 {'mention_id': '27887026.29',
  'test_set_idx': 20814,
  'type': 'T103',
  'name': 'CmFDH',
  'context': '[CLS] ##O ##3 ( - ) as a substrate was evaluated by measuring the kinetic rates and conducting productivity ass ##ays . Can ##di ##da met ##hyl ##ica ( C ##m ##F ##D ##H ) and Cha ##eto ##mium the ##rm ##op ##hil ##um showed a higher efficiency in converting HC ##O ##3 ( - ) to format ##e than [unused1] C ##m ##F ##D ##H [unused2] , whereas C ##m ##F ##D ##H was better in the oxidation of format ##e . The pH op ##ti ##mum of the reduction was at pH 7 - 8 . However , the high concentrations of HC ##O ##3 ( - ) reduced the reaction rate . Can ##di ##da met ##hyl ##ica ( C ##m ##F ##D ##H [SEP]',
  'existing_cui': 'C0342185',
  'existing_cui_name': 'Hyperthyroxinemia , Familial Dysalbuminemic',
  'existing_cui_desc': '[CLS] H ##yper ##thy ##ro ##xin ##emia , F ##ami ##lial D ##ys ##al ##bu ##mine ##mic [unused3] ( Ana ##tom ##ical Structure , disorder : H ##YP ##ER ##TH ##Y ##RO ##X ##IN ##EM ##IA , FA ##MI ##L ##IA ##L D ##Y ##SA ##L ##B ##UM ##IN ##EM ##IC ; F ##ami ##lial d ##ys ##al ##bu ##mina ##em ##ic h ##yper ##thy ##ro ##xin ##ae ##mia ; H ##yper ##thy ##ro ##xin ##emia , F ##ami ##lial D ##ys ##al ##bu ##mine ##mic [ Disease / Finding ] ; D ##ys ##al ##bu ##mine ##mic H ##yper ##thy ##ro ##xin ##emia , F ##ami ##lial ; F ##ami ##lial D ##ys ##al ##bu ##mine ##mic H ##yper ##thy ##ro ##xin ##emia ; F ##ami [SEP]',
  'our_bi_cui': 'C0060647',
  'our_bi_cui_name': 'Formate dehydrogenase',
  'our_bi_cui_desc': '[CLS] Form ##ate de ##hy ##dr ##ogen ##ase [unused3] ( Chemical , c ##yt ##och ##rome , c ##yt ##och ##rome , substance : Form ##ate de ##hy ##dr ##ogen ##ase ( c ##yt ##och ##rome ) ) [SEP]',
  'our_cross_cui': 'C0016570',
  'our_cross_cui_name': 'Formate dehydrogenase',
  'our_cross_cui_desc': '[CLS] Form ##ate de ##hy ##dr ##ogen ##ase [unused3] ( Chemical , substance : De ##hy ##dr ##ogen ##ase , N ##AD - Form ##ate ; N ##AD Form ##ate De ##hy ##dr ##ogen ##ase ; De ##hy ##dr ##ogen ##ases , Form ##ate ; N ##AD - Form ##ate De ##hy ##dr ##ogen ##ase ; Form ##ate H ##ydro ##gen ##lya ##ses ; Form ##ate De ##hy ##dr ##ogen ##ases ; Form ##ate De ##hy ##dr ##ogen ##ases [ Chemical / In ##g ##red ##ient ] ; Form ##ate : N ##AD + o ##xi ##dor ##ed ##uc ##tase ; H ##ydro ##gen ##lya ##ses , Form ##ate ) [SEP]'},
 {'mention_id': '27478553.20',
  'test_set_idx': 7719,
  'type': 'T098',
  'name': 'non - night eaters',
  'context': "[CLS] National Health and N ##utrition Examination Survey . ' Night eating ' was defined as consuming 25 % or more of one ' s daily energy intake between 21 : 00 and 06 : 00 . Sub ##jects comply ##ing with the preceding condition were classified as ' night eat ##ers ' , whereas the rest were considered ' [unused1] non - night eat ##ers [unused2] ' . Lo ##gis ##tic re ##gression analysis examined factors related to night eating . Multiple linear re ##gression analyses were used to examine the relationship between night eating and B ##MI z - scores , whereas multi ##no ##mia ##l log ##istic re ##gression analysis was used to examine the relationship between night eating and weight status . About [SEP]",
  'existing_cui': 'C2350309',
  'existing_cui_name': 'Night Vision',
  'existing_cui_desc': '[CLS] Night Vision [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , finding , o ##bs ##er ##vable entity : Vision , Sc ##oto ##pic ; Vision , Rod ; Night vision ; Vision , Night ; s ##cot ##op ##ic vision ; Rod Vision ; Sc ##oto ##pic vision ; Sc ##oto ##pic Vision ) [SEP]',
  'our_bi_cui': 'C0243095',
  'our_bi_cui_name': 'Finding',
  'our_bi_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]',
  'our_cross_cui': 'C1257890',
  'our_cross_cui_name': 'Population Group',
  'our_cross_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]'},
 {'mention_id': '28295414.12',
  'test_set_idx': 31550,
  'type': 'T168',
  'name': 'drinks',
  'context': '[CLS] ##IN ##E Study Recently , the Food and Drug Administration proposed to expand the options for primary end points in the development of medications for alcohol use disorder to include either a ##bs ##tine ##nce from alcohol or a non ##ab ##st ##inen ##t outcome : no heavy drinking days ( with a heavy drinking day defined as more than 3 [unused1] drinks [unused2] per day for women and more than 4 drinks per day for men [ > 3 / > 4 cut ##off ] ) . The Food and Drug Administration also suggested that 6 months would be the most appropriate length for a clinical trial to demonstrate the stability of this non ##ab ##st ##inen ##t drinking outcome . However , few alcohol [SEP]',
  'existing_cui': 'C0452428',
  'existing_cui_name': 'Drink',
  'existing_cui_desc': '[CLS] Dr ##ink [unused3] ( Food , diet ##ary substance , diet ##ary substance , attribute , substance : Dr ##ink ##s ; drinks ; drink [ substance ] ; drink ) [SEP]',
  'our_bi_cui': 'C0452428',
  'our_bi_cui_name': 'Drink',
  'our_bi_cui_desc': '[CLS] Dr ##ink [unused3] ( Food , diet ##ary substance , diet ##ary substance , attribute , substance : Dr ##ink ##s ; drinks ; drink [ substance ] ; drink ) [SEP]',
  'our_cross_cui': 'C0001967',
  'our_cross_cui_name': 'Alcoholic Beverages',
  'our_cross_cui_desc': '[CLS] Al ##co ##hol ##ic Be ##ver ##ages [unused3] ( Food , Al ##co ##hol ##ic , substance : Be ##ver ##age , Al ##co ##hol ##ic ; Al ##co ##hol ##ic Be ##ver ##age ; Al ##co ##hol ##ic beverages ; alcoholic drinks ; Be ##ver ##ages ; Be ##ver ##ages , Al ##co ##hol ##ic ; alcoholic beverages ; alcoholic beverage ; alcoholic ##s beverages ; Al ##co ##hol ##ic beverage ; Al ##co ##hol ##ic drink ; alcoholic drinking ) [SEP]'},
 {'mention_id': '28027760.29',
  'test_set_idx': 23870,
  'type': 'T058',
  'name': '3rd generation dual - source CT',
  'context': '[CLS] ##iac CT an ##gio ##graphy : 2 . 4 ± 0 . 9 ##m ##S ##v - 6 . 8 ± 2 . 7 m ##S ##v , all p ≤ 0 . 04 ##14 ) . In 39 patients trans ##cat ##he ##ter a ##ort ##ic valve replacement was performed and an ##nu ##lus diameter was within the recommended range in all patients . No severe cardiac or vascular complications were noted . [unused1] 3rd generation dual - source CT [unused2] provides diagnostic image quality in trans ##cat ##he ##ter a ##ort ##ic valve replacement - planning CT an ##gio ##graphy and facilitate ##s reliable assessment of trans ##cat ##he ##ter a ##ort ##ic valve replacement device and delivery option while reducing radiation dose . [SEP]',
  'existing_cui': 'C2021220',
  'existing_cui_name': 'third generation thyroid stimulating hormone assay',
  'existing_cui_desc': '[CLS] third generation thy ##roid s ##ti ##mulating hormone ass ##ay [unused3] ( Health Care Act ##ivity , lab test : T ##S ##H ( 3rd generation ) ; 3rd generation T ##S ##H ass ##ay ) [SEP]',
  'our_bi_cui': 'C4055114',
  'our_bi_cui_name': 'Dual energy computed tomography',
  'our_bi_cui_desc': '[CLS] Dual energy com ##puted to ##mo ##graphy [unused3] ( Health Care Act ##ivity , procedure : Dual - Energy Co ##mp ##uted Tom ##ography ; DE ##CT - dual energy com ##puted to ##mo ##graphy ) [SEP]',
  'our_cross_cui': 'C0040405',
  'our_cross_cui_name': 'X-Ray Computed Tomography',
  'our_cross_cui_desc': '[CLS] X - Ray Co ##mp ##uted Tom ##ography [unused3] ( Health Care Act ##ivity , computer ##ized to ##mo ##graphy , procedure : Sc ##an , X - Ray CT ; com ##puted to ##mo ##graphy x rays ; EMI scan ; em ##i scan ; CT X ##RA ##Y ; Co ##mp ##uted X - Ray Tom ##ography ; Computer ##ised trans ##ax ##ial to ##mo ##graphy ; to ##mo ##den ##si ##tom ##et ##ry ; CA ##T scan - ; CT Sc ##an , X Ray ; Computer ##ized to ##mo ##gram ; X - Ray CA ##T Sc ##an ; Computer ##ized trans ##ax ##ial to ##mo ##graphy ; x - ray com ##puted to ##mo ##graphy ; Tom ##ography , Trans ##mission [SEP]'},
 {'mention_id': '27273231.23',
  'test_set_idx': 1204,
  'type': 'T204',
  'name': 'survival model',
  'context': '[CLS] ##l , l ##iga ##tion , and pro ##ximal l ##iga ##tion plus di ##stal En ##sea ##l ) and extracted to evaluate the pressure tolerance up to 75 mm H ##g at the sealed end . A left lower lobe ##ct ##omy was performed to evaluate chronic - phase du ##rab ##ility of the sealed s ##tum ##ps in a [unused1] survival model [unused2] . Two or three branches of the pulmonary art ##eries in each dog were allocated to each of the 3 groups . After the scheduled survival period , the pulmonary art ##eries were sampled . Press ##ure tolerance at the sealed end was evaluated in 91 pulmonary artery sections . All sealed ends showed pressure tolerance > 75 mm H ##g [SEP]',
  'existing_cui': 'C0007620',
  'existing_cui_name': 'Cell Survival',
  'existing_cui_desc': '[CLS] Cell Survival [unused3] ( B ##iol ##og ##ic Fun ##ction : Via ##bilities , Cell ; cell via ##bility ; Cell Via ##bility ; Via ##bility , Cell ; cell survival ; Cell Via ##bilities ; Survival , Cell ) [SEP]',
  'our_bi_cui': 'C0086272',
  'our_bi_cui_name': 'Experimental Model',
  'our_bi_cui_desc': '[CLS] Experimental Model [unused3] ( Intel ##lect ##ual Product : experimental model ; E ##X ##P ##ER M ##OD ##EL ; models experimental ; Experimental Models ; experimental ##s models ; Models , Experimental ; experimental models ; Experimental Models , Other ; Model , Experimental ; M ##OD ##EL E ##X ##P ##ER ; E ##X ##P ##ER M ##OD ##EL ##S ; M ##OD ##EL ##S E ##X ##P ##ER ) [SEP]',
  'our_cross_cui': 'C0599779',
  'our_cross_cui_name': 'Animal Model',
  'our_cross_cui_desc': '[CLS] Animal Model [unused3] ( E ##uka ##ryo ##te : animal model ; Animal Models ; Models , Animal ; Animal Model , Gene ##ric ; Research Model , Animal ; animals models ; animal models ; Model , Animal ) [SEP]'},
 {'mention_id': '27986797.2',
  'test_set_idx': 22899,
  'type': 'T103',
  'name': 'myomitokine',
  'context': '[CLS] Growth differentiation factor 15 is a [unused1] my ##omi ##tok ##ine [unused2] governing systemic energy home ##ost ##asis Red ##uce ##d mitochondrial electron transport chain activity promotes long ##evity and improve ##s energy home ##ost ##asis via cell - autonomous and - non - autonomous factors in multiple model systems . This mit ##oh ##orm ##etic effect is thought to involve the mitochondrial unfolded protein response ( UP ##R ( m ##t ) ) , an adaptive stress - response pathway activated by mitochondrial pro ##te ##oto ##xi ##c stress . Using mice with skeletal muscle - specific deficiency of C ##ri ##f ##1 ( muscle - specific knockout [ M ##KO ] ) , an integral protein of the large mit ##ori ##bos ##oma ##l [SEP]',
  'existing_cui': 'C0950133',
  'existing_cui_name': 'Online Mendelian Inheritance In Man',
  'existing_cui_desc': '[CLS] Online Men ##del ##ian In ##her ##ita ##nce In Man [unused3] ( Intel ##lect ##ual Product : online men ##del ##ian inheritance in man ( O ##MI ##M ) database ; Online Men ##del ##ian In ##her ##ita ##nce in Man ) [SEP]',
  'our_bi_cui': 'C0026484',
  'our_bi_cui_name': 'Monokines',
  'our_bi_cui_desc': '[CLS] Mon ##oki ##nes [unused3] ( Chemical , substance : Mon ##oki ##ne ; mon ##oki ##ne ; Mon ##oki ##nes [ Chemical / In ##g ##red ##ient ] ) [SEP]',
  'our_cross_cui': 'C0574031',
  'our_cross_cui_name': 'Biologically active substance',
  'our_cross_cui_desc': '[CLS] Biological ##ly active substance [unused3] ( Chemical , substance : Biological ##ly Active Sub ##stance ) [SEP]'},
 {'mention_id': '27482869.1',
  'test_set_idx': 7894,
  'type': 'T204',
  'name': 'Allen J. Wilcox',
  'context': "[CLS] Voices : A Con ##vers ##ation with [unused1] Allen J . Wilcox [unused2] Allen James Wilcox was born on 30 September 1946 in Columbus , OH . He studied medicine at the University of Michigan , graduated in 1973 , and after a rotating internship , he completed a master ' s degree in maternal and child health ( 1976 ) and a PhD in e ##pid ##em ##iology ( 1979 ) at the University of North Carolina in Chapel Hill . After graduation , he went to work at the National Institute of Environmental Health Sciences ( National Institute of Environmental Health Sciences , one of the US National Institutes of Health ) in Durham , NC , where he has spent his career . [SEP]",
  'existing_cui': 'C3810728',
  'existing_cui_name': 'Wilcox County , AL',
  'existing_cui_desc': '[CLS] Wilcox County , AL [unused3] ( Spa ##tial Concept : Wilcox County ; AL ##13 ##1 ; Wilcox County , Alabama ) [SEP]',
  'our_bi_cui': 'C0027361',
  'our_bi_cui_name': 'Persons',
  'our_bi_cui_desc': '[CLS] Persons [unused3] ( Population Group , person : Individual ; Person ; persons ; peoples ; Person [ du ##p ] ; People ; person ; people ) [SEP]',
  'our_cross_cui': 'C0086418',
  'our_cross_cui_name': 'Homo sapiens',
  'our_cross_cui_desc': '[CLS] Ho ##mo sa ##pie ##ns [unused3] ( E ##uka ##ryo ##te , Tax ##onomy , organism , qualifier value : Humans ; Man , Modern ; humans ; man ; MA ##N ; Ho ##mo sa ##pie ##ns ( living organism ) [ Am ##bi ##guous ] ; ho ##mo sa ##pie ##ns ; Human , General ; Ho ##mo sa ##pie ##ns Linnaeus , 1758 ; Human ; Human - origin ; human ; Ho ##mo sa ##pie ##ns ( living organism ) ; Modern Man ; Man ) [SEP]'},
 {'mention_id': '28197156.18',
  'test_set_idx': 28088,
  'type': 'T017',
  'name': 'Carbon Assimilation Rate 8',
  'context': '[CLS] ##rs using quantitative genetics is one promising means to identify genes contributing to higher photos ##ynth ##esis . In this study , we determined precise location of Carbon As ##si ##mi ##lation Rate 8 by crossing a high - yielding in ##dic ##a cult ##iva ##r with a Japanese commercial cult ##iva ##r . Fine mapping suggested that [unused1] Carbon As ##si ##mi ##lation Rate 8 [unused2] en ##codes a put ##ative He ##me Act ##iva ##tor Pro ##tein 3 ( O ##s ##HA ##P ##3 ) subunit of a CC ##AA ##T - box - binding transcription factor called O ##s ##HA ##P ##3 ##H . Se ##quencing analysis revealed that the in ##dic ##a all ##ele of Carbon As ##si ##mi ##lation Rate 8 [SEP]',
  'existing_cui': 'C2246620',
  'existing_cui_name': 'iron assimilation',
  'existing_cui_desc': '[CLS] iron ass ##im ##ilation [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_bi_cui': 'C1413023',
  'our_bi_cui_name': 'C8A gene',
  'our_bi_cui_desc': '[CLS] C ##8 ##A gene [unused3] ( Ana ##tom ##ical Structure : complement component 8 alpha subunit ; CO ##MP ##LE ##ME ##NT CO ##MP ##ON ##EN ##T 8 , AL ##P ##HA S ##U ##B ##UN ##IT ; C ##8 AL ##P ##HA ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27372076.8',
  'test_set_idx': 3959,
  'type': 'T092',
  'name': 'industrially',
  'context': '[CLS] the St ##re ##pt ##omy ##ces ah ##y ##gro ##scopic ##us strain C ##K - 15 that acts as a re ##press ##or of m ##or ##phological development Wu ##yi ##en ##cin is produced by St ##re ##pt ##omy ##ces ah ##y ##gro ##scopic ##us var . w ##uy ##iens ##is , which has been widely used in China as an [unused1] industrial ##ly [unused2] produced bio ##pes ##tic ##ide to control various fun ##gal diseases . Although its mechanism of action , breeding , and f ##er ##mentation had been extensively characterized , less is known about the regulatory functions that affect its bio ##sy ##nt ##hesis or m ##or ##phological development . The w ##ys ##R ##3 gene of S . ah ##y ##gro [SEP]',
  'existing_cui': 'C1552816',
  'existing_cui_name': 'all - LocalMarkupIgnore',
  'existing_cui_desc': '[CLS] all - Local ##M ##ark ##up ##I ##gno ##re [unused3] ( Intel ##lect ##ual Product ) [SEP]',
  'our_bi_cui': 'C0557758',
  'our_bi_cui_name': 'Industrial environment',
  'our_bi_cui_desc': '[CLS] Industrial environment [unused3] ( Spa ##tial Concept , environment ) [SEP]',
  'our_cross_cui': 'C0007983',
  'our_cross_cui_name': 'Chemical Industry',
  'our_cross_cui_desc': '[CLS] Chemical Industry [unused3] ( Organization : Chemical Industries ; IN ##D ##US ##T CH ##EM ; Industry , Chemical ; chemical industry ; chemical industries ; CH ##EM IN ##D ##US ##T ; Industries , Chemical ; Chemical industry ) [SEP]'},
 {'mention_id': '27442216.17',
  'test_set_idx': 5813,
  'type': 'T098',
  'name': 'Snowmobile',
  'context': '[CLS] ##s ( 63 % , 64 % ) than the All - terrain vehicles group ( 50 % ) ( P = 0 . 000 ##8 ) . Snow ##mobile injuries had the highest rate of spinal ( 23 % ) and lower ex ##tre ##mity fracture ##s ( 53 % ) ( P = 0 . 000 ##4 ) . [unused1] Snow ##mobile [unused2] and dirt ##bi ##ke co ##hor ##ts had higher rate of f ##em ##ur fracture ##s ( 22 % , 17 % , P = 0 . 00 ##1 ) whereas the All - terrain vehicles co ##hor ##t had higher rates of upper ex ##tre ##mity ( 18 % ) , hand ( 11 % ) , s ##cap ##ula [SEP]',
  'existing_cui': 'C1427826',
  'existing_cui_name': 'SBNO1 gene',
  'existing_cui_desc': '[CLS] S ##B ##N ##O ##1 gene [unused3] ( Ana ##tom ##ical Structure , Dr ##oso ##phi ##la : FL ##J ##10 ##70 ##1 ; FL ##J ##10 ##8 ##33 ; ST ##RA ##W ##BE ##R ##R ##Y NO ##TC ##H , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF , 1 ; straw ##berry notch ho ##mo ##log 1 ; M ##OP ##3 ) [SEP]',
  'our_bi_cui': 'C0597177',
  'our_bi_cui_name': 'Particle',
  'our_bi_cui_desc': '[CLS] Part ##icle [unused3] ( Chemical : particle ; particles ; Part ##icles ; a very small piece or part ) [SEP]',
  'our_cross_cui': 'C0599755',
  'our_cross_cui_name': 'Cohort',
  'our_cross_cui_desc': '[CLS] Co ##hor ##t [unused3] ( Population Group : co ##hor ##ted ; co ##hor ##ting ; co ##hor ##t ; co ##hor ##ts ) [SEP]'},
 {'mention_id': '27299916.29',
  'test_set_idx': 1879,
  'type': 'T103',
  'name': 'CQ',
  'context': '[CLS] or synthesized . Only derivatives containing an electron def ##icient a ##romatic ring and capable of adopting flat conform ##ations , optimal for π - π interactions with Fe ( III ) PP ##IX , in ##hibit ##ed β - hem ##ati ##n formation . The two most potent analogue ##s showed na ##no ##mo ##lar parasite activity , with little [unused1] C ##Q [unused2] cross - resistance , low c ##yt ##oto ##xi ##city , and high in v ##it ##ro micro ##so ##mal stability . Selected analogue ##s in ##hibit ##ed hem ##oz ##oi ##n formation in P ##las ##mo ##dium f ##al ##ci ##par ##um causing high levels of free hem ##e . In contrast to q ##uin ##olin ##es , introduction of [SEP]',
  'existing_cui': 'C0282245',
  'existing_cui_name': 'Northern Mariana Islands',
  'existing_cui_desc': '[CLS] Northern Mariana Islands [unused3] ( Spa ##tial Concept : MP ; NO ##RT ##H ##ER ##N MA ##RI ##AN ##A IS ##LA ##ND ##S ; C ##Q ; M ##NP ) [SEP]',
  'our_bi_cui': 'C0072849',
  'our_bi_cui_name': 'quinacillin',
  'our_bi_cui_desc': '[CLS] q ##uin ##ac ##ill ##in [unused3] ( Chemical : 3 - car ##box ##y - 2 - q ##uin ##ox ##ali ##ny ##l ##pen ##ici ##llan ##ic acid ; Q ##uin ##ac ##ill ##in ; Q ##UI ##NA ##CI ##LL ##IN ) [SEP]',
  'our_cross_cui': 'C0008269',
  'our_cross_cui_name': 'Chloroquine',
  'our_cross_cui_desc': '[CLS] Ch ##lor ##o ##quin ##e [unused3] ( Chemical , medication , product , substance : Ch ##lor ##o ##quin ##e [ Chemical / In ##g ##red ##ient ] ; Ch ##lor ##och ##in ; Ch ##lor ##o ##quin ##e [ anti - malaria ##l ] ; Ch ##lor ##o ##quin ##ium ; N ( 4 ) - ( 7 - ch ##lor ##o - 4 - Q ##uin ##olin ##yl ) - N ( 1 ) , N ( 1 ) - diet ##hyl - 1 , 4 - pen ##tan ##edia ##mine ; ch ##lor ##o ##quin ##e ; Ch ##lor ##och ##ine ; Ch ##lor ##o ##quin ##a ; CH ##L ##OR ##O ##Q ##UI ##NE ; C ##lor ##o ##quin ##a ; Ch [SEP]'},
 {'mention_id': '27623277.40',
  'test_set_idx': 12224,
  'type': 'T058',
  'name': 'procedures',
  'context': '[CLS] ( relative weight of 0 . 21 ) , risk of major complications ( relative weight of 0 . 18 ) , and distance needed to travel for treatment ( relative weight of 0 . 15 ) . Pro ##cedural and treatment preferences did not vary with demographic ##s , self - reported health status , or familiar ##ity with the [unused1] procedures [unused2] . Survey respond ##ents preferred minimal ##ly invasive operations over stereo ##ta ##ctic body radiation therapy or th ##ora ##cot ##omy for treatment of early - stage non - small cell lung cancer . Treatment m ##oda ##lity and risk of cancer re ##cu ##rrence were the most important factors associated with treatment preferences . Pro ##vide ##r experience out ##wei ##gh [SEP]',
  'existing_cui': 'C0184661',
  'existing_cui_name': 'Interventional procedure',
  'existing_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_bi_cui': 'C0184661',
  'our_bi_cui_name': 'Interventional procedure',
  'our_bi_cui_desc': '[CLS] Inter ##vention ##al procedure [unused3] ( Health Care Act ##ivity , procedure : intervention ##al procedures ; procedure ; Inter ##vention or Pro ##ced ##ure ; intervention ##al procedure ; intervention ##D ##es ##cription ; { Pro ##ced ##ure } ; PR ##OC ##ED ##UR ##ES ; Inter ##vention ##al ; Pro ##ced ##ure ; Inter ##vention ; Pro ##ced ##ures ; intervention ; Inter ##vention St ##rate ##gies ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27500205.1',
  'test_set_idx': 8433,
  'type': 'T103',
  'name': 'Kindlins',
  'context': '[CLS] Of [unused1] Kind ##lins [unused2] and Cancer Kind ##lins are 4 . 1 - e ##z ##rin - rid ##ix ##in - m ##oes ##in ( F ##ER ##M ) domain containing proteins . There are three kind ##lins in mammals , which share high sequence identity . Kind ##lin - 1 is expressed primarily in e ##pit ##hel ##ial cells , kind ##lin - 2 is widely distributed and is particularly abundant in ad ##here ##nt cells , and kind ##lin - 3 is expressed primarily in hem ##ato ##po ##iet ##ic cells . These distributions are not exclusive ; some cells express multiple kind ##lins , and transformed cells often exhibit a ##ber ##rant expression , both in the is ##of ##orms and the [SEP]',
  'existing_cui': 'C1720776',
  'existing_cui_name': 'Kindling , Neurologic',
  'existing_cui_desc': '[CLS] Kind ##ling , N ##eur ##olo ##gic [unused3] ( B ##iol ##og ##ic Fun ##ction , N ##eur ##ology : kind ##ling ; Kind ##lings , N ##eur ##olo ##gic ; Kind ##ling ; N ##eur ##olo ##gic Kind ##lings ; N ##eur ##olo ##gic Kind ##ling ) [SEP]',
  'our_bi_cui': 'C0010422',
  'our_bi_cui_name': 'Crystallins',
  'our_bi_cui_desc': '[CLS] Crystal ##lins [unused3] ( Chemical , substance : Crystal ##lins [ Chemical / In ##g ##red ##ient ] ; Crystal ##lin ; Pro ##tein ##s , Len ##s ; C ##rist ##all ##in ; Len ##s Pro ##tein ##s ; crystal ##lin ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28495651.5',
  'test_set_idx': 37780,
  'type': 'T201',
  'name': 'cardiovascular death',
  'context': '[CLS] Per ##cent ##ile cat ##ego ##riz ##ation of Q ##T interval as an approach for identifying adult patients at risk for [unused1] card ##iovascular death [unused2] The results from studies of the association of Q ##T pro ##long ##ation with card ##iovascular death have been inconsistent . The purpose of this study was to compare the major correction formula ##s to percent ##ile values of Q ##T for heart rate ranges as to their ability to remove the relationship of Q ##T to heart rate and to predict card ##iovascular death . Part ##ici ##pants were 16 , 53 ##1 veterans who had an initial EC ##G at the Veterans Affairs Medical Center , Pa ##lo Alto , between March 31 , 1987 , and December [SEP]',
  'existing_cui': 'C0007226',
  'existing_cui_name': 'Cardiovascular system',
  'existing_cui_desc': '[CLS] Card ##iovascular system [unused3] ( Body System , body structure : Structure of card ##iovascular system , un ##sp ##ec ##ified ; CA ##RD ##IO ##VA ##SC ##U ##LA ##R S ##Y ##ST ##EM ; Card ##iovascular ; Organ System , Card ##iovascular ; CA ##RD ##IO ##VA ##SC S ##Y ##ST ##EM ; Card ##iovascular structure ; card ##iovascular system ; Card ##iovascular Organ System ; C ##ir ##cula ##tory ; Structure of card ##iovascular system ; Card ##iovascular Systems ; card ##iovascular structure ; C ##ir ##cula ##tory System ; C ##ir ##cula ##tory Systems ; { C ##ir ##cula ##tory system } ; c ##ir ##cula ##tory ; C ##ir ##cula ##tory system ; Body System , Card ##iovascular ; card ##iovascular system [SEP]',
  'our_bi_cui': 'C0376297',
  'our_bi_cui_name': 'Cardiac Death',
  'our_bi_cui_desc': '[CLS] Card ##iac Death [unused3] ( B ##iol ##og ##ic Fun ##ction : death ; cardiac ; cardiac death ; cardiac deaths ; Card ##iac death ; Death , Card ##iac ; cardiac ; death ) [SEP]',
  'our_cross_cui': 'C2926098',
  'our_cross_cui_name': "Mechanism of death in patient 's dying of cardiovascular causes : Type : Point in time : ^ Patient : Nominal",
  'our_cross_cui_desc': "[CLS] Me ##chan ##ism of death in patient ' s dying of card ##iovascular causes : Type : Point in time : ^ Pat ##ient : No ##mina ##l [unused3] ( Clinical At ##tri ##but ##e : Death due to C ##V causes ; Me ##chan ##ism of death in patient ' s dying of card ##iovascular causes : Type : P ##t : ^ Pat ##ient : No ##m ; Me ##chan ##ism of death in patient ' s dying of card ##iovascular causes ) [SEP]"},
 {'mention_id': '27298278.22',
  'test_set_idx': 1823,
  'type': 'T103',
  'name': 'E',
  'context': '[CLS] ##one , an ant ##hra ##quin ##one , and a nap ##ht ##ho ##hy ##dr ##o ##quin ##one dim ##er , together with three known q ##uin ##ones and seven other known compounds , were isolated from the aerial parts of Mo ##rin ##da par ##vi ##folia . The structures of m ##ori ##nda ##par ##vin ##s C , D , [unused1] E [unused2] , F , and G were el ##uc ##idated on the basis of s ##pect ##ros ##copic or X - ray di ##ff ##raction analysis as met ##hyl 4 - h ##ydro ##xy - 1 , 6 - dim ##eth ##ox ##y - nap ##ht ##hale ##ne - 2 - car ##box ##yla ##te , met ##hyl 4 , 8 - di [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0010207',
  'our_bi_cui_name': 'Coumarins',
  'our_bi_cui_desc': '[CLS] Co ##uma ##rin ##s [unused3] ( Chemical : co ##uma ##rin ##s ; Co ##uma ##rine ##s ; Der ##via ##tives , 1 , 2 - Benz ##opy ##rone ; 1 , 2 - Benz ##opy ##rone ##s ; Benz ##opy ##ran 2 ones ; 1 , 2 - Benz ##opy ##rone Der ##via ##tives ; Co ##uma ##rin ##s [ Chemical / In ##g ##red ##ient ] ; 1 , 2 Benz ##opy ##rone ##s ; Co ##uma ##rin Der ##iva ##tives ; Der ##iva ##tives , Co ##uma ##rin ; Benz ##opy ##ran - 2 - ones ; B ##EN ##Z ##OP ##Y ##RO ##NE ##S 01 02 ; 1 , 2 Benz ##o P ##yr ##ones ; 1 , 2 - Benz ##o [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28334615.30',
  'test_set_idx': 32328,
  'type': 'T033',
  'name': 'molecular underpinnings',
  'context': '[CLS] white matter property - within the force ##ps minor and the left superior temporal longitudinal f ##as ##ci ##culus . Within each of these tracts , mean fraction ##al an ##is ##ot ##rop ##y value of depression score - associated v ##ox ##els was used as a p ##hen ##otype in a subsequent Gene set en ##rich ##ment analysis . The NC ##AM ##1 Inter ##actions gene set was significantly en ##rich ##ed in these tracts . By linking the NC ##AM ##1 Inter ##actions gene set to depression scores and their structural brain co ##rrel ##ates in healthy participants , the current study contributes to the understanding of the [unused1] molecular under ##pin ##ning ##s [unused2] of de ##pressive s ##ym ##pt ##oma ##tology . [SEP]',
  'existing_cui': 'C0019247',
  'existing_cui_name': 'Hereditary Diseases',
  'existing_cui_desc': '[CLS] Here ##dit ##ary Disease ##s [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : Gene ##tic disorder ; Di ##sor ##der , Gene ##tic ; genetics disease ; Gene ##tic Di ##sor ##der ; Disease , Gene ##tic ; Di ##sor ##ders , Gene ##tic ; Disease , Here ##dit ##ary ; genetic disorder ; hereditary disease ; Disease ##s , Here ##dit ##ary ; inherited disease ; Here ##dit ##ary disorder ; Here ##dit ##ary Di ##sor ##ders ; Disease ##s , Gene ##tic ; genetic disorders ; In ##her ##ited disease ; In ##her ##ited Disease ; hereditary diseases ; Molecular Disease ; H ##ER ##ED ##IT ##AR ##Y D ##IS ; Gene ##tic Disease ##s ; Gene ##tic Con ##dition ; [SEP]',
  'our_bi_cui': 'C1148560',
  'our_bi_cui_name': 'molecular_function',
  'our_bi_cui_desc': '[CLS] molecular _ function [unused3] ( B ##iol ##og ##ic Fun ##ction : molecular function ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28197133.33',
  'test_set_idx': 28049,
  'type': 'T103',
  'name': 'GH48',
  'context': '[CLS] 1 % of the trans ##cripts . The CA ##Z ##yme profile was dominated by families G ##H ##9 ##4 ( cello ##bio ##se - p ##hos ##ph ##ory ##lase ) , G ##H1 ##3 ( am ##yla ##se ) , G ##H ##43 and G ##H1 ##0 ( hem ##ice ##ll ##ula ##ses ) , G ##H ##9 and [unused1] G ##H ##48 [unused2] ( cell ##ula ##ses ) , P ##L ##11 ( p ##ec ##tina ##se ) as well as G ##H ##2 and G ##H ##3 ( o ##li ##gos ##ac ##cha ##rida ##ses ) . Our data support the pivotal role of the most characterized fi ##bro ##ly ##tic bacteria ( Pre ##vo ##tel ##la , R ##umi ##no ##cc ##cus [SEP]',
  'existing_cui': 'C1333673',
  'existing_cui_name': 'GH1 gene',
  'existing_cui_desc': '[CLS] G ##H1 gene [unused3] ( Ana ##tom ##ical Structure : G ##RO ##W ##TH H ##OR ##MO ##NE , NO ##R ##MA ##L ; G ##RO ##W ##TH H ##OR ##MO ##NE 1 ; G ##H1 Gene ; growth hormone 1 ; so ##mat ##ot ##rop ##in ; pit ##uit ##ary growth hormone ; G ##RO ##W ##TH H ##OR ##MO ##NE , P ##IT ##UI ##TA ##R ##Y ; h ##G ##H - N ; G ##H - N ; Growth Ho ##rmon ##e 1 Gene ) [SEP]',
  'our_bi_cui': 'C0016799',
  'our_bi_cui_name': 'Fumarate Hydratase',
  'our_bi_cui_desc': '[CLS] Fu ##mara ##te H ##yd ##rata ##se [unused3] ( Chemical , substance : Fu ##mara ##te H ##yd ##rata ##se [ Chemical / In ##g ##red ##ient ] ; Fu ##mara ##te h ##yd ##rata ##se ; Fu ##mara ##se ; f ##uma ##rase ; Fu ##mara ##te h ##yd ##rata ##se ( substance ) [ Am ##bi ##guous ] ; H ##yd ##rata ##se , Fu ##mara ##te ; ( S ) - Mal ##ate h ##ydro - l ##yas ##e ; f ##uma ##rate h ##yd ##rata ##se ) [SEP]',
  'our_cross_cui': 'C1260229',
  'our_cross_cui_name': 'Cellulases',
  'our_cross_cui_desc': '[CLS] Cell ##ula ##ses [unused3] ( Chemical : Cell ##ula ##ses [ Chemical / In ##g ##red ##ient ] ) [SEP]'},
 {'mention_id': '28273336.12',
  'test_set_idx': 30879,
  'type': 'T038',
  'name': 'BV - allergic',
  'context': '[CLS] ##A ##pi m 1 has been available on the I ##mm ##uno ##CA ##P system for routine diagnosis of bee venom all ##er ##gy . Now , there are five components available , and we aimed to detect the sensitivity and specific ##ity of r ##A ##pi m 1 , 2 , 3 , 5 , and 10 in [unused1] B ##V - all ##ergic [unused2] patients . We further evaluated the sensitivity of r ##A ##pi m 1 and 2 of an alternative platform and investigated possible differences in the se ##ns ##iti ##zation profile between mon ##ose ##ns ##iti ##zation and clinical ##ly relevant double se ##ns ##iti ##zation . Analysis of the whole panel of bee venom all ##er ##gens of the CA [SEP]',
  'existing_cui': 'C0282555',
  'existing_cui_name': 'Anti-Allergic Agents',
  'existing_cui_desc': '[CLS] Anti - All ##ergic Agents [unused3] ( Chemical , product , substance : Anti All ##ergic Agents ; AN ##TI ##AL ##LE ##R ##GI ##C AG ##EN ##TS ; Agents , Anti ##alle ##rgy ; Anti ##alle ##rgy Agents ; anti all ##er ##gy drugs ; Agents , Anti ##alle ##rg ##ic ; anti ##alle ##rg ##ic drugs ; anti - all ##er ##gy drugs ; anti ##alle ##rg ##ic drug ; Drugs , Anti ##alle ##rg ##ic ; Anti All ##ergic ##s ; Drugs , Anti - All ##er ##gy ; all ##ergic anti ; Anti - All ##ergic ##s ; Anti ##alle ##rgen ##ic drugs ; Anti ##alle ##rg ##ics ; anti ##alle ##rg ##ic ; Agents , Anti - All ##ergic ; anti [SEP]',
  'our_bi_cui': 'C0571832',
  'our_bi_cui_name': 'Vitamin B group allergy',
  'our_bi_cui_desc': '[CLS] Vita ##min B group all ##er ##gy [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder ) [SEP]',
  'our_cross_cui': 'C1828283',
  'our_cross_cui_name': 'Allergy to bee venom',
  'our_cross_cui_desc': '[CLS] All ##er ##gy to bee venom [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder ) [SEP]'},
 {'mention_id': '27507702.51',
  'test_set_idx': 8767,
  'type': 'T103',
  'name': 'tri -',
  'context': '[CLS] ##ed . The most prominent protein motifs include the zinc finger , rib ##on ##uc ##lea ##se H , reverse trans ##cript ##ase , and an ##ky ##rin repeat domains . The simple sequence repeats identified from > 1 k ##b length of un ##ige ##nes show a do ##mina ##ncy of din ##uc ##leo ##tide repeat motifs followed with [unused1] t ##ri - [unused2] and te ##tra ##nu ##cle ##ot ##ide motifs . A number of un ##ige ##nes were put ##ative ##ly assessed to belong to adaptation and defense mechanisms including heat shock proteins 70 , To ##ll - like receptor 4 , AM ##P - activated protein kinase , a ##qua ##por ##in - 2 , etc . Our data provide a rich [SEP]',
  'existing_cui': 'C0065404',
  'existing_cui_name': 'lysyl-5-fluorotryptophyl-lysine',
  'existing_cui_desc': '[CLS] l ##ys ##yl - 5 - flu ##oro ##try ##pt ##op ##hyl - l ##ys ##ine [unused3] ( Chemical , L : L ##ys - F - T ##rp - L ##ys ; Tri ) [SEP]',
  'our_bi_cui': 'C0032550',
  'our_bi_cui_name': 'Polynucleotides',
  'our_bi_cui_desc': '[CLS] Pol ##yn ##uc ##leo ##tide ##s [unused3] ( Chemical , substance : p ##oly ##nu ##cle ##ot ##ide ; Pol ##yn ##uc ##leo ##tide ##s [ Chemical / In ##g ##red ##ient ] ; Pol ##yn ##uc ##leo ##tide ) [SEP]',
  'our_cross_cui': 'C0282537',
  'our_cross_cui_name': 'Trinucleotide Repeats',
  'our_cross_cui_desc': '[CLS] Tri ##nu ##cle ##ot ##ide Rep ##eat ##s [unused3] ( Chemical : Rep ##eat ##s , Triple ##t ; triple ##t repeats ; Triple ##t Rep ##eat ##s ; t ##rin ##uc ##leo ##tide repeats ; triple ##t repeat ; Rep ##eat , Tri ##nu ##cle ##ot ##ide ; Triple ##t Rep ##eat ; Rep ##eat ##s , Tri ##nu ##cle ##ot ##ide ; Tri ##nu ##cle ##ot ##ide repeats ; Rep ##eat , Triple ##t ; t ##rin ##uc ##leo ##tide repeat ; Tri ##nu ##cle ##ot ##ide Rep ##eat ) [SEP]'},
 {'mention_id': '27669217.53',
  'test_set_idx': 13704,
  'type': 'T017',
  'name': 'ermB',
  'context': '[CLS] ##ed with T ##n ##9 ##16 and IS ##12 ##16 , respectively . The expression of both er ##m ##B and er ##m ##T in all is ##olate ##s was er ##yt ##hr ##omy ##cin in ##du ##cible and yielded comparable mac ##rol ##ide MI ##Cs in all six is ##olate ##s . Take ##n together , in ##du ##cible expression of both er ##m ##B and er ##m ##T conferred high mac ##rol ##ide resistance in these S . gal ##lo ##ly ##ticus subsp . paste ##rian ##us is ##olate ##s . Our findings reveal new mac ##rol ##ide resistance features in S . gal ##lo ##ly ##ticus subsp . paste ##uria ##nus by both [unused1] er ##m ##B [unused2] and er ##m ##T . [SEP]',
  'existing_cui': 'C0732611',
  'existing_cui_name': 'Selective Estrogen Receptor Modulators',
  'existing_cui_desc': '[CLS] Select ##ive E ##stro ##gen Re ##ceptor Mo ##du ##lator ##s [unused3] ( Chemical , SE ##R ##M , medication , product , substance : E ##ST ##RO ##GE ##N R ##EC ##EP ##T M ##OD ##U ##LA ##TO ##RS SE ##LE ##CT ##IVE ; Select ##ive E ##stro ##gen Re ##ceptor Mo ##di ##fying Agents ; Select ##ive est ##rogen receptor m ##od ##ulator ##s ; Select ##ive o ##est ##rogen receptor m ##od ##ulator ; E ##stro ##gen Re ##ceptor Mo ##du ##lator , Select ##ive ; SE ##R ##M - Select ##ive est ##rogen receptor m ##od ##ulator ; SE ##R ##Ms ; SE ##R ##M - Select ##ive o ##est ##rogen receptor m ##od ##ulator ; SE ##LE ##CT ##IVE E ##ST [SEP]',
  'our_bi_cui': 'C0243036',
  'our_bi_cui_name': 'Genes , erbB',
  'our_bi_cui_desc': '[CLS] Gene ##s , er ##b ##B [unused3] ( Ana ##tom ##ical Structure : A ##vian E ##ry ##th ##ro ##blast ##osis V ##ir ##us On ##co ##gene B ; E ##ry ##th ##ro ##blast Trans ##form ##ing Gene B ; On ##co ##gene , E ##G ##F ##R ; On ##co ##gene ER ##B B ; er ##b ##B Gene ##s ; er ##b ##B Gene ; ER ##B - B ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '28107521.38',
  'test_set_idx': 25785,
  'type': 'T017',
  'name': 'SW480',
  'context': '[CLS] the W ##nt signaling pathway still remains incomplete ##ly understood , we sought to el ##uc ##ida ##te their relative contribution in the formation of de ##grad ##as ##ome ##s , as these protein assemblies most likely represent the m ##or ##phological and functional co ##rrel ##ates of end ##ogen ##ous β - cat ##eni ##n destruction complexes . In [unused1] S ##W ##48 ##0 [unused2] color ##ec ##tal cancer cells treated with the tank ##yra ##se inhibitor G ##00 ##7 - L ##K we found that A ##X ##IN ##1 was not required for de ##grad ##as ##ome formation . In contrast , the formation of de ##grad ##as ##ome ##s as well as their capacity to de ##grade β - cat ##eni ##n were [SEP]',
  'existing_cui': 'C3538902',
  'existing_cui_name': 'SLC4A1 wt Allele',
  'existing_cui_desc': '[CLS] SL ##C ##4 ##A ##1 w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : F ##R ; Sol ##ute Carrier Family 4 , An ##ion Exchange ##r , Member 1 ( E ##ry ##th ##ro ##cy ##te Me ##mb ##rane Pro ##tein Band 3 , Diego Blood Group ) w ##t All ##ele ; W ##D ; D ##I ; A ##E 1 ; W ##D ##1 ; CD ##23 ##3 ; WR ; B ##ND ##3 ; R ##TA ##1 ##A ) [SEP]',
  'our_bi_cui': 'C3006953',
  'our_bi_cui_name': 'Staphylococcus sp . SW48',
  'our_bi_cui_desc': '[CLS] St ##aph ##yl ##oc ##oc ##cus s ##p . S ##W ##48 [unused3] ( Ba ##cter ##ium ) [SEP]',
  'our_cross_cui': 'C0007634',
  'our_cross_cui_name': 'Cells',
  'our_cross_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]'},
 {'mention_id': '27621200.11',
  'test_set_idx': 12043,
  'type': 'T038',
  'name': 'kindness',
  'context': '[CLS] exploring the relationship between self - compassion , self - judgement , self - kindness , compassion , professional quality of life and well ##bei ##ng among UK community nurses Co ##mp ##ass ##ion fatigue and burn ##out can impact on performance of nurses . This paper explores the relationship between self - compassion , self - judgement , self - [unused1] kindness [unused2] , compassion , professional quality of life , and well ##bei ##ng among community nurses . To measure associations between self - compassion , compassion fatigue , well ##bei ##ng , and burn ##out in community nurses . Q ##uant ##itative data were collected using standard ##ised ps ##ych ##ometric question ##naire ##s : ( 1 ) Professional Quality of Life Scale [SEP]',
  'existing_cui': 'C0456909',
  'existing_cui_name': 'Blind Vision',
  'existing_cui_desc': '[CLS] Blind Vision [unused3] ( Finding , finding : Blind ##ness ; blind vision ; Unable to see ; Blind ##ness [ Disease / Finding ] ; Can not see ; Blind ; blind ##ness ; U ##TS - Unable to see ; blind ) [SEP]',
  'our_bi_cui': 'C0020039',
  'our_bi_cui_name': 'Hostility',
  'our_bi_cui_desc': '[CLS] Host ##ility [unused3] ( B ##iol ##og ##ic Fun ##ction , context - dependent category , finding , physical finding , situation , s ##ym ##pt ##om : [ D ] Host ##ility ; attitude hostile ; Host ##ile behavior ; Host ##ile behaviour ; hostile ; hostilities ; hostility ; Host ##ilities ) [SEP]',
  'our_cross_cui': 'C0871641',
  'our_cross_cui_name': 'affection',
  'our_cross_cui_desc': '[CLS] affection [unused3] ( B ##iol ##og ##ic Fun ##ction : A ##ffe ##ction ; affection ##s ) [SEP]'},
 {'mention_id': '28072576.24',
  'test_set_idx': 24800,
  'type': 'T033',
  'name': 'detection',
  'context': '[CLS] ( - 1 ) Iron oxide na ##no ##par ##tic ##les ) . I ##maging based on this property has shown a substantial contrast - to - noise ratio improvement ( up to 5 fold , p < 0 . 01 ) . The speed of sound - related effect generated a well disc ##ern ##ible image contrast and allowed the [unused1] detection [unused2] of the particles existence and location , in both r ##aster - scan projection and ultra ##sonic com ##puted to ##mo ##graphy imaging . Conversely , no significant change in the acoustic at ##ten ##uation coefficient was noted . Based on these findings , it is concluded that Iron oxide na ##no ##par ##tic ##les can be used as an effective speed [SEP]',
  'existing_cui': 'C0206100',
  'existing_cui_name': 'Signal Detection',
  'existing_cui_desc': '[CLS] Signal Det ##ec ##tion [unused3] ( B ##iol ##og ##ic Fun ##ction , Psychology , Psychology , Signal : Signal Det ##ec ##tion Theo ##ries ; Signal Det ##ec ##tions , Psychological ; Theory , Signal Det ##ec ##tion ; Signal detection ; S ##IG ##NA ##L DE ##TE ##CT ##ION AN ##AL ; Psychological Signal Det ##ec ##tions ; Det ##ec ##tion ; Signal Det ##ec ##tion Ana ##ly ##ses ; S ##IG ##NA ##L DE ##TE ##CT ##ION ; Det ##ec ##tions , Psychological Signal ; Psychological Signal Det ##ec ##tion ; Signal Det ##ec ##tion Theory ; Theo ##ries , Signal Det ##ec ##tion ; Signal Det ##ec ##tions ; Ana ##ly ##ses , Signal Det ##ec ##tion ; Det ##ec ##tion , [SEP]',
  'our_bi_cui': 'C1511790',
  'our_bi_cui_name': 'Detection',
  'our_bi_cui_desc': '[CLS] Det ##ec ##tion [unused3] ( Health Care Act ##ivity : Det ##ec ##ted ) [SEP]',
  'our_cross_cui': 'C0442726',
  'our_cross_cui_name': 'Detected',
  'our_cross_cui_desc': '[CLS] Det ##ec ##ted [unused3] ( Finding , finding , finding , qualifier value : detected ; detect ) [SEP]'},
 {'mention_id': '28424683.67',
  'test_set_idx': 35445,
  'type': 'T038',
  'name': 'expand',
  'context': '[CLS] ##city but higher secret ##ion of IF ##N - γ . Ex ##pan ##sion and functional activation of super - charged Natural killer cells by O ##Cs is dependent on inter ##le ##uki ##n ( inter ##le ##uki ##n ) - 12 and IL - 15 . Thus , in this report , we not only provide a novel strategy to [unused1] expand [unused2] super - charged Natural killer cells , but also demonstrate that rapid and sustained expansion of residual T cells within the pu ##rified Natural killer cells during expansion with den ##dr ##itic cells or O ##Cs could be a potential mechanism by which the numbers and function of Natural killer cells decline in cancer patients and in B ##LT - human ##ized [SEP]',
  'existing_cui': 'C1263424',
  'existing_cui_name': 'LYSP100-associated nuclear domain',
  'existing_cui_desc': '[CLS] L ##Y ##SP ##100 - associated nuclear domain [unused3] ( Ana ##tom ##ical Structure : LA ##ND ##s ; lands ; landed ) [SEP]',
  'our_bi_cui': 'C0205229',
  'our_bi_cui_name': 'Expanding',
  'our_bi_cui_desc': '[CLS] Ex ##pan ##ding [unused3] ( Spa ##tial Concept , qualifier value : Ex ##pan ##d ; Ex ##pan ##ded ; expanding ; expand ; expanded ) [SEP]',
  'our_cross_cui': 'C0007595',
  'our_cross_cui_name': 'cell growth',
  'our_cross_cui_desc': '[CLS] cell growth [unused3] ( B ##iol ##og ##ic Fun ##ction : cell expansion ; cellular growth ; Cell ##s - - Growth ; Cell Growth ; cell growth ##s ; cells growth ; growth cell ; growth of cell ; Cell ##ular Growth ; Cell ##ular Ex ##pan ##sion ) [SEP]'},
 {'mention_id': '27921269.21',
  'test_set_idx': 21918,
  'type': 'T033',
  'name': 'had sex',
  'context': '[CLS] structure , arising from goodness of membership judgment ##s . Mode ##ration of this graded structure is also predicted , with the focus here on how sexual orientation identity affects sexual definitions . A total of 300 18 - to 30 - year - old participants completed an online survey , rating 18 behaviors for how far each constitutes having " [unused1] had sex [unused2] " and virgin ##ity loss . Part ##ici ##pants fell into one of four groups : he ##tero ##sexual male or female , gay male or lesbian . The predicted ratings hierarchy emerged , in which bid ##ire ##ctional g ##eni ##tal acts were rated significantly higher than un ##id ##ire ##ctional or non ##pen ##et ##rative contact , which was [SEP]',
  'existing_cui': 'C1314687',
  'existing_cui_name': 'Sexual intercourse - finding',
  'existing_cui_desc': '[CLS] Sexual intercourse - finding [unused3] ( Finding , finding : Finding of sexual intercourse ; O ##bs ##er ##vation of sexual intercourse ; Making love ; Sexual intercourse ; Sexual act ; Co ##it ##us ; Finding of sexual intercourse [ du ##p ] ; Sex ; S ##I - Sexual intercourse ; V ##agi ##nal intercourse ) [SEP]',
  'our_bi_cui': 'C3829825',
  'our_bi_cui_name': 'Has HIV or AIDS',
  'our_bi_cui_desc': '[CLS] Has HIV or AIDS [unused3] ( Intel ##lect ##ual Product : Does the patient have HIV or AIDS ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27905972.10',
  'test_set_idx': 21624,
  'type': 'T170',
  'name': 'paper',
  'context': '[CLS] models to describe the full age pattern of mortality are not able to capture mortality delay versus mortality compression . More recent models that assess delay versus compression mostly focused on the adult or old ages alone and did not distinguish mortality compression below and above the m ##oda ##l age at death , although they represent different mechanisms . This [unused1] paper [unused2] presents a new para ##metric model that describes the full age pattern of mortality and assess ##es compression - at different stages of life - and delay of mortality : the Co ##D ##e model . The model includes 10 parameters , of which five are constant over time . The five time - varying parameters reflect delay of mortality and compression [SEP]',
  'existing_cui': 'C0462999',
  'existing_cui_name': 'Litmus paper',
  'existing_cui_desc': '[CLS] Li ##t ##mus paper [unused3] ( Medical Devi ##ce , physical object : Li ##t ##mus Paper ; lit ##mus papers ; Re ##age ##nts , Clinical Chemistry , Rapid Test , pH , Strip ; IV ##D Test Re ##age ##nt / Kit ##s , Clinical Chemistry , Rapid Test , pH , Strip ; pH Strip In ##dic ##ators ; lit ##mus paper ) [SEP]',
  'our_bi_cui': 'C0600659',
  'our_bi_cui_name': 'Manuscripts',
  'our_bi_cui_desc': '[CLS] Man ##us ##cripts [unused3] ( Intel ##lect ##ual Product : manuscript ; manuscripts ; Man ##us ##cript ) [SEP]',
  'our_cross_cui': 'C1706852',
  'our_cross_cui_name': 'Article',
  'our_cross_cui_desc': '[CLS] Article [unused3] ( Intel ##lect ##ual Product ) [SEP]'},
 {'mention_id': '28277189.51',
  'test_set_idx': 31002,
  'type': 'T017',
  'name': 'hFoB1 .19 cells',
  'context': '[CLS] significantly suppressed U2 ##OS cell migration and invasion . On the contrary , over ##ex ##press ##ion of C ##X ##LC ##5 remarkably promoted the migration and invasion of U2 ##OS cells . Interest ##ingly , both ex ##ogen ##ous C ##X ##CL ##5 treatment and the conditioned medium of C ##X ##CL ##5 - over ##ex ##pressing [unused1] h ##F ##o ##B ##1 . 19 cells [unused2] could also enhance the migration and invasion of U2 ##OS cells , suggesting that the promoting role of C ##X ##CL ##5 in U2 ##OS cell migration and invasion is also in a para ##c ##rine - dependent manner . According to these data , our study demonstrates that C ##X ##CL ##5 is up ##re ##gu ##lated [SEP]',
  'existing_cui': 'C3640056',
  'existing_cui_name': 'CD19CAR-CD3zeta-4-1BB-expressing Allogeneic T-lymphocyte Cells',
  'existing_cui_desc': '[CLS] CD ##19 ##CA ##R - CD ##3 ##ze ##ta - 4 - 1 ##BB - expressing All ##ogen ##ei ##c T - l ##ymph ##oc ##yte Cell ##s [unused3] ( Ana ##tom ##ical Structure : CA ##RT - 19 Cell ##s ) [SEP]',
  'our_bi_cui': 'C0007634',
  'our_bi_cui_name': 'Cells',
  'our_bi_cui_desc': '[CLS] Cell ##s [unused3] ( Ana ##tom ##ical Structure , Biology , body structure , cell structure : Cell ##s set ; Cell Type ; cells ; { Cell ##s } ; Normal Cell ; Set of cells ; Cell ##ular structures ; Cell structure ; Cell ; cell ; Cell ##ular structure ; the cell ; Cell ##ular ; Cell Type ##s ; Normal cell ; Cell ##ula ) [SEP]',
  'our_cross_cui': 'C0029418',
  'our_cross_cui_name': 'Osteoblasts',
  'our_cross_cui_desc': '[CLS] O ##ste ##ob ##last ##s [unused3] ( Ana ##tom ##ical Structure , cell : o ##ste ##ob ##last ##s ; o ##ste ##ob ##last ; cells o ##ste ##ob ##last ##s ; cells o ##ste ##ob ##last ; O ##ste ##ob ##last ) [SEP]'},
 {'mention_id': '28441489.18',
  'test_set_idx': 36140,
  'type': 'T033',
  'name': 'cold - start',
  'context': '[CLS] ##yna ##mo ##meter using the cold - start unified cycle . The fleet included 15 gasoline direct - injection vehicles , including 8 G ##DI ##s certified to the most - string ##ent emissions standard , super ##ult ##ra - low - emission vehicles . We q ##uant ##ified the effects of engine technology , emission certification standards , and [unused1] cold - start [unused2] on emissions . For vehicles certified to the same emissions standard , there is no statistical difference of regulated gas - phase poll ##uta ##nt emissions between P ##FI ##s and G ##DI ##s . However , G ##DI ##s had , on average , a factor of 2 higher part ##iculate matter mass emissions than P ##FI ##s due to [SEP]',
  'existing_cui': 'C0452588',
  'existing_cui_name': 'Start brand of breakfast cereal',
  'existing_cui_desc': '[CLS] Start brand of breakfast cereal [unused3] ( Food , substance : Start ) [SEP]',
  'our_bi_cui': 'C0022885',
  'our_bi_cui_name': 'Laboratory Procedures',
  'our_bi_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27587730.21',
  'test_set_idx': 10760,
  'type': 'T103',
  'name': 'gga-miR-21',
  'context': "[CLS] ##ga - mi ##R - 21 in chicken pre - ad ##ip ##oc ##yte proliferation . Cell proliferation ass ##ay and gene expression analysis of pro ##life ##rating cell nuclear anti ##gen showed that the g ##ga - mi ##R - 21 mimic in ##hibit ##ed pre - ad ##ip ##oc ##yte proliferation . In contrast , the [unused1] g ##ga - mi ##R - 21 [unused2] inhibitor enhanced pre - ad ##ip ##oc ##yte proliferation . The subsequent investigation identified K ##rup ##pel - like factor 5 m ##RNA as a target of g ##ga - mi ##R - 21 . The g ##ga - mi ##R - 21 mimic in ##hibit ##ed K ##rup ##pel - like factor 5 3 ( ' ) U [SEP]",
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1537719',
  'our_bi_cui_name': 'MIR21 gene',
  'our_bi_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '27430240.52',
  'test_set_idx': 5378,
  'type': 'T058',
  'name': 'dual - luciferase reporter assay',
  'context': '[CLS] E ##2 ##F transcription factor 1 or c ##yclic ad ##eno ##sin ##e mon ##op ##hos ##phate response element - binding protein ( CR ##E ##B ) , which are predicted transcription regulatory factors of CD ##39 or CD ##7 ##3 . The regulatory relationships among these factors were then determined by western b ##lot analysis and [unused1] dual - l ##uc ##ifer ##ase reporter ass ##ay [unused2] . In addition , changes in ad ##eno ##sin ##e metabolism were measured in the treated cells . The results revealed that ad ##eno ##sin ##e and C ##GS ##21 ##6 ##80 significantly up ##re ##gu ##lated CD ##39 and CD ##7 ##3 expression ( P < 0 . 01 ) . E ##2 ##F transcription factor 1 [SEP]',
  'existing_cui': 'C1324309',
  'existing_cui_name': 'Latia-luciferin monooxygenase ( demethylating ) activity',
  'existing_cui_desc': '[CLS] La ##tia - l ##uc ##ifer ##in mon ##oo ##xy ##gen ##ase ( dem ##eth ##yla ##ting ) activity [unused3] ( B ##iol ##og ##ic Fun ##ction , La ##tia l ##uc ##ifer ##in , dem ##eth ##yla ##ting : l ##uc ##ifer ##ase ; La ##tia - l ##uc ##ifer ##in , hydrogen - donor : oxygen o ##xi ##dor ##ed ##uc ##tase ; La ##tia l ##uc ##ifer ##in mon ##oo ##xy ##gen ##ase ) [SEP]',
  'our_bi_cui': 'C0005507',
  'our_bi_cui_name': 'Biological Assay',
  'our_bi_cui_desc': '[CLS] Biological As ##say [unused3] ( Health Care Act ##ivity , procedure : B ##io ##ass ##ays ; B ##iol ##og ##ic As ##say ##s ; B ##IO ##L AS ##SA ##Y ; B ##IO ##L AS ##SA ##Y ##S ; As ##say ; As ##say ##s , Biological ; bio ##ass ##ay ; As ##say , Biological ; bio ##ass ##ays ; B ##io ##ass ##ay ; Biological As ##say ##s ; AS ##SA ##Y B ##IO ##L ; B ##iol ##og ##ic As ##say ; As ##say , B ##iol ##og ##ic ; AS ##SA ##Y ##S B ##IO ##L ; As ##say ##s , B ##iol ##og ##ic ; Biological ass ##ay ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '27762482.20',
  'test_set_idx': 15898,
  'type': 'T074',
  'name': 'PDL',
  'context': '[CLS] ##ura , we designed the fraction ##ation of high flu ##ence using five passes and longer pulse duration ( 6 mill ##ise ##con ##ds ) of a PD ##L in succession . A total of eight patients with er ##yt ##hem ##ato ##tel ##ang ##ie ##ct ##atic r ##osa ##cea were enrolled in this study ; all patients were treated with [unused1] PD ##L [unused2] 10 times at 2 - week intervals . E ##ry ##them ##a and te ##lang ##ie ##ct ##asi ##a scores , as well as improvement , were assessed by two physicians using the digital photographs . Mode ##rate - to - marked improvement was achieved in most of the patients , and er ##yt ##hem ##a and te ##lang ##ie ##ct [SEP]',
  'existing_cui': 'C1414066',
  'existing_cui_name': 'DLG5 gene',
  'existing_cui_desc': '[CLS] DL ##G ##5 gene [unused3] ( Ana ##tom ##ical Structure : K ##IA ##A ##0 ##5 ##8 ##3 ; D ##IS ##CS LA ##R ##GE , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF , 5 ; P - d ##l ##g ; discs large ho ##mo ##log 5 ; P ##LA ##CE ##NT ##A AND PR ##OS ##TA ##TE DL ##G ) [SEP]',
  'our_bi_cui': 'C0031740',
  'our_bi_cui_name': 'Photochemotherapy',
  'our_bi_cui_desc': '[CLS] Photo ##che ##mother ##ap ##y [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : PD ##T - Photo ##dynamic therapy ; Photo ##che ##mother ##ap ##ies ; Photo ##dynamic The ##rap ##ies ; therapy , photo ##dynamic ; Therapy , Photo ##dynamic ; THE ##R P ##H ##OT ##OD ##Y ##NA ##MI ##C ; Photo ##rad ##iation Therapy ; photo ##dynamic therapy ; photo ##che ##mother ##ap ##y ; P ##H ##OT ##OC ##H ##EM ##OT ##H ##ER ; The ##rap ##ies , Photo ##dynamic ; Photo ##dynamic Therapy Pro ##ced ##ures ; neo ##p ##las ##m / cancer photo ##rad ##iation therapy ; Photo ##che ##mother ##ap ##y [ Am ##bi ##guous ] ; Photo ##dynamic therapy ; PD [SEP]',
  'our_cross_cui': 'C1289859',
  'our_cross_cui_name': 'Pulsed Dye Lasers',
  'our_cross_cui_desc': '[CLS] P ##ulse ##d D ##ye Las ##ers [unused3] ( Medical Devi ##ce , physical object : D ##ye Las ##ers , P ##ulse ##d ; P ##ulse ##d dye laser device ; Las ##er , P ##ulse ##d D ##ye ; D ##ye Las ##er , P ##ulse ##d ; Las ##ers , P ##ulse ##d D ##ye ; P ##ulse ##d D ##ye Las ##er ; pulsed dye laser ; P ##ulse ##d dye laser ) [SEP]'},
 {'mention_id': '28197156.12',
  'test_set_idx': 28082,
  'type': 'T017',
  'name': 'Carbon Assimilation Rate 8',
  'context': '[CLS] in rice ( Or ##y ##za sat ##iva ) . Ex ##p ##lo ##iting the natural variation in CO ##2 ass ##im ##ilation rate ( A ) between rice cult ##iva ##rs using quantitative genetics is one promising means to identify genes contributing to higher photos ##ynth ##esis . In this study , we determined precise location of [unused1] Carbon As ##si ##mi ##lation Rate 8 [unused2] by crossing a high - yielding in ##dic ##a cult ##iva ##r with a Japanese commercial cult ##iva ##r . Fine mapping suggested that Carbon As ##si ##mi ##lation Rate 8 en ##codes a put ##ative He ##me Act ##iva ##tor Pro ##tein 3 ( O ##s ##HA ##P ##3 ) subunit of a CC ##AA ##T - box [SEP]',
  'existing_cui': 'C2246620',
  'existing_cui_name': 'iron assimilation',
  'existing_cui_desc': '[CLS] iron ass ##im ##ilation [unused3] ( B ##iol ##og ##ic Fun ##ction ) [SEP]',
  'our_bi_cui': 'C1413023',
  'our_bi_cui_name': 'C8A gene',
  'our_bi_cui_desc': '[CLS] C ##8 ##A gene [unused3] ( Ana ##tom ##ical Structure : complement component 8 alpha subunit ; CO ##MP ##LE ##ME ##NT CO ##MP ##ON ##EN ##T 8 , AL ##P ##HA S ##U ##B ##UN ##IT ; C ##8 AL ##P ##HA ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27373772.13',
  'test_set_idx': 4071,
  'type': 'T038',
  'name': 'functioning',
  'context': '[CLS] and ne ##uron ##al commitment in the rat den ##tate g ##yr ##us Adult hip ##po ##cam ##pal ne ##uro ##genesis can be m ##od ##ulated by various physiological and path ##ological conditions , including stress , affect ##ive disorders , and several ne ##uro ##logical conditions . Given the proposed role of this form of structural plastic ##ity in the [unused1] functioning [unused2] of the hip ##po ##cam ##pus ( namely learning and memory and affect ##ive behaviors ) , it is believed that alterations in hip ##po ##cam ##pal ne ##uro ##genesis might under ##lie some of the behavioral deficit ##s associated with these psychiatric and ne ##uro ##logical conditions . Thus , the search for compounds that can reverse these deficit ##s with [SEP]',
  'existing_cui': 'C2986655',
  'existing_cui_name': 'Functioning Endocrine Neoplasm',
  'existing_cui_desc': '[CLS] Fun ##ction ##ing End ##oc ##rine Neo ##p ##las ##m [unused3] ( B ##iol ##og ##ic Fun ##ction : functioning tumor ) [SEP]',
  'our_bi_cui': 'C0007613',
  'our_bi_cui_name': 'Cell physiology',
  'our_bi_cui_desc': '[CLS] Cell physiology [unused3] ( B ##iol ##og ##ic Fun ##ction : cell physiology ; Cell ##ular Process ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##a ; Cell ##ular Fun ##ction ; cells physiology ; Cell ##ular Ph ##ys ##iology ; Sub ##cellular Process ; cells function ; cell function ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##on ; Cell Ph ##ys ##iology ; Cell Process ; Cell Fun ##ction ; CE ##LL P ##H ##Y ##SI ##OL ; P ##H ##Y ##SI ##OL CE ##LL ; cell functions ; Ph ##eno ##men ##a , Cell Ph ##ys ##iol ##ogical ; Ph ##ys ##iology , Cell ; Ph ##eno ##men ##on , Cell Ph ##ys ##iol ##ogical ) [SEP]',
  'our_cross_cui': 'C0031843',
  'our_cross_cui_name': 'physiological aspects',
  'our_cross_cui_desc': '[CLS] physiological aspects [unused3] ( B ##iol ##og ##ic Fun ##ction : P ##H ; physiology ; P ##H ##Y ##SI ##OL ; function ) [SEP]'},
 {'mention_id': '28125734.41',
  'test_set_idx': 26195,
  'type': 'T103',
  'name': 'miR-21-5p',
  'context': '[CLS] o ##li ##gos were employed to is ##olate mi ##RNA : : MS ##L ##N m ##RNA complexes from a normal cell line ( Met - 5 ##A ) which expresses low levels of me ##so ##the ##lin . Mi ##RNA ##s targeting the me ##so ##the ##lin m ##RNA were identified by next generation se ##quencing and [unused1] mi ##R - 21 - 5 ##p [unused2] and mi ##R - 100 - 5 ##p were selected for further valid ##ation analyses . Mi ##R - 21 - 5 ##p was shown to be able to m ##od ##ulate me ##so ##the ##lin expression in mi ##RNA mimic experiments in a panel of ma ##li ##gnant and non - ma ##li ##gnant cell lines . Further [SEP]',
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1999986',
  'our_bi_cui_name': 'MIRN21 microRNA , human',
  'our_bi_cui_desc': '[CLS] MI ##R ##N ##21 micro ##RNA , human [unused3] ( Chemical : micro ##RNA - 21 , human ; mi ##RNA - 21 , human ; h ##sa - mi ##r - 21 micro ##RNA ; mi ##R - 21 , human ; mi ##R ##21 , human ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '28183596.8',
  'test_set_idx': 27754,
  'type': 'T033',
  'name': 'reduce/minimize',
  'context': '[CLS] A numerical study on the application of the functional ##ly graded materials in the s ##ten ##t design Un ##des ##ira ##ble def ##orm ##ation of the s ##ten ##t can induce a significant amount of in ##jure not only to the blood vessel but also to the plaque . The objective of this study was to [unused1] reduce / minimize [unused2] these und ##esi ##rable def ##orm ##ations by the application of Fun ##ctional ##ly Grade ##d Materials . To do this , Fin ##ite El ##ement method was employed to si ##mu ##late the expansion of a s ##ten ##t and the corresponding displacement of the s ##ten ##osis plaque . Three h ##yper ##ela ##stic plaque types as well as five el ##ast [SEP]',
  'existing_cui': 'C0340305',
  'existing_cui_name': 'Inferior Wall Myocardial Infarction',
  'existing_cui_desc': '[CLS] In ##fer ##ior Wall My ##oc ##ard ##ial In ##far ##ction [unused3] ( B ##iol ##og ##ic Fun ##ction , disorder : Di ##aph ##rag ##matic My ##oc ##ard ##ial In ##far ##ction ; inferior my ##oc ##ard ##ial in ##far ##ction ; In ##fer ##ior MI ; My ##oc ##ard ##ial In ##far ##ction , In ##fer ##ior ; Di ##aph ##rag ##matic My ##oc ##ard ##ial In ##far ##ctions ; In ##fer ##ior my ##oc ##ard ##ial in ##far ##ction ; In ##far ##ctions , Di ##aph ##rag ##matic My ##oc ##ard ##ial ; inferior mi ; My ##oc ##ard ##ial In ##far ##ction , In ##fer ##ior Wall ; In ##fer ##ior My ##oc ##ard ##ial In ##far ##ctions ; My ##oc ##ard [SEP]',
  'our_bi_cui': 'C0441610',
  'our_bi_cui_name': 'Reduction - action',
  'our_bi_cui_desc': '[CLS] Red ##uction - action [unused3] ( Health Care Act ##ivity , qualifier value : Sur ##gical reduction ; reduction ##s ; reduction ; Red ##uction ; Red ##uction p ##last ##y ; surgical reduction ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27280034.3',
  'test_set_idx': 1332,
  'type': 'T082',
  'name': 'Newcastle upon Tyne',
  'context': "[CLS] Changes in risk factors for young male suicide in [unused1] Newcastle upon Tyne [unused2] , 1961 - 2009 Ai ##ms and method To as ##cer ##tain differences in patterns of suicide in young men over three decades ( 1960s , 1990s and 2000s ) and discuss implications for suicide prevention . Data on suicide ##s and open verdict ##s in men aged 15 - 34 were obtained from co ##roner ' s records in Newcastle upon Tyne and anal ##ys ##ed using SP ##SS software . Results An increase in suicide rates from the first to the second decade was followed by a fall in the third decade . This was associated with an increasing proportion of single men , those living alone , unemployment , [SEP]",
  'existing_cui': 'C0027983',
  'existing_cui_name': 'Newcastle Disease',
  'existing_cui_desc': "[CLS] Newcastle Disease [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , et ##iology , manifest ##ation : Disease , Newcastle ; new ##castle disease ; Newcastle ' s disease ; A ##vian p ##ne ##um ##oe ##nce ##pha ##lit ##is ; Newcastle disease [ Am ##bi ##guous ] ; P ##se ##udo - f ##ow ##l ##pes ##t ; disease ( or disorder ) ; Newcastle ; Newcastle disease ; Newcastle ; NE ##WC ##AS ##TL ##E D ##IS ; Newcastle fever ; Newcastle Disease [ Disease / Finding ] ) [SEP]",
  'our_bi_cui': 'C0454880',
  'our_bi_cui_name': 'Tyne and Wear',
  'our_bi_cui_desc': '[CLS] Tyne and We ##ar [unused3] ( Spa ##tial Concept , geographic location : T and W - Tyne and We ##ar ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '28077286.22',
  'test_set_idx': 25072,
  'type': 'T103',
  'name': 'MeHg',
  'context': '[CLS] ##air ##ments to cognitive motor functions in ma ##mmal ##ian models . However , in birds the ne ##uro ##to ##xi ##c effects of Me ##H ##g on the developing pre - hatch ##ing brain and con ##se ##quent behavioral alterations in adult birds have not received much attention . Moreover , pass ##eri ##ne birds are poorly represented in [unused1] Me ##H ##g [unused2] ne ##uro ##to ##xi ##cology studies in comparison to other a ##vian orders . Hence in this study , we used the egg injection method to investigate the long term effects of in o ##vo Me ##H ##g exposure on brain his ##top ##ath ##ology and courts ##hip behavior in a model song ##bird species , the z ##eb ##ra fin [SEP]',
  'existing_cui': 'C0228520',
  'existing_cui_name': 'Structure of reticular formation of medulla oblongata',
  'existing_cui_desc': '[CLS] Structure of re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata [unused3] ( Ana ##tom ##ical Structure , body structure : bulb ##ar re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla ; B ##ul ##bar re ##tic ##ular formation ; Form ##ati ##o re ##tic ##ular ##is my ##ele ##nce ##pha ##li ; me ##du ##llar ##y re ##tic ##ular formation ; re ##tic ##ular formation of me ##du ##lla ; Me ##du ##llar ##y re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata ) [SEP]',
  'our_bi_cui': 'C0025424',
  'our_bi_cui_name': 'Mercury',
  'our_bi_cui_desc': '[CLS] Mercury [unused3] ( Chemical , H ##g , Metal , elemental , substance : mercury ; Mercury [ Chemical / In ##g ##red ##ient ] ; metal mercury ; H ##yd ##rar ##gy ##rum ; H ##g element ; Li ##quid Silver ; Quick ##si ##lve ##r ; H ##g ; H ##g - Mercury ; h ##g ; ME ##RC ##UR ##Y ) [SEP]',
  'our_cross_cui': 'C0025794',
  'our_cross_cui_name': 'Methylmercury Compounds',
  'our_cross_cui_desc': '[CLS] Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [unused3] ( Chemical : Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [ Chemical / In ##g ##red ##ient ] ; Mercury Co ##mpo ##unds , Met ##hyl ; ME ##TH ##Y ##L ME ##RC ##UR ##Y CP ##DS ; Met ##hyl Mercury Co ##mpo ##unds ; ME ##TH ##Y ##LM ##ER ##CU ##R ##Y CP ##DS ; Co ##mpo ##unds , Met ##hyl ##mer ##cu ##ry ; met ##hyl ##mer ##cu ##ry ; Co ##mpo ##unds , Met ##hyl Mercury ; Met ##hyl ##mer ##cu ##ry ) [SEP]'},
 {'mention_id': '28249376.9',
  'test_set_idx': 29879,
  'type': 'T168',
  'name': 'concentrate diet',
  'context': '[CLS] Effects of a specific blend of essential oils on apparent nut ##rient dig ##est ##ion , r ##ume ##n f ##er ##mentation and r ##ume ##n micro ##bial populations in sheep fed a 50 : 50 al ##fa ##lf ##a hay : [unused1] concentrate diet [unused2] An experiment was conducted to investigate the effects of a specific mixture of essential oils , containing thy ##me , c ##love and c ##innamon E ##O , on r ##ume ##n micro ##bial f ##er ##mentation , nut ##rient apparent dig ##est ##ibility and blood meta ##bol ##ites in fist ##ulated sheep . Six sheep fitted with r ##umi ##nal fist ##ula ##s were used in a repeated measurement design with two 24 - d periods to investigate the [SEP]',
  'existing_cui': 'C1880165',
  'existing_cui_name': 'Concentrate Dosage Form',
  'existing_cui_desc': '[CLS] Con ##cent ##rate Do ##sa ##ge Form [unused3] ( Chemical : Con ##cent ##rated ; Con ##cent ##rate ; CO ##NC ; CO ##NC ##EN ##TR ##AT ##E ; Con ##c ; Con ##cent ##rated Do ##sa ##ge Form ; Con ##cent ##rated Do ##se Form ) [SEP]',
  'our_bi_cui': 'C0003050',
  'our_bi_cui_name': 'Animal feed',
  'our_bi_cui_desc': '[CLS] Animal feed [unused3] ( Food , substance , substance : animal food ; animal feeds ; animal feeding ; animals food ; food animal ; Fe ##eds , Animal ; Animal Fe ##ed ; Fe ##ed , Animal ; animal feed ; Animal Fe ##eds ) [SEP]',
  'our_cross_cui': 'C0012155',
  'our_cross_cui_name': 'Diet',
  'our_cross_cui_desc': '[CLS] Diet [unused3] ( Food , finding : Diet ##s ; regular diet ? ; Diet ##ary ; diet ; Diet ##ary finding ; diet ##s ) [SEP]'},
 {'mention_id': '27886280.72',
  'test_set_idx': 20784,
  'type': 'T038',
  'name': 'production',
  'context': '[CLS] C ##ip ##ro ##f ##lo ##xa ##cin ( F ##lu ##oro ##quin ##olo ##ne ) at a low concentration resulted in killing of both cultures of P . a ##er ##ug ##ino ##sa , producing per ##sist ##ers that were in ##vu ##ln ##era ##ble to killing . Station ##ary cells appear to be somewhat more tolerant than ex ##po ##nent ##ial cells in all of these ass ##ays . We also showed that nut ##rient de ##p ##ri ##vation ( se ##rine starvation ) regulated by string ##ent and general stress response , contribute to the increased tolerance of P . a ##er ##ug ##ino ##sa ex ##po ##nent ##ial and stationary plan ##kt ##onic cells via [unused1] production [unused2] of per ##sist ##ers . [SEP]',
  'existing_cui': 'C1548180',
  'existing_cui_name': 'Production Processing ID',
  'existing_cui_desc': '[CLS] Production Process ##ing ID [unused3] ( Intel ##lect ##ual Product : Production ) [SEP]',
  'our_bi_cui': 'C0220781',
  'our_bi_cui_name': 'Anabolism',
  'our_bi_cui_desc': '[CLS] Ana ##bol ##ism [unused3] ( B ##iol ##og ##ic Fun ##ction , function , o ##bs ##er ##vable entity : bio ##sy ##nt ##hetic process ; Ana ##bol ##ism , function ; synthesis ; bio ##sy ##nt ##hesis ; formation ; B ##ios ##ynth ##etic ; B ##ios ##ynth ##esis ; B ##io ##genesis ; B ##ios ##ynth ##etic Process ; an ##ab ##olis ##m ) [SEP]',
  'our_cross_cui': 'C0007613',
  'our_cross_cui_name': 'Cell physiology',
  'our_cross_cui_desc': '[CLS] Cell physiology [unused3] ( B ##iol ##og ##ic Fun ##ction : cell physiology ; Cell ##ular Process ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##a ; Cell ##ular Fun ##ction ; cells physiology ; Cell ##ular Ph ##ys ##iology ; Sub ##cellular Process ; cells function ; cell function ; Cell Ph ##ys ##iol ##ogical Ph ##eno ##men ##on ; Cell Ph ##ys ##iology ; Cell Process ; Cell Fun ##ction ; CE ##LL P ##H ##Y ##SI ##OL ; P ##H ##Y ##SI ##OL CE ##LL ; cell functions ; Ph ##eno ##men ##a , Cell Ph ##ys ##iol ##ogical ; Ph ##ys ##iology , Cell ; Ph ##eno ##men ##on , Cell Ph ##ys ##iol ##ogical ) [SEP]'},
 {'mention_id': '27669217.30',
  'test_set_idx': 13681,
  'type': 'T033',
  'name': 'MIC',
  'context': '[CLS] ##hr ##omy ##cin , and 512 mg / L for c ##lar ##ith ##rom ##y ##cin ) . E ##ff ##lux - encoding me ##f ##A and me ##f ##E genes were not detect ##able in these is ##olate ##s . The presence of 23 ##S r ##RNA mutations in specific is ##olate ##s did not significantly change mac ##rol ##ide [unused1] MI ##C [unused2] s . No n ##uc ##leo ##tide substitution ##s were found in genes encoding rib ##oso ##mal proteins L ##4 or L ##22 . The er ##m ##B and er ##m ##T genes were found in the genome ##s of all is ##olate ##s . These two genes were acquired independently in one highly v ##ir ##ulent is ##olate AL ##10 [SEP]',
  'existing_cui': 'C2732473',
  'existing_cui_name': 'Ductal Carcinoma In Situ with Microinvasion',
  'existing_cui_desc': '[CLS] Du ##ct ##al Car ##cin ##oma In Sit ##u with Micro ##in ##vas ##ion [unused3] ( B ##iol ##og ##ic Fun ##ction , m ##or ##ph ##olo ##gic abnormal ##ity : Micro ##in ##vasive B ##rea ##st Car ##cin ##oma ; MI ##C ; Du ##ct ##al car ##cin ##oma in sit ##u with micro ##in ##vas ##ion ) [SEP]',
  'our_bi_cui': 'C0427978',
  'our_bi_cui_name': 'Minimum Inhibitory Concentration measurement',
  'our_bi_cui_desc': '[CLS] Mini ##mum In ##hibit ##ory Con ##cent ##ration measurement [unused3] ( Health Care Act ##ivity , MI ##C , procedure , qualifier value : Mini ##mum inhibitor ##y concentration su ##s ##ce ##pt ##ibility test ; Con ##cent ##rations , Mini ##mum In ##hibit ##ory ; Anti ##biotic sensitivity , minimum inhibitor ##y concentration ; MI ##C su ##s ##ce ##pt ##ibility test ; Mini ##mum In ##hibit ##ory Con ##cent ##rations ; MI ##C - Mini ##mum inhibitor ##y concentration ; Con ##cent ##ration , Mini ##mum In ##hibit ##ory ; Mini ##mum inhibitor ##y concentration ; Mini ##mum inhibitor ##y concentration su ##s ##ce ##pt ##ibility test technique ; In ##hibit ##ory Con ##cent ##rations , Mini ##mum ; Mini ##mum In ##hibit ##ory [SEP]',
  'our_cross_cui': 'C1304747',
  'our_cross_cui_name': 'Minimum inhibitory concentration result',
  'our_cross_cui_desc': '[CLS] Mini ##mum inhibitor ##y concentration result [unused3] ( Finding , finding : minimum inhibitor ##y concentration ; Mini ##mum inhibitor ##y concentration finding ) [SEP]'},
 {'mention_id': '27444636.25',
  'test_set_idx': 5976,
  'type': 'T170',
  'name': 'Adverse Events Profile',
  'context': '[CLS] at least 4 weeks were switched overnight to es ##lica ##rb ##az ##ep ##ine ace ##tate . Pat ##ients were switched because they experienced persistent seizure ##s with o ##x ##car ##ba ##ze ##pine but were unable to tolerate increased o ##x ##car ##ba ##ze ##pine dos ##ing due to adverse events . To ##ler ##ability was assessed using the [unused1] Ad ##verse Events Profile [unused2] , quality of life was assessed using the Quality of Life in E ##pile ##psy In ##vent ##ory 10 , and alert ##ness was assessed as reaction time using a sub ##test of the Test Battery for At ##ten ##tion Performance version 2 . 3 . Assessment ##s were performed immediately prior to and 5 days after switching from o [SEP]',
  'existing_cui': 'C0877248',
  'existing_cui_name': 'Adverse event',
  'existing_cui_desc': '[CLS] Ad ##verse event [unused3] ( B ##iol ##og ##ic Fun ##ction : AD ##VE ##RS ##E E ##VE ##NT ; adverse events ; Ad ##verse Event ; adverse event ; Ad ##verse ##E ##vent ; Ad ##verse Experience ) [SEP]',
  'our_bi_cui': 'C1521800',
  'our_bi_cui_name': 'Adverse Drug Experience Report',
  'our_bi_cui_desc': '[CLS] Ad ##verse Drug Experience Report [unused3] ( Intel ##lect ##ual Product : Ad ##verse Drug Report ; Ad ##verse Experience Report ; Ad ##verse Event Report ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28254329.46',
  'test_set_idx': 30171,
  'type': 'T033',
  'name': 'hydroxylation degree',
  'context': '[CLS] ##or ##ption , blood wet ##ta ##bility , bacterial and cell ad ##hesion were evaluated in order to investigate the correlation ##s between the surface p ##hy ##sio ##chemical properties and biological responses . From a method ##ological stand ##point , X ##PS and electro ##kin ##etic measurements emerged as the more suitable techniques respectively for the evaluation of [unused1] h ##ydro ##xy ##lation degree [unused2] and surface charge / is ##oe ##lect ##ric point . Moreover , determination of wet ##ta ##bility by blood appeared a specific and crucial test , the results of which are not easily predictable by using other type of tests . H ##ydro ##xy ##lation degree resulted correlated to the wet ##ta ##bility by water , but not directly to [SEP]',
  'existing_cui': 'C1159294',
  'existing_cui_name': 'protein hydroxylation',
  'existing_cui_desc': '[CLS] protein h ##ydro ##xy ##lation [unused3] ( B ##iol ##og ##ic Fun ##ction : protein amino acid h ##ydro ##xy ##lation ) [SEP]',
  'our_bi_cui': 'C0700307',
  'our_bi_cui_name': 'hydroxyl group',
  'our_bi_cui_desc': '[CLS] h ##ydro ##xy ##l group [unused3] ( Chemical : H ##ydro ##xy ##l group ; H ##ydro ##xy ##l Group ; h ##ydro ##xy ##l group ( OH ) - that consists of an oxygen atom joined to a hydrogen atom ; H ##ydro ##xy ; h ##ydro ##xy ##l group ( OH ) [ functional group ] - that consists of an oxygen atom joined to a hydrogen atom ; H ##ydro ##xy ##l ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28400056.14',
  'test_set_idx': 34669,
  'type': 'T103',
  'name': 'EN 12791',
  'context': '[CLS] study was to determine if me ##ce ##tron ##ium et ##ils ##ulf ##ate contributes to overall efficacy in surgical hand di ##sin ##fect ##ion . Three blinded hand rub ##s ( 45 % is ##o - prop ##ano ##l , 30 % n - prop ##ano ##l ) were applied for 1 . 5 min and compared with the [unused1] E ##N 127 ##9 ##1 [unused2] reference procedure ( crossover design ) . One commercial hand rub contained 0 . 2 % me ##ce ##tron ##ium et ##ils ##ulf ##ate , and the two other hand rub ##s were identical apart from 0 . 2 % me ##ce ##tron ##ium et ##ils ##ulf ##ate . None of the formulation ##s had a log ##10 reduction after [SEP]',
  'existing_cui': 'C3899313',
  'existing_cui_name': 'EN1 wt Allele',
  'existing_cui_desc': '[CLS] E ##N ##1 w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : En - 1 ; En ##gra ##iled Home ##ob ##ox 1 w ##t All ##ele ) [SEP]',
  'our_bi_cui': 'C0013983',
  'our_bi_cui_name': 'Emollients',
  'our_bi_cui_desc': '[CLS] Em ##oll ##ients [unused3] ( Chemical , product , substance : Em ##oll ##ient - Mi ##s ##c ; [ DE ##35 ##0 ] E ##MO ##LL ##IE ##NT ##S ; em ##oll ##ients ; E ##MO ##LL ##IE ##NT ##S ; em ##oll ##ient ; Em ##oll ##ient ; Em ##oll ##ient product ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28392427.22',
  'test_set_idx': 34338,
  'type': 'T058',
  'name': 'CBA',
  'context': '[CLS] s ##CD ##25 have repeatedly been reported altered in patients with mood disorders . The aim of this study was to investigate levels of these factors in serum of adolescent bi ##pol ##ar offspring , who have a heightened risk for mood disorder development and to also analyze the data combined with previously published data . Growth factors were assessed by [unused1] CB ##A [unused2] / E ##L ##IS ##A in adolescent bi ##pol ##ar offspring ( n = 96 , mean age = 16 years ) and in age - and gender - matched healthy controls ( n = 50 ) . E ##pid ##er ##mal Growth Factor belonged to a mutually co ##rrel ##ating cluster of mainly ne ##uro ##tro ##phic compounds including S [SEP]',
  'existing_cui': 'C0025922',
  'existing_cui_name': 'Mice , Inbred CBA',
  'existing_cui_desc': '[CLS] Mi ##ce , In ##bred CB ##A [unused3] ( E ##uka ##ryo ##te : CB ##A ; CB ##A Mi ##ce , In ##bred ; Mi ##ce , CB ##A ; CB ##A Mouse , In ##bred ; CB ##A Mouse ; Mouse , In ##bred CB ##A ; Mouse , CB ##A ; CB ##A Mi ##ce ; In ##bred CB ##A Mouse ; In ##bred CB ##A Mi ##ce ) [SEP]',
  'our_bi_cui': 'C0201563',
  'our_bi_cui_name': 'Concanavalin A assay',
  'our_bi_cui_desc': '[CLS] Con ##can ##ava ##lin A ass ##ay [unused3] ( Health Care Act ##ivity , procedure : Con A ass ##ay ) [SEP]',
  'our_cross_cui': 'C0005507',
  'our_cross_cui_name': 'Biological Assay',
  'our_cross_cui_desc': '[CLS] Biological As ##say [unused3] ( Health Care Act ##ivity , procedure : B ##io ##ass ##ays ; B ##iol ##og ##ic As ##say ##s ; B ##IO ##L AS ##SA ##Y ; B ##IO ##L AS ##SA ##Y ##S ; As ##say ; As ##say ##s , Biological ; bio ##ass ##ay ; As ##say , Biological ; bio ##ass ##ays ; B ##io ##ass ##ay ; Biological As ##say ##s ; AS ##SA ##Y B ##IO ##L ; B ##iol ##og ##ic As ##say ; As ##say , B ##iol ##og ##ic ; AS ##SA ##Y ##S B ##IO ##L ; As ##say ##s , B ##iol ##og ##ic ; Biological ass ##ay ) [SEP]'},
 {'mention_id': '28436493.26',
  'test_set_idx': 35792,
  'type': 'T103',
  'name': 'EsxA',
  'context': '[CLS] , it suggested a defect in the E ##S ##X - 1 type VII secret ##ion system . Gene ##tic complement ##ation with a p ##las ##mi ##d carrying the My ##co ##ba ##cter ##ium tuberculosis H ##37 ##R ##v sequence of e ##cc ##C ##a ##1 - e ##cc ##C ##b ##1 - p ##e ##35 re - established [unused1] E ##s ##x ##A [unused2] secret ##ion , host specific E ##s ##x ##A T - cell responses , and increased strain v ##ir ##ule ##nce . The E ##S ##X - 1 secret ##ion defect prevents several v ##ir ##ule ##nce factors from being functional during infection and therefore at ##ten ##uate ##s My ##co ##ba ##cter ##ium tuberculosis . It pre ##c ##lude [SEP]',
  'existing_cui': 'C1706322',
  'existing_cui_name': 'ELF3 wt Allele',
  'existing_cui_desc': '[CLS] E ##LF ##3 w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : E ##SE - 1 ; ER ##T ; J ##EN ; E ##S ##X ; E ##7 ##4 - Like Factor 3 ( ET ##S Domain Trans ##cription Factor , E ##pit ##hel ##ial - S ##pecific ) w ##t All ##ele ; EP ##R - 1 ) [SEP]',
  'our_bi_cui': 'C1822656',
  'our_bi_cui_name': 'SCXA gene',
  'our_bi_cui_desc': '[CLS] SC ##X ##A gene [unused3] ( Ana ##tom ##ical Structure : SC ##LE ##RA ##X ##IS , M ##O ##US ##E , H ##OM ##OL ##O ##G OF , A ; SC ##LE ##RA ##X ##IS ) [SEP]',
  'our_cross_cui': 'C1744314',
  'our_cross_cui_name': 'ESAT-6 protein , Mycobacterium tuberculosis',
  'our_cross_cui_desc': '[CLS] E ##SA ##T - 6 protein , My ##co ##ba ##cter ##ium tuberculosis [unused3] ( Chemical : E ##SA ##T - 6 System 1 protein , My ##co ##ba ##cter ##ium tuberculosis ; 6 - k ##D ##a early secret ##ory anti ##genic target , My ##co ##ba ##cter ##ium tuberculosis ; E ##s ##x - 1 protein , My ##co ##ba ##cter ##ium tuberculosis ; E ##s ##x ##A protein , My ##co ##ba ##cter ##ium tuberculosis ) [SEP]'},
 {'mention_id': '27857693.31',
  'test_set_idx': 19410,
  'type': 'T058',
  'name': 'IHRT',
  'context': '[CLS] . 2 % greater change , 90 % Con ##fi ##dence Inter ##val 1 . 6 : 14 . 9 % , effect size 0 . 49 ) . Only I ##H ##RT increased counter ##mo ##ve ##ment jump peak power at Post ( 4 . 9 % , effect size 0 . 35 ) , however the difference between [unused1] I ##H ##RT [unused2] and place ##bo was unclear ( 2 . 7 , 90 % Con ##fi ##dence Inter ##val - 2 . 0 : 7 . 6 % , effect size 0 . 20 ) with no clear differences in speed or body composition throughout . Con ##c ##lusion : Heavy resistance training in h ##y ##pox ##ia is more effective than place [SEP]',
  'existing_cui': 'C0020669',
  'existing_cui_name': 'Hypothalamus , Middle',
  'existing_cui_desc': '[CLS] H ##y ##pot ##hala ##mus , Middle [unused3] ( Ana ##tom ##ical Structure : Area h ##y ##pot ##hala ##mic ##a inter ##media ; Media ##l H ##y ##pot ##hala ##mus ; Intermediate h ##y ##pot ##hala ##mic area ; Intermediate H ##y ##pot ##hala ##mic Region ; Region , Intermediate H ##y ##pot ##hala ##mic ; Reg ##io h ##y ##pot ##hala ##mic ##a inter ##media ; Middle H ##y ##pot ##hala ##mus ; H ##y ##pot ##hala ##mic Region ##s , Intermediate ; H ##y ##pot ##hala ##mic Region , Intermediate ; Intermediate H ##y ##pot ##hala ##mic Region ##s ; Intermediate h ##y ##pot ##hala ##mic region ; intermediate h ##y ##pot ##hala ##mic region ; H ##y ##pot ##hala ##mus , Media [SEP]',
  'our_bi_cui': 'C0282402',
  'our_bi_cui_name': 'Hormone replacement therapy',
  'our_bi_cui_desc': '[CLS] Ho ##rmon ##e replacement therapy [unused3] ( Health Care Act ##ivity , H ##RT , hormone replacement therapy , procedure : hormone replacement therapy ; Therapy , Ho ##rmon ##e Rep ##lace ##ment ; hormone replacement the ##rap ##ies ; H ##OR ##MO ##NE R ##EP ##LA ##CE THE ##R ; hormone replacement ##s ; H ##RT - Ho ##rmon ##e replacement therapy ; ER ##T ; Ho ##rmon ##e Rep ##lace ##ment Therapy ; Rep ##lace ##ment The ##rap ##ies , Ho ##rmon ##e ; hormones replacement ; men ##op ##aus ##al hormone therapy ; Rep ##lace ##ment Therapy , Ho ##rmon ##e ; R ##EP ##LA ##CE THE ##R H ##OR ##MO ##NE ; Ho ##rmon ##e Rep ##lace ##ment R ##x ; THE [SEP]',
  'our_cross_cui': 'C0872279',
  'our_cross_cui_name': 'Resistance Training',
  'our_cross_cui_desc': '[CLS] Resistance Training [unused3] ( Health Care Act ##ivity : Training , Resistance ; St ##ren ##gth Training ; strength train ; strength training ; Training , St ##ren ##gth ) [SEP]'},
 {'mention_id': '27587730.26',
  'test_set_idx': 10765,
  'type': 'T103',
  'name': 'gga-miR-21',
  'context': "[CLS] the g ##ga - mi ##R - 21 mimic in ##hibit ##ed pre - ad ##ip ##oc ##yte proliferation . In contrast , the g ##ga - mi ##R - 21 inhibitor enhanced pre - ad ##ip ##oc ##yte proliferation . The subsequent investigation identified K ##rup ##pel - like factor 5 m ##RNA as a target of [unused1] g ##ga - mi ##R - 21 [unused2] . The g ##ga - mi ##R - 21 mimic in ##hibit ##ed K ##rup ##pel - like factor 5 3 ( ' ) U ##TR reporter activity and decreased end ##ogen ##ous K ##rup ##pel - like factor 5 expression in primary pre - ad ##ip ##ocytes . K ##rup ##pel - like factor 5 knock ##down using [SEP]",
  'existing_cui': 'C1537719',
  'existing_cui_name': 'MIR21 gene',
  'existing_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_bi_cui': 'C1537719',
  'our_bi_cui_name': 'MIR21 gene',
  'our_bi_cui_desc': '[CLS] MI ##R ##21 gene [unused3] ( Ana ##tom ##ical Structure : micro ##RNA 21 ; MI ##R ##N ##21 ; Micro ##RNA 21 Gene ; h ##sa - mi ##r - 21 ; MI ##R ##21 Gene ; MI ##R - 21 ; mi ##RNA ##21 ; MI ##CR ##O RNA 21 ) [SEP]',
  'our_cross_cui': 'C1101610',
  'our_cross_cui_name': 'MicroRNAs',
  'our_cross_cui_desc': '[CLS] Micro ##RNA ##s [unused3] ( Chemical : micro ##rna ; Micro ##RNA ##s [ Chemical / In ##g ##red ##ient ] ; micro ##RNA ; mi ##RNA ##s ; mi ##RNA ; mi ##rna ; micro - RNA ; RNA , Micro ; Micro RNA ; MI ##R ; Micro ##RNA ) [SEP]'},
 {'mention_id': '28127853.14',
  'test_set_idx': 26308,
  'type': 'T170',
  'name': 'Spiritual Health Scale',
  'context': '[CLS] attended the 12 - week program and completed the pre - test and post - test . There were 16 middle - aged and nine older adults recruited from a district of Taipei , and nine young adults recruited from the principal investigator \' s university . The " At ##ti ##tudes toward A ##ging Scale " and the " [unused1] Spiritual Health Scale [unused2] " were two assessment instruments used in the study . The results showed that there were significant improvements in the At ##ti ##tudes toward A ##ging Scale for the young adult group ( aged 18 - 29 years ) and in the Spiritual Health Scale for the older adult group ( aged 65 - 80 years ) . The evaluation showed [SEP]',
  'existing_cui': 'C0686997',
  'existing_cui_name': 'Spiritual health of patient',
  'existing_cui_desc': '[CLS] Spiritual health of patient [unused3] ( Finding , o ##bs ##er ##vable entity : Spiritual health ; Spiritual condition of patient ; spiritual health ; health spiritual ; Spiritual Health ) [SEP]',
  'our_bi_cui': 'C1998888',
  'our_bi_cui_name': 'Spiritual Well Being',
  'our_bi_cui_desc': '[CLS] Spiritual Well Being [unused3] ( Intel ##lect ##ual Product : Spiritual Well - being ; spiritual well - being ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28477843.3',
  'test_set_idx': 37379,
  'type': 'T017',
  'name': 'low back',
  'context': '[CLS] Development and valid ##ation of an easy - to - use risk assessment tool for cumulative [unused1] low back [unused2] loading : The Li ##fting Fat ##ig ##ue F ##ail ##ure Too ##l ( Li ##fting Fat ##ig ##ue F ##ail ##ure Too ##l ) Recent evidence suggests that m ##us ##cu ##los ##kel ##etal disorders may be the result of a fatigue failure process in affected tissues . This paper describes a new low back exposure assessment tool ( the Li ##fting Fat ##ig ##ue F ##ail ##ure Too ##l [ Li ##FF ##T ] ) , which estimates a " daily dose " of cumulative loading on the low back using fatigue failure principles . Only three variables are necessary to derive the cumulative [SEP]',
  'existing_cui': 'C0024031',
  'existing_cui_name': 'Low Back Pain',
  'existing_cui_desc': '[CLS] Low Back Pain [unused3] ( Finding , diagnosis , disorder , finding , s ##ym ##pt ##om : low ; back ##ache ; Back Pain ##s , Lower ; A ##ches , Low Back ; Pain ; back ; l ##umba ##r ; do ##rs ##op ##athy low back pain ; Low Back A ##che ; back ##ache ; low ; A ##che , Low Back ; lower back pain ; Lu ##mba ##r pain ; Back ##ache ##s , Low ; Back Pain ##s , Low ; Low Back Pain [ Disease / Finding ] ; l ##umba ##go ; Back A ##che , Low ; back ; pain , low ; pain ; low back ; l ##umba ##r pain ; Lu ##mba ##go [SEP]',
  'our_bi_cui': 'C2939142',
  'our_bi_cui_name': 'Lower back',
  'our_bi_cui_desc': '[CLS] Lower back [unused3] ( Spa ##tial Concept , surface region , surface region , body structure : Lower back ( surface region ) ) [SEP]',
  'our_cross_cui': 'C0230102',
  'our_cross_cui_name': 'Lower back structure',
  'our_cross_cui_desc': '[CLS] Lower back structure [unused3] ( Ana ##tom ##ical Structure , body structure : Lower back ; lower back ; Di ##stal back ; lower back structure ) [SEP]'},
 {'mention_id': '27842161.5',
  'test_set_idx': 19098,
  'type': 'T103',
  'name': 'Netarsudil',
  'context': '[CLS] Net ##ars ##udi ##l Inc ##rease ##s Out ##flow Facility in Human Eyes Through Multiple Me ##chan ##isms [unused1] Net ##ars ##udi ##l [unused2] is a R ##ho kinase / nor ##ep ##ine ##ph ##rine transport ##er inhibitor currently in phase 3 clinical development for g ##lau ##com ##a treatment . We investigated the effects of its active meta ##bol ##ite , net ##ars ##udi ##l - M1 , on out ##flow facility ( C ) , out ##flow h ##ydro ##dynamic ##s , and morphology of the conventional out ##flow pathway in en ##uc ##lea ##ted human eyes . Pa ##ired human eyes ( n = 5 ) were per ##fused with either 0 . 3 μ ##M net ##ars ##udi ##l - M1 or [SEP]',
  'existing_cui': 'C0013443',
  'existing_cui_name': 'Ear structure',
  'existing_cui_desc': '[CLS] E ##ar structure [unused3] ( Ana ##tom ##ical Structure , Anatomy , body structure : V ##est ##ib ##ulo ##co ##ch ##lea ##r System ; System , V ##est ##ib ##ulo ##co ##ch ##lea ##r ; ear structures ; Organ ##um vest ##ib ##ulo ##co ##ch ##lea ##re ; ear structure ; E ##ar - related structure ; au ##ris ; A ##ppa ##rat ##us , V ##est ##ib ##ulo ##co ##ch ##lea ##r ; E ##ar and related structures ; Au ##ris ; V ##est ##ib ##ulo ##co ##ch ##lea ##r organ ; V ##est ##ib ##ulo ##co ##ch ##lea ##r A ##ppa ##rat ##us ) [SEP]',
  'our_bi_cui': 'C0027783',
  'our_bi_cui_name': 'Netropsin',
  'our_bi_cui_desc': '[CLS] Net ##rops ##in [unused3] ( Chemical : Sin ##ano ##my ##cin ; Net ##rops ##in [ Chemical / In ##g ##red ##ient ] ; net ##rops ##in ; Congo ##cid ##ine ; 1 ##H - P ##yr ##rol ##e - 2 - car ##box ##ami ##de , 4 - ( ( ( ( amino ##im ##ino ##met ##hyl ) amino ) ace ##ty ##l ) amino ) - N - ( 5 - ( ( ( 3 - amino - 3 - im ##ino ##p ##rop ##yl ) amino ) carbon ##yl ) - 1 - met ##hyl - 1 ##H - p ##yr ##rol - 3 - y ##l ) - 1 - met ##hyl - ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28459124.18',
  'test_set_idx': 36727,
  'type': 'T038',
  'name': 'Zebra Chip',
  'context': '[CLS] Mexico , and Central America have experienced severe yield loss from Z ##eb ##ra Chip disease and have responded by increasing insect ##icide use to suppress populations of the insect vector , the potato ps ##yl ##lid , Ba ##cter ##ice ##ra cock ##ere ##lli ( He ##mi ##ptera : Trio ##zi ##dae ) . Despite the severe nature of [unused1] Z ##eb ##ra Chip [unused2] outbreak ##s , the causes of emergence remain unknown . We tested the h ##y ##pot ##heses that 1 ) B . cock ##ere ##lli o ##cc ##up ##ancy has increased over the last century in California and 2 ) such increases are related to climate change , specifically warmer winters . We compiled a data ##set of 87 , [SEP]',
  'existing_cui': 'C3890232',
  'existing_cui_name': 'Trans-Activator Protein BZLF1',
  'existing_cui_desc': '[CLS] Trans - Act ##iva ##tor Pro ##tein B ##Z ##LF ##1 [unused3] ( Chemical : Trans - Act ##iva ##tor Pro ##tein B ##Z ##LF - 1 ; E ##ps ##tein - Barr V ##ir ##us Trans - Act ##iva ##tor Pro ##tein B ##Z ##LF ##1 ; E ##ps ##tein - Barr V ##ir ##us Trans - Act ##iva ##tor Pro ##tein B ##Z ##LF - 1 ; E ##ps ##tein - Barr V ##ir ##us B ##Z ##LF - 1 ; E ##B ##V B ##Z ##LF - 1 ; Z ##eb ##ra ; E ##B ##V B ##Z ##LF ##1 ) [SEP]',
  'our_bi_cui': 'C0325177',
  'our_bi_cui_name': 'Equus zebra',
  'our_bi_cui_desc': '[CLS] E ##q ##u ##us z ##eb ##ra [unused3] ( E ##uka ##ryo ##te , organism : Mountain z ##eb ##ra ; E ##q ##u ##us z ##eb ##ra z ##eb ##ra ) [SEP]',
  'our_cross_cui': 'C0012634',
  'our_cross_cui_name': 'Disease',
  'our_cross_cui_desc': '[CLS] Disease [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , or disorder : disease / disorder ; D ##IS ; Clinical disease or syndrome present ; Disease [ Disease / Finding ] ; Disease ##s ; Di ##sor ##der ; Clinical disease or syndrome ; Disease AND / OR syndrome present ; di ##ag ##nose ##s , syndrome ##s , and conditions ; Clinical disease AND / OR syndrome present ; Di ##sor ##ders ; Disease or Di ##sor ##der ; disease ; Disease ##s and Di ##sor ##ders ; condition ; Disease or syndrome present ; diseases ; disorders ; Clinical disease AND / OR syndrome ; disorder ) [SEP]'},
 {'mention_id': '27598388.17',
  'test_set_idx': 10966,
  'type': 'T058',
  'name': 'elemental mapping',
  'context': '[CLS] V ##is spectrum which showed the surface p ##las ##mon resonance band at 42 ##2 nm . Trans ##mission electron micro ##scope analysis revealed that the particles were spherical , he ##xa ##gonal , and irregular in shape and had size ranging from 30 to 80 nm . The energy di ##sper ##sive X - ray s ##pect ##ros ##copy and [unused1] elemental mapping [unused2] have displayed the purity and maximum distribution of silver in the A ##g ##NP ##s . The crystal ##line nature of A ##g ##NP ##s had been identified using X - ray di ##ff ##raction and selected area di ##ff ##raction pattern . The particle size analysis revealed that the Z - average diameter of the A ##g ##NP ##s was [SEP]',
  'existing_cui': 'C0008630',
  'existing_cui_name': 'Chromosome Mapping',
  'existing_cui_desc': '[CLS] Ch ##rom ##oso ##me Map ##ping [unused3] ( Health Care Act ##ivity : Gene ##tics , Gene Map ##ping ; Map ##ping , Gene ; gene mapped ; Gene Map ##ping ##s ; Link ##age Map ##ping ; Gene Map ##ping ; Gene mapping ; chromosome mapping ; genetic mapping ; gene map ; link ##age map ; Gene ##tic Map ##ping ; Gene Map ##ping , Total Human and Non - Human ; Ch ##rom ##oso ##me Map ##ping ##s ; Map ##ping , Ch ##rom ##oso ##me ; Gene Local ##ization ; link ##age mapping ; Map ##ping , Link ##age ; Map ##ping ##s , Gene ; gene mapping ; Link ##age Map ##ping ##s ; Map ##ping ##s , Link ##age ; [SEP]',
  'our_bi_cui': 'C3898095',
  'our_bi_cui_name': 'Optical Mapping',
  'our_bi_cui_desc': '[CLS] Op ##tical Map ##ping [unused3] ( Research Act ##ivity : Op ##tical Map ; O ##P ##TI ##CA ##L MA ##PP ##ING ; Op ##tical mapping ) [SEP]',
  'our_cross_cui': 'C0022885',
  'our_cross_cui_name': 'Laboratory Procedures',
  'our_cross_cui_desc': '[CLS] Laboratory Pro ##ced ##ures [unused3] ( Health Care Act ##ivity , C ##MS Te ##mpo ##rary Code ##s , context - dependent category , procedure , situation : Laboratory test panel ; Tests ; Laboratory procedure ; lab test ; laboratory tests ; Pro ##ced ##ure , lab . - general ; test ; laboratory ; Laboratory procedure - general - ; Lab . test - general ; Lab test ; Laboratory procedures - general ; laboratory procedures ; General laboratory procedure - R ##ET ##IR ##ED - ; lab testing ; General laboratory procedure ; Laboratory Test ; lab tests ; laboratory test ; Lab Test ; Test ; Laboratory Tests ; Laboratory Pro ##ced ##ure ; Investigation - lab . , general ; Test [SEP]'},
 {'mention_id': '27611704.12',
  'test_set_idx': 11686,
  'type': 'T017',
  'name': 'D',
  'context': "[CLS] B ##read wheat ( Tri ##tic ##um a ##est ##iv ##um , 2 ##n = 6 ##x = 42 , AA ##BB ##D ##D ) has a complex all ##oh ##ex ##ap ##lo ##id genome , which makes it difficult to differentiate between the ho ##mo ##eo ##log ##ous sequences and assign them to the chromosome A , B , or [unused1] D [unused2] sub ##gen ##ome ##s . The chromosome - based draft genome sequence of the ' Chinese Spring ' common wheat cult ##iva ##r enables the large - scale development of polymer ##ase chain reaction ( polymer ##ase chain reaction ) - based markers specific for ho ##mo ##eo ##log ##s . Based on high - confidence ' Chinese Spring ' genes with [SEP]",
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0008645',
  'our_bi_cui_name': 'Chromosomes , Human , 13-15',
  'our_bi_cui_desc': '[CLS] Ch ##rom ##oso ##mes , Human , 13 - 15 [unused3] ( Ana ##tom ##ical Structure , body structure , cell structure : Ch ##rom ##oso ##mes , group D ; Ch ##rom ##oso ##mes , Group D ; Group D Ch ##rom ##oso ##me ; Ch ##rom ##oso ##me pairs 13 - 15 ; Ch ##rom ##oso ##mes D ; Ch ##rom ##oso ##me , Group D ; Group D Ch ##rom ##oso ##mes ) [SEP]',
  'our_cross_cui': 'C0008633',
  'our_cross_cui_name': 'Chromosomes',
  'our_cross_cui_desc': '[CLS] Ch ##rom ##oso ##mes [unused3] ( Ana ##tom ##ical Structure , body structure , cell structure : chromosome ; Ch ##rom ##oso ##mal ; chromosome ##s ; Ch ##rom ##oso ##me ) [SEP]'},
 {'mention_id': '27892235.1',
  'test_set_idx': 21013,
  'type': 'T058',
  'name': 'NHS',
  'context': '[CLS] S ##hout louder about poor pay It seems increasingly clear to me that the [unused1] NHS [unused2] pay campaign is not making waves either in the NHS or with the public . [SEP]',
  'existing_cui': 'C1417725',
  'existing_cui_name': 'NHS gene',
  'existing_cui_desc': '[CLS] NHS gene [unused3] ( Ana ##tom ##ical Structure : NHS act ##in re ##mo ##del ##ing regulator ; NHS GE ##NE ) [SEP]',
  'our_bi_cui': 'C0018104',
  'our_bi_cui_name': 'Government',
  'our_bi_cui_desc': '[CLS] Government [unused3] ( Organization : governments ; Government ##s ; government ) [SEP]',
  'our_cross_cui': 'C0027462',
  'our_cross_cui_name': 'Health Services , National',
  'our_cross_cui_desc': '[CLS] Health Services , National [unused3] ( Health Care Act ##ivity : National health services ; H ##EA ##LT ##H SE ##R ##V N ##AT ##ION ##AL ; SE ##R ##V N ##AT ##ION ##AL H ##EA ##LT ##H ; National Health Service ; Service , National Health ; national health services ; national health service ; Health Service , National ; N ##AT ##ION ##AL H ##EA ##LT ##H SE ##R ##V ; Services , National Health ; National Health Services ) [SEP]'},
 {'mention_id': '27802570.2',
  'test_set_idx': 17754,
  'type': 'T058',
  'name': 'Treated by',
  'context': '[CLS] Overall Quality Properties of Ki ##wi ##f ##ruit [unused1] T ##reate ##d by [unused2] C ##inn ##ama ##lde ##hy ##de and C ##it ##ral : Micro ##bial , Anti ##ox ##ida ##nt Cap ##acity during Cold St ##orage This work was undertaken to evaluate the micro ##bio ##logical characteristics and anti ##ox ##ida ##nt and physiological activities in k ##i ##wi ##f ##ruit ##s ( Act ##ini ##da del ##ici ##osa var . Qin ##mei ) with / without c ##inn ##ama ##lde ##hy ##de ( C ##1 ) and c ##it ##ral ( C ##2 ) f ##umi ##gation treatments ( 5 μ ##L / L ) during 0 , 3 , 6 , 9 , and 12 d of storage at 4 °C . [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0087111',
  'our_bi_cui_name': 'Therapeutic procedure',
  'our_bi_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]',
  'our_cross_cui': 'C0332293',
  'our_cross_cui_name': 'Treated with',
  'our_cross_cui_desc': '[CLS] T ##reate ##d with [unused3] ( Health Care Act ##ivity , attribute : T ##reate ##d by ) [SEP]'},
 {'mention_id': '27832198.46',
  'test_set_idx': 18668,
  'type': 'T017',
  'name': 'Cam734',
  'context': '[CLS] ##al ##ci ##par ##um ch ##lor ##o ##quin ##e resistance transport ##er - specific zinc - finger n ##uc ##lea ##ses to genetically di ##sse ##ct this all ##ele in the path ##ogenic setting of as ##ex ##ual blood - stage infection . Comparative analysis of drug resistance and growth profiles of re ##comb ##ina ##nt parasite ##s that express [unused1] Cam ##7 ##34 [unused2] or variants thereof , D ##d ##2 ( the most common Southeast Asian variant ) , or wild - type P . f ##al ##ci ##par ##um ch ##lor ##o ##quin ##e resistance transport ##er , revealed previously unknown roles for P ##f ##CR ##T mutations in m ##od ##ulating parasite su ##s ##ce ##pt ##ibility to multiple anti ##mal ##aria [SEP]',
  'existing_cui': 'C0006675',
  'existing_cui_name': 'Calcium',
  'existing_cui_desc': '[CLS] Cal ##ci ##um [unused3] ( Chemical , medication , substance : Co ##agu ##lation factor IV - R ##ET ##IR ##ED - ; CO ##AG FA ##CT ##OR IV ; Co ##agu ##lation Factor IV ; B ##L ##O ##OD CO ##AG FA ##CT ##OR IV ; C ##a + + element ; factor i ##v ; calcium ; Blood Co ##agu ##lation Factor IV ; Cal ##ci ##um co ##agu ##lation factor ; Co ##agu ##lation factor IV ; calcium ##s ; Cal ##ci ##um [ Chemical / In ##g ##red ##ient ] ; C ##a - Cal ##ci ##um ; CA ##LC ##I ##UM ; Factor IV ; C ##a element ; Cal ##ci ##um anal ##yte ; Factor IV , Co ##agu ##lation [SEP]',
  'our_bi_cui': 'C1708291',
  'our_bi_cui_name': 'CDC73 gene',
  'our_bi_cui_desc': '[CLS] CD ##C ##7 ##3 gene [unused3] ( Ana ##tom ##ical Structure : H ##Y ##RA ##X , DR ##OS ##OP ##H ##IL ##A , H ##OM ##OL ##O ##G OF ; cell division cycle 73 ; para ##fi ##bro ##min ; F ##I ##HP ; PA ##RA ##FI ##BR ##OM ##IN ; CD ##C ##7 ##3 Gene ; CE ##LL D ##I ##VI ##SI ##ON C ##Y ##CL ##E PR ##OT ##EI ##N 73 , S . CE ##RE ##VI ##SI ##A ##E , H ##OM ##OL ##O ##G OF ; C ##1 ##OR ##F ##28 ; Cell Division Cycle 73 , Pa ##f ##1 / RNA Pol ##yme ##rase II Complex Co ##mpo ##nent , Ho ##mo ##log ( S . c ##ere ##vis ##iae [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27671177.37',
  'test_set_idx': 13814,
  'type': 'T082',
  'name': 'Quetta',
  'context': "[CLS] ##sis , had 4 times more chances to develop car ##ies than those who did not have flu ##oro ##sis . There was no significant association between children ' s car ##ies status and use of paste , brushing habit , mi ##s ##wa ##k , and visit to the den ##tist . The use of past ##ries and juice ##s had a direct relation with the children ' s dental car ##ies status . Dental car ##ies in children of [unused1] Que ##tta [unused2] is not so much frequent as compared to the flu ##ori ##de def ##icient countries . However , the high prevalence of moderate dental flu ##oro ##sis and consumption of past ##ries and juice ##s resulted in dental car ##ies . [SEP]",
  'existing_cui': 'C0262345',
  'existing_cui_name': 'Tenia of thalamus',
  'existing_cui_desc': '[CLS] Ten ##ia of th ##ala ##mus [unused3] ( Ana ##tom ##ical Structure : ten ##ia of th ##ala ##mi ; Ten ##ia of th ##ala ##mi ; Ta ##enia th ##ala ##mi ; T ##hala ##mus ten ##ia ; ten ##ia of the th ##ala ##mus ; Ten ##ia th ##ala ##mi ) [SEP]',
  'our_bi_cui': 'C0337884',
  'our_bi_cui_name': 'Quechua',
  'our_bi_cui_desc': '[CLS] Que ##chu ##a [unused3] ( Population Group , ethnic group , ethnic group ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '27798906.12',
  'test_set_idx': 17532,
  'type': 'T058',
  'name': 'measured',
  'context': '[CLS] ##s were enrolled and 93 i ##rra ##dia ##ted P ##G ##s were analyzed . Thirty and 24 patients were treated with total doses [ relative biological effectiveness ( relative biological effectiveness ) ] of 57 . 6 G ##y and 64 . 0 G ##y , respectively , in 16 fraction ##s . par ##ot ##id g ##land volumes were [unused1] measured [unused2] using com ##puted to ##mo ##graphic images obtained before carbon ion radio ##ther ##ap ##y and every 3 - 6 months thereafter . The median follow - up period was 46 . 4 months ( range = 24 . 0 - 123 . 0 months ) . Un ##iva ##ria ##te analysis showed that par ##ot ##id g ##land volumes receiving more [SEP]',
  'existing_cui': 'C3541902',
  'existing_cui_name': 'Measured Tumor Identification',
  'existing_cui_desc': '[CLS] Me ##as ##ured Tu ##mor I ##dent ##ification [unused3] ( Health Care Act ##ivity : ME ##AS ##UR ##ED ) [SEP]',
  'our_bi_cui': 'C2734710',
  'our_bi_cui_name': 'Measure observations',
  'our_bi_cui_desc': '[CLS] Me ##as ##ure observations [unused3] ( Finding ) [SEP]',
  'our_cross_cui': 'C2700258',
  'our_cross_cui_name': 'Volume Measurement',
  'our_cross_cui_desc': '[CLS] Volume Me ##as ##ure ##ment [unused3] ( Health Care Act ##ivity : V ##OL ##UM ##E ; Volume measurement ; Volume ) [SEP]'},
 {'mention_id': '27555898.19',
  'test_set_idx': 10352,
  'type': 'T103',
  'name': 'PAA1',
  'context': '[CLS] the specimens , 3 mm of a ##c ##ryl ##ic were grin ##ded off and separated parts were re ##att ##ache ##d by re ##si ##lient liner ##s . The specimens were divided into 2 control ( control ##1 , control ##7 ) and 4 test groups of para ##ce ##tic acid and oz ##one di ##sin ##fect ##ion ( [unused1] PA ##A ##1 [unused2] , PA ##A ##7 , oz ##one ##1 and oz ##one ##7 ; n = 10 ) . While control groups were im ##mers ##ed in di ##st ##ille ##d water for 10 min ( control ##1 ) and 7 days ( control ##7 ) , test groups were subjected to para ##ce ##tic acid ( 16 g / L ) [SEP]',
  'existing_cui': 'C1415190',
  'existing_cui_name': 'GPAA1 gene',
  'existing_cui_desc': '[CLS] GP ##AA ##1 gene [unused3] ( Ana ##tom ##ical Structure : GP ##I trans ##ami ##das ##e subunit ; G ##L ##Y ##CO ##S ##Y ##LP ##H ##OS ##P ##HA ##TI ##D ##Y ##L ##IN ##OS ##IT ##OL AN ##CH ##OR AT ##TA ##CH ##ME ##NT PR ##OT ##EI ##N 1 ; g ##ly ##cos ##yl ##ph ##os ##pha ##ti ##dy ##lino ##si ##to ##l anchor attachment 1 ; h ##GA ##A ##1 ) [SEP]',
  'our_bi_cui': 'C1528733',
  'our_bi_cui_name': 'diphenylacetaldehyde',
  'our_bi_cui_desc': '[CLS] dip ##hen ##yla ##ce ##tal ##de ##hy ##de [unused3] ( Chemical : D ##PA ##A c ##p ##d ) [SEP]',
  'our_cross_cui': 'C0030968',
  'our_cross_cui_name': 'Peracetic Acid',
  'our_cross_cui_desc': '[CLS] Per ##ace ##tic A ##cid [unused3] ( Chemical , product , substance : A ##cid , Per ##ox ##ye ##than ##oi ##c ; per ##ace ##tic acid ; Per ##ox ##ya ##ce ##tic A ##cid ; Per ##ace ##tic A ##cid [ Chemical / In ##g ##red ##ient ] ; Ethan ##eper ##ox ##oi ##c acid ; A ##cid , Per ##ace ##tic ; Ace ##ty ##l H ##ydro ##per ##oxide ; A ##cid , Per ##ox ##ya ##ce ##tic ; Per ##ox ##ye ##than ##oi ##c A ##cid ; V ##ap ##ori ##zed Para ##ce ##tic A ##cid St ##eri ##li ##zation ; Per ##ox ##ya ##ce ##tic acid ; Per ##ace ##tic acid ; per ##ox ##ya ##ce ##tic acid ) [SEP]'},
 {'mention_id': '28477843.12',
  'test_set_idx': 37388,
  'type': 'T017',
  'name': 'low back',
  'context': '[CLS] m ##us ##cu ##los ##kel ##etal disorders may be the result of a fatigue failure process in affected tissues . This paper describes a new low back exposure assessment tool ( the Li ##fting Fat ##ig ##ue F ##ail ##ure Too ##l [ Li ##FF ##T ] ) , which estimates a " daily dose " of cumulative loading on the [unused1] low back [unused2] using fatigue failure principles . Only three variables are necessary to derive the cumulative load associated with a lifting task : the weight of the load , the maximum horizontal distance from the spine to the load , and the number of repetition ##s for tasks performed during the work ##day . The new tool was valid ##ated using two existing [SEP]',
  'existing_cui': 'C0024031',
  'existing_cui_name': 'Low Back Pain',
  'existing_cui_desc': '[CLS] Low Back Pain [unused3] ( Finding , diagnosis , disorder , finding , s ##ym ##pt ##om : low ; back ##ache ; Back Pain ##s , Lower ; A ##ches , Low Back ; Pain ; back ; l ##umba ##r ; do ##rs ##op ##athy low back pain ; Low Back A ##che ; back ##ache ; low ; A ##che , Low Back ; lower back pain ; Lu ##mba ##r pain ; Back ##ache ##s , Low ; Back Pain ##s , Low ; Low Back Pain [ Disease / Finding ] ; l ##umba ##go ; Back A ##che , Low ; back ; pain , low ; pain ; low back ; l ##umba ##r pain ; Lu ##mba ##go [SEP]',
  'our_bi_cui': 'C2939142',
  'our_bi_cui_name': 'Lower back',
  'our_bi_cui_desc': '[CLS] Lower back [unused3] ( Spa ##tial Concept , surface region , surface region , body structure : Lower back ( surface region ) ) [SEP]',
  'our_cross_cui': 'C0230102',
  'our_cross_cui_name': 'Lower back structure',
  'our_cross_cui_desc': '[CLS] Lower back structure [unused3] ( Ana ##tom ##ical Structure , body structure : Lower back ; lower back ; Di ##stal back ; lower back structure ) [SEP]'},
 {'mention_id': '27388786.22',
  'test_set_idx': 4445,
  'type': 'T098',
  'name': 'signers',
  'context': "[CLS] explained by hand movement alone , the contribution of motor movement versus ' linguistic ' processes to the strength of hem ##is ##pheric lateral ##ization during sign production remains unclear . Here we directly contrast lateral ##ization strength of covert versus over ##t signing during p ##hon ##ological and semantic flu ##ency tasks . To address the possibility that hearing native [unused1] sign ##ers [unused2] ' elevated lateral ##ization in ##dices were due to performing a task in their less dominant language , here we test deaf native sign ##ers , whose dominant language is British Sign Language . Sign ##ers were more strongly left lateral ##ized for over ##t than covert sign generation . However , the strength of lateral ##ization was not correlated with [SEP]",
  'existing_cui': 'C0035127',
  'existing_cui_name': 'Cumulative Trauma Disorders',
  'existing_cui_desc': '[CLS] C ##um ##ula ##tive T ##ra ##uma Di ##sor ##ders [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , event : Over ##use S ##yn ##drome ; Rep ##eti ##tive St ##ress In ##ju ##ries ; Di ##sor ##ders , C ##um ##ula ##tive T ##ra ##uma ; RS ##I - Rep ##eti ##tive strain injury ; R ##EP ##ET M ##OT ##ION D ##IS ; repetitive strain injury ; St ##rain In ##ju ##ry , Rep ##eti ##tive ; T ##ra ##uma Di ##sor ##ders , C ##um ##ula ##tive ; Rep ##eti ##tive Motion Di ##sor ##ders ; In ##ju ##ries , Rep ##eti ##tive St ##ress ; Motion Di ##sor ##ders , Rep ##eti ##tive ; cumulative trauma disorders [SEP]',
  'our_bi_cui': 'C0336303',
  'our_bi_cui_name': 'Sign-painter',
  'our_bi_cui_desc': '[CLS] Sign - painter [unused3] ( Professional or O ##cc ##upation ##al Group , occupation : Sign ##writer ) [SEP]',
  'our_cross_cui': 'C1257890',
  'our_cross_cui_name': 'Population Group',
  'our_cross_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]'},
 {'mention_id': '27580908.24',
  'test_set_idx': 10668,
  'type': 'T098',
  'name': "learners '",
  'context': "[CLS] to participate . Student co ##hor ##ts were alternatively given a faculty - supervised 1 ##h session playing a simple interactive digital T ##ic - ta ##c - toe q ##uiz module on p ##ediatric gas ##tro ##int ##est ##inal radio ##logy or a 1 ##h did ##actic intro ##ductory lecture on the same topic . Survey questions assessed the [unused1] learn ##ers ' [unused2] perceived ability to recall the material as well as their satisfaction with the educational experience . Results of an end - of - rotation exam were reviewed to evaluate a quantitative measure of learning between groups . Survey responses were analyzed with a ch ##i - squared test . Ex ##am results for both groups were analyzed with a paired Student [SEP]",
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0520955',
  'our_bi_cui_name': 'Slow learner',
  'our_bi_cui_desc': '[CLS] Slow learn ##er [unused3] ( Finding , finding , organism : slow learn ##er ; delayed learning ; Slow Lea ##rner ##s ; slow learn ##ers ; Learning delay ) [SEP]',
  'our_cross_cui': 'C1257890',
  'our_cross_cui_name': 'Population Group',
  'our_cross_cui_desc': '[CLS] Population Group [unused3] ( Population Group , social concept : Group ; groups population ; Group , Population ; group populations ; population ; Sub ##pop ##ulation ; Population Groups ; population group ; Groups , Population ; Population ; population groups ) [SEP]'},
 {'mention_id': '27383326.9',
  'test_set_idx': 4277,
  'type': 'T038',
  'name': 'advanced',
  'context': '[CLS] Ad ##verse events risk associated with an ##gio ##genesis inhibitor ##s addition to therapy in o ##var ##ian cancer : a meta - analysis of random ##ized controlled trials In ##hibition of an ##gio ##genesis has been regarded as an attractive treatment strategy for [unused1] advanced [unused2] or re ##current o ##var ##ian cancer . We conduct this meta - analysis to investigate the risk of adverse events of special interest related to an ##gio ##genesis inhibitor ##s in o ##var ##ian cancer . Database ##s from Pub ##M ##ed , Web of Science and Cochrane library up to December 2015 were searched to identify relevant studies . Eli ##gible studies included prospective random ##ized controlled phase II / III clinical trials evaluating therapy with or [SEP]',
  'existing_cui': 'C0877373',
  'existing_cui_name': 'Advanced cancer',
  'existing_cui_desc': '[CLS] Advanced cancer [unused3] ( B ##iol ##og ##ic Fun ##ction : Advanced Mali ##gnant Neo ##p ##las ##m ; Advanced Cancer ; cancer advanced ; advanced cancer ) [SEP]',
  'our_bi_cui': 'C0877373',
  'our_bi_cui_name': 'Advanced cancer',
  'our_bi_cui_desc': '[CLS] Advanced cancer [unused3] ( B ##iol ##og ##ic Fun ##ction : Advanced Mali ##gnant Neo ##p ##las ##m ; Advanced Cancer ; cancer advanced ; advanced cancer ) [SEP]',
  'our_cross_cui': 'C0029925',
  'our_cross_cui_name': 'Ovarian Carcinoma',
  'our_cross_cui_desc': '[CLS] O ##var ##ian Car ##cin ##oma [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis : o ##vary cancer ; o ##var ##ian car ##cin ##oma ; o ##vary car ##cin ##oma ; car ##cin ##oma o ##var ##ian ; ca o ##var ##ian ; cancer o ##var ##ian ; cancer ##s o ##var ##ian ; cancer of o ##vary ; cancer o ##vary ; o ##var ##ian cancer ; ma ##li ##gnant o ##var ##ies tumor ; o ##var ##ian cancer ##s ; o ##var ##ian car ##cin ##oma ##s ; cancer of the o ##vary ; car ##cin ##oma of o ##vary ; car ##cin ##oma o ##var ##ies ; car ##cin ##oma o ##vary ; cancer o ##var ##ies ; O ##var [SEP]'},
 {'mention_id': '27608421.15',
  'test_set_idx': 11552,
  'type': 'T017',
  'name': 'woody tissue',
  'context': '[CLS] ##cc ##um par ##vu ##m The as ##com ##y ##ce ##te Neo ##fu ##si ##co ##cc ##um par ##vu ##m , one of the ca ##usal agents of Bo ##try ##os ##pha ##eria die ##back , is a destructive wood - in ##fect ##ing fungus and a serious threat to grape production worldwide . The capability of co ##lon ##izing [unused1] wood ##y tissue [unused2] combined with the secret ##ion of p ##hy ##to ##to ##xi ##c compounds is thought to under ##lie its path ##ogenic ##ity and v ##ir ##ule ##nce . Here , we describe the repertoire of v ##ir ##ule ##nce factors and their transcription ##al dynamics as the fungus feeds on different substrates and co ##lon ##izes the wood ##y stem . [SEP]',
  'existing_cui': 'C0040300',
  'existing_cui_name': 'Body tissue',
  'existing_cui_desc': '[CLS] Body tissue [unused3] ( Ana ##tom ##ical Structure , Body , body structure : Port ##ion of tissue ; Normal T ##iss ##ue ; T ##iss ##ue ; Text ##us ; Body T ##iss ##ues ; tissues ; Body tissue structure ; body tissues ; T ##iss ; body tissue ; tissue ; T ##iss ##ues ; T ##IS ##SU ##E ) [SEP]',
  'our_bi_cui': 'C2699642',
  'our_bi_cui_name': 'Stem , Woody - plant part',
  'our_bi_cui_desc': '[CLS] St ##em , Woody - plant part [unused3] ( E ##uka ##ryo ##te : St ##em , Woody ; Li ##gn ##um ; St ##em , wood ##y ) [SEP]',
  'our_cross_cui': 'C1514137',
  'our_cross_cui_name': 'Plant Tissue , Cells',
  'our_cross_cui_desc': '[CLS] Plant T ##iss ##ue , Cell ##s [unused3] ( Ana ##tom ##ical Structure : Plant cells and tissues ) [SEP]'},
 {'mention_id': '28353042.21',
  'test_set_idx': 32949,
  'type': 'T204',
  'name': 'rats',
  'context': '[CLS] - f ##ru ##ct ##ose fed male W ##ista ##r rats . high - f ##ru ##ct ##ose diet ( 66 % of f ##ru ##ct ##ose ) and PA ##A ##E ( 200 mg / kg body weight / day ) were given concurrently to the rats for a period of 60 days . Fr ##uc ##tos ##e - fed [unused1] rats [unused2] showed weight gain , h ##yper ##gly ##ce ##mia , h ##yper ##ins ##ulin ##emia , impaired glucose tolerance , impaired insulin sensitivity , d ##ys ##lip ##ide ##mia , h ##yper ##le ##pt ##ine ##mia , and h ##y ##po ##adi ##po ##nect ##ine ##mia ( P < 0 . 05 ) after 60 days . Co - administration of PA [SEP]',
  'existing_cui': 'C2097385',
  'existing_cui_name': 'allergy testing rat',
  'existing_cui_desc': '[CLS] all ##er ##gy testing rat [unused3] ( Health Care Act ##ivity , lab test : rat ) [SEP]',
  'our_bi_cui': 'C0034721',
  'our_bi_cui_name': 'Rattus',
  'our_bi_cui_desc': '[CLS] Rat ##tus [unused3] ( E ##uka ##ryo ##te , organism : Rat ; Genus rat ##tus ; Genus Rat ##tus ; rats ; rat ; rat ##tus ; Rat ##s ) [SEP]',
  'our_cross_cui': 'C0034716',
  'our_cross_cui_name': 'Rats , Wistar',
  'our_cross_cui_desc': '[CLS] Rat ##s , W ##ista ##r [unused3] ( E ##uka ##ryo ##te , organism : W ##ista ##r rat ; W ##ista ##r Rat ##s ; w ##ista ##r rat ; w ##ista ##r rats ) [SEP]'},
 {'mention_id': '28521168.28',
  'test_set_idx': 38297,
  'type': 'T204',
  'name': 'L. variegatus',
  'context': '[CLS] of eight e ##cot ##ox ##icit ##y tests on bacteria ( V . fish ##eri , E . co ##li ) , algae ( P . sub ##cap ##ita ##ta , Ch ##lam ##yd ##omo ##nas s ##p . ) , crust ##ace ##ans ( D . ma ##gna ) , worms ( E . f ##eti ##da , [unused1] L . var ##ie ##gat ##us [unused2] ) and plants ( R . sat ##iv ##us , L . multi ##f ##lor ##um ) . The tested materials are commercially available and include Fe oxide and na ##nos ##cale zero v ##ale ##nt iron , but also hybrid products with Fe loaded into a matrix . All but one material , a ball mill ##ed na [SEP]',
  'existing_cui': 'C3754415',
  'existing_cui_name': 'Clania variegata',
  'existing_cui_desc': '[CLS] Clan ##ia var ##ie ##gata [unused3] ( E ##uka ##ryo ##te : E ##ume ##ta var ##ie ##gata ; Clan ##ia var ##ie ##gata ( S ##nell ##en , 1879 ) ; O ##ike ##ticus var ##ie ##gat ##us ) [SEP]',
  'our_bi_cui': 'C1022967',
  'our_bi_cui_name': 'Lumbriculus variegatus',
  'our_bi_cui_desc': '[CLS] Lu ##mb ##ric ##ulus var ##ie ##gat ##us [unused3] ( E ##uka ##ryo ##te : Lu ##mb ##ric ##ulus var ##ie ##gat ##us ( Muller , 1774 ) ) [SEP]',
  'our_cross_cui': 'C0998692',
  'our_cross_cui_name': 'Lytechinus variegatus',
  'our_cross_cui_desc': '[CLS] L ##yte ##chin ##us var ##ie ##gat ##us [unused3] ( E ##uka ##ryo ##te : L ##yte ##chin ##us var ##ie ##gat ##us ( Lamar ##ck , 1816 ) ) [SEP]'},
 {'mention_id': '27252199.9',
  'test_set_idx': 851,
  'type': 'T017',
  'name': 'vascular tissues',
  'context': '[CLS] Mo ##rp ##hol ##ogical characteristics regulating p ##hall ##ic g ##lan ##s en ##gor ##gement in the American all ##iga ##tor The di ##stal part of the c ##ro ##co ##di ##lian p ##hall ##us consists of a bulb ##ous g ##lan ##s containing well - developed [unused1] vascular tissues [unused2] that can in ##f ##late before or during sexual activity , en ##lar ##ging and el ##ab ##ora ##ting the g ##lan ##s into a complex , though still functional ##ly und ##ef ##ined , cop ##ulatory structure . An enlarged g ##lan ##s put ##ative ##ly interact ##s with the female c ##lo ##aca and may change the shape of her reproductive tract to facilitate ins ##em ##ination and increase the probability of f [SEP]',
  'existing_cui': 'C0282607',
  'existing_cui_name': 'Vascular Neoplasms',
  'existing_cui_desc': '[CLS] V ##as ##cular Neo ##p ##las ##ms [unused3] ( B ##iol ##og ##ic Fun ##ction : V ##as ##cular Tu ##mor ##s ; V ##as ##cular Tu ##mor ; vascular neo ##p ##las ##ms ; NE ##OP ##L VA ##SC ##U ##LA ##R ; Neo ##p ##las ##m of V ##as ##cular T ##iss ##ue ; V ##as ##cular neo ##p ##las ##m ; Neo ##p ##las ##ms , V ##as ##cular ; V ##as ##cular neo ##p ##las ##ms ; Neo ##p ##las ##m , V ##as ##cular ; VA ##SC ##U ##LA ##R NE ##OP ##L ; V ##as ##cular Neo ##p ##las ##ms [ Disease / Finding ] ; V ##as ##cular T ##iss ##ue Neo ##p ##las ##m ; V ##as ##cular [SEP]',
  'our_bi_cui': 'C0458940',
  'our_bi_cui_name': 'Vulval vascular structure',
  'our_bi_cui_desc': '[CLS] V ##ul ##val vascular structure [unused3] ( Ana ##tom ##ical Structure , body structure ) [SEP]',
  'our_cross_cui': 'C1511228',
  'our_cross_cui_name': 'Blood Vessel Tissue',
  'our_cross_cui_desc': '[CLS] Blood V ##ess ##el T ##iss ##ue [unused3] ( Ana ##tom ##ical Structure : V ##ES ##SE ##L , B ##L ##O ##OD ; V ##ess ##el ) [SEP]'},
 {'mention_id': '27395442.13',
  'test_set_idx': 4644,
  'type': 'T082',
  'name': 'Lima',
  'context': '[CLS] ##mo ##x are key processes for reducing the external nitrogen loads delivered to coastal ecosystems , and these processes can be affected by poll ##uta ##nts . In this study , we investigated the effect of crude oil on den ##it ##rification and an ##am ##mo ##x . Control ##led laboratory experiments were performed using sediment s ##lu ##rries from the [unused1] Lima [unused2] E ##st ##uary ( NW Portugal ) . Ana ##mm ##ox and den ##it ##rification rates were measured using ( 15 ) N - labeled NO ##3 ( - ) , and the production of ( 29 ) N ##2 and ( 30 ) N ##2 q ##uant ##ified by membrane inlet mass s ##pect ##rome ##try . Results revealed that while [SEP]',
  'existing_cui': 'C1047251',
  'existing_cui_name': 'Lima',
  'existing_cui_desc': '[CLS] Lima [unused3] ( E ##uka ##ryo ##te : l ##ima ) [SEP]',
  'our_bi_cui': 'C1047251',
  'our_bi_cui_name': 'Lima',
  'our_bi_cui_desc': '[CLS] Lima [unused3] ( E ##uka ##ryo ##te : l ##ima ) [SEP]',
  'our_cross_cui': 'C0017446',
  'our_cross_cui_name': 'Geographic Locations',
  'our_cross_cui_desc': '[CLS] Geographic Location ##s [unused3] ( Spa ##tial Concept : areas geographic ; Geographic Location ; Region ; GE ##O ##GR L ##OC ##AT ##ION ##S ; geographic region ; Geographic Region ; geographic location ; Geographic Region ##s ; Area ; geographic site ; Geographic Area ) [SEP]'},
 {'mention_id': '28427655.15',
  'test_set_idx': 35585,
  'type': 'T033',
  'name': 'not',
  'context': '[CLS] ##tina ##se concentration ( 0 and 0 . 2 % ) . The responses studied were yield , v ##isco ##sity , clarity , total soluble solid ##s and pH . It was observed that p ##ec ##tina ##se was more effective in increasing the yield of juice compared to cell ##ula ##se . Ultra ##sonic pre - treatment alone did [unused1] not [unused2] significantly increase the yield of juice . When ultra ##sound was combined with pre - treatment with both the enzymes maximum yield of 89 . 40 % was obtained compared to 47 . 30 % in the control . The v ##isco ##sity of the juice decreased with addition of enzymes and with application of ultra ##sound . The clarity of the [SEP]',
  'existing_cui': 'C1140150',
  'existing_cui_name': 'NOC - CodeSystem',
  'existing_cui_desc': '[CLS] NO ##C - Code ##S ##ys ##tem [unused3] ( Intel ##lect ##ual Product , NO ##C : nursing outcomes classification ) [SEP]',
  'our_bi_cui': 'C0205160',
  'our_bi_cui_name': 'Negative',
  'our_bi_cui_desc': '[CLS] N ##eg ##ative [unused3] ( Finding , qualifier value : Rule ##d out ; N ##eg ##ative ( m ##od ##ifier ) [ Am ##bi ##guous ] ; N ##eg ##ative for ; N ##eg ##ative ( m ##od ##ifier ) ; ruled out ; for negative ; out ruled ; negative ; negative ##s ) [SEP]',
  'our_cross_cui': 'C1513916',
  'our_cross_cui_name': 'Negative Finding',
  'our_cross_cui_desc': '[CLS] N ##eg ##ative Finding [unused3] ( Finding : NE ##GA ##TI ##VE ; N ##eg ##ative ) [SEP]'},
 {'mention_id': '27876084.11',
  'test_set_idx': 20251,
  'type': 'T092',
  'name': 'American College of Critical Care Medicine',
  'context': '[CLS] determine whether se ##ptic shock - reversal is quicker in p ##ediatric patients random ##ized to an early goal - directed fluid - spa ##ring strategy versus usual care ( S ##Q ##UE ##E ##Z ##E ) : study protocol for a pilot random ##ized controlled trial Current p ##ediatric se ##ptic shock re ##sus ##citation guidelines from the [unused1] American College of Critical Care Medicine [unused2] focus on the early and goal - directed administration of in ##tra ##vas ##cular fluid followed by v ##as ##oa ##ctive medication in ##fusion ##s for persistent and fluid - re ##fra ##ctor ##y shock . However , a ##cc ##um ##ulating adult and p ##ediatric data suggest that excessive fluid administration is associated with worse patient outcomes and [SEP]',
  'existing_cui': 'C1319411',
  'existing_cui_name': 'Critical care medicine specialty',
  'existing_cui_desc': '[CLS] Critical care medicine specialty [unused3] ( B ##io ##medical O ##cc ##upation or Disc ##ip ##line , qualifier value : Critical care medicine ; critical care medicine ; care critical medicine ) [SEP]',
  'our_bi_cui': 'C1527272',
  'our_bi_cui_name': 'American College of Physicians',
  'our_bi_cui_desc': '[CLS] American College of Physicians [unused3] ( Professional or O ##cc ##upation ##al Group ) [SEP]',
  'our_cross_cui': 'C1708333',
  'our_cross_cui_name': 'Health Care Organization',
  'our_cross_cui_desc': '[CLS] Health Care Organization [unused3] ( Organization : Healthcare Organization ) [SEP]'},
 {'mention_id': '27539945.45',
  'test_set_idx': 9870,
  'type': 'T103',
  'name': 'cationic CnDMAOH ( + )',
  'context': '[CLS] / C ##10 ##DM ##A ##O system , however , no DNA compact ##ion was observed even in Tri ##s - HC ##l buffer solution with a much lower pH and a much higher C ##10 ##DM ##A ##O concentration . The negative charges of DNA molecules can easily be neutral ##ized by positive charges of [unused1] cat ##ion ##ic C ##n ##DM ##A ##OH ( + ) [unused2] ( n = 12 and 14 ) mice ##lles . DNA was compact ##ed and then ins ##ol ##ub ##le DNA / C ##n ##DM ##A ##OH ( + ) complexes were formed . Because of the much higher critical mice ##lle concentration of the shorter chain length C ##10 ##DM ##A ##OH ( + ) , [SEP]',
  'existing_cui': 'C1685101',
  'existing_cui_name': 'N-deoxychoylethylenediamine',
  'existing_cui_desc': '[CLS] N - de ##ox ##ych ##oy ##let ##hyl ##ened ##iam ##ine [unused3] ( Chemical : cat ##ion ##ic DC ##EA ) [SEP]',
  'our_bi_cui': 'C0007447',
  'our_bi_cui_name': 'Cations',
  'our_bi_cui_desc': '[CLS] Cat ##ions [unused3] ( Chemical , substance : Cat ##ions [ Chemical / In ##g ##red ##ient ] ; Cat ##ion ; cat ##ions ; cat ##ion ) [SEP]',
  'our_cross_cui': 'C0029224',
  'our_cross_cui_name': 'Organic Chemicals',
  'our_cross_cui_desc': '[CLS] Organic Chemical ##s [unused3] ( Chemical , substance : organic chemicals ; compound organic ; Organic Chemical ##s [ Chemical / In ##g ##red ##ient ] ; organic compound ; Chemical ##s , Organic ; organic chemical ; Organic Chemical ; chemical organic ; organic compounds ; Organic compound ; Organic compounds ; Organic chemical ) [SEP]'},
 {'mention_id': '27334471.4',
  'test_set_idx': 2661,
  'type': 'T103',
  'name': 'TSSs',
  'context': '[CLS] CA ##GE ##d - o ##PO ##SS ##UM : motif en ##rich ##ment analysis from Cap Analysis of Gene Express ##ion - derived [unused1] T ##SS ##s [unused2] With the emergence of large - scale Cap Analysis of Gene Express ##ion data ##sets from individual labs and the FA ##NT ##OM consortium , one can now analyze the c ##is - regulatory regions associated with gene transcription at an unprecedented level of re ##fine ##ment . By coupling transcription factor binding site en ##rich ##ment analysis with Cap Analysis of Gene Express ##ion - derived g ##eno ##mic regions , CA ##GE ##d - o ##PO ##SS ##UM can identify T ##F ##s that act as key regulator ##s of genes involved in specific ma ##mmal [SEP]',
  'existing_cui': 'C0600327',
  'existing_cui_name': 'Toxic Shock Syndrome',
  'existing_cui_desc': '[CLS] To ##xi ##c Shock S ##yn ##drome [unused3] ( B ##iol ##og ##ic Fun ##ction , T ##SS , diagnosis , disorder , syndrome : toxic shock syndrome ; Shock , To ##xi ##c ; toxic ; shock ; shock ; toxic ; To ##xi ##c shock syndrome ; S ##yn ##drome ##s , To ##xi ##c Shock ; S ##yn ##drome , To ##xi ##c Shock ; shock syndrome ; toxic ; toxic ; shock syndrome ; S ##yn ##drome toxic shock ; Shock S ##yn ##drome , To ##xi ##c ; T ##SS , To ##xi ##c Shock S ##yn ##drome ; To ##xi ##c Shock S ##yn ##drome ##s ; T ##SS - To ##xi ##c shock syndrome ; syndrome ; toxic shock [SEP]',
  'our_bi_cui': 'C0017337',
  'our_bi_cui_name': 'Genes',
  'our_bi_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]',
  'our_cross_cui': 'C0949639',
  'our_cross_cui_name': 'Transcription Initiation Site',
  'our_cross_cui_desc': '[CLS] Trans ##cription In ##iti ##ation Site [unused3] ( Chemical : Start Sites , Trans ##cription ; In ##iti ##ation Site , Trans ##cription ; In ##iti ##ation Sites , Trans ##cription ; Site , Trans ##cription Start ; Start Site , Trans ##cription ; Trans ##cription Start Sites ; Site , Trans ##cription In ##iti ##ation ; Trans ##cription Start Site ; Trans ##cription In ##iti ##ation Sites ; Sites , Trans ##cription Start ; Sites , Trans ##cription In ##iti ##ation ) [SEP]'},
 {'mention_id': '28195652.2',
  'test_set_idx': 27976,
  'type': 'T103',
  'name': 'piperidinol derivatives',
  'context': '[CLS] I ##dent ##ification of the anti - my ##co ##ba ##cter ##ial functional properties of [unused1] pipe ##rid ##ino ##l derivatives [unused2] Tu ##ber ##cu ##los ##is remains a major global health threat and is now the leading cause of death from a single infectious agent worldwide . The current Tu ##ber ##cu ##los ##is drug regime ##n is inadequate , and new anti - tube ##rc ##ular agents are urgent ##ly required to be able to successfully combat the increasing prevalence of drug - resistant Tu ##ber ##cu ##los ##is . The purpose of this study was to investigate a pipe ##rid ##ino ##l compound derivative that is highly active against the My ##co ##ba ##cter ##ium tuberculosis b ##ac ##ill ##us . The anti [SEP]',
  'existing_cui': 'C1268914',
  'existing_cui_name': 'Piperidine derivative',
  'existing_cui_desc': '[CLS] Piper ##id ##ine derivative [unused3] ( Chemical , product , substance ) [SEP]',
  'our_bi_cui': 'C1268914',
  'our_bi_cui_name': 'Piperidine derivative',
  'our_bi_cui_desc': '[CLS] Piper ##id ##ine derivative [unused3] ( Chemical , product , substance ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '28468980.17',
  'test_set_idx': 37011,
  'type': 'T103',
  'name': 'rPfSEA-1A',
  'context': '[CLS] ##long ##s survival following P ##las ##mo ##dium be ##rg ##he ##i AN ##KA challenge . To enhance the im ##mu ##no ##genic ##ity of P ##f ##SE ##A - 1A , we identified five linear , B - cell e ##pit ##ope ##s using p ##eptide micro ##arra ##ys probe ##d with anti - se ##ra from [unused1] r ##P ##f ##SE ##A - 1A [unused2] - v ##ac ##cin ##ated non - human p ##rima ##tes . We evaluated the relationship between e ##pit ##ope - specific anti ##body levels and protection from parasite ##mia in a longitudinal treatment - re ##in ##fect ##ion co ##hor ##t in Western Kenya . Anti ##bo ##dies to three e ##pit ##ope ##s were associated with 16 [SEP]',
  'existing_cui': 'C0120659',
  'existing_cui_name': 'HA-1A',
  'existing_cui_desc': '[CLS] H ##A - 1A [unused3] ( Chemical , product , substance ) [SEP]',
  'our_bi_cui': 'C0487378',
  'our_bi_cui_name': 'Plasmodium falciparum antigen',
  'our_bi_cui_desc': '[CLS] P ##las ##mo ##dium f ##al ##ci ##par ##um anti ##gen [unused3] ( Chemical , substance : P ##las ##mo ##dium f ##al ##ci ##par ##um A ##g ; P ##las ##mo ##dium f ##al ##ci ##par ##um Anti ##gen ) [SEP]',
  'our_cross_cui': 'C3849939',
  'our_cross_cui_name': 'SEA-1 protein , Plasmodium falciparum',
  'our_cross_cui_desc': '[CLS] SE ##A - 1 protein , P ##las ##mo ##dium f ##al ##ci ##par ##um [unused3] ( Chemical : P ##f ##SE ##A - 1 protein , P ##las ##mo ##dium f ##al ##ci ##par ##um ; s ##chi ##zon ##t e ##g ##ress anti ##gen - 1 , P ##las ##mo ##dium f ##al ##ci ##par ##um ) [SEP]'},
 {'mention_id': '28523425.26',
  'test_set_idx': 38391,
  'type': 'T082',
  'name': 'M1',
  'context': '[CLS] Specifically , subjects performed intermittent pin ##cer grasp / release , elbow fl ##ex ##ion / extension , and shoulder fl ##ex ##ion / extension at slow , moderate , and fast speeds . In all movements , bursts of power in the high - [ Formula : see text ] band ( 80 - 160 Hz ) were observed in [unused1] M1 [unused2] . In addition , the amplitude of these power bursts and the area of M1 with elevated high - [ Formula : see text ] activity were directly proportional to the movement speed . Likewise , the duration of elevated high - [ Formula : see text ] activity increased with movement duration . Based on linear re ##gression , M1 high [SEP]',
  'existing_cui': 'C1705014',
  'existing_cui_name': 'CHRM1 wt Allele',
  'existing_cui_desc': '[CLS] CH ##R ##M ##1 w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : M1 ##R ; HM ##1 ; Cho ##liner ##gic Re ##ceptor , Mu ##sca ##rini ##c , 1 Gene ; Ace ##ty ##l ##cho ##line Re ##ceptor , Mu ##sca ##rini ##c 1 Gene ; Cho ##liner ##gic Re ##ceptor , Mu ##sca ##rini ##c 1 w ##t All ##ele ; Ace ##ty ##l ##cho ##line Re ##ceptor , Mu ##sca ##rini ##c , 1 Gene ) [SEP]',
  'our_bi_cui': 'C0026607',
  'our_bi_cui_name': 'Motor Cortex',
  'our_bi_cui_desc': '[CLS] Motor Co ##rte ##x [unused3] ( Spa ##tial Concept , Motor , body structure : Co ##rte ##x ; Area , Motor ; Motor Area ; Areas , So ##matic Motor ; so ##mat ##omo ##tor areas ; Motor area ; Co ##rte ##x , Motor ; So ##mat ##omo ##tor Area ; Areas , Motor ; motor cortex ; Area , So ##matic Motor ; Motor Areas ; so ##matic motor areas ; So ##mat ##omo ##tor Areas ; Motor Areas , So ##matic ; motor area ; areas motor ; So ##matic Motor Area ; Area , So ##mat ##omo ##tor ; Motor Area , So ##matic ; Areas , So ##mat ##omo ##tor ; Motor cortex ; cortex motor ; So ##matic Motor [SEP]',
  'our_cross_cui': 'C3495441',
  'our_cross_cui_name': 'primary motor cortex',
  'our_cross_cui_desc': '[CLS] primary motor cortex [unused3] ( Spa ##tial Concept , Carpenter , Crosby , M I : area F1 ; Motor Area , Primary ; Motor Co ##rt ##ices , Primary ; primary so ##mat ##omo ##tor area ; primary motor area ; Motor Strip ; Pre ##cent ##ral Motor Co ##rte ##x ; Pre ##cent ##ral Motor Areas ; Area , Primary Motor ; motor cortex ; Motor Co ##rt ##ices , Pre ##cent ##ral ; Primary Motor Co ##rt ##ices ; pre ##cent ##ral motor cortex ; motor strip ; Primary Motor Area ; Motor cortex ; Area , Pre ##cent ##ral Motor ; Motor Co ##rte ##x , Pre ##cent ##ral ; pre ##cent ##ral motor area ; Primary Motor Areas ; Co ##rt [SEP]'},
 {'mention_id': '28282592.40',
  'test_set_idx': 31122,
  'type': 'T038',
  'name': 'activation pathway',
  'context': '[CLS] Trans ##ient rotor ##s terminated when t ##rave ##rsing a region with differing re ##fra ##ctor ##y periods , if no obstacle to conduct ##ion was present to sufficiently delay wave ##front arrival beyond the longest re ##fra ##ctor ##y period . Other rotor ##s were an ##ni ##hil ##ated when a nearby rotor with faster spin rate gradually interrupted the [unused1] activation pathway [unused2] . Eli ##mination of anchor ##s by removal , or by simulated a ##bla ##tion over a sufficient region , prevented rotor onset at a particular location where it would otherwise form . The presence of obstacles to conduct ##ion and spatial differences in re ##fra ##ctor ##y period are important parameters for in ##iti ##ating and maintaining rotation ##al activity [SEP]',
  'existing_cui': 'C3156336',
  'existing_cui_name': 'protein activation cascade',
  'existing_cui_desc': '[CLS] protein activation ca ##sca ##de [unused3] ( B ##iol ##og ##ic Fun ##ction : protein act ##iv ##itor ##y ca ##sca ##de ; protein activation pathway ) [SEP]',
  'our_bi_cui': 'C1704259',
  'our_bi_cui_name': 'Biochemical Pathway',
  'our_bi_cui_desc': '[CLS] B ##io ##chemical Path ##way [unused3] ( B ##iol ##og ##ic Fun ##ction : molecular pathway ; Path ##ways ; Path ##way ) [SEP]',
  'our_cross_cui': 'C1148560',
  'our_cross_cui_name': 'molecular_function',
  'our_cross_cui_desc': '[CLS] molecular _ function [unused3] ( B ##iol ##og ##ic Fun ##ction : molecular function ) [SEP]'},
 {'mention_id': '27692319.46',
  'test_set_idx': 14317,
  'type': 'T204',
  'name': 'miracidia',
  'context': '[CLS] mg / m ##L . Egg ##s in ##cu ##bate ##d with crude extract sub - fraction ##s at concentrations of 1000 , 100 , 10 , 1 , 0 . 1 , 0 . 01 μ ##g / m ##L affected em ##b ##ryo ##nic development , with n - but ##ano ##l presenting the strongest in ##hibition of [unused1] mi ##rac ##id ##ia [unused2] formation . In contrast , on the 12th day , 90 % of the mi ##rac ##id ##ia hatch ##ed in the control experiments using 0 . 03 % D ##MS ##O whereas em ##b ##ryo ##genesis was completely abolished with any concentration of al ##ben ##da ##zo ##le su ##l ##ph ##oxide AB ##Z ( S ##O ) . [SEP]',
  'existing_cui': 'C0451085',
  'existing_cui_name': 'Composite international diagnostic interview',
  'existing_cui_desc': '[CLS] Co ##mpo ##site international diagnostic interview [unused3] ( Intel ##lect ##ual Product , C ##ID ##I , assessment scale : composite international diagnostic interview ; C ##ID ##I - Co ##mpo ##site international diagnostic interview ) [SEP]',
  'our_bi_cui': 'C2807025',
  'our_bi_cui_name': 'Focillidia',
  'our_bi_cui_desc': '[CLS] F ##oc ##ill ##id ##ia [unused3] ( E ##uka ##ryo ##te ) [SEP]',
  'our_cross_cui': 'C0023047',
  'our_cross_cui_name': 'Larva',
  'our_cross_cui_desc': '[CLS] La ##rva [unused3] ( E ##uka ##ryo ##te : La ##rva ##e ; la ##rva ; larvae ) [SEP]'},
 {'mention_id': '28145492.37',
  'test_set_idx': 26797,
  'type': 'T103',
  'name': 'OX2R receptor',
  'context': '[CLS] in the presence of te ##tro ##do ##to ##xin ) , appeared to be carried by a Na ( + ) - dependent trans ##ient receptor potential - like channel ( as it was blocked by 2 - AP ##B and was diminished by removal of extra ##cellular Na ( + ) ) , and was a consequence of [unused1] O ##X ##2 ##R receptor [unused2] activation ( as it was blocked by the O ##X ##2 ##R receptor antagonist T ##CS O ##X ##2 29 , but not the O ##X ##1 ##R receptor antagonist S ##B 33 ##48 ##6 ##7 ) . Application of the hormone , me ##lat ##oni ##n , failed to alter tube ##roi ##n ##fu ##ndi ##bula ##r do ##pa [SEP]',
  'existing_cui': 'C0913649',
  'existing_cui_name': 'taste receptors , type 2',
  'existing_cui_desc': '[CLS] taste receptors , type 2 [unused3] ( Chemical , taste : G protein - coupled receptor T ##R ##2 ; T ##2 ##R taste receptors ; T ##2 ##R receptor ; T ##2 ##R receptors ; taste receptor , type 2 ; T ##AS ##2 ##R ##13 protein , human ; receptor , T ##2 ##R ) [SEP]',
  'our_bi_cui': 'C0069841',
  'our_bi_cui_name': 'Oxytocin Receptor',
  'our_bi_cui_desc': '[CLS] O ##xy ##to ##cin Re ##ceptor [unused3] ( Chemical : Re ##ceptor ##s , O ##xy ##to ##cin [ Chemical / In ##g ##red ##ient ] ; R ##EC ##EP ##T O ##X ##Y ##TO ##CI ##N ; Re ##ceptor ##s , O ##xy ##to ##cin ; O ##X ##Y ##TO ##CI ##N R ##EC ##EP ##T ; Re ##ceptor , O ##xy ##to ##cin ; O ##xy ##to ##cin Re ##ceptor ##s ) [SEP]',
  'our_cross_cui': 'C0671880',
  'our_cross_cui_name': 'orexin receptor type 2',
  'our_cross_cui_desc': '[CLS] ore ##xin receptor type 2 [unused3] ( Chemical : Or ##ex ##in - B Re ##ceptor ##s ; H ##y ##po ##cre ##tin - 2 Re ##ceptor ; Re ##ceptor , H ##y ##po ##cre ##tin - 2 ; H ##y ##po ##cre ##tin Re ##ceptor ##s 2 ; O ##X ##2 Re ##ceptor ##s ; Or ##ex ##in B Re ##ceptor ; Re ##ceptor ##s , Or ##ex ##in - B ; Re ##ceptor ##s 2 , H ##y ##po ##cre ##tin ; Or ##ex ##in Re ##ceptor 2 ; Re ##ceptor 2 , H ##y ##po ##cre ##tin ; HC ##TR ##2 Pro ##tein ; Re ##ceptor , Or ##ex ##in - B ; Or ##ex ##in Re ##ceptor Type 2 ; Pro ##tein [SEP]'},
 {'mention_id': '27838742.73',
  'test_set_idx': 19040,
  'type': 'T103',
  'name': 'Delicaflavone',
  'context': '[CLS] m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . Del ##ica ##f ##lav ##one showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . Del ##ica ##f ##lav ##one induced auto ##pha ##gic cell death via A ##kt / m ##TO ##R / p ##70 ##S ##6 ##K signaling pathway . [unused1] Del ##ica ##f ##lav ##one [unused2] did not show o ##bs ##er ##vable side effects in a x ##eno ##gra ##ft mouse model . Del ##ica ##f ##lav ##one may represent a potential therapeutic agent for lung cancer . Del ##ica ##f ##lav ##one showed anti - lung cancer effects in v ##it ##ro and in v ##ivo . Del ##ica ##f ##lav ##one induced auto [SEP]',
  'existing_cui': 'C1870828',
  'existing_cui_name': 'sumaflavone',
  'existing_cui_desc': '[CLS] sum ##af ##lav ##one [unused3] ( Chemical ) [SEP]',
  'our_bi_cui': 'C2828290',
  'our_bi_cui_name': 'delafloxacin',
  'our_bi_cui_desc': '[CLS] del ##af ##lo ##xa ##cin [unused3] ( Chemical : 1 - ( 6 - Am ##ino - 3 , 5 - Di ##f ##lu ##oro ##py ##rid ##in - 2 - Y ##l ) - 8 - Ch ##lor ##o - 6 - F ##lu ##oro - 7 - ( 3 - H ##ydro ##xy ##az ##eti ##din - 1 - y ##l ) - 4 - O ##x ##o - 1 , 4 - Di ##hy ##dr ##o ##quin ##olin ##e - 3 - Car ##box ##yl ##ic A ##cid ; Del ##af ##lo ##xa ##cin ; DE ##LA ##FL ##OX ##AC ##IN ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27894019.20',
  'test_set_idx': 21081,
  'type': 'T103',
  'name': 'CYP315a1',
  'context': '[CLS] ##e p ##le ##bs . X ##ant ##hur ##eni ##c acid is a known end ##ogen ##ous m ##olt regulator present in art ##hr ##op ##ods . Experimental studies have confirmed that X ##ant ##hur ##eni ##c acid in ##hibit ##s m ##olt ##ing by binding to either ( or both ) of two P ##45 ##0 enzymes ( [unused1] C ##YP ##31 ##5 ##a ##1 [unused2] or C ##YP ##31 ##4 ##a ##1 ) that are responsible for the final two h ##ydro ##xy ##lation ##s in the production of the m ##olt - in ##ducing hormone , 20 - h ##ydro ##xy ##ec ##dy ##son ##e . The lack of crystal structures and bio ##chemical ass ##ays for C ##YP ##31 ##5 ##a ##1 [SEP]',
  'existing_cui': 'C1418799',
  'existing_cui_name': 'PPIF gene',
  'existing_cui_desc': '[CLS] PP ##IF gene [unused3] ( Ana ##tom ##ical Structure : C ##Y ##CL ##OP ##H ##IL ##IN 3 ; P ##EP ##TI ##D ##Y ##L - PR ##OL ##Y ##L C ##IS / T ##RA ##NS ISO ##ME ##RA ##SE , MIT ##OC ##H ##ON ##DR ##IA ##L ; P ##ept ##idy ##l ##p ##rol ##yl Is ##omer ##ase F Gene ; PP ##IF Gene ; C ##Y ##CL ##OP ##H ##IL ##IN F ; p ##ept ##idy ##l ##p ##rol ##yl is ##omer ##ase F ; Cy ##p - D ; h ##C ##y ##P ##3 ; c ##y ##c ##lop ##hil ##in D ) [SEP]',
  'our_bi_cui': 'C0059735',
  'our_bi_cui_name': 'Cytochrome P-450 CYP1A1',
  'our_bi_cui_desc': '[CLS] Cy ##to ##ch ##rome P - 450 C ##YP ##1 ##A ##1 [unused3] ( Chemical , substance : Cy ##to ##ch ##rome P 450 C ##YP ##1 ##A ##1 ; Cy ##to ##ch ##rome p ##45 ##0 C ##YP ##1 ##A ##1 enzyme ; er ##od ##ing ; 7 - E ##th ##ox ##yres ##or ##uf ##in O - Dee ##thy ##lase ; C ##YP 1A ##1 ; E ##thy ##l ##res ##or ##uf ##in O - Dee ##thy ##lase ; Cy ##to ##ch ##rome P - 450 C ##YP ##1 ##A ##1 [ Chemical / In ##g ##red ##ient ] ; Cy ##to ##ch ##rome P ##45 ##0 I ##A ##1 ; C ##YP ##1 ##A ##1 ; E ##th ##ox ##yres ##or ##uf ##in [SEP]',
  'our_cross_cui': 'C0010762',
  'our_cross_cui_name': 'Cytochrome P450',
  'our_cross_cui_desc': '[CLS] Cy ##to ##ch ##rome P ##45 ##0 [unused3] ( Chemical , substance : c ##yt ##och ##rome ##s p ##45 ##0 ; c ##yt ##och ##rome P ##45 ##0 ; C ##Y ##TO ##CH ##RO ##ME P 450 E ##N ##Z ##Y ##ME S ##Y ##ST ##EM ; Cy ##to ##ch ##rome p ##45 ##0 enzyme ; Super ##family , Cy ##to ##ch ##rome P ##45 ##0 ; P ##45 ##0 En ##zy ##mes ; c ##yt ##och ##rome p 450 ; En ##zy ##mes , P ##45 ##0 ; Cy ##to ##ch ##rome P 450 Families ; Cy ##to ##ch ##rome P ##45 ##0 En ##zy ##me ; P ##45 ##0 ; C ##YP ##45 ##0 ; Cy ##to ##ch ##rome p ##45 ##0 ; [SEP]'},
 {'mention_id': '27669217.40',
  'test_set_idx': 13691,
  'type': 'T017',
  'name': 'ermB',
  'context': '[CLS] . The er ##m ##B and er ##m ##T genes were found in the genome ##s of all is ##olate ##s . These two genes were acquired independently in one highly v ##ir ##ulent is ##olate AL ##10 ##100 ##2 , and cluster ##ed with T ##n ##9 ##16 and IS ##12 ##16 , respectively . The expression of both [unused1] er ##m ##B [unused2] and er ##m ##T in all is ##olate ##s was er ##yt ##hr ##omy ##cin in ##du ##cible and yielded comparable mac ##rol ##ide MI ##Cs in all six is ##olate ##s . Take ##n together , in ##du ##cible expression of both er ##m ##B and er ##m ##T conferred high mac ##rol ##ide resistance in these S . gal [SEP]',
  'existing_cui': 'C0732611',
  'existing_cui_name': 'Selective Estrogen Receptor Modulators',
  'existing_cui_desc': '[CLS] Select ##ive E ##stro ##gen Re ##ceptor Mo ##du ##lator ##s [unused3] ( Chemical , SE ##R ##M , medication , product , substance : E ##ST ##RO ##GE ##N R ##EC ##EP ##T M ##OD ##U ##LA ##TO ##RS SE ##LE ##CT ##IVE ; Select ##ive E ##stro ##gen Re ##ceptor Mo ##di ##fying Agents ; Select ##ive est ##rogen receptor m ##od ##ulator ##s ; Select ##ive o ##est ##rogen receptor m ##od ##ulator ; E ##stro ##gen Re ##ceptor Mo ##du ##lator , Select ##ive ; SE ##R ##M - Select ##ive est ##rogen receptor m ##od ##ulator ; SE ##R ##Ms ; SE ##R ##M - Select ##ive o ##est ##rogen receptor m ##od ##ulator ; SE ##LE ##CT ##IVE E ##ST [SEP]',
  'our_bi_cui': 'C0017339',
  'our_bi_cui_name': 'Genes , Bacterial',
  'our_bi_cui_desc': '[CLS] Gene ##s , Ba ##cter ##ial [unused3] ( Ana ##tom ##ical Structure , substance : Ba ##cter ##ial Gene ##s ; BA ##CT GE ##NE ##S ; GE ##NE ##S BA ##CT ; bacterial gene ; Ba ##cter ##ial genes ; Gene , Ba ##cter ##ial ; BA ##CT GE ##NE ; Ba ##cter ##ial gene ; Ba ##cter ##ial Gene ; GE ##NE BA ##CT ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '28522288.2',
  'test_set_idx': 38314,
  'type': 'T033',
  'name': 'Underestimation',
  'context': '[CLS] Grade Group [unused1] Under ##est ##imation [unused2] in Pro ##state B ##io ##psy : Pre ##dict ##ive Factor ##s and Out ##come ##s in Can ##di ##dates for Active Sur ##ve ##illa ##nce We intended to analyze the outcomes and predict ##ive factors for under ##est ##imating the pro ##state cancer grade group from pro ##state bio ##ps ##ies in a large mon ##oc ##ent ##ric co ##hor ##t of patients treated by minimal ##ly invasive radical pro ##state ##ct ##omy . Using a mon ##oc ##ent ##ric prospective ##ly maintained database , we included 306 ##2 patients who underwent minimal ##ly invasive radical pro ##state ##ct ##omy between 2006 and 2013 . We explored clinic ##op ##ath ##olo ##gic features and outcomes associated with a [SEP]',
  'existing_cui': 'C0680844',
  'existing_cui_name': 'estimation',
  'existing_cui_desc': '[CLS] est ##imation [unused3] ( B ##iol ##og ##ic Fun ##ction , subjective , subjective : E ##st ##imation ; est ##imation ##s ) [SEP]',
  'our_bi_cui': 'C0680844',
  'our_bi_cui_name': 'estimation',
  'our_bi_cui_desc': '[CLS] est ##imation [unused3] ( B ##iol ##og ##ic Fun ##ction , subjective , subjective : E ##st ##imation ; est ##imation ##s ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28484550.10',
  'test_set_idx': 37646,
  'type': 'T038',
  'name': 'giant',
  'context': '[CLS] Di ##stal posterior cerebral artery re ##vas ##cular ##ization for a f ##us ##iform PC ##A an ##eur ##ys ##m : A lesson learned The need for re ##vas ##cular ##ization with pro ##ximal posterior cerebral artery o ##cc ##lusion in the treatment of [unused1] giant [unused2] and f ##us ##iform an ##eur ##ys ##ms is unclear . While early series demonstrated only about a 10 % chance of in ##far ##ction following posterior cerebral artery o ##cc ##lusion , recently several authors have advocated a bypass prior to parent vessel sacrifice in all cases . We present the case of an adult man with a f ##us ##iform an ##eur ##ys ##m of the right posterior cerebral artery at the P ##2 - P ##3 [SEP]',
  'existing_cui': 'C0017547',
  'existing_cui_name': 'Gigantism',
  'existing_cui_desc': '[CLS] G ##igan ##tism [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , physical finding : G ##igan ##tism , Pit ##uit ##ary ; pit ##uit ##ary ; gig ##anti ##sm ; giants ; gig ##anti ##sm disorder ; giant ; gig ##anti ##sm ; pit ##uit ##ary ; Giant ; Normal giant ; gig ##anti ##sm was noted ; Pit ##uit ##ary G ##igan ##tism ; Giant ##ism ; gig ##anti ##sm ; P ##rim ##ord ##ial giant ; Gene ##tic giant ; genetics giant ; giant ##ism ; Pit ##uit ##ary gig ##anti ##sm ; Giant - R ##ET ##IR ##ED - ; G ##igan ##tism [ Disease / Finding ] ) [SEP]',
  'our_bi_cui': 'C0017547',
  'our_bi_cui_name': 'Gigantism',
  'our_bi_cui_desc': '[CLS] G ##igan ##tism [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , physical finding : G ##igan ##tism , Pit ##uit ##ary ; pit ##uit ##ary ; gig ##anti ##sm ; giants ; gig ##anti ##sm disorder ; giant ; gig ##anti ##sm ; pit ##uit ##ary ; Giant ; Normal giant ; gig ##anti ##sm was noted ; Pit ##uit ##ary G ##igan ##tism ; Giant ##ism ; gig ##anti ##sm ; P ##rim ##ord ##ial giant ; Gene ##tic giant ; genetics giant ; giant ##ism ; Pit ##uit ##ary gig ##anti ##sm ; Giant - R ##ET ##IR ##ED - ; G ##igan ##tism [ Disease / Finding ] ) [SEP]',
  'our_cross_cui': 'C0751004',
  'our_cross_cui_name': 'Giant Intracranial Aneurysm',
  'our_cross_cui_desc': '[CLS] Giant In ##tra ##c ##rani ##al An ##eur ##ys ##m [unused3] ( B ##iol ##og ##ic Fun ##ction : In ##tra ##c ##rani ##al An ##eur ##ys ##m , Giant ; An ##eur ##ys ##ms , Giant In ##tra ##c ##rani ##al ; In ##tra ##c ##rani ##al An ##eur ##ys ##ms , Giant ; Giant In ##tra ##c ##rani ##al An ##eur ##ys ##ms ; An ##eur ##ys ##m , Giant In ##tra ##c ##rani ##al ) [SEP]'},
 {'mention_id': '27887587.40',
  'test_set_idx': 20928,
  'type': 'T033',
  'name': 'practical',
  'context': "[CLS] Community Mental Health Centre in their work towards their recovery goals were el ##icit ##ed , as follows : developing an understanding of ones ##elf and one ' s mental health problems ; learning how to change feelings and behaviour ##s ; being ' pushed ' into social arena ##s ; finding helpful medication ; and counsel ##ling in family , [unused1] practical [unused2] and financial issues . The participants ' expectations about counsel ##ling with regard to longer - term family , practical , and financial challenges were insufficient ##ly met by the Community Mental Health Centre . In the experience of the service users , recovery occurred within the context of their everyday life with or without the support of their professional help ##ers [SEP]",
  'existing_cui': 'C0335403',
  'existing_cui_name': 'Practical aid',
  'existing_cui_desc': '[CLS] P ##ractical aid [unused3] ( B ##io ##medical O ##cc ##upation or Disc ##ip ##line , pharmacy , pharmacy , occupation : P ##ractical aid ( pharmacy ) ) [SEP]',
  'our_bi_cui': 'C4054160',
  'our_bi_cui_name': 'Practical Domain',
  'our_bi_cui_desc': '[CLS] P ##ractical Domain [unused3] ( Intel ##lect ##ual Product ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '28436493.28',
  'test_set_idx': 35794,
  'type': 'T103',
  'name': 'EsxA',
  'context': '[CLS] ##X - 1 type VII secret ##ion system . Gene ##tic complement ##ation with a p ##las ##mi ##d carrying the My ##co ##ba ##cter ##ium tuberculosis H ##37 ##R ##v sequence of e ##cc ##C ##a ##1 - e ##cc ##C ##b ##1 - p ##e ##35 re - established E ##s ##x ##A secret ##ion , host specific [unused1] E ##s ##x ##A [unused2] T - cell responses , and increased strain v ##ir ##ule ##nce . The E ##S ##X - 1 secret ##ion defect prevents several v ##ir ##ule ##nce factors from being functional during infection and therefore at ##ten ##uate ##s My ##co ##ba ##cter ##ium tuberculosis . It pre ##c ##lude ##s specific T - cell responses against strong anti [SEP]',
  'existing_cui': 'C1706322',
  'existing_cui_name': 'ELF3 wt Allele',
  'existing_cui_desc': '[CLS] E ##LF ##3 w ##t All ##ele [unused3] ( Ana ##tom ##ical Structure : E ##SE - 1 ; ER ##T ; J ##EN ; E ##S ##X ; E ##7 ##4 - Like Factor 3 ( ET ##S Domain Trans ##cription Factor , E ##pit ##hel ##ial - S ##pecific ) w ##t All ##ele ; EP ##R - 1 ) [SEP]',
  'our_bi_cui': 'C0052654',
  'our_bi_cui_name': 'atroxase',
  'our_bi_cui_desc': '[CLS] at ##ro ##xa ##se [unused3] ( Chemical , substance : At ##ro ##xa ##se ) [SEP]',
  'our_cross_cui': 'C1744314',
  'our_cross_cui_name': 'ESAT-6 protein , Mycobacterium tuberculosis',
  'our_cross_cui_desc': '[CLS] E ##SA ##T - 6 protein , My ##co ##ba ##cter ##ium tuberculosis [unused3] ( Chemical : E ##SA ##T - 6 System 1 protein , My ##co ##ba ##cter ##ium tuberculosis ; 6 - k ##D ##a early secret ##ory anti ##genic target , My ##co ##ba ##cter ##ium tuberculosis ; E ##s ##x - 1 protein , My ##co ##ba ##cter ##ium tuberculosis ; E ##s ##x ##A protein , My ##co ##ba ##cter ##ium tuberculosis ) [SEP]'},
 {'mention_id': '27347668.1',
  'test_set_idx': 3121,
  'type': 'T058',
  'name': 'Spectral fingerprints',
  'context': '[CLS] [unused1] S ##pect ##ral finger ##prints [unused2] of large - scale co ##rt ##ical dynamics during ambiguous motion perception Am ##bi ##guous stimuli have been widely used to study the ne ##uron ##al co ##rrel ##ates of consciousness . Recently , it has been suggested that conscious perception might arise from the dynamic inter ##play of functional ##ly specialized but widely distributed co ##rt ##ical areas . While previous research mainly focused on phase coupling as a co ##rrel ##ate of co ##rt ##ical communication , more recent findings indicated that additional coupling modes might co ##ex ##ist and possibly sub ##serve distinct co ##rt ##ical functions . Here , we studied two coupling modes , namely phase and envelope coupling , which might differ in [SEP]',
  'existing_cui': 'C0016128',
  'existing_cui_name': 'Fingerprints , Peptide',
  'existing_cui_desc': '[CLS] Fin ##ger ##prints , P ##eptide [unused3] ( Health Care Act ##ivity : Fin ##ger ##print , P ##eptide ; Fin ##ger ##prints , Pro ##tein ; Pro ##tein Fin ##ger ##prints ; Pro ##tein Fin ##ger ##print ; P ##eptide Fin ##ger ##print ; P ##eptide Fin ##ger ##prints ; Fin ##ger ##print , Pro ##tein ) [SEP]',
  'our_bi_cui': 'C0079356',
  'our_bi_cui_name': 'DNA Fingerprints',
  'our_bi_cui_desc': '[CLS] DNA Fin ##ger ##prints [unused3] ( Finding : Fin ##ger ##print , DNA ; Fin ##ger ##prints , DNA ; d ##na finger ##prints ; DNA Fin ##ger ##print ; d ##na finger ##print ) [SEP]',
  'our_cross_cui': 'C0037812',
  'our_cross_cui_name': 'Spectrum Analysis',
  'our_cross_cui_desc': '[CLS] Spectrum Analysis [unused3] ( Health Care Act ##ivity : s ##pect ##ros ##copy ; spectrum analysis ; SP ##EC ##TR ##UM AN ##AL ; S ##pect ##ros ##copy ; Analysis , Spectrum ; AN ##AL SP ##EC ##TR ##UM ; Spectrum analysis ) [SEP]'},
 {'mention_id': '27984742.11',
  'test_set_idx': 22893,
  'type': 'T103',
  'name': 'DarG',
  'context': '[CLS] Back to the Roots : Deep View into the Evolution ##ary History of AD ##P - R ##ib ##osy ##lation Opened by the DNA - Target ##ing To ##xin - Anti ##to ##xin Mo ##du ##le Dar ##T ##G In this issue of Molecular Cell , Jan ##ke ##vic ##ius et al . ( 2016 ) character ##ize the Dar ##T ##G to ##xin - anti ##to ##xin module in which the Dar ##T to ##xin AD ##P - rib ##osy ##lates single - stranded DNA and the [unused1] Dar ##G [unused2] anti ##to ##xin counter ##acts Dar ##T by direct binding and by en ##zy ##matic re ##mo ##va l of the AD ##P - rib ##osy ##lation . [SEP]',
  'existing_cui': 'C2767915',
  'existing_cui_name': 'Dargida',
  'existing_cui_desc': '[CLS] Dar ##gi ##da [unused3] ( E ##uka ##ryo ##te ) [SEP]',
  'our_bi_cui': 'C0969663',
  'our_bi_cui_name': 'DnaG',
  'our_bi_cui_desc': '[CLS] D ##na ##G [unused3] ( Chemical , P ##rima ##se , P ##rima ##se : D ##na ##G Pro ##tein ; D ##na ##G Gene Product ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '27442216.12',
  'test_set_idx': 5808,
  'type': 'T037',
  'name': 'appendicular fractures',
  'context': '[CLS] total of 75 ##8 patients who presented following either snow ##mobile ( n = 87 ) , AT ##V - related ( n = 308 ) or dirt ##bi ##ke ( n = 36 ##3 ) - related trauma at our institution between 1996 and 2015 were retrospective ##ly reviewed . A total of 44 ##1 a ##xial and [unused1] app ##end ##icular fracture ##s [unused2] occurred requiring 53 ##3 procedures . Snow ##mobile and dirt ##bi ##ke accidents were associated with a higher rate of fracture ##s ( 63 % , 64 % ) than the All - terrain vehicles group ( 50 % ) ( P = 0 . 000 ##8 ) . Snow ##mobile injuries had the highest rate of spinal ( 23 [SEP]',
  'existing_cui': 'C0226745',
  'existing_cui_name': 'Structure of appendicular vein',
  'existing_cui_desc': '[CLS] Structure of app ##end ##icular vein [unused3] ( Ana ##tom ##ical Structure , body structure : V ##ena app ##end ##icular ##is ; A ##ppe ##ndi ##cular vein ) [SEP]',
  'our_bi_cui': 'C0003611',
  'our_bi_cui_name': 'Appendectomy',
  'our_bi_cui_desc': '[CLS] A ##ppe ##nde ##ct ##omy [unused3] ( Health Care Act ##ivity , procedure , treatment : removal of app ##end ##ix ; app ##end ##ec ##tom ##ies ; A ##ppe ##nde ##ct ##omi ##es ; app ##end ##ice ##ct ##omy ; Re ##mo ##val of app ##end ##ix ; A ##ppe ##ndi ##x : Ex ##cision ; app ##end ##ec ##tom ##y procedure ; A ##ppe ##ndi ##ce ##ct ##omy ; app ##end ##ec ##tom ##y ; app ##end ##ice ##ct ##omi ##es ; Ex ##cision of app ##end ##ix ; app ##end ##ec ##tom ##y procedures ; Primary app ##end ##ec ##tom ##y ; app ##end ##ix ex ##cision ; Ex ##cision Pro ##ced ##ures on the A ##ppe ##ndi ##x ; AP ##P ##EN [SEP]',
  'our_cross_cui': 'C0016658',
  'our_cross_cui_name': 'Fracture',
  'our_cross_cui_desc': '[CLS] Fr ##act ##ure [unused3] ( In ##ju ##ry or Po ##ison ##ing , diagnosis , disorder , finding , m ##or ##ph ##olo ##gic abnormal ##ity : Broken bones ; Broken Bones ; broken bone ; Fr ##act ##ure of Bone ; bone ; fracture ; bones fractured ; Fr ##act ##ures ; fracture ; bone ; Fr ##act ##ures , Bone [ Disease / Finding ] ; fracture ##s ; fractured ; Bone , Broken ; Fr ##act ##ure of bones ; Fr ##act ##ure : [ of un ##sp ##ec ##ified bones ] or [ NO ##S ] ; Fr ##act ##ure , cause ; Fr ##act ##ure bone ; bone fracture ; bone fracture ##s ; bone fractured ; Bone Fr ##act ##ure [SEP]'},
 {'mention_id': '27722881.48',
  'test_set_idx': 15035,
  'type': 'T007',
  'name': 'JPM1',
  'context': '[CLS] strain JP ##M ##1 was proposed based on the meta ##bol ##ites and the interaction analyses . Additionally , the the ##rm ##ody ##nam ##ic analysis showed that the strain JP ##M ##1 had high tolerance for flu ##oran ##the ##ne , and the influence of flu ##oran ##the ##ne for the bacterial growth activity was ne ##gli ##gible under 100 to 400 mg L ( - 1 ) concentrations . Take ##n together , this study indicates that the strain [unused1] JP ##M ##1 [unused2] has high potential for further study in bio ##rem ##edia ##tion of p ##oly ##cy ##c ##lic a ##romatic h ##ydro ##car ##bon ( p ##oly ##cy ##c ##lic a ##romatic h ##ydro ##car ##bon ) - contaminated sites . [SEP]',
  'existing_cui': 'C1421124',
  'existing_cui_name': 'TRGJP gene',
  'existing_cui_desc': '[CLS] T ##R ##G ##JP gene [unused3] ( Ana ##tom ##ical Structure : T - cell receptor , gamma , joining segment JP ; T cell receptor gamma joining P ) [SEP]',
  'our_bi_cui': 'C3629619',
  'our_bi_cui_name': 'bacterium JP1',
  'our_bi_cui_desc': '[CLS] bacterium JP ##1 [unused3] ( Ba ##cter ##ium ) [SEP]',
  'our_cross_cui': 'C4153571',
  'our_cross_cui_name': 'Microbacterium sp . JPM1',
  'our_cross_cui_desc': '[CLS] Micro ##ba ##cter ##ium s ##p . JP ##M ##1 [unused3] ( Ba ##cter ##ium ) [SEP]'},
 {'mention_id': '27933285.26',
  'test_set_idx': 22294,
  'type': 'T170',
  'name': 'Autism Diagnostic Observation Schedule - second edition',
  'context': "[CLS] ##eti ##tive Be ##ha ##vior Scale - Rev ##ised , the Raven ' s Color ##ed Progressive Mat ##rice ##s , the Child Be ##ha ##vior Check ##L ##ist for ages 1 ½ - 5 or 4 - 18 ( CBC ##L ) , the Social Re ##sp ##ons ##ive ##ness Scale , and the [unused1] Au ##tism Di ##ag ##nos ##tic O ##bs ##er ##vation Schedule - second edition [unused2] ( AD ##OS 2 ) . All patients were showing motor stereo ##ty ##pies for periods of time varying from 6 to 77 months . The Motor Se ##ver ##ity St ##ere ##ot ##y ##py Scale showed that each child had a limited number of stereo ##ty ##pies ; their frequency and intensity were mild [SEP]",
  'existing_cui': 'C0441795',
  'existing_cui_name': 'Second edition',
  'existing_cui_desc': '[CLS] Second edition [unused3] ( Intel ##lect ##ual Product , qualifier value : Second Edition ) [SEP]',
  'our_bi_cui': 'C2828257',
  'our_bi_cui_name': 'International Classification of Diseases for Oncology , Second Edition',
  'our_bi_cui_desc': '[CLS] International Classification of Disease ##s for On ##cology , Second Edition [unused3] ( Intel ##lect ##ual Product : I ##CD - O - 2 ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28077286.39',
  'test_set_idx': 25089,
  'type': 'T103',
  'name': 'MeHg',
  'context': '[CLS] on brain his ##top ##ath ##ology and courts ##hip behavior in a model song ##bird species , the z ##eb ##ra fin ##ch ( Ta ##eni ##opy ##gia gut ##tata ) . Egg treatment groups included : a low Me ##H ##g dose of 0 . 2 μ ##g H ##g g ( - 1 ) egg , a high [unused1] Me ##H ##g [unused2] dose of 3 . 2 μ ##g H ##g g ( - 1 ) egg , and a vehicle control ( water ) . No adverse effects of in o ##vo Me ##H ##g treatment were detected on courts ##hip song quality or on mating behavior in experimental males at sexually maturity which would suggest that o ##bs ##er ##vable ne [SEP]',
  'existing_cui': 'C0228520',
  'existing_cui_name': 'Structure of reticular formation of medulla oblongata',
  'existing_cui_desc': '[CLS] Structure of re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata [unused3] ( Ana ##tom ##ical Structure , body structure : bulb ##ar re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla ; B ##ul ##bar re ##tic ##ular formation ; Form ##ati ##o re ##tic ##ular ##is my ##ele ##nce ##pha ##li ; me ##du ##llar ##y re ##tic ##ular formation ; re ##tic ##ular formation of me ##du ##lla ; Me ##du ##llar ##y re ##tic ##ular formation ; Re ##tic ##ular formation of me ##du ##lla o ##b ##long ##ata ) [SEP]',
  'our_bi_cui': 'C0166979',
  'our_bi_cui_name': 'dimethyl mercury',
  'our_bi_cui_desc': '[CLS] dim ##eth ##yl mercury [unused3] ( Chemical , substance : Di ##met ##hyl ##mer ##cu ##ry ; Me ##rc ##uri ##us Met ##hyl ##en ##us ; Met ##hyl Mercury ; D ##IM ##ET ##H ##Y ##LM ##ER ##CU ##R ##Y ; Di ##met ##hyl mercury ; dim ##eth ##yl ##mer ##cu ##ry ; Mercury , Di ##met ##hyl ) [SEP]',
  'our_cross_cui': 'C0025794',
  'our_cross_cui_name': 'Methylmercury Compounds',
  'our_cross_cui_desc': '[CLS] Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [unused3] ( Chemical : Met ##hyl ##mer ##cu ##ry Co ##mpo ##unds [ Chemical / In ##g ##red ##ient ] ; Mercury Co ##mpo ##unds , Met ##hyl ; ME ##TH ##Y ##L ME ##RC ##UR ##Y CP ##DS ; Met ##hyl Mercury Co ##mpo ##unds ; ME ##TH ##Y ##LM ##ER ##CU ##R ##Y CP ##DS ; Co ##mpo ##unds , Met ##hyl ##mer ##cu ##ry ; met ##hyl ##mer ##cu ##ry ; Co ##mpo ##unds , Met ##hyl Mercury ; Met ##hyl ##mer ##cu ##ry ) [SEP]'},
 {'mention_id': '27503899.39',
  'test_set_idx': 8574,
  'type': 'T033',
  'name': 'underlying misconceptions',
  'context': '[CLS] time and logistics . Various forms of generic feedback can provide rapid and cost - effect feedback to large co ##hor ##ts but may be of limited benefit to students other than signaling weaknesses in knowledge . The present study describes a method that was used to provide format ##ive task - related feedback to a large co ##hor ##t of first - year physiology and anatomy students . Based on student evaluation ##s presented in this study , this method provided feedback in a manner that engaged students , uncovered [unused1] underlying mi ##sco ##nce ##ption ##s [unused2] , facilitated peer discussion , and provided opportunity for new instruction while allowing the lecturer to recognize common gaps in knowledge and inform ongoing curriculum development . [SEP]',
  'existing_cui': 'C4313726',
  'existing_cui_name': 'Underlying erythema',
  'existing_cui_desc': '[CLS] Under ##lying er ##yt ##hem ##a [unused3] ( Finding , in some patients , in some patients ) [SEP]',
  'our_bi_cui': 'C0561845',
  'our_bi_cui_name': 'False memories',
  'our_bi_cui_desc': '[CLS] F ##als ##e memories [unused3] ( B ##iol ##og ##ic Fun ##ction , finding : P ##se ##udo ##me ##mor ##y ; F ##als ##e Memory ; false memory ; false memories ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27259647.14',
  'test_set_idx': 945,
  'type': 'T103',
  'name': '9-cis-11-trans-CLA',
  'context': '[CLS] - surface protein belonging to the conserved family of en ##ola ##ses with a central role in g ##ly ##co ##ly ##tic metabolism , was characterized to have a side role in the intricate metabolism of bio ##hy ##dr ##ogen ##ation of l ##ino ##le ##ic acid , cat ##aly ##zing the formation of bio ##active [unused1] 9 - c ##is - 11 - trans - C ##LA [unused2] through de ##hy ##dra ##tion and is ##omer ##ization of 10 - h ##ydro ##xy - 12 - c ##is - o ##ct ##ade ##ce ##no ##ic acid . The identity of the en ##ola ##se was confirmed through mass s ##pect ##rome ##tric analysis that showed the characteristic 44 ##2 amino acid sequence with a molecular [SEP]',
  'existing_cui': 'C0281666',
  'existing_cui_name': 'alitretinoin',
  'existing_cui_desc': '[CLS] al ##it ##ret ##ino ##in [unused3] ( Chemical , product , substance : 9 - CR ##A ; 9 - c ##is - T ##ret ##ino ##in ; Ali ##tre ##tino ##ín ##a ; 9 - c ##is re ##tino ##ic acid ; 9 - c ##is - Re ##tino ##ic acid ; 9 - c ##is - Re ##tino ##ic A ##cid ; Ali ##tre ##tino ##in ; Ali ##tre ##tino ##in ##um ; 3 , 7 - Di ##met ##hyl - 9 - ( 2 , 6 , 6 - trim ##eth ##yl - 1 - c ##y ##c ##lo ##he ##xen - 1 - y ##l ) 2 - trans - 4 - trans - 6 - c ##is - 8 - trans [SEP]',
  'our_bi_cui': 'C0902483',
  'our_bi_cui_name': 'cis-9-trans-11-octadecadienoic acid',
  'our_bi_cui_desc': '[CLS] c ##is - 9 - trans - 11 - o ##ct ##ade ##ca ##die ##no ##ic acid [unused3] ( Chemical : 9 - c ##is - 11 - trans - o ##ct ##ade ##ca ##die ##no ##ic acid ; c ##9 ##t ##11 C ##LA ) [SEP]',
  'our_cross_cui': 'C1609806',
  'our_cross_cui_name': 'cis-9 , trans-11-conjugated linoleic acid',
  'our_cross_cui_desc': '[CLS] c ##is - 9 , trans - 11 - con ##ju ##gated l ##ino ##le ##ic acid [unused3] ( Chemical : c ##9 - t ##11 - C ##LA ) [SEP]'},
 {'mention_id': '28179911.55',
  'test_set_idx': 27649,
  'type': 'T017',
  'name': 'KASI',
  'context': '[CLS] ##ido ##psis , high over - expressions of Co ##WR ##I ##1 in seven independent T ##2 lines were detected by quantitative real - time PC ##R . The relative m ##RNA accumulation of genes encoding enzymes involved in either fatty acid bio ##sy ##nt ##hesis or t ##ria ##cy ##l ##gly ##cer ##ols assembly ( BC ##CP ##2 , [unused1] K ##AS ##I [unused2] , MA ##T , E ##NR , FA ##TA , and GP ##D ##H ) were also ass ##ay ##ed in mature seeds . Furthermore , lip ##id and fatty acids C ##16 : 0 and C ##18 : 0 significantly increased . In two ho ##mo ##zy ##go ##us T ##2 trans ##genic rice lines ( G ##5 and G [SEP]',
  'existing_cui': 'C1151968',
  'existing_cui_name': '3-oxoacyl - [ acyl-carrier-protein ] synthase activity',
  'existing_cui_desc': '[CLS] 3 - o ##x ##oa ##cy ##l - [ a ##cy ##l - carrier - protein ] s ##ynth ##ase activity [unused3] ( B ##iol ##og ##ic Fun ##ction , de ##car ##box ##yla ##ting : beta - k ##eto ##acy ##l - AC ##P s ##ynth ##ase I activity ; 3 - o ##x ##oa ##cy ##l : AC ##P s ##ynth ##ase I ; beta - k ##eto ##acy ##l s ##ynth ##eta ##se activity ; 3 - o ##x ##oa ##cy ##l - [ a ##cy ##l - carrier protein ] s ##ynth ##ase activity ; beta - k ##eto ##acy ##l a ##cy ##l carrier protein s ##ynth ##ase activity ; beta - k ##eto ##acy ##l - AC ##P s ##ynth [SEP]',
  'our_bi_cui': 'C1418256',
  'our_bi_cui_name': 'PAICS gene',
  'our_bi_cui_desc': '[CLS] PA ##IC ##S gene [unused3] ( Ana ##tom ##ical Structure : SA ##IC ##AR S ##Y ##NT ##H ##ET ##AS ##E ; P ##H ##OS ##P ##H ##OR ##IB ##OS ##Y ##LA ##MI ##N ##O ##IM ##ID ##A ##Z ##OL ##E CA ##RB ##OX ##Y ##LA ##SE ; AI ##R CA ##RB ##OX ##Y ##LA ##SE ; AD ##E ##2 ##H1 ; p ##hos ##ph ##ori ##bos ##yla ##min ##oi ##mi ##da ##zo ##le car ##box ##yla ##se ; p ##hos ##ph ##ori ##bos ##yla ##min ##oi ##mi ##da ##zo ##les ##ucci ##no ##car ##box ##ami ##de s ##ynth ##ase ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27647760.43',
  'test_set_idx': 12810,
  'type': 'T170',
  'name': 'recessive model',
  'context': "[CLS] ##D ##NF ##OS had a protective role for heroin addiction in the add ##itive model and dominant model ( p < 0 . 05 ) . By contrast , the all ##ele ' T ' of r ##s ##7 ##48 ##13 ##11 in B ##D ##NF ##OS significantly elevated the risk of heroin addiction in the add ##itive model , [unused1] re ##cess ##ive model [unused2] and dominant model ( p < 0 . 05 ) . We also found that all ##ele ' C ' of r ##s ##11 ##0 ##30 ##0 ##9 ##6 was associated with an increased risk of addiction in the dominant model and add ##itive model ( p < 0 . 05 ) . Additionally , we found that r ##s [SEP]",
  'existing_cui': 'C0017361',
  'existing_cui_name': 'Genes , Recessive',
  'existing_cui_desc': '[CLS] Gene ##s , Re ##cess ##ive [unused3] ( Ana ##tom ##ical Structure , function , o ##bs ##er ##vable entity , substance : Re ##cess ##ive Gene ##s ; gene re ##cess ##ive ; genes re ##cess ##ive ; Re ##cess ##ive Gene ; re ##cess ##ive genes ; re ##cess ##ive gene ; Gene , Re ##cess ##ive ; Re ##cess ##ive gene , function ; Re ##cess ##ive gene ) [SEP]',
  'our_bi_cui': 'C0017361',
  'our_bi_cui_name': 'Genes , Recessive',
  'our_bi_cui_desc': '[CLS] Gene ##s , Re ##cess ##ive [unused3] ( Ana ##tom ##ical Structure , function , o ##bs ##er ##vable entity , substance : Re ##cess ##ive Gene ##s ; gene re ##cess ##ive ; genes re ##cess ##ive ; Re ##cess ##ive Gene ; re ##cess ##ive genes ; re ##cess ##ive gene ; Gene , Re ##cess ##ive ; Re ##cess ##ive gene , function ; Re ##cess ##ive gene ) [SEP]',
  'our_cross_cui': 'C0026343',
  'our_cross_cui_name': 'Models , Genetic',
  'our_cross_cui_desc': '[CLS] Models , Gene ##tic [unused3] ( Intel ##lect ##ual Product : GE ##NE ##T M ##OD ##EL ##S ; genetic model ; Gene ##tic Models ; Gene ##tic Model ; Model , Gene ##tic ; M ##OD ##EL ##S GE ##NE ##T ) [SEP]'},
 {'mention_id': '27894020.12',
  'test_set_idx': 21121,
  'type': 'T204',
  'name': 'Giuseppe Paravicini',
  'context': '[CLS] Mom ##bell ##o P ##sy ##chia ##tric Hospital in Lim ##bia ##te , near Milan , replaced the old Sen ##av ##ra Hospital as the P ##sy ##chia ##tric Hospital for the Province of Milan in the 19th century . During the early 20th century , bodies of several Mom ##bell ##o patients were di ##sse ##cted and preserved by [unused1] Giuseppe Para ##vic ##ini [unused2] , an an ##ato ##mist who operated within the asylum . The aim of the present study was to examine and memorial ##ize this important ass ##em ##bla ##ge . To this end , we were allowed to sample the head hair of six such preparations for toxic ##ological analysis . By means of high performance liquid ch ##roma ##tography [SEP]',
  'existing_cui': 'C0162568',
  'existing_cui_name': 'Erythropoietic Protoporphyria',
  'existing_cui_desc': '[CLS] E ##ry ##throp ##oi ##etic Proto ##por ##phy ##ria [unused3] ( B ##iol ##og ##ic Fun ##ction , diagnosis , disorder , er ##yt ##hr ##op ##oi ##etic pro ##top ##or ##phy ##ria p ##or ##phy ##ria : pro ##top ##or ##phy ##ria er ##yt ##hr ##op ##oi ##etic ; Proto ##por ##phy ##ria , E ##ry ##throp ##oi ##etic [ Disease / Finding ] ; E ##ry ##throp ##oi ##etic pro ##top ##or ##phy ##ria ; E ##ry ##th ##ro ##he ##pa ##tic Proto ##por ##phy ##ria ; PR ##OT ##OP ##OR ##P ##H ##Y ##RI ##A , ER ##Y ##TH ##RO ##PO ##IE ##TI ##C ; ma ##gn ##us syndrome ; Proto ##por ##phy ##ria ; er ##yt ##hr ##op ##oi ##etic ; pro [SEP]',
  'our_bi_cui': 'C0015621',
  'our_bi_cui_name': 'Famous Persons',
  'our_bi_cui_desc': '[CLS] Famous Persons [unused3] ( Population Group : celebrities ; famous persons ; Ce ##le ##bri ##ties ; Person , Famous ; Persons , Famous ; famous person ; Famous Person ; Celebrity ; celebrity ) [SEP]',
  'our_cross_cui': 'C0086418',
  'our_cross_cui_name': 'Homo sapiens',
  'our_cross_cui_desc': '[CLS] Ho ##mo sa ##pie ##ns [unused3] ( E ##uka ##ryo ##te , Tax ##onomy , organism , qualifier value : Humans ; Man , Modern ; humans ; man ; MA ##N ; Ho ##mo sa ##pie ##ns ( living organism ) [ Am ##bi ##guous ] ; ho ##mo sa ##pie ##ns ; Human , General ; Ho ##mo sa ##pie ##ns Linnaeus , 1758 ; Human ; Human - origin ; human ; Ho ##mo sa ##pie ##ns ( living organism ) ; Modern Man ; Man ) [SEP]'},
 {'mention_id': '27629431.19',
  'test_set_idx': 12256,
  'type': 'T017',
  'name': 'initiation gene',
  'context': '[CLS] ##tra ##cellular degradation , which is rapidly activated under starvation conditions . Recent data demonstrate a transcription ##al up ##re ##gu ##lation of several auto ##pha ##gy genes as a mechanism that controls auto ##pha ##gy in response to starvation . Here we report that despite the significant up ##re ##gu ##lation of m ##RNA of the essential auto ##pha ##gy [unused1] initiation gene [unused2] U ##L ##K ##1 , its protein level is rapidly reduced under starvation . Although both auto ##pha ##gic and pro ##te ##as ##oma ##l systems contribute to the degradation of U ##L ##K ##1 , under prolonged nitrogen de ##p ##ri ##vation , its level was still reduced in AT ##G ##7 knockout cells , and only initially stab ##ilized [SEP]',
  'existing_cui': 'C0589507',
  'existing_cui_name': 'Cognitive function : initiation',
  'existing_cui_desc': '[CLS] Co ##gni ##tive function : initiation [unused3] ( B ##iol ##og ##ic Fun ##ction , o ##bs ##er ##vable entity , qualifier value : In ##iti ##ation ; initiation ) [SEP]',
  'our_bi_cui': 'C1335755',
  'our_bi_cui_name': 'Replication Initiation Gene',
  'our_bi_cui_desc': '[CLS] Rep ##lication In ##iti ##ation Gene [unused3] ( Ana ##tom ##ical Structure : DNA Rep ##lication In ##iti ##ation Gene ) [SEP]',
  'our_cross_cui': 'C0017337',
  'our_cross_cui_name': 'Genes',
  'our_cross_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]'},
 {'mention_id': '27932585.31',
  'test_set_idx': 22258,
  'type': 'T103',
  'name': 'Taw3',
  'context': '[CLS] novel α / β fold . The sequence motif ( S / T ) x ##SS ##C ##x ##GR and invariant as ##par ##tate and his ##ti ##dine , conserved in T ##Y ##W ##3 / Ta ##w ##3 , cluster to form the cat ##alytic center . These structural and sequence features indicate that T ##Y ##W ##3 / [unused1] Ta ##w ##3 [unused2] proteins constitute a distinct class of SA ##M - dependent met ##hyl ##tra ##ns ##fer ##ases . Using site - directed m ##uta ##genesis along with in v ##ivo complement ##ation ass ##ays combined with mass s ##pect ##rome ##try as well as l ##igan ##d dock ##ing and co ##fa ##ctor binding ass ##ays , we have identified the active [SEP]',
  'existing_cui': 'C3496263',
  'existing_cui_name': 'annectant white matter',
  'existing_cui_desc': '[CLS] an ##nect ##ant white matter [unused3] ( Ana ##tom ##ical Structure : Sub ##stant ##ia me ##du ##llar ##is an ##nect ##ens ) [SEP]',
  'our_bi_cui': 'C0039341',
  'our_bi_cui_name': 'tat Protein',
  'our_bi_cui_desc': '[CLS] ta ##t Pro ##tein [unused3] ( Chemical : ta ##t Gene Product ; Gene Products , ta ##t [ Chemical / In ##g ##red ##ient ] ; Trans ##act ##ivating Reg ##ulatory Pro ##tein ; T ##AT Pro ##tein ; Trans - Act ##ivation of Trans ##cription Pro ##tein ; Trans - Act ##iva ##tor of Trans ##cription of HIV ; ta ##t Gene Products ; ta ##t protein ; Gene Product , ta ##t ; Gene Products , ta ##t ) [SEP]',
  'our_cross_cui': 'C0033684',
  'our_cross_cui_name': 'Proteins',
  'our_cross_cui_desc': '[CLS] Pro ##tein ##s [unused3] ( Chemical , medication , obsolete , substance : protein ; gene product ; Pro ##tein ; Pro ##tein ##s [ Chemical / In ##g ##red ##ient ] ; proteins ; protein preparations ; PR ##OT ##EI ##N ) [SEP]'},
 {'mention_id': '28539192.14',
  'test_set_idx': 38798,
  'type': 'T033',
  'name': 'basic level',
  'context': '[CLS] poor literacy level of the population specifically in health . Currently , there is a fragment ##ed model of patient care , in which the systems are exclusive of each other . They do not cooperate or communicate with each other ; and if there is no vertical communication within the system , preventing refer ##ral of patients directly from the [unused1] basic level [unused2] to the complex level of care when needed . In addition , there has been no progress in the development of an em ##path ##etic , respect ##ful or person - centered clinical practice ; instead , economic , social and educational differences per ##pet ##uate a paternal ##istic clinical practice . The task of or ##ient ##ing medical training towards [SEP]',
  'existing_cui': 'C0178499',
  'existing_cui_name': 'Base',
  'existing_cui_desc': '[CLS] Base [unused3] ( Chemical , chemistry , medication , substance : base ; bases ; Basic ; al ##kal ##ini ##ty ) [SEP]',
  'our_bi_cui': 'C0871796',
  'our_bi_cui_name': 'Knowledge level',
  'our_bi_cui_desc': '[CLS] Knowledge level [unused3] ( B ##iol ##og ##ic Fun ##ction , o ##bs ##er ##vable entity : Knowledge Level ) [SEP]',
  'our_cross_cui': 'C1547668',
  'our_cross_cui_name': 'Level of Care',
  'our_cross_cui_desc': '[CLS] Level of Care [unused3] ( Finding : Level of care ) [SEP]'},
 {'mention_id': '27400734.19',
  'test_set_idx': 4723,
  'type': 'T103',
  'name': 'E',
  'context': '[CLS] national database A random ##ized clinical trial has found that the addition of er ##lot ##ini ##b to g ##em ##ci ##ta ##bine ( GE ##M - E ) for pan ##cre ##atic cancer led to a modest increase in survival . The aim of this national population - based retrospective study was to compare the effectiveness of GE ##M - [unused1] E [unused2] to GE ##M alone for pan ##cre ##atic cancer patients in real clinical practice . Pat ##ients with pan ##cre ##atic cancer ( I ##CD - 10 : C ##25 ) with prescription claims of g ##em ##ci ##ta ##bine or er ##lot ##ini ##b between Jan 1 , 2007 and Dec 31 , 2012 were retrospective ##ly identified from the Korean [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C1142985',
  'our_bi_cui_name': 'ezetimibe',
  'our_bi_cui_desc': '[CLS] e ##ze ##ti ##mi ##be [unused3] ( Chemical , medication , product , substance : E ##ze ##ti ##mi ##be ; E ##ze ##ti ##mi ##bu ##m ; E ##ze ##ti ##mi ##b ; ( 1 - ( 4 - flu ##oro ##phe ##ny ##l ) - ( 3 ##R ) - ( 3 - ( 4 - flu ##oro ##phe ##ny ##l ) - ( 3 ##S ) - h ##ydro ##xy ##p ##rop ##yl ) - ( 4 ##S ) - ( 4 - h ##ydro ##xy ##phe ##ny ##l ) - 2 - a ##ze ##ti ##din ##one ) ; E ##Z ##ET ##IM ##IB ##E ; E ##ze ##ti ##mi ##be [ Chemical / In ##g ##red ##ient ] ; e ##ze [SEP]',
  'our_cross_cui': 'C1135135',
  'our_cross_cui_name': 'erlotinib',
  'our_cross_cui_desc': '[CLS] er ##lot ##ini ##b [unused3] ( Chemical , medication , product , substance : N - ( 3 - et ##hy ##ny ##l ##phe ##ny ##l ) - 6 , 7 - bi ##s ( 2 - met ##ho ##xy ##eth ##ox ##y ) q ##uin ##az ##olin - 4 - am ##ine ; [ 6 , 7 - B ##is - ( 2 - met ##ho ##xy - et ##ho ##xy ) - q ##uin ##az ##olin - 4 - y ##l ] - ( 3 - et ##hy ##ny ##l - p ##hen ##yl ) - am ##ine ; ER ##L ##OT ##IN ##IB ; E ##rl ##ot ##ini ##b ) [SEP]'},
 {'mention_id': '28478603.28',
  'test_set_idx': 37430,
  'type': 'T170',
  'name': 'h',
  'context': "[CLS] ##ei ' s gene diversity ( h = 0 . 28 ) , Shannon ' s information index ( I = 0 . 43 ) , and p ##oly ##mor ##phism information content ( P ##IC = 0 . 29 ) generated using the simple sequence repeat prime ##rs were higher than that with Inter simple sequence repeat prime ##rs ( [unused1] h [unused2] = 0 . 23 , I = 0 . 37 , P ##IC = 0 . 24 ) , thereby suggesting that the simple sequence repeat system is more efficient than the Inter simple sequence repeat system in assessing genetic diversity in various pine ##ap ##ple accession ##s . Mean genetic similarities were 0 . 74 , 0 . 61 , and [SEP]",
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0017337',
  'our_bi_cui_name': 'Genes',
  'our_bi_cui_desc': '[CLS] Gene ##s [unused3] ( Ana ##tom ##ical Structure , substance : Gene ; gene ; genes ) [SEP]',
  'our_cross_cui': 'C0282574',
  'our_cross_cui_name': 'Intellectual Property',
  'our_cross_cui_desc': '[CLS] Intel ##lect ##ual Property [unused3] ( Intel ##lect ##ual Product : Intel ##lect ##ual Product ; Property , Intel ##lect ##ual ; Properties , Intel ##lect ##ual ; Intel ##lect ##ual Properties ; Intel ##lect ##ual property ; intellectual property ) [SEP]'},
 {'mention_id': '28296885.51',
  'test_set_idx': 31634,
  'type': 'T103',
  'name': 'marijuana .',
  'context': '[CLS] - food diet had more than those living in the village consuming a mostly agricultural diet . These findings are explained by the high incidence of maize consumption in village settings , along with previously recognized variation in rate of car ##ies between men and women . The unexpected discovery of high car ##ies incidence ##s for men in the bush is likely explained by heavy reliance on honey , and perhaps differential access to tobacco and [unused1] marijuana . [unused2] These data support the notions that mechanisms of car ##io ##genesis are multi ##fa ##ctor ##ial and that the relationships between oral health and the shift from a predominantly wild - food diet to one dominated by cult ##ige ##ns are n ##uan ##ced . [SEP]',
  'existing_cui': None,
  'existing_cui_name': None,
  'existing_cui_desc': None,
  'our_bi_cui': 'C0024809',
  'our_bi_cui_name': 'Marijuana Abuse',
  'our_bi_cui_desc': '[CLS] Mari ##ju ##ana Abu ##se [unused3] ( B ##iol ##og ##ic Fun ##ction : abuse marijuana ; Abu ##se , Mari ##hua ##na ; Has ##h ; Abu ##se , Mari ##ju ##ana ; marijuana abuse ; marijuana drug abuse ; Mari ##ju ##ana Abu ##se [ Disease / Finding ] ; abuse ; ma ##ri ##hua ##na ; Mari ##hua ##na Abu ##se ; Mari ##ju ##ana ; Mari ##hua ##na abuse ; ma ##ri ##hua ##na ; abuse ; Mari ##ju ##ana abuse ; We ##ed ; drug abuse marijuana ; abuse drug marijuana ; Abu ##se ; drug ( s ) ; marijuana ; Po ##t ; Can ##na ##bis ) [SEP]',
  'our_cross_cui': 'C0024808',
  'our_cross_cui_name': 'Marihuana',
  'our_cross_cui_desc': '[CLS] Mari ##hua ##na [unused3] ( Chemical : Mari ##ju ##ana ##s ; marijuana ; Mari ##ju ##ana ; ma ##ri ##hua ##na ; MA ##RI ##J ##U ##AN ##A [ VA Product ] ; Mari ##hua ##nas ; MA ##RI ##J ##U ##AN ##A ; cannabis ; Can ##na ##bis ) [SEP]'},
 {'mention_id': '27645371.31',
  'test_set_idx': 12724,
  'type': 'T033',
  'name': 'Children abandoning treatment',
  'context': '[CLS] the 77 ( of 41 ##8 ) children abandoning therapy , 17 ( 22 % ) refused up ##front , whereas the rest abandoned during various phases of ch ##em ##otherapy . Only 39 ( 50 . 6 % ) of these 77 families could be subsequently contacted . Financial problems , too many dependent ##s at home , and wrong perception ##s about cancer led to abandonment in majority . [unused1] Children abandoning treatment [unused2] before completion of induction had a significantly shorter survival than who abandoned post induction ( P < . 000 ##1 ) . In ##tens ##ified pre ##aba ##ndon ##ment counseling and sub ##si ##dized treatment led to significant reduction in abandonment rates ( P < . 000 ##1 ) . [SEP]',
  'existing_cui': 'C0013806',
  'existing_cui_name': 'Electroconvulsive Therapy',
  'existing_cui_desc': '[CLS] El ##ec ##tro ##con ##vu ##lsive Therapy [unused3] ( Health Care Act ##ivity , EC ##T , P ##sy ##cho ##ther ##ap ##y , Therapy , electro - con ##vu ##lsive therapy , procedure , regime / therapy , treatment : electric shock treatments ; El ##ec ##tro ##sh ##ock treatment ; Therapy , Electric Shock ; Therapy , El ##ec ##tro ##sh ##ock ; Therapy , Electric Con ##vu ##lsive ; e ##cs ; electric shock therapy ; electro ##con ##vu ##lsive treatment ; Con ##vu ##lsive Therapy , Electric ; EC ##T therapy ; El ##ec ##tro ##con ##vu ##lsive treatment ; CO ##N ##V ##U ##LS ##IVE THE ##R E ##LE ##CT ##RI ##C ; The ##rap ##ies , El ##ec ##tro ##con [SEP]',
  'our_bi_cui': 'C1298653',
  'our_bi_cui_name': 'Patient declined anti-cancer treatment',
  'our_bi_cui_desc': '[CLS] Pat ##ient declined anti - cancer treatment [unused3] ( Finding , finding ) [SEP]',
  'our_cross_cui': 'C0243095',
  'our_cross_cui_name': 'Finding',
  'our_cross_cui_desc': '[CLS] Finding [unused3] ( Finding , finding : Find ; Investigation Finding ) [SEP]'},
 {'mention_id': '27842161.46',
  'test_set_idx': 19139,
  'type': 'T103',
  'name': 'Netarsudil',
  'context': '[CLS] ##age effective fi ##ltration length in e ##pis ##cle ##ral veins positively correlated with the percentage change in C ( R ##2 = 0 . 58 , P = 0 . 01 ) . A significant increase in j ##ux ##ta ##can ##ali ##cular connect ##ive tissue thickness ( P < 0 . 05 ) was found in treated eyes compared to controls . [unused1] Net ##ars ##udi ##l [unused2] acute ##ly increased C by expansion of the j ##ux ##ta ##can ##ali ##cular connect ##ive tissue and di ##lating the e ##pis ##cle ##ral veins , which led to red ##ist ##ri ##but ##ion of a ##que ##ous out ##flow through a larger area of the inner wall and e ##pis ##cle ##ral veins . [SEP]',
  'existing_cui': 'C0013443',
  'existing_cui_name': 'Ear structure',
  'existing_cui_desc': '[CLS] E ##ar structure [unused3] ( Ana ##tom ##ical Structure , Anatomy , body structure : V ##est ##ib ##ulo ##co ##ch ##lea ##r System ; System , V ##est ##ib ##ulo ##co ##ch ##lea ##r ; ear structures ; Organ ##um vest ##ib ##ulo ##co ##ch ##lea ##re ; ear structure ; E ##ar - related structure ; au ##ris ; A ##ppa ##rat ##us , V ##est ##ib ##ulo ##co ##ch ##lea ##r ; E ##ar and related structures ; Au ##ris ; V ##est ##ib ##ulo ##co ##ch ##lea ##r organ ; V ##est ##ib ##ulo ##co ##ch ##lea ##r A ##ppa ##rat ##us ) [SEP]',
  'our_bi_cui': 'C0701205',
  'our_bi_cui_name': 'Rantodil',
  'our_bi_cui_desc': '[CLS] Ra ##nto ##di ##l [unused3] ( Chemical : Bay ##er brand of ace ##met ##ac ##in ; Ra ##nt ##udi ##l ; T ##rop ##on brand of ace ##met ##ac ##in ) [SEP]',
  'our_cross_cui': 'C1254351',
  'our_cross_cui_name': 'Pharmacologic Substance',
  'our_cross_cui_desc': '[CLS] Ph ##arma ##co ##log ##ic Sub ##stance [unused3] ( Chemical : Ph ##arma ##cological Sub ##stance ; Agent ; Drug Sub ##stance ; Ph ##arma ##ce ##utical agent ; Ph ##arma ##ce ##utical Agent ; DR ##U ##G ; Drug ; Ph ##arma ##co ##log ##ic Agent ; Ph ##arma ##ce ##utical ##s ; drug ) [SEP]'},
 {'mention_id': '27325547.7',
  'test_set_idx': 2505,
  'type': 'T103',
  'name': 'PM2 .5',
  'context': '[CLS] deposited in the human respiratory tract It is well - known that the health effects of PM increase as particle size decreases : particularly , great concern has risen on the role of Ultra ##F ##ine Part ##icles . Starting from the knowledge that the main fraction of atmospheric a ##ero ##sol in Rome is characterized by significant levels of [unused1] PM ##2 . 5 [unused2] ( almost 75 % of PM ##10 fraction is PM ##2 . 5 ) , the paper is focused on sub ##mic ##ron particles in such great urban area . The daytime / night ##time , work - / weekdays and cold / hot seasonal trends of sub ##mic ##ron particles will be investigated and discussed along with NO ##x [SEP]',
  'existing_cui': 'C0665987',
  'existing_cui_name': '1-(4-aminobutyl)-6-benzylindane',
  'existing_cui_desc': '[CLS] 1 - ( 4 - amino ##but ##yl ) - 6 - ben ##zy ##lind ##ane [unused3] ( Chemical : PM ##2 c ##p ##d ) [SEP]',
  'our_bi_cui': 'C0597177',
  'our_bi_cui_name': 'Particle',
  'our_bi_cui_desc': '[CLS] Part ##icle [unused3] ( Chemical : particle ; particles ; Part ##icles ; a very small piece or part ) [SEP]',
  'our_cross_cui': 'C1510837',
  'our_cross_cui_name': 'Airborne Particulate Matter',
  'our_cross_cui_desc': '[CLS] Airborne Part ##iculate Matter [unused3] ( Chemical : Air Poll ##uta ##nts , Part ##iculate ; Am ##bie ##nt Part ##iculate Matter ; Part ##iculate Matter , Am ##bie ##nt ; Part ##iculate Matter , Airborne ; Poll ##uta ##nts , Part ##iculate Air ; Part ##iculate Air Poll ##uta ##nts ) [SEP]'},
 {'mention_id': '27251800.39',
  'test_set_idx': 834,
  'type': 'T058',
  'name': 'Stoppa',
  'context': '[CLS] during the 1 s ##t and 7 th post - operative , physical functioning , physical limitation , the impact of pain on daily activities , and the Carolina ##s Co ##m ##fort Scale during the 7 th and 15 th post - operative ( P > 0 . 05 ) . Co ##mp ##lication ##s occurred in 88 % of [unused1] Stop ##pa [unused2] group ( 22 patients ) and 64 % in lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair group ( 16 patients ) ( P < 0 . 05 ) . The comparative study between the Stop ##pa and lap ##aro ##scopic total extra ##per ##ito ##nea ##l repair approaches for the bilateral ing ##uin ##al her ##nia repair demonstrated [SEP]',
  'existing_cui': 'C1538901',
  'existing_cui_name': 'ACD gene',
  'existing_cui_desc': '[CLS] AC ##D gene [unused3] ( Ana ##tom ##ical Structure : T ##EL ##OM ##ER ##E PR ##OT ##EI ##N T ##PP ##1 ; ad ##ren ##oc ##ort ##ical d ##ys ##p ##lasia ho ##mo ##log ; AC ##D Gene ; Ad ##ren ##oc ##ort ##ical D ##ys ##p ##lasia Ho ##mo ##log ( Mouse ) Gene ; P ##OT ##1 - IN ##TE ##RA ##CT ##ING PR ##OT ##EI ##N 1 ; P ##OT ##1 - AND T ##IN ##2 - OR ##GA ##NI ##Z ##ING PR ##OT ##EI ##N ; P ##OT ##1 and T ##IN ##2 organizing protein ; AC ##D , M ##O ##US ##E , H ##OM ##OL ##O ##G OF ; Tin ##t ##1 ; T ##IN ##2 interacting protein 1 [SEP]',
  'our_bi_cui': 'C1080020',
  'our_bi_cui_name': 'Lopa',
  'our_bi_cui_desc': '[CLS] Lo ##pa [unused3] ( E ##uka ##ryo ##te : lo ##pa ) [SEP]',
  'our_cross_cui': 'C0087111',
  'our_cross_cui_name': 'Therapeutic procedure',
  'our_cross_cui_desc': '[CLS] The ##rap ##eu ##tic procedure [unused3] ( Health Care Act ##ivity , procedure , regime / therapy , treatment : The ##rap ##eu ##tic Tech ##ni ##que ; The ##rap ##eu ##tic Method ; TX ; method treatment ; Pro ##ced ##ure - therapeutic ; T ##RE ##AT ; methods treatment ; therapeutic procedure ; The ##rap ##eu ##tic pro ##ced ; The ##rap ##eu ##tics ; Therapy ; The ##rap ##eu ##tic Pro ##ced ##ures ; Treatment ; treatment method ; The ##rap ##eu ##tic Inter ##vention ##s ; the ##rap ##ies ; therapy ; Therapy ( regime / therapy ) ; treatment ; therapeutic ##s ; Treatment ##s ; The ##rap ##eu ##tic Pro ##ced ##ure ; The ##rap ##eu ##tic ; THE ##R [SEP]'},
 {'mention_id': '27261783.3',
  'test_set_idx': 1081,
  'type': 'T082',
  'name': 'organic - rich anaerobic sedimentary habitats',
  'context': '[CLS] Analysis of Low - B ##io ##mass Micro ##bial Communities in the Deep B ##ios ##phere Over the past few decades , the sub ##sea ##f ##lo ##or bio ##sp ##here has been explored by scientific ocean drilling to depths of about 2 . 5 km below the sea ##f ##lo ##or . Although [unused1] organic - rich an ##ae ##ro ##bic sediment ##ary habitats [unused2] in the ocean margins harbor large numbers of micro ##bial cells , micro ##bial populations in ultra ##oli ##got ##rop ##hic a ##ero ##bic sediment ##ary habitats in the open ocean g ##yres are several orders of magnitude less abundant . Despite advances in cultivation - independent molecular ecological techniques , exploring the low - bio ##mass environment remains technological [SEP]',
  'existing_cui': 'C0004613',
  'existing_cui_name': 'Bacteria , Anaerobic',
  'existing_cui_desc': '[CLS] Ba ##cter ##ia , Ana ##ero ##bic [unused3] ( Ba ##cter ##ium , organism : an ##ae ##ro ##bic bacterium ; BA ##CT AN ##A ##ER ##O ##BI ##C ; Ana ##ero ##bic bacterium ; Ana ##ero ##bic Ba ##cter ##ia ; Ana ##ero ##bic bacteria ; AN ##A ##ER ##O ##BI ##C BA ##CT ; an ##ae ##robe ; an ##ae ##ro ##bic bacteria ; Ana ##ero ##be ; an ##ae ##robe ##s ; AN ##A ##ER ##O ##BI ##C BA ##CT ##ER ##IA ) [SEP]',
  'our_bi_cui': 'C0486320',
  'our_bi_cui_name': 'Amorphous sediment',
  'our_bi_cui_desc': '[CLS] Amor ##ph ##ous sediment [unused3] ( Body Sub ##stance , substance ) [SEP]',
  'our_cross_cui': 'C0871648',
  'our_cross_cui_name': 'Habitat',
  'our_cross_cui_desc': '[CLS] Ha ##bit ##at [unused3] ( Spa ##tial Concept , Animal : habitats ; Animal Environment ##s ; Ha ##bit ##ats ; environment animal ; habitat ) [SEP]'},
 {'mention_id': '27427386.16',
  'test_set_idx': 5254,
  'type': 'T017',
  'name': 'm. 14502T > C',
  'context': "[CLS] ##rial m ##od ##ifier ##s are proposed to modify the p ##hen ##otypic expression of primary Le ##ber ' s hereditary op ##tic ne ##uro ##pathy - associated mitochondrial DNA mutations . In this study , we demonstrated that the Le ##ber ' s hereditary op ##tic ne ##uro ##pathy su ##s ##ce ##pt ##ibility all ##ele ( [unused1] m . 145 ##0 ##2 ##T > C [unused2] , p . 58 ##I > V ) in the N ##D ##6 gene m ##od ##ulated the p ##hen ##otypic expression of primary Le ##ber ' s hereditary op ##tic ne ##uro ##pathy - associated m . 117 ##7 ##8 ##G > A mutation . Twenty - two Han Chinese p ##ed ##ig ##ree ##s carrying m [SEP]",
  'existing_cui': 'C0035609',
  'existing_cui_name': 'Rifamycins',
  'existing_cui_desc': '[CLS] R ##if ##am ##y ##cin ##s [unused3] ( Chemical , medication , substance : r ##if ##am ##y ##cin ; M - 14 ; R ##if ##am ##y ##cin ; R ##if ##am ##y ##cin SV ; R ##if ##omy ##cin ##s ; R ##if ##am ##y ##cin ##s [ Chemical / In ##g ##red ##ient ] ; r ##if ##am ##y ##cin ##s ; 5 , 6 , 9 , 17 , 19 , 21 - he ##xa ##hy ##dr ##ox ##y - 23 - met ##ho ##xy - 2 , 4 , 12 , 16 , 18 , 20 , 22 - he ##pta ##met ##hyl - 2 , 7 - ( e ##pox ##ype ##nta ##de ##ca ( 1 , 11 , 13 [SEP]',
  'our_bi_cui': 'C0596611',
  'our_bi_cui_name': 'Gene Mutation',
  'our_bi_cui_desc': '[CLS] Gene Mu ##tation [unused3] ( B ##iol ##og ##ic Fun ##ction : DNA Se ##quence Alt ##eration ; Gene Alt ##eration ; DNA Alt ##eration ; gene mutation ; genes mutation ; Gene Mu ##tation Type ; M ##OL ##PA ##TH . M ##UT ; gene mutations ; Se ##quence Alt ##eration ; Gene mutation ; mutation gene ) [SEP]',
  'our_cross_cui': 'C0002085',
  'our_cross_cui_name': 'Alleles',
  'our_cross_cui_desc': '[CLS] All ##ele ##s [unused3] ( Ana ##tom ##ical Structure , finding : All ##ele ; All ##elo ##mor ##ph ; Gene ##tic all ##ele ##s ; all ##ele ; All ##elo ##mor ##phs ; all ##ele ##s ; AL ##LE ##LE ) [SEP]'}]

In [25]:
selected  = []
for r in res:
    if r['test_set_idx'] in chosen:
        selected.append(r)

In [27]:
selected

[{'mention_id': '27660632.56',
  'test_set_idx': 13478,
  'type': 'T017',
  'name': 'regions',
  'context': ') traits , mainly focused on Chr05 , Chr09 , Chr10 , Chr14 , Chr19 , and Chr20 . Nineteen pairs of epistatic quantitative trait locus were identified , of which two pairs involved in two additive quantitative trait locus . These additive quantitative trait locus , epistatic quantitative trait locus , and QTL clusters were tightly linked to single nucleotide polymorphism markers , which may serve as target [unused1] regions [unused2] for map - based cloning , gene discovery , and marker - assisted selection in cotton breeding .',
  'existing_cui': 'C0017446',
  'existing_cui_name': 'Geographic Locations',
  'existing_cui_desc': 'Geographic Locations [unused3] ( Spatial Concept : areas geographic ; Geographic Location ; Region ; GEOGR LOCATIONS ; geographic region ; Geographic Region ; geographic location ; Geographic Regions ; Area ; geographic site ; Geographic Area )',
  'our_b

In [31]:
for s in selected:
    for attr in ['context', 'existing_cui_desc', 'our_bi_cui_desc', 'our_cross_cui_desc']:
        split = s[attr].split()
        final = []
        for token in split:
            if token in ['[CLS]', '[SEP]']:
                continue
            if token[:2] == "##":
                if len(final) == 0:
                    final.append(token[2:])
                else:
                    final[-1] += token[2:]
            else:
                final.append(token)
        s[attr] = ' '.join(final)

In [32]:
selected

[{'mention_id': '27660632.56',
  'test_set_idx': 13478,
  'type': 'T017',
  'name': 'regions',
  'context': ') traits , mainly focused on Chr05 , Chr09 , Chr10 , Chr14 , Chr19 , and Chr20 . Nineteen pairs of epistatic quantitative trait locus were identified , of which two pairs involved in two additive quantitative trait locus . These additive quantitative trait locus , epistatic quantitative trait locus , and QTL clusters were tightly linked to single nucleotide polymorphism markers , which may serve as target [unused1] regions [unused2] for map - based cloning , gene discovery , and marker - assisted selection in cotton breeding .',
  'existing_cui': 'C0017446',
  'existing_cui_name': 'Geographic Locations',
  'existing_cui_desc': 'Geographic Locations [unused3] ( Spatial Concept : areas geographic ; Geographic Location ; Region ; GEOGR LOCATIONS ; geographic region ; Geographic Region ; geographic location ; Geographic Regions ; Area ; geographic site ; Geographic Area )',
  'our_b